# Open and Shut Case: The Optimal Strategy for Shut the Box

David Castner

Shut the Box is a simple dice roll game where the player has a set of tiles numbered 1 to 9. The player rolls two dice and then flips down tiles that sum to the dice roll. The player wins if all tiles are flipped down. The player loses if there are no valid moves left. The player can also choose to stop rolling the dice and flip down the remaining tiles. The player's score is the sum of the tiles that were flipped down. The goal of this project is to find the optimal strategy for Shut the Box.

Example: The player rolls a 6 and a 3. The player can flip down the 6 and the 3, or the 6 and the 2 and the 1, or the 5 and the 4. The player can also choose to stop rolling the dice and flip down the 7, 8, and 9. If the player chooses to stop rolling the dice, the player's score is 24. If the player chooses to roll again, the player's score is 11.

## Hypothesis

I hypothesize that the optimal strategy for Shut the Box is to flip down the least amount of tiles and then choose to flip the highest possible tiles each turn. This would leave lower tiles numbers allowing for more possible combinations of dice rolls later in the game. For example, if the first roll was a 9, then flipping over the 9 would be the best move. If another 9 is rolled, then flipping the 1 and 8 next would be the best move.

## Finding the Optimal Strategy

> Definitions: A **board state** is a unique combination of tiles that are up or down.

Finding the optimal strategy can be done using information theory. Each possible board state and roll can be calculated since the amount of combinations is small for modern computing. If a tile can either be flipped or not be flipped, it is equivalent to a single bit of information. With 9 different tiles, there are 2^9 possible board states. With 2 dice, there are 6^2 possible rolls. This means there are 2^9 * 6^2 = 18432 possible board states and rolls. This is a small enough number to calculate all possible board states and rolls.

Calculating the expected score of a board state can be calculated by summing the probability of each roll multiplied by the maximum of each possible resulting board state's expected score. Below is the formula for that calculation: $$S(b) = \sum_{i=d_n}^{d_m} P(i) \cdot F(\{S(b') | b' \in B(i, b)\})$$

Where $S(b)$ is the expected score of board state $b$, $P(i)$ is the probability of roll $i$, $d_n$ is the minimum roll, $d_m$ is the maximum roll, $B(i, b)$ is the set of all possible board states resulting from roll $i$ and board state $b$, and $S(b')$ is the expected score of board state $b'$.

$F$ is a function that represents a strategy. For the optimal strategy, $F$ is the $max$ function. For the worst strategy, $F$ is the $min$ function. $F$ could also be a function that represents a human player's strategy, such as choosing the combination of tiles that contains the highest tile number.

Additionally in order to use this formula, the expected score of each board state $b$ where $B(i, b)$ is the empty set must be defined (i.e. what is the score of a board state when a roll causes the game to be over or all numbers are flipped). From the traditional rules of the game, the expected score of a board state when the game is over is the sum of all tiles that were flipped down. Other scoring systems will be explored further below.

> Note: Because each board state is always increasing the sum of all tiles, the formula above does not need to consider infinite loops.



### Example Calculation

Below is an example calculation of the expected score of the board state where the only remaining tiles are 2 and 1.

| Roll | Probability | Resulting Board States (Remaining Unflipped) | Expected Score |
| ---- | ----------- | ---------------------- | -------------- |
| 2    | 1/36        | 1                      | 44             |
| 3    | 2/36        | -                      | 45             |
| everything else | 33/36 | 1 2               | 42             |

$S(b) = \frac{1}{36} \cdot 44 + \frac{2}{36} \cdot 45 + \frac{33}{36} \cdot 42 = 42.222$


### Overall Expected Score and Optimal Strategy

The overall expected score of the game is the expected score, $S(b)$, of the initial board state.
The optimal strategy is the strategy, $F$, that returns the maximum expected score.

## Code for Solving

### Dependencies

Imports, constants, and dependency code for displaying results

In [1]:
import itertools
from typing import Callable, Iterable
from IPython.display import display, Markdown

DEFAULT_DICE_COUNT = 2
DEFAULT_DICE_SIZE = 6
DEFAULT_MAX_DIGIT = 9

Table = list[dict[str, str|int|float]]
"""table of data"""

def display_table(table: Table) -> None:
    """display table of data"""
    if len(table) == 0:
        return
    keys = table[0].keys()
    output = "| " + " | ".join(keys) + " |\n"
    output += "| " + " | ".join(["---"] * len(keys)) + " |\n"
    for row in table:
        output += "| " + " | ".join([str(row[key]) for key in keys]) + " |\n"
    display(Markdown(output))

### Classes for Dice and Boards

In [2]:
class Dice:
    """Utility class for calculating dice probabilities"""

    def __init__(self, dice_count: int = DEFAULT_DICE_COUNT, dice_size: int = DEFAULT_DICE_SIZE):
        assert isinstance(dice_count, int)
        assert 0 < dice_count < 8
        assert isinstance(dice_size, int)
        assert 1 < dice_size < 21
        dice_rolls = list(range(1, dice_size + 1))
        dice_outcomes = itertools.product(dice_rolls, repeat=dice_count)
        self._probabilities = {}
        for outcome in dice_outcomes:
            outcome_sum = sum(outcome)
            self._probabilities[outcome_sum] = self._probabilities.get(outcome_sum, 0) + 1
        for roll in self._probabilities:
            self._probabilities[roll] /= dice_size ** dice_count

    def all_rolls(self) -> Iterable[int]:
        """generates all possible rolls"""
        return self._probabilities.keys()

    def get_probability(self, roll: int) -> float:
        """returns the probability of rolling the given roll"""
        return self._probabilities.get(roll, 0)


class Board:
    """represents an immutable board state with a unique id"""

    def __init__(self, board_id: int, max_digit: int = DEFAULT_MAX_DIGIT):
        assert isinstance(max_digit, int)
        assert 0 < max_digit < 16 # 16 avoids long calculations
        assert isinstance(board_id, int)
        assert 0 <= board_id < 2 ** max_digit
        self._id = (max_digit, board_id) # tuple for immutability and hashing
        # calculate the board using binary representation of board_id
        binary = bin(board_id)[2:].zfill(max_digit)
        self._flipped_tiles = {i+1: char == '1' for i, char in enumerate(binary)}
        self._sum = None # lazy evaluation

    # required for use in sets and dicts
    def __hash__(self) -> int:
        """hashes the board id"""
        return hash(self._id)

    # required for use in sets and dicts
    def __eq__(self, other) -> bool:
        """compares two boards by id"""
        return self.max_digit() == other.max_digit() and self.get_id() == other.get_id()

    def __str__(self) -> str:
        """returns a string representation of the board"""
        return ' '.join('-' if is_flipped else str(tile) for tile, is_flipped in self._flipped_tiles.items())

    def __repr__(self) -> str:
        """returns Board(unique_id, sum, flipped_digits)"""
        return f'Board({self._id[1]}, {self.get_sum()}, {str(self)})'

    def max_digit(self) -> int:
        """returns the max digit of the board"""
        return self._id[0]

    def get_id(self) -> int:
        """returns the unique id of the board"""
        return self._id[1]

    def max_board_id(self) -> int:
        """returns the max possible score of the board"""
        return 2 ** self.max_digit() - 1

    def is_flipped(self, digit: int) -> bool:
        """returns True if the tile is flipped"""
        return self._flipped_tiles.get(digit, False)

    def get_sum(self) -> int:
        """returns the sum of the flipped digits"""
        if self._sum is None:
            self._sum = sum(tile for tile, is_flipped in self._flipped_tiles.items() if is_flipped)
        return self._sum

    def can_make_next_board(self, next_board: 'Board') -> bool:
        """returns True if the board can make a next board,
        the next board cannot have unflipped digits that the current board does have flipped"""
        assert isinstance(next_board, Board)
        assert self.max_digit() == next_board.max_digit()
        flipped_tiles = (tile for tile, is_flipped in self._flipped_tiles.items() if is_flipped)
        return all(next_board.is_flipped(tile) for tile in flipped_tiles)

    def is_all_flipped(self) -> bool:
        """returns True if all digits are flipped"""
        return self.get_id() == self.max_board_id()

    def get_flipped_count(self) -> int:
        """returns the number of flipped tiles"""
        return bin(self.get_id()).count('1')


### Type Definitions for Strategies and Scoring Systems

In [3]:
BoardSet = Iterable[tuple[Board, float]]
"""a set of boards with their expected scores"""

BoardSums = dict[int, set[Board]]
"""a dictionary of sums and the boards that have that sum"""

Strategy = Callable[[BoardSet], Board]
"""a function that takes a set of boards and returns a chosen board"""

ScoringSystem = Callable[[Board], int]
"""a function that takes a board and returns a score, it is assumed that the board given is game over"""

ExpectedScores = dict[Board, float]
"""a dictionary of boards and their expected scores"""

pass # avoids doctype output for this cell

### Utility Class for Managing Boards

In [4]:
class BoardManager:
    """manages all boards for a given max digit"""

    def __init__(self, max_digit: int = DEFAULT_MAX_DIGIT):
        assert isinstance(max_digit, int)
        assert 0 < max_digit < 16 # 16 avoids long calculations
        self._max_digit = max_digit
        self._max_board_sum = max_digit * (max_digit + 1) // 2
        self._boards = [Board(board_id, max_digit) for board_id in range(2 ** max_digit)]
        self._boards_by_sum = dict()
        for board in self._boards:
            board_sum = board.get_sum()
            if board_sum not in self._boards_by_sum:
                self._boards_by_sum[board_sum] = set()
            self._boards_by_sum[board_sum].add(board)

    def get_max_board_sum(self) -> int:
        """returns the max board sum"""
        return self._max_board_sum

    def get_boards(self) -> Iterable[Board]:
        """returns all boards"""
        return (board for board in self._boards)

    def get_boards_by_sum(self) -> BoardSums:
        """returns a dictionary of boards by sum"""
        # deep copy to avoid mutation
        dict_copy = dict()
        for key, value in self._boards_by_sum.items():
            dict_copy[key] = value.copy()
        return dict_copy

    def get_empty_expected_scores(self) -> ExpectedScores:
        """returns an empty dictionary of boards and their expected scores"""
        return {board: 0.0 for board in self._boards}

### Code for Calculating Expected Scores

In [5]:
def find_possible_moves(boards_by_sum: dict[int, set[Board]], current_board: Board, roll: int) -> set[Board]:
    """helper for finding possible moves"""
    next_board_sum = current_board.get_sum() + roll
    return {board for board in boards_by_sum.get(next_board_sum, set()) if current_board.can_make_next_board(board)}


def create_board_set(possible_boards: set[Board], expected_scores: ExpectedScores) -> BoardSet:
    """helper for creating a board set"""
    return ((board, expected_scores[board]) for board in possible_boards)


def calculate_expected_scores(
        scoring_system: ScoringSystem,
        strategy: Strategy,
        board_manager: BoardManager,
        dice: Dice,
) -> ExpectedScores:
    """returns a dictionary of boards and their expected scores"""
    # initialize boards_by_sum and expected_scores
    boards_by_sum = board_manager.get_boards_by_sum()
    expected_scores = board_manager.get_empty_expected_scores()
    # calculate expected scores in reverse order of board sum
    for score in range(board_manager.get_max_board_sum(), -1, -1):
        for board in boards_by_sum.get(score, set()):
            cumulative_expected = 0.0
            for roll in dice.all_rolls():
                possible_boards = find_possible_moves(boards_by_sum, board, roll)
                if not possible_boards:
                    multiplier = scoring_system(board)
                else:
                    board_set = create_board_set(possible_boards, expected_scores)
                    multiplier = expected_scores[strategy(board_set)]
                cumulative_expected += dice.get_probability(roll) * multiplier
            expected_scores[board] = cumulative_expected
    return expected_scores

## Strategies

Below are the three strategies that were tested.

**`strategy_max`**: The optimal strategy. Chooses the board state with the highest expected score. This is the mathematically optimal strategy.

**`strategy_min`**: The worst strategy. Chooses the board state with the lowest expected score. This is the mathematically worst strategy.

**`strategy_highest_tile`**: A strategy that chooses the board state that flips the least amount fo tiles and flips the highest tile number possible. This is a strategy that a human player might use and what is hypothesized to be the optimal strategy.

In [6]:
def strategy_max(board_set: BoardSet) -> Board:
    """returns the board with the highest expected score"""
    return max(board_set, key=lambda board_and_score: board_and_score[1])[0]


def strategy_min(board_set: BoardSet) -> Board:
    """returns the board with the lowest expected score"""
    return min(board_set, key=lambda board_and_score: board_and_score[1])[0]


def strategy_highest_tile(board_set: BoardSet) -> Board:
    """returns the board with the highest tile number and least amount of flipped tiles"""
    def flipped_tiles(board: Board) -> str:
        """helper for sorting by flipped tiles"""
        tile_string = ''
        for tile in range(board.max_digit(), 0, -1):
            tile_string += str(tile) if board.is_flipped(tile) else '0'
        return int(tile_string)
    boards = [board_and_score[0] for board_and_score in board_set]
    return max(boards, key=lambda board: (-board.get_flipped_count(), flipped_tiles(board)))

## Scoring Systems

Three different scoring systems are defined below:

**`score_traditional`**: The traditional scoring system. The score is the sum of all tiles that were flipped down.

**`score_all_or_nothing`**: The score is 0 if the game is lost and 1 if all tiles are flipped down. The traditional scoring system may produce a local maximum where certain moves are picked to increase the overall sum but make it more likely to lose the game. This scoring system is designed to avoid that local maximum. Hypothetical example: if the 3 tile is extremely hard to flip, it may be better to flip it down early when rolling a 9 instead of flipping down the 8 and 1, but flipping down the 8 and 1 would increase the overall expected sum.

**`score_flipped_count`**: The score is the count of tiles that were flipped down. This is my own added scoring system. If the strategy for the traditional scoring system is both simple and optimal, then the game is less interesting. This scoring system may produce a more interesting game that introduces more nuance, risk and reward.

In [7]:
def score_traditional(board: Board) -> int:
    """returns the traditional score of a board"""
    return board.get_sum()


def score_all_or_nothing(board: Board) -> int:
    """returns 1 if all tiles are flipped, otherwise 0"""
    return 1 if board.is_all_flipped() else 0


def score_flipped_count(board: Board) -> int:
    """returns the all or nothing score of a board"""
    return board.get_flipped_count()

## Results

Calculating the expected scores for each strategy and scoring system combination

In [8]:
def get_optimal_move(current_board: Board, possible_boards: set[Board], expected_scores: ExpectedScores, strategy: Strategy) -> str:
    """returns the optimal move(s) for a given board
    as the string representation of list[list]
    where each tuple is the digits flipped on the board

    if the expected score is within 0.0001 of the max expected score, then the move is considered optimal"""
    board_set = create_board_set(possible_boards, expected_scores)
    optimal_moves = list()
    optimal_expected_score = expected_scores[strategy(board_set)]
    for board in possible_boards:
        if abs(expected_scores[board] - optimal_expected_score) <= 0.0001:
            flipped_tiles = [tile for tile, is_flipped in board._flipped_tiles.items() if is_flipped and not current_board.is_flipped(tile)]
            flipped_tiles.sort()
            optimal_moves.append((flipped_tiles))
    optimal_moves.sort(key=lambda move: tuple(move))
    return str(optimal_moves)[1:-1]


def calculate_results() -> dict:
    """calculates the results for all scoring systems and strategies"""
    board_manager = BoardManager()
    dice = Dice()
    # run all combinations of scoring systems and strategies
    es_traditional_max = calculate_expected_scores(score_traditional, strategy_max, board_manager, dice)
    es_traditional_min = calculate_expected_scores(score_traditional, strategy_min, board_manager, dice)
    es_traditional_highest_tile = calculate_expected_scores(score_traditional, strategy_highest_tile, board_manager, dice)
    es_all_or_nothing_max = calculate_expected_scores(score_all_or_nothing, strategy_max, board_manager, dice)
    es_all_or_nothing_min = calculate_expected_scores(score_all_or_nothing, strategy_min, board_manager, dice)
    es_all_or_nothing_highest_tile = calculate_expected_scores(score_all_or_nothing, strategy_highest_tile, board_manager, dice)
    es_flipped_count_max = calculate_expected_scores(score_flipped_count, strategy_max, board_manager, dice)
    es_flipped_count_min = calculate_expected_scores(score_flipped_count, strategy_min, board_manager, dice)
    es_flipped_count_highest_tile = calculate_expected_scores(score_flipped_count, strategy_highest_tile, board_manager, dice)
    # create tabular data for expected scores by board state
    traditional_board_table: Table = []
    all_or_nothing_board_table: Table = []
    flipped_count_board_table: Table = []
    for board in board_manager.get_boards():
        traditional_board_table.append({
            'Board Id': board.get_id(),
            'Board': str(board),
            'Sum': board.get_sum(),
            'Max ES': es_traditional_max[board],
            'Min ES': es_traditional_min[board],
            'Highest Tile ES': es_traditional_highest_tile[board],
        })
        all_or_nothing_board_table.append({
            'Board Id': board.get_id(),
            'Board': str(board),
            'Sum': board.get_sum(),
            'Max ES': es_all_or_nothing_max[board],
            'Min ES': es_all_or_nothing_min[board],
            'Highest Tile ES': es_all_or_nothing_highest_tile[board],
        })
        flipped_count_board_table.append({
            'Board Id': board.get_id(),
            'Board': str(board),
            'Sum': board.get_sum(),
            'Max ES': es_flipped_count_max[board],
            'Min ES': es_flipped_count_min[board],
            'Highest Tile ES': es_flipped_count_highest_tile[board],
        })
    # create tabular data for optimal moves
    traditional_move_table: Table = []
    all_or_nothing_move_table: Table = []
    flipped_count_move_table: Table = []
    for board in board_manager.get_boards():
        for roll in dice.all_rolls():
            possible_boards = find_possible_moves(board_manager.get_boards_by_sum(), board, roll)
            # skip if no possible moves
            if not possible_boards:
                continue
            traditional_move_table.append({
                'Board Id': board.get_id(),
                'Board': str(board),
                'Sum': board.get_sum(),
                'Roll': roll,
                'Max Move': get_optimal_move(board, possible_boards, es_traditional_max, strategy_max),
                'Min Move': get_optimal_move(board, possible_boards, es_traditional_min, strategy_min),
                'Highest Tile Move': get_optimal_move(board, possible_boards, es_traditional_highest_tile, strategy_highest_tile),
            })
            all_or_nothing_move_table.append({
                'Board Id': board.get_id(),
                'Board': str(board),
                'Sum': board.get_sum(),
                'Roll': roll,
                'Max Move': get_optimal_move(board, possible_boards, es_all_or_nothing_max, strategy_max),
                'Min Move': get_optimal_move(board, possible_boards, es_all_or_nothing_min, strategy_min),
                'Highest Tile Move': get_optimal_move(board, possible_boards, es_all_or_nothing_highest_tile, strategy_highest_tile),
            })
            flipped_count_move_table.append({
                'Board Id': board.get_id(),
                'Board': str(board),
                'Sum': board.get_sum(),
                'Roll': roll,
                'Max Move': get_optimal_move(board, possible_boards, es_flipped_count_max, strategy_max),
                'Min Move': get_optimal_move(board, possible_boards, es_flipped_count_min, strategy_min),
                'Highest Tile Move': get_optimal_move(board, possible_boards, es_flipped_count_highest_tile, strategy_highest_tile),
            })
    return {
        'board_manager': board_manager,
        'dice': dice,
        'traditional_board_table': traditional_board_table,
        'all_or_nothing_board_table': all_or_nothing_board_table,
        'flipped_count_board_table': flipped_count_board_table,
        'traditional_move_table': traditional_move_table,
        'all_or_nothing_move_table': all_or_nothing_move_table,
        'flipped_count_move_table': flipped_count_move_table,
        'es_traditional_max': es_traditional_max,
        'es_traditional_min': es_traditional_min,
        'es_traditional_highest_tile': es_traditional_highest_tile,
        'es_all_or_nothing_max': es_all_or_nothing_max,
        'es_all_or_nothing_min': es_all_or_nothing_min,
        'es_all_or_nothing_highest_tile': es_all_or_nothing_highest_tile,
        'es_flipped_count_max': es_flipped_count_max,
        'es_flipped_count_min': es_flipped_count_min,
        'es_flipped_count_highest_tile': es_flipped_count_highest_tile
    }

full_results = calculate_results()

## Results for Traditional Scoring System

> See **Exhibit D** for the optimal moves for the traditional scoring system.

After looking at the results of the optimal moves for the traditional scoring system, flipping down the highest possible tiles each turn is clear pattern. It is not the best move 100% of the time. However, when it is not the best move, the expected score is still very close to the optimal expected score. This is thebest fitting pattern that a human can use without having to memorize the optimal moves for every possible board state. Below is table of exceptions to the pattern.

> See **Exhibit G** for the full table comparing the optimal moves to the highest tile moves.

In [9]:
def results_traditional_scoring_system(results: dict) -> Table:
    """a comparison of the optimal strategy and the highest tile strategy for the traditional scoring system"""
    exhibit_table = []
    results_subtable = results['traditional_move_table']
    es_max = results['es_traditional_max']
    es_highest_tile = results['es_traditional_highest_tile']
    for record in results_subtable:
        if record['Max Move'] == record['Highest Tile Move']:
            continue
        board = Board(record['Board Id'])
        exhibit_table.append({
            'Board (Remaining)': f'`{record["Board"]}`',
            'Board Sum': record['Sum'],
            'Roll': record['Roll'],
            'Max Move': record['Max Move'],
            'Max ES': f'{es_max[board]:.3f}',
            'Highest Tile Move': record['Highest Tile Move'],
            'Highest Tile ES': f'{es_highest_tile[board]:.3f}'
        })
    # sort by board sum ascending
    exhibit_table.sort(key=lambda record: (record['Board Sum'], record['Board (Remaining)'], record['Roll']))
    return exhibit_table


display_table(results_traditional_scoring_system(full_results))

| Board (Remaining) | Board Sum | Roll | Max Move | Max ES | Highest Tile Move | Highest Tile ES |
| --- | --- | --- | --- | --- | --- | --- |
| `1 - 3 4 5 6 7 8 9` | 2 | 12 | [4, 8] | 29.161 | [3, 9] | 29.036 |
| `- - 3 4 5 6 7 8 9` | 3 | 10 | [4, 6] | 24.603 | [3, 7] | 24.502 |
| `1 2 - 4 5 6 7 8 9` | 3 | 12 | [5, 7] | 30.686 | [4, 8] | 30.584 |
| `1 2 3 - 5 6 7 8 9` | 4 | 10 | [2, 8] | 31.294 | [1, 9] | 31.103 |
| `1 - - 4 5 6 7 8 9` | 5 | 12 | [5, 7] | 25.583 | [4, 8] | 25.560 |
| `1 2 3 4 - 6 7 8 9` | 5 | 5 | [2, 3] | 32.485 | [1, 4] | 32.311 |
| `- - - 4 5 6 7 8 9` | 6 | 11 | [5, 6] | 21.798 | [4, 7] | 21.759 |
| `- 2 3 4 - 6 7 8 9` | 6 | 10 | [4, 6] | 28.199 | [2, 8] | 28.073 |
| `- 2 3 4 - 6 7 8 9` | 6 | 12 | [4, 8] | 28.199 | [3, 9] | 28.073 |
| `1 - 3 - 5 6 7 8 9` | 6 | 12 | [5, 7] | 27.551 | [3, 9] | 27.504 |
| `- - 3 - 5 6 7 8 9` | 7 | 11 | [5, 6] | 22.449 | [3, 8] | 22.399 |
| `- 2 3 4 5 - 7 8 9` | 7 | 10 | [3, 7] | 28.744 | [2, 8] | 28.599 |
| `- - 3 4 - 6 7 8 9` | 8 | 10 | [4, 6] | 23.194 | [3, 7] | 23.085 |
| `1 - 3 4 5 - 7 8 9` | 8 | 12 | [4, 8] | 28.841 | [3, 9] | 28.798 |
| `1 2 - 4 - 6 7 8 9` | 8 | 10 | [2, 8] | 29.820 | [1, 9] | 29.699 |
| `- 2 3 4 5 6 7 - 9` | 9 | 8 | [3, 5] | 31.565 | [2, 6] | 31.478 |
| `1 - 3 4 5 6 - 8 9` | 9 | 7 | [3, 4] | 30.157 | [1, 6] | 29.765 |
| `1 - 3 4 5 6 - 8 9` | 9 | 11 | [5, 6] | 30.157 | [3, 8] | 29.765 |
| `- - 3 4 5 6 - 8 9` | 10 | 11 | [5, 6] | 26.024 | [3, 8] | 25.976 |
| `- 2 - 4 5 - 7 8 9` | 10 | 12 | [5, 7] | 25.884 | [4, 8] | 25.825 |
| `- 2 3 - - 6 7 8 9` | 10 | 10 | [3, 7] | 25.672 | [2, 8] | 25.663 |
| `- 2 3 4 5 6 7 8 -` | 10 | 11 | [5, 6] | 33.860 | [3, 8] | 33.785 |
| `- 2 3 4 5 6 7 8 -` | 10 | 12 | [5, 7] | 33.860 | [4, 8] | 33.785 |
| `1 2 3 - 5 - 7 8 9` | 10 | 10 | [2, 8] | 31.333 | [1, 9] | 31.107 |
| `- - 3 4 5 6 7 - 9` | 11 | 10 | [4, 6] | 28.009 | [3, 7] | 27.869 |
| `- 2 - 4 5 6 - 8 9` | 11 | 10 | [4, 6] | 27.189 | [2, 8] | 27.049 |
| `1 - 3 4 5 6 7 8 -` | 11 | 10 | [4, 6] | 33.806 | [3, 7] | 33.705 |
| `1 2 3 - 5 6 - 8 9` | 11 | 7 | [2, 5] | 32.395 | [1, 6] | 32.075 |
| `1 2 3 - 5 6 - 8 9` | 11 | 10 | [2, 8] | 32.395 | [1, 9] | 32.075 |
| `1 2 3 4 - - 7 8 9` | 11 | 11 | [3, 8] | 32.123 | [2, 9] | 31.989 |
| `- - 3 4 5 6 7 8 -` | 12 | 9 | [4, 5] | 30.311 | [3, 6] | 30.192 |
| `- 2 3 - 5 6 - 8 9` | 12 | 11 | [3, 8] | 27.142 | [2, 9] | 27.052 |
| `- 2 3 4 - - 7 8 9` | 12 | 10 | [3, 7] | 28.868 | [2, 8] | 28.800 |
| `1 2 - 4 5 6 7 8 -` | 12 | 11 | [5, 6] | 35.064 | [4, 7] | 34.938 |
| `1 2 - 4 5 6 7 8 -` | 12 | 12 | [5, 7] | 35.064 | [4, 8] | 34.938 |
| `1 2 3 - 5 6 7 - 9` | 12 | 8 | [2, 6] | 34.134 | [1, 7] | 33.900 |
| `1 2 3 4 - 6 - 8 9` | 12 | 5 | [2, 3] | 33.649 | [1, 4] | 33.313 |
| `- 2 - 4 5 6 7 8 -` | 13 | 11 | [5, 6] | 30.745 | [4, 7] | 30.641 |
| `- 2 3 4 - 6 - 8 9` | 13 | 12 | [4, 8] | 29.572 | [3, 9] | 29.533 |
| `1 - - 4 5 6 7 - 9` | 13 | 11 | [5, 6] | 29.029 | [4, 7] | 28.980 |
| `1 - 3 4 - - 7 8 9` | 13 | 12 | [4, 8] | 27.045 | [3, 9] | 27.043 |
| `1 2 - - 5 - 7 8 9` | 13 | 10 | [2, 8] | 28.519 | [1, 9] | 28.441 |
| `1 2 3 - 5 6 7 8 -` | 13 | 11 | [5, 6] | 35.639 | [3, 8] | 35.521 |
| `1 2 3 4 - 6 7 - 9` | 13 | 5 | [2, 3] | 35.097 | [1, 4] | 34.956 |
| `1 2 3 4 5 - - 8 9` | 13 | 7 | [3, 4] | 34.274 | [2, 5] | 33.982 |
| `- - - 4 5 6 7 - 9` | 14 | 11 | [5, 6] | 24.448 | [4, 7] | 24.442 |
| `- - 3 - 5 6 - 8 9` | 14 | 11 | [5, 6] | 22.328 | [3, 8] | 22.322 |
| `- - 3 4 - - 7 8 9` | 14 | 11 | [4, 7] | 23.949 | [3, 8] | 23.949 |
| `- 2 3 - 5 6 7 8 -` | 14 | 9 | [3, 6] | 30.712 | [2, 7] | 30.591 |
| `- 2 3 4 5 - - 8 9` | 14 | 7 | [3, 4] | 30.555 | [2, 5] | 30.397 |
| `1 - - 4 5 6 7 8 -` | 14 | 12 | [5, 7] | 30.954 | [4, 8] | 30.875 |
| `1 - 3 - 5 6 7 - 9` | 14 | 8 | [3, 5] | 31.182 | [1, 7] | 31.073 |
| `1 - 3 4 - 6 - 8 9` | 14 | 7 | [3, 4] | 30.367 | [1, 6] | 30.259 |
| `1 2 - - 5 6 - 8 9` | 14 | 12 | [1, 5, 6] | 29.623 | [1, 2, 9] | 29.583 |
| `1 2 3 4 5 - 7 - 9` | 14 | 6 | [2, 4] | 35.899 | [1, 5] | 35.418 |
| `1 2 3 4 5 - 7 - 9` | 14 | 8 | [3, 5] | 35.899 | [1, 7] | 35.418 |
| `- - - 4 5 6 7 8 -` | 15 | 11 | [5, 6] | 28.111 | [4, 7] | 28.081 |
| `- 2 - - 5 6 - 8 9` | 15 | 11 | [5, 6] | 26.220 | [2, 9] | 26.203 |
| `- 2 3 4 - 6 7 8 -` | 15 | 10 | [4, 6] | 33.386 | [2, 8] | 33.295 |
| `1 - 3 - 5 6 7 8 -` | 15 | 11 | [5, 6] | 32.668 | [3, 8] | 32.610 |
| `1 - 3 4 - 6 7 - 9` | 15 | 10 | [4, 6] | 31.344 | [1, 9] | 31.313 |
| `1 - 3 4 5 - - 8 9` | 15 | 12 | [4, 8] | 30.357 | [3, 9] | 30.355 |
| `1 2 - - 5 6 7 - 9` | 15 | 8 | [2, 6] | 31.099 | [1, 7] | 30.972 |
| `1 2 - - 5 6 7 - 9` | 15 | 11 | [5, 6] | 31.099 | [2, 9] | 30.972 |
| `1 2 - 4 - 6 - 8 9` | 15 | 10 | [2, 8] | 31.603 | [1, 9] | 31.419 |
| `1 2 3 4 5 6 - - 9` | 15 | 8 | [3, 5] | 36.633 | [2, 6] | 36.322 |
| `- - 3 - 5 6 7 8 -` | 16 | 11 | [5, 6] | 28.596 | [3, 8] | 28.579 |
| `- - 3 4 - 6 7 - 9` | 16 | 10 | [4, 6] | 25.689 | [3, 7] | 25.640 |
| `- 2 - 4 - 6 - 8 9` | 16 | 10 | [4, 6] | 25.078 | [2, 8] | 25.029 |
| `- 2 3 - - - 7 8 9` | 16 | 10 | [3, 7] | 26.606 | [2, 8] | 26.605 |
| `- 2 3 4 5 - 7 8 -` | 16 | 9 | [4, 5] | 33.861 | [2, 7] | 33.668 |
| `- 2 3 4 5 - 7 8 -` | 16 | 10 | [3, 7] | 33.861 | [2, 8] | 33.668 |
| `- 2 3 4 5 - 7 8 -` | 16 | 12 | [5, 7] | 33.861 | [4, 8] | 33.668 |
| `- 2 3 4 5 6 - - 9` | 16 | 7 | [3, 4] | 33.302 | [2, 5] | 33.066 |
| `- 2 3 4 5 6 - - 9` | 16 | 8 | [3, 5] | 33.302 | [2, 6] | 33.066 |
| `1 - 3 4 - 6 7 8 -` | 16 | 11 | [4, 7] | 33.480 | [3, 8] | 33.440 |
| `1 - 3 4 5 - 7 - 9` | 16 | 8 | [3, 5] | 32.505 | [1, 7] | 32.278 |
| `1 2 - 4 - 6 7 - 9` | 16 | 8 | [2, 6] | 33.192 | [1, 7] | 33.000 |
| `1 2 - 4 - 6 7 - 9` | 16 | 12 | [2, 4, 6] | 33.192 | [1, 2, 9] | 33.000 |
| `- - 3 4 - 6 7 8 -` | 17 | 10 | [4, 6] | 29.399 | [3, 7] | 29.353 |
| `1 - 3 4 5 6 - - 9` | 17 | 7 | [3, 4] | 33.490 | [1, 6] | 33.333 |
| `1 2 - 4 - 6 7 8 -` | 17 | 9 | [2, 7] | 34.715 | [1, 8] | 34.650 |
| `1 2 - 4 5 - 7 - 9` | 17 | 6 | [2, 4] | 33.831 | [1, 5] | 33.469 |
| `1 2 3 - - 6 7 - 9` | 17 | 8 | [2, 6] | 33.922 | [1, 7] | 33.775 |
| `1 2 3 - 5 - - 8 9` | 17 | 10 | [2, 8] | 32.750 | [1, 9] | 32.611 |
| `1 2 3 - 5 - - 8 9` | 17 | 11 | [3, 8] | 32.750 | [2, 9] | 32.611 |
| `1 2 3 4 5 6 7 - -` | 17 | 9 | [3, 6] | 39.466 | [2, 7] | 39.339 |
| `- 2 - 4 - 6 7 8 -` | 18 | 10 | [4, 6] | 29.768 | [2, 8] | 29.740 |
| `- 2 3 - 5 - - 8 9` | 18 | 11 | [3, 8] | 28.213 | [2, 9] | 28.159 |
| `- 2 3 4 5 6 7 - -` | 18 | 8 | [3, 5] | 36.797 | [2, 6] | 36.776 |
| `1 - - 4 - 6 7 - 9` | 18 | 10 | [4, 6] | 30.356 | [1, 9] | 30.345 |
| `1 - 3 4 5 6 - 8 -` | 18 | 7 | [3, 4] | 35.317 | [1, 6] | 35.239 |
| `1 2 - 4 5 - 7 8 -` | 18 | 12 | [5, 7] | 35.748 | [4, 8] | 35.704 |
| `1 2 - 4 5 6 - - 9` | 18 | 7 | [2, 5] | 34.288 | [1, 6] | 34.120 |
| `1 2 3 - 5 - 7 - 9` | 18 | 10 | [3, 7] | 34.571 | [1, 9] | 34.465 |
| `1 2 3 4 - - - 8 9` | 18 | 11 | [3, 8] | 33.497 | [2, 9] | 33.477 |
| `- - 3 4 5 6 - 8 -` | 19 | 9 | [4, 5] | 32.285 | [3, 6] | 32.233 |
| `- 2 - 4 5 - 7 8 -` | 19 | 9 | [4, 5] | 32.217 | [2, 7] | 32.093 |
| `- 2 - 4 5 - 7 8 -` | 19 | 12 | [5, 7] | 32.217 | [4, 8] | 32.093 |
| `- 2 3 - - 6 7 8 -` | 19 | 9 | [3, 6] | 31.854 | [2, 7] | 31.792 |
| `1 - 3 4 5 6 7 - -` | 19 | 10 | [4, 6] | 37.126 | [3, 7] | 36.957 |
| `1 2 - - - 6 - 8 9` | 19 | 10 | [2, 8] | 29.737 | [1, 9] | 29.652 |
| `1 2 - 4 5 6 - 8 -` | 19 | 10 | [4, 6] | 36.835 | [2, 8] | 36.761 |
| `1 2 3 - 5 6 - - 9` | 19 | 8 | [3, 5] | 35.815 | [2, 6] | 35.633 |
| `1 2 3 4 - - 7 - 9` | 19 | 5 | [2, 3] | 35.391 | [1, 4] | 35.220 |
| `- - 3 4 5 6 7 - -` | 20 | 9 | [4, 5] | 34.087 | [3, 6] | 34.073 |
| `- - 3 4 5 6 7 - -` | 20 | 10 | [4, 6] | 34.087 | [3, 7] | 34.073 |
| `- 2 3 - 5 6 - - 9` | 20 | 8 | [3, 5] | 31.740 | [2, 6] | 31.667 |
| `1 - 3 - 5 - 7 - 9` | 20 | 8 | [3, 5] | 32.718 | [1, 7] | 32.558 |
| `1 2 - - - 6 7 - 9` | 20 | 8 | [2, 6] | 31.090 | [1, 7] | 31.004 |
| `1 2 - 4 5 6 7 - -` | 20 | 11 | [5, 6] | 37.863 | [4, 7] | 37.757 |
| `1 2 3 - 5 6 - 8 -` | 20 | 7 | [2, 5] | 37.228 | [1, 6] | 36.801 |
| `1 2 3 4 - - 7 8 -` | 20 | 10 | [3, 7] | 37.140 | [2, 8] | 36.971 |
| `1 2 3 4 - 6 - - 9` | 20 | 5 | [2, 3] | 36.481 | [1, 4] | 36.169 |
| `1 2 3 4 - 6 - - 9` | 20 | 7 | [3, 4] | 36.481 | [1, 6] | 36.169 |
| `- 2 - 4 5 6 7 - -` | 21 | 9 | [4, 5] | 34.382 | [2, 7] | 34.363 |
| `- 2 3 4 - - 7 8 -` | 21 | 10 | [3, 7] | 34.507 | [2, 8] | 34.479 |
| `1 2 3 - 5 6 7 - -` | 21 | 8 | [2, 6] | 38.638 | [1, 7] | 38.501 |
| `1 2 3 4 - 6 - 8 -` | 21 | 5 | [2, 3] | 38.476 | [1, 4] | 38.236 |
| `1 2 3 4 5 - - - 9` | 21 | 6 | [2, 4] | 37.234 | [1, 5] | 36.952 |
| `1 2 3 4 5 - - - 9` | 21 | 7 | [3, 4] | 37.234 | [2, 5] | 36.952 |
| `- 2 3 - 5 6 7 - -` | 22 | 8 | [3, 5] | 35.071 | [2, 6] | 35.056 |
| `- 2 3 4 5 - - - 9` | 22 | 7 | [3, 4] | 34.175 | [2, 5] | 34.099 |
| `1 - - 4 5 6 7 - -` | 22 | 11 | [5, 6] | 34.791 | [4, 7] | 34.757 |
| `1 - 3 4 - 6 - - 9` | 22 | 7 | [3, 4] | 33.842 | [1, 6] | 33.674 |
| `1 2 - - 5 - 7 8 -` | 22 | 9 | [2, 7] | 34.247 | [1, 8] | 34.207 |
| `1 2 - - 5 6 - - 9` | 22 | 7 | [2, 5] | 34.022 | [1, 6] | 33.931 |
| `1 2 3 4 - 6 7 - -` | 22 | 9 | [3, 6] | 39.474 | [2, 7] | 39.399 |
| `1 2 3 4 - 6 7 - -` | 22 | 12 | [2, 3, 7] | 39.474 | [1, 4, 7] | 39.399 |
| `1 2 3 4 5 - - 8 -` | 22 | 7 | [3, 4] | 38.987 | [2, 5] | 38.733 |
| `- - - 4 5 6 7 - -` | 23 | 11 | [5, 6] | 31.726 | [4, 7] | 31.721 |
| `- 2 3 4 - 6 7 - -` | 23 | 10 | [4, 6] | 36.545 | [3, 7] | 36.486 |
| `- 2 3 4 - 6 7 - -` | 23 | 12 | [2, 4, 6] | 36.545 | [2, 3, 7] | 36.486 |
| `- 2 3 4 5 - - 8 -` | 23 | 7 | [3, 4] | 36.019 | [2, 5] | 36.010 |
| `1 - 3 4 - 6 - 8 -` | 23 | 7 | [3, 4] | 35.993 | [1, 6] | 35.899 |
| `1 2 - - 5 6 - 8 -` | 23 | 7 | [2, 5] | 35.254 | [1, 6] | 35.170 |
| `1 2 3 4 5 - 7 - -` | 23 | 6 | [2, 4] | 40.138 | [1, 5] | 39.869 |
| `1 2 3 4 5 - 7 - -` | 23 | 9 | [4, 5] | 40.138 | [2, 7] | 39.869 |
| `1 2 - 4 5 - - - 9` | 24 | 6 | [2, 4] | 35.670 | [1, 5] | 35.604 |
| `1 2 3 4 5 6 - - -` | 24 | 8 | [3, 5] | 40.698 | [2, 6] | 40.554 |
| `1 2 3 4 5 6 - - -` | 24 | 12 | [3, 4, 5] | 40.698 | [1, 5, 6] | 40.554 |
| `- - 3 4 - 6 7 - -` | 25 | 10 | [4, 6] | 32.542 | [3, 7] | 32.530 |
| `- 2 3 4 5 6 - - -` | 25 | 7 | [3, 4] | 38.645 | [2, 5] | 38.630 |
| `- 2 3 4 5 6 - - -` | 25 | 12 | [3, 4, 5] | 38.645 | [2, 4, 6] | 38.630 |
| `1 2 - 4 - 6 7 - -` | 25 | 8 | [2, 6] | 38.247 | [1, 7] | 38.144 |
| `1 2 - 4 - 6 7 - -` | 25 | 12 | [2, 4, 6] | 38.247 | [1, 4, 7] | 38.144 |
| `1 2 - 4 5 - - 8 -` | 25 | 6 | [2, 4] | 37.592 | [1, 5] | 37.571 |
| `1 2 - 4 5 - - 8 -` | 25 | 11 | [2, 4, 5] | 37.592 | [1, 2, 8] | 37.571 |
| `1 - 3 4 5 6 - - -` | 26 | 9 | [4, 5] | 38.856 | [3, 6] | 38.754 |
| `1 - 3 4 5 6 - - -` | 26 | 12 | [3, 4, 5] | 38.856 | [1, 5, 6] | 38.754 |
| `1 2 - 4 5 - 7 - -` | 26 | 6 | [2, 4] | 38.968 | [1, 5] | 38.854 |
| `1 2 - 4 5 - 7 - -` | 26 | 9 | [4, 5] | 38.968 | [2, 7] | 38.854 |
| `1 2 - 4 5 - 7 - -` | 26 | 10 | [1, 4, 5] | 38.968 | [1, 2, 7] | 38.854 |
| `1 2 3 - - 6 7 - -` | 26 | 11 | [2, 3, 6] | 38.792 | [1, 3, 7] | 38.776 |
| `1 2 3 4 - - - - 9` | 26 | 5 | [2, 3] | 36.991 | [1, 4] | 36.984 |
| `- - 3 4 5 6 - - -` | 27 | 9 | [4, 5] | 37.100 | [3, 6] | 37.089 |
| `- 2 - 4 5 - 7 - -` | 27 | 9 | [4, 5] | 34.401 | [2, 7] | 34.369 |
| `- 2 3 - - 6 7 - -` | 27 | 9 | [3, 6] | 36.594 | [2, 7] | 36.573 |
| `1 - - 4 5 - - 8 -` | 27 | 9 | [4, 5] | 33.929 | [1, 8] | 33.913 |
| `1 - 3 - - 6 - 8 -` | 27 | 9 | [3, 6] | 35.815 | [1, 8] | 35.810 |
| `1 2 - 4 5 6 - - -` | 27 | 12 | [2, 4, 6] | 39.672 | [1, 5, 6] | 39.662 |
| `1 2 3 - 5 - 7 - -` | 27 | 8 | [3, 5] | 39.467 | [1, 7] | 39.418 |
| `1 2 3 4 - - - 8 -` | 27 | 5 | [2, 3] | 38.835 | [1, 4] | 38.828 |
| `1 2 3 - 5 6 - - -` | 28 | 7 | [2, 5] | 40.217 | [1, 6] | 40.086 |
| `1 2 3 - 5 6 - - -` | 28 | 8 | [3, 5] | 40.217 | [2, 6] | 40.086 |
| `1 2 3 - 5 6 - - -` | 28 | 10 | [2, 3, 5] | 40.217 | [1, 3, 6] | 40.086 |
| `1 2 3 4 - - 7 - -` | 28 | 5 | [2, 3] | 40.054 | [1, 4] | 40.044 |
| `1 2 3 4 - - 7 - -` | 28 | 12 | [2, 3, 7] | 40.054 | [1, 4, 7] | 40.044 |
| `- 2 3 - 5 6 - - -` | 29 | 8 | [3, 5] | 37.801 | [2, 6] | 37.781 |
| `1 - 3 - 5 - 7 - -` | 29 | 8 | [3, 5] | 38.523 | [1, 7] | 38.468 |
| `1 2 - - - 6 7 - -` | 29 | 8 | [2, 6] | 36.382 | [1, 7] | 36.348 |
| `1 2 3 4 - 6 - - -` | 29 | 5 | [2, 3] | 40.676 | [1, 4] | 40.644 |
| `1 2 3 4 - 6 - - -` | 29 | 7 | [3, 4] | 40.676 | [1, 6] | 40.644 |
| `1 2 3 4 - 6 - - -` | 29 | 11 | [2, 3, 6] | 40.676 | [1, 4, 6] | 40.644 |
| `1 2 3 4 5 - - - -` | 30 | 7 | [3, 4] | 41.101 | [2, 5] | 40.999 |
| `1 2 3 4 5 - - - -` | 30 | 10 | [2, 3, 5] | 41.101 | [1, 4, 5] | 40.999 |
| `- 2 3 4 5 - - - -` | 31 | 7 | [3, 4] | 40.117 | [2, 5] | 40.100 |
| `1 - 3 4 - 6 - - -` | 31 | 7 | [3, 4] | 39.567 | [1, 6] | 39.517 |
| `1 2 - - 5 6 - - -` | 31 | 7 | [2, 5] | 38.719 | [1, 6] | 38.685 |
| `1 2 - 4 5 - - - -` | 33 | 6 | [2, 4] | 41.318 | [1, 5] | 41.297 |
| `1 2 3 4 - - - - -` | 35 | 5 | [2, 3] | 41.705 | [1, 4] | 41.694 |


## Appendix

### Exhibit A: All Possible Board States with Expected Score of Each Strategy for Traditional Scoring System

In [10]:
def exhibit_a(results: dict) -> Table:
    """returns a table for expected scores of the traditional scoring system"""
    exhibit_table = []
    results_subtable = results['traditional_board_table']
    for record in results_subtable:
        strategic_difference = record['Max ES'] - record['Min ES']
        exhibit_table.append({
            'Board (Remaining)': f'`{record["Board"]}`',
            'Board Sum': record['Sum'],
            'Max ES': f'{record["Max ES"]:.3f}',
            'Min ES': f'{record["Min ES"]:.3f}',
            'Highest Tile ES': f'{record["Highest Tile ES"]:.3f}',
            'Strategic Difference': f'{strategic_difference:.3f}',
        })
    # sort by board sum ascending
    exhibit_table.sort(key=lambda record: record['Board Sum'])
    return exhibit_table

display_table(exhibit_a(full_results))

| Board (Remaining) | Board Sum | Max ES | Min ES | Highest Tile ES | Strategic Difference |
| --- | --- | --- | --- | --- | --- |
| `1 2 3 4 5 6 7 8 9` | 0 | 33.842 | 20.655 | 33.677 | 13.187 |
| `- 2 3 4 5 6 7 8 9` | 1 | 29.366 | 20.803 | 29.269 | 8.563 |
| `1 - 3 4 5 6 7 8 9` | 2 | 29.161 | 20.471 | 29.036 | 8.690 |
| `1 2 - 4 5 6 7 8 9` | 3 | 30.686 | 21.106 | 30.584 | 9.580 |
| `- - 3 4 5 6 7 8 9` | 3 | 24.603 | 20.663 | 24.502 | 3.940 |
| `1 2 3 - 5 6 7 8 9` | 4 | 31.294 | 21.437 | 31.103 | 9.856 |
| `- 2 - 4 5 6 7 8 9` | 4 | 25.143 | 20.503 | 25.095 | 4.640 |
| `1 2 3 4 - 6 7 8 9` | 5 | 32.485 | 22.252 | 32.311 | 10.233 |
| `1 - - 4 5 6 7 8 9` | 5 | 25.583 | 20.123 | 25.560 | 5.460 |
| `- 2 3 - 5 6 7 8 9` | 5 | 25.242 | 20.310 | 25.206 | 4.932 |
| `1 2 3 4 5 - 7 8 9` | 6 | 33.273 | 22.649 | 33.079 | 10.625 |
| `1 - 3 - 5 6 7 8 9` | 6 | 27.551 | 21.660 | 27.504 | 5.891 |
| `- 2 3 4 - 6 7 8 9` | 6 | 28.199 | 22.669 | 28.073 | 5.530 |
| `- - - 4 5 6 7 8 9` | 6 | 21.798 | 20.754 | 21.759 | 1.044 |
| `1 2 3 4 5 6 - 8 9` | 7 | 34.175 | 23.495 | 33.964 | 10.680 |
| `1 2 - - 5 6 7 8 9` | 7 | 26.837 | 21.003 | 26.794 | 5.834 |
| `1 - 3 4 - 6 7 8 9` | 7 | 28.263 | 22.510 | 28.231 | 5.752 |
| `- 2 3 4 5 - 7 8 9` | 7 | 28.744 | 23.098 | 28.599 | 5.646 |
| `- - 3 - 5 6 7 8 9` | 7 | 22.449 | 20.574 | 22.399 | 1.875 |
| `1 2 3 4 5 6 7 - 9` | 8 | 35.759 | 25.405 | 35.535 | 10.354 |
| `1 2 - 4 - 6 7 8 9` | 8 | 29.820 | 22.974 | 29.699 | 6.845 |
| `1 - 3 4 5 - 7 8 9` | 8 | 28.841 | 22.974 | 28.798 | 5.867 |
| `- 2 3 4 5 6 - 8 9` | 8 | 30.233 | 23.697 | 30.154 | 6.536 |
| `- 2 - - 5 6 7 8 9` | 8 | 23.170 | 20.795 | 23.167 | 2.374 |
| `- - 3 4 - 6 7 8 9` | 8 | 23.194 | 21.238 | 23.085 | 1.955 |
| `1 2 3 4 5 6 7 8 -` | 9 | 37.376 | 27.492 | 37.263 | 9.884 |
| `1 2 3 - - 6 7 8 9` | 9 | 30.156 | 24.020 | 30.127 | 6.136 |
| `1 2 - 4 5 - 7 8 9` | 9 | 30.779 | 23.742 | 30.705 | 7.037 |
| `1 - 3 4 5 6 - 8 9` | 9 | 30.157 | 23.516 | 29.765 | 6.641 |
| `1 - - - 5 6 7 8 9` | 9 | 23.704 | 20.856 | 23.704 | 2.848 |
| `- 2 3 4 5 6 7 - 9` | 9 | 31.565 | 25.500 | 31.478 | 6.065 |
| `- 2 - 4 - 6 7 8 9` | 9 | 23.631 | 21.248 | 23.620 | 2.384 |
| `- - 3 4 5 - 7 8 9` | 9 | 22.911 | 21.085 | 22.910 | 1.826 |
| `1 2 3 - 5 - 7 8 9` | 10 | 31.333 | 24.644 | 31.107 | 6.689 |
| `1 2 - 4 5 6 - 8 9` | 10 | 31.684 | 24.504 | 31.624 | 7.180 |
| `1 - 3 4 5 6 7 - 9` | 10 | 32.078 | 25.257 | 31.984 | 6.821 |
| `1 - - 4 - 6 7 8 9` | 10 | 26.089 | 23.039 | 26.087 | 3.049 |
| `- 2 3 4 5 6 7 8 -` | 10 | 33.860 | 27.743 | 33.785 | 6.117 |
| `- 2 3 - - 6 7 8 9` | 10 | 25.672 | 23.558 | 25.663 | 2.114 |
| `- 2 - 4 5 - 7 8 9` | 10 | 25.884 | 23.427 | 25.825 | 2.457 |
| `- - 3 4 5 6 - 8 9` | 10 | 26.024 | 23.719 | 25.976 | 2.305 |
| `- - - - 5 6 7 8 9` | 10 | 20.289 | 20.289 | 20.289 | 0.000 |
| `1 2 3 4 - - 7 8 9` | 11 | 32.123 | 25.352 | 31.989 | 6.771 |
| `1 2 3 - 5 6 - 8 9` | 11 | 32.395 | 25.406 | 32.075 | 6.989 |
| `1 2 - 4 5 6 7 - 9` | 11 | 33.146 | 25.927 | 33.019 | 7.219 |
| `1 - 3 4 5 6 7 8 -` | 11 | 33.806 | 27.176 | 33.705 | 6.630 |
| `1 - 3 - - 6 7 8 9` | 11 | 25.885 | 22.899 | 25.885 | 2.987 |
| `1 - - 4 5 - 7 8 9` | 11 | 26.441 | 23.479 | 26.440 | 2.962 |
| `- 2 3 - 5 - 7 8 9` | 11 | 24.420 | 21.816 | 24.410 | 2.604 |
| `- 2 - 4 5 6 - 8 9` | 11 | 27.189 | 23.880 | 27.049 | 3.309 |
| `- - 3 4 5 6 7 - 9` | 11 | 28.009 | 25.434 | 27.869 | 2.575 |
| `- - - 4 - 6 7 8 9` | 11 | 22.320 | 22.320 | 22.320 | 0.000 |
| `1 2 3 4 - 6 - 8 9` | 12 | 33.649 | 25.748 | 33.313 | 7.901 |
| `1 2 3 - 5 6 7 - 9` | 12 | 34.134 | 26.691 | 33.900 | 7.444 |
| `1 2 - 4 5 6 7 8 -` | 12 | 35.064 | 27.732 | 34.938 | 7.332 |
| `1 2 - - - 6 7 8 9` | 12 | 25.576 | 22.788 | 25.549 | 2.788 |
| `1 - 3 - 5 - 7 8 9` | 12 | 28.349 | 25.467 | 28.326 | 2.882 |
| `1 - - 4 5 6 - 8 9` | 12 | 26.967 | 24.380 | 26.966 | 2.586 |
| `- 2 3 4 - - 7 8 9` | 12 | 28.868 | 26.271 | 28.800 | 2.597 |
| `- 2 3 - 5 6 - 8 9` | 12 | 27.142 | 24.620 | 27.052 | 2.522 |
| `- 2 - 4 5 6 7 - 9` | 12 | 27.930 | 25.286 | 27.927 | 2.644 |
| `- - 3 4 5 6 7 8 -` | 12 | 30.311 | 27.507 | 30.192 | 2.804 |
| `- - 3 - - 6 7 8 9` | 12 | 22.962 | 22.390 | 22.962 | 0.572 |
| `- - - 4 5 - 7 8 9` | 12 | 21.159 | 20.684 | 21.159 | 0.475 |
| `1 2 3 4 5 - - 8 9` | 13 | 34.274 | 26.242 | 33.982 | 8.032 |
| `1 2 3 4 - 6 7 - 9` | 13 | 35.097 | 27.342 | 34.956 | 7.755 |
| `1 2 3 - 5 6 7 8 -` | 13 | 35.639 | 28.214 | 35.521 | 7.426 |
| `1 2 - - 5 - 7 8 9` | 13 | 28.519 | 24.765 | 28.441 | 3.755 |
| `1 - 3 4 - - 7 8 9` | 13 | 27.045 | 23.990 | 27.043 | 3.056 |
| `1 - 3 - 5 6 - 8 9` | 13 | 28.874 | 25.882 | 28.874 | 2.992 |
| `1 - - 4 5 6 7 - 9` | 13 | 29.029 | 25.514 | 28.980 | 3.515 |
| `- 2 3 4 - 6 - 8 9` | 13 | 29.572 | 26.179 | 29.533 | 3.393 |
| `- 2 3 - 5 6 7 - 9` | 13 | 29.283 | 25.650 | 29.268 | 3.633 |
| `- 2 - 4 5 6 7 8 -` | 13 | 30.745 | 27.388 | 30.641 | 3.357 |
| `- 2 - - - 6 7 8 9` | 13 | 23.225 | 22.323 | 23.225 | 0.903 |
| `- - 3 - 5 - 7 8 9` | 13 | 23.335 | 22.795 | 23.335 | 0.539 |
| `- - - 4 5 6 - 8 9` | 13 | 22.839 | 22.268 | 22.839 | 0.571 |
| `1 2 3 4 5 - 7 - 9` | 14 | 35.899 | 27.532 | 35.418 | 8.368 |
| `1 2 3 4 - 6 7 8 -` | 14 | 36.729 | 29.009 | 36.629 | 7.720 |
| `1 2 - 4 - - 7 8 9` | 14 | 30.591 | 26.733 | 30.591 | 3.858 |
| `1 2 - - 5 6 - 8 9` | 14 | 29.623 | 26.125 | 29.583 | 3.498 |
| `1 - 3 4 - 6 - 8 9` | 14 | 30.367 | 26.847 | 30.259 | 3.520 |
| `1 - 3 - 5 6 7 - 9` | 14 | 31.182 | 27.299 | 31.073 | 3.884 |
| `1 - - 4 5 6 7 8 -` | 14 | 30.954 | 27.175 | 30.875 | 3.779 |
| `1 - - - - 6 7 8 9` | 14 | 23.044 | 21.740 | 23.044 | 1.304 |
| `- 2 3 4 5 - - 8 9` | 14 | 30.555 | 26.955 | 30.397 | 3.600 |
| `- 2 3 4 - 6 7 - 9` | 14 | 30.867 | 27.624 | 30.859 | 3.243 |
| `- 2 3 - 5 6 7 8 -` | 14 | 30.712 | 27.411 | 30.591 | 3.302 |
| `- 2 - - 5 - 7 8 9` | 14 | 24.152 | 23.042 | 24.152 | 1.110 |
| `- - 3 4 - - 7 8 9` | 14 | 23.949 | 23.470 | 23.949 | 0.479 |
| `- - 3 - 5 6 - 8 9` | 14 | 22.328 | 21.522 | 22.322 | 0.805 |
| `- - - 4 5 6 7 - 9` | 14 | 24.448 | 23.813 | 24.442 | 0.635 |
| `1 2 3 4 5 6 - - 9` | 15 | 36.633 | 28.776 | 36.322 | 7.857 |
| `1 2 3 4 5 - 7 8 -` | 15 | 37.510 | 29.567 | 37.396 | 7.944 |
| `1 2 3 - - - 7 8 9` | 15 | 30.599 | 28.066 | 30.599 | 2.533 |
| `1 2 - 4 - 6 - 8 9` | 15 | 31.603 | 27.536 | 31.419 | 4.067 |
| `1 2 - - 5 6 7 - 9` | 15 | 31.099 | 26.872 | 30.972 | 4.227 |
| `1 - 3 4 5 - - 8 9` | 15 | 30.357 | 26.738 | 30.355 | 3.620 |
| `1 - 3 4 - 6 7 - 9` | 15 | 31.344 | 28.274 | 31.313 | 3.070 |
| `1 - 3 - 5 6 7 8 -` | 15 | 32.668 | 28.450 | 32.610 | 4.218 |
| `1 - - - 5 - 7 8 9` | 15 | 25.568 | 24.368 | 25.568 | 1.200 |
| `- 2 3 4 5 - 7 - 9` | 15 | 31.501 | 27.770 | 31.487 | 3.731 |
| `- 2 3 4 - 6 7 8 -` | 15 | 33.386 | 29.280 | 33.295 | 4.106 |
| `- 2 - 4 - - 7 8 9` | 15 | 26.333 | 25.603 | 26.333 | 0.730 |
| `- 2 - - 5 6 - 8 9` | 15 | 26.220 | 25.514 | 26.203 | 0.707 |
| `- - 3 4 - 6 - 8 9` | 15 | 26.420 | 25.651 | 26.420 | 0.770 |
| `- - 3 - 5 6 7 - 9` | 15 | 27.195 | 26.379 | 27.195 | 0.815 |
| `- - - 4 5 6 7 8 -` | 15 | 28.111 | 28.074 | 28.081 | 0.037 |
| `- - - - - 6 7 8 9` | 15 | 21.354 | 21.354 | 21.354 | 0.000 |
| `1 2 3 4 5 6 - 8 -` | 16 | 38.329 | 30.436 | 38.189 | 7.893 |
| `1 2 3 - - 6 - 8 9` | 16 | 32.024 | 28.260 | 32.019 | 3.764 |
| `1 2 - 4 5 - - 8 9` | 16 | 32.177 | 28.132 | 32.165 | 4.045 |
| `1 2 - 4 - 6 7 - 9` | 16 | 33.192 | 28.852 | 33.000 | 4.340 |
| `1 2 - - 5 6 7 8 -` | 16 | 32.068 | 28.012 | 32.047 | 4.056 |
| `1 - 3 4 5 - 7 - 9` | 16 | 32.505 | 28.635 | 32.278 | 3.870 |
| `1 - 3 4 - 6 7 8 -` | 16 | 33.480 | 29.528 | 33.440 | 3.952 |
| `1 - - 4 - - 7 8 9` | 16 | 26.563 | 25.336 | 26.563 | 1.228 |
| `1 - - - 5 6 - 8 9` | 16 | 27.178 | 26.298 | 27.178 | 0.880 |
| `- 2 3 4 5 6 - - 9` | 16 | 33.302 | 29.171 | 33.066 | 4.131 |
| `- 2 3 4 5 - 7 8 -` | 16 | 33.861 | 30.080 | 33.668 | 3.781 |
| `- 2 3 - - - 7 8 9` | 16 | 26.606 | 25.875 | 26.605 | 0.732 |
| `- 2 - 4 - 6 - 8 9` | 16 | 25.078 | 23.940 | 25.029 | 1.139 |
| `- 2 - - 5 6 7 - 9` | 16 | 26.556 | 25.223 | 26.556 | 1.332 |
| `- - 3 4 5 - - 8 9` | 16 | 25.336 | 24.380 | 25.336 | 0.955 |
| `- - 3 4 - 6 7 - 9` | 16 | 25.689 | 24.713 | 25.640 | 0.976 |
| `- - 3 - 5 6 7 8 -` | 16 | 28.596 | 27.661 | 28.579 | 0.935 |
| `- - - - 5 - 7 8 9` | 16 | 22.273 | 22.273 | 22.273 | 0.000 |
| `1 2 3 4 5 6 7 - -` | 17 | 39.466 | 32.581 | 39.339 | 6.885 |
| `1 2 3 - 5 - - 8 9` | 17 | 32.750 | 28.747 | 32.611 | 4.003 |
| `1 2 3 - - 6 7 - 9` | 17 | 33.922 | 29.424 | 33.775 | 4.498 |
| `1 2 - 4 5 - 7 - 9` | 17 | 33.831 | 29.014 | 33.469 | 4.818 |
| `1 2 - 4 - 6 7 8 -` | 17 | 34.715 | 29.650 | 34.650 | 5.065 |
| `1 - 3 4 5 6 - - 9` | 17 | 33.490 | 28.925 | 33.333 | 4.565 |
| `1 - 3 4 5 - 7 8 -` | 17 | 34.048 | 29.819 | 34.044 | 4.229 |
| `1 - 3 - - - 7 8 9` | 17 | 26.952 | 25.658 | 26.952 | 1.294 |
| `1 - - 4 - 6 - 8 9` | 17 | 28.869 | 28.082 | 28.869 | 0.787 |
| `1 - - - 5 6 7 - 9` | 17 | 28.709 | 27.340 | 28.709 | 1.369 |
| `- 2 3 4 5 6 - 8 -` | 17 | 35.450 | 30.508 | 35.443 | 4.942 |
| `- 2 3 - - 6 - 8 9` | 17 | 26.667 | 25.785 | 26.667 | 0.882 |
| `- 2 - 4 5 - - 8 9` | 17 | 28.930 | 28.173 | 28.930 | 0.757 |
| `- 2 - 4 - 6 7 - 9` | 17 | 28.220 | 27.253 | 28.220 | 0.967 |
| `- 2 - - 5 6 7 8 -` | 17 | 29.231 | 27.953 | 29.231 | 1.278 |
| `- - 3 4 5 - 7 - 9` | 17 | 27.962 | 26.862 | 27.962 | 1.100 |
| `- - 3 4 - 6 7 8 -` | 17 | 29.399 | 28.383 | 29.353 | 1.016 |
| `- - - 4 - - 7 8 9` | 17 | 23.722 | 23.722 | 23.722 | 0.000 |
| `- - - - 5 6 - 8 9` | 17 | 23.100 | 23.100 | 23.100 | 0.000 |
| `1 2 3 4 - - - 8 9` | 18 | 33.497 | 29.411 | 33.477 | 4.086 |
| `1 2 3 - 5 - 7 - 9` | 18 | 34.571 | 30.404 | 34.465 | 4.167 |
| `1 2 3 - - 6 7 8 -` | 18 | 35.176 | 31.119 | 35.169 | 4.057 |
| `1 2 - 4 5 6 - - 9` | 18 | 34.288 | 30.246 | 34.120 | 4.043 |
| `1 2 - 4 5 - 7 8 -` | 18 | 35.748 | 31.046 | 35.704 | 4.703 |
| `1 2 - - - - 7 8 9` | 18 | 27.104 | 25.913 | 27.104 | 1.191 |
| `1 - 3 4 5 6 - 8 -` | 18 | 35.317 | 31.060 | 35.239 | 4.257 |
| `1 - 3 - - 6 - 8 9` | 18 | 29.231 | 28.185 | 29.231 | 1.047 |
| `1 - - 4 5 - - 8 9` | 18 | 26.547 | 25.533 | 26.545 | 1.013 |
| `1 - - 4 - 6 7 - 9` | 18 | 30.356 | 29.561 | 30.345 | 0.795 |
| `1 - - - 5 6 7 8 -` | 18 | 28.500 | 26.849 | 28.500 | 1.650 |
| `- 2 3 4 5 6 7 - -` | 18 | 36.797 | 32.958 | 36.776 | 3.839 |
| `- 2 3 - 5 - - 8 9` | 18 | 28.213 | 27.153 | 28.159 | 1.060 |
| `- 2 3 - - 6 7 - 9` | 18 | 30.135 | 29.008 | 30.133 | 1.128 |
| `- 2 - 4 5 - 7 - 9` | 18 | 26.978 | 25.744 | 26.974 | 1.234 |
| `- 2 - 4 - 6 7 8 -` | 18 | 29.768 | 28.221 | 29.740 | 1.547 |
| `- - 3 4 5 6 - - 9` | 18 | 30.420 | 29.314 | 30.419 | 1.106 |
| `- - 3 4 5 - 7 8 -` | 18 | 29.716 | 28.254 | 29.716 | 1.462 |
| `- - 3 - - - 7 8 9` | 18 | 25.645 | 25.645 | 25.645 | 0.000 |
| `- - - 4 - 6 - 8 9` | 18 | 24.483 | 24.483 | 24.483 | 0.000 |
| `- - - - 5 6 7 - 9` | 18 | 24.804 | 24.804 | 24.804 | 0.000 |
| `1 2 3 4 - - 7 - 9` | 19 | 35.391 | 30.500 | 35.220 | 4.891 |
| `1 2 3 - 5 6 - - 9` | 19 | 35.815 | 30.931 | 35.633 | 4.883 |
| `1 2 3 - 5 - 7 8 -` | 19 | 36.241 | 31.617 | 36.227 | 4.624 |
| `1 2 - 4 5 6 - 8 -` | 19 | 36.835 | 31.376 | 36.761 | 5.459 |
| `1 2 - - - 6 - 8 9` | 19 | 29.737 | 28.146 | 29.652 | 1.591 |
| `1 - 3 4 5 6 7 - -` | 19 | 37.126 | 32.332 | 36.957 | 4.793 |
| `1 - 3 - 5 - - 8 9` | 19 | 28.920 | 27.794 | 28.920 | 1.126 |
| `1 - 3 - - 6 7 - 9` | 19 | 30.256 | 29.215 | 30.256 | 1.040 |
| `1 - - 4 5 - 7 - 9` | 19 | 30.837 | 29.530 | 30.837 | 1.307 |
| `1 - - 4 - 6 7 8 -` | 19 | 31.801 | 30.101 | 31.801 | 1.701 |
| `- 2 3 4 - - - 8 9` | 19 | 31.411 | 30.480 | 31.411 | 0.930 |
| `- 2 3 - 5 - 7 - 9` | 19 | 29.412 | 27.889 | 29.412 | 1.523 |
| `- 2 3 - - 6 7 8 -` | 19 | 31.854 | 30.765 | 31.792 | 1.089 |
| `- 2 - 4 5 6 - - 9` | 19 | 31.000 | 29.824 | 31.000 | 1.177 |
| `- 2 - 4 5 - 7 8 -` | 19 | 32.217 | 31.048 | 32.093 | 1.169 |
| `- 2 - - - - 7 8 9` | 19 | 26.017 | 25.693 | 26.017 | 0.324 |
| `- - 3 4 5 6 - 8 -` | 19 | 32.285 | 31.204 | 32.233 | 1.081 |
| `- - 3 - - 6 - 8 9` | 19 | 24.896 | 24.635 | 24.896 | 0.261 |
| `- - - 4 5 - - 8 9` | 19 | 23.835 | 23.627 | 23.835 | 0.208 |
| `- - - 4 - 6 7 - 9` | 19 | 26.110 | 26.110 | 26.110 | 0.000 |
| `- - - - 5 6 7 8 -` | 19 | 26.144 | 26.144 | 26.144 | 0.000 |
| `1 2 3 4 - 6 - - 9` | 20 | 36.481 | 31.251 | 36.169 | 5.231 |
| `1 2 3 4 - - 7 8 -` | 20 | 37.140 | 32.260 | 36.971 | 4.880 |
| `1 2 3 - 5 6 - 8 -` | 20 | 37.228 | 32.142 | 36.801 | 5.086 |
| `1 2 - 4 5 6 7 - -` | 20 | 37.863 | 33.038 | 37.757 | 4.825 |
| `1 2 - - 5 - - 8 9` | 20 | 31.619 | 29.997 | 31.619 | 1.622 |
| `1 2 - - - 6 7 - 9` | 20 | 31.090 | 29.338 | 31.004 | 1.753 |
| `1 - 3 4 - - - 8 9` | 20 | 30.369 | 29.011 | 30.369 | 1.358 |
| `1 - 3 - 5 - 7 - 9` | 20 | 32.718 | 31.558 | 32.558 | 1.160 |
| `1 - 3 - - 6 7 8 -` | 20 | 32.090 | 30.364 | 32.089 | 1.726 |
| `1 - - 4 5 6 - - 9` | 20 | 29.697 | 28.393 | 29.697 | 1.305 |
| `1 - - 4 5 - 7 8 -` | 20 | 32.438 | 31.072 | 32.435 | 1.366 |
| `1 - - - - - 7 8 9` | 20 | 26.052 | 25.500 | 26.052 | 0.551 |
| `- 2 3 4 - - 7 - 9` | 20 | 30.648 | 29.232 | 30.648 | 1.416 |
| `- 2 3 - 5 6 - - 9` | 20 | 31.740 | 30.589 | 31.667 | 1.152 |
| `- 2 3 - 5 - 7 8 -` | 20 | 30.559 | 29.194 | 30.559 | 1.365 |
| `- 2 - 4 5 6 - 8 -` | 20 | 32.845 | 30.950 | 32.845 | 1.894 |
| `- 2 - - - 6 - 8 9` | 20 | 26.432 | 26.148 | 26.432 | 0.284 |
| `- - 3 4 5 6 7 - -` | 20 | 34.087 | 32.865 | 34.073 | 1.222 |
| `- - 3 - 5 - - 8 9` | 20 | 25.412 | 25.202 | 25.412 | 0.210 |
| `- - 3 - - 6 7 - 9` | 20 | 26.421 | 26.121 | 26.421 | 0.300 |
| `- - - 4 5 - 7 - 9` | 20 | 25.842 | 25.549 | 25.842 | 0.293 |
| `- - - 4 - 6 7 8 -` | 20 | 27.982 | 27.982 | 27.982 | 0.000 |
| `1 2 3 4 5 - - - 9` | 21 | 37.234 | 31.689 | 36.952 | 5.545 |
| `1 2 3 4 - 6 - 8 -` | 21 | 38.476 | 32.759 | 38.236 | 5.717 |
| `1 2 3 - 5 6 7 - -` | 21 | 38.638 | 33.738 | 38.501 | 4.900 |
| `1 2 - 4 - - - 8 9` | 21 | 32.836 | 31.916 | 32.836 | 0.920 |
| `1 2 - - 5 - 7 - 9` | 21 | 32.968 | 31.381 | 32.968 | 1.588 |
| `1 2 - - - 6 7 8 -` | 21 | 31.533 | 30.002 | 31.528 | 1.531 |
| `1 - 3 4 - - 7 - 9` | 21 | 31.718 | 30.622 | 31.718 | 1.096 |
| `1 - 3 - 5 6 - - 9` | 21 | 33.353 | 31.568 | 33.353 | 1.785 |
| `1 - 3 - 5 - 7 8 -` | 21 | 34.002 | 32.481 | 33.994 | 1.521 |
| `1 - - 4 5 6 - 8 -` | 21 | 33.451 | 32.038 | 33.449 | 1.413 |
| `1 - - - - 6 - 8 9` | 21 | 28.197 | 27.858 | 28.197 | 0.339 |
| `- 2 3 4 - 6 - - 9` | 21 | 33.390 | 31.690 | 33.390 | 1.700 |
| `- 2 3 4 - - 7 8 -` | 21 | 34.507 | 33.355 | 34.479 | 1.152 |
| `- 2 3 - 5 6 - 8 -` | 21 | 32.844 | 31.114 | 32.841 | 1.730 |
| `- 2 - 4 5 6 7 - -` | 21 | 34.382 | 32.874 | 34.363 | 1.508 |
| `- 2 - - 5 - - 8 9` | 21 | 28.603 | 28.603 | 28.603 | 0.000 |
| `- 2 - - - 6 7 - 9` | 21 | 28.134 | 27.773 | 28.134 | 0.361 |
| `- - 3 4 - - - 8 9` | 21 | 27.755 | 27.745 | 27.755 | 0.010 |
| `- - 3 - 5 - 7 - 9` | 21 | 28.604 | 28.584 | 28.604 | 0.020 |
| `- - 3 - - 6 7 8 -` | 21 | 29.684 | 29.684 | 29.684 | 0.000 |
| `- - - 4 5 6 - - 9` | 21 | 27.008 | 26.665 | 27.008 | 0.343 |
| `- - - 4 5 - 7 8 -` | 21 | 28.785 | 28.775 | 28.785 | 0.010 |
| `- - - - - - 7 8 9` | 21 | 25.307 | 25.307 | 25.307 | 0.000 |
| `1 2 3 4 5 - - 8 -` | 22 | 38.987 | 33.578 | 38.733 | 5.410 |
| `1 2 3 4 - 6 7 - -` | 22 | 39.474 | 34.391 | 39.399 | 5.083 |
| `1 2 3 - - - - 8 9` | 22 | 31.616 | 30.780 | 31.616 | 0.836 |
| `1 2 - 4 - - 7 - 9` | 22 | 34.522 | 32.618 | 34.522 | 1.903 |
| `1 2 - - 5 6 - - 9` | 22 | 34.022 | 32.329 | 33.931 | 1.693 |
| `1 2 - - 5 - 7 8 -` | 22 | 34.247 | 32.345 | 34.207 | 1.902 |
| `1 - 3 4 - 6 - - 9` | 22 | 33.842 | 32.509 | 33.674 | 1.333 |
| `1 - 3 4 - - 7 8 -` | 22 | 32.978 | 31.314 | 32.978 | 1.664 |
| `1 - 3 - 5 6 - 8 -` | 22 | 34.460 | 33.171 | 34.460 | 1.289 |
| `1 - - 4 5 6 7 - -` | 22 | 34.791 | 32.773 | 34.757 | 2.018 |
| `1 - - - 5 - - 8 9` | 22 | 28.194 | 27.912 | 28.194 | 0.282 |
| `1 - - - - 6 7 - 9` | 22 | 29.293 | 28.930 | 29.293 | 0.363 |
| `- 2 3 4 5 - - - 9` | 22 | 34.175 | 32.425 | 34.099 | 1.750 |
| `- 2 3 4 - 6 - 8 -` | 22 | 35.077 | 33.116 | 35.077 | 1.961 |
| `- 2 3 - 5 6 7 - -` | 22 | 35.071 | 33.161 | 35.056 | 1.911 |
| `- 2 - 4 - - - 8 9` | 22 | 29.081 | 29.081 | 29.081 | 0.000 |
| `- 2 - - 5 - 7 - 9` | 22 | 27.485 | 26.980 | 27.485 | 0.505 |
| `- 2 - - - 6 7 8 -` | 22 | 29.566 | 29.195 | 29.566 | 0.370 |
| `- - 3 4 - - 7 - 9` | 22 | 28.298 | 28.103 | 28.298 | 0.195 |
| `- - 3 - 5 6 - - 9` | 22 | 28.671 | 28.286 | 28.671 | 0.385 |
| `- - 3 - 5 - 7 8 -` | 22 | 29.095 | 28.737 | 29.095 | 0.358 |
| `- - - 4 5 6 - 8 -` | 22 | 30.257 | 30.257 | 30.257 | 0.000 |
| `- - - - - 6 - 8 9` | 22 | 25.807 | 25.807 | 25.807 | 0.000 |
| `1 2 3 4 5 - 7 - -` | 23 | 40.138 | 35.032 | 39.869 | 5.106 |
| `1 2 3 - - - 7 - 9` | 23 | 34.880 | 33.589 | 34.880 | 1.291 |
| `1 2 - 4 - 6 - - 9` | 23 | 35.202 | 33.510 | 35.202 | 1.692 |
| `1 2 - 4 - - 7 8 -` | 23 | 36.164 | 33.869 | 36.164 | 2.295 |
| `1 2 - - 5 6 - 8 -` | 23 | 35.254 | 33.181 | 35.170 | 2.074 |
| `1 - 3 4 5 - - - 9` | 23 | 33.965 | 31.909 | 33.965 | 2.056 |
| `1 - 3 4 - 6 - 8 -` | 23 | 35.993 | 34.054 | 35.899 | 1.939 |
| `1 - 3 - 5 6 7 - -` | 23 | 36.472 | 34.256 | 36.464 | 2.216 |
| `1 - - 4 - - - 8 9` | 23 | 28.789 | 28.518 | 28.789 | 0.270 |
| `1 - - - 5 - 7 - 9` | 23 | 31.354 | 31.354 | 31.354 | 0.000 |
| `1 - - - - 6 7 8 -` | 23 | 29.298 | 28.655 | 29.298 | 0.642 |
| `- 2 3 4 5 - - 8 -` | 23 | 36.019 | 33.924 | 36.010 | 2.095 |
| `- 2 3 4 - 6 7 - -` | 23 | 36.545 | 34.829 | 36.486 | 1.716 |
| `- 2 3 - - - - 8 9` | 23 | 29.417 | 29.402 | 29.417 | 0.015 |
| `- 2 - 4 - - 7 - 9` | 23 | 28.936 | 28.537 | 28.936 | 0.399 |
| `- 2 - - 5 6 - - 9` | 23 | 30.746 | 30.720 | 30.746 | 0.026 |
| `- 2 - - 5 - 7 8 -` | 23 | 30.465 | 30.141 | 30.465 | 0.324 |
| `- - 3 4 - 6 - - 9` | 23 | 29.707 | 29.298 | 29.707 | 0.409 |
| `- - 3 4 - - 7 8 -` | 23 | 29.817 | 29.486 | 29.817 | 0.331 |
| `- - 3 - 5 6 - 8 -` | 23 | 29.465 | 29.042 | 29.465 | 0.423 |
| `- - - 4 5 6 7 - -` | 23 | 31.726 | 31.721 | 31.721 | 0.005 |
| `- - - - 5 - - 8 9` | 23 | 26.378 | 26.378 | 26.378 | 0.000 |
| `- - - - - 6 7 - 9` | 23 | 26.942 | 26.942 | 26.942 | 0.000 |
| `1 2 3 4 5 6 - - -` | 24 | 40.698 | 36.448 | 40.554 | 4.250 |
| `1 2 3 - - 6 - - 9` | 24 | 35.911 | 33.840 | 35.911 | 2.071 |
| `1 2 3 - - - 7 8 -` | 24 | 36.421 | 35.157 | 36.421 | 1.265 |
| `1 2 - 4 5 - - - 9` | 24 | 35.670 | 33.986 | 35.604 | 1.683 |
| `1 2 - 4 - 6 - 8 -` | 24 | 36.763 | 34.807 | 36.763 | 1.955 |
| `1 2 - - 5 6 7 - -` | 24 | 36.068 | 34.249 | 36.059 | 1.819 |
| `1 - 3 4 5 - - 8 -` | 24 | 36.185 | 34.196 | 36.184 | 1.990 |
| `1 - 3 4 - 6 7 - -` | 24 | 36.999 | 35.040 | 36.990 | 1.959 |
| `1 - 3 - - - - 8 9` | 24 | 29.941 | 29.650 | 29.941 | 0.291 |
| `1 - - 4 - - 7 - 9` | 24 | 32.180 | 32.180 | 32.180 | 0.000 |
| `1 - - - 5 6 - - 9` | 24 | 31.405 | 31.160 | 31.405 | 0.245 |
| `1 - - - 5 - 7 8 -` | 24 | 31.476 | 31.061 | 31.476 | 0.415 |
| `- 2 3 4 5 - 7 - -` | 24 | 37.432 | 35.318 | 37.428 | 2.115 |
| `- 2 3 - - - 7 - 9` | 24 | 30.660 | 30.283 | 30.660 | 0.377 |
| `- 2 - 4 - 6 - - 9` | 24 | 30.961 | 30.782 | 30.961 | 0.179 |
| `- 2 - 4 - - 7 8 -` | 24 | 33.098 | 33.098 | 33.098 | 0.000 |
| `- 2 - - 5 6 - 8 -` | 24 | 31.662 | 31.200 | 31.662 | 0.462 |
| `- - 3 4 5 - - - 9` | 24 | 30.646 | 30.202 | 30.646 | 0.445 |
| `- - 3 4 - 6 - 8 -` | 24 | 33.249 | 33.249 | 33.249 | 0.000 |
| `- - 3 - 5 6 7 - -` | 24 | 33.903 | 33.903 | 33.903 | 0.000 |
| `- - - 4 - - - 8 9` | 24 | 27.418 | 27.418 | 27.418 | 0.000 |
| `- - - - 5 - 7 - 9` | 24 | 27.898 | 27.898 | 27.898 | 0.000 |
| `- - - - - 6 7 8 -` | 24 | 28.164 | 28.164 | 28.164 | 0.000 |
| `1 2 3 - 5 - - - 9` | 25 | 36.533 | 34.544 | 36.533 | 1.989 |
| `1 2 3 - - 6 - 8 -` | 25 | 37.514 | 35.337 | 37.514 | 2.178 |
| `1 2 - 4 5 - - 8 -` | 25 | 37.592 | 35.481 | 37.571 | 2.111 |
| `1 2 - 4 - 6 7 - -` | 25 | 38.247 | 35.684 | 38.144 | 2.563 |
| `1 2 - - - - - 8 9` | 25 | 30.532 | 30.201 | 30.532 | 0.331 |
| `1 - 3 4 5 - 7 - -` | 25 | 37.756 | 35.637 | 37.751 | 2.118 |
| `1 - 3 - - - 7 - 9` | 25 | 32.552 | 32.516 | 32.552 | 0.035 |
| `1 - - 4 - 6 - - 9` | 25 | 32.378 | 32.355 | 32.378 | 0.024 |
| `1 - - 4 - - 7 8 -` | 25 | 32.449 | 32.027 | 32.449 | 0.422 |
| `1 - - - 5 6 - 8 -` | 25 | 32.995 | 32.716 | 32.995 | 0.279 |
| `- 2 3 4 5 6 - - -` | 25 | 38.645 | 36.722 | 38.630 | 1.924 |
| `- 2 3 - - 6 - - 9` | 25 | 31.559 | 31.160 | 31.559 | 0.400 |
| `- 2 3 - - - 7 8 -` | 25 | 33.369 | 33.351 | 33.369 | 0.018 |
| `- 2 - 4 5 - - - 9` | 25 | 31.477 | 31.050 | 31.477 | 0.428 |
| `- 2 - 4 - 6 - 8 -` | 25 | 30.262 | 29.678 | 30.262 | 0.584 |
| `- 2 - - 5 6 7 - -` | 25 | 33.477 | 32.998 | 33.477 | 0.479 |
| `- - 3 4 5 - - 8 -` | 25 | 32.680 | 32.183 | 32.680 | 0.497 |
| `- - 3 4 - 6 7 - -` | 25 | 32.542 | 32.060 | 32.530 | 0.482 |
| `- - 3 - - - - 8 9` | 25 | 28.878 | 28.878 | 28.878 | 0.000 |
| `- - - 4 - - 7 - 9` | 25 | 28.866 | 28.866 | 28.866 | 0.000 |
| `- - - - 5 6 - - 9` | 25 | 28.766 | 28.766 | 28.766 | 0.000 |
| `- - - - 5 - 7 8 -` | 25 | 29.117 | 29.117 | 29.117 | 0.000 |
| `1 2 3 4 - - - - 9` | 26 | 36.991 | 35.234 | 36.984 | 1.756 |
| `1 2 3 - 5 - - 8 -` | 26 | 38.136 | 35.870 | 38.136 | 2.266 |
| `1 2 3 - - 6 7 - -` | 26 | 38.792 | 36.631 | 38.776 | 2.161 |
| `1 2 - 4 5 - 7 - -` | 26 | 38.968 | 36.677 | 38.854 | 2.291 |
| `1 2 - - - - 7 - 9` | 26 | 33.158 | 32.756 | 33.158 | 0.402 |
| `1 - 3 4 5 6 - - -` | 26 | 38.856 | 36.535 | 38.754 | 2.321 |
| `1 - 3 - - 6 - - 9` | 26 | 32.500 | 32.102 | 32.500 | 0.398 |
| `1 - 3 - - - 7 8 -` | 26 | 33.861 | 33.429 | 33.861 | 0.432 |
| `1 - - 4 5 - - - 9` | 26 | 31.005 | 30.550 | 31.005 | 0.455 |
| `1 - - 4 - 6 - 8 -` | 26 | 35.474 | 35.474 | 35.474 | 0.000 |
| `1 - - - 5 6 7 - -` | 26 | 32.849 | 32.186 | 32.849 | 0.663 |
| `- 2 3 - 5 - - - 9` | 26 | 33.703 | 33.551 | 33.703 | 0.152 |
| `- 2 3 - - 6 - 8 -` | 26 | 33.426 | 32.985 | 33.426 | 0.442 |
| `- 2 - 4 5 - - 8 -` | 26 | 35.555 | 35.555 | 35.555 | 0.000 |
| `- 2 - 4 - 6 7 - -` | 26 | 35.028 | 34.691 | 35.028 | 0.337 |
| `- 2 - - - - - 8 9` | 26 | 30.285 | 30.285 | 30.285 | 0.000 |
| `- - 3 4 5 - 7 - -` | 26 | 34.986 | 34.461 | 34.986 | 0.525 |
| `- - 3 - - - 7 - 9` | 26 | 30.266 | 30.266 | 30.266 | 0.000 |
| `- - - 4 - 6 - - 9` | 26 | 29.692 | 29.692 | 29.692 | 0.000 |
| `- - - 4 - - 7 8 -` | 26 | 30.502 | 30.502 | 30.502 | 0.000 |
| `- - - - 5 6 - 8 -` | 26 | 29.980 | 29.980 | 29.980 | 0.000 |
| `1 2 3 4 - - - 8 -` | 27 | 38.835 | 36.521 | 38.828 | 2.313 |
| `1 2 3 - 5 - 7 - -` | 27 | 39.467 | 37.277 | 39.418 | 2.189 |
| `1 2 - 4 5 6 - - -` | 27 | 39.672 | 37.363 | 39.662 | 2.309 |
| `1 2 - - - 6 - - 9` | 27 | 35.126 | 34.740 | 35.126 | 0.386 |
| `1 2 - - - - 7 8 -` | 27 | 33.991 | 33.530 | 33.991 | 0.461 |
| `1 - 3 - 5 - - - 9` | 27 | 33.980 | 33.624 | 33.980 | 0.356 |
| `1 - 3 - - 6 - 8 -` | 27 | 35.815 | 35.810 | 35.810 | 0.005 |
| `1 - - 4 5 - - 8 -` | 27 | 33.929 | 33.694 | 33.913 | 0.235 |
| `1 - - 4 - 6 7 - -` | 27 | 35.412 | 34.936 | 35.412 | 0.476 |
| `1 - - - - - - 8 9` | 27 | 30.526 | 30.403 | 30.526 | 0.123 |
| `- 2 3 4 - - - - 9` | 27 | 33.872 | 33.498 | 33.872 | 0.374 |
| `- 2 3 - 5 - - 8 -` | 27 | 33.415 | 32.843 | 33.415 | 0.572 |
| `- 2 3 - - 6 7 - -` | 27 | 36.594 | 36.573 | 36.573 | 0.022 |
| `- 2 - 4 5 - 7 - -` | 27 | 34.401 | 33.863 | 34.369 | 0.538 |
| `- 2 - - - - 7 - 9` | 27 | 30.502 | 30.361 | 30.502 | 0.140 |
| `- - 3 4 5 6 - - -` | 27 | 37.100 | 37.089 | 37.089 | 0.011 |
| `- - 3 - - 6 - - 9` | 27 | 29.920 | 29.801 | 29.920 | 0.119 |
| `- - 3 - - - 7 8 -` | 27 | 31.835 | 31.835 | 31.835 | 0.000 |
| `- - - 4 5 - - - 9` | 27 | 29.414 | 29.306 | 29.414 | 0.108 |
| `- - - 4 - 6 - 8 -` | 27 | 31.305 | 31.305 | 31.305 | 0.000 |
| `- - - - 5 6 7 - -` | 27 | 31.522 | 31.522 | 31.522 | 0.000 |
| `1 2 3 4 - - 7 - -` | 28 | 40.054 | 37.726 | 40.044 | 2.328 |
| `1 2 3 - 5 6 - - -` | 28 | 40.217 | 38.090 | 40.086 | 2.128 |
| `1 2 - - 5 - - - 9` | 28 | 36.582 | 36.582 | 36.582 | 0.000 |
| `1 2 - - - 6 - 8 -` | 28 | 35.985 | 35.555 | 35.985 | 0.431 |
| `1 - 3 4 - - - - 9` | 28 | 35.085 | 34.961 | 35.085 | 0.124 |
| `1 - 3 - 5 - - 8 -` | 28 | 34.918 | 34.542 | 34.918 | 0.376 |
| `1 - 3 - - 6 7 - -` | 28 | 36.617 | 36.148 | 36.617 | 0.469 |
| `1 - - 4 5 - 7 - -` | 28 | 37.362 | 37.086 | 37.362 | 0.276 |
| `1 - - - - - 7 - 9` | 28 | 32.906 | 32.906 | 32.906 | 0.000 |
| `- 2 3 4 - - - 8 -` | 28 | 37.451 | 37.451 | 37.451 | 0.000 |
| `- 2 3 - 5 - 7 - -` | 28 | 35.472 | 34.834 | 35.472 | 0.639 |
| `- 2 - 4 5 6 - - -` | 28 | 37.621 | 37.195 | 37.621 | 0.427 |
| `- 2 - - - 6 - - 9` | 28 | 32.316 | 32.316 | 32.316 | 0.000 |
| `- 2 - - - - 7 8 -` | 28 | 33.114 | 33.114 | 33.114 | 0.000 |
| `- - 3 - 5 - - - 9` | 28 | 31.801 | 31.801 | 31.801 | 0.000 |
| `- - 3 - - 6 - 8 -` | 28 | 32.577 | 32.577 | 32.577 | 0.000 |
| `- - - 4 5 - - 8 -` | 28 | 32.115 | 32.115 | 32.115 | 0.000 |
| `- - - 4 - 6 7 - -` | 28 | 32.780 | 32.780 | 32.780 | 0.000 |
| `- - - - - - - 8 9` | 28 | 30.373 | 30.373 | 30.373 | 0.000 |
| `1 2 3 4 - 6 - - -` | 29 | 40.676 | 38.593 | 40.644 | 2.083 |
| `1 2 - 4 - - - - 9` | 29 | 36.443 | 36.443 | 36.443 | 0.000 |
| `1 2 - - 5 - - 8 -` | 29 | 37.525 | 37.125 | 37.525 | 0.400 |
| `1 2 - - - 6 7 - -` | 29 | 36.382 | 35.883 | 36.348 | 0.499 |
| `1 - 3 4 - - - 8 -` | 29 | 36.111 | 35.621 | 36.111 | 0.490 |
| `1 - 3 - 5 - 7 - -` | 29 | 38.523 | 38.468 | 38.468 | 0.055 |
| `1 - - 4 5 6 - - -` | 29 | 36.786 | 36.190 | 36.786 | 0.597 |
| `1 - - - - 6 - - 9` | 29 | 33.546 | 33.546 | 33.546 | 0.000 |
| `1 - - - - - 7 8 -` | 29 | 33.076 | 32.914 | 33.076 | 0.162 |
| `- 2 3 4 - - 7 - -` | 29 | 37.271 | 36.859 | 37.271 | 0.411 |
| `- 2 3 - 5 6 - - -` | 29 | 37.801 | 37.480 | 37.781 | 0.322 |
| `- 2 - - 5 - - - 9` | 29 | 33.023 | 33.023 | 33.023 | 0.000 |
| `- 2 - - - 6 - 8 -` | 29 | 32.529 | 32.402 | 32.529 | 0.128 |
| `- - 3 4 - - - - 9` | 29 | 32.577 | 32.577 | 32.577 | 0.000 |
| `- - 3 - 5 - - 8 -` | 29 | 32.060 | 31.953 | 32.060 | 0.107 |
| `- - 3 - - 6 7 - -` | 29 | 33.986 | 33.986 | 33.986 | 0.000 |
| `- - - 4 5 - 7 - -` | 29 | 33.940 | 33.940 | 33.940 | 0.000 |
| `- - - - - - 7 - 9` | 29 | 31.463 | 31.463 | 31.463 | 0.000 |
| `1 2 3 4 5 - - - -` | 30 | 41.101 | 39.372 | 40.999 | 1.729 |
| `1 2 3 - - - - - 9` | 30 | 35.781 | 35.692 | 35.781 | 0.089 |
| `1 2 - 4 - - - 8 -` | 30 | 38.922 | 38.922 | 38.922 | 0.000 |
| `1 2 - - 5 - 7 - -` | 30 | 38.250 | 37.877 | 38.250 | 0.373 |
| `1 - 3 4 - - 7 - -` | 30 | 36.915 | 36.447 | 36.915 | 0.468 |
| `1 - 3 - 5 6 - - -` | 30 | 39.268 | 38.906 | 39.268 | 0.362 |
| `1 - - - 5 - - - 9` | 30 | 33.765 | 33.765 | 33.765 | 0.000 |
| `1 - - - - 6 - 8 -` | 30 | 34.960 | 34.960 | 34.960 | 0.000 |
| `- 2 3 4 - 6 - - -` | 30 | 39.396 | 39.029 | 39.396 | 0.366 |
| `- 2 - 4 - - - - 9` | 30 | 33.310 | 33.310 | 33.310 | 0.000 |
| `- 2 - - 5 - - 8 -` | 30 | 34.482 | 34.482 | 34.482 | 0.000 |
| `- 2 - - - 6 7 - -` | 30 | 35.138 | 35.138 | 35.138 | 0.000 |
| `- - 3 4 - - - 8 -` | 30 | 34.438 | 34.438 | 34.438 | 0.000 |
| `- - 3 - 5 - 7 - -` | 30 | 35.071 | 35.071 | 35.071 | 0.000 |
| `- - - 4 5 6 - - -` | 30 | 35.014 | 35.014 | 35.014 | 0.000 |
| `- - - - - 6 - - 9` | 30 | 32.065 | 32.065 | 32.065 | 0.000 |
| `- - - - - - 7 8 -` | 30 | 32.625 | 32.625 | 32.625 | 0.000 |
| `1 2 3 - - - - 8 -` | 31 | 38.135 | 38.022 | 38.135 | 0.112 |
| `1 2 - 4 - - 7 - -` | 31 | 40.040 | 39.704 | 40.040 | 0.335 |
| `1 2 - - 5 6 - - -` | 31 | 38.719 | 38.334 | 38.685 | 0.385 |
| `1 - 3 4 - 6 - - -` | 31 | 39.567 | 39.293 | 39.517 | 0.274 |
| `1 - - 4 - - - - 9` | 31 | 34.113 | 34.113 | 34.113 | 0.000 |
| `1 - - - 5 - - 8 -` | 31 | 35.144 | 35.144 | 35.144 | 0.000 |
| `1 - - - - 6 7 - -` | 31 | 34.877 | 34.738 | 34.877 | 0.139 |
| `- 2 3 4 5 - - - -` | 31 | 40.117 | 39.807 | 40.100 | 0.310 |
| `- 2 3 - - - - - 9` | 31 | 34.070 | 34.070 | 34.070 | 0.000 |
| `- 2 - 4 - - - 8 -` | 31 | 35.078 | 35.078 | 35.078 | 0.000 |
| `- 2 - - 5 - 7 - -` | 31 | 34.789 | 34.684 | 34.789 | 0.105 |
| `- - 3 4 - - 7 - -` | 31 | 34.745 | 34.657 | 34.745 | 0.089 |
| `- - 3 - 5 6 - - -` | 31 | 36.076 | 36.076 | 36.076 | 0.000 |
| `- - - - 5 - - - 9` | 31 | 32.728 | 32.728 | 32.728 | 0.000 |
| `- - - - - 6 - 8 -` | 31 | 33.215 | 33.215 | 33.215 | 0.000 |
| `1 2 3 - - - 7 - -` | 32 | 40.496 | 40.366 | 40.496 | 0.130 |
| `1 2 - 4 - 6 - - -` | 32 | 40.678 | 40.447 | 40.678 | 0.231 |
| `1 - 3 4 5 - - - -` | 32 | 39.938 | 39.512 | 39.938 | 0.426 |
| `1 - 3 - - - - - 9` | 32 | 34.921 | 34.921 | 34.921 | 0.000 |
| `1 - - 4 - - - 8 -` | 32 | 35.793 | 35.793 | 35.793 | 0.000 |
| `1 - - - 5 - 7 - -` | 32 | 36.724 | 36.724 | 36.724 | 0.000 |
| `- 2 3 - - - - 8 -` | 32 | 35.751 | 35.751 | 35.751 | 0.000 |
| `- 2 - 4 - - 7 - -` | 32 | 36.660 | 36.660 | 36.660 | 0.000 |
| `- 2 - - 5 6 - - -` | 32 | 37.088 | 37.088 | 37.088 | 0.000 |
| `- - 3 4 - 6 - - -` | 32 | 37.049 | 37.049 | 37.049 | 0.000 |
| `- - - 4 - - - - 9` | 32 | 33.454 | 33.454 | 33.454 | 0.000 |
| `- - - - 5 - - 8 -` | 32 | 33.867 | 33.867 | 33.867 | 0.000 |
| `- - - - - 6 7 - -` | 32 | 34.301 | 34.301 | 34.301 | 0.000 |
| `1 2 3 - - 6 - - -` | 33 | 41.272 | 40.974 | 41.272 | 0.298 |
| `1 2 - 4 5 - - - -` | 33 | 41.318 | 41.087 | 41.297 | 0.231 |
| `1 2 - - - - - - 9` | 33 | 36.136 | 36.136 | 36.136 | 0.000 |
| `1 - 3 - - - - 8 -` | 33 | 36.853 | 36.853 | 36.853 | 0.000 |
| `1 - - 4 - - 7 - -` | 33 | 37.625 | 37.625 | 37.625 | 0.000 |
| `1 - - - 5 6 - - -` | 33 | 37.008 | 36.931 | 37.008 | 0.077 |
| `- 2 3 - - - 7 - -` | 33 | 37.583 | 37.583 | 37.583 | 0.000 |
| `- 2 - 4 - 6 - - -` | 33 | 36.951 | 36.895 | 36.951 | 0.056 |
| `- - 3 4 5 - - - -` | 33 | 38.279 | 38.279 | 38.279 | 0.000 |
| `- - 3 - - - - - 9` | 33 | 34.574 | 34.574 | 34.574 | 0.000 |
| `- - - 4 - - - 8 -` | 33 | 34.917 | 34.917 | 34.917 | 0.000 |
| `- - - - 5 - 7 - -` | 33 | 35.278 | 35.278 | 35.278 | 0.000 |
| `1 2 3 - 5 - - - -` | 34 | 41.604 | 41.382 | 41.604 | 0.222 |
| `1 2 - - - - - 8 -` | 34 | 37.935 | 37.935 | 37.935 | 0.000 |
| `1 - 3 - - - 7 - -` | 34 | 38.552 | 38.552 | 38.552 | 0.000 |
| `1 - - 4 - 6 - - -` | 34 | 38.772 | 38.772 | 38.772 | 0.000 |
| `- 2 3 - - 6 - - -` | 34 | 38.735 | 38.735 | 38.735 | 0.000 |
| `- 2 - 4 5 - - - -` | 34 | 39.037 | 39.037 | 39.037 | 0.000 |
| `- 2 - - - - - - 9` | 34 | 35.701 | 35.701 | 35.701 | 0.000 |
| `- - 3 - - - - 8 -` | 34 | 35.974 | 35.974 | 35.974 | 0.000 |
| `- - - 4 - - 7 - -` | 34 | 36.264 | 36.264 | 36.264 | 0.000 |
| `- - - - 5 6 - - -` | 34 | 36.170 | 36.170 | 36.170 | 0.000 |
| `1 2 3 4 - - - - -` | 35 | 41.705 | 41.517 | 41.694 | 0.188 |
| `1 2 - - - - 7 - -` | 35 | 39.506 | 39.506 | 39.506 | 0.000 |
| `1 - 3 - - 6 - - -` | 35 | 39.611 | 39.611 | 39.611 | 0.000 |
| `1 - - 4 5 - - - -` | 35 | 38.816 | 38.779 | 38.816 | 0.037 |
| `1 - - - - - - - 9` | 35 | 36.833 | 36.833 | 36.833 | 0.000 |
| `- 2 3 - 5 - - - -` | 35 | 39.164 | 39.138 | 39.164 | 0.026 |
| `- 2 - - - - - 8 -` | 35 | 37.039 | 37.039 | 37.039 | 0.000 |
| `- - 3 - - - 7 - -` | 35 | 37.259 | 37.259 | 37.259 | 0.000 |
| `- - - 4 - 6 - - -` | 35 | 37.116 | 37.116 | 37.116 | 0.000 |
| `1 2 - - - 6 - - -` | 36 | 40.475 | 40.475 | 40.475 | 0.000 |
| `1 - 3 - 5 - - - -` | 36 | 40.202 | 40.202 | 40.202 | 0.000 |
| `1 - - - - - - 8 -` | 36 | 38.111 | 38.111 | 38.111 | 0.000 |
| `- 2 3 4 - - - - -` | 36 | 40.362 | 40.362 | 40.362 | 0.000 |
| `- 2 - - - - 7 - -` | 36 | 38.264 | 38.264 | 38.264 | 0.000 |
| `- - 3 - - 6 - - -` | 36 | 38.069 | 38.069 | 38.069 | 0.000 |
| `- - - 4 5 - - - -` | 36 | 37.972 | 37.972 | 37.972 | 0.000 |
| `- - - - - - - - 9` | 36 | 37.000 | 37.000 | 37.000 | 0.000 |
| `1 2 - - 5 - - - -` | 37 | 40.991 | 40.991 | 40.991 | 0.000 |
| `1 - 3 4 - - - - -` | 37 | 40.443 | 40.429 | 40.443 | 0.014 |
| `1 - - - - - 7 - -` | 37 | 39.278 | 39.278 | 39.278 | 0.000 |
| `- 2 - - - 6 - - -` | 37 | 39.031 | 39.031 | 39.031 | 0.000 |
| `- - 3 - 5 - - - -` | 37 | 38.883 | 38.883 | 38.883 | 0.000 |
| `- - - - - - - 8 -` | 37 | 38.111 | 38.111 | 38.111 | 0.000 |
| `1 2 - 4 - - - - -` | 38 | 41.179 | 41.179 | 41.179 | 0.000 |
| `1 - - - - 6 - - -` | 38 | 40.000 | 40.000 | 40.000 | 0.000 |
| `- 2 - - 5 - - - -` | 38 | 39.799 | 39.799 | 39.799 | 0.000 |
| `- - 3 4 - - - - -` | 38 | 39.699 | 39.699 | 39.699 | 0.000 |
| `- - - - - - 7 - -` | 38 | 39.167 | 39.167 | 39.167 | 0.000 |
| `1 2 3 - - - - - -` | 39 | 40.975 | 40.972 | 40.975 | 0.003 |
| `1 - - - 5 - - - -` | 39 | 40.389 | 40.389 | 40.389 | 0.000 |
| `- 2 - 4 - - - - -` | 39 | 40.236 | 40.236 | 40.236 | 0.000 |
| `- - - - - 6 - - -` | 39 | 39.833 | 39.833 | 39.833 | 0.000 |
| `1 - - 4 - - - - -` | 40 | 40.889 | 40.889 | 40.889 | 0.000 |
| `- 2 3 - - - - - -` | 40 | 40.785 | 40.785 | 40.785 | 0.000 |
| `- - - - 5 - - - -` | 40 | 40.556 | 40.556 | 40.556 | 0.000 |
| `1 - 3 - - - - - -` | 41 | 41.500 | 41.500 | 41.500 | 0.000 |
| `- - - 4 - - - - -` | 41 | 41.333 | 41.333 | 41.333 | 0.000 |
| `1 2 - - - - - - -` | 42 | 42.222 | 42.222 | 42.222 | 0.000 |
| `- - 3 - - - - - -` | 42 | 42.167 | 42.167 | 42.167 | 0.000 |
| `- 2 - - - - - - -` | 43 | 43.056 | 43.056 | 43.056 | 0.000 |
| `1 - - - - - - - -` | 44 | 44.000 | 44.000 | 44.000 | 0.000 |
| `- - - - - - - - -` | 45 | 45.000 | 45.000 | 45.000 | 0.000 |


### Exhibit B: All Possible Board States with Expected Score of Each Strategy for All or Nothing Scoring System

In [11]:
def exhibit_b(results: dict) -> Table:
    """returns a table for expected scores of the all or nothing scoring system"""
    exhibit_table = []
    results_subtable = results['all_or_nothing_board_table']
    for record in results_subtable:
        strategic_difference = record['Max ES'] - record['Min ES']
        exhibit_table.append({
            'Board (Remaining)': f'`{record["Board"]}`',
            'Board Sum': record['Sum'],
            'Max ES': f'{record["Max ES"]:.3f}',
            'Min ES': f'{record["Min ES"]:.3f}',
            'Highest Tile ES': f'{record["Highest Tile ES"]:.3f}',
            'Strategic Difference': f'{strategic_difference:.3f}',
        })
    # sort by board sum ascending
    exhibit_table.sort(key=lambda record: record['Board Sum'])
    return exhibit_table

display_table(exhibit_b(full_results))

| Board (Remaining) | Board Sum | Max ES | Min ES | Highest Tile ES | Strategic Difference |
| --- | --- | --- | --- | --- | --- |
| `1 2 3 4 5 6 7 8 9` | 0 | 0.071 | 0.010 | 0.070 | 0.062 |
| `- 2 3 4 5 6 7 8 9` | 1 | 0.042 | 0.010 | 0.041 | 0.032 |
| `1 - 3 4 5 6 7 8 9` | 2 | 0.045 | 0.010 | 0.044 | 0.035 |
| `1 2 - 4 5 6 7 8 9` | 3 | 0.051 | 0.010 | 0.050 | 0.041 |
| `- - 3 4 5 6 7 8 9` | 3 | 0.023 | 0.010 | 0.022 | 0.013 |
| `1 2 3 - 5 6 7 8 9` | 4 | 0.055 | 0.011 | 0.054 | 0.045 |
| `- 2 - 4 5 6 7 8 9` | 4 | 0.026 | 0.010 | 0.026 | 0.016 |
| `1 2 3 4 - 6 7 8 9` | 5 | 0.061 | 0.012 | 0.060 | 0.049 |
| `1 - - 4 5 6 7 8 9` | 5 | 0.029 | 0.009 | 0.029 | 0.020 |
| `- 2 3 - 5 6 7 8 9` | 5 | 0.030 | 0.010 | 0.029 | 0.020 |
| `1 2 3 4 5 - 7 8 9` | 6 | 0.065 | 0.012 | 0.064 | 0.053 |
| `1 - 3 - 5 6 7 8 9` | 6 | 0.035 | 0.011 | 0.035 | 0.024 |
| `- 2 3 4 - 6 7 8 9` | 6 | 0.035 | 0.013 | 0.034 | 0.022 |
| `- - - 4 5 6 7 8 9` | 6 | 0.013 | 0.009 | 0.013 | 0.004 |
| `1 2 3 4 5 6 - 8 9` | 7 | 0.071 | 0.012 | 0.069 | 0.059 |
| `1 2 - - 5 6 7 8 9` | 7 | 0.038 | 0.010 | 0.037 | 0.027 |
| `1 - 3 4 - 6 7 8 9` | 7 | 0.039 | 0.012 | 0.038 | 0.026 |
| `- 2 3 4 5 - 7 8 9` | 7 | 0.037 | 0.013 | 0.037 | 0.025 |
| `- - 3 - 5 6 7 8 9` | 7 | 0.017 | 0.010 | 0.016 | 0.007 |
| `1 2 3 4 5 6 7 - 9` | 8 | 0.084 | 0.017 | 0.082 | 0.068 |
| `1 2 - 4 - 6 7 8 9` | 8 | 0.045 | 0.013 | 0.044 | 0.032 |
| `1 - 3 4 5 - 7 8 9` | 8 | 0.040 | 0.012 | 0.040 | 0.028 |
| `- 2 3 4 5 6 - 8 9` | 8 | 0.043 | 0.012 | 0.042 | 0.031 |
| `- 2 - - 5 6 7 8 9` | 8 | 0.020 | 0.010 | 0.019 | 0.009 |
| `- - 3 4 - 6 7 8 9` | 8 | 0.019 | 0.011 | 0.019 | 0.008 |
| `1 2 3 4 5 6 7 8 -` | 9 | 0.102 | 0.025 | 0.100 | 0.076 |
| `1 2 3 - - 6 7 8 9` | 9 | 0.048 | 0.015 | 0.048 | 0.033 |
| `1 2 - 4 5 - 7 8 9` | 9 | 0.049 | 0.012 | 0.048 | 0.036 |
| `1 - 3 4 5 6 - 8 9` | 9 | 0.045 | 0.012 | 0.043 | 0.033 |
| `1 - - - 5 6 7 8 9` | 9 | 0.022 | 0.010 | 0.022 | 0.012 |
| `- 2 3 4 5 6 7 - 9` | 9 | 0.052 | 0.017 | 0.051 | 0.035 |
| `- 2 - 4 - 6 7 8 9` | 9 | 0.022 | 0.012 | 0.022 | 0.011 |
| `- - 3 4 5 - 7 8 9` | 9 | 0.018 | 0.011 | 0.018 | 0.007 |
| `1 2 3 - 5 - 7 8 9` | 10 | 0.052 | 0.016 | 0.051 | 0.036 |
| `1 2 - 4 5 6 - 8 9` | 10 | 0.052 | 0.014 | 0.051 | 0.039 |
| `1 - 3 4 5 6 7 - 9` | 10 | 0.057 | 0.016 | 0.056 | 0.041 |
| `1 - - 4 - 6 7 8 9` | 10 | 0.028 | 0.013 | 0.028 | 0.014 |
| `- 2 3 4 5 6 7 8 -` | 10 | 0.069 | 0.026 | 0.068 | 0.043 |
| `- 2 3 - - 6 7 8 9` | 10 | 0.027 | 0.015 | 0.027 | 0.012 |
| `- 2 - 4 5 - 7 8 9` | 10 | 0.024 | 0.012 | 0.024 | 0.012 |
| `- - 3 4 5 6 - 8 9` | 10 | 0.023 | 0.012 | 0.023 | 0.011 |
| `- - - - 5 6 7 8 9` | 10 | 0.010 | 0.010 | 0.010 | 0.000 |
| `1 2 3 4 - - 7 8 9` | 11 | 0.057 | 0.018 | 0.056 | 0.039 |
| `1 2 3 - 5 6 - 8 9` | 11 | 0.057 | 0.017 | 0.055 | 0.041 |
| `1 2 - 4 5 6 7 - 9` | 11 | 0.063 | 0.017 | 0.062 | 0.045 |
| `1 - 3 4 5 6 7 8 -` | 11 | 0.073 | 0.025 | 0.072 | 0.048 |
| `1 - 3 - - 6 7 8 9` | 11 | 0.030 | 0.014 | 0.030 | 0.016 |
| `1 - - 4 5 - 7 8 9` | 11 | 0.027 | 0.012 | 0.027 | 0.016 |
| `- 2 3 - 5 - 7 8 9` | 11 | 0.026 | 0.013 | 0.026 | 0.013 |
| `- 2 - 4 5 6 - 8 9` | 11 | 0.028 | 0.013 | 0.027 | 0.016 |
| `- - 3 4 5 6 7 - 9` | 11 | 0.032 | 0.016 | 0.030 | 0.015 |
| `- - - 4 - 6 7 8 9` | 11 | 0.013 | 0.013 | 0.013 | 0.000 |
| `1 2 3 4 - 6 - 8 9` | 12 | 0.064 | 0.017 | 0.063 | 0.047 |
| `1 2 3 - 5 6 7 - 9` | 12 | 0.071 | 0.020 | 0.069 | 0.050 |
| `1 2 - 4 5 6 7 8 -` | 12 | 0.080 | 0.026 | 0.079 | 0.054 |
| `1 2 - - - 6 7 8 9` | 12 | 0.031 | 0.014 | 0.031 | 0.018 |
| `1 - 3 - 5 - 7 8 9` | 12 | 0.035 | 0.017 | 0.034 | 0.018 |
| `1 - - 4 5 6 - 8 9` | 12 | 0.028 | 0.013 | 0.028 | 0.015 |
| `- 2 3 4 - - 7 8 9` | 12 | 0.035 | 0.020 | 0.034 | 0.015 |
| `- 2 3 - 5 6 - 8 9` | 12 | 0.031 | 0.016 | 0.030 | 0.015 |
| `- 2 - 4 5 6 7 - 9` | 12 | 0.033 | 0.016 | 0.032 | 0.017 |
| `- - 3 4 5 6 7 8 -` | 12 | 0.044 | 0.026 | 0.043 | 0.018 |
| `- - 3 - - 6 7 8 9` | 12 | 0.016 | 0.012 | 0.016 | 0.003 |
| `- - - 4 5 - 7 8 9` | 12 | 0.011 | 0.008 | 0.011 | 0.003 |
| `1 2 3 4 5 - - 8 9` | 13 | 0.069 | 0.016 | 0.066 | 0.053 |
| `1 2 3 4 - 6 7 - 9` | 13 | 0.076 | 0.022 | 0.075 | 0.054 |
| `1 2 3 - 5 6 7 8 -` | 13 | 0.085 | 0.027 | 0.084 | 0.058 |
| `1 2 - - 5 - 7 8 9` | 13 | 0.039 | 0.016 | 0.039 | 0.023 |
| `1 - 3 4 - - 7 8 9` | 13 | 0.033 | 0.015 | 0.033 | 0.018 |
| `1 - 3 - 5 6 - 8 9` | 13 | 0.035 | 0.017 | 0.035 | 0.018 |
| `1 - - 4 5 6 7 - 9` | 13 | 0.037 | 0.017 | 0.037 | 0.020 |
| `- 2 3 4 - 6 - 8 9` | 13 | 0.037 | 0.018 | 0.037 | 0.020 |
| `- 2 3 - 5 6 7 - 9` | 13 | 0.042 | 0.018 | 0.041 | 0.024 |
| `- 2 - 4 5 6 7 8 -` | 13 | 0.048 | 0.026 | 0.047 | 0.022 |
| `- 2 - - - 6 7 8 9` | 13 | 0.018 | 0.012 | 0.018 | 0.006 |
| `- - 3 - 5 - 7 8 9` | 13 | 0.016 | 0.013 | 0.015 | 0.003 |
| `- - - 4 5 6 - 8 9` | 13 | 0.013 | 0.009 | 0.013 | 0.003 |
| `1 2 3 4 5 - 7 - 9` | 14 | 0.082 | 0.021 | 0.078 | 0.061 |
| `1 2 3 4 - 6 7 8 -` | 14 | 0.094 | 0.031 | 0.093 | 0.063 |
| `1 2 - 4 - - 7 8 9` | 14 | 0.046 | 0.020 | 0.046 | 0.026 |
| `1 2 - - 5 6 - 8 9` | 14 | 0.044 | 0.019 | 0.042 | 0.024 |
| `1 - 3 4 - 6 - 8 9` | 14 | 0.044 | 0.019 | 0.042 | 0.025 |
| `1 - 3 - 5 6 7 - 9` | 14 | 0.049 | 0.022 | 0.047 | 0.027 |
| `1 - - 4 5 6 7 8 -` | 14 | 0.052 | 0.024 | 0.051 | 0.027 |
| `1 - - - - 6 7 8 9` | 14 | 0.019 | 0.010 | 0.019 | 0.008 |
| `- 2 3 4 5 - - 8 9` | 14 | 0.041 | 0.019 | 0.040 | 0.022 |
| `- 2 3 4 - 6 7 - 9` | 14 | 0.044 | 0.022 | 0.044 | 0.022 |
| `- 2 3 - 5 6 7 8 -` | 14 | 0.051 | 0.025 | 0.050 | 0.026 |
| `- 2 - - 5 - 7 8 9` | 14 | 0.019 | 0.013 | 0.019 | 0.006 |
| `- - 3 4 - - 7 8 9` | 14 | 0.017 | 0.015 | 0.017 | 0.002 |
| `- - 3 - 5 6 - 8 9` | 14 | 0.014 | 0.009 | 0.014 | 0.005 |
| `- - - 4 5 6 7 - 9` | 14 | 0.017 | 0.013 | 0.017 | 0.005 |
| `1 2 3 4 5 6 - - 9` | 15 | 0.089 | 0.022 | 0.086 | 0.067 |
| `1 2 3 4 5 - 7 8 -` | 15 | 0.100 | 0.033 | 0.099 | 0.067 |
| `1 2 3 - - - 7 8 9` | 15 | 0.044 | 0.024 | 0.044 | 0.020 |
| `1 2 - 4 - 6 - 8 9` | 15 | 0.048 | 0.021 | 0.047 | 0.027 |
| `1 2 - - 5 6 7 - 9` | 15 | 0.052 | 0.021 | 0.051 | 0.030 |
| `1 - 3 4 5 - - 8 9` | 15 | 0.041 | 0.017 | 0.041 | 0.024 |
| `1 - 3 4 - 6 7 - 9` | 15 | 0.048 | 0.025 | 0.048 | 0.023 |
| `1 - 3 - 5 6 7 8 -` | 15 | 0.060 | 0.027 | 0.060 | 0.033 |
| `1 - - - 5 - 7 8 9` | 15 | 0.022 | 0.014 | 0.022 | 0.008 |
| `- 2 3 4 5 - 7 - 9` | 15 | 0.049 | 0.021 | 0.048 | 0.027 |
| `- 2 3 4 - 6 7 8 -` | 15 | 0.062 | 0.032 | 0.062 | 0.031 |
| `- 2 - 4 - - 7 8 9` | 15 | 0.023 | 0.018 | 0.023 | 0.006 |
| `- 2 - - 5 6 - 8 9` | 15 | 0.023 | 0.018 | 0.022 | 0.005 |
| `- - 3 4 - 6 - 8 9` | 15 | 0.022 | 0.016 | 0.022 | 0.006 |
| `- - 3 - 5 6 7 - 9` | 15 | 0.026 | 0.019 | 0.026 | 0.007 |
| `- - - 4 5 6 7 8 -` | 15 | 0.029 | 0.028 | 0.028 | 0.001 |
| `- - - - - 6 7 8 9` | 15 | 0.009 | 0.009 | 0.009 | 0.000 |
| `1 2 3 4 5 6 - 8 -` | 16 | 0.109 | 0.034 | 0.108 | 0.075 |
| `1 2 3 - - 6 - 8 9` | 16 | 0.050 | 0.024 | 0.050 | 0.027 |
| `1 2 - 4 5 - - 8 9` | 16 | 0.051 | 0.020 | 0.051 | 0.031 |
| `1 2 - 4 - 6 7 - 9` | 16 | 0.060 | 0.027 | 0.059 | 0.033 |
| `1 2 - - 5 6 7 8 -` | 16 | 0.061 | 0.026 | 0.060 | 0.034 |
| `1 - 3 4 5 - 7 - 9` | 16 | 0.055 | 0.024 | 0.053 | 0.031 |
| `1 - 3 4 - 6 7 8 -` | 16 | 0.066 | 0.033 | 0.066 | 0.034 |
| `1 - - 4 - - 7 8 9` | 16 | 0.026 | 0.017 | 0.026 | 0.009 |
| `1 - - - 5 6 - 8 9` | 16 | 0.025 | 0.019 | 0.025 | 0.006 |
| `- 2 3 4 5 6 - - 9` | 16 | 0.059 | 0.024 | 0.057 | 0.036 |
| `- 2 3 4 5 - 7 8 -` | 16 | 0.067 | 0.035 | 0.066 | 0.031 |
| `- 2 3 - - - 7 8 9` | 16 | 0.027 | 0.020 | 0.026 | 0.006 |
| `- 2 - 4 - 6 - 8 9` | 16 | 0.021 | 0.014 | 0.021 | 0.007 |
| `- 2 - - 5 6 7 - 9` | 16 | 0.026 | 0.017 | 0.026 | 0.010 |
| `- - 3 4 5 - - 8 9` | 16 | 0.019 | 0.012 | 0.019 | 0.006 |
| `- - 3 4 - 6 7 - 9` | 16 | 0.023 | 0.016 | 0.022 | 0.007 |
| `- - 3 - 5 6 7 8 -` | 16 | 0.033 | 0.025 | 0.032 | 0.008 |
| `- - - - 5 - 7 8 9` | 16 | 0.009 | 0.009 | 0.009 | 0.000 |
| `1 2 3 4 5 6 7 - -` | 17 | 0.123 | 0.052 | 0.122 | 0.071 |
| `1 2 3 - 5 - - 8 9` | 17 | 0.053 | 0.025 | 0.052 | 0.028 |
| `1 2 3 - - 6 7 - 9` | 17 | 0.065 | 0.029 | 0.064 | 0.036 |
| `1 2 - 4 5 - 7 - 9` | 17 | 0.063 | 0.025 | 0.061 | 0.039 |
| `1 2 - 4 - 6 7 8 -` | 17 | 0.075 | 0.032 | 0.074 | 0.043 |
| `1 - 3 4 5 6 - - 9` | 17 | 0.063 | 0.023 | 0.061 | 0.040 |
| `1 - 3 4 5 - 7 8 -` | 17 | 0.071 | 0.033 | 0.071 | 0.037 |
| `1 - 3 - - - 7 8 9` | 17 | 0.030 | 0.020 | 0.030 | 0.010 |
| `1 - - 4 - 6 - 8 9` | 17 | 0.031 | 0.024 | 0.031 | 0.007 |
| `1 - - - 5 6 7 - 9` | 17 | 0.034 | 0.025 | 0.034 | 0.009 |
| `- 2 3 4 5 6 - 8 -` | 17 | 0.080 | 0.034 | 0.080 | 0.046 |
| `- 2 3 - - 6 - 8 9` | 17 | 0.025 | 0.019 | 0.025 | 0.006 |
| `- 2 - 4 5 - - 8 9` | 17 | 0.028 | 0.022 | 0.028 | 0.006 |
| `- 2 - 4 - 6 7 - 9` | 17 | 0.031 | 0.023 | 0.031 | 0.009 |
| `- 2 - - 5 6 7 8 -` | 17 | 0.037 | 0.027 | 0.037 | 0.011 |
| `- - 3 4 5 - 7 - 9` | 17 | 0.029 | 0.020 | 0.029 | 0.009 |
| `- - 3 4 - 6 7 8 -` | 17 | 0.039 | 0.031 | 0.038 | 0.008 |
| `- - - 4 - - 7 8 9` | 17 | 0.013 | 0.013 | 0.013 | 0.000 |
| `- - - - 5 6 - 8 9` | 17 | 0.011 | 0.011 | 0.011 | 0.000 |
| `1 2 3 4 - - - 8 9` | 18 | 0.060 | 0.028 | 0.060 | 0.032 |
| `1 2 3 - 5 - 7 - 9` | 18 | 0.069 | 0.034 | 0.069 | 0.036 |
| `1 2 3 - - 6 7 8 -` | 18 | 0.080 | 0.041 | 0.080 | 0.038 |
| `1 2 - 4 5 6 - - 9` | 18 | 0.067 | 0.031 | 0.066 | 0.037 |
| `1 2 - 4 5 - 7 8 -` | 18 | 0.083 | 0.040 | 0.083 | 0.044 |
| `1 2 - - - - 7 8 9` | 18 | 0.032 | 0.021 | 0.032 | 0.010 |
| `1 - 3 4 5 6 - 8 -` | 18 | 0.081 | 0.039 | 0.080 | 0.042 |
| `1 - 3 - - 6 - 8 9` | 18 | 0.037 | 0.025 | 0.037 | 0.011 |
| `1 - - 4 5 - - 8 9` | 18 | 0.024 | 0.014 | 0.024 | 0.009 |
| `1 - - 4 - 6 7 - 9` | 18 | 0.040 | 0.033 | 0.039 | 0.006 |
| `1 - - - 5 6 7 8 -` | 18 | 0.037 | 0.022 | 0.037 | 0.014 |
| `- 2 3 4 5 6 7 - -` | 18 | 0.097 | 0.056 | 0.097 | 0.041 |
| `- 2 3 - 5 - - 8 9` | 18 | 0.032 | 0.025 | 0.031 | 0.007 |
| `- 2 3 - - 6 7 - 9` | 18 | 0.041 | 0.028 | 0.041 | 0.013 |
| `- 2 - 4 5 - 7 - 9` | 18 | 0.027 | 0.015 | 0.027 | 0.011 |
| `- 2 - 4 - 6 7 8 -` | 18 | 0.042 | 0.029 | 0.042 | 0.014 |
| `- - 3 4 5 6 - - 9` | 18 | 0.037 | 0.024 | 0.037 | 0.013 |
| `- - 3 4 5 - 7 8 -` | 18 | 0.043 | 0.029 | 0.043 | 0.014 |
| `- - 3 - - - 7 8 9` | 18 | 0.021 | 0.021 | 0.021 | 0.000 |
| `- - - 4 - 6 - 8 9` | 18 | 0.015 | 0.015 | 0.015 | 0.000 |
| `- - - - 5 6 7 - 9` | 18 | 0.016 | 0.016 | 0.016 | 0.000 |
| `1 2 3 4 - - 7 - 9` | 19 | 0.074 | 0.034 | 0.073 | 0.040 |
| `1 2 3 - 5 6 - - 9` | 19 | 0.080 | 0.034 | 0.078 | 0.046 |
| `1 2 3 - 5 - 7 8 -` | 19 | 0.087 | 0.044 | 0.087 | 0.043 |
| `1 2 - 4 5 6 - 8 -` | 19 | 0.094 | 0.040 | 0.093 | 0.054 |
| `1 2 - - - 6 - 8 9` | 19 | 0.043 | 0.027 | 0.041 | 0.016 |
| `1 - 3 4 5 6 7 - -` | 19 | 0.102 | 0.051 | 0.100 | 0.051 |
| `1 - 3 - 5 - - 8 9` | 19 | 0.032 | 0.023 | 0.032 | 0.009 |
| `1 - 3 - - 6 7 - 9` | 19 | 0.041 | 0.031 | 0.041 | 0.009 |
| `1 - - 4 5 - 7 - 9` | 19 | 0.040 | 0.028 | 0.040 | 0.012 |
| `1 - - 4 - 6 7 8 -` | 19 | 0.053 | 0.037 | 0.053 | 0.016 |
| `- 2 3 4 - - - 8 9` | 19 | 0.043 | 0.033 | 0.043 | 0.010 |
| `- 2 3 - 5 - 7 - 9` | 19 | 0.040 | 0.027 | 0.040 | 0.014 |
| `- 2 3 - - 6 7 8 -` | 19 | 0.054 | 0.041 | 0.053 | 0.013 |
| `- 2 - 4 5 6 - - 9` | 19 | 0.041 | 0.029 | 0.041 | 0.012 |
| `- 2 - 4 5 - 7 8 -` | 19 | 0.053 | 0.042 | 0.051 | 0.011 |
| `- 2 - - - - 7 8 9` | 19 | 0.024 | 0.020 | 0.024 | 0.004 |
| `- - 3 4 5 6 - 8 -` | 19 | 0.051 | 0.039 | 0.050 | 0.012 |
| `- - 3 - - 6 - 8 9` | 19 | 0.016 | 0.014 | 0.016 | 0.003 |
| `- - - 4 5 - - 8 9` | 19 | 0.011 | 0.009 | 0.011 | 0.002 |
| `- - - 4 - 6 7 - 9` | 19 | 0.020 | 0.020 | 0.020 | 0.000 |
| `- - - - 5 6 7 8 -` | 19 | 0.020 | 0.020 | 0.020 | 0.000 |
| `1 2 3 4 - 6 - - 9` | 20 | 0.086 | 0.033 | 0.083 | 0.053 |
| `1 2 3 4 - - 7 8 -` | 20 | 0.095 | 0.048 | 0.093 | 0.047 |
| `1 2 3 - 5 6 - 8 -` | 20 | 0.098 | 0.044 | 0.094 | 0.054 |
| `1 2 - 4 5 6 7 - -` | 20 | 0.106 | 0.054 | 0.105 | 0.052 |
| `1 2 - - 5 - - 8 9` | 20 | 0.050 | 0.032 | 0.050 | 0.018 |
| `1 2 - - - 6 7 - 9` | 20 | 0.051 | 0.033 | 0.050 | 0.019 |
| `1 - 3 4 - - - 8 9` | 20 | 0.041 | 0.028 | 0.041 | 0.013 |
| `1 - 3 - 5 - 7 - 9` | 20 | 0.054 | 0.038 | 0.052 | 0.016 |
| `1 - 3 - - 6 7 8 -` | 20 | 0.059 | 0.041 | 0.059 | 0.018 |
| `1 - - 4 5 6 - - 9` | 20 | 0.035 | 0.023 | 0.035 | 0.012 |
| `1 - - 4 5 - 7 8 -` | 20 | 0.056 | 0.041 | 0.056 | 0.015 |
| `1 - - - - - 7 8 9` | 20 | 0.024 | 0.018 | 0.024 | 0.006 |
| `- 2 3 4 - - 7 - 9` | 20 | 0.041 | 0.027 | 0.041 | 0.014 |
| `- 2 3 - 5 6 - - 9` | 20 | 0.048 | 0.035 | 0.047 | 0.014 |
| `- 2 3 - 5 - 7 8 -` | 20 | 0.046 | 0.034 | 0.046 | 0.012 |
| `- 2 - 4 5 6 - 8 -` | 20 | 0.058 | 0.038 | 0.058 | 0.020 |
| `- 2 - - - 6 - 8 9` | 20 | 0.022 | 0.019 | 0.022 | 0.003 |
| `- - 3 4 5 6 7 - -` | 20 | 0.070 | 0.055 | 0.069 | 0.015 |
| `- - 3 - 5 - - 8 9` | 20 | 0.015 | 0.014 | 0.015 | 0.002 |
| `- - 3 - - 6 7 - 9` | 20 | 0.021 | 0.017 | 0.021 | 0.004 |
| `- - - 4 5 - 7 - 9` | 20 | 0.016 | 0.013 | 0.016 | 0.003 |
| `- - - 4 - 6 7 8 -` | 20 | 0.028 | 0.028 | 0.028 | 0.000 |
| `1 2 3 4 5 - - - 9` | 21 | 0.091 | 0.032 | 0.088 | 0.059 |
| `1 2 3 4 - 6 - 8 -` | 21 | 0.110 | 0.047 | 0.108 | 0.062 |
| `1 2 3 - 5 6 7 - -` | 21 | 0.113 | 0.061 | 0.113 | 0.052 |
| `1 2 - 4 - - - 8 9` | 21 | 0.052 | 0.040 | 0.052 | 0.012 |
| `1 2 - - 5 - 7 - 9` | 21 | 0.058 | 0.040 | 0.057 | 0.018 |
| `1 2 - - - 6 7 8 -` | 21 | 0.056 | 0.038 | 0.056 | 0.018 |
| `1 - 3 4 - - 7 - 9` | 21 | 0.047 | 0.038 | 0.047 | 0.009 |
| `1 - 3 - 5 6 - - 9` | 21 | 0.056 | 0.035 | 0.056 | 0.021 |
| `1 - 3 - 5 - 7 8 -` | 21 | 0.066 | 0.047 | 0.066 | 0.019 |
| `1 - - 4 5 6 - 8 -` | 21 | 0.062 | 0.047 | 0.060 | 0.015 |
| `1 - - - - 6 - 8 9` | 21 | 0.030 | 0.025 | 0.030 | 0.005 |
| `- 2 3 4 - 6 - - 9` | 21 | 0.055 | 0.034 | 0.055 | 0.020 |
| `- 2 3 4 - - 7 8 -` | 21 | 0.068 | 0.056 | 0.068 | 0.012 |
| `- 2 3 - 5 6 - 8 -` | 21 | 0.061 | 0.040 | 0.061 | 0.022 |
| `- 2 - 4 5 6 7 - -` | 21 | 0.073 | 0.054 | 0.071 | 0.019 |
| `- 2 - - 5 - - 8 9` | 21 | 0.028 | 0.028 | 0.028 | 0.000 |
| `- 2 - - - 6 7 - 9` | 21 | 0.030 | 0.024 | 0.030 | 0.006 |
| `- - 3 4 - - - 8 9` | 21 | 0.024 | 0.023 | 0.024 | 0.000 |
| `- - 3 - 5 - 7 - 9` | 21 | 0.029 | 0.028 | 0.029 | 0.001 |
| `- - 3 - - 6 7 8 -` | 21 | 0.037 | 0.037 | 0.037 | 0.000 |
| `- - - 4 5 6 - - 9` | 21 | 0.018 | 0.014 | 0.018 | 0.004 |
| `- - - 4 5 - 7 8 -` | 21 | 0.031 | 0.030 | 0.031 | 0.000 |
| `- - - - - - 7 8 9` | 21 | 0.015 | 0.015 | 0.015 | 0.000 |
| `1 2 3 4 5 - - 8 -` | 22 | 0.114 | 0.050 | 0.111 | 0.064 |
| `1 2 3 4 - 6 7 - -` | 22 | 0.122 | 0.065 | 0.122 | 0.057 |
| `1 2 3 - - - - 8 9` | 22 | 0.042 | 0.032 | 0.042 | 0.010 |
| `1 2 - 4 - - 7 - 9` | 22 | 0.065 | 0.040 | 0.065 | 0.025 |
| `1 2 - - 5 6 - - 9` | 22 | 0.066 | 0.044 | 0.064 | 0.022 |
| `1 2 - - 5 - 7 8 -` | 22 | 0.070 | 0.048 | 0.070 | 0.022 |
| `1 - 3 4 - 6 - - 9` | 22 | 0.059 | 0.040 | 0.057 | 0.019 |
| `1 - 3 4 - - 7 8 -` | 22 | 0.058 | 0.041 | 0.058 | 0.016 |
| `1 - 3 - 5 6 - 8 -` | 22 | 0.066 | 0.051 | 0.066 | 0.015 |
| `1 - - 4 5 6 7 - -` | 22 | 0.076 | 0.053 | 0.075 | 0.022 |
| `1 - - - 5 - - 8 9` | 22 | 0.025 | 0.020 | 0.025 | 0.004 |
| `1 - - - - 6 7 - 9` | 22 | 0.034 | 0.030 | 0.034 | 0.004 |
| `- 2 3 4 5 - - - 9` | 22 | 0.059 | 0.036 | 0.058 | 0.023 |
| `- 2 3 4 - 6 - 8 -` | 22 | 0.073 | 0.048 | 0.073 | 0.024 |
| `- 2 3 - 5 6 7 - -` | 22 | 0.081 | 0.058 | 0.080 | 0.023 |
| `- 2 - 4 - - - 8 9` | 22 | 0.026 | 0.026 | 0.026 | 0.000 |
| `- 2 - - 5 - 7 - 9` | 22 | 0.025 | 0.018 | 0.025 | 0.006 |
| `- 2 - - - 6 7 8 -` | 22 | 0.036 | 0.031 | 0.036 | 0.005 |
| `- - 3 4 - - 7 - 9` | 22 | 0.023 | 0.022 | 0.023 | 0.001 |
| `- - 3 - 5 6 - - 9` | 22 | 0.027 | 0.021 | 0.027 | 0.006 |
| `- - 3 - 5 - 7 8 -` | 22 | 0.031 | 0.027 | 0.031 | 0.004 |
| `- - - 4 5 6 - 8 -` | 22 | 0.034 | 0.034 | 0.034 | 0.000 |
| `- - - - - 6 - 8 9` | 22 | 0.013 | 0.013 | 0.013 | 0.000 |
| `1 2 3 4 5 - 7 - -` | 23 | 0.128 | 0.069 | 0.126 | 0.059 |
| `1 2 3 - - - 7 - 9` | 23 | 0.062 | 0.045 | 0.062 | 0.017 |
| `1 2 - 4 - 6 - - 9` | 23 | 0.067 | 0.044 | 0.067 | 0.022 |
| `1 2 - 4 - - 7 8 -` | 23 | 0.083 | 0.055 | 0.082 | 0.028 |
| `1 2 - - 5 6 - 8 -` | 23 | 0.078 | 0.052 | 0.076 | 0.026 |
| `1 - 3 4 5 - - - 9` | 23 | 0.059 | 0.033 | 0.059 | 0.026 |
| `1 - 3 4 - 6 - 8 -` | 23 | 0.083 | 0.057 | 0.080 | 0.026 |
| `1 - 3 - 5 6 7 - -` | 23 | 0.090 | 0.066 | 0.090 | 0.024 |
| `1 - - 4 - - - 8 9` | 23 | 0.024 | 0.020 | 0.024 | 0.003 |
| `1 - - - 5 - 7 - 9` | 23 | 0.040 | 0.040 | 0.040 | 0.000 |
| `1 - - - - 6 7 8 -` | 23 | 0.035 | 0.026 | 0.035 | 0.009 |
| `- 2 3 4 5 - - 8 -` | 23 | 0.080 | 0.052 | 0.080 | 0.028 |
| `- 2 3 4 - 6 7 - -` | 23 | 0.087 | 0.068 | 0.087 | 0.019 |
| `- 2 3 - - - - 8 9` | 23 | 0.026 | 0.025 | 0.026 | 0.001 |
| `- 2 - 4 - - 7 - 9` | 23 | 0.028 | 0.021 | 0.028 | 0.007 |
| `- 2 - - 5 6 - - 9` | 23 | 0.036 | 0.034 | 0.036 | 0.001 |
| `- 2 - - 5 - 7 8 -` | 23 | 0.038 | 0.034 | 0.038 | 0.004 |
| `- - 3 4 - 6 - - 9` | 23 | 0.030 | 0.023 | 0.030 | 0.007 |
| `- - 3 4 - - 7 8 -` | 23 | 0.036 | 0.032 | 0.036 | 0.004 |
| `- - 3 - 5 6 - 8 -` | 23 | 0.031 | 0.025 | 0.031 | 0.006 |
| `- - - 4 5 6 7 - -` | 23 | 0.048 | 0.048 | 0.048 | 0.000 |
| `- - - - 5 - - 8 9` | 23 | 0.010 | 0.010 | 0.010 | 0.000 |
| `- - - - - 6 7 - 9` | 23 | 0.015 | 0.015 | 0.015 | 0.000 |
| `1 2 3 4 5 6 - - -` | 24 | 0.134 | 0.085 | 0.133 | 0.049 |
| `1 2 3 - - 6 - - 9` | 24 | 0.069 | 0.044 | 0.069 | 0.024 |
| `1 2 3 - - - 7 8 -` | 24 | 0.081 | 0.064 | 0.081 | 0.016 |
| `1 2 - 4 5 - - - 9` | 24 | 0.069 | 0.045 | 0.068 | 0.024 |
| `1 2 - 4 - 6 - 8 -` | 24 | 0.086 | 0.060 | 0.086 | 0.026 |
| `1 2 - - 5 6 7 - -` | 24 | 0.086 | 0.064 | 0.086 | 0.021 |
| `1 - 3 4 5 - - 8 -` | 24 | 0.082 | 0.056 | 0.082 | 0.026 |
| `1 - 3 4 - 6 7 - -` | 24 | 0.094 | 0.069 | 0.094 | 0.025 |
| `1 - 3 - - - - 8 9` | 24 | 0.029 | 0.026 | 0.029 | 0.003 |
| `1 - - 4 - - 7 - 9` | 24 | 0.042 | 0.042 | 0.042 | 0.000 |
| `1 - - - 5 6 - - 9` | 24 | 0.042 | 0.039 | 0.042 | 0.003 |
| `1 - - - 5 - 7 8 -` | 24 | 0.044 | 0.038 | 0.044 | 0.006 |
| `- 2 3 4 5 - 7 - -` | 24 | 0.099 | 0.071 | 0.099 | 0.028 |
| `- 2 3 - - - 7 - 9` | 24 | 0.036 | 0.031 | 0.036 | 0.005 |
| `- 2 - 4 - 6 - - 9` | 24 | 0.036 | 0.034 | 0.036 | 0.002 |
| `- 2 - 4 - - 7 8 -` | 24 | 0.053 | 0.053 | 0.053 | 0.000 |
| `- 2 - - 5 6 - 8 -` | 24 | 0.045 | 0.038 | 0.045 | 0.007 |
| `- - 3 4 5 - - - 9` | 24 | 0.031 | 0.024 | 0.031 | 0.008 |
| `- - 3 4 - 6 - 8 -` | 24 | 0.054 | 0.054 | 0.054 | 0.000 |
| `- - 3 - 5 6 7 - -` | 24 | 0.063 | 0.063 | 0.063 | 0.000 |
| `- - - 4 - - - 8 9` | 24 | 0.014 | 0.014 | 0.014 | 0.000 |
| `- - - - 5 - 7 - 9` | 24 | 0.019 | 0.019 | 0.019 | 0.000 |
| `- - - - - 6 7 8 -` | 24 | 0.019 | 0.019 | 0.019 | 0.000 |
| `1 2 3 - 5 - - - 9` | 25 | 0.072 | 0.051 | 0.072 | 0.020 |
| `1 2 3 - - 6 - 8 -` | 25 | 0.091 | 0.065 | 0.091 | 0.027 |
| `1 2 - 4 5 - - 8 -` | 25 | 0.096 | 0.066 | 0.095 | 0.030 |
| `1 2 - 4 - 6 7 - -` | 25 | 0.104 | 0.078 | 0.103 | 0.026 |
| `1 2 - - - - - 8 9` | 25 | 0.035 | 0.033 | 0.035 | 0.002 |
| `1 - 3 4 5 - 7 - -` | 25 | 0.100 | 0.077 | 0.100 | 0.023 |
| `1 - 3 - - - 7 - 9` | 25 | 0.046 | 0.044 | 0.046 | 0.002 |
| `1 - - 4 - 6 - - 9` | 25 | 0.045 | 0.043 | 0.045 | 0.002 |
| `1 - - 4 - - 7 8 -` | 25 | 0.050 | 0.044 | 0.050 | 0.005 |
| `1 - - - 5 6 - 8 -` | 25 | 0.054 | 0.050 | 0.054 | 0.003 |
| `- 2 3 4 5 6 - - -` | 25 | 0.116 | 0.089 | 0.115 | 0.027 |
| `- 2 3 - - 6 - - 9` | 25 | 0.040 | 0.034 | 0.040 | 0.006 |
| `- 2 3 - - - 7 8 -` | 25 | 0.056 | 0.055 | 0.056 | 0.001 |
| `- 2 - 4 5 - - - 9` | 25 | 0.037 | 0.029 | 0.037 | 0.008 |
| `- 2 - 4 - 6 - 8 -` | 25 | 0.036 | 0.029 | 0.036 | 0.008 |
| `- 2 - - 5 6 7 - -` | 25 | 0.061 | 0.055 | 0.061 | 0.006 |
| `- - 3 4 5 - - 8 -` | 25 | 0.051 | 0.044 | 0.051 | 0.008 |
| `- - 3 4 - 6 7 - -` | 25 | 0.054 | 0.048 | 0.054 | 0.006 |
| `- - 3 - - - - 8 9` | 25 | 0.025 | 0.025 | 0.025 | 0.000 |
| `- - - 4 - - 7 - 9` | 25 | 0.022 | 0.022 | 0.022 | 0.000 |
| `- - - - 5 6 - - 9` | 25 | 0.023 | 0.023 | 0.023 | 0.000 |
| `- - - - 5 - 7 8 -` | 25 | 0.023 | 0.023 | 0.023 | 0.000 |
| `1 2 3 4 - - - - 9` | 26 | 0.079 | 0.062 | 0.079 | 0.018 |
| `1 2 3 - 5 - - 8 -` | 26 | 0.099 | 0.073 | 0.099 | 0.026 |
| `1 2 3 - - 6 7 - -` | 26 | 0.112 | 0.088 | 0.112 | 0.024 |
| `1 2 - 4 5 - 7 - -` | 26 | 0.117 | 0.088 | 0.116 | 0.028 |
| `1 2 - - - - 7 - 9` | 26 | 0.054 | 0.051 | 0.054 | 0.004 |
| `1 - 3 4 5 6 - - -` | 26 | 0.118 | 0.091 | 0.117 | 0.027 |
| `1 - 3 - - 6 - - 9` | 26 | 0.046 | 0.040 | 0.046 | 0.006 |
| `1 - 3 - - - 7 8 -` | 26 | 0.064 | 0.060 | 0.064 | 0.004 |
| `1 - - 4 5 - - - 9` | 26 | 0.035 | 0.028 | 0.035 | 0.007 |
| `1 - - 4 - 6 - 8 -` | 26 | 0.075 | 0.075 | 0.075 | 0.000 |
| `1 - - - 5 6 7 - -` | 26 | 0.056 | 0.049 | 0.056 | 0.008 |
| `- 2 3 - 5 - - - 9` | 26 | 0.059 | 0.058 | 0.059 | 0.001 |
| `- 2 3 - - 6 - 8 -` | 26 | 0.058 | 0.053 | 0.058 | 0.006 |
| `- 2 - 4 5 - - 8 -` | 26 | 0.076 | 0.076 | 0.076 | 0.000 |
| `- 2 - 4 - 6 7 - -` | 26 | 0.077 | 0.074 | 0.077 | 0.003 |
| `- 2 - - - - - 8 9` | 26 | 0.037 | 0.037 | 0.037 | 0.000 |
| `- - 3 4 5 - 7 - -` | 26 | 0.080 | 0.073 | 0.080 | 0.007 |
| `- - 3 - - - 7 - 9` | 26 | 0.034 | 0.034 | 0.034 | 0.000 |
| `- - - 4 - 6 - - 9` | 26 | 0.026 | 0.026 | 0.026 | 0.000 |
| `- - - 4 - - 7 8 -` | 26 | 0.036 | 0.036 | 0.036 | 0.000 |
| `- - - - 5 6 - 8 -` | 26 | 0.028 | 0.028 | 0.028 | 0.000 |
| `1 2 3 4 - - - 8 -` | 27 | 0.108 | 0.085 | 0.108 | 0.024 |
| `1 2 3 - 5 - 7 - -` | 27 | 0.122 | 0.098 | 0.122 | 0.023 |
| `1 2 - 4 5 6 - - -` | 27 | 0.126 | 0.104 | 0.126 | 0.022 |
| `1 2 - - - 6 - - 9` | 27 | 0.072 | 0.069 | 0.072 | 0.003 |
| `1 2 - - - - 7 8 -` | 27 | 0.066 | 0.063 | 0.066 | 0.004 |
| `1 - 3 - 5 - - - 9` | 27 | 0.058 | 0.054 | 0.058 | 0.004 |
| `1 - 3 - - 6 - 8 -` | 27 | 0.084 | 0.083 | 0.084 | 0.002 |
| `1 - - 4 5 - - 8 -` | 27 | 0.064 | 0.060 | 0.064 | 0.004 |
| `1 - - 4 - 6 7 - -` | 27 | 0.081 | 0.076 | 0.081 | 0.005 |
| `1 - - - - - - 8 9` | 27 | 0.035 | 0.035 | 0.035 | 0.000 |
| `- 2 3 4 - - - - 9` | 27 | 0.055 | 0.050 | 0.055 | 0.005 |
| `- 2 3 - 5 - - 8 -` | 27 | 0.061 | 0.054 | 0.061 | 0.007 |
| `- 2 3 - - 6 7 - -` | 27 | 0.095 | 0.094 | 0.095 | 0.001 |
| `- 2 - 4 5 - 7 - -` | 27 | 0.072 | 0.064 | 0.072 | 0.008 |
| `- 2 - - - - 7 - 9` | 27 | 0.034 | 0.033 | 0.034 | 0.001 |
| `- - 3 4 5 6 - - -` | 27 | 0.103 | 0.102 | 0.103 | 0.000 |
| `- - 3 - - 6 - - 9` | 27 | 0.025 | 0.023 | 0.025 | 0.002 |
| `- - 3 - - - 7 8 -` | 27 | 0.049 | 0.049 | 0.049 | 0.000 |
| `- - - 4 5 - - - 9` | 27 | 0.019 | 0.016 | 0.019 | 0.002 |
| `- - - 4 - 6 - 8 -` | 27 | 0.041 | 0.041 | 0.041 | 0.000 |
| `- - - - 5 6 7 - -` | 27 | 0.042 | 0.042 | 0.042 | 0.000 |
| `1 2 3 4 - - 7 - -` | 28 | 0.132 | 0.106 | 0.132 | 0.026 |
| `1 2 3 - 5 6 - - -` | 28 | 0.135 | 0.115 | 0.134 | 0.020 |
| `1 2 - - 5 - - - 9` | 28 | 0.088 | 0.088 | 0.088 | 0.000 |
| `1 2 - - - 6 - 8 -` | 28 | 0.087 | 0.083 | 0.087 | 0.004 |
| `1 - 3 4 - - - - 9` | 28 | 0.074 | 0.073 | 0.074 | 0.001 |
| `1 - 3 - 5 - - 8 -` | 28 | 0.071 | 0.065 | 0.071 | 0.006 |
| `1 - 3 - - 6 7 - -` | 28 | 0.097 | 0.093 | 0.097 | 0.004 |
| `1 - - 4 5 - 7 - -` | 28 | 0.106 | 0.103 | 0.106 | 0.003 |
| `1 - - - - - 7 - 9` | 28 | 0.059 | 0.059 | 0.059 | 0.000 |
| `- 2 3 4 - - - 8 -` | 28 | 0.102 | 0.102 | 0.102 | 0.000 |
| `- 2 3 - 5 - 7 - -` | 28 | 0.086 | 0.080 | 0.086 | 0.006 |
| `- 2 - 4 5 6 - - -` | 28 | 0.112 | 0.106 | 0.112 | 0.005 |
| `- 2 - - - 6 - - 9` | 28 | 0.049 | 0.049 | 0.049 | 0.000 |
| `- 2 - - - - 7 8 -` | 28 | 0.063 | 0.063 | 0.063 | 0.000 |
| `- - 3 - 5 - - - 9` | 28 | 0.041 | 0.041 | 0.041 | 0.000 |
| `- - 3 - - 6 - 8 -` | 28 | 0.053 | 0.053 | 0.053 | 0.000 |
| `- - - 4 5 - - 8 -` | 28 | 0.045 | 0.045 | 0.045 | 0.000 |
| `- - - 4 - 6 7 - -` | 28 | 0.055 | 0.055 | 0.055 | 0.000 |
| `- - - - - - - 8 9` | 28 | 0.031 | 0.031 | 0.031 | 0.000 |
| `1 2 3 4 - 6 - - -` | 29 | 0.143 | 0.121 | 0.142 | 0.022 |
| `1 2 - 4 - - - - 9` | 29 | 0.083 | 0.083 | 0.083 | 0.000 |
| `1 2 - - 5 - - 8 -` | 29 | 0.102 | 0.099 | 0.102 | 0.003 |
| `1 2 - - - 6 7 - -` | 29 | 0.093 | 0.089 | 0.093 | 0.004 |
| `1 - 3 4 - - - 8 -` | 29 | 0.084 | 0.081 | 0.084 | 0.004 |
| `1 - 3 - 5 - 7 - -` | 29 | 0.119 | 0.117 | 0.119 | 0.002 |
| `1 - - 4 5 6 - - -` | 29 | 0.096 | 0.092 | 0.096 | 0.005 |
| `1 - - - - 6 - - 9` | 29 | 0.060 | 0.060 | 0.060 | 0.000 |
| `1 - - - - - 7 8 -` | 29 | 0.056 | 0.056 | 0.056 | 0.000 |
| `- 2 3 4 - - 7 - -` | 29 | 0.102 | 0.095 | 0.102 | 0.007 |
| `- 2 3 - 5 6 - - -` | 29 | 0.115 | 0.112 | 0.115 | 0.003 |
| `- 2 - - 5 - - - 9` | 29 | 0.051 | 0.051 | 0.051 | 0.000 |
| `- 2 - - - 6 - 8 -` | 29 | 0.046 | 0.045 | 0.046 | 0.001 |
| `- - 3 4 - - - - 9` | 29 | 0.045 | 0.045 | 0.045 | 0.000 |
| `- - 3 - 5 - - 8 -` | 29 | 0.037 | 0.035 | 0.037 | 0.002 |
| `- - 3 - - 6 7 - -` | 29 | 0.068 | 0.068 | 0.068 | 0.000 |
| `- - - 4 5 - 7 - -` | 29 | 0.063 | 0.063 | 0.063 | 0.000 |
| `- - - - - - 7 - 9` | 29 | 0.037 | 0.037 | 0.037 | 0.000 |
| `1 2 3 4 5 - - - -` | 30 | 0.144 | 0.130 | 0.144 | 0.013 |
| `1 2 3 - - - - - 9` | 30 | 0.065 | 0.065 | 0.065 | 0.000 |
| `1 2 - 4 - - - 8 -` | 30 | 0.116 | 0.116 | 0.116 | 0.000 |
| `1 2 - - 5 - 7 - -` | 30 | 0.111 | 0.107 | 0.111 | 0.004 |
| `1 - 3 4 - - 7 - -` | 30 | 0.091 | 0.085 | 0.091 | 0.006 |
| `1 - 3 - 5 6 - - -` | 30 | 0.125 | 0.123 | 0.125 | 0.003 |
| `1 - - - 5 - - - 9` | 30 | 0.049 | 0.049 | 0.049 | 0.000 |
| `1 - - - - 6 - 8 -` | 30 | 0.077 | 0.077 | 0.077 | 0.000 |
| `- 2 3 4 - 6 - - -` | 30 | 0.129 | 0.125 | 0.129 | 0.005 |
| `- 2 - 4 - - - - 9` | 30 | 0.042 | 0.042 | 0.042 | 0.000 |
| `- 2 - - 5 - - 8 -` | 30 | 0.067 | 0.067 | 0.067 | 0.000 |
| `- 2 - - - 6 7 - -` | 30 | 0.081 | 0.081 | 0.081 | 0.000 |
| `- - 3 4 - - - 8 -` | 30 | 0.063 | 0.063 | 0.063 | 0.000 |
| `- - 3 - 5 - 7 - -` | 30 | 0.074 | 0.074 | 0.074 | 0.000 |
| `- - - 4 5 6 - - -` | 30 | 0.066 | 0.066 | 0.066 | 0.000 |
| `- - - - - 6 - - 9` | 30 | 0.031 | 0.031 | 0.031 | 0.000 |
| `- - - - - - 7 8 -` | 30 | 0.046 | 0.046 | 0.046 | 0.000 |
| `1 2 3 - - - - 8 -` | 31 | 0.089 | 0.089 | 0.089 | 0.000 |
| `1 2 - 4 - - 7 - -` | 31 | 0.122 | 0.120 | 0.122 | 0.003 |
| `1 2 - - 5 6 - - -` | 31 | 0.109 | 0.106 | 0.109 | 0.003 |
| `1 - 3 4 - 6 - - -` | 31 | 0.124 | 0.121 | 0.124 | 0.003 |
| `1 - - 4 - - - - 9` | 31 | 0.039 | 0.039 | 0.039 | 0.000 |
| `1 - - - 5 - - 8 -` | 31 | 0.063 | 0.063 | 0.063 | 0.000 |
| `1 - - - - 6 7 - -` | 31 | 0.066 | 0.066 | 0.066 | 0.000 |
| `- 2 3 4 5 - - - -` | 31 | 0.131 | 0.127 | 0.131 | 0.004 |
| `- 2 3 - - - - - 9` | 31 | 0.033 | 0.033 | 0.033 | 0.000 |
| `- 2 - 4 - - - 8 -` | 31 | 0.056 | 0.056 | 0.056 | 0.000 |
| `- 2 - - 5 - 7 - -` | 31 | 0.057 | 0.056 | 0.057 | 0.001 |
| `- - 3 4 - - 7 - -` | 31 | 0.052 | 0.051 | 0.052 | 0.002 |
| `- - 3 - 5 6 - - -` | 31 | 0.075 | 0.075 | 0.075 | 0.000 |
| `- - - - 5 - - - 9` | 31 | 0.025 | 0.025 | 0.025 | 0.000 |
| `- - - - - 6 - 8 -` | 31 | 0.039 | 0.039 | 0.039 | 0.000 |
| `1 2 3 - - - 7 - -` | 32 | 0.113 | 0.113 | 0.113 | 0.000 |
| `1 2 - 4 - 6 - - -` | 32 | 0.117 | 0.114 | 0.117 | 0.003 |
| `1 - 3 4 5 - - - -` | 32 | 0.113 | 0.111 | 0.113 | 0.002 |
| `1 - 3 - - - - - 9` | 32 | 0.028 | 0.028 | 0.028 | 0.000 |
| `1 - - 4 - - - 8 -` | 32 | 0.049 | 0.049 | 0.049 | 0.000 |
| `1 - - - 5 - 7 - -` | 32 | 0.077 | 0.077 | 0.077 | 0.000 |
| `- 2 3 - - - - 8 -` | 32 | 0.044 | 0.044 | 0.044 | 0.000 |
| `- 2 - 4 - - 7 - -` | 32 | 0.070 | 0.070 | 0.070 | 0.000 |
| `- 2 - - 5 6 - - -` | 32 | 0.083 | 0.083 | 0.083 | 0.000 |
| `- - 3 4 - 6 - - -` | 32 | 0.078 | 0.078 | 0.078 | 0.000 |
| `- - - 4 - - - - 9` | 32 | 0.019 | 0.019 | 0.019 | 0.000 |
| `- - - - 5 - - 8 -` | 32 | 0.031 | 0.031 | 0.031 | 0.000 |
| `- - - - - 6 7 - -` | 32 | 0.046 | 0.046 | 0.046 | 0.000 |
| `1 2 3 - - 6 - - -` | 33 | 0.129 | 0.127 | 0.129 | 0.002 |
| `1 2 - 4 5 - - - -` | 33 | 0.130 | 0.129 | 0.130 | 0.002 |
| `1 2 - - - - - - 9` | 33 | 0.045 | 0.045 | 0.045 | 0.000 |
| `1 - 3 - - - - 8 -` | 33 | 0.063 | 0.063 | 0.063 | 0.000 |
| `1 - - 4 - - 7 - -` | 33 | 0.088 | 0.088 | 0.088 | 0.000 |
| `1 - - - 5 6 - - -` | 33 | 0.084 | 0.084 | 0.084 | 0.000 |
| `- 2 3 - - - 7 - -` | 33 | 0.083 | 0.083 | 0.083 | 0.000 |
| `- 2 - 4 - 6 - - -` | 33 | 0.077 | 0.076 | 0.077 | 0.001 |
| `- - 3 4 5 - - - -` | 33 | 0.103 | 0.103 | 0.103 | 0.000 |
| `- - 3 - - - - - 9` | 33 | 0.040 | 0.040 | 0.040 | 0.000 |
| `- - - 4 - - - 8 -` | 33 | 0.051 | 0.051 | 0.051 | 0.000 |
| `- - - - 5 - 7 - -` | 33 | 0.065 | 0.065 | 0.065 | 0.000 |
| `1 2 3 - 5 - - - -` | 34 | 0.140 | 0.138 | 0.140 | 0.002 |
| `1 2 - - - - - 8 -` | 34 | 0.077 | 0.077 | 0.077 | 0.000 |
| `1 - 3 - - - 7 - -` | 34 | 0.099 | 0.099 | 0.099 | 0.000 |
| `1 - - 4 - 6 - - -` | 34 | 0.114 | 0.114 | 0.114 | 0.000 |
| `- 2 3 - - 6 - - -` | 34 | 0.109 | 0.109 | 0.109 | 0.000 |
| `- 2 - 4 5 - - - -` | 34 | 0.122 | 0.122 | 0.122 | 0.000 |
| `- 2 - - - - - - 9` | 34 | 0.062 | 0.062 | 0.062 | 0.000 |
| `- - 3 - - - - 8 -` | 34 | 0.071 | 0.071 | 0.071 | 0.000 |
| `- - - 4 - - 7 - -` | 34 | 0.083 | 0.083 | 0.083 | 0.000 |
| `- - - - 5 6 - - -` | 34 | 0.086 | 0.086 | 0.086 | 0.000 |
| `1 2 3 4 - - - - -` | 35 | 0.147 | 0.146 | 0.147 | 0.001 |
| `1 2 - - - - 7 - -` | 35 | 0.110 | 0.110 | 0.110 | 0.000 |
| `1 - 3 - - 6 - - -` | 35 | 0.125 | 0.125 | 0.125 | 0.000 |
| `1 - - 4 5 - - - -` | 35 | 0.119 | 0.119 | 0.119 | 0.000 |
| `1 - - - - - - - 9` | 35 | 0.083 | 0.083 | 0.083 | 0.000 |
| `- 2 3 - 5 - - - -` | 35 | 0.123 | 0.123 | 0.123 | 0.000 |
| `- 2 - - - - - 8 -` | 35 | 0.091 | 0.091 | 0.091 | 0.000 |
| `- - 3 - - - 7 - -` | 35 | 0.102 | 0.102 | 0.102 | 0.000 |
| `- - - 4 - 6 - - -` | 35 | 0.106 | 0.106 | 0.106 | 0.000 |
| `1 2 - - - 6 - - -` | 36 | 0.136 | 0.136 | 0.136 | 0.000 |
| `1 - 3 - 5 - - - -` | 36 | 0.145 | 0.145 | 0.145 | 0.000 |
| `1 - - - - - - 8 -` | 36 | 0.111 | 0.111 | 0.111 | 0.000 |
| `- 2 3 4 - - - - -` | 36 | 0.155 | 0.155 | 0.155 | 0.000 |
| `- 2 - - - - 7 - -` | 36 | 0.120 | 0.120 | 0.120 | 0.000 |
| `- - 3 - - 6 - - -` | 36 | 0.127 | 0.127 | 0.127 | 0.000 |
| `- - - 4 5 - - - -` | 36 | 0.130 | 0.130 | 0.130 | 0.000 |
| `- - - - - - - - 9` | 36 | 0.111 | 0.111 | 0.111 | 0.000 |
| `1 2 - - 5 - - - -` | 37 | 0.159 | 0.159 | 0.159 | 0.000 |
| `1 - 3 4 - - - - -` | 37 | 0.158 | 0.158 | 0.158 | 0.000 |
| `1 - - - - - 7 - -` | 37 | 0.139 | 0.139 | 0.139 | 0.000 |
| `- 2 - - - 6 - - -` | 37 | 0.147 | 0.147 | 0.147 | 0.000 |
| `- - 3 - 5 - - - -` | 37 | 0.151 | 0.151 | 0.151 | 0.000 |
| `- - - - - - - 8 -` | 37 | 0.139 | 0.139 | 0.139 | 0.000 |
| `1 2 - 4 - - - - -` | 38 | 0.182 | 0.182 | 0.182 | 0.000 |
| `1 - - - - 6 - - -` | 38 | 0.167 | 0.167 | 0.167 | 0.000 |
| `- 2 - - 5 - - - -` | 38 | 0.173 | 0.173 | 0.173 | 0.000 |
| `- - 3 4 - - - - -` | 38 | 0.176 | 0.176 | 0.176 | 0.000 |
| `- - - - - - 7 - -` | 38 | 0.167 | 0.167 | 0.167 | 0.000 |
| `1 2 3 - - - - - -` | 39 | 0.147 | 0.147 | 0.147 | 0.000 |
| `1 - - - 5 - - - -` | 39 | 0.139 | 0.139 | 0.139 | 0.000 |
| `- 2 - 4 - - - - -` | 39 | 0.144 | 0.144 | 0.144 | 0.000 |
| `- - - - - 6 - - -` | 39 | 0.139 | 0.139 | 0.139 | 0.000 |
| `1 - - 4 - - - - -` | 40 | 0.111 | 0.111 | 0.111 | 0.000 |
| `- 2 3 - - - - - -` | 40 | 0.114 | 0.114 | 0.114 | 0.000 |
| `- - - - 5 - - - -` | 40 | 0.111 | 0.111 | 0.111 | 0.000 |
| `1 - 3 - - - - - -` | 41 | 0.083 | 0.083 | 0.083 | 0.000 |
| `- - - 4 - - - - -` | 41 | 0.083 | 0.083 | 0.083 | 0.000 |
| `1 2 - - - - - - -` | 42 | 0.056 | 0.056 | 0.056 | 0.000 |
| `- - 3 - - - - - -` | 42 | 0.056 | 0.056 | 0.056 | 0.000 |
| `- 2 - - - - - - -` | 43 | 0.028 | 0.028 | 0.028 | 0.000 |
| `1 - - - - - - - -` | 44 | 0.000 | 0.000 | 0.000 | 0.000 |
| `- - - - - - - - -` | 45 | 1.000 | 1.000 | 1.000 | 0.000 |


### Exhibit C: All Possible Board States with Expected Score of Each Strategy for Flipped Count Scoring System

In [12]:
def exhibit_c(results: dict) -> Table:
    """returns a table for expected scores of the flipped count scoring system"""
    exhibit_table = []
    results_subtable = results['flipped_count_board_table']
    for record in results_subtable:
        strategic_difference = record['Max ES'] - record['Min ES']
        exhibit_table.append({
            'Board (Remaining)': f'`{record["Board"]}`',
            'Board Sum': record['Sum'],
            'Max ES': f'{record["Max ES"]:.3f}',
            'Min ES': f'{record["Min ES"]:.3f}',
            'Highest Tile ES': f'{record["Highest Tile ES"]:.3f}',
            'Strategic Difference': f'{strategic_difference:.3f}',
        })
    # sort by board sum ascending
    exhibit_table.sort(key=lambda record: record['Board Sum'])
    return exhibit_table

display_table(exhibit_c(full_results))

| Board (Remaining) | Board Sum | Max ES | Min ES | Highest Tile ES | Strategic Difference |
| --- | --- | --- | --- | --- | --- |
| `1 2 3 4 5 6 7 8 9` | 0 | 6.804 | 5.372 | 6.659 | 1.433 |
| `- 2 3 4 5 6 7 8 9` | 1 | 6.289 | 5.395 | 6.223 | 0.894 |
| `1 - 3 4 5 6 7 8 9` | 2 | 6.129 | 5.304 | 6.024 | 0.825 |
| `1 2 - 4 5 6 7 8 9` | 3 | 6.439 | 5.470 | 6.313 | 0.969 |
| `- - 3 4 5 6 7 8 9` | 3 | 5.724 | 5.393 | 5.681 | 0.331 |
| `1 2 3 - 5 6 7 8 9` | 4 | 6.483 | 5.446 | 6.323 | 1.037 |
| `- 2 - 4 5 6 7 8 9` | 4 | 5.714 | 5.354 | 5.665 | 0.360 |
| `1 2 3 4 - 6 7 8 9` | 5 | 6.670 | 5.525 | 6.522 | 1.145 |
| `1 - - 4 5 6 7 8 9` | 5 | 5.684 | 5.260 | 5.604 | 0.424 |
| `- 2 3 - 5 6 7 8 9` | 5 | 5.610 | 5.260 | 5.551 | 0.349 |
| `1 2 3 4 5 - 7 8 9` | 6 | 6.765 | 5.566 | 6.605 | 1.199 |
| `1 - 3 - 5 6 7 8 9` | 6 | 6.039 | 5.521 | 5.967 | 0.519 |
| `- 2 3 4 - 6 7 8 9` | 6 | 6.205 | 5.665 | 6.133 | 0.539 |
| `- - - 4 5 6 7 8 9` | 6 | 5.523 | 5.460 | 5.507 | 0.063 |
| `1 2 3 4 5 6 - 8 9` | 7 | 6.875 | 5.654 | 6.725 | 1.220 |
| `1 2 - - 5 6 7 8 9` | 7 | 5.743 | 5.328 | 5.653 | 0.415 |
| `1 - 3 4 - 6 7 8 9` | 7 | 6.089 | 5.555 | 6.005 | 0.534 |
| `- 2 3 4 5 - 7 8 9` | 7 | 6.241 | 5.667 | 6.146 | 0.574 |
| `- - 3 - 5 6 7 8 9` | 7 | 5.524 | 5.404 | 5.490 | 0.120 |
| `1 2 3 4 5 6 7 - 9` | 8 | 7.027 | 5.849 | 6.878 | 1.177 |
| `1 2 - 4 - 6 7 8 9` | 8 | 6.383 | 5.675 | 6.255 | 0.708 |
| `1 - 3 4 5 - 7 8 9` | 8 | 6.101 | 5.568 | 6.012 | 0.534 |
| `- 2 3 4 5 6 - 8 9` | 8 | 6.455 | 5.712 | 6.402 | 0.743 |
| `- 2 - - 5 6 7 8 9` | 8 | 5.524 | 5.426 | 5.491 | 0.098 |
| `- - 3 4 - 6 7 8 9` | 8 | 5.539 | 5.414 | 5.498 | 0.126 |
| `1 2 3 4 5 6 7 8 -` | 9 | 7.199 | 6.064 | 7.037 | 1.135 |
| `1 2 3 - - 6 7 8 9` | 9 | 6.383 | 5.721 | 6.256 | 0.662 |
| `1 2 - 4 5 - 7 8 9` | 9 | 6.499 | 5.781 | 6.401 | 0.718 |
| `1 - 3 4 5 6 - 8 9` | 9 | 6.302 | 5.632 | 6.157 | 0.670 |
| `1 - - - 5 6 7 8 9` | 9 | 5.508 | 5.369 | 5.428 | 0.140 |
| `- 2 3 4 5 6 7 - 9` | 9 | 6.544 | 5.871 | 6.475 | 0.673 |
| `- 2 - 4 - 6 7 8 9` | 9 | 5.492 | 5.371 | 5.442 | 0.121 |
| `- - 3 4 5 - 7 8 9` | 9 | 5.426 | 5.338 | 5.362 | 0.088 |
| `1 2 3 - 5 - 7 8 9` | 10 | 6.519 | 5.817 | 6.345 | 0.703 |
| `1 2 - 4 5 6 - 8 9` | 10 | 6.624 | 5.800 | 6.502 | 0.825 |
| `1 - 3 4 5 6 7 - 9` | 10 | 6.519 | 5.851 | 6.432 | 0.668 |
| `1 - - 4 - 6 7 8 9` | 10 | 5.947 | 5.687 | 5.889 | 0.260 |
| `- 2 3 4 5 6 7 8 -` | 10 | 6.817 | 6.135 | 6.737 | 0.682 |
| `- 2 3 - - 6 7 8 9` | 10 | 5.852 | 5.684 | 5.806 | 0.167 |
| `- 2 - 4 5 - 7 8 9` | 10 | 5.934 | 5.702 | 5.876 | 0.232 |
| `- - 3 4 5 6 - 8 9` | 10 | 5.982 | 5.755 | 5.943 | 0.227 |
| `- - - - 5 6 7 8 9` | 10 | 5.512 | 5.512 | 5.512 | 0.000 |
| `1 2 3 4 - - 7 8 9` | 11 | 6.618 | 5.869 | 6.430 | 0.749 |
| `1 2 3 - 5 6 - 8 9` | 11 | 6.631 | 5.931 | 6.453 | 0.699 |
| `1 2 - 4 5 6 7 - 9` | 11 | 6.731 | 5.937 | 6.596 | 0.794 |
| `1 - 3 4 5 6 7 8 -` | 11 | 6.677 | 6.017 | 6.556 | 0.660 |
| `1 - 3 - - 6 7 8 9` | 11 | 5.767 | 5.538 | 5.681 | 0.229 |
| `1 - - 4 5 - 7 8 9` | 11 | 5.890 | 5.637 | 5.822 | 0.253 |
| `- 2 3 - 5 - 7 8 9` | 11 | 5.438 | 5.289 | 5.334 | 0.149 |
| `- 2 - 4 5 6 - 8 9` | 11 | 6.070 | 5.755 | 6.008 | 0.315 |
| `- - 3 4 5 6 7 - 9` | 11 | 6.168 | 5.914 | 6.116 | 0.254 |
| `- - - 4 - 6 7 8 9` | 11 | 5.737 | 5.737 | 5.737 | 0.000 |
| `1 2 3 4 - 6 - 8 9` | 12 | 6.846 | 5.978 | 6.657 | 0.868 |
| `1 2 3 - 5 6 7 - 9` | 12 | 6.823 | 6.067 | 6.672 | 0.756 |
| `1 2 - 4 5 6 7 8 -` | 12 | 6.908 | 6.131 | 6.774 | 0.777 |
| `1 2 - - - 6 7 8 9` | 12 | 5.593 | 5.388 | 5.442 | 0.204 |
| `1 - 3 - 5 - 7 8 9` | 12 | 6.197 | 5.947 | 6.144 | 0.250 |
| `1 - - 4 5 6 - 8 9` | 12 | 5.950 | 5.668 | 5.815 | 0.281 |
| `- 2 3 4 - - 7 8 9` | 12 | 6.338 | 6.083 | 6.290 | 0.255 |
| `- 2 3 - 5 6 - 8 9` | 12 | 5.957 | 5.747 | 5.853 | 0.210 |
| `- 2 - 4 5 6 7 - 9` | 12 | 6.077 | 5.806 | 5.978 | 0.271 |
| `- - 3 4 5 6 7 8 -` | 12 | 6.396 | 6.124 | 6.362 | 0.272 |
| `- - 3 - - 6 7 8 9` | 12 | 5.725 | 5.701 | 5.701 | 0.024 |
| `- - - 4 5 - 7 8 9` | 12 | 5.445 | 5.399 | 5.402 | 0.045 |
| `1 2 3 4 5 - - 8 9` | 13 | 6.896 | 5.958 | 6.687 | 0.938 |
| `1 2 3 4 - 6 7 - 9` | 13 | 6.958 | 6.127 | 6.828 | 0.831 |
| `1 2 3 - 5 6 7 8 -` | 13 | 6.976 | 6.107 | 6.813 | 0.869 |
| `1 2 - - 5 - 7 8 9` | 13 | 6.107 | 5.780 | 5.988 | 0.327 |
| `1 - 3 4 - - 7 8 9` | 13 | 5.761 | 5.535 | 5.645 | 0.227 |
| `1 - 3 - 5 6 - 8 9` | 13 | 6.245 | 5.973 | 6.154 | 0.272 |
| `1 - - 4 5 6 7 - 9` | 13 | 6.107 | 5.895 | 6.025 | 0.212 |
| `- 2 3 4 - 6 - 8 9` | 13 | 6.397 | 6.049 | 6.335 | 0.349 |
| `- 2 3 - 5 6 7 - 9` | 13 | 6.173 | 5.845 | 6.125 | 0.328 |
| `- 2 - 4 5 6 7 8 -` | 13 | 6.368 | 6.066 | 6.313 | 0.302 |
| `- 2 - - - 6 7 8 9` | 13 | 5.683 | 5.596 | 5.596 | 0.087 |
| `- - 3 - 5 - 7 8 9` | 13 | 5.694 | 5.644 | 5.646 | 0.050 |
| `- - - 4 5 6 - 8 9` | 13 | 5.612 | 5.596 | 5.596 | 0.017 |
| `1 2 3 4 5 - 7 - 9` | 14 | 7.059 | 6.065 | 6.846 | 0.994 |
| `1 2 3 4 - 6 7 8 -` | 14 | 7.137 | 6.239 | 6.985 | 0.899 |
| `1 2 - 4 - - 7 8 9` | 14 | 6.515 | 6.148 | 6.427 | 0.366 |
| `1 2 - - 5 6 - 8 9` | 14 | 6.217 | 5.922 | 6.153 | 0.295 |
| `1 - 3 4 - 6 - 8 9` | 14 | 6.420 | 6.107 | 6.353 | 0.313 |
| `1 - 3 - 5 6 7 - 9` | 14 | 6.503 | 6.185 | 6.429 | 0.318 |
| `1 - - 4 5 6 7 8 -` | 14 | 6.303 | 5.995 | 6.201 | 0.308 |
| `1 - - - - 6 7 8 9` | 14 | 5.554 | 5.401 | 5.401 | 0.152 |
| `- 2 3 4 5 - - 8 9` | 14 | 6.510 | 6.137 | 6.436 | 0.372 |
| `- 2 3 4 - 6 7 - 9` | 14 | 6.480 | 6.164 | 6.412 | 0.316 |
| `- 2 3 - 5 6 7 8 -` | 14 | 6.245 | 5.979 | 6.161 | 0.266 |
| `- 2 - - 5 - 7 8 9` | 14 | 5.702 | 5.634 | 5.634 | 0.068 |
| `- - 3 4 - - 7 8 9` | 14 | 5.698 | 5.618 | 5.621 | 0.079 |
| `- - 3 - 5 6 - 8 9` | 14 | 5.430 | 5.341 | 5.342 | 0.090 |
| `- - - 4 5 6 7 - 9` | 14 | 5.751 | 5.730 | 5.737 | 0.022 |
| `1 2 3 4 5 6 - - 9` | 15 | 7.142 | 6.235 | 6.948 | 0.907 |
| `1 2 3 4 5 - 7 8 -` | 15 | 7.231 | 6.296 | 7.055 | 0.935 |
| `1 2 3 - - - 7 8 9` | 15 | 6.533 | 6.151 | 6.428 | 0.382 |
| `1 2 - 4 - 6 - 8 9` | 15 | 6.663 | 6.197 | 6.511 | 0.465 |
| `1 2 - - 5 6 7 - 9` | 15 | 6.325 | 5.995 | 6.226 | 0.330 |
| `1 - 3 4 5 - - 8 9` | 15 | 6.299 | 5.991 | 6.215 | 0.307 |
| `1 - 3 4 - 6 7 - 9` | 15 | 6.503 | 6.195 | 6.379 | 0.307 |
| `1 - 3 - 5 6 7 8 -` | 15 | 6.632 | 6.207 | 6.545 | 0.425 |
| `1 - - - 5 - 7 8 9` | 15 | 5.882 | 5.818 | 5.840 | 0.064 |
| `- 2 3 4 5 - 7 - 9` | 15 | 6.516 | 6.144 | 6.458 | 0.373 |
| `- 2 3 4 - 6 7 8 -` | 15 | 6.800 | 6.342 | 6.728 | 0.458 |
| `- 2 - 4 - - 7 8 9` | 15 | 6.017 | 5.970 | 6.007 | 0.047 |
| `- 2 - - 5 6 - 8 9` | 15 | 6.034 | 6.007 | 6.021 | 0.027 |
| `- - 3 4 - 6 - 8 9` | 15 | 6.078 | 6.022 | 6.071 | 0.056 |
| `- - 3 - 5 6 7 - 9` | 15 | 6.168 | 6.108 | 6.157 | 0.060 |
| `- - - 4 5 6 7 8 -` | 15 | 6.266 | 6.262 | 6.263 | 0.005 |
| `- - - - - 6 7 8 9` | 15 | 5.857 | 5.857 | 5.857 | 0.000 |
| `1 2 3 4 5 6 - 8 -` | 16 | 7.322 | 6.374 | 7.149 | 0.949 |
| `1 2 3 - - 6 - 8 9` | 16 | 6.585 | 6.188 | 6.474 | 0.397 |
| `1 2 - 4 5 - - 8 9` | 16 | 6.718 | 6.199 | 6.585 | 0.519 |
| `1 2 - 4 - 6 7 - 9` | 16 | 6.764 | 6.318 | 6.649 | 0.446 |
| `1 2 - - 5 6 7 8 -` | 16 | 6.354 | 6.020 | 6.231 | 0.334 |
| `1 - 3 4 5 - 7 - 9` | 16 | 6.564 | 6.212 | 6.438 | 0.352 |
| `1 - 3 4 - 6 7 8 -` | 16 | 6.659 | 6.310 | 6.572 | 0.350 |
| `1 - - 4 - - 7 8 9` | 16 | 5.919 | 5.845 | 5.867 | 0.074 |
| `1 - - - 5 6 - 8 9` | 16 | 6.119 | 6.035 | 6.037 | 0.084 |
| `- 2 3 4 5 6 - - 9` | 16 | 6.790 | 6.348 | 6.713 | 0.442 |
| `- 2 3 4 5 - 7 8 -` | 16 | 6.808 | 6.402 | 6.683 | 0.406 |
| `- 2 3 - - - 7 8 9` | 16 | 5.917 | 5.867 | 5.897 | 0.050 |
| `- 2 - 4 - 6 - 8 9` | 16 | 5.621 | 5.537 | 5.537 | 0.084 |
| `- 2 - - 5 6 7 - 9` | 16 | 5.909 | 5.854 | 5.878 | 0.055 |
| `- - 3 4 5 - - 8 9` | 16 | 5.785 | 5.738 | 5.744 | 0.047 |
| `- - 3 4 - 6 7 - 9` | 16 | 5.751 | 5.672 | 5.673 | 0.079 |
| `- - 3 - 5 6 7 8 -` | 16 | 6.231 | 6.166 | 6.213 | 0.065 |
| `- - - - 5 - 7 8 9` | 16 | 5.875 | 5.875 | 5.875 | 0.000 |
| `1 2 3 4 5 6 7 - -` | 17 | 7.436 | 6.579 | 7.243 | 0.857 |
| `1 2 3 - 5 - - 8 9` | 17 | 6.659 | 6.236 | 6.466 | 0.423 |
| `1 2 3 - - 6 7 - 9` | 17 | 6.808 | 6.382 | 6.688 | 0.426 |
| `1 2 - 4 5 - 7 - 9` | 17 | 6.853 | 6.318 | 6.683 | 0.534 |
| `1 2 - 4 - 6 7 8 -` | 17 | 6.897 | 6.350 | 6.778 | 0.547 |
| `1 - 3 4 5 6 - - 9` | 17 | 6.687 | 6.240 | 6.600 | 0.447 |
| `1 - 3 4 5 - 7 8 -` | 17 | 6.707 | 6.316 | 6.597 | 0.391 |
| `1 - 3 - - - 7 8 9` | 17 | 5.845 | 5.745 | 5.770 | 0.100 |
| `1 - - 4 - 6 - 8 9` | 17 | 6.373 | 6.291 | 6.363 | 0.082 |
| `1 - - - 5 6 7 - 9` | 17 | 6.169 | 6.101 | 6.111 | 0.068 |
| `- 2 3 4 5 6 - 8 -` | 17 | 7.046 | 6.383 | 6.990 | 0.663 |
| `- 2 3 - - 6 - 8 9` | 17 | 5.874 | 5.795 | 5.797 | 0.080 |
| `- 2 - 4 5 - - 8 9` | 17 | 6.423 | 6.323 | 6.380 | 0.100 |
| `- 2 - 4 - 6 7 - 9` | 17 | 6.158 | 6.039 | 6.069 | 0.119 |
| `- 2 - - 5 6 7 8 -` | 17 | 6.225 | 6.174 | 6.182 | 0.051 |
| `- - 3 4 5 - 7 - 9` | 17 | 6.109 | 6.020 | 6.048 | 0.088 |
| `- - 3 4 - 6 7 8 -` | 17 | 6.248 | 6.179 | 6.224 | 0.069 |
| `- - - 4 - - 7 8 9` | 17 | 5.975 | 5.975 | 5.975 | 0.000 |
| `- - - - 5 6 - 8 9` | 17 | 5.898 | 5.898 | 5.898 | 0.000 |
| `1 2 3 4 - - - 8 9` | 18 | 6.809 | 6.325 | 6.552 | 0.484 |
| `1 2 3 - 5 - 7 - 9` | 18 | 6.919 | 6.458 | 6.717 | 0.461 |
| `1 2 3 - - 6 7 8 -` | 18 | 6.943 | 6.496 | 6.812 | 0.447 |
| `1 2 - 4 5 6 - - 9` | 18 | 6.901 | 6.417 | 6.694 | 0.484 |
| `1 2 - 4 5 - 7 8 -` | 18 | 7.014 | 6.521 | 6.923 | 0.494 |
| `1 2 - - - - 7 8 9` | 18 | 5.757 | 5.614 | 5.619 | 0.143 |
| `1 - 3 4 5 6 - 8 -` | 18 | 6.878 | 6.470 | 6.776 | 0.407 |
| `1 - 3 - - 6 - 8 9` | 18 | 6.266 | 6.137 | 6.199 | 0.129 |
| `1 - - 4 5 - - 8 9` | 18 | 5.748 | 5.610 | 5.628 | 0.138 |
| `1 - - 4 - 6 7 - 9` | 18 | 6.481 | 6.433 | 6.441 | 0.049 |
| `1 - - - 5 6 7 8 -` | 18 | 5.973 | 5.864 | 5.864 | 0.109 |
| `- 2 3 4 5 6 7 - -` | 18 | 7.139 | 6.701 | 7.080 | 0.438 |
| `- 2 3 - 5 - - 8 9` | 18 | 6.066 | 5.937 | 5.947 | 0.128 |
| `- 2 3 - - 6 7 - 9` | 18 | 6.423 | 6.300 | 6.377 | 0.123 |
| `- 2 - 4 5 - 7 - 9` | 18 | 5.810 | 5.710 | 5.744 | 0.100 |
| `- 2 - 4 - 6 7 8 -` | 18 | 6.189 | 6.087 | 6.133 | 0.102 |
| `- - 3 4 5 6 - - 9` | 18 | 6.531 | 6.415 | 6.514 | 0.116 |
| `- - 3 4 5 - 7 8 -` | 18 | 6.208 | 6.127 | 6.174 | 0.082 |
| `- - 3 - - - 7 8 9` | 18 | 6.162 | 6.162 | 6.162 | 0.000 |
| `- - - 4 - 6 - 8 9` | 18 | 5.999 | 5.999 | 5.999 | 0.000 |
| `- - - - 5 6 7 - 9` | 18 | 6.040 | 6.040 | 6.040 | 0.000 |
| `1 2 3 4 - - 7 - 9` | 19 | 6.974 | 6.474 | 6.785 | 0.501 |
| `1 2 3 - 5 6 - - 9` | 19 | 7.020 | 6.510 | 6.863 | 0.510 |
| `1 2 3 - 5 - 7 8 -` | 19 | 7.049 | 6.524 | 6.910 | 0.525 |
| `1 2 - 4 5 6 - 8 -` | 19 | 7.183 | 6.536 | 7.055 | 0.646 |
| `1 2 - - - 6 - 8 9` | 19 | 6.215 | 6.056 | 6.097 | 0.159 |
| `1 - 3 4 5 6 7 - -` | 19 | 7.101 | 6.534 | 6.950 | 0.567 |
| `1 - 3 - 5 - - 8 9` | 19 | 6.126 | 5.997 | 6.012 | 0.129 |
| `1 - 3 - - 6 7 - 9` | 19 | 6.370 | 6.249 | 6.266 | 0.122 |
| `1 - - 4 5 - 7 - 9` | 19 | 6.460 | 6.327 | 6.367 | 0.133 |
| `1 - - 4 - 6 7 8 -` | 19 | 6.563 | 6.410 | 6.487 | 0.153 |
| `- 2 3 4 - - - 8 9` | 19 | 6.676 | 6.638 | 6.665 | 0.038 |
| `- 2 3 - 5 - 7 - 9` | 19 | 6.135 | 5.986 | 5.997 | 0.148 |
| `- 2 3 - - 6 7 8 -` | 19 | 6.578 | 6.460 | 6.539 | 0.118 |
| `- 2 - 4 5 6 - - 9` | 19 | 6.550 | 6.422 | 6.488 | 0.128 |
| `- 2 - 4 5 - 7 8 -` | 19 | 6.663 | 6.532 | 6.606 | 0.132 |
| `- 2 - - - - 7 8 9` | 19 | 6.118 | 6.062 | 6.062 | 0.055 |
| `- - 3 4 5 6 - 8 -` | 19 | 6.692 | 6.580 | 6.662 | 0.112 |
| `- - 3 - - 6 - 8 9` | 19 | 5.959 | 5.896 | 5.896 | 0.063 |
| `- - - 4 5 - - 8 9` | 19 | 5.806 | 5.736 | 5.736 | 0.070 |
| `- - - 4 - 6 7 - 9` | 19 | 6.141 | 6.141 | 6.141 | 0.000 |
| `- - - - 5 6 7 8 -` | 19 | 6.112 | 6.112 | 6.112 | 0.000 |
| `1 2 3 4 - 6 - - 9` | 20 | 7.096 | 6.538 | 6.917 | 0.558 |
| `1 2 3 4 - - 7 8 -` | 20 | 7.147 | 6.597 | 6.938 | 0.550 |
| `1 2 3 - 5 6 - 8 -` | 20 | 7.166 | 6.576 | 6.937 | 0.590 |
| `1 2 - 4 5 6 7 - -` | 20 | 7.217 | 6.681 | 7.072 | 0.536 |
| `1 2 - - 5 - - 8 9` | 20 | 6.556 | 6.439 | 6.482 | 0.118 |
| `1 2 - - - 6 7 - 9` | 20 | 6.323 | 6.149 | 6.191 | 0.174 |
| `1 - 3 4 - - - 8 9` | 20 | 6.296 | 6.140 | 6.146 | 0.155 |
| `1 - 3 - 5 - 7 - 9` | 20 | 6.752 | 6.678 | 6.719 | 0.074 |
| `1 - 3 - - 6 7 8 -` | 20 | 6.514 | 6.335 | 6.420 | 0.179 |
| `1 - - 4 5 6 - - 9` | 20 | 6.151 | 5.955 | 5.979 | 0.196 |
| `1 - - 4 5 - 7 8 -` | 20 | 6.610 | 6.471 | 6.562 | 0.139 |
| `1 - - - - - 7 8 9` | 20 | 6.035 | 5.889 | 5.889 | 0.146 |
| `- 2 3 4 - - 7 - 9` | 20 | 6.334 | 6.207 | 6.270 | 0.127 |
| `- 2 3 - 5 6 - - 9` | 20 | 6.589 | 6.455 | 6.516 | 0.133 |
| `- 2 3 - 5 - 7 8 -` | 20 | 6.181 | 5.997 | 6.012 | 0.185 |
| `- 2 - 4 5 6 - 8 -` | 20 | 6.687 | 6.478 | 6.622 | 0.208 |
| `- 2 - - - 6 - 8 9` | 20 | 6.096 | 6.016 | 6.016 | 0.080 |
| `- - 3 4 5 6 7 - -` | 20 | 6.853 | 6.728 | 6.836 | 0.125 |
| `- - 3 - 5 - - 8 9` | 20 | 5.941 | 5.853 | 5.853 | 0.088 |
| `- - 3 - - 6 7 - 9` | 20 | 6.087 | 6.031 | 6.031 | 0.056 |
| `- - - 4 5 - 7 - 9` | 20 | 5.995 | 5.939 | 5.939 | 0.056 |
| `- - - 4 - 6 7 8 -` | 20 | 6.302 | 6.302 | 6.302 | 0.000 |
| `1 2 3 4 5 - - - 9` | 21 | 7.203 | 6.532 | 6.995 | 0.671 |
| `1 2 3 4 - 6 - 8 -` | 21 | 7.373 | 6.691 | 7.176 | 0.682 |
| `1 2 3 - 5 6 7 - -` | 21 | 7.325 | 6.718 | 7.152 | 0.607 |
| `1 2 - 4 - - - 8 9` | 21 | 6.890 | 6.729 | 6.830 | 0.161 |
| `1 2 - - 5 - 7 - 9` | 21 | 6.730 | 6.478 | 6.565 | 0.253 |
| `1 2 - - - 6 7 8 -` | 21 | 6.267 | 6.113 | 6.135 | 0.153 |
| `1 - 3 4 - - 7 - 9` | 21 | 6.420 | 6.249 | 6.256 | 0.171 |
| `1 - 3 - 5 6 - - 9` | 21 | 6.750 | 6.604 | 6.691 | 0.146 |
| `1 - 3 - 5 - 7 8 -` | 21 | 6.849 | 6.640 | 6.757 | 0.209 |
| `1 - - 4 5 6 - 8 -` | 21 | 6.698 | 6.517 | 6.569 | 0.181 |
| `1 - - - - 6 - 8 9` | 21 | 6.307 | 6.263 | 6.263 | 0.044 |
| `- 2 3 4 - 6 - - 9` | 21 | 6.802 | 6.648 | 6.760 | 0.153 |
| `- 2 3 4 - - 7 8 -` | 21 | 6.993 | 6.818 | 6.912 | 0.174 |
| `- 2 3 - 5 6 - 8 -` | 21 | 6.543 | 6.373 | 6.476 | 0.170 |
| `- 2 - 4 5 6 7 - -` | 21 | 6.808 | 6.665 | 6.734 | 0.143 |
| `- 2 - - 5 - - 8 9` | 21 | 6.383 | 6.383 | 6.383 | 0.000 |
| `- 2 - - - 6 7 - 9` | 21 | 6.268 | 6.228 | 6.228 | 0.040 |
| `- - 3 4 - - - 8 9` | 21 | 6.235 | 6.233 | 6.235 | 0.002 |
| `- - 3 - 5 - 7 - 9` | 21 | 6.357 | 6.353 | 6.357 | 0.004 |
| `- - 3 - - 6 7 8 -` | 21 | 6.492 | 6.492 | 6.492 | 0.000 |
| `- - - 4 5 6 - - 9` | 21 | 6.079 | 6.038 | 6.038 | 0.041 |
| `- - - 4 5 - 7 8 -` | 21 | 6.336 | 6.334 | 6.336 | 0.002 |
| `- - - - - - 7 8 9` | 21 | 6.546 | 6.546 | 6.546 | 0.000 |
| `1 2 3 4 5 - - 8 -` | 22 | 7.408 | 6.716 | 7.161 | 0.693 |
| `1 2 3 4 - 6 7 - -` | 22 | 7.454 | 6.788 | 7.280 | 0.666 |
| `1 2 3 - - - - 8 9` | 22 | 6.381 | 6.225 | 6.287 | 0.156 |
| `1 2 - 4 - - 7 - 9` | 22 | 6.964 | 6.765 | 6.874 | 0.199 |
| `1 2 - - 5 6 - - 9` | 22 | 6.793 | 6.600 | 6.671 | 0.194 |
| `1 2 - - 5 - 7 8 -` | 22 | 6.751 | 6.557 | 6.646 | 0.195 |
| `1 - 3 4 - 6 - - 9` | 22 | 6.895 | 6.704 | 6.826 | 0.191 |
| `1 - 3 4 - - 7 8 -` | 22 | 6.447 | 6.257 | 6.269 | 0.190 |
| `1 - 3 - 5 6 - 8 -` | 22 | 6.918 | 6.745 | 6.826 | 0.172 |
| `1 - - 4 5 6 7 - -` | 22 | 6.765 | 6.584 | 6.633 | 0.182 |
| `1 - - - 5 - - 8 9` | 22 | 6.178 | 6.128 | 6.128 | 0.050 |
| `1 - - - - 6 7 - 9` | 22 | 6.362 | 6.275 | 6.275 | 0.087 |
| `- 2 3 4 5 - - - 9` | 22 | 6.912 | 6.739 | 6.873 | 0.173 |
| `- 2 3 4 - 6 - 8 -` | 22 | 7.030 | 6.737 | 6.913 | 0.293 |
| `- 2 3 - 5 6 7 - -` | 22 | 6.827 | 6.632 | 6.750 | 0.195 |
| `- 2 - 4 - - - 8 9` | 22 | 6.315 | 6.315 | 6.315 | 0.000 |
| `- 2 - - 5 - 7 - 9` | 22 | 6.051 | 5.901 | 5.901 | 0.150 |
| `- 2 - - - 6 7 8 -` | 22 | 6.368 | 6.306 | 6.306 | 0.063 |
| `- - 3 4 - - 7 - 9` | 22 | 6.193 | 6.085 | 6.085 | 0.108 |
| `- - 3 - 5 6 - - 9` | 22 | 6.255 | 6.228 | 6.228 | 0.026 |
| `- - 3 - 5 - 7 8 -` | 22 | 6.273 | 6.210 | 6.210 | 0.063 |
| `- - - 4 5 6 - 8 -` | 22 | 6.490 | 6.490 | 6.490 | 0.000 |
| `- - - - - 6 - 8 9` | 22 | 6.502 | 6.502 | 6.502 | 0.000 |
| `1 2 3 4 5 - 7 - -` | 23 | 7.528 | 6.824 | 7.271 | 0.704 |
| `1 2 3 - - - 7 - 9` | 23 | 6.959 | 6.761 | 6.885 | 0.198 |
| `1 2 - 4 - 6 - - 9` | 23 | 7.035 | 6.824 | 6.903 | 0.210 |
| `1 2 - 4 - - 7 8 -` | 23 | 7.095 | 6.882 | 6.996 | 0.213 |
| `1 2 - - 5 6 - 8 -` | 23 | 6.840 | 6.625 | 6.715 | 0.216 |
| `1 - 3 4 5 - - - 9` | 23 | 6.710 | 6.472 | 6.570 | 0.238 |
| `1 - 3 4 - 6 - 8 -` | 23 | 7.014 | 6.848 | 6.913 | 0.166 |
| `1 - 3 - 5 6 7 - -` | 23 | 7.118 | 6.817 | 6.986 | 0.301 |
| `1 - - 4 - - - 8 9` | 23 | 6.117 | 6.067 | 6.067 | 0.051 |
| `1 - - - 5 - 7 - 9` | 23 | 6.621 | 6.621 | 6.621 | 0.000 |
| `1 - - - - 6 7 8 -` | 23 | 6.220 | 6.066 | 6.066 | 0.154 |
| `- 2 3 4 5 - - 8 -` | 23 | 7.104 | 6.847 | 7.006 | 0.256 |
| `- 2 3 4 - 6 7 - -` | 23 | 7.131 | 6.945 | 7.040 | 0.186 |
| `- 2 3 - - - - 8 9` | 23 | 6.200 | 6.197 | 6.200 | 0.004 |
| `- 2 - 4 - - 7 - 9` | 23 | 6.161 | 6.128 | 6.135 | 0.034 |
| `- 2 - - 5 6 - - 9` | 23 | 6.559 | 6.552 | 6.559 | 0.008 |
| `- 2 - - 5 - 7 8 -` | 23 | 6.418 | 6.331 | 6.331 | 0.088 |
| `- - 3 4 - 6 - - 9` | 23 | 6.339 | 6.327 | 6.327 | 0.012 |
| `- - 3 4 - - 7 8 -` | 23 | 6.283 | 6.191 | 6.195 | 0.092 |
| `- - 3 - 5 6 - 8 -` | 23 | 6.227 | 6.178 | 6.182 | 0.049 |
| `- - - 4 5 6 7 - -` | 23 | 6.610 | 6.610 | 6.610 | 0.000 |
| `- - - - 5 - - 8 9` | 23 | 6.458 | 6.458 | 6.458 | 0.000 |
| `- - - - - 6 7 - 9` | 23 | 6.546 | 6.546 | 6.546 | 0.000 |
| `1 2 3 4 5 6 - - -` | 24 | 7.594 | 7.044 | 7.342 | 0.549 |
| `1 2 3 - - 6 - - 9` | 24 | 7.016 | 6.753 | 6.854 | 0.263 |
| `1 2 3 - - - 7 8 -` | 24 | 7.136 | 6.913 | 7.038 | 0.222 |
| `1 2 - 4 5 - - - 9` | 24 | 7.161 | 6.859 | 6.995 | 0.302 |
| `1 2 - 4 - 6 - 8 -` | 24 | 7.229 | 6.882 | 7.068 | 0.348 |
| `1 2 - - 5 6 7 - -` | 24 | 6.900 | 6.686 | 6.764 | 0.214 |
| `1 - 3 4 5 - - 8 -` | 24 | 6.971 | 6.771 | 6.890 | 0.200 |
| `1 - 3 4 - 6 7 - -` | 24 | 7.085 | 6.855 | 6.978 | 0.230 |
| `1 - 3 - - - - 8 9` | 24 | 6.157 | 6.088 | 6.088 | 0.069 |
| `1 - - 4 - - 7 - 9` | 24 | 6.636 | 6.636 | 6.636 | 0.000 |
| `1 - - - 5 6 - - 9` | 24 | 6.534 | 6.451 | 6.451 | 0.083 |
| `1 - - - 5 - 7 8 -` | 24 | 6.481 | 6.437 | 6.437 | 0.044 |
| `- 2 3 4 5 - 7 - -` | 24 | 7.186 | 6.951 | 7.119 | 0.236 |
| `- 2 3 - - - 7 - 9` | 24 | 6.325 | 6.282 | 6.282 | 0.043 |
| `- 2 - 4 - 6 - - 9` | 24 | 6.448 | 6.356 | 6.356 | 0.092 |
| `- 2 - 4 - - 7 8 -` | 24 | 6.777 | 6.777 | 6.777 | 0.000 |
| `- 2 - - 5 6 - 8 -` | 24 | 6.533 | 6.507 | 6.507 | 0.026 |
| `- - 3 4 5 - - - 9` | 24 | 6.445 | 6.413 | 6.422 | 0.032 |
| `- - 3 4 - 6 - 8 -` | 24 | 6.850 | 6.850 | 6.850 | 0.000 |
| `- - 3 - 5 6 7 - -` | 24 | 6.929 | 6.929 | 6.929 | 0.000 |
| `- - - 4 - - - 8 9` | 24 | 6.478 | 6.478 | 6.478 | 0.000 |
| `- - - - 5 - 7 - 9` | 24 | 6.565 | 6.565 | 6.565 | 0.000 |
| `- - - - - 6 7 8 -` | 24 | 6.595 | 6.595 | 6.595 | 0.000 |
| `1 2 3 - 5 - - - 9` | 25 | 7.137 | 6.815 | 6.857 | 0.322 |
| `1 2 3 - - 6 - 8 -` | 25 | 7.194 | 6.930 | 7.045 | 0.264 |
| `1 2 - 4 5 - - 8 -` | 25 | 7.292 | 6.981 | 7.131 | 0.311 |
| `1 2 - 4 - 6 7 - -` | 25 | 7.278 | 6.981 | 7.144 | 0.297 |
| `1 2 - - - - - 8 9` | 25 | 6.105 | 6.013 | 6.018 | 0.092 |
| `1 - 3 4 5 - 7 - -` | 25 | 7.180 | 6.889 | 6.988 | 0.290 |
| `1 - 3 - - - 7 - 9` | 25 | 6.547 | 6.536 | 6.547 | 0.011 |
| `1 - - 4 - 6 - - 9` | 25 | 6.629 | 6.618 | 6.629 | 0.011 |
| `1 - - 4 - - 7 8 -` | 25 | 6.515 | 6.455 | 6.455 | 0.060 |
| `1 - - - 5 6 - 8 -` | 25 | 6.686 | 6.610 | 6.610 | 0.076 |
| `- 2 3 4 5 6 - - -` | 25 | 7.367 | 7.127 | 7.305 | 0.240 |
| `- 2 3 - - 6 - - 9` | 25 | 6.427 | 6.383 | 6.383 | 0.044 |
| `- 2 3 - - - 7 8 -` | 25 | 6.684 | 6.678 | 6.684 | 0.006 |
| `- 2 - 4 5 - - - 9` | 25 | 6.496 | 6.445 | 6.456 | 0.050 |
| `- 2 - 4 - 6 - 8 -` | 25 | 6.119 | 5.967 | 5.973 | 0.151 |
| `- 2 - - 5 6 7 - -` | 25 | 6.719 | 6.673 | 6.673 | 0.047 |
| `- - 3 4 5 - - 8 -` | 25 | 6.629 | 6.599 | 6.602 | 0.029 |
| `- - 3 4 - 6 7 - -` | 25 | 6.512 | 6.465 | 6.465 | 0.048 |
| `- - 3 - - - - 8 9` | 25 | 6.566 | 6.566 | 6.566 | 0.000 |
| `- - - 4 - - 7 - 9` | 25 | 6.583 | 6.583 | 6.583 | 0.000 |
| `- - - - 5 6 - - 9` | 25 | 6.587 | 6.587 | 6.587 | 0.000 |
| `- - - - 5 - 7 8 -` | 25 | 6.613 | 6.613 | 6.613 | 0.000 |
| `1 2 3 4 - - - - 9` | 26 | 7.235 | 6.911 | 6.936 | 0.324 |
| `1 2 3 - 5 - - 8 -` | 26 | 7.257 | 6.945 | 7.058 | 0.312 |
| `1 2 3 - - 6 7 - -` | 26 | 7.334 | 7.098 | 7.195 | 0.237 |
| `1 2 - 4 5 - 7 - -` | 26 | 7.362 | 7.102 | 7.258 | 0.260 |
| `1 2 - - - - 7 - 9` | 26 | 6.503 | 6.427 | 6.427 | 0.076 |
| `1 - 3 4 5 6 - - -` | 26 | 7.293 | 6.999 | 7.131 | 0.294 |
| `1 - 3 - - 6 - - 9` | 26 | 6.471 | 6.411 | 6.411 | 0.060 |
| `1 - 3 - - - 7 8 -` | 26 | 6.651 | 6.574 | 6.574 | 0.077 |
| `1 - - 4 5 - - - 9` | 26 | 6.216 | 6.067 | 6.067 | 0.150 |
| `1 - - 4 - 6 - 8 -` | 26 | 7.076 | 7.076 | 7.076 | 0.000 |
| `1 - - - 5 6 7 - -` | 26 | 6.484 | 6.331 | 6.331 | 0.153 |
| `- 2 3 - 5 - - - 9` | 26 | 6.796 | 6.720 | 6.720 | 0.076 |
| `- 2 3 - - 6 - 8 -` | 26 | 6.614 | 6.556 | 6.556 | 0.058 |
| `- 2 - 4 5 - - 8 -` | 26 | 7.149 | 7.149 | 7.149 | 0.000 |
| `- 2 - 4 - 6 7 - -` | 26 | 6.905 | 6.844 | 6.844 | 0.061 |
| `- 2 - - - - - 8 9` | 26 | 6.654 | 6.654 | 6.654 | 0.000 |
| `- - 3 4 5 - 7 - -` | 26 | 6.901 | 6.861 | 6.861 | 0.040 |
| `- - 3 - - - 7 - 9` | 26 | 6.671 | 6.671 | 6.671 | 0.000 |
| `- - - 4 - 6 - - 9` | 26 | 6.608 | 6.608 | 6.608 | 0.000 |
| `- - - 4 - - 7 8 -` | 26 | 6.701 | 6.701 | 6.701 | 0.000 |
| `- - - - 5 6 - 8 -` | 26 | 6.636 | 6.636 | 6.636 | 0.000 |
| `1 2 3 4 - - - 8 -` | 27 | 7.373 | 7.024 | 7.068 | 0.349 |
| `1 2 3 - 5 - 7 - -` | 27 | 7.452 | 7.112 | 7.215 | 0.340 |
| `1 2 - 4 5 6 - - -` | 27 | 7.448 | 7.123 | 7.268 | 0.325 |
| `1 2 - - - 6 - - 9` | 27 | 6.874 | 6.780 | 6.780 | 0.094 |
| `1 2 - - - - 7 8 -` | 27 | 6.524 | 6.423 | 6.431 | 0.101 |
| `1 - 3 - 5 - - - 9` | 27 | 6.708 | 6.616 | 6.616 | 0.092 |
| `1 - 3 - - 6 - 8 -` | 27 | 7.015 | 7.007 | 7.015 | 0.008 |
| `1 - - 4 5 - - 8 -` | 27 | 6.621 | 6.549 | 6.557 | 0.072 |
| `1 - - 4 - 6 7 - -` | 27 | 6.903 | 6.827 | 6.827 | 0.076 |
| `1 - - - - - - 8 9` | 27 | 6.590 | 6.507 | 6.507 | 0.083 |
| `- 2 3 4 - - - - 9` | 27 | 6.763 | 6.686 | 6.686 | 0.077 |
| `- 2 3 - 5 - - 8 -` | 27 | 6.517 | 6.378 | 6.378 | 0.139 |
| `- 2 3 - - 6 7 - -` | 27 | 7.154 | 7.154 | 7.154 | 0.001 |
| `- 2 - 4 5 - 7 - -` | 27 | 6.674 | 6.619 | 6.620 | 0.055 |
| `- 2 - - - - 7 - 9` | 27 | 6.601 | 6.525 | 6.525 | 0.076 |
| `- - 3 4 5 6 - - -` | 27 | 7.281 | 7.281 | 7.281 | 0.000 |
| `- - 3 - - 6 - - 9` | 27 | 6.537 | 6.461 | 6.461 | 0.075 |
| `- - 3 - - - 7 8 -` | 27 | 6.789 | 6.789 | 6.789 | 0.000 |
| `- - - 4 5 - - - 9` | 27 | 6.473 | 6.398 | 6.398 | 0.075 |
| `- - - 4 - 6 - 8 -` | 27 | 6.724 | 6.724 | 6.724 | 0.000 |
| `- - - - 5 6 7 - -` | 27 | 6.752 | 6.752 | 6.752 | 0.000 |
| `1 2 3 4 - - 7 - -` | 28 | 7.480 | 7.113 | 7.222 | 0.367 |
| `1 2 3 - 5 6 - - -` | 28 | 7.496 | 7.184 | 7.268 | 0.311 |
| `1 2 - - 5 - - - 9` | 28 | 7.164 | 7.164 | 7.164 | 0.000 |
| `1 2 - - - 6 - 8 -` | 28 | 6.891 | 6.802 | 6.802 | 0.089 |
| `1 - 3 4 - - - - 9` | 28 | 6.855 | 6.796 | 6.796 | 0.058 |
| `1 - 3 - 5 - - 8 -` | 28 | 6.718 | 6.631 | 6.631 | 0.086 |
| `1 - 3 - - 6 7 - -` | 28 | 7.038 | 6.946 | 6.946 | 0.091 |
| `1 - - 4 5 - 7 - -` | 28 | 7.210 | 7.139 | 7.139 | 0.072 |
| `1 - - - - - 7 - 9` | 28 | 6.847 | 6.847 | 6.847 | 0.000 |
| `- 2 3 4 - - - 8 -` | 28 | 7.365 | 7.365 | 7.365 | 0.000 |
| `- 2 3 - 5 - 7 - -` | 28 | 6.783 | 6.618 | 6.618 | 0.165 |
| `- 2 - 4 5 6 - - -` | 28 | 7.271 | 7.212 | 7.212 | 0.059 |
| `- 2 - - - 6 - - 9` | 28 | 6.783 | 6.783 | 6.783 | 0.000 |
| `- 2 - - - - 7 8 -` | 28 | 6.877 | 6.877 | 6.877 | 0.000 |
| `- - 3 - 5 - - - 9` | 28 | 6.720 | 6.720 | 6.720 | 0.000 |
| `- - 3 - - 6 - 8 -` | 28 | 6.812 | 6.812 | 6.812 | 0.000 |
| `- - - 4 5 - - 8 -` | 28 | 6.747 | 6.747 | 6.747 | 0.000 |
| `- - - 4 - 6 7 - -` | 28 | 6.840 | 6.840 | 6.840 | 0.000 |
| `- - - - - - - 8 9` | 28 | 7.281 | 7.281 | 7.281 | 0.000 |
| `1 2 3 4 - 6 - - -` | 29 | 7.552 | 7.177 | 7.275 | 0.376 |
| `1 2 - 4 - - - - 9` | 29 | 7.055 | 7.055 | 7.055 | 0.000 |
| `1 2 - - 5 - - 8 -` | 29 | 7.189 | 7.074 | 7.074 | 0.115 |
| `1 2 - - - 6 7 - -` | 29 | 6.833 | 6.725 | 6.726 | 0.108 |
| `1 - 3 4 - - - 8 -` | 29 | 6.870 | 6.671 | 6.671 | 0.199 |
| `1 - 3 - 5 - 7 - -` | 29 | 7.384 | 7.383 | 7.384 | 0.002 |
| `1 - - 4 5 6 - - -` | 29 | 6.969 | 6.780 | 6.780 | 0.189 |
| `1 - - - - 6 - - 9` | 29 | 6.871 | 6.871 | 6.871 | 0.000 |
| `1 - - - - - 7 8 -` | 29 | 6.765 | 6.668 | 6.668 | 0.096 |
| `- 2 3 4 - - 7 - -` | 29 | 7.105 | 7.024 | 7.024 | 0.081 |
| `- 2 3 - 5 6 - - -` | 29 | 7.210 | 7.133 | 7.134 | 0.077 |
| `- 2 - - 5 - - - 9` | 29 | 6.808 | 6.808 | 6.808 | 0.000 |
| `- 2 - - - 6 - 8 -` | 29 | 6.698 | 6.603 | 6.603 | 0.095 |
| `- - 3 4 - - - - 9` | 29 | 6.745 | 6.745 | 6.745 | 0.000 |
| `- - 3 - 5 - - 8 -` | 29 | 6.632 | 6.538 | 6.538 | 0.095 |
| `- - 3 - - 6 7 - -` | 29 | 6.928 | 6.928 | 6.928 | 0.000 |
| `- - - 4 5 - 7 - -` | 29 | 6.924 | 6.924 | 6.924 | 0.000 |
| `- - - - - - 7 - 9` | 29 | 7.315 | 7.315 | 7.315 | 0.000 |
| `1 2 3 4 5 - - - -` | 30 | 7.616 | 7.186 | 7.258 | 0.430 |
| `1 2 3 - - - - - 9` | 30 | 6.660 | 6.596 | 6.596 | 0.064 |
| `1 2 - 4 - - - 8 -` | 30 | 7.447 | 7.447 | 7.447 | 0.000 |
| `1 2 - - 5 - 7 - -` | 30 | 7.190 | 7.073 | 7.073 | 0.117 |
| `1 - 3 4 - - 7 - -` | 30 | 6.877 | 6.660 | 6.660 | 0.217 |
| `1 - 3 - 5 6 - - -` | 30 | 7.467 | 7.358 | 7.358 | 0.109 |
| `1 - - - 5 - - - 9` | 30 | 6.765 | 6.765 | 6.765 | 0.000 |
| `1 - - - - 6 - 8 -` | 30 | 6.988 | 6.988 | 6.988 | 0.000 |
| `- 2 3 4 - 6 - - -` | 30 | 7.512 | 7.422 | 7.422 | 0.091 |
| `- 2 - 4 - - - - 9` | 30 | 6.702 | 6.702 | 6.702 | 0.000 |
| `- 2 - - 5 - - 8 -` | 30 | 6.922 | 6.922 | 6.922 | 0.000 |
| `- 2 - - - 6 7 - -` | 30 | 7.016 | 7.016 | 7.016 | 0.000 |
| `- - 3 4 - - - 8 -` | 30 | 6.917 | 6.917 | 6.917 | 0.000 |
| `- - 3 - 5 - 7 - -` | 30 | 7.009 | 7.009 | 7.009 | 0.000 |
| `- - - 4 5 6 - - -` | 30 | 7.002 | 7.002 | 7.002 | 0.000 |
| `- - - - - 6 - - 9` | 30 | 7.281 | 7.281 | 7.281 | 0.000 |
| `- - - - - - 7 8 -` | 30 | 7.352 | 7.352 | 7.352 | 0.000 |
| `1 2 3 - - - - 8 -` | 31 | 7.041 | 6.958 | 6.958 | 0.083 |
| `1 2 - 4 - - 7 - -` | 31 | 7.551 | 7.398 | 7.398 | 0.153 |
| `1 2 - - 5 6 - - -` | 31 | 7.179 | 7.051 | 7.052 | 0.127 |
| `1 - 3 4 - 6 - - -` | 31 | 7.436 | 7.339 | 7.340 | 0.097 |
| `1 - - 4 - - - - 9` | 31 | 6.660 | 6.660 | 6.660 | 0.000 |
| `1 - - - 5 - - 8 -` | 31 | 6.876 | 6.876 | 6.876 | 0.000 |
| `1 - - - - 6 7 - -` | 31 | 6.859 | 6.743 | 6.743 | 0.116 |
| `- 2 3 4 5 - - - -` | 31 | 7.595 | 7.496 | 7.496 | 0.099 |
| `- 2 3 - - - - - 9` | 31 | 6.655 | 6.655 | 6.655 | 0.000 |
| `- 2 - 4 - - - 8 -` | 31 | 6.868 | 6.868 | 6.868 | 0.000 |
| `- 2 - - 5 - 7 - -` | 31 | 6.849 | 6.734 | 6.734 | 0.115 |
| `- - 3 4 - - 7 - -` | 31 | 6.843 | 6.729 | 6.729 | 0.114 |
| `- - 3 - 5 6 - - -` | 31 | 7.085 | 7.085 | 7.085 | 0.000 |
| `- - - - 5 - - - 9` | 31 | 7.247 | 7.247 | 7.247 | 0.000 |
| `- - - - - 6 - 8 -` | 31 | 7.316 | 7.316 | 7.316 | 0.000 |
| `1 2 3 - - - 7 - -` | 32 | 7.505 | 7.403 | 7.403 | 0.102 |
| `1 2 - 4 - 6 - - -` | 32 | 7.616 | 7.461 | 7.461 | 0.155 |
| `1 - 3 4 5 - - - -` | 32 | 7.449 | 7.190 | 7.190 | 0.259 |
| `1 - 3 - - - - - 9` | 32 | 6.610 | 6.610 | 6.610 | 0.000 |
| `1 - - 4 - - - 8 -` | 32 | 6.819 | 6.819 | 6.819 | 0.000 |
| `1 - - - 5 - 7 - -` | 32 | 7.042 | 7.042 | 7.042 | 0.000 |
| `- 2 3 - - - - 8 -` | 32 | 6.815 | 6.815 | 6.815 | 0.000 |
| `- 2 - 4 - - 7 - -` | 32 | 7.035 | 7.035 | 7.035 | 0.000 |
| `- 2 - - 5 6 - - -` | 32 | 7.168 | 7.168 | 7.168 | 0.000 |
| `- - 3 4 - 6 - - -` | 32 | 7.163 | 7.163 | 7.163 | 0.000 |
| `- - - 4 - - - - 9` | 32 | 7.213 | 7.213 | 7.213 | 0.000 |
| `- - - - 5 - - 8 -` | 32 | 7.281 | 7.281 | 7.281 | 0.000 |
| `- - - - - 6 7 - -` | 32 | 7.352 | 7.352 | 7.352 | 0.000 |
| `1 2 3 - - 6 - - -` | 33 | 7.675 | 7.393 | 7.393 | 0.282 |
| `1 2 - 4 5 - - - -` | 33 | 7.709 | 7.546 | 7.547 | 0.163 |
| `1 2 - - - - - - 9` | 33 | 6.643 | 6.643 | 6.643 | 0.000 |
| `1 - 3 - - - - 8 -` | 33 | 6.846 | 6.846 | 6.846 | 0.000 |
| `1 - - 4 - - 7 - -` | 33 | 7.063 | 7.063 | 7.063 | 0.000 |
| `1 - - - 5 6 - - -` | 33 | 7.007 | 6.903 | 6.903 | 0.104 |
| `- 2 3 - - - 7 - -` | 33 | 7.058 | 7.058 | 7.058 | 0.000 |
| `- 2 - 4 - 6 - - -` | 33 | 6.999 | 6.895 | 6.895 | 0.104 |
| `- - 3 4 5 - - - -` | 33 | 7.319 | 7.319 | 7.319 | 0.000 |
| `- - 3 - - - - - 9` | 33 | 7.235 | 7.235 | 7.235 | 0.000 |
| `- - - 4 - - - 8 -` | 33 | 7.301 | 7.301 | 7.301 | 0.000 |
| `- - - - 5 - 7 - -` | 33 | 7.370 | 7.370 | 7.370 | 0.000 |
| `1 2 3 - 5 - - - -` | 34 | 7.707 | 7.407 | 7.407 | 0.300 |
| `1 2 - - - - - 8 -` | 34 | 6.873 | 6.873 | 6.873 | 0.000 |
| `1 - 3 - - - 7 - -` | 34 | 7.083 | 7.083 | 7.083 | 0.000 |
| `1 - - 4 - 6 - - -` | 34 | 7.222 | 7.222 | 7.222 | 0.000 |
| `- 2 3 - - 6 - - -` | 34 | 7.217 | 7.217 | 7.217 | 0.000 |
| `- 2 - 4 5 - - - -` | 34 | 7.350 | 7.350 | 7.350 | 0.000 |
| `- 2 - - - - - - 9` | 34 | 7.256 | 7.256 | 7.256 | 0.000 |
| `- - 3 - - - - 8 -` | 34 | 7.321 | 7.321 | 7.321 | 0.000 |
| `- - - 4 - - 7 - -` | 34 | 7.389 | 7.389 | 7.389 | 0.000 |
| `- - - - 5 6 - - -` | 34 | 7.392 | 7.392 | 7.392 | 0.000 |
| `1 2 3 4 - - - - -` | 35 | 7.680 | 7.352 | 7.352 | 0.328 |
| `1 2 - - - - 7 - -` | 35 | 7.104 | 7.104 | 7.104 | 0.000 |
| `1 - 3 - - 6 - - -` | 35 | 7.245 | 7.245 | 7.245 | 0.000 |
| `1 - - 4 5 - - - -` | 35 | 7.112 | 7.022 | 7.022 | 0.090 |
| `1 - - - - - - - 9` | 35 | 7.278 | 7.278 | 7.278 | 0.000 |
| `- 2 3 - 5 - - - -` | 35 | 7.231 | 7.142 | 7.142 | 0.089 |
| `- 2 - - - - - 8 -` | 35 | 7.341 | 7.341 | 7.341 | 0.000 |
| `- - 3 - - - 7 - -` | 35 | 7.407 | 7.407 | 7.407 | 0.000 |
| `- - - 4 - 6 - - -` | 35 | 7.412 | 7.412 | 7.412 | 0.000 |
| `1 2 - - - 6 - - -` | 36 | 7.267 | 7.267 | 7.267 | 0.000 |
| `1 - 3 - 5 - - - -` | 36 | 7.285 | 7.285 | 7.285 | 0.000 |
| `1 - - - - - - 8 -` | 36 | 7.361 | 7.361 | 7.361 | 0.000 |
| `- 2 3 4 - - - - -` | 36 | 7.416 | 7.416 | 7.416 | 0.000 |
| `- 2 - - - - 7 - -` | 36 | 7.426 | 7.426 | 7.426 | 0.000 |
| `- - 3 - - 6 - - -` | 36 | 7.432 | 7.432 | 7.432 | 0.000 |
| `- - - 4 5 - - - -` | 36 | 7.435 | 7.435 | 7.435 | 0.000 |
| `- - - - - - - - 9` | 36 | 8.111 | 8.111 | 8.111 | 0.000 |
| `1 2 - - 5 - - - -` | 37 | 7.314 | 7.314 | 7.314 | 0.000 |
| `1 - 3 4 - - - - -` | 37 | 7.225 | 7.153 | 7.153 | 0.072 |
| `1 - - - - - 7 - -` | 37 | 7.444 | 7.444 | 7.444 | 0.000 |
| `- 2 - - - 6 - - -` | 37 | 7.452 | 7.452 | 7.452 | 0.000 |
| `- - 3 - 5 - - - -` | 37 | 7.457 | 7.457 | 7.457 | 0.000 |
| `- - - - - - - 8 -` | 37 | 8.139 | 8.139 | 8.139 | 0.000 |
| `1 2 - 4 - - - - -` | 38 | 7.250 | 7.250 | 7.250 | 0.000 |
| `1 - - - - 6 - - -` | 38 | 7.472 | 7.472 | 7.472 | 0.000 |
| `- 2 - - 5 - - - -` | 38 | 7.478 | 7.478 | 7.478 | 0.000 |
| `- - 3 4 - - - - -` | 38 | 7.481 | 7.481 | 7.481 | 0.000 |
| `- - - - - - 7 - -` | 38 | 8.167 | 8.167 | 8.167 | 0.000 |
| `1 2 3 - - - - - -` | 39 | 6.956 | 6.905 | 6.905 | 0.051 |
| `1 - - - 5 - - - -` | 39 | 7.389 | 7.389 | 7.389 | 0.000 |
| `- 2 - 4 - - - - -` | 39 | 7.394 | 7.394 | 7.394 | 0.000 |
| `- - - - - 6 - - -` | 39 | 8.139 | 8.139 | 8.139 | 0.000 |
| `1 - - 4 - - - - -` | 40 | 7.306 | 7.306 | 7.306 | 0.000 |
| `- 2 3 - - - - - -` | 40 | 7.309 | 7.309 | 7.309 | 0.000 |
| `- - - - 5 - - - -` | 40 | 8.111 | 8.111 | 8.111 | 0.000 |
| `1 - 3 - - - - - -` | 41 | 7.222 | 7.222 | 7.222 | 0.000 |
| `- - - 4 - - - - -` | 41 | 8.083 | 8.083 | 8.083 | 0.000 |
| `1 2 - - - - - - -` | 42 | 7.139 | 7.139 | 7.139 | 0.000 |
| `- - 3 - - - - - -` | 42 | 8.056 | 8.056 | 8.056 | 0.000 |
| `- 2 - - - - - - -` | 43 | 8.028 | 8.028 | 8.028 | 0.000 |
| `1 - - - - - - - -` | 44 | 8.000 | 8.000 | 8.000 | 0.000 |
| `- - - - - - - - -` | 45 | 9.000 | 9.000 | 9.000 | 0.000 |


### Exhibit D: Optimal Moves for Traditional Scoring System

Note: Only showing results where there is a difference between the optimal strategy and the worst strategy

In [13]:
def exhibit_d(results: dict) -> Table:
    """returns a table for optimal moves of the traditional scoring system"""
    exhibit_table = []
    results_subtable = results['traditional_move_table']
    for record in results_subtable:
        if record['Max Move'] == record['Min Move']:
            continue
        exhibit_table.append({
            'Board (Remaining)': f'`{record["Board"]}`',
            'Board Sum': record['Sum'],
            'Roll': record['Roll'],
            'Max Move': record['Max Move'],
            'Min Move': record['Min Move'],
            'Highest Tile Move': record['Highest Tile Move'],
        })
    # sort by board sum ascending
    exhibit_table.sort(key=lambda record: (record['Board Sum'], record['Board (Remaining)'], record['Roll']))
    return exhibit_table


display_table(exhibit_d(full_results))

| Board (Remaining) | Board Sum | Roll | Max Move | Min Move | Highest Tile Move |
| --- | --- | --- | --- | --- | --- |
| `1 2 3 4 5 6 7 8 9` | 0 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 4 5 6 7 8 9` | 0 | 4 | [4] | [1, 3] | [4] |
| `1 2 3 4 5 6 7 8 9` | 0 | 5 | [5] | [2, 3] | [5] |
| `1 2 3 4 5 6 7 8 9` | 0 | 6 | [6] | [1, 2, 3] | [6] |
| `1 2 3 4 5 6 7 8 9` | 0 | 7 | [7] | [1, 2, 4] | [7] |
| `1 2 3 4 5 6 7 8 9` | 0 | 8 | [8] | [1, 3, 4] | [8] |
| `1 2 3 4 5 6 7 8 9` | 0 | 9 | [9] | [2, 3, 4] | [9] |
| `1 2 3 4 5 6 7 8 9` | 0 | 10 | [1, 9] | [1, 2, 3, 4] | [1, 9] |
| `1 2 3 4 5 6 7 8 9` | 0 | 11 | [2, 9] | [1, 4, 6] | [2, 9] |
| `1 2 3 4 5 6 7 8 9` | 0 | 12 | [3, 9] | [1, 2, 3, 6] | [3, 9] |
| `- 2 3 4 5 6 7 8 9` | 1 | 5 | [5] | [2, 3] | [5] |
| `- 2 3 4 5 6 7 8 9` | 1 | 6 | [6] | [2, 4] | [6] |
| `- 2 3 4 5 6 7 8 9` | 1 | 7 | [7] | [3, 4] | [7] |
| `- 2 3 4 5 6 7 8 9` | 1 | 8 | [8] | [2, 6] | [8] |
| `- 2 3 4 5 6 7 8 9` | 1 | 9 | [9] | [2, 3, 4] | [9] |
| `- 2 3 4 5 6 7 8 9` | 1 | 10 | [2, 8] | [4, 6] | [2, 8] |
| `- 2 3 4 5 6 7 8 9` | 1 | 11 | [2, 9] | [2, 3, 6] | [2, 9] |
| `- 2 3 4 5 6 7 8 9` | 1 | 12 | [3, 9] | [2, 3, 7] | [3, 9] |
| `1 - 3 4 5 6 7 8 9` | 2 | 4 | [4] | [1, 3] | [4] |
| `1 - 3 4 5 6 7 8 9` | 2 | 5 | [5] | [1, 4] | [5] |
| `1 - 3 4 5 6 7 8 9` | 2 | 6 | [6] | [1, 5] | [6] |
| `1 - 3 4 5 6 7 8 9` | 2 | 7 | [7] | [3, 4] | [7] |
| `1 - 3 4 5 6 7 8 9` | 2 | 8 | [8] | [1, 3, 4] | [8] |
| `1 - 3 4 5 6 7 8 9` | 2 | 9 | [9] | [1, 3, 5] | [9] |
| `1 - 3 4 5 6 7 8 9` | 2 | 10 | [1, 9] | [1, 3, 6] | [1, 9] |
| `1 - 3 4 5 6 7 8 9` | 2 | 11 | [3, 8] | [1, 3, 7] | [3, 8] |
| `1 - 3 4 5 6 7 8 9` | 2 | 12 | [4, 8] | [1, 4, 7] | [3, 9] |
| `- - 3 4 5 6 7 8 9` | 3 | 7 | [7] | [3, 4] | [7] |
| `- - 3 4 5 6 7 8 9` | 3 | 8 | [8] | [3, 5] | [8] |
| `- - 3 4 5 6 7 8 9` | 3 | 9 | [9] | [3, 6] | [9] |
| `- - 3 4 5 6 7 8 9` | 3 | 10 | [4, 6] | [3, 7] | [3, 7] |
| `- - 3 4 5 6 7 8 9` | 3 | 11 | [3, 8] | [4, 7] | [3, 8] |
| `- - 3 4 5 6 7 8 9` | 3 | 12 | [3, 9] | [3, 4, 5] | [3, 9] |
| `1 2 - 4 5 6 7 8 9` | 3 | 5 | [5] | [1, 4] | [5] |
| `1 2 - 4 5 6 7 8 9` | 3 | 6 | [6] | [2, 4] | [6] |
| `1 2 - 4 5 6 7 8 9` | 3 | 7 | [7] | [1, 2, 4] | [7] |
| `1 2 - 4 5 6 7 8 9` | 3 | 8 | [8] | [1, 2, 5] | [8] |
| `1 2 - 4 5 6 7 8 9` | 3 | 9 | [9] | [1, 2, 6] | [9] |
| `1 2 - 4 5 6 7 8 9` | 3 | 10 | [1, 9] | [1, 2, 7] | [1, 9] |
| `1 2 - 4 5 6 7 8 9` | 3 | 11 | [2, 9] | [2, 4, 5] | [2, 9] |
| `1 2 - 4 5 6 7 8 9` | 3 | 12 | [5, 7] | [1, 2, 4, 5] | [4, 8] |
| `- 2 - 4 5 6 7 8 9` | 4 | 6 | [6] | [2, 4] | [6] |
| `- 2 - 4 5 6 7 8 9` | 4 | 7 | [7] | [2, 5] | [7] |
| `- 2 - 4 5 6 7 8 9` | 4 | 8 | [8] | [2, 6] | [8] |
| `- 2 - 4 5 6 7 8 9` | 4 | 9 | [9] | [2, 7] | [9] |
| `- 2 - 4 5 6 7 8 9` | 4 | 10 | [2, 8] | [4, 6] | [2, 8] |
| `- 2 - 4 5 6 7 8 9` | 4 | 11 | [2, 9] | [2, 4, 5] | [2, 9] |
| `- 2 - 4 5 6 7 8 9` | 4 | 12 | [4, 8] | [2, 4, 6] | [4, 8] |
| `1 2 3 - 5 6 7 8 9` | 4 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 - 5 6 7 8 9` | 4 | 5 | [5] | [2, 3] | [5] |
| `1 2 3 - 5 6 7 8 9` | 4 | 6 | [6] | [1, 2, 3] | [6] |
| `1 2 3 - 5 6 7 8 9` | 4 | 7 | [7] | [1, 6] | [7] |
| `1 2 3 - 5 6 7 8 9` | 4 | 8 | [8] | [1, 2, 5] | [8] |
| `1 2 3 - 5 6 7 8 9` | 4 | 9 | [9] | [1, 3, 5] | [9] |
| `1 2 3 - 5 6 7 8 9` | 4 | 10 | [2, 8] | [1, 2, 7] | [1, 9] |
| `1 2 3 - 5 6 7 8 9` | 4 | 11 | [2, 9] | [1, 2, 3, 5] | [2, 9] |
| `1 2 3 - 5 6 7 8 9` | 4 | 12 | [3, 9] | [1, 2, 3, 6] | [3, 9] |
| `- 2 3 - 5 6 7 8 9` | 5 | 5 | [5] | [2, 3] | [5] |
| `- 2 3 - 5 6 7 8 9` | 5 | 7 | [7] | [2, 5] | [7] |
| `- 2 3 - 5 6 7 8 9` | 5 | 8 | [8] | [3, 5] | [8] |
| `- 2 3 - 5 6 7 8 9` | 5 | 9 | [9] | [2, 7] | [9] |
| `- 2 3 - 5 6 7 8 9` | 5 | 10 | [2, 8] | [2, 3, 5] | [2, 8] |
| `- 2 3 - 5 6 7 8 9` | 5 | 11 | [2, 9] | [2, 3, 6] | [2, 9] |
| `- 2 3 - 5 6 7 8 9` | 5 | 12 | [3, 9] | [2, 3, 7] | [3, 9] |
| `1 - - 4 5 6 7 8 9` | 5 | 5 | [5] | [1, 4] | [5] |
| `1 - - 4 5 6 7 8 9` | 5 | 6 | [6] | [1, 5] | [6] |
| `1 - - 4 5 6 7 8 9` | 5 | 7 | [7] | [1, 6] | [7] |
| `1 - - 4 5 6 7 8 9` | 5 | 8 | [8] | [1, 7] | [8] |
| `1 - - 4 5 6 7 8 9` | 5 | 9 | [9] | [4, 5] | [9] |
| `1 - - 4 5 6 7 8 9` | 5 | 10 | [1, 9] | [1, 4, 5] | [1, 9] |
| `1 - - 4 5 6 7 8 9` | 5 | 11 | [4, 7] | [1, 4, 6] | [4, 7] |
| `1 - - 4 5 6 7 8 9` | 5 | 12 | [5, 7] | [1, 4, 7] | [4, 8] |
| `1 2 3 4 - 6 7 8 9` | 5 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 4 - 6 7 8 9` | 5 | 4 | [4] | [1, 3] | [4] |
| `1 2 3 4 - 6 7 8 9` | 5 | 6 | [6] | [1, 2, 3] | [6] |
| `1 2 3 4 - 6 7 8 9` | 5 | 7 | [7] | [1, 2, 4] | [7] |
| `1 2 3 4 - 6 7 8 9` | 5 | 8 | [8] | [1, 3, 4] | [8] |
| `1 2 3 4 - 6 7 8 9` | 5 | 9 | [9] | [2, 3, 4] | [9] |
| `1 2 3 4 - 6 7 8 9` | 5 | 10 | [1, 9] | [1, 2, 3, 4] | [1, 9] |
| `1 2 3 4 - 6 7 8 9` | 5 | 11 | [2, 9] | [1, 3, 7] | [2, 9] |
| `1 2 3 4 - 6 7 8 9` | 5 | 12 | [3, 9] | [1, 2, 3, 6] | [3, 9] |
| `- - - 4 5 6 7 8 9` | 6 | 9 | [9] | [4, 5] | [9] |
| `- - - 4 5 6 7 8 9` | 6 | 11 | [5, 6] | [4, 7] | [4, 7] |
| `- - - 4 5 6 7 8 9` | 6 | 12 | [4, 8] | [5, 7] | [4, 8] |
| `- 2 3 4 - 6 7 8 9` | 6 | 6 | [6] | [2, 4] | [6] |
| `- 2 3 4 - 6 7 8 9` | 6 | 7 | [7] | [3, 4] | [7] |
| `- 2 3 4 - 6 7 8 9` | 6 | 8 | [8] | [2, 6] | [8] |
| `- 2 3 4 - 6 7 8 9` | 6 | 9 | [9] | [2, 3, 4] | [9] |
| `- 2 3 4 - 6 7 8 9` | 6 | 10 | [4, 6] | [3, 7] | [2, 8] |
| `- 2 3 4 - 6 7 8 9` | 6 | 11 | [2, 9] | [2, 3, 6] | [2, 9] |
| `- 2 3 4 - 6 7 8 9` | 6 | 12 | [4, 8] | [2, 3, 7] | [3, 9] |
| `1 - 3 - 5 6 7 8 9` | 6 | 6 | [6] | [1, 5] | [6] |
| `1 - 3 - 5 6 7 8 9` | 6 | 7 | [7] | [1, 6] | [7] |
| `1 - 3 - 5 6 7 8 9` | 6 | 8 | [8] | [1, 7] | [8] |
| `1 - 3 - 5 6 7 8 9` | 6 | 9 | [9] | [1, 3, 5] | [9] |
| `1 - 3 - 5 6 7 8 9` | 6 | 10 | [1, 9] | [1, 3, 6] | [1, 9] |
| `1 - 3 - 5 6 7 8 9` | 6 | 11 | [3, 8] | [1, 3, 7] | [3, 8] |
| `1 - 3 - 5 6 7 8 9` | 6 | 12 | [5, 7] | [1, 3, 8] | [3, 9] |
| `1 2 3 4 5 - 7 8 9` | 6 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 4 5 - 7 8 9` | 6 | 4 | [4] | [1, 3] | [4] |
| `1 2 3 4 5 - 7 8 9` | 6 | 5 | [5] | [1, 4] | [5] |
| `1 2 3 4 5 - 7 8 9` | 6 | 6 | [1, 5] | [1, 2, 3] | [1, 5] |
| `1 2 3 4 5 - 7 8 9` | 6 | 7 | [7] | [1, 2, 4] | [7] |
| `1 2 3 4 5 - 7 8 9` | 6 | 8 | [8] | [1, 3, 4] | [8] |
| `1 2 3 4 5 - 7 8 9` | 6 | 9 | [9] | [2, 3, 4] | [9] |
| `1 2 3 4 5 - 7 8 9` | 6 | 10 | [1, 9] | [1, 2, 3, 4] | [1, 9] |
| `1 2 3 4 5 - 7 8 9` | 6 | 11 | [2, 9] | [1, 2, 3, 5] | [2, 9] |
| `1 2 3 4 5 - 7 8 9` | 6 | 12 | [3, 9] | [2, 3, 7] | [3, 9] |
| `- - 3 - 5 6 7 8 9` | 7 | 8 | [8] | [3, 5] | [8] |
| `- - 3 - 5 6 7 8 9` | 7 | 9 | [9] | [3, 6] | [9] |
| `- - 3 - 5 6 7 8 9` | 7 | 11 | [5, 6] | [3, 8] | [3, 8] |
| `- - 3 - 5 6 7 8 9` | 7 | 12 | [3, 9] | [5, 7] | [3, 9] |
| `- 2 3 4 5 - 7 8 9` | 7 | 5 | [5] | [2, 3] | [5] |
| `- 2 3 4 5 - 7 8 9` | 7 | 7 | [7] | [3, 4] | [7] |
| `- 2 3 4 5 - 7 8 9` | 7 | 8 | [8] | [3, 5] | [8] |
| `- 2 3 4 5 - 7 8 9` | 7 | 9 | [9] | [2, 3, 4] | [9] |
| `- 2 3 4 5 - 7 8 9` | 7 | 10 | [3, 7] | [2, 3, 5] | [2, 8] |
| `- 2 3 4 5 - 7 8 9` | 7 | 11 | [2, 9] | [2, 4, 5] | [2, 9] |
| `- 2 3 4 5 - 7 8 9` | 7 | 12 | [3, 9] | [2, 3, 7] | [3, 9] |
| `1 - 3 4 - 6 7 8 9` | 7 | 4 | [4] | [1, 3] | [4] |
| `1 - 3 4 - 6 7 8 9` | 7 | 7 | [7] | [3, 4] | [7] |
| `1 - 3 4 - 6 7 8 9` | 7 | 8 | [8] | [1, 3, 4] | [8] |
| `1 - 3 4 - 6 7 8 9` | 7 | 9 | [9] | [1, 8] | [9] |
| `1 - 3 4 - 6 7 8 9` | 7 | 10 | [1, 9] | [1, 3, 6] | [1, 9] |
| `1 - 3 4 - 6 7 8 9` | 7 | 11 | [3, 8] | [1, 3, 7] | [3, 8] |
| `1 - 3 4 - 6 7 8 9` | 7 | 12 | [3, 9] | [1, 4, 7] | [3, 9] |
| `1 2 - - 5 6 7 8 9` | 7 | 6 | [6] | [1, 5] | [6] |
| `1 2 - - 5 6 7 8 9` | 7 | 7 | [7] | [2, 5] | [7] |
| `1 2 - - 5 6 7 8 9` | 7 | 8 | [8] | [1, 2, 5] | [8] |
| `1 2 - - 5 6 7 8 9` | 7 | 9 | [9] | [1, 2, 6] | [9] |
| `1 2 - - 5 6 7 8 9` | 7 | 10 | [1, 9] | [1, 2, 7] | [1, 9] |
| `1 2 - - 5 6 7 8 9` | 7 | 11 | [2, 9] | [1, 2, 8] | [2, 9] |
| `1 2 - - 5 6 7 8 9` | 7 | 12 | [5, 7] | [1, 5, 6] | [5, 7] |
| `1 2 3 4 5 6 - 8 9` | 7 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 4 5 6 - 8 9` | 7 | 4 | [4] | [1, 3] | [4] |
| `1 2 3 4 5 6 - 8 9` | 7 | 5 | [5] | [2, 3] | [5] |
| `1 2 3 4 5 6 - 8 9` | 7 | 6 | [6] | [1, 2, 3] | [6] |
| `1 2 3 4 5 6 - 8 9` | 7 | 7 | [1, 6] | [1, 2, 4] | [1, 6] |
| `1 2 3 4 5 6 - 8 9` | 7 | 8 | [8] | [1, 3, 4] | [8] |
| `1 2 3 4 5 6 - 8 9` | 7 | 9 | [9] | [1, 3, 5] | [9] |
| `1 2 3 4 5 6 - 8 9` | 7 | 10 | [1, 9] | [1, 2, 3, 4] | [1, 9] |
| `1 2 3 4 5 6 - 8 9` | 7 | 11 | [2, 9] | [1, 2, 3, 5] | [2, 9] |
| `1 2 3 4 5 6 - 8 9` | 7 | 12 | [3, 9] | [1, 2, 3, 6] | [3, 9] |
| `- - 3 4 - 6 7 8 9` | 8 | 7 | [7] | [3, 4] | [7] |
| `- - 3 4 - 6 7 8 9` | 8 | 9 | [9] | [3, 6] | [9] |
| `- - 3 4 - 6 7 8 9` | 8 | 10 | [4, 6] | [3, 7] | [3, 7] |
| `- - 3 4 - 6 7 8 9` | 8 | 11 | [3, 8] | [4, 7] | [3, 8] |
| `- - 3 4 - 6 7 8 9` | 8 | 12 | [3, 9] | [4, 8] | [3, 9] |
| `- 2 - - 5 6 7 8 9` | 8 | 7 | [7] | [2, 5] | [7] |
| `- 2 - - 5 6 7 8 9` | 8 | 8 | [8] | [2, 6] | [8] |
| `- 2 - - 5 6 7 8 9` | 8 | 9 | [9] | [2, 7] | [9] |
| `- 2 - - 5 6 7 8 9` | 8 | 11 | [2, 9] | [5, 6] | [2, 9] |
| `- 2 3 4 5 6 - 8 9` | 8 | 5 | [5] | [2, 3] | [5] |
| `- 2 3 4 5 6 - 8 9` | 8 | 6 | [6] | [2, 4] | [6] |
| `- 2 3 4 5 6 - 8 9` | 8 | 7 | [2, 5] | [3, 4] | [2, 5] |
| `- 2 3 4 5 6 - 8 9` | 8 | 8 | [8] | [3, 5] | [8] |
| `- 2 3 4 5 6 - 8 9` | 8 | 9 | [9] | [2, 3, 4] | [9] |
| `- 2 3 4 5 6 - 8 9` | 8 | 10 | [2, 8] | [2, 3, 5] | [2, 8] |
| `- 2 3 4 5 6 - 8 9` | 8 | 11 | [2, 9] | [2, 3, 6] | [2, 9] |
| `- 2 3 4 5 6 - 8 9` | 8 | 12 | [3, 9] | [2, 4, 6] | [3, 9] |
| `1 - 3 4 5 - 7 8 9` | 8 | 4 | [4] | [1, 3] | [4] |
| `1 - 3 4 5 - 7 8 9` | 8 | 5 | [5] | [1, 4] | [5] |
| `1 - 3 4 5 - 7 8 9` | 8 | 7 | [7] | [3, 4] | [7] |
| `1 - 3 4 5 - 7 8 9` | 8 | 8 | [8] | [1, 3, 4] | [8] |
| `1 - 3 4 5 - 7 8 9` | 8 | 9 | [9] | [1, 3, 5] | [9] |
| `1 - 3 4 5 - 7 8 9` | 8 | 10 | [1, 9] | [3, 7] | [1, 9] |
| `1 - 3 4 5 - 7 8 9` | 8 | 11 | [3, 8] | [1, 3, 7] | [3, 8] |
| `1 - 3 4 5 - 7 8 9` | 8 | 12 | [4, 8] | [1, 4, 7] | [3, 9] |
| `1 2 - 4 - 6 7 8 9` | 8 | 6 | [6] | [2, 4] | [6] |
| `1 2 - 4 - 6 7 8 9` | 8 | 7 | [7] | [1, 2, 4] | [7] |
| `1 2 - 4 - 6 7 8 9` | 8 | 8 | [8] | [1, 7] | [8] |
| `1 2 - 4 - 6 7 8 9` | 8 | 9 | [9] | [1, 2, 6] | [9] |
| `1 2 - 4 - 6 7 8 9` | 8 | 10 | [2, 8] | [1, 2, 7] | [1, 9] |
| `1 2 - 4 - 6 7 8 9` | 8 | 11 | [2, 9] | [1, 4, 6] | [2, 9] |
| `1 2 - 4 - 6 7 8 9` | 8 | 12 | [4, 8] | [2, 4, 6] | [4, 8] |
| `1 2 3 4 5 6 7 - 9` | 8 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 4 5 6 7 - 9` | 8 | 4 | [4] | [1, 3] | [4] |
| `1 2 3 4 5 6 7 - 9` | 8 | 5 | [5] | [2, 3] | [5] |
| `1 2 3 4 5 6 7 - 9` | 8 | 6 | [6] | [1, 2, 3] | [6] |
| `1 2 3 4 5 6 7 - 9` | 8 | 7 | [7] | [1, 2, 4] | [7] |
| `1 2 3 4 5 6 7 - 9` | 8 | 8 | [1, 7] | [1, 2, 5] | [1, 7] |
| `1 2 3 4 5 6 7 - 9` | 8 | 9 | [9] | [1, 2, 6] | [9] |
| `1 2 3 4 5 6 7 - 9` | 8 | 10 | [1, 9] | [1, 2, 3, 4] | [1, 9] |
| `1 2 3 4 5 6 7 - 9` | 8 | 11 | [2, 9] | [1, 2, 3, 5] | [2, 9] |
| `1 2 3 4 5 6 7 - 9` | 8 | 12 | [3, 9] | [1, 2, 3, 6] | [3, 9] |
| `- - 3 4 5 - 7 8 9` | 9 | 7 | [7] | [3, 4] | [7] |
| `- - 3 4 5 - 7 8 9` | 9 | 8 | [8] | [3, 5] | [8] |
| `- - 3 4 5 - 7 8 9` | 9 | 9 | [9] | [4, 5] | [9] |
| `- - 3 4 5 - 7 8 9` | 9 | 11 | [3, 8] | [4, 7] | [3, 8] |
| `- - 3 4 5 - 7 8 9` | 9 | 12 | [3, 9] | [3, 4, 5] | [3, 9] |
| `- 2 - 4 - 6 7 8 9` | 9 | 6 | [6] | [2, 4] | [6] |
| `- 2 - 4 - 6 7 8 9` | 9 | 8 | [8] | [2, 6] | [8] |
| `- 2 - 4 - 6 7 8 9` | 9 | 9 | [9] | [2, 7] | [9] |
| `- 2 - 4 - 6 7 8 9` | 9 | 10 | [2, 8] | [4, 6] | [2, 8] |
| `- 2 - 4 - 6 7 8 9` | 9 | 11 | [2, 9] | [4, 7] | [2, 9] |
| `- 2 - 4 - 6 7 8 9` | 9 | 12 | [4, 8] | [2, 4, 6] | [4, 8] |
| `- 2 3 4 5 6 7 - 9` | 9 | 5 | [5] | [2, 3] | [5] |
| `- 2 3 4 5 6 7 - 9` | 9 | 6 | [6] | [2, 4] | [6] |
| `- 2 3 4 5 6 7 - 9` | 9 | 7 | [7] | [2, 5] | [7] |
| `- 2 3 4 5 6 7 - 9` | 9 | 8 | [3, 5] | [2, 6] | [2, 6] |
| `- 2 3 4 5 6 7 - 9` | 9 | 9 | [9] | [2, 3, 4] | [9] |
| `- 2 3 4 5 6 7 - 9` | 9 | 10 | [3, 7] | [2, 3, 5] | [3, 7] |
| `- 2 3 4 5 6 7 - 9` | 9 | 11 | [2, 9] | [2, 3, 6] | [2, 9] |
| `- 2 3 4 5 6 7 - 9` | 9 | 12 | [3, 9] | [2, 3, 7] | [3, 9] |
| `1 - - - 5 6 7 8 9` | 9 | 6 | [6] | [1, 5] | [6] |
| `1 - - - 5 6 7 8 9` | 9 | 7 | [7] | [1, 6] | [7] |
| `1 - - - 5 6 7 8 9` | 9 | 8 | [8] | [1, 7] | [8] |
| `1 - - - 5 6 7 8 9` | 9 | 9 | [9] | [1, 8] | [9] |
| `1 - - - 5 6 7 8 9` | 9 | 12 | [5, 7] | [1, 5, 6] | [5, 7] |
| `1 - 3 4 5 6 - 8 9` | 9 | 4 | [4] | [1, 3] | [4] |
| `1 - 3 4 5 6 - 8 9` | 9 | 5 | [5] | [1, 4] | [5] |
| `1 - 3 4 5 6 - 8 9` | 9 | 6 | [6] | [1, 5] | [6] |
| `1 - 3 4 5 6 - 8 9` | 9 | 7 | [3, 4] | [1, 6] | [1, 6] |
| `1 - 3 4 5 6 - 8 9` | 9 | 8 | [8] | [1, 3, 4] | [8] |
| `1 - 3 4 5 6 - 8 9` | 9 | 9 | [9] | [1, 3, 5] | [9] |
| `1 - 3 4 5 6 - 8 9` | 9 | 10 | [1, 9] | [1, 3, 6] | [1, 9] |
| `1 - 3 4 5 6 - 8 9` | 9 | 11 | [5, 6] | [1, 4, 6] | [3, 8] |
| `1 - 3 4 5 6 - 8 9` | 9 | 12 | [3, 9] | [1, 3, 8] | [3, 9] |
| `1 2 - 4 5 - 7 8 9` | 9 | 5 | [5] | [1, 4] | [5] |
| `1 2 - 4 5 - 7 8 9` | 9 | 6 | [1, 5] | [2, 4] | [1, 5] |
| `1 2 - 4 5 - 7 8 9` | 9 | 7 | [7] | [1, 2, 4] | [7] |
| `1 2 - 4 5 - 7 8 9` | 9 | 8 | [8] | [1, 2, 5] | [8] |
| `1 2 - 4 5 - 7 8 9` | 9 | 9 | [9] | [2, 7] | [9] |
| `1 2 - 4 5 - 7 8 9` | 9 | 10 | [1, 9] | [1, 2, 7] | [1, 9] |
| `1 2 - 4 5 - 7 8 9` | 9 | 11 | [2, 9] | [2, 4, 5] | [2, 9] |
| `1 2 - 4 5 - 7 8 9` | 9 | 12 | [4, 8] | [1, 2, 4, 5] | [4, 8] |
| `1 2 3 - - 6 7 8 9` | 9 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 - - 6 7 8 9` | 9 | 6 | [6] | [1, 2, 3] | [6] |
| `1 2 3 - - 6 7 8 9` | 9 | 7 | [7] | [1, 6] | [7] |
| `1 2 3 - - 6 7 8 9` | 9 | 8 | [8] | [2, 6] | [8] |
| `1 2 3 - - 6 7 8 9` | 9 | 9 | [9] | [1, 2, 6] | [9] |
| `1 2 3 - - 6 7 8 9` | 9 | 10 | [1, 9] | [1, 2, 7] | [1, 9] |
| `1 2 3 - - 6 7 8 9` | 9 | 11 | [2, 9] | [2, 3, 6] | [2, 9] |
| `1 2 3 - - 6 7 8 9` | 9 | 12 | [3, 9] | [1, 2, 3, 6] | [3, 9] |
| `1 2 3 4 5 6 7 8 -` | 9 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 4 5 6 7 8 -` | 9 | 4 | [4] | [1, 3] | [4] |
| `1 2 3 4 5 6 7 8 -` | 9 | 5 | [5] | [2, 3] | [5] |
| `1 2 3 4 5 6 7 8 -` | 9 | 6 | [6] | [1, 2, 3] | [6] |
| `1 2 3 4 5 6 7 8 -` | 9 | 7 | [7] | [1, 2, 4] | [7] |
| `1 2 3 4 5 6 7 8 -` | 9 | 8 | [8] | [1, 3, 4] | [8] |
| `1 2 3 4 5 6 7 8 -` | 9 | 9 | [1, 8] | [2, 3, 4] | [1, 8] |
| `1 2 3 4 5 6 7 8 -` | 9 | 10 | [2, 8] | [1, 2, 3, 4] | [2, 8] |
| `1 2 3 4 5 6 7 8 -` | 9 | 11 | [3, 8] | [1, 2, 3, 5] | [3, 8] |
| `1 2 3 4 5 6 7 8 -` | 9 | 12 | [4, 8] | [1, 2, 3, 6] | [4, 8] |
| `- - 3 4 5 6 - 8 9` | 10 | 8 | [8] | [3, 5] | [8] |
| `- - 3 4 5 6 - 8 9` | 10 | 9 | [9] | [3, 6] | [9] |
| `- - 3 4 5 6 - 8 9` | 10 | 11 | [5, 6] | [3, 8] | [3, 8] |
| `- - 3 4 5 6 - 8 9` | 10 | 12 | [3, 9] | [3, 4, 5] | [3, 9] |
| `- 2 - 4 5 - 7 8 9` | 10 | 7 | [7] | [2, 5] | [7] |
| `- 2 - 4 5 - 7 8 9` | 10 | 9 | [9] | [2, 7] | [9] |
| `- 2 - 4 5 - 7 8 9` | 10 | 11 | [2, 9] | [2, 4, 5] | [2, 9] |
| `- 2 - 4 5 - 7 8 9` | 10 | 12 | [5, 7] | [4, 8] | [4, 8] |
| `- 2 3 - - 6 7 8 9` | 10 | 8 | [8] | [2, 6] | [8] |
| `- 2 3 - - 6 7 8 9` | 10 | 9 | [9] | [2, 7] | [9] |
| `- 2 3 - - 6 7 8 9` | 10 | 10 | [3, 7] | [2, 8] | [2, 8] |
| `- 2 3 - - 6 7 8 9` | 10 | 11 | [2, 9] | [2, 3, 6] | [2, 9] |
| `- 2 3 - - 6 7 8 9` | 10 | 12 | [3, 9] | [2, 3, 7] | [3, 9] |
| `- 2 3 4 5 6 7 8 -` | 10 | 5 | [5] | [2, 3] | [5] |
| `- 2 3 4 5 6 7 8 -` | 10 | 6 | [6] | [2, 4] | [6] |
| `- 2 3 4 5 6 7 8 -` | 10 | 7 | [7] | [3, 4] | [7] |
| `- 2 3 4 5 6 7 8 -` | 10 | 8 | [8] | [3, 5] | [8] |
| `- 2 3 4 5 6 7 8 -` | 10 | 9 | [2, 7] | [2, 3, 4] | [2, 7] |
| `- 2 3 4 5 6 7 8 -` | 10 | 10 | [2, 8] | [2, 3, 5] | [2, 8] |
| `- 2 3 4 5 6 7 8 -` | 10 | 11 | [5, 6] | [2, 3, 6] | [3, 8] |
| `- 2 3 4 5 6 7 8 -` | 10 | 12 | [5, 7] | [2, 4, 6] | [4, 8] |
| `1 - - 4 - 6 7 8 9` | 10 | 7 | [7] | [1, 6] | [7] |
| `1 - - 4 - 6 7 8 9` | 10 | 8 | [8] | [1, 7] | [8] |
| `1 - - 4 - 6 7 8 9` | 10 | 9 | [9] | [1, 8] | [9] |
| `1 - - 4 - 6 7 8 9` | 10 | 10 | [1, 9] | [4, 6] | [1, 9] |
| `1 - - 4 - 6 7 8 9` | 10 | 11 | [4, 7] | [1, 4, 6] | [4, 7] |
| `1 - - 4 - 6 7 8 9` | 10 | 12 | [4, 8] | [1, 4, 7] | [4, 8] |
| `1 - 3 4 5 6 7 - 9` | 10 | 4 | [4] | [1, 3] | [4] |
| `1 - 3 4 5 6 7 - 9` | 10 | 5 | [5] | [1, 4] | [5] |
| `1 - 3 4 5 6 7 - 9` | 10 | 6 | [6] | [1, 5] | [6] |
| `1 - 3 4 5 6 7 - 9` | 10 | 7 | [7] | [1, 6] | [7] |
| `1 - 3 4 5 6 7 - 9` | 10 | 8 | [1, 7] | [1, 3, 4] | [1, 7] |
| `1 - 3 4 5 6 7 - 9` | 10 | 9 | [9] | [1, 3, 5] | [9] |
| `1 - 3 4 5 6 7 - 9` | 10 | 10 | [1, 9] | [1, 3, 6] | [1, 9] |
| `1 - 3 4 5 6 7 - 9` | 10 | 11 | [4, 7] | [1, 3, 7] | [4, 7] |
| `1 - 3 4 5 6 7 - 9` | 10 | 12 | [3, 9] | [1, 5, 6] | [3, 9] |
| `1 2 - 4 5 6 - 8 9` | 10 | 5 | [5] | [1, 4] | [5] |
| `1 2 - 4 5 6 - 8 9` | 10 | 6 | [6] | [1, 5] | [6] |
| `1 2 - 4 5 6 - 8 9` | 10 | 7 | [1, 6] | [1, 2, 4] | [1, 6] |
| `1 2 - 4 5 6 - 8 9` | 10 | 8 | [8] | [1, 2, 5] | [8] |
| `1 2 - 4 5 6 - 8 9` | 10 | 9 | [9] | [1, 2, 6] | [9] |
| `1 2 - 4 5 6 - 8 9` | 10 | 10 | [1, 9] | [1, 4, 5] | [1, 9] |
| `1 2 - 4 5 6 - 8 9` | 10 | 11 | [2, 9] | [1, 2, 8] | [2, 9] |
| `1 2 - 4 5 6 - 8 9` | 10 | 12 | [4, 8] | [1, 2, 4, 5] | [4, 8] |
| `1 2 3 - 5 - 7 8 9` | 10 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 - 5 - 7 8 9` | 10 | 5 | [5] | [2, 3] | [5] |
| `1 2 3 - 5 - 7 8 9` | 10 | 6 | [1, 5] | [1, 2, 3] | [1, 5] |
| `1 2 3 - 5 - 7 8 9` | 10 | 7 | [7] | [2, 5] | [7] |
| `1 2 3 - 5 - 7 8 9` | 10 | 8 | [8] | [1, 2, 5] | [8] |
| `1 2 3 - 5 - 7 8 9` | 10 | 9 | [9] | [1, 3, 5] | [9] |
| `1 2 3 - 5 - 7 8 9` | 10 | 10 | [2, 8] | [1, 2, 7] | [1, 9] |
| `1 2 3 - 5 - 7 8 9` | 10 | 11 | [2, 9] | [1, 2, 3, 5] | [2, 9] |
| `1 2 3 - 5 - 7 8 9` | 10 | 12 | [3, 9] | [1, 3, 8] | [3, 9] |
| `- - 3 4 5 6 7 - 9` | 11 | 7 | [7] | [3, 4] | [7] |
| `- - 3 4 5 6 7 - 9` | 11 | 9 | [9] | [3, 6] | [9] |
| `- - 3 4 5 6 7 - 9` | 11 | 10 | [4, 6] | [3, 7] | [3, 7] |
| `- - 3 4 5 6 7 - 9` | 11 | 11 | [4, 7] | [5, 6] | [4, 7] |
| `- - 3 4 5 6 7 - 9` | 11 | 12 | [3, 9] | [3, 4, 5] | [3, 9] |
| `- 2 - 4 5 6 - 8 9` | 11 | 6 | [6] | [2, 4] | [6] |
| `- 2 - 4 5 6 - 8 9` | 11 | 8 | [8] | [2, 6] | [8] |
| `- 2 - 4 5 6 - 8 9` | 11 | 9 | [9] | [4, 5] | [9] |
| `- 2 - 4 5 6 - 8 9` | 11 | 10 | [4, 6] | [2, 8] | [2, 8] |
| `- 2 - 4 5 6 - 8 9` | 11 | 11 | [2, 9] | [2, 4, 5] | [2, 9] |
| `- 2 - 4 5 6 - 8 9` | 11 | 12 | [4, 8] | [2, 4, 6] | [4, 8] |
| `- 2 3 - 5 - 7 8 9` | 11 | 5 | [5] | [2, 3] | [5] |
| `- 2 3 - 5 - 7 8 9` | 11 | 7 | [7] | [2, 5] | [7] |
| `- 2 3 - 5 - 7 8 9` | 11 | 8 | [8] | [3, 5] | [8] |
| `- 2 3 - 5 - 7 8 9` | 11 | 9 | [9] | [2, 7] | [9] |
| `- 2 3 - 5 - 7 8 9` | 11 | 10 | [2, 8] | [2, 3, 5] | [2, 8] |
| `- 2 3 - 5 - 7 8 9` | 11 | 11 | [2, 9] | [3, 8] | [2, 9] |
| `- 2 3 - 5 - 7 8 9` | 11 | 12 | [3, 9] | [2, 3, 7] | [3, 9] |
| `1 - - 4 5 - 7 8 9` | 11 | 5 | [5] | [1, 4] | [5] |
| `1 - - 4 5 - 7 8 9` | 11 | 8 | [8] | [1, 7] | [8] |
| `1 - - 4 5 - 7 8 9` | 11 | 9 | [9] | [4, 5] | [9] |
| `1 - - 4 5 - 7 8 9` | 11 | 10 | [1, 9] | [1, 4, 5] | [1, 9] |
| `1 - - 4 5 - 7 8 9` | 11 | 12 | [4, 8] | [1, 4, 7] | [4, 8] |
| `1 - 3 - - 6 7 8 9` | 11 | 7 | [7] | [1, 6] | [7] |
| `1 - 3 - - 6 7 8 9` | 11 | 8 | [8] | [1, 7] | [8] |
| `1 - 3 - - 6 7 8 9` | 11 | 9 | [9] | [3, 6] | [9] |
| `1 - 3 - - 6 7 8 9` | 11 | 10 | [1, 9] | [1, 3, 6] | [1, 9] |
| `1 - 3 - - 6 7 8 9` | 11 | 11 | [3, 8] | [1, 3, 7] | [3, 8] |
| `1 - 3 - - 6 7 8 9` | 11 | 12 | [3, 9] | [1, 3, 8] | [3, 9] |
| `1 - 3 4 5 6 7 8 -` | 11 | 4 | [4] | [1, 3] | [4] |
| `1 - 3 4 5 6 7 8 -` | 11 | 5 | [5] | [1, 4] | [5] |
| `1 - 3 4 5 6 7 8 -` | 11 | 6 | [6] | [1, 5] | [6] |
| `1 - 3 4 5 6 7 8 -` | 11 | 7 | [7] | [3, 4] | [7] |
| `1 - 3 4 5 6 7 8 -` | 11 | 8 | [8] | [1, 3, 4] | [8] |
| `1 - 3 4 5 6 7 8 -` | 11 | 9 | [1, 8] | [1, 3, 5] | [1, 8] |
| `1 - 3 4 5 6 7 8 -` | 11 | 10 | [4, 6] | [1, 3, 6] | [3, 7] |
| `1 - 3 4 5 6 7 8 -` | 11 | 11 | [3, 8] | [1, 4, 6] | [3, 8] |
| `1 - 3 4 5 6 7 8 -` | 11 | 12 | [4, 8] | [3, 4, 5] | [4, 8] |
| `1 2 - 4 5 6 7 - 9` | 11 | 5 | [5] | [1, 4] | [5] |
| `1 2 - 4 5 6 7 - 9` | 11 | 6 | [6] | [1, 5] | [6] |
| `1 2 - 4 5 6 7 - 9` | 11 | 7 | [7] | [1, 2, 4] | [7] |
| `1 2 - 4 5 6 7 - 9` | 11 | 8 | [1, 7] | [1, 2, 5] | [1, 7] |
| `1 2 - 4 5 6 7 - 9` | 11 | 9 | [9] | [1, 2, 6] | [9] |
| `1 2 - 4 5 6 7 - 9` | 11 | 10 | [1, 9] | [1, 2, 7] | [1, 9] |
| `1 2 - 4 5 6 7 - 9` | 11 | 11 | [2, 9] | [1, 4, 6] | [2, 9] |
| `1 2 - 4 5 6 7 - 9` | 11 | 12 | [5, 7] | [1, 2, 4, 5] | [5, 7] |
| `1 2 3 - 5 6 - 8 9` | 11 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 - 5 6 - 8 9` | 11 | 5 | [5] | [2, 3] | [5] |
| `1 2 3 - 5 6 - 8 9` | 11 | 6 | [6] | [1, 2, 3] | [6] |
| `1 2 3 - 5 6 - 8 9` | 11 | 7 | [2, 5] | [1, 6] | [1, 6] |
| `1 2 3 - 5 6 - 8 9` | 11 | 8 | [8] | [1, 2, 5] | [8] |
| `1 2 3 - 5 6 - 8 9` | 11 | 9 | [9] | [1, 2, 6] | [9] |
| `1 2 3 - 5 6 - 8 9` | 11 | 10 | [2, 8] | [2, 3, 5] | [1, 9] |
| `1 2 3 - 5 6 - 8 9` | 11 | 11 | [2, 9] | [1, 2, 3, 5] | [2, 9] |
| `1 2 3 - 5 6 - 8 9` | 11 | 12 | [3, 9] | [1, 2, 3, 6] | [3, 9] |
| `1 2 3 4 - - 7 8 9` | 11 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 4 - - 7 8 9` | 11 | 4 | [4] | [1, 3] | [4] |
| `1 2 3 4 - - 7 8 9` | 11 | 5 | [1, 4] | [2, 3] | [1, 4] |
| `1 2 3 4 - - 7 8 9` | 11 | 6 | [2, 4] | [1, 2, 3] | [2, 4] |
| `1 2 3 4 - - 7 8 9` | 11 | 7 | [7] | [1, 2, 4] | [7] |
| `1 2 3 4 - - 7 8 9` | 11 | 8 | [8] | [1, 3, 4] | [8] |
| `1 2 3 4 - - 7 8 9` | 11 | 9 | [9] | [2, 3, 4] | [9] |
| `1 2 3 4 - - 7 8 9` | 11 | 10 | [1, 9] | [1, 2, 3, 4] | [1, 9] |
| `1 2 3 4 - - 7 8 9` | 11 | 11 | [3, 8] | [1, 2, 8] | [2, 9] |
| `1 2 3 4 - - 7 8 9` | 11 | 12 | [3, 9] | [2, 3, 7] | [3, 9] |
| `- - - 4 5 - 7 8 9` | 12 | 9 | [9] | [4, 5] | [9] |
| `- - - 4 5 - 7 8 9` | 12 | 12 | [4, 8] | [5, 7] | [4, 8] |
| `- - 3 - - 6 7 8 9` | 12 | 9 | [9] | [3, 6] | [9] |
| `- - 3 4 5 6 7 8 -` | 12 | 7 | [7] | [3, 4] | [7] |
| `- - 3 4 5 6 7 8 -` | 12 | 8 | [8] | [3, 5] | [8] |
| `- - 3 4 5 6 7 8 -` | 12 | 9 | [4, 5] | [3, 6] | [3, 6] |
| `- - 3 4 5 6 7 8 -` | 12 | 10 | [3, 7] | [4, 6] | [3, 7] |
| `- - 3 4 5 6 7 8 -` | 12 | 11 | [3, 8] | [4, 7] | [3, 8] |
| `- - 3 4 5 6 7 8 -` | 12 | 12 | [4, 8] | [3, 4, 5] | [4, 8] |
| `- 2 - 4 5 6 7 - 9` | 12 | 6 | [6] | [2, 4] | [6] |
| `- 2 - 4 5 6 7 - 9` | 12 | 7 | [7] | [2, 5] | [7] |
| `- 2 - 4 5 6 7 - 9` | 12 | 9 | [9] | [2, 7] | [9] |
| `- 2 - 4 5 6 7 - 9` | 12 | 11 | [2, 9] | [2, 4, 5] | [2, 9] |
| `- 2 - 4 5 6 7 - 9` | 12 | 12 | [5, 7] | [2, 4, 6] | [5, 7] |
| `- 2 3 - 5 6 - 8 9` | 12 | 5 | [5] | [2, 3] | [5] |
| `- 2 3 - 5 6 - 8 9` | 12 | 8 | [8] | [2, 6] | [8] |
| `- 2 3 - 5 6 - 8 9` | 12 | 9 | [9] | [3, 6] | [9] |
| `- 2 3 - 5 6 - 8 9` | 12 | 10 | [2, 8] | [2, 3, 5] | [2, 8] |
| `- 2 3 - 5 6 - 8 9` | 12 | 11 | [3, 8] | [2, 3, 6] | [2, 9] |
| `- 2 3 4 - - 7 8 9` | 12 | 7 | [7] | [3, 4] | [7] |
| `- 2 3 4 - - 7 8 9` | 12 | 9 | [9] | [2, 3, 4] | [9] |
| `- 2 3 4 - - 7 8 9` | 12 | 10 | [3, 7] | [2, 8] | [2, 8] |
| `- 2 3 4 - - 7 8 9` | 12 | 11 | [2, 9] | [3, 8] | [2, 9] |
| `- 2 3 4 - - 7 8 9` | 12 | 12 | [3, 9] | [2, 3, 7] | [3, 9] |
| `1 - - 4 5 6 - 8 9` | 12 | 5 | [5] | [1, 4] | [5] |
| `1 - - 4 5 6 - 8 9` | 12 | 6 | [6] | [1, 5] | [6] |
| `1 - - 4 5 6 - 8 9` | 12 | 9 | [9] | [1, 8] | [9] |
| `1 - - 4 5 6 - 8 9` | 12 | 10 | [1, 9] | [1, 4, 5] | [1, 9] |
| `1 - - 4 5 6 - 8 9` | 12 | 11 | [5, 6] | [1, 4, 6] | [5, 6] |
| `1 - - 4 5 6 - 8 9` | 12 | 12 | [4, 8] | [1, 5, 6] | [4, 8] |
| `1 - 3 - 5 - 7 8 9` | 12 | 8 | [8] | [1, 7] | [8] |
| `1 - 3 - 5 - 7 8 9` | 12 | 9 | [9] | [1, 3, 5] | [9] |
| `1 - 3 - 5 - 7 8 9` | 12 | 10 | [1, 9] | [3, 7] | [1, 9] |
| `1 - 3 - 5 - 7 8 9` | 12 | 11 | [3, 8] | [1, 3, 7] | [3, 8] |
| `1 - 3 - 5 - 7 8 9` | 12 | 12 | [3, 9] | [1, 3, 8] | [3, 9] |
| `1 2 - - - 6 7 8 9` | 12 | 7 | [7] | [1, 6] | [7] |
| `1 2 - - - 6 7 8 9` | 12 | 8 | [8] | [2, 6] | [8] |
| `1 2 - - - 6 7 8 9` | 12 | 9 | [9] | [1, 2, 6] | [9] |
| `1 2 - - - 6 7 8 9` | 12 | 10 | [1, 9] | [1, 2, 7] | [1, 9] |
| `1 2 - - - 6 7 8 9` | 12 | 11 | [2, 9] | [1, 2, 8] | [2, 9] |
| `1 2 - 4 5 6 7 8 -` | 12 | 5 | [5] | [1, 4] | [5] |
| `1 2 - 4 5 6 7 8 -` | 12 | 6 | [6] | [2, 4] | [6] |
| `1 2 - 4 5 6 7 8 -` | 12 | 7 | [7] | [1, 2, 4] | [7] |
| `1 2 - 4 5 6 7 8 -` | 12 | 8 | [8] | [1, 2, 5] | [8] |
| `1 2 - 4 5 6 7 8 -` | 12 | 9 | [1, 8] | [1, 2, 6] | [1, 8] |
| `1 2 - 4 5 6 7 8 -` | 12 | 10 | [2, 8] | [1, 4, 5] | [2, 8] |
| `1 2 - 4 5 6 7 8 -` | 12 | 11 | [5, 6] | [2, 4, 5] | [4, 7] |
| `1 2 - 4 5 6 7 8 -` | 12 | 12 | [5, 7] | [1, 2, 4, 5] | [4, 8] |
| `1 2 3 - 5 6 7 - 9` | 12 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 - 5 6 7 - 9` | 12 | 5 | [5] | [2, 3] | [5] |
| `1 2 3 - 5 6 7 - 9` | 12 | 6 | [6] | [1, 2, 3] | [6] |
| `1 2 3 - 5 6 7 - 9` | 12 | 7 | [7] | [1, 6] | [7] |
| `1 2 3 - 5 6 7 - 9` | 12 | 8 | [2, 6] | [1, 2, 5] | [1, 7] |
| `1 2 3 - 5 6 7 - 9` | 12 | 9 | [9] | [1, 3, 5] | [9] |
| `1 2 3 - 5 6 7 - 9` | 12 | 10 | [1, 9] | [1, 3, 6] | [1, 9] |
| `1 2 3 - 5 6 7 - 9` | 12 | 11 | [2, 9] | [1, 2, 3, 5] | [2, 9] |
| `1 2 3 - 5 6 7 - 9` | 12 | 12 | [3, 9] | [1, 2, 3, 6] | [3, 9] |
| `1 2 3 4 - 6 - 8 9` | 12 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 4 - 6 - 8 9` | 12 | 4 | [4] | [1, 3] | [4] |
| `1 2 3 4 - 6 - 8 9` | 12 | 5 | [2, 3] | [1, 4] | [1, 4] |
| `1 2 3 4 - 6 - 8 9` | 12 | 6 | [6] | [1, 2, 3] | [6] |
| `1 2 3 4 - 6 - 8 9` | 12 | 7 | [1, 6] | [1, 2, 4] | [1, 6] |
| `1 2 3 4 - 6 - 8 9` | 12 | 8 | [8] | [1, 3, 4] | [8] |
| `1 2 3 4 - 6 - 8 9` | 12 | 9 | [9] | [1, 2, 6] | [9] |
| `1 2 3 4 - 6 - 8 9` | 12 | 10 | [1, 9] | [1, 2, 3, 4] | [1, 9] |
| `1 2 3 4 - 6 - 8 9` | 12 | 11 | [2, 9] | [2, 3, 6] | [2, 9] |
| `1 2 3 4 - 6 - 8 9` | 12 | 12 | [3, 9] | [1, 2, 3, 6] | [3, 9] |
| `- - - 4 5 6 - 8 9` | 13 | 9 | [9] | [4, 5] | [9] |
| `- - 3 - 5 - 7 8 9` | 13 | 8 | [8] | [3, 5] | [8] |
| `- - 3 - 5 - 7 8 9` | 13 | 12 | [3, 9] | [5, 7] | [3, 9] |
| `- 2 - - - 6 7 8 9` | 13 | 8 | [8] | [2, 6] | [8] |
| `- 2 - - - 6 7 8 9` | 13 | 9 | [9] | [2, 7] | [9] |
| `- 2 - 4 5 6 7 8 -` | 13 | 6 | [6] | [2, 4] | [6] |
| `- 2 - 4 5 6 7 8 -` | 13 | 7 | [7] | [2, 5] | [7] |
| `- 2 - 4 5 6 7 8 -` | 13 | 8 | [8] | [2, 6] | [8] |
| `- 2 - 4 5 6 7 8 -` | 13 | 9 | [2, 7] | [4, 5] | [2, 7] |
| `- 2 - 4 5 6 7 8 -` | 13 | 10 | [2, 8] | [4, 6] | [2, 8] |
| `- 2 - 4 5 6 7 8 -` | 13 | 11 | [5, 6] | [2, 4, 5] | [4, 7] |
| `- 2 - 4 5 6 7 8 -` | 13 | 12 | [4, 8] | [2, 4, 6] | [4, 8] |
| `- 2 3 - 5 6 7 - 9` | 13 | 5 | [5] | [2, 3] | [5] |
| `- 2 3 - 5 6 7 - 9` | 13 | 7 | [7] | [2, 5] | [7] |
| `- 2 3 - 5 6 7 - 9` | 13 | 8 | [2, 6] | [3, 5] | [2, 6] |
| `- 2 3 - 5 6 7 - 9` | 13 | 9 | [9] | [3, 6] | [9] |
| `- 2 3 - 5 6 7 - 9` | 13 | 10 | [3, 7] | [2, 3, 5] | [3, 7] |
| `- 2 3 - 5 6 7 - 9` | 13 | 11 | [2, 9] | [2, 3, 6] | [2, 9] |
| `- 2 3 - 5 6 7 - 9` | 13 | 12 | [3, 9] | [2, 3, 7] | [3, 9] |
| `- 2 3 4 - 6 - 8 9` | 13 | 6 | [6] | [2, 4] | [6] |
| `- 2 3 4 - 6 - 8 9` | 13 | 8 | [8] | [2, 6] | [8] |
| `- 2 3 4 - 6 - 8 9` | 13 | 9 | [9] | [2, 3, 4] | [9] |
| `- 2 3 4 - 6 - 8 9` | 13 | 11 | [2, 9] | [2, 3, 6] | [2, 9] |
| `- 2 3 4 - 6 - 8 9` | 13 | 12 | [4, 8] | [2, 4, 6] | [3, 9] |
| `1 - - 4 5 6 7 - 9` | 13 | 5 | [5] | [1, 4] | [5] |
| `1 - - 4 5 6 7 - 9` | 13 | 6 | [6] | [1, 5] | [6] |
| `1 - - 4 5 6 7 - 9` | 13 | 7 | [7] | [1, 6] | [7] |
| `1 - - 4 5 6 7 - 9` | 13 | 9 | [9] | [4, 5] | [9] |
| `1 - - 4 5 6 7 - 9` | 13 | 10 | [1, 9] | [1, 4, 5] | [1, 9] |
| `1 - - 4 5 6 7 - 9` | 13 | 11 | [5, 6] | [1, 4, 6] | [4, 7] |
| `1 - - 4 5 6 7 - 9` | 13 | 12 | [5, 7] | [1, 4, 7] | [5, 7] |
| `1 - 3 - 5 6 - 8 9` | 13 | 6 | [6] | [1, 5] | [6] |
| `1 - 3 - 5 6 - 8 9` | 13 | 8 | [8] | [3, 5] | [8] |
| `1 - 3 - 5 6 - 8 9` | 13 | 9 | [9] | [1, 3, 5] | [9] |
| `1 - 3 - 5 6 - 8 9` | 13 | 10 | [1, 9] | [1, 3, 6] | [1, 9] |
| `1 - 3 - 5 6 - 8 9` | 13 | 11 | [3, 8] | [5, 6] | [3, 8] |
| `1 - 3 - 5 6 - 8 9` | 13 | 12 | [3, 9] | [1, 3, 8] | [3, 9] |
| `1 - 3 4 - - 7 8 9` | 13 | 4 | [4] | [1, 3] | [4] |
| `1 - 3 4 - - 7 8 9` | 13 | 7 | [7] | [3, 4] | [7] |
| `1 - 3 4 - - 7 8 9` | 13 | 8 | [8] | [1, 3, 4] | [8] |
| `1 - 3 4 - - 7 8 9` | 13 | 9 | [9] | [1, 8] | [9] |
| `1 - 3 4 - - 7 8 9` | 13 | 10 | [1, 9] | [3, 7] | [1, 9] |
| `1 - 3 4 - - 7 8 9` | 13 | 11 | [3, 8] | [1, 3, 7] | [3, 8] |
| `1 - 3 4 - - 7 8 9` | 13 | 12 | [4, 8] | [1, 3, 8] | [3, 9] |
| `1 2 - - 5 - 7 8 9` | 13 | 7 | [7] | [2, 5] | [7] |
| `1 2 - - 5 - 7 8 9` | 13 | 8 | [8] | [1, 2, 5] | [8] |
| `1 2 - - 5 - 7 8 9` | 13 | 9 | [9] | [1, 8] | [9] |
| `1 2 - - 5 - 7 8 9` | 13 | 10 | [2, 8] | [1, 2, 7] | [1, 9] |
| `1 2 - - 5 - 7 8 9` | 13 | 11 | [2, 9] | [1, 2, 8] | [2, 9] |
| `1 2 - - 5 - 7 8 9` | 13 | 12 | [5, 7] | [1, 2, 9] | [5, 7] |
| `1 2 3 - 5 6 7 8 -` | 13 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 - 5 6 7 8 -` | 13 | 5 | [5] | [2, 3] | [5] |
| `1 2 3 - 5 6 7 8 -` | 13 | 6 | [6] | [1, 2, 3] | [6] |
| `1 2 3 - 5 6 7 8 -` | 13 | 7 | [7] | [1, 6] | [7] |
| `1 2 3 - 5 6 7 8 -` | 13 | 8 | [8] | [1, 2, 5] | [8] |
| `1 2 3 - 5 6 7 8 -` | 13 | 9 | [1, 8] | [1, 2, 6] | [1, 8] |
| `1 2 3 - 5 6 7 8 -` | 13 | 10 | [2, 8] | [2, 3, 5] | [2, 8] |
| `1 2 3 - 5 6 7 8 -` | 13 | 11 | [5, 6] | [1, 2, 3, 5] | [3, 8] |
| `1 2 3 - 5 6 7 8 -` | 13 | 12 | [5, 7] | [1, 2, 3, 6] | [5, 7] |
| `1 2 3 4 - 6 7 - 9` | 13 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 4 - 6 7 - 9` | 13 | 4 | [4] | [1, 3] | [4] |
| `1 2 3 4 - 6 7 - 9` | 13 | 5 | [2, 3] | [1, 4] | [1, 4] |
| `1 2 3 4 - 6 7 - 9` | 13 | 6 | [6] | [1, 2, 3] | [6] |
| `1 2 3 4 - 6 7 - 9` | 13 | 7 | [7] | [1, 2, 4] | [7] |
| `1 2 3 4 - 6 7 - 9` | 13 | 8 | [1, 7] | [1, 3, 4] | [1, 7] |
| `1 2 3 4 - 6 7 - 9` | 13 | 9 | [9] | [1, 2, 6] | [9] |
| `1 2 3 4 - 6 7 - 9` | 13 | 10 | [1, 9] | [1, 2, 3, 4] | [1, 9] |
| `1 2 3 4 - 6 7 - 9` | 13 | 11 | [2, 9] | [1, 4, 6] | [2, 9] |
| `1 2 3 4 - 6 7 - 9` | 13 | 12 | [3, 9] | [1, 2, 3, 6] | [3, 9] |
| `1 2 3 4 5 - - 8 9` | 13 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 4 5 - - 8 9` | 13 | 4 | [4] | [1, 3] | [4] |
| `1 2 3 4 5 - - 8 9` | 13 | 5 | [5] | [2, 3] | [5] |
| `1 2 3 4 5 - - 8 9` | 13 | 6 | [1, 5] | [1, 2, 3] | [1, 5] |
| `1 2 3 4 5 - - 8 9` | 13 | 7 | [3, 4] | [1, 2, 4] | [2, 5] |
| `1 2 3 4 5 - - 8 9` | 13 | 8 | [8] | [1, 2, 5] | [8] |
| `1 2 3 4 5 - - 8 9` | 13 | 9 | [9] | [2, 3, 4] | [9] |
| `1 2 3 4 5 - - 8 9` | 13 | 10 | [1, 9] | [1, 2, 3, 4] | [1, 9] |
| `1 2 3 4 5 - - 8 9` | 13 | 11 | [2, 9] | [1, 2, 3, 5] | [2, 9] |
| `1 2 3 4 5 - - 8 9` | 13 | 12 | [3, 9] | [1, 2, 4, 5] | [3, 9] |
| `- - - 4 5 6 7 - 9` | 14 | 9 | [9] | [4, 5] | [9] |
| `- - - 4 5 6 7 - 9` | 14 | 11 | [5, 6] | [4, 7] | [4, 7] |
| `- - 3 - 5 6 - 8 9` | 14 | 8 | [8] | [3, 5] | [8] |
| `- - 3 - 5 6 - 8 9` | 14 | 9 | [9] | [3, 6] | [9] |
| `- - 3 - 5 6 - 8 9` | 14 | 11 | [5, 6] | [3, 8] | [3, 8] |
| `- - 3 4 - - 7 8 9` | 14 | 7 | [7] | [3, 4] | [7] |
| `- - 3 4 - - 7 8 9` | 14 | 11 | [4, 7] | [3, 8] | [3, 8] |
| `- - 3 4 - - 7 8 9` | 14 | 12 | [3, 9] | [4, 8] | [3, 9] |
| `- 2 - - 5 - 7 8 9` | 14 | 7 | [7] | [2, 5] | [7] |
| `- 2 - - 5 - 7 8 9` | 14 | 9 | [9] | [2, 7] | [9] |
| `- 2 3 - 5 6 7 8 -` | 14 | 5 | [5] | [2, 3] | [5] |
| `- 2 3 - 5 6 7 8 -` | 14 | 7 | [7] | [2, 5] | [7] |
| `- 2 3 - 5 6 7 8 -` | 14 | 8 | [8] | [2, 6] | [8] |
| `- 2 3 - 5 6 7 8 -` | 14 | 9 | [3, 6] | [2, 7] | [2, 7] |
| `- 2 3 - 5 6 7 8 -` | 14 | 10 | [2, 8] | [2, 3, 5] | [2, 8] |
| `- 2 3 - 5 6 7 8 -` | 14 | 11 | [3, 8] | [2, 3, 6] | [3, 8] |
| `- 2 3 - 5 6 7 8 -` | 14 | 12 | [5, 7] | [2, 3, 7] | [5, 7] |
| `- 2 3 4 - 6 7 - 9` | 14 | 6 | [6] | [2, 4] | [6] |
| `- 2 3 4 - 6 7 - 9` | 14 | 7 | [7] | [3, 4] | [7] |
| `- 2 3 4 - 6 7 - 9` | 14 | 9 | [9] | [2, 3, 4] | [9] |
| `- 2 3 4 - 6 7 - 9` | 14 | 10 | [3, 7] | [4, 6] | [3, 7] |
| `- 2 3 4 - 6 7 - 9` | 14 | 11 | [2, 9] | [2, 3, 6] | [2, 9] |
| `- 2 3 4 - 6 7 - 9` | 14 | 12 | [3, 9] | [2, 3, 7] | [3, 9] |
| `- 2 3 4 5 - - 8 9` | 14 | 5 | [5] | [2, 3] | [5] |
| `- 2 3 4 5 - - 8 9` | 14 | 7 | [3, 4] | [2, 5] | [2, 5] |
| `- 2 3 4 5 - - 8 9` | 14 | 8 | [8] | [3, 5] | [8] |
| `- 2 3 4 5 - - 8 9` | 14 | 9 | [9] | [2, 3, 4] | [9] |
| `- 2 3 4 5 - - 8 9` | 14 | 10 | [2, 8] | [2, 3, 5] | [2, 8] |
| `- 2 3 4 5 - - 8 9` | 14 | 11 | [2, 9] | [2, 4, 5] | [2, 9] |
| `- 2 3 4 5 - - 8 9` | 14 | 12 | [3, 9] | [3, 4, 5] | [3, 9] |
| `1 - - - - 6 7 8 9` | 14 | 7 | [7] | [1, 6] | [7] |
| `1 - - - - 6 7 8 9` | 14 | 8 | [8] | [1, 7] | [8] |
| `1 - - - - 6 7 8 9` | 14 | 9 | [9] | [1, 8] | [9] |
| `1 - - 4 5 6 7 8 -` | 14 | 5 | [5] | [1, 4] | [5] |
| `1 - - 4 5 6 7 8 -` | 14 | 6 | [6] | [1, 5] | [6] |
| `1 - - 4 5 6 7 8 -` | 14 | 7 | [7] | [1, 6] | [7] |
| `1 - - 4 5 6 7 8 -` | 14 | 8 | [8] | [1, 7] | [8] |
| `1 - - 4 5 6 7 8 -` | 14 | 9 | [1, 8] | [4, 5] | [1, 8] |
| `1 - - 4 5 6 7 8 -` | 14 | 10 | [4, 6] | [1, 4, 5] | [4, 6] |
| `1 - - 4 5 6 7 8 -` | 14 | 11 | [4, 7] | [1, 4, 6] | [4, 7] |
| `1 - - 4 5 6 7 8 -` | 14 | 12 | [5, 7] | [1, 4, 7] | [4, 8] |
| `1 - 3 - 5 6 7 - 9` | 14 | 6 | [6] | [1, 5] | [6] |
| `1 - 3 - 5 6 7 - 9` | 14 | 7 | [7] | [1, 6] | [7] |
| `1 - 3 - 5 6 7 - 9` | 14 | 8 | [3, 5] | [1, 7] | [1, 7] |
| `1 - 3 - 5 6 7 - 9` | 14 | 9 | [9] | [1, 3, 5] | [9] |
| `1 - 3 - 5 6 7 - 9` | 14 | 10 | [1, 9] | [1, 3, 6] | [1, 9] |
| `1 - 3 - 5 6 7 - 9` | 14 | 11 | [5, 6] | [1, 3, 7] | [5, 6] |
| `1 - 3 - 5 6 7 - 9` | 14 | 12 | [3, 9] | [1, 5, 6] | [3, 9] |
| `1 - 3 4 - 6 - 8 9` | 14 | 4 | [4] | [1, 3] | [4] |
| `1 - 3 4 - 6 - 8 9` | 14 | 7 | [3, 4] | [1, 6] | [1, 6] |
| `1 - 3 4 - 6 - 8 9` | 14 | 8 | [8] | [1, 3, 4] | [8] |
| `1 - 3 4 - 6 - 8 9` | 14 | 9 | [9] | [3, 6] | [9] |
| `1 - 3 4 - 6 - 8 9` | 14 | 10 | [1, 9] | [1, 3, 6] | [1, 9] |
| `1 - 3 4 - 6 - 8 9` | 14 | 11 | [3, 8] | [1, 4, 6] | [3, 8] |
| `1 - 3 4 - 6 - 8 9` | 14 | 12 | [3, 9] | [1, 3, 8] | [3, 9] |
| `1 2 - - 5 6 - 8 9` | 14 | 6 | [6] | [1, 5] | [6] |
| `1 2 - - 5 6 - 8 9` | 14 | 7 | [1, 6] | [2, 5] | [1, 6] |
| `1 2 - - 5 6 - 8 9` | 14 | 8 | [8] | [1, 2, 5] | [8] |
| `1 2 - - 5 6 - 8 9` | 14 | 9 | [9] | [1, 2, 6] | [9] |
| `1 2 - - 5 6 - 8 9` | 14 | 10 | [1, 9] | [2, 8] | [1, 9] |
| `1 2 - - 5 6 - 8 9` | 14 | 11 | [2, 9] | [1, 2, 8] | [2, 9] |
| `1 2 - - 5 6 - 8 9` | 14 | 12 | [1, 5, 6] | [1, 2, 9] | [1, 2, 9] |
| `1 2 - 4 - - 7 8 9` | 14 | 7 | [7] | [1, 2, 4] | [7] |
| `1 2 - 4 - - 7 8 9` | 14 | 8 | [8] | [1, 7] | [8] |
| `1 2 - 4 - - 7 8 9` | 14 | 9 | [9] | [2, 7] | [9] |
| `1 2 - 4 - - 7 8 9` | 14 | 10 | [1, 9] | [1, 2, 7] | [1, 9] |
| `1 2 - 4 - - 7 8 9` | 14 | 11 | [2, 9] | [1, 2, 8] | [2, 9] |
| `1 2 - 4 - - 7 8 9` | 14 | 12 | [4, 8] | [1, 4, 7] | [4, 8] |
| `1 2 3 4 - 6 7 8 -` | 14 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 4 - 6 7 8 -` | 14 | 4 | [4] | [1, 3] | [4] |
| `1 2 3 4 - 6 7 8 -` | 14 | 5 | [1, 4] | [2, 3] | [1, 4] |
| `1 2 3 4 - 6 7 8 -` | 14 | 6 | [6] | [1, 2, 3] | [6] |
| `1 2 3 4 - 6 7 8 -` | 14 | 7 | [7] | [1, 2, 4] | [7] |
| `1 2 3 4 - 6 7 8 -` | 14 | 8 | [8] | [1, 3, 4] | [8] |
| `1 2 3 4 - 6 7 8 -` | 14 | 9 | [1, 8] | [2, 3, 4] | [1, 8] |
| `1 2 3 4 - 6 7 8 -` | 14 | 10 | [2, 8] | [1, 2, 3, 4] | [2, 8] |
| `1 2 3 4 - 6 7 8 -` | 14 | 11 | [3, 8] | [1, 3, 7] | [3, 8] |
| `1 2 3 4 - 6 7 8 -` | 14 | 12 | [4, 8] | [1, 2, 3, 6] | [4, 8] |
| `1 2 3 4 5 - 7 - 9` | 14 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 4 5 - 7 - 9` | 14 | 4 | [4] | [1, 3] | [4] |
| `1 2 3 4 5 - 7 - 9` | 14 | 5 | [5] | [1, 4] | [5] |
| `1 2 3 4 5 - 7 - 9` | 14 | 6 | [2, 4] | [1, 2, 3] | [1, 5] |
| `1 2 3 4 5 - 7 - 9` | 14 | 7 | [7] | [1, 2, 4] | [7] |
| `1 2 3 4 5 - 7 - 9` | 14 | 8 | [3, 5] | [1, 3, 4] | [1, 7] |
| `1 2 3 4 5 - 7 - 9` | 14 | 9 | [9] | [1, 3, 5] | [9] |
| `1 2 3 4 5 - 7 - 9` | 14 | 10 | [1, 9] | [1, 2, 3, 4] | [1, 9] |
| `1 2 3 4 5 - 7 - 9` | 14 | 11 | [2, 9] | [1, 2, 3, 5] | [2, 9] |
| `1 2 3 4 5 - 7 - 9` | 14 | 12 | [3, 9] | [1, 2, 4, 5] | [3, 9] |
| `- - - 4 5 6 7 8 -` | 15 | 11 | [5, 6] | [4, 7] | [4, 7] |
| `- - - 4 5 6 7 8 -` | 15 | 12 | [4, 8] | [5, 7] | [4, 8] |
| `- - 3 - 5 6 7 - 9` | 15 | 9 | [9] | [3, 6] | [9] |
| `- - 3 - 5 6 7 - 9` | 15 | 12 | [3, 9] | [5, 7] | [3, 9] |
| `- - 3 4 - 6 - 8 9` | 15 | 9 | [9] | [3, 6] | [9] |
| `- - 3 4 - 6 - 8 9` | 15 | 12 | [3, 9] | [4, 8] | [3, 9] |
| `- 2 - - 5 6 - 8 9` | 15 | 8 | [8] | [2, 6] | [8] |
| `- 2 - - 5 6 - 8 9` | 15 | 11 | [5, 6] | [2, 9] | [2, 9] |
| `- 2 - 4 - - 7 8 9` | 15 | 9 | [9] | [2, 7] | [9] |
| `- 2 - 4 - - 7 8 9` | 15 | 11 | [2, 9] | [4, 7] | [2, 9] |
| `- 2 3 4 - 6 7 8 -` | 15 | 6 | [6] | [2, 4] | [6] |
| `- 2 3 4 - 6 7 8 -` | 15 | 7 | [7] | [3, 4] | [7] |
| `- 2 3 4 - 6 7 8 -` | 15 | 8 | [8] | [2, 6] | [8] |
| `- 2 3 4 - 6 7 8 -` | 15 | 9 | [2, 7] | [2, 3, 4] | [2, 7] |
| `- 2 3 4 - 6 7 8 -` | 15 | 10 | [4, 6] | [3, 7] | [2, 8] |
| `- 2 3 4 - 6 7 8 -` | 15 | 11 | [3, 8] | [2, 3, 6] | [3, 8] |
| `- 2 3 4 - 6 7 8 -` | 15 | 12 | [4, 8] | [2, 3, 7] | [4, 8] |
| `- 2 3 4 5 - 7 - 9` | 15 | 5 | [5] | [2, 3] | [5] |
| `- 2 3 4 5 - 7 - 9` | 15 | 7 | [7] | [3, 4] | [7] |
| `- 2 3 4 5 - 7 - 9` | 15 | 9 | [9] | [2, 3, 4] | [9] |
| `- 2 3 4 5 - 7 - 9` | 15 | 10 | [3, 7] | [2, 3, 5] | [3, 7] |
| `- 2 3 4 5 - 7 - 9` | 15 | 11 | [2, 9] | [2, 4, 5] | [2, 9] |
| `- 2 3 4 5 - 7 - 9` | 15 | 12 | [3, 9] | [2, 3, 7] | [3, 9] |
| `1 - - - 5 - 7 8 9` | 15 | 8 | [8] | [1, 7] | [8] |
| `1 - - - 5 - 7 8 9` | 15 | 9 | [9] | [1, 8] | [9] |
| `1 - 3 - 5 6 7 8 -` | 15 | 6 | [6] | [1, 5] | [6] |
| `1 - 3 - 5 6 7 8 -` | 15 | 7 | [7] | [1, 6] | [7] |
| `1 - 3 - 5 6 7 8 -` | 15 | 8 | [8] | [3, 5] | [8] |
| `1 - 3 - 5 6 7 8 -` | 15 | 9 | [1, 8] | [1, 3, 5] | [1, 8] |
| `1 - 3 - 5 6 7 8 -` | 15 | 10 | [3, 7] | [1, 3, 6] | [3, 7] |
| `1 - 3 - 5 6 7 8 -` | 15 | 11 | [5, 6] | [1, 3, 7] | [3, 8] |
| `1 - 3 - 5 6 7 8 -` | 15 | 12 | [5, 7] | [1, 3, 8] | [5, 7] |
| `1 - 3 4 - 6 7 - 9` | 15 | 4 | [4] | [1, 3] | [4] |
| `1 - 3 4 - 6 7 - 9` | 15 | 7 | [7] | [1, 6] | [7] |
| `1 - 3 4 - 6 7 - 9` | 15 | 8 | [1, 7] | [1, 3, 4] | [1, 7] |
| `1 - 3 4 - 6 7 - 9` | 15 | 9 | [9] | [3, 6] | [9] |
| `1 - 3 4 - 6 7 - 9` | 15 | 10 | [4, 6] | [1, 3, 6] | [1, 9] |
| `1 - 3 4 - 6 7 - 9` | 15 | 11 | [4, 7] | [1, 3, 7] | [4, 7] |
| `1 - 3 4 - 6 7 - 9` | 15 | 12 | [3, 9] | [1, 4, 7] | [3, 9] |
| `1 - 3 4 5 - - 8 9` | 15 | 4 | [4] | [1, 3] | [4] |
| `1 - 3 4 5 - - 8 9` | 15 | 5 | [5] | [1, 4] | [5] |
| `1 - 3 4 5 - - 8 9` | 15 | 8 | [8] | [1, 3, 4] | [8] |
| `1 - 3 4 5 - - 8 9` | 15 | 9 | [9] | [1, 3, 5] | [9] |
| `1 - 3 4 5 - - 8 9` | 15 | 10 | [1, 9] | [1, 4, 5] | [1, 9] |
| `1 - 3 4 5 - - 8 9` | 15 | 12 | [4, 8] | [1, 3, 8] | [3, 9] |
| `1 2 - - 5 6 7 - 9` | 15 | 6 | [6] | [1, 5] | [6] |
| `1 2 - - 5 6 7 - 9` | 15 | 7 | [7] | [1, 6] | [7] |
| `1 2 - - 5 6 7 - 9` | 15 | 8 | [2, 6] | [1, 2, 5] | [1, 7] |
| `1 2 - - 5 6 7 - 9` | 15 | 9 | [9] | [1, 2, 6] | [9] |
| `1 2 - - 5 6 7 - 9` | 15 | 10 | [1, 9] | [1, 2, 7] | [1, 9] |
| `1 2 - - 5 6 7 - 9` | 15 | 11 | [5, 6] | [2, 9] | [2, 9] |
| `1 2 - - 5 6 7 - 9` | 15 | 12 | [5, 7] | [1, 5, 6] | [5, 7] |
| `1 2 - 4 - 6 - 8 9` | 15 | 6 | [6] | [2, 4] | [6] |
| `1 2 - 4 - 6 - 8 9` | 15 | 7 | [1, 6] | [1, 2, 4] | [1, 6] |
| `1 2 - 4 - 6 - 8 9` | 15 | 8 | [8] | [2, 6] | [8] |
| `1 2 - 4 - 6 - 8 9` | 15 | 9 | [9] | [1, 2, 6] | [9] |
| `1 2 - 4 - 6 - 8 9` | 15 | 10 | [2, 8] | [1, 9] | [1, 9] |
| `1 2 - 4 - 6 - 8 9` | 15 | 11 | [2, 9] | [1, 2, 8] | [2, 9] |
| `1 2 - 4 - 6 - 8 9` | 15 | 12 | [4, 8] | [2, 4, 6] | [4, 8] |
| `1 2 3 - - - 7 8 9` | 15 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 - - - 7 8 9` | 15 | 8 | [8] | [1, 7] | [8] |
| `1 2 3 - - - 7 8 9` | 15 | 9 | [9] | [2, 7] | [9] |
| `1 2 3 - - - 7 8 9` | 15 | 10 | [1, 9] | [1, 2, 7] | [1, 9] |
| `1 2 3 - - - 7 8 9` | 15 | 11 | [2, 9] | [1, 2, 8] | [2, 9] |
| `1 2 3 - - - 7 8 9` | 15 | 12 | [3, 9] | [1, 3, 8] | [3, 9] |
| `1 2 3 4 5 - 7 8 -` | 15 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 4 5 - 7 8 -` | 15 | 4 | [4] | [1, 3] | [4] |
| `1 2 3 4 5 - 7 8 -` | 15 | 5 | [5] | [1, 4] | [5] |
| `1 2 3 4 5 - 7 8 -` | 15 | 6 | [1, 5] | [1, 2, 3] | [1, 5] |
| `1 2 3 4 5 - 7 8 -` | 15 | 7 | [7] | [1, 2, 4] | [7] |
| `1 2 3 4 5 - 7 8 -` | 15 | 8 | [8] | [1, 2, 5] | [8] |
| `1 2 3 4 5 - 7 8 -` | 15 | 9 | [1, 8] | [2, 3, 4] | [1, 8] |
| `1 2 3 4 5 - 7 8 -` | 15 | 10 | [2, 8] | [1, 2, 3, 4] | [2, 8] |
| `1 2 3 4 5 - 7 8 -` | 15 | 11 | [3, 8] | [1, 2, 3, 5] | [3, 8] |
| `1 2 3 4 5 - 7 8 -` | 15 | 12 | [4, 8] | [1, 2, 4, 5] | [4, 8] |
| `1 2 3 4 5 6 - - 9` | 15 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 4 5 6 - - 9` | 15 | 4 | [4] | [1, 3] | [4] |
| `1 2 3 4 5 6 - - 9` | 15 | 5 | [5] | [2, 3] | [5] |
| `1 2 3 4 5 6 - - 9` | 15 | 6 | [6] | [1, 2, 3] | [6] |
| `1 2 3 4 5 6 - - 9` | 15 | 7 | [1, 6] | [1, 2, 4] | [1, 6] |
| `1 2 3 4 5 6 - - 9` | 15 | 8 | [3, 5] | [1, 2, 5] | [2, 6] |
| `1 2 3 4 5 6 - - 9` | 15 | 9 | [9] | [1, 2, 6] | [9] |
| `1 2 3 4 5 6 - - 9` | 15 | 10 | [1, 9] | [1, 2, 3, 4] | [1, 9] |
| `1 2 3 4 5 6 - - 9` | 15 | 11 | [2, 9] | [1, 2, 3, 5] | [2, 9] |
| `1 2 3 4 5 6 - - 9` | 15 | 12 | [3, 9] | [1, 2, 3, 6] | [3, 9] |
| `- - 3 - 5 6 7 8 -` | 16 | 8 | [8] | [3, 5] | [8] |
| `- - 3 - 5 6 7 8 -` | 16 | 11 | [5, 6] | [3, 8] | [3, 8] |
| `- - 3 4 - 6 7 - 9` | 16 | 7 | [7] | [3, 4] | [7] |
| `- - 3 4 - 6 7 - 9` | 16 | 9 | [9] | [3, 6] | [9] |
| `- - 3 4 - 6 7 - 9` | 16 | 10 | [4, 6] | [3, 7] | [3, 7] |
| `- - 3 4 5 - - 8 9` | 16 | 8 | [8] | [3, 5] | [8] |
| `- - 3 4 5 - - 8 9` | 16 | 9 | [9] | [4, 5] | [9] |
| `- - 3 4 5 - - 8 9` | 16 | 12 | [3, 9] | [3, 4, 5] | [3, 9] |
| `- 2 - - 5 6 7 - 9` | 16 | 7 | [7] | [2, 5] | [7] |
| `- 2 - - 5 6 7 - 9` | 16 | 9 | [9] | [2, 7] | [9] |
| `- 2 - - 5 6 7 - 9` | 16 | 11 | [2, 9] | [5, 6] | [2, 9] |
| `- 2 - 4 - 6 - 8 9` | 16 | 6 | [6] | [2, 4] | [6] |
| `- 2 - 4 - 6 - 8 9` | 16 | 8 | [8] | [2, 6] | [8] |
| `- 2 - 4 - 6 - 8 9` | 16 | 10 | [4, 6] | [2, 8] | [2, 8] |
| `- 2 - 4 - 6 - 8 9` | 16 | 12 | [4, 8] | [2, 4, 6] | [4, 8] |
| `- 2 3 - - - 7 8 9` | 16 | 9 | [9] | [2, 7] | [9] |
| `- 2 3 - - - 7 8 9` | 16 | 10 | [3, 7] | [2, 8] | [2, 8] |
| `- 2 3 - - - 7 8 9` | 16 | 11 | [2, 9] | [3, 8] | [2, 9] |
| `- 2 3 - - - 7 8 9` | 16 | 12 | [3, 9] | [2, 3, 7] | [3, 9] |
| `- 2 3 4 5 - 7 8 -` | 16 | 5 | [5] | [2, 3] | [5] |
| `- 2 3 4 5 - 7 8 -` | 16 | 7 | [7] | [2, 5] | [7] |
| `- 2 3 4 5 - 7 8 -` | 16 | 8 | [8] | [3, 5] | [8] |
| `- 2 3 4 5 - 7 8 -` | 16 | 9 | [4, 5] | [2, 3, 4] | [2, 7] |
| `- 2 3 4 5 - 7 8 -` | 16 | 10 | [3, 7] | [2, 3, 5] | [2, 8] |
| `- 2 3 4 5 - 7 8 -` | 16 | 11 | [3, 8] | [2, 4, 5] | [3, 8] |
| `- 2 3 4 5 - 7 8 -` | 16 | 12 | [5, 7] | [2, 3, 7] | [4, 8] |
| `- 2 3 4 5 6 - - 9` | 16 | 5 | [5] | [2, 3] | [5] |
| `- 2 3 4 5 6 - - 9` | 16 | 6 | [6] | [2, 4] | [6] |
| `- 2 3 4 5 6 - - 9` | 16 | 7 | [3, 4] | [2, 5] | [2, 5] |
| `- 2 3 4 5 6 - - 9` | 16 | 8 | [3, 5] | [2, 6] | [2, 6] |
| `- 2 3 4 5 6 - - 9` | 16 | 9 | [9] | [2, 3, 4] | [9] |
| `- 2 3 4 5 6 - - 9` | 16 | 10 | [4, 6] | [2, 3, 5] | [4, 6] |
| `- 2 3 4 5 6 - - 9` | 16 | 11 | [2, 9] | [2, 3, 6] | [2, 9] |
| `- 2 3 4 5 6 - - 9` | 16 | 12 | [3, 9] | [2, 4, 6] | [3, 9] |
| `1 - - - 5 6 - 8 9` | 16 | 6 | [6] | [1, 5] | [6] |
| `1 - - - 5 6 - 8 9` | 16 | 9 | [9] | [1, 8] | [9] |
| `1 - - 4 - - 7 8 9` | 16 | 8 | [8] | [1, 7] | [8] |
| `1 - - 4 - - 7 8 9` | 16 | 9 | [9] | [1, 8] | [9] |
| `1 - - 4 - - 7 8 9` | 16 | 12 | [4, 8] | [1, 4, 7] | [4, 8] |
| `1 - 3 4 - 6 7 8 -` | 16 | 4 | [4] | [1, 3] | [4] |
| `1 - 3 4 - 6 7 8 -` | 16 | 7 | [7] | [3, 4] | [7] |
| `1 - 3 4 - 6 7 8 -` | 16 | 8 | [8] | [1, 3, 4] | [8] |
| `1 - 3 4 - 6 7 8 -` | 16 | 9 | [1, 8] | [3, 6] | [1, 8] |
| `1 - 3 4 - 6 7 8 -` | 16 | 10 | [3, 7] | [1, 3, 6] | [3, 7] |
| `1 - 3 4 - 6 7 8 -` | 16 | 11 | [4, 7] | [1, 3, 7] | [3, 8] |
| `1 - 3 4 - 6 7 8 -` | 16 | 12 | [4, 8] | [1, 4, 7] | [4, 8] |
| `1 - 3 4 5 - 7 - 9` | 16 | 4 | [4] | [1, 3] | [4] |
| `1 - 3 4 5 - 7 - 9` | 16 | 5 | [5] | [1, 4] | [5] |
| `1 - 3 4 5 - 7 - 9` | 16 | 7 | [7] | [3, 4] | [7] |
| `1 - 3 4 5 - 7 - 9` | 16 | 8 | [3, 5] | [1, 3, 4] | [1, 7] |
| `1 - 3 4 5 - 7 - 9` | 16 | 9 | [9] | [1, 3, 5] | [9] |
| `1 - 3 4 5 - 7 - 9` | 16 | 10 | [1, 9] | [1, 4, 5] | [1, 9] |
| `1 - 3 4 5 - 7 - 9` | 16 | 11 | [4, 7] | [1, 3, 7] | [4, 7] |
| `1 - 3 4 5 - 7 - 9` | 16 | 12 | [3, 9] | [1, 4, 7] | [3, 9] |
| `1 2 - - 5 6 7 8 -` | 16 | 6 | [6] | [1, 5] | [6] |
| `1 2 - - 5 6 7 8 -` | 16 | 7 | [7] | [2, 5] | [7] |
| `1 2 - - 5 6 7 8 -` | 16 | 8 | [8] | [1, 2, 5] | [8] |
| `1 2 - - 5 6 7 8 -` | 16 | 9 | [1, 8] | [1, 2, 6] | [1, 8] |
| `1 2 - - 5 6 7 8 -` | 16 | 10 | [2, 8] | [1, 2, 7] | [2, 8] |
| `1 2 - - 5 6 7 8 -` | 16 | 11 | [5, 6] | [1, 2, 8] | [5, 6] |
| `1 2 - - 5 6 7 8 -` | 16 | 12 | [5, 7] | [1, 5, 6] | [5, 7] |
| `1 2 - 4 - 6 7 - 9` | 16 | 6 | [6] | [2, 4] | [6] |
| `1 2 - 4 - 6 7 - 9` | 16 | 7 | [7] | [1, 2, 4] | [7] |
| `1 2 - 4 - 6 7 - 9` | 16 | 8 | [2, 6] | [1, 7] | [1, 7] |
| `1 2 - 4 - 6 7 - 9` | 16 | 9 | [9] | [1, 2, 6] | [9] |
| `1 2 - 4 - 6 7 - 9` | 16 | 10 | [1, 9] | [1, 2, 7] | [1, 9] |
| `1 2 - 4 - 6 7 - 9` | 16 | 11 | [2, 9] | [1, 4, 6] | [2, 9] |
| `1 2 - 4 - 6 7 - 9` | 16 | 12 | [2, 4, 6] | [1, 4, 7] | [1, 2, 9] |
| `1 2 - 4 5 - - 8 9` | 16 | 5 | [5] | [1, 4] | [5] |
| `1 2 - 4 5 - - 8 9` | 16 | 6 | [1, 5] | [2, 4] | [1, 5] |
| `1 2 - 4 5 - - 8 9` | 16 | 7 | [2, 5] | [1, 2, 4] | [2, 5] |
| `1 2 - 4 5 - - 8 9` | 16 | 8 | [8] | [1, 2, 5] | [8] |
| `1 2 - 4 5 - - 8 9` | 16 | 9 | [9] | [4, 5] | [9] |
| `1 2 - 4 5 - - 8 9` | 16 | 10 | [1, 9] | [1, 4, 5] | [1, 9] |
| `1 2 - 4 5 - - 8 9` | 16 | 11 | [2, 9] | [1, 2, 8] | [2, 9] |
| `1 2 - 4 5 - - 8 9` | 16 | 12 | [4, 8] | [1, 2, 4, 5] | [4, 8] |
| `1 2 3 - - 6 - 8 9` | 16 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 - - 6 - 8 9` | 16 | 6 | [6] | [1, 2, 3] | [6] |
| `1 2 3 - - 6 - 8 9` | 16 | 8 | [8] | [2, 6] | [8] |
| `1 2 3 - - 6 - 8 9` | 16 | 9 | [9] | [1, 2, 6] | [9] |
| `1 2 3 - - 6 - 8 9` | 16 | 10 | [1, 9] | [1, 3, 6] | [1, 9] |
| `1 2 3 - - 6 - 8 9` | 16 | 11 | [2, 9] | [1, 2, 8] | [2, 9] |
| `1 2 3 - - 6 - 8 9` | 16 | 12 | [3, 9] | [1, 2, 3, 6] | [3, 9] |
| `1 2 3 4 5 6 - 8 -` | 16 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 4 5 6 - 8 -` | 16 | 4 | [4] | [1, 3] | [4] |
| `1 2 3 4 5 6 - 8 -` | 16 | 5 | [5] | [1, 4] | [5] |
| `1 2 3 4 5 6 - 8 -` | 16 | 6 | [6] | [1, 2, 3] | [6] |
| `1 2 3 4 5 6 - 8 -` | 16 | 7 | [1, 6] | [1, 2, 4] | [1, 6] |
| `1 2 3 4 5 6 - 8 -` | 16 | 8 | [8] | [1, 3, 4] | [8] |
| `1 2 3 4 5 6 - 8 -` | 16 | 9 | [1, 8] | [1, 3, 5] | [1, 8] |
| `1 2 3 4 5 6 - 8 -` | 16 | 10 | [2, 8] | [1, 2, 3, 4] | [2, 8] |
| `1 2 3 4 5 6 - 8 -` | 16 | 11 | [3, 8] | [1, 2, 3, 5] | [3, 8] |
| `1 2 3 4 5 6 - 8 -` | 16 | 12 | [4, 8] | [1, 2, 3, 6] | [4, 8] |
| `- - 3 4 - 6 7 8 -` | 17 | 7 | [7] | [3, 4] | [7] |
| `- - 3 4 - 6 7 8 -` | 17 | 10 | [4, 6] | [3, 7] | [3, 7] |
| `- - 3 4 - 6 7 8 -` | 17 | 11 | [3, 8] | [4, 7] | [3, 8] |
| `- - 3 4 5 - 7 - 9` | 17 | 7 | [7] | [3, 4] | [7] |
| `- - 3 4 5 - 7 - 9` | 17 | 9 | [9] | [4, 5] | [9] |
| `- - 3 4 5 - 7 - 9` | 17 | 12 | [3, 9] | [3, 4, 5] | [3, 9] |
| `- 2 - - 5 6 7 8 -` | 17 | 7 | [7] | [2, 5] | [7] |
| `- 2 - - 5 6 7 8 -` | 17 | 8 | [8] | [2, 6] | [8] |
| `- 2 - 4 - 6 7 - 9` | 17 | 6 | [6] | [2, 4] | [6] |
| `- 2 - 4 - 6 7 - 9` | 17 | 9 | [9] | [2, 7] | [9] |
| `- 2 - 4 - 6 7 - 9` | 17 | 11 | [2, 9] | [4, 7] | [2, 9] |
| `- 2 - 4 5 - - 8 9` | 17 | 9 | [9] | [4, 5] | [9] |
| `- 2 - 4 5 - - 8 9` | 17 | 11 | [2, 9] | [2, 4, 5] | [2, 9] |
| `- 2 3 - - 6 - 8 9` | 17 | 8 | [8] | [2, 6] | [8] |
| `- 2 3 - - 6 - 8 9` | 17 | 9 | [9] | [3, 6] | [9] |
| `- 2 3 - - 6 - 8 9` | 17 | 11 | [2, 9] | [2, 3, 6] | [2, 9] |
| `- 2 3 4 5 6 - 8 -` | 17 | 5 | [5] | [2, 3] | [5] |
| `- 2 3 4 5 6 - 8 -` | 17 | 6 | [6] | [2, 4] | [6] |
| `- 2 3 4 5 6 - 8 -` | 17 | 7 | [2, 5] | [3, 4] | [2, 5] |
| `- 2 3 4 5 6 - 8 -` | 17 | 8 | [8] | [3, 5] | [8] |
| `- 2 3 4 5 6 - 8 -` | 17 | 9 | [3, 6] | [2, 3, 4] | [3, 6] |
| `- 2 3 4 5 6 - 8 -` | 17 | 10 | [2, 8] | [2, 3, 5] | [2, 8] |
| `- 2 3 4 5 6 - 8 -` | 17 | 11 | [3, 8] | [2, 3, 6] | [3, 8] |
| `- 2 3 4 5 6 - 8 -` | 17 | 12 | [4, 8] | [2, 4, 6] | [4, 8] |
| `1 - - - 5 6 7 - 9` | 17 | 6 | [6] | [1, 5] | [6] |
| `1 - - - 5 6 7 - 9` | 17 | 7 | [7] | [1, 6] | [7] |
| `1 - - - 5 6 7 - 9` | 17 | 12 | [5, 7] | [1, 5, 6] | [5, 7] |
| `1 - - 4 - 6 - 8 9` | 17 | 9 | [9] | [1, 8] | [9] |
| `1 - - 4 - 6 - 8 9` | 17 | 10 | [1, 9] | [4, 6] | [1, 9] |
| `1 - 3 - - - 7 8 9` | 17 | 8 | [8] | [1, 7] | [8] |
| `1 - 3 - - - 7 8 9` | 17 | 9 | [9] | [1, 8] | [9] |
| `1 - 3 - - - 7 8 9` | 17 | 10 | [1, 9] | [3, 7] | [1, 9] |
| `1 - 3 - - - 7 8 9` | 17 | 11 | [3, 8] | [1, 3, 7] | [3, 8] |
| `1 - 3 - - - 7 8 9` | 17 | 12 | [3, 9] | [1, 3, 8] | [3, 9] |
| `1 - 3 4 5 - 7 8 -` | 17 | 4 | [4] | [1, 3] | [4] |
| `1 - 3 4 5 - 7 8 -` | 17 | 5 | [5] | [1, 4] | [5] |
| `1 - 3 4 5 - 7 8 -` | 17 | 7 | [7] | [3, 4] | [7] |
| `1 - 3 4 5 - 7 8 -` | 17 | 8 | [8] | [1, 3, 4] | [8] |
| `1 - 3 4 5 - 7 8 -` | 17 | 9 | [1, 8] | [1, 3, 5] | [1, 8] |
| `1 - 3 4 5 - 7 8 -` | 17 | 10 | [3, 7] | [1, 4, 5] | [3, 7] |
| `1 - 3 4 5 - 7 8 -` | 17 | 11 | [3, 8] | [1, 3, 7] | [3, 8] |
| `1 - 3 4 5 - 7 8 -` | 17 | 12 | [4, 8] | [1, 4, 7] | [4, 8] |
| `1 - 3 4 5 6 - - 9` | 17 | 4 | [4] | [1, 3] | [4] |
| `1 - 3 4 5 6 - - 9` | 17 | 5 | [5] | [1, 4] | [5] |
| `1 - 3 4 5 6 - - 9` | 17 | 6 | [6] | [1, 5] | [6] |
| `1 - 3 4 5 6 - - 9` | 17 | 7 | [3, 4] | [1, 6] | [1, 6] |
| `1 - 3 4 5 6 - - 9` | 17 | 8 | [3, 5] | [1, 3, 4] | [3, 5] |
| `1 - 3 4 5 6 - - 9` | 17 | 9 | [9] | [1, 3, 5] | [9] |
| `1 - 3 4 5 6 - - 9` | 17 | 10 | [1, 9] | [1, 3, 6] | [1, 9] |
| `1 - 3 4 5 6 - - 9` | 17 | 11 | [5, 6] | [1, 4, 6] | [5, 6] |
| `1 - 3 4 5 6 - - 9` | 17 | 12 | [3, 9] | [1, 5, 6] | [3, 9] |
| `1 2 - 4 - 6 7 8 -` | 17 | 6 | [6] | [2, 4] | [6] |
| `1 2 - 4 - 6 7 8 -` | 17 | 7 | [7] | [1, 2, 4] | [7] |
| `1 2 - 4 - 6 7 8 -` | 17 | 8 | [8] | [1, 7] | [8] |
| `1 2 - 4 - 6 7 8 -` | 17 | 9 | [2, 7] | [1, 2, 6] | [1, 8] |
| `1 2 - 4 - 6 7 8 -` | 17 | 10 | [2, 8] | [1, 2, 7] | [2, 8] |
| `1 2 - 4 - 6 7 8 -` | 17 | 11 | [4, 7] | [1, 2, 8] | [4, 7] |
| `1 2 - 4 - 6 7 8 -` | 17 | 12 | [4, 8] | [1, 4, 7] | [4, 8] |
| `1 2 - 4 5 - 7 - 9` | 17 | 5 | [5] | [1, 4] | [5] |
| `1 2 - 4 5 - 7 - 9` | 17 | 6 | [2, 4] | [1, 5] | [1, 5] |
| `1 2 - 4 5 - 7 - 9` | 17 | 7 | [7] | [1, 2, 4] | [7] |
| `1 2 - 4 5 - 7 - 9` | 17 | 8 | [1, 7] | [1, 2, 5] | [1, 7] |
| `1 2 - 4 5 - 7 - 9` | 17 | 9 | [9] | [2, 7] | [9] |
| `1 2 - 4 5 - 7 - 9` | 17 | 10 | [1, 9] | [1, 2, 7] | [1, 9] |
| `1 2 - 4 5 - 7 - 9` | 17 | 11 | [2, 9] | [2, 4, 5] | [2, 9] |
| `1 2 - 4 5 - 7 - 9` | 17 | 12 | [5, 7] | [1, 2, 4, 5] | [5, 7] |
| `1 2 3 - - 6 7 - 9` | 17 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 - - 6 7 - 9` | 17 | 6 | [6] | [1, 2, 3] | [6] |
| `1 2 3 - - 6 7 - 9` | 17 | 7 | [7] | [1, 6] | [7] |
| `1 2 3 - - 6 7 - 9` | 17 | 8 | [2, 6] | [1, 7] | [1, 7] |
| `1 2 3 - - 6 7 - 9` | 17 | 9 | [9] | [1, 2, 6] | [9] |
| `1 2 3 - - 6 7 - 9` | 17 | 10 | [1, 9] | [1, 2, 7] | [1, 9] |
| `1 2 3 - - 6 7 - 9` | 17 | 11 | [2, 9] | [1, 3, 7] | [2, 9] |
| `1 2 3 - - 6 7 - 9` | 17 | 12 | [3, 9] | [1, 2, 3, 6] | [3, 9] |
| `1 2 3 - 5 - - 8 9` | 17 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 - 5 - - 8 9` | 17 | 5 | [5] | [2, 3] | [5] |
| `1 2 3 - 5 - - 8 9` | 17 | 6 | [1, 5] | [1, 2, 3] | [1, 5] |
| `1 2 3 - 5 - - 8 9` | 17 | 8 | [8] | [1, 2, 5] | [8] |
| `1 2 3 - 5 - - 8 9` | 17 | 9 | [9] | [1, 3, 5] | [9] |
| `1 2 3 - 5 - - 8 9` | 17 | 10 | [2, 8] | [2, 3, 5] | [1, 9] |
| `1 2 3 - 5 - - 8 9` | 17 | 11 | [3, 8] | [1, 2, 3, 5] | [2, 9] |
| `1 2 3 - 5 - - 8 9` | 17 | 12 | [3, 9] | [1, 2, 9] | [3, 9] |
| `1 2 3 4 5 6 7 - -` | 17 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 4 5 6 7 - -` | 17 | 4 | [4] | [1, 3] | [4] |
| `1 2 3 4 5 6 7 - -` | 17 | 5 | [5] | [2, 3] | [5] |
| `1 2 3 4 5 6 7 - -` | 17 | 6 | [6] | [1, 2, 3] | [6] |
| `1 2 3 4 5 6 7 - -` | 17 | 7 | [7] | [1, 2, 4] | [7] |
| `1 2 3 4 5 6 7 - -` | 17 | 8 | [1, 7] | [1, 2, 5] | [1, 7] |
| `1 2 3 4 5 6 7 - -` | 17 | 9 | [3, 6] | [2, 3, 4] | [2, 7] |
| `1 2 3 4 5 6 7 - -` | 17 | 10 | [3, 7] | [1, 2, 3, 4] | [3, 7] |
| `1 2 3 4 5 6 7 - -` | 17 | 11 | [4, 7] | [1, 2, 3, 5] | [4, 7] |
| `1 2 3 4 5 6 7 - -` | 17 | 12 | [5, 7] | [1, 2, 3, 6] | [5, 7] |
| `- - 3 4 5 - 7 8 -` | 18 | 7 | [7] | [3, 4] | [7] |
| `- - 3 4 5 - 7 8 -` | 18 | 8 | [8] | [3, 5] | [8] |
| `- - 3 4 5 - 7 8 -` | 18 | 11 | [3, 8] | [4, 7] | [3, 8] |
| `- - 3 4 5 - 7 8 -` | 18 | 12 | [4, 8] | [3, 4, 5] | [4, 8] |
| `- - 3 4 5 6 - - 9` | 18 | 9 | [9] | [3, 6] | [9] |
| `- - 3 4 5 6 - - 9` | 18 | 12 | [3, 9] | [3, 4, 5] | [3, 9] |
| `- 2 - 4 - 6 7 8 -` | 18 | 6 | [6] | [2, 4] | [6] |
| `- 2 - 4 - 6 7 8 -` | 18 | 8 | [8] | [2, 6] | [8] |
| `- 2 - 4 - 6 7 8 -` | 18 | 10 | [4, 6] | [2, 8] | [2, 8] |
| `- 2 - 4 - 6 7 8 -` | 18 | 12 | [4, 8] | [2, 4, 6] | [4, 8] |
| `- 2 - 4 5 - 7 - 9` | 18 | 7 | [7] | [2, 5] | [7] |
| `- 2 - 4 5 - 7 - 9` | 18 | 9 | [9] | [2, 7] | [9] |
| `- 2 - 4 5 - 7 - 9` | 18 | 11 | [2, 9] | [2, 4, 5] | [2, 9] |
| `- 2 3 - - 6 7 - 9` | 18 | 9 | [9] | [2, 7] | [9] |
| `- 2 3 - - 6 7 - 9` | 18 | 11 | [2, 9] | [2, 3, 6] | [2, 9] |
| `- 2 3 - - 6 7 - 9` | 18 | 12 | [3, 9] | [2, 3, 7] | [3, 9] |
| `- 2 3 - 5 - - 8 9` | 18 | 5 | [5] | [2, 3] | [5] |
| `- 2 3 - 5 - - 8 9` | 18 | 8 | [8] | [3, 5] | [8] |
| `- 2 3 - 5 - - 8 9` | 18 | 10 | [2, 8] | [2, 3, 5] | [2, 8] |
| `- 2 3 - 5 - - 8 9` | 18 | 11 | [3, 8] | [2, 9] | [2, 9] |
| `- 2 3 4 5 6 7 - -` | 18 | 5 | [5] | [2, 3] | [5] |
| `- 2 3 4 5 6 7 - -` | 18 | 6 | [6] | [2, 4] | [6] |
| `- 2 3 4 5 6 7 - -` | 18 | 7 | [7] | [2, 5] | [7] |
| `- 2 3 4 5 6 7 - -` | 18 | 8 | [3, 5] | [2, 6] | [2, 6] |
| `- 2 3 4 5 6 7 - -` | 18 | 9 | [2, 7] | [2, 3, 4] | [2, 7] |
| `- 2 3 4 5 6 7 - -` | 18 | 10 | [3, 7] | [2, 3, 5] | [3, 7] |
| `- 2 3 4 5 6 7 - -` | 18 | 11 | [4, 7] | [2, 3, 6] | [4, 7] |
| `- 2 3 4 5 6 7 - -` | 18 | 12 | [5, 7] | [2, 3, 7] | [5, 7] |
| `1 - - - 5 6 7 8 -` | 18 | 6 | [6] | [1, 5] | [6] |
| `1 - - - 5 6 7 8 -` | 18 | 7 | [7] | [1, 6] | [7] |
| `1 - - - 5 6 7 8 -` | 18 | 8 | [8] | [1, 7] | [8] |
| `1 - - - 5 6 7 8 -` | 18 | 12 | [5, 7] | [1, 5, 6] | [5, 7] |
| `1 - - 4 - 6 7 - 9` | 18 | 7 | [7] | [1, 6] | [7] |
| `1 - - 4 - 6 7 - 9` | 18 | 10 | [4, 6] | [1, 9] | [1, 9] |
| `1 - - 4 - 6 7 - 9` | 18 | 11 | [4, 7] | [1, 4, 6] | [4, 7] |
| `1 - - 4 5 - - 8 9` | 18 | 5 | [5] | [1, 4] | [5] |
| `1 - - 4 5 - - 8 9` | 18 | 9 | [9] | [1, 8] | [9] |
| `1 - - 4 5 - - 8 9` | 18 | 10 | [1, 9] | [1, 4, 5] | [1, 9] |
| `1 - 3 - - 6 - 8 9` | 18 | 9 | [9] | [1, 8] | [9] |
| `1 - 3 - - 6 - 8 9` | 18 | 10 | [1, 9] | [1, 3, 6] | [1, 9] |
| `1 - 3 - - 6 - 8 9` | 18 | 12 | [3, 9] | [1, 3, 8] | [3, 9] |
| `1 - 3 4 5 6 - 8 -` | 18 | 4 | [4] | [1, 3] | [4] |
| `1 - 3 4 5 6 - 8 -` | 18 | 5 | [5] | [1, 4] | [5] |
| `1 - 3 4 5 6 - 8 -` | 18 | 6 | [6] | [1, 5] | [6] |
| `1 - 3 4 5 6 - 8 -` | 18 | 7 | [3, 4] | [1, 6] | [1, 6] |
| `1 - 3 4 5 6 - 8 -` | 18 | 8 | [8] | [1, 3, 4] | [8] |
| `1 - 3 4 5 6 - 8 -` | 18 | 9 | [1, 8] | [1, 3, 5] | [1, 8] |
| `1 - 3 4 5 6 - 8 -` | 18 | 10 | [4, 6] | [1, 3, 6] | [4, 6] |
| `1 - 3 4 5 6 - 8 -` | 18 | 11 | [3, 8] | [1, 4, 6] | [3, 8] |
| `1 - 3 4 5 6 - 8 -` | 18 | 12 | [4, 8] | [1, 5, 6] | [4, 8] |
| `1 2 - - - - 7 8 9` | 18 | 8 | [8] | [1, 7] | [8] |
| `1 2 - - - - 7 8 9` | 18 | 9 | [9] | [1, 8] | [9] |
| `1 2 - - - - 7 8 9` | 18 | 10 | [1, 9] | [1, 2, 7] | [1, 9] |
| `1 2 - - - - 7 8 9` | 18 | 11 | [2, 9] | [1, 2, 8] | [2, 9] |
| `1 2 - 4 5 - 7 8 -` | 18 | 5 | [5] | [1, 4] | [5] |
| `1 2 - 4 5 - 7 8 -` | 18 | 6 | [1, 5] | [2, 4] | [1, 5] |
| `1 2 - 4 5 - 7 8 -` | 18 | 7 | [7] | [1, 2, 4] | [7] |
| `1 2 - 4 5 - 7 8 -` | 18 | 8 | [8] | [1, 2, 5] | [8] |
| `1 2 - 4 5 - 7 8 -` | 18 | 9 | [1, 8] | [4, 5] | [1, 8] |
| `1 2 - 4 5 - 7 8 -` | 18 | 10 | [2, 8] | [1, 2, 7] | [2, 8] |
| `1 2 - 4 5 - 7 8 -` | 18 | 11 | [4, 7] | [2, 4, 5] | [4, 7] |
| `1 2 - 4 5 - 7 8 -` | 18 | 12 | [5, 7] | [1, 2, 4, 5] | [4, 8] |
| `1 2 - 4 5 6 - - 9` | 18 | 5 | [5] | [1, 4] | [5] |
| `1 2 - 4 5 6 - - 9` | 18 | 6 | [6] | [1, 5] | [6] |
| `1 2 - 4 5 6 - - 9` | 18 | 7 | [2, 5] | [1, 2, 4] | [1, 6] |
| `1 2 - 4 5 6 - - 9` | 18 | 8 | [2, 6] | [1, 2, 5] | [2, 6] |
| `1 2 - 4 5 6 - - 9` | 18 | 9 | [9] | [1, 2, 6] | [9] |
| `1 2 - 4 5 6 - - 9` | 18 | 10 | [1, 9] | [1, 4, 5] | [1, 9] |
| `1 2 - 4 5 6 - - 9` | 18 | 11 | [2, 9] | [1, 4, 6] | [2, 9] |
| `1 2 - 4 5 6 - - 9` | 18 | 12 | [1, 2, 9] | [1, 2, 4, 5] | [1, 2, 9] |
| `1 2 3 - - 6 7 8 -` | 18 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 - - 6 7 8 -` | 18 | 6 | [6] | [1, 2, 3] | [6] |
| `1 2 3 - - 6 7 8 -` | 18 | 7 | [7] | [1, 6] | [7] |
| `1 2 3 - - 6 7 8 -` | 18 | 8 | [8] | [1, 7] | [8] |
| `1 2 3 - - 6 7 8 -` | 18 | 9 | [1, 8] | [1, 2, 6] | [1, 8] |
| `1 2 3 - - 6 7 8 -` | 18 | 10 | [2, 8] | [1, 2, 7] | [2, 8] |
| `1 2 3 - - 6 7 8 -` | 18 | 11 | [3, 8] | [1, 3, 7] | [3, 8] |
| `1 2 3 - - 6 7 8 -` | 18 | 12 | [1, 3, 8] | [1, 2, 3, 6] | [1, 3, 8] |
| `1 2 3 - 5 - 7 - 9` | 18 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 - 5 - 7 - 9` | 18 | 5 | [5] | [2, 3] | [5] |
| `1 2 3 - 5 - 7 - 9` | 18 | 6 | [1, 5] | [1, 2, 3] | [1, 5] |
| `1 2 3 - 5 - 7 - 9` | 18 | 7 | [7] | [2, 5] | [7] |
| `1 2 3 - 5 - 7 - 9` | 18 | 8 | [1, 7] | [1, 2, 5] | [1, 7] |
| `1 2 3 - 5 - 7 - 9` | 18 | 9 | [9] | [1, 3, 5] | [9] |
| `1 2 3 - 5 - 7 - 9` | 18 | 10 | [3, 7] | [1, 2, 7] | [1, 9] |
| `1 2 3 - 5 - 7 - 9` | 18 | 11 | [2, 9] | [1, 2, 3, 5] | [2, 9] |
| `1 2 3 - 5 - 7 - 9` | 18 | 12 | [3, 9] | [2, 3, 7] | [3, 9] |
| `1 2 3 4 - - - 8 9` | 18 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 4 - - - 8 9` | 18 | 4 | [4] | [1, 3] | [4] |
| `1 2 3 4 - - - 8 9` | 18 | 5 | [1, 4] | [2, 3] | [1, 4] |
| `1 2 3 4 - - - 8 9` | 18 | 6 | [2, 4] | [1, 2, 3] | [2, 4] |
| `1 2 3 4 - - - 8 9` | 18 | 7 | [3, 4] | [1, 2, 4] | [3, 4] |
| `1 2 3 4 - - - 8 9` | 18 | 8 | [8] | [1, 3, 4] | [8] |
| `1 2 3 4 - - - 8 9` | 18 | 9 | [9] | [2, 3, 4] | [9] |
| `1 2 3 4 - - - 8 9` | 18 | 10 | [1, 9] | [1, 2, 3, 4] | [1, 9] |
| `1 2 3 4 - - - 8 9` | 18 | 11 | [3, 8] | [1, 2, 8] | [2, 9] |
| `1 2 3 4 - - - 8 9` | 18 | 12 | [3, 9] | [1, 3, 8] | [3, 9] |
| `- - - 4 5 - - 8 9` | 19 | 9 | [9] | [4, 5] | [9] |
| `- - 3 - - 6 - 8 9` | 19 | 9 | [9] | [3, 6] | [9] |
| `- - 3 4 5 6 - 8 -` | 19 | 8 | [8] | [3, 5] | [8] |
| `- - 3 4 5 6 - 8 -` | 19 | 9 | [4, 5] | [3, 6] | [3, 6] |
| `- - 3 4 5 6 - 8 -` | 19 | 11 | [3, 8] | [5, 6] | [3, 8] |
| `- - 3 4 5 6 - 8 -` | 19 | 12 | [4, 8] | [3, 4, 5] | [4, 8] |
| `- 2 - - - - 7 8 9` | 19 | 9 | [9] | [2, 7] | [9] |
| `- 2 - 4 5 - 7 8 -` | 19 | 7 | [7] | [2, 5] | [7] |
| `- 2 - 4 5 - 7 8 -` | 19 | 9 | [4, 5] | [2, 7] | [2, 7] |
| `- 2 - 4 5 - 7 8 -` | 19 | 11 | [4, 7] | [2, 4, 5] | [4, 7] |
| `- 2 - 4 5 - 7 8 -` | 19 | 12 | [5, 7] | [4, 8] | [4, 8] |
| `- 2 - 4 5 6 - - 9` | 19 | 6 | [6] | [2, 4] | [6] |
| `- 2 - 4 5 6 - - 9` | 19 | 9 | [9] | [4, 5] | [9] |
| `- 2 - 4 5 6 - - 9` | 19 | 11 | [2, 9] | [2, 4, 5] | [2, 9] |
| `- 2 3 - - 6 7 8 -` | 19 | 8 | [8] | [2, 6] | [8] |
| `- 2 3 - - 6 7 8 -` | 19 | 9 | [3, 6] | [2, 7] | [2, 7] |
| `- 2 3 - - 6 7 8 -` | 19 | 10 | [2, 8] | [3, 7] | [2, 8] |
| `- 2 3 - - 6 7 8 -` | 19 | 11 | [3, 8] | [2, 3, 6] | [3, 8] |
| `- 2 3 - 5 - 7 - 9` | 19 | 5 | [5] | [2, 3] | [5] |
| `- 2 3 - 5 - 7 - 9` | 19 | 7 | [7] | [2, 5] | [7] |
| `- 2 3 - 5 - 7 - 9` | 19 | 9 | [9] | [2, 7] | [9] |
| `- 2 3 - 5 - 7 - 9` | 19 | 10 | [3, 7] | [2, 3, 5] | [3, 7] |
| `- 2 3 - 5 - 7 - 9` | 19 | 12 | [3, 9] | [2, 3, 7] | [3, 9] |
| `- 2 3 4 - - - 8 9` | 19 | 9 | [9] | [2, 3, 4] | [9] |
| `- 2 3 4 - - - 8 9` | 19 | 11 | [2, 9] | [3, 8] | [2, 9] |
| `- 2 3 4 - - - 8 9` | 19 | 12 | [3, 9] | [4, 8] | [3, 9] |
| `1 - - 4 - 6 7 8 -` | 19 | 7 | [7] | [1, 6] | [7] |
| `1 - - 4 - 6 7 8 -` | 19 | 8 | [8] | [1, 7] | [8] |
| `1 - - 4 - 6 7 8 -` | 19 | 11 | [4, 7] | [1, 4, 6] | [4, 7] |
| `1 - - 4 - 6 7 8 -` | 19 | 12 | [4, 8] | [1, 4, 7] | [4, 8] |
| `1 - - 4 5 - 7 - 9` | 19 | 5 | [5] | [1, 4] | [5] |
| `1 - - 4 5 - 7 - 9` | 19 | 9 | [9] | [4, 5] | [9] |
| `1 - - 4 5 - 7 - 9` | 19 | 10 | [1, 9] | [1, 4, 5] | [1, 9] |
| `1 - - 4 5 - 7 - 9` | 19 | 12 | [5, 7] | [1, 4, 7] | [5, 7] |
| `1 - 3 - - 6 7 - 9` | 19 | 7 | [7] | [1, 6] | [7] |
| `1 - 3 - - 6 7 - 9` | 19 | 9 | [9] | [3, 6] | [9] |
| `1 - 3 - - 6 7 - 9` | 19 | 10 | [1, 9] | [1, 3, 6] | [1, 9] |
| `1 - 3 - 5 - - 8 9` | 19 | 8 | [8] | [3, 5] | [8] |
| `1 - 3 - 5 - - 8 9` | 19 | 9 | [9] | [1, 3, 5] | [9] |
| `1 - 3 - 5 - - 8 9` | 19 | 12 | [3, 9] | [1, 3, 8] | [3, 9] |
| `1 - 3 4 5 6 7 - -` | 19 | 4 | [4] | [1, 3] | [4] |
| `1 - 3 4 5 6 7 - -` | 19 | 5 | [5] | [1, 4] | [5] |
| `1 - 3 4 5 6 7 - -` | 19 | 6 | [6] | [1, 5] | [6] |
| `1 - 3 4 5 6 7 - -` | 19 | 7 | [7] | [3, 4] | [7] |
| `1 - 3 4 5 6 7 - -` | 19 | 8 | [1, 7] | [1, 3, 4] | [1, 7] |
| `1 - 3 4 5 6 7 - -` | 19 | 9 | [3, 6] | [1, 3, 5] | [3, 6] |
| `1 - 3 4 5 6 7 - -` | 19 | 10 | [4, 6] | [1, 3, 6] | [3, 7] |
| `1 - 3 4 5 6 7 - -` | 19 | 11 | [4, 7] | [1, 3, 7] | [4, 7] |
| `1 - 3 4 5 6 7 - -` | 19 | 12 | [5, 7] | [1, 5, 6] | [5, 7] |
| `1 2 - - - 6 - 8 9` | 19 | 8 | [8] | [2, 6] | [8] |
| `1 2 - - - 6 - 8 9` | 19 | 9 | [9] | [1, 2, 6] | [9] |
| `1 2 - - - 6 - 8 9` | 19 | 10 | [2, 8] | [1, 9] | [1, 9] |
| `1 2 - - - 6 - 8 9` | 19 | 11 | [2, 9] | [1, 2, 8] | [2, 9] |
| `1 2 - 4 5 6 - 8 -` | 19 | 5 | [5] | [1, 4] | [5] |
| `1 2 - 4 5 6 - 8 -` | 19 | 6 | [6] | [1, 5] | [6] |
| `1 2 - 4 5 6 - 8 -` | 19 | 7 | [1, 6] | [1, 2, 4] | [1, 6] |
| `1 2 - 4 5 6 - 8 -` | 19 | 8 | [8] | [1, 2, 5] | [8] |
| `1 2 - 4 5 6 - 8 -` | 19 | 9 | [1, 8] | [1, 2, 6] | [1, 8] |
| `1 2 - 4 5 6 - 8 -` | 19 | 10 | [4, 6] | [1, 4, 5] | [2, 8] |
| `1 2 - 4 5 6 - 8 -` | 19 | 11 | [5, 6] | [1, 4, 6] | [5, 6] |
| `1 2 - 4 5 6 - 8 -` | 19 | 12 | [4, 8] | [1, 2, 4, 5] | [4, 8] |
| `1 2 3 - 5 - 7 8 -` | 19 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 - 5 - 7 8 -` | 19 | 5 | [5] | [2, 3] | [5] |
| `1 2 3 - 5 - 7 8 -` | 19 | 6 | [1, 5] | [1, 2, 3] | [1, 5] |
| `1 2 3 - 5 - 7 8 -` | 19 | 7 | [7] | [2, 5] | [7] |
| `1 2 3 - 5 - 7 8 -` | 19 | 8 | [8] | [1, 2, 5] | [8] |
| `1 2 3 - 5 - 7 8 -` | 19 | 9 | [1, 8] | [1, 3, 5] | [1, 8] |
| `1 2 3 - 5 - 7 8 -` | 19 | 10 | [2, 8] | [1, 2, 7] | [2, 8] |
| `1 2 3 - 5 - 7 8 -` | 19 | 11 | [3, 8] | [1, 2, 3, 5] | [3, 8] |
| `1 2 3 - 5 - 7 8 -` | 19 | 12 | [5, 7] | [1, 3, 8] | [5, 7] |
| `1 2 3 - 5 6 - - 9` | 19 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 - 5 6 - - 9` | 19 | 5 | [5] | [2, 3] | [5] |
| `1 2 3 - 5 6 - - 9` | 19 | 6 | [6] | [1, 2, 3] | [6] |
| `1 2 3 - 5 6 - - 9` | 19 | 7 | [1, 6] | [2, 5] | [1, 6] |
| `1 2 3 - 5 6 - - 9` | 19 | 8 | [3, 5] | [1, 2, 5] | [2, 6] |
| `1 2 3 - 5 6 - - 9` | 19 | 9 | [9] | [1, 2, 6] | [9] |
| `1 2 3 - 5 6 - - 9` | 19 | 10 | [1, 9] | [1, 3, 6] | [1, 9] |
| `1 2 3 - 5 6 - - 9` | 19 | 11 | [2, 9] | [1, 2, 3, 5] | [2, 9] |
| `1 2 3 - 5 6 - - 9` | 19 | 12 | [3, 9] | [1, 2, 3, 6] | [3, 9] |
| `1 2 3 4 - - 7 - 9` | 19 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 4 - - 7 - 9` | 19 | 4 | [4] | [1, 3] | [4] |
| `1 2 3 4 - - 7 - 9` | 19 | 5 | [2, 3] | [1, 4] | [1, 4] |
| `1 2 3 4 - - 7 - 9` | 19 | 6 | [2, 4] | [1, 2, 3] | [2, 4] |
| `1 2 3 4 - - 7 - 9` | 19 | 7 | [7] | [1, 2, 4] | [7] |
| `1 2 3 4 - - 7 - 9` | 19 | 8 | [1, 7] | [1, 3, 4] | [1, 7] |
| `1 2 3 4 - - 7 - 9` | 19 | 9 | [9] | [2, 3, 4] | [9] |
| `1 2 3 4 - - 7 - 9` | 19 | 10 | [1, 9] | [1, 2, 3, 4] | [1, 9] |
| `1 2 3 4 - - 7 - 9` | 19 | 11 | [2, 9] | [1, 3, 7] | [2, 9] |
| `1 2 3 4 - - 7 - 9` | 19 | 12 | [3, 9] | [1, 4, 7] | [3, 9] |
| `- - - 4 5 - 7 - 9` | 20 | 9 | [9] | [4, 5] | [9] |
| `- - 3 - - 6 7 - 9` | 20 | 9 | [9] | [3, 6] | [9] |
| `- - 3 - 5 - - 8 9` | 20 | 8 | [8] | [3, 5] | [8] |
| `- - 3 4 5 6 7 - -` | 20 | 7 | [7] | [3, 4] | [7] |
| `- - 3 4 5 6 7 - -` | 20 | 9 | [4, 5] | [3, 6] | [3, 6] |
| `- - 3 4 5 6 7 - -` | 20 | 10 | [4, 6] | [3, 7] | [3, 7] |
| `- - 3 4 5 6 7 - -` | 20 | 11 | [4, 7] | [5, 6] | [4, 7] |
| `- - 3 4 5 6 7 - -` | 20 | 12 | [5, 7] | [3, 4, 5] | [5, 7] |
| `- 2 - - - 6 - 8 9` | 20 | 8 | [8] | [2, 6] | [8] |
| `- 2 - 4 5 6 - 8 -` | 20 | 6 | [6] | [2, 4] | [6] |
| `- 2 - 4 5 6 - 8 -` | 20 | 8 | [8] | [2, 6] | [8] |
| `- 2 - 4 5 6 - 8 -` | 20 | 10 | [2, 8] | [4, 6] | [2, 8] |
| `- 2 - 4 5 6 - 8 -` | 20 | 11 | [5, 6] | [2, 4, 5] | [5, 6] |
| `- 2 - 4 5 6 - 8 -` | 20 | 12 | [4, 8] | [2, 4, 6] | [4, 8] |
| `- 2 3 - 5 - 7 8 -` | 20 | 5 | [5] | [2, 3] | [5] |
| `- 2 3 - 5 - 7 8 -` | 20 | 7 | [7] | [2, 5] | [7] |
| `- 2 3 - 5 - 7 8 -` | 20 | 8 | [8] | [3, 5] | [8] |
| `- 2 3 - 5 - 7 8 -` | 20 | 10 | [2, 8] | [2, 3, 5] | [2, 8] |
| `- 2 3 - 5 - 7 8 -` | 20 | 12 | [5, 7] | [2, 3, 7] | [5, 7] |
| `- 2 3 - 5 6 - - 9` | 20 | 5 | [5] | [2, 3] | [5] |
| `- 2 3 - 5 6 - - 9` | 20 | 8 | [3, 5] | [2, 6] | [2, 6] |
| `- 2 3 - 5 6 - - 9` | 20 | 9 | [9] | [3, 6] | [9] |
| `- 2 3 - 5 6 - - 9` | 20 | 11 | [2, 9] | [2, 3, 6] | [2, 9] |
| `- 2 3 4 - - 7 - 9` | 20 | 7 | [7] | [3, 4] | [7] |
| `- 2 3 4 - - 7 - 9` | 20 | 9 | [9] | [2, 3, 4] | [9] |
| `- 2 3 4 - - 7 - 9` | 20 | 11 | [2, 9] | [4, 7] | [2, 9] |
| `- 2 3 4 - - 7 - 9` | 20 | 12 | [3, 9] | [2, 3, 7] | [3, 9] |
| `1 - - - - - 7 8 9` | 20 | 8 | [8] | [1, 7] | [8] |
| `1 - - - - - 7 8 9` | 20 | 9 | [9] | [1, 8] | [9] |
| `1 - - 4 5 - 7 8 -` | 20 | 5 | [5] | [1, 4] | [5] |
| `1 - - 4 5 - 7 8 -` | 20 | 8 | [8] | [1, 7] | [8] |
| `1 - - 4 5 - 7 8 -` | 20 | 9 | [1, 8] | [4, 5] | [1, 8] |
| `1 - - 4 5 - 7 8 -` | 20 | 12 | [4, 8] | [1, 4, 7] | [4, 8] |
| `1 - - 4 5 6 - - 9` | 20 | 5 | [5] | [1, 4] | [5] |
| `1 - - 4 5 6 - - 9` | 20 | 6 | [6] | [1, 5] | [6] |
| `1 - - 4 5 6 - - 9` | 20 | 9 | [9] | [4, 5] | [9] |
| `1 - - 4 5 6 - - 9` | 20 | 10 | [1, 9] | [1, 4, 5] | [1, 9] |
| `1 - - 4 5 6 - - 9` | 20 | 11 | [5, 6] | [1, 4, 6] | [5, 6] |
| `1 - 3 - - 6 7 8 -` | 20 | 7 | [7] | [1, 6] | [7] |
| `1 - 3 - - 6 7 8 -` | 20 | 8 | [8] | [1, 7] | [8] |
| `1 - 3 - - 6 7 8 -` | 20 | 9 | [1, 8] | [3, 6] | [1, 8] |
| `1 - 3 - - 6 7 8 -` | 20 | 10 | [3, 7] | [1, 3, 6] | [3, 7] |
| `1 - 3 - - 6 7 8 -` | 20 | 11 | [3, 8] | [1, 3, 7] | [3, 8] |
| `1 - 3 - 5 - 7 - 9` | 20 | 8 | [3, 5] | [1, 7] | [1, 7] |
| `1 - 3 - 5 - 7 - 9` | 20 | 9 | [9] | [1, 3, 5] | [9] |
| `1 - 3 - 5 - 7 - 9` | 20 | 10 | [1, 9] | [3, 7] | [1, 9] |
| `1 - 3 - 5 - 7 - 9` | 20 | 12 | [3, 9] | [5, 7] | [3, 9] |
| `1 - 3 4 - - - 8 9` | 20 | 4 | [4] | [1, 3] | [4] |
| `1 - 3 4 - - - 8 9` | 20 | 8 | [8] | [1, 3, 4] | [8] |
| `1 - 3 4 - - - 8 9` | 20 | 9 | [9] | [1, 8] | [9] |
| `1 - 3 4 - - - 8 9` | 20 | 12 | [3, 9] | [1, 3, 8] | [3, 9] |
| `1 2 - - - 6 7 - 9` | 20 | 7 | [7] | [1, 6] | [7] |
| `1 2 - - - 6 7 - 9` | 20 | 8 | [2, 6] | [1, 7] | [1, 7] |
| `1 2 - - - 6 7 - 9` | 20 | 9 | [9] | [1, 2, 6] | [9] |
| `1 2 - - - 6 7 - 9` | 20 | 10 | [1, 9] | [1, 2, 7] | [1, 9] |
| `1 2 - - 5 - - 8 9` | 20 | 8 | [8] | [1, 2, 5] | [8] |
| `1 2 - - 5 - - 8 9` | 20 | 9 | [9] | [1, 8] | [9] |
| `1 2 - - 5 - - 8 9` | 20 | 10 | [1, 9] | [2, 8] | [1, 9] |
| `1 2 - - 5 - - 8 9` | 20 | 11 | [2, 9] | [1, 2, 8] | [2, 9] |
| `1 2 - 4 5 6 7 - -` | 20 | 5 | [5] | [1, 4] | [5] |
| `1 2 - 4 5 6 7 - -` | 20 | 6 | [6] | [2, 4] | [6] |
| `1 2 - 4 5 6 7 - -` | 20 | 7 | [7] | [1, 2, 4] | [7] |
| `1 2 - 4 5 6 7 - -` | 20 | 8 | [1, 7] | [1, 2, 5] | [1, 7] |
| `1 2 - 4 5 6 7 - -` | 20 | 9 | [2, 7] | [1, 2, 6] | [2, 7] |
| `1 2 - 4 5 6 7 - -` | 20 | 10 | [4, 6] | [1, 2, 7] | [4, 6] |
| `1 2 - 4 5 6 7 - -` | 20 | 11 | [5, 6] | [1, 4, 6] | [4, 7] |
| `1 2 - 4 5 6 7 - -` | 20 | 12 | [5, 7] | [1, 2, 4, 5] | [5, 7] |
| `1 2 3 - 5 6 - 8 -` | 20 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 - 5 6 - 8 -` | 20 | 5 | [5] | [2, 3] | [5] |
| `1 2 3 - 5 6 - 8 -` | 20 | 6 | [6] | [1, 2, 3] | [6] |
| `1 2 3 - 5 6 - 8 -` | 20 | 7 | [2, 5] | [1, 6] | [1, 6] |
| `1 2 3 - 5 6 - 8 -` | 20 | 8 | [8] | [1, 2, 5] | [8] |
| `1 2 3 - 5 6 - 8 -` | 20 | 9 | [1, 8] | [1, 2, 6] | [1, 8] |
| `1 2 3 - 5 6 - 8 -` | 20 | 10 | [2, 8] | [1, 3, 6] | [2, 8] |
| `1 2 3 - 5 6 - 8 -` | 20 | 11 | [3, 8] | [1, 2, 3, 5] | [3, 8] |
| `1 2 3 - 5 6 - 8 -` | 20 | 12 | [1, 3, 8] | [1, 2, 3, 6] | [1, 3, 8] |
| `1 2 3 4 - - 7 8 -` | 20 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 4 - - 7 8 -` | 20 | 4 | [4] | [1, 3] | [4] |
| `1 2 3 4 - - 7 8 -` | 20 | 5 | [1, 4] | [2, 3] | [1, 4] |
| `1 2 3 4 - - 7 8 -` | 20 | 6 | [2, 4] | [1, 2, 3] | [2, 4] |
| `1 2 3 4 - - 7 8 -` | 20 | 7 | [7] | [1, 2, 4] | [7] |
| `1 2 3 4 - - 7 8 -` | 20 | 8 | [8] | [1, 3, 4] | [8] |
| `1 2 3 4 - - 7 8 -` | 20 | 9 | [1, 8] | [2, 3, 4] | [1, 8] |
| `1 2 3 4 - - 7 8 -` | 20 | 10 | [3, 7] | [1, 2, 3, 4] | [2, 8] |
| `1 2 3 4 - - 7 8 -` | 20 | 11 | [3, 8] | [1, 2, 8] | [3, 8] |
| `1 2 3 4 - - 7 8 -` | 20 | 12 | [4, 8] | [1, 4, 7] | [4, 8] |
| `1 2 3 4 - 6 - - 9` | 20 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 4 - 6 - - 9` | 20 | 4 | [4] | [1, 3] | [4] |
| `1 2 3 4 - 6 - - 9` | 20 | 5 | [2, 3] | [1, 4] | [1, 4] |
| `1 2 3 4 - 6 - - 9` | 20 | 6 | [6] | [1, 2, 3] | [6] |
| `1 2 3 4 - 6 - - 9` | 20 | 7 | [3, 4] | [1, 2, 4] | [1, 6] |
| `1 2 3 4 - 6 - - 9` | 20 | 8 | [2, 6] | [1, 3, 4] | [2, 6] |
| `1 2 3 4 - 6 - - 9` | 20 | 9 | [9] | [1, 2, 6] | [9] |
| `1 2 3 4 - 6 - - 9` | 20 | 10 | [1, 9] | [1, 2, 3, 4] | [1, 9] |
| `1 2 3 4 - 6 - - 9` | 20 | 11 | [2, 9] | [1, 4, 6] | [2, 9] |
| `1 2 3 4 - 6 - - 9` | 20 | 12 | [3, 9] | [1, 2, 3, 6] | [3, 9] |
| `- - - 4 5 - 7 8 -` | 21 | 12 | [4, 8] | [5, 7] | [4, 8] |
| `- - - 4 5 6 - - 9` | 21 | 9 | [9] | [4, 5] | [9] |
| `- - 3 - 5 - 7 - 9` | 21 | 12 | [3, 9] | [5, 7] | [3, 9] |
| `- - 3 4 - - - 8 9` | 21 | 12 | [3, 9] | [4, 8] | [3, 9] |
| `- 2 - - - 6 7 - 9` | 21 | 9 | [9] | [2, 7] | [9] |
| `- 2 - 4 5 6 7 - -` | 21 | 6 | [6] | [2, 4] | [6] |
| `- 2 - 4 5 6 7 - -` | 21 | 7 | [7] | [2, 5] | [7] |
| `- 2 - 4 5 6 7 - -` | 21 | 9 | [4, 5] | [2, 7] | [2, 7] |
| `- 2 - 4 5 6 7 - -` | 21 | 11 | [4, 7] | [2, 4, 5] | [4, 7] |
| `- 2 - 4 5 6 7 - -` | 21 | 12 | [5, 7] | [2, 4, 6] | [5, 7] |
| `- 2 3 - 5 6 - 8 -` | 21 | 5 | [5] | [2, 3] | [5] |
| `- 2 3 - 5 6 - 8 -` | 21 | 8 | [8] | [2, 6] | [8] |
| `- 2 3 - 5 6 - 8 -` | 21 | 10 | [2, 8] | [2, 3, 5] | [2, 8] |
| `- 2 3 - 5 6 - 8 -` | 21 | 11 | [3, 8] | [2, 3, 6] | [3, 8] |
| `- 2 3 4 - - 7 8 -` | 21 | 7 | [7] | [3, 4] | [7] |
| `- 2 3 4 - - 7 8 -` | 21 | 9 | [2, 7] | [2, 3, 4] | [2, 7] |
| `- 2 3 4 - - 7 8 -` | 21 | 10 | [3, 7] | [2, 8] | [2, 8] |
| `- 2 3 4 - - 7 8 -` | 21 | 11 | [3, 8] | [4, 7] | [3, 8] |
| `- 2 3 4 - - 7 8 -` | 21 | 12 | [4, 8] | [2, 3, 7] | [4, 8] |
| `- 2 3 4 - 6 - - 9` | 21 | 6 | [6] | [2, 4] | [6] |
| `- 2 3 4 - 6 - - 9` | 21 | 9 | [9] | [2, 3, 4] | [9] |
| `- 2 3 4 - 6 - - 9` | 21 | 11 | [2, 9] | [2, 3, 6] | [2, 9] |
| `- 2 3 4 - 6 - - 9` | 21 | 12 | [3, 9] | [2, 4, 6] | [3, 9] |
| `1 - - - - 6 - 8 9` | 21 | 9 | [9] | [1, 8] | [9] |
| `1 - - 4 5 6 - 8 -` | 21 | 5 | [5] | [1, 4] | [5] |
| `1 - - 4 5 6 - 8 -` | 21 | 6 | [6] | [1, 5] | [6] |
| `1 - - 4 5 6 - 8 -` | 21 | 9 | [1, 8] | [4, 5] | [1, 8] |
| `1 - - 4 5 6 - 8 -` | 21 | 10 | [4, 6] | [1, 4, 5] | [4, 6] |
| `1 - - 4 5 6 - 8 -` | 21 | 11 | [5, 6] | [1, 4, 6] | [5, 6] |
| `1 - - 4 5 6 - 8 -` | 21 | 12 | [4, 8] | [1, 5, 6] | [4, 8] |
| `1 - 3 - 5 - 7 8 -` | 21 | 8 | [8] | [1, 7] | [8] |
| `1 - 3 - 5 - 7 8 -` | 21 | 9 | [1, 8] | [1, 3, 5] | [1, 8] |
| `1 - 3 - 5 - 7 8 -` | 21 | 11 | [3, 8] | [1, 3, 7] | [3, 8] |
| `1 - 3 - 5 - 7 8 -` | 21 | 12 | [5, 7] | [1, 3, 8] | [5, 7] |
| `1 - 3 - 5 6 - - 9` | 21 | 6 | [6] | [1, 5] | [6] |
| `1 - 3 - 5 6 - - 9` | 21 | 9 | [9] | [1, 3, 5] | [9] |
| `1 - 3 - 5 6 - - 9` | 21 | 10 | [1, 9] | [1, 3, 6] | [1, 9] |
| `1 - 3 - 5 6 - - 9` | 21 | 12 | [3, 9] | [1, 5, 6] | [3, 9] |
| `1 - 3 4 - - 7 - 9` | 21 | 4 | [4] | [1, 3] | [4] |
| `1 - 3 4 - - 7 - 9` | 21 | 7 | [7] | [3, 4] | [7] |
| `1 - 3 4 - - 7 - 9` | 21 | 8 | [1, 7] | [1, 3, 4] | [1, 7] |
| `1 - 3 4 - - 7 - 9` | 21 | 10 | [1, 9] | [3, 7] | [1, 9] |
| `1 - 3 4 - - 7 - 9` | 21 | 11 | [4, 7] | [1, 3, 7] | [4, 7] |
| `1 - 3 4 - - 7 - 9` | 21 | 12 | [3, 9] | [1, 4, 7] | [3, 9] |
| `1 2 - - - 6 7 8 -` | 21 | 7 | [7] | [1, 6] | [7] |
| `1 2 - - - 6 7 8 -` | 21 | 8 | [8] | [1, 7] | [8] |
| `1 2 - - - 6 7 8 -` | 21 | 9 | [1, 8] | [1, 2, 6] | [1, 8] |
| `1 2 - - - 6 7 8 -` | 21 | 10 | [2, 8] | [1, 2, 7] | [2, 8] |
| `1 2 - - 5 - 7 - 9` | 21 | 7 | [7] | [2, 5] | [7] |
| `1 2 - - 5 - 7 - 9` | 21 | 8 | [1, 7] | [1, 2, 5] | [1, 7] |
| `1 2 - - 5 - 7 - 9` | 21 | 9 | [9] | [2, 7] | [9] |
| `1 2 - - 5 - 7 - 9` | 21 | 10 | [1, 9] | [1, 2, 7] | [1, 9] |
| `1 2 - - 5 - 7 - 9` | 21 | 12 | [5, 7] | [1, 2, 9] | [5, 7] |
| `1 2 - 4 - - - 8 9` | 21 | 9 | [9] | [1, 8] | [9] |
| `1 2 - 4 - - - 8 9` | 21 | 10 | [1, 9] | [2, 8] | [1, 9] |
| `1 2 - 4 - - - 8 9` | 21 | 11 | [2, 9] | [1, 2, 8] | [2, 9] |
| `1 2 - 4 - - - 8 9` | 21 | 12 | [4, 8] | [1, 2, 9] | [4, 8] |
| `1 2 3 - 5 6 7 - -` | 21 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 - 5 6 7 - -` | 21 | 5 | [5] | [2, 3] | [5] |
| `1 2 3 - 5 6 7 - -` | 21 | 6 | [6] | [1, 2, 3] | [6] |
| `1 2 3 - 5 6 7 - -` | 21 | 7 | [7] | [1, 6] | [7] |
| `1 2 3 - 5 6 7 - -` | 21 | 8 | [2, 6] | [1, 2, 5] | [1, 7] |
| `1 2 3 - 5 6 7 - -` | 21 | 9 | [2, 7] | [1, 2, 6] | [2, 7] |
| `1 2 3 - 5 6 7 - -` | 21 | 10 | [3, 7] | [1, 3, 6] | [3, 7] |
| `1 2 3 - 5 6 7 - -` | 21 | 11 | [5, 6] | [1, 2, 3, 5] | [5, 6] |
| `1 2 3 - 5 6 7 - -` | 21 | 12 | [5, 7] | [1, 2, 3, 6] | [5, 7] |
| `1 2 3 4 - 6 - 8 -` | 21 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 4 - 6 - 8 -` | 21 | 4 | [4] | [1, 3] | [4] |
| `1 2 3 4 - 6 - 8 -` | 21 | 5 | [2, 3] | [1, 4] | [1, 4] |
| `1 2 3 4 - 6 - 8 -` | 21 | 6 | [6] | [1, 2, 3] | [6] |
| `1 2 3 4 - 6 - 8 -` | 21 | 7 | [1, 6] | [1, 2, 4] | [1, 6] |
| `1 2 3 4 - 6 - 8 -` | 21 | 8 | [8] | [1, 3, 4] | [8] |
| `1 2 3 4 - 6 - 8 -` | 21 | 9 | [1, 8] | [1, 2, 6] | [1, 8] |
| `1 2 3 4 - 6 - 8 -` | 21 | 10 | [2, 8] | [1, 2, 3, 4] | [2, 8] |
| `1 2 3 4 - 6 - 8 -` | 21 | 11 | [3, 8] | [1, 4, 6] | [3, 8] |
| `1 2 3 4 - 6 - 8 -` | 21 | 12 | [4, 8] | [1, 2, 3, 6] | [4, 8] |
| `1 2 3 4 5 - - - 9` | 21 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 4 5 - - - 9` | 21 | 4 | [4] | [1, 3] | [4] |
| `1 2 3 4 5 - - - 9` | 21 | 5 | [5] | [2, 3] | [5] |
| `1 2 3 4 5 - - - 9` | 21 | 6 | [2, 4] | [1, 2, 3] | [1, 5] |
| `1 2 3 4 5 - - - 9` | 21 | 7 | [3, 4] | [1, 2, 4] | [2, 5] |
| `1 2 3 4 5 - - - 9` | 21 | 8 | [3, 5] | [1, 2, 5] | [3, 5] |
| `1 2 3 4 5 - - - 9` | 21 | 9 | [9] | [1, 3, 5] | [9] |
| `1 2 3 4 5 - - - 9` | 21 | 10 | [1, 9] | [1, 2, 3, 4] | [1, 9] |
| `1 2 3 4 5 - - - 9` | 21 | 11 | [2, 9] | [1, 2, 3, 5] | [2, 9] |
| `1 2 3 4 5 - - - 9` | 21 | 12 | [3, 9] | [1, 2, 4, 5] | [3, 9] |
| `- - 3 - 5 - 7 8 -` | 22 | 8 | [8] | [3, 5] | [8] |
| `- - 3 - 5 6 - - 9` | 22 | 9 | [9] | [3, 6] | [9] |
| `- - 3 4 - - 7 - 9` | 22 | 7 | [7] | [3, 4] | [7] |
| `- 2 - - - 6 7 8 -` | 22 | 8 | [8] | [2, 6] | [8] |
| `- 2 - - 5 - 7 - 9` | 22 | 7 | [7] | [2, 5] | [7] |
| `- 2 - - 5 - 7 - 9` | 22 | 9 | [9] | [2, 7] | [9] |
| `- 2 3 - 5 6 7 - -` | 22 | 5 | [5] | [2, 3] | [5] |
| `- 2 3 - 5 6 7 - -` | 22 | 7 | [7] | [2, 5] | [7] |
| `- 2 3 - 5 6 7 - -` | 22 | 8 | [3, 5] | [2, 6] | [2, 6] |
| `- 2 3 - 5 6 7 - -` | 22 | 9 | [2, 7] | [3, 6] | [2, 7] |
| `- 2 3 - 5 6 7 - -` | 22 | 10 | [3, 7] | [2, 3, 5] | [3, 7] |
| `- 2 3 - 5 6 7 - -` | 22 | 11 | [5, 6] | [2, 3, 6] | [5, 6] |
| `- 2 3 - 5 6 7 - -` | 22 | 12 | [5, 7] | [2, 3, 7] | [5, 7] |
| `- 2 3 4 - 6 - 8 -` | 22 | 6 | [6] | [2, 4] | [6] |
| `- 2 3 4 - 6 - 8 -` | 22 | 8 | [8] | [2, 6] | [8] |
| `- 2 3 4 - 6 - 8 -` | 22 | 9 | [3, 6] | [2, 3, 4] | [3, 6] |
| `- 2 3 4 - 6 - 8 -` | 22 | 10 | [2, 8] | [4, 6] | [2, 8] |
| `- 2 3 4 - 6 - 8 -` | 22 | 11 | [3, 8] | [2, 3, 6] | [3, 8] |
| `- 2 3 4 - 6 - 8 -` | 22 | 12 | [4, 8] | [2, 4, 6] | [4, 8] |
| `- 2 3 4 5 - - - 9` | 22 | 5 | [5] | [2, 3] | [5] |
| `- 2 3 4 5 - - - 9` | 22 | 7 | [3, 4] | [2, 5] | [2, 5] |
| `- 2 3 4 5 - - - 9` | 22 | 9 | [9] | [2, 3, 4] | [9] |
| `- 2 3 4 5 - - - 9` | 22 | 11 | [2, 9] | [2, 4, 5] | [2, 9] |
| `- 2 3 4 5 - - - 9` | 22 | 12 | [3, 9] | [3, 4, 5] | [3, 9] |
| `1 - - - - 6 7 - 9` | 22 | 7 | [7] | [1, 6] | [7] |
| `1 - - - 5 - - 8 9` | 22 | 9 | [9] | [1, 8] | [9] |
| `1 - - 4 5 6 7 - -` | 22 | 5 | [5] | [1, 4] | [5] |
| `1 - - 4 5 6 7 - -` | 22 | 6 | [6] | [1, 5] | [6] |
| `1 - - 4 5 6 7 - -` | 22 | 7 | [7] | [1, 6] | [7] |
| `1 - - 4 5 6 7 - -` | 22 | 10 | [4, 6] | [1, 4, 5] | [4, 6] |
| `1 - - 4 5 6 7 - -` | 22 | 11 | [5, 6] | [1, 4, 6] | [4, 7] |
| `1 - - 4 5 6 7 - -` | 22 | 12 | [5, 7] | [1, 4, 7] | [5, 7] |
| `1 - 3 - 5 6 - 8 -` | 22 | 6 | [6] | [1, 5] | [6] |
| `1 - 3 - 5 6 - 8 -` | 22 | 8 | [8] | [3, 5] | [8] |
| `1 - 3 - 5 6 - 8 -` | 22 | 9 | [1, 8] | [1, 3, 5] | [1, 8] |
| `1 - 3 - 5 6 - 8 -` | 22 | 11 | [3, 8] | [5, 6] | [3, 8] |
| `1 - 3 - 5 6 - 8 -` | 22 | 12 | [1, 3, 8] | [1, 5, 6] | [1, 3, 8] |
| `1 - 3 4 - - 7 8 -` | 22 | 4 | [4] | [1, 3] | [4] |
| `1 - 3 4 - - 7 8 -` | 22 | 7 | [7] | [3, 4] | [7] |
| `1 - 3 4 - - 7 8 -` | 22 | 8 | [8] | [1, 3, 4] | [8] |
| `1 - 3 4 - - 7 8 -` | 22 | 11 | [3, 8] | [1, 3, 7] | [3, 8] |
| `1 - 3 4 - - 7 8 -` | 22 | 12 | [4, 8] | [1, 4, 7] | [4, 8] |
| `1 - 3 4 - 6 - - 9` | 22 | 4 | [4] | [1, 3] | [4] |
| `1 - 3 4 - 6 - - 9` | 22 | 7 | [3, 4] | [1, 6] | [1, 6] |
| `1 - 3 4 - 6 - - 9` | 22 | 9 | [9] | [3, 6] | [9] |
| `1 - 3 4 - 6 - - 9` | 22 | 10 | [1, 9] | [1, 3, 6] | [1, 9] |
| `1 2 - - 5 - 7 8 -` | 22 | 7 | [7] | [2, 5] | [7] |
| `1 2 - - 5 - 7 8 -` | 22 | 8 | [8] | [1, 2, 5] | [8] |
| `1 2 - - 5 - 7 8 -` | 22 | 9 | [2, 7] | [1, 8] | [1, 8] |
| `1 2 - - 5 - 7 8 -` | 22 | 10 | [2, 8] | [1, 2, 7] | [2, 8] |
| `1 2 - - 5 6 - - 9` | 22 | 6 | [6] | [1, 5] | [6] |
| `1 2 - - 5 6 - - 9` | 22 | 7 | [2, 5] | [1, 6] | [1, 6] |
| `1 2 - - 5 6 - - 9` | 22 | 8 | [2, 6] | [1, 2, 5] | [2, 6] |
| `1 2 - - 5 6 - - 9` | 22 | 9 | [9] | [1, 2, 6] | [9] |
| `1 2 - - 5 6 - - 9` | 22 | 11 | [2, 9] | [5, 6] | [2, 9] |
| `1 2 - - 5 6 - - 9` | 22 | 12 | [1, 2, 9] | [1, 5, 6] | [1, 2, 9] |
| `1 2 - 4 - - 7 - 9` | 22 | 7 | [7] | [1, 2, 4] | [7] |
| `1 2 - 4 - - 7 - 9` | 22 | 9 | [9] | [2, 7] | [9] |
| `1 2 - 4 - - 7 - 9` | 22 | 10 | [1, 9] | [1, 2, 7] | [1, 9] |
| `1 2 - 4 - - 7 - 9` | 22 | 11 | [2, 9] | [4, 7] | [2, 9] |
| `1 2 - 4 - - 7 - 9` | 22 | 12 | [1, 2, 9] | [1, 4, 7] | [1, 2, 9] |
| `1 2 3 - - - - 8 9` | 22 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 - - - - 8 9` | 22 | 9 | [9] | [1, 8] | [9] |
| `1 2 3 - - - - 8 9` | 22 | 10 | [1, 9] | [2, 8] | [1, 9] |
| `1 2 3 - - - - 8 9` | 22 | 11 | [2, 9] | [1, 2, 8] | [2, 9] |
| `1 2 3 - - - - 8 9` | 22 | 12 | [3, 9] | [1, 3, 8] | [3, 9] |
| `1 2 3 4 - 6 7 - -` | 22 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 4 - 6 7 - -` | 22 | 4 | [4] | [1, 3] | [4] |
| `1 2 3 4 - 6 7 - -` | 22 | 5 | [1, 4] | [2, 3] | [1, 4] |
| `1 2 3 4 - 6 7 - -` | 22 | 6 | [6] | [1, 2, 3] | [6] |
| `1 2 3 4 - 6 7 - -` | 22 | 7 | [7] | [1, 2, 4] | [7] |
| `1 2 3 4 - 6 7 - -` | 22 | 8 | [1, 7] | [1, 3, 4] | [1, 7] |
| `1 2 3 4 - 6 7 - -` | 22 | 9 | [3, 6] | [1, 2, 6] | [2, 7] |
| `1 2 3 4 - 6 7 - -` | 22 | 10 | [3, 7] | [1, 2, 3, 4] | [3, 7] |
| `1 2 3 4 - 6 7 - -` | 22 | 11 | [4, 7] | [1, 3, 7] | [4, 7] |
| `1 2 3 4 - 6 7 - -` | 22 | 12 | [2, 3, 7] | [1, 2, 3, 6] | [1, 4, 7] |
| `1 2 3 4 5 - - 8 -` | 22 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 4 5 - - 8 -` | 22 | 4 | [4] | [1, 3] | [4] |
| `1 2 3 4 5 - - 8 -` | 22 | 5 | [5] | [1, 4] | [5] |
| `1 2 3 4 5 - - 8 -` | 22 | 6 | [1, 5] | [1, 2, 3] | [1, 5] |
| `1 2 3 4 5 - - 8 -` | 22 | 7 | [3, 4] | [1, 2, 4] | [2, 5] |
| `1 2 3 4 5 - - 8 -` | 22 | 8 | [8] | [1, 2, 5] | [8] |
| `1 2 3 4 5 - - 8 -` | 22 | 9 | [1, 8] | [1, 3, 5] | [1, 8] |
| `1 2 3 4 5 - - 8 -` | 22 | 10 | [2, 8] | [1, 2, 3, 4] | [2, 8] |
| `1 2 3 4 5 - - 8 -` | 22 | 11 | [3, 8] | [1, 2, 3, 5] | [3, 8] |
| `1 2 3 4 5 - - 8 -` | 22 | 12 | [4, 8] | [1, 2, 4, 5] | [4, 8] |
| `- - - 4 5 6 7 - -` | 23 | 11 | [5, 6] | [4, 7] | [4, 7] |
| `- - 3 - 5 6 - 8 -` | 23 | 8 | [8] | [3, 5] | [8] |
| `- - 3 - 5 6 - 8 -` | 23 | 11 | [3, 8] | [5, 6] | [3, 8] |
| `- - 3 4 - - 7 8 -` | 23 | 7 | [7] | [3, 4] | [7] |
| `- - 3 4 - - 7 8 -` | 23 | 11 | [3, 8] | [4, 7] | [3, 8] |
| `- - 3 4 - 6 - - 9` | 23 | 9 | [9] | [3, 6] | [9] |
| `- 2 - - 5 - 7 8 -` | 23 | 7 | [7] | [2, 5] | [7] |
| `- 2 - - 5 6 - - 9` | 23 | 11 | [2, 9] | [5, 6] | [2, 9] |
| `- 2 - 4 - - 7 - 9` | 23 | 9 | [9] | [2, 7] | [9] |
| `- 2 - 4 - - 7 - 9` | 23 | 11 | [2, 9] | [4, 7] | [2, 9] |
| `- 2 3 - - - - 8 9` | 23 | 11 | [2, 9] | [3, 8] | [2, 9] |
| `- 2 3 4 - 6 7 - -` | 23 | 6 | [6] | [2, 4] | [6] |
| `- 2 3 4 - 6 7 - -` | 23 | 7 | [7] | [3, 4] | [7] |
| `- 2 3 4 - 6 7 - -` | 23 | 9 | [2, 7] | [2, 3, 4] | [2, 7] |
| `- 2 3 4 - 6 7 - -` | 23 | 10 | [4, 6] | [3, 7] | [3, 7] |
| `- 2 3 4 - 6 7 - -` | 23 | 11 | [4, 7] | [2, 3, 6] | [4, 7] |
| `- 2 3 4 - 6 7 - -` | 23 | 12 | [2, 4, 6] | [2, 3, 7] | [2, 3, 7] |
| `- 2 3 4 5 - - 8 -` | 23 | 5 | [5] | [2, 3] | [5] |
| `- 2 3 4 5 - - 8 -` | 23 | 7 | [3, 4] | [2, 5] | [2, 5] |
| `- 2 3 4 5 - - 8 -` | 23 | 8 | [8] | [3, 5] | [8] |
| `- 2 3 4 5 - - 8 -` | 23 | 9 | [4, 5] | [2, 3, 4] | [4, 5] |
| `- 2 3 4 5 - - 8 -` | 23 | 10 | [2, 8] | [2, 3, 5] | [2, 8] |
| `- 2 3 4 5 - - 8 -` | 23 | 11 | [3, 8] | [2, 4, 5] | [3, 8] |
| `- 2 3 4 5 - - 8 -` | 23 | 12 | [4, 8] | [3, 4, 5] | [4, 8] |
| `1 - - - - 6 7 8 -` | 23 | 7 | [7] | [1, 6] | [7] |
| `1 - - - - 6 7 8 -` | 23 | 8 | [8] | [1, 7] | [8] |
| `1 - - 4 - - - 8 9` | 23 | 9 | [9] | [1, 8] | [9] |
| `1 - 3 - 5 6 7 - -` | 23 | 6 | [6] | [1, 5] | [6] |
| `1 - 3 - 5 6 7 - -` | 23 | 7 | [7] | [1, 6] | [7] |
| `1 - 3 - 5 6 7 - -` | 23 | 8 | [1, 7] | [3, 5] | [1, 7] |
| `1 - 3 - 5 6 7 - -` | 23 | 9 | [3, 6] | [1, 3, 5] | [3, 6] |
| `1 - 3 - 5 6 7 - -` | 23 | 10 | [3, 7] | [1, 3, 6] | [3, 7] |
| `1 - 3 - 5 6 7 - -` | 23 | 11 | [5, 6] | [1, 3, 7] | [5, 6] |
| `1 - 3 - 5 6 7 - -` | 23 | 12 | [5, 7] | [1, 5, 6] | [5, 7] |
| `1 - 3 4 - 6 - 8 -` | 23 | 4 | [4] | [1, 3] | [4] |
| `1 - 3 4 - 6 - 8 -` | 23 | 7 | [3, 4] | [1, 6] | [1, 6] |
| `1 - 3 4 - 6 - 8 -` | 23 | 8 | [8] | [1, 3, 4] | [8] |
| `1 - 3 4 - 6 - 8 -` | 23 | 9 | [1, 8] | [3, 6] | [1, 8] |
| `1 - 3 4 - 6 - 8 -` | 23 | 10 | [4, 6] | [1, 3, 6] | [4, 6] |
| `1 - 3 4 - 6 - 8 -` | 23 | 11 | [3, 8] | [1, 4, 6] | [3, 8] |
| `1 - 3 4 - 6 - 8 -` | 23 | 12 | [4, 8] | [1, 3, 8] | [4, 8] |
| `1 - 3 4 5 - - - 9` | 23 | 4 | [4] | [1, 3] | [4] |
| `1 - 3 4 5 - - - 9` | 23 | 5 | [5] | [1, 4] | [5] |
| `1 - 3 4 5 - - - 9` | 23 | 8 | [3, 5] | [1, 3, 4] | [3, 5] |
| `1 - 3 4 5 - - - 9` | 23 | 9 | [9] | [1, 3, 5] | [9] |
| `1 - 3 4 5 - - - 9` | 23 | 10 | [1, 9] | [1, 4, 5] | [1, 9] |
| `1 - 3 4 5 - - - 9` | 23 | 12 | [3, 9] | [3, 4, 5] | [3, 9] |
| `1 2 - - 5 6 - 8 -` | 23 | 6 | [6] | [1, 5] | [6] |
| `1 2 - - 5 6 - 8 -` | 23 | 7 | [2, 5] | [1, 6] | [1, 6] |
| `1 2 - - 5 6 - 8 -` | 23 | 8 | [8] | [1, 2, 5] | [8] |
| `1 2 - - 5 6 - 8 -` | 23 | 9 | [1, 8] | [1, 2, 6] | [1, 8] |
| `1 2 - - 5 6 - 8 -` | 23 | 11 | [5, 6] | [1, 2, 8] | [5, 6] |
| `1 2 - 4 - - 7 8 -` | 23 | 7 | [7] | [1, 2, 4] | [7] |
| `1 2 - 4 - - 7 8 -` | 23 | 8 | [8] | [1, 7] | [8] |
| `1 2 - 4 - - 7 8 -` | 23 | 9 | [1, 8] | [2, 7] | [1, 8] |
| `1 2 - 4 - - 7 8 -` | 23 | 10 | [2, 8] | [1, 2, 7] | [2, 8] |
| `1 2 - 4 - - 7 8 -` | 23 | 11 | [4, 7] | [1, 2, 8] | [4, 7] |
| `1 2 - 4 - - 7 8 -` | 23 | 12 | [4, 8] | [1, 4, 7] | [4, 8] |
| `1 2 - 4 - 6 - - 9` | 23 | 6 | [6] | [2, 4] | [6] |
| `1 2 - 4 - 6 - - 9` | 23 | 7 | [1, 6] | [1, 2, 4] | [1, 6] |
| `1 2 - 4 - 6 - - 9` | 23 | 9 | [9] | [1, 2, 6] | [9] |
| `1 2 - 4 - 6 - - 9` | 23 | 10 | [1, 9] | [4, 6] | [1, 9] |
| `1 2 - 4 - 6 - - 9` | 23 | 11 | [2, 9] | [1, 4, 6] | [2, 9] |
| `1 2 - 4 - 6 - - 9` | 23 | 12 | [1, 2, 9] | [2, 4, 6] | [1, 2, 9] |
| `1 2 3 - - - 7 - 9` | 23 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 - - - 7 - 9` | 23 | 9 | [9] | [2, 7] | [9] |
| `1 2 3 - - - 7 - 9` | 23 | 10 | [1, 9] | [1, 2, 7] | [1, 9] |
| `1 2 3 - - - 7 - 9` | 23 | 11 | [2, 9] | [1, 3, 7] | [2, 9] |
| `1 2 3 - - - 7 - 9` | 23 | 12 | [3, 9] | [2, 3, 7] | [3, 9] |
| `1 2 3 4 5 - 7 - -` | 23 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 4 5 - 7 - -` | 23 | 4 | [4] | [1, 3] | [4] |
| `1 2 3 4 5 - 7 - -` | 23 | 5 | [5] | [1, 4] | [5] |
| `1 2 3 4 5 - 7 - -` | 23 | 6 | [2, 4] | [1, 2, 3] | [1, 5] |
| `1 2 3 4 5 - 7 - -` | 23 | 7 | [7] | [1, 2, 4] | [7] |
| `1 2 3 4 5 - 7 - -` | 23 | 8 | [1, 7] | [1, 2, 5] | [1, 7] |
| `1 2 3 4 5 - 7 - -` | 23 | 9 | [4, 5] | [1, 3, 5] | [2, 7] |
| `1 2 3 4 5 - 7 - -` | 23 | 10 | [3, 7] | [1, 2, 3, 4] | [3, 7] |
| `1 2 3 4 5 - 7 - -` | 23 | 11 | [4, 7] | [1, 2, 3, 5] | [4, 7] |
| `1 2 3 4 5 - 7 - -` | 23 | 12 | [5, 7] | [1, 2, 4, 5] | [5, 7] |
| `- - 3 4 5 - - - 9` | 24 | 9 | [9] | [4, 5] | [9] |
| `- - 3 4 5 - - - 9` | 24 | 12 | [3, 9] | [3, 4, 5] | [3, 9] |
| `- 2 - - 5 6 - 8 -` | 24 | 8 | [8] | [2, 6] | [8] |
| `- 2 - 4 - 6 - - 9` | 24 | 6 | [6] | [2, 4] | [6] |
| `- 2 3 - - - 7 - 9` | 24 | 9 | [9] | [2, 7] | [9] |
| `- 2 3 - - - 7 - 9` | 24 | 12 | [3, 9] | [2, 3, 7] | [3, 9] |
| `- 2 3 4 5 - 7 - -` | 24 | 5 | [5] | [2, 3] | [5] |
| `- 2 3 4 5 - 7 - -` | 24 | 7 | [7] | [2, 5] | [7] |
| `- 2 3 4 5 - 7 - -` | 24 | 9 | [2, 7] | [2, 3, 4] | [2, 7] |
| `- 2 3 4 5 - 7 - -` | 24 | 10 | [3, 7] | [2, 3, 5] | [3, 7] |
| `- 2 3 4 5 - 7 - -` | 24 | 11 | [4, 7] | [2, 4, 5] | [4, 7] |
| `- 2 3 4 5 - 7 - -` | 24 | 12 | [5, 7] | [2, 3, 7] | [5, 7] |
| `1 - - - 5 - 7 8 -` | 24 | 8 | [8] | [1, 7] | [8] |
| `1 - - - 5 6 - - 9` | 24 | 6 | [6] | [1, 5] | [6] |
| `1 - 3 - - - - 8 9` | 24 | 9 | [9] | [1, 8] | [9] |
| `1 - 3 - - - - 8 9` | 24 | 12 | [3, 9] | [1, 3, 8] | [3, 9] |
| `1 - 3 4 - 6 7 - -` | 24 | 4 | [4] | [1, 3] | [4] |
| `1 - 3 4 - 6 7 - -` | 24 | 7 | [7] | [1, 6] | [7] |
| `1 - 3 4 - 6 7 - -` | 24 | 8 | [1, 7] | [1, 3, 4] | [1, 7] |
| `1 - 3 4 - 6 7 - -` | 24 | 10 | [3, 7] | [1, 3, 6] | [3, 7] |
| `1 - 3 4 - 6 7 - -` | 24 | 11 | [4, 7] | [1, 3, 7] | [4, 7] |
| `1 - 3 4 5 - - 8 -` | 24 | 4 | [4] | [1, 3] | [4] |
| `1 - 3 4 5 - - 8 -` | 24 | 5 | [5] | [1, 4] | [5] |
| `1 - 3 4 5 - - 8 -` | 24 | 8 | [8] | [1, 3, 4] | [8] |
| `1 - 3 4 5 - - 8 -` | 24 | 9 | [1, 8] | [1, 3, 5] | [1, 8] |
| `1 - 3 4 5 - - 8 -` | 24 | 12 | [4, 8] | [1, 3, 8] | [4, 8] |
| `1 2 - - 5 6 7 - -` | 24 | 6 | [6] | [1, 5] | [6] |
| `1 2 - - 5 6 7 - -` | 24 | 7 | [7] | [1, 6] | [7] |
| `1 2 - - 5 6 7 - -` | 24 | 8 | [1, 7] | [1, 2, 5] | [1, 7] |
| `1 2 - - 5 6 7 - -` | 24 | 9 | [2, 7] | [1, 2, 6] | [2, 7] |
| `1 2 - - 5 6 7 - -` | 24 | 12 | [5, 7] | [1, 5, 6] | [5, 7] |
| `1 2 - 4 - 6 - 8 -` | 24 | 6 | [6] | [2, 4] | [6] |
| `1 2 - 4 - 6 - 8 -` | 24 | 7 | [1, 6] | [1, 2, 4] | [1, 6] |
| `1 2 - 4 - 6 - 8 -` | 24 | 8 | [8] | [2, 6] | [8] |
| `1 2 - 4 - 6 - 8 -` | 24 | 9 | [1, 8] | [1, 2, 6] | [1, 8] |
| `1 2 - 4 - 6 - 8 -` | 24 | 10 | [2, 8] | [4, 6] | [2, 8] |
| `1 2 - 4 - 6 - 8 -` | 24 | 11 | [1, 2, 8] | [1, 4, 6] | [1, 2, 8] |
| `1 2 - 4 - 6 - 8 -` | 24 | 12 | [4, 8] | [2, 4, 6] | [4, 8] |
| `1 2 - 4 5 - - - 9` | 24 | 5 | [5] | [1, 4] | [5] |
| `1 2 - 4 5 - - - 9` | 24 | 6 | [2, 4] | [1, 5] | [1, 5] |
| `1 2 - 4 5 - - - 9` | 24 | 7 | [2, 5] | [1, 2, 4] | [2, 5] |
| `1 2 - 4 5 - - - 9` | 24 | 9 | [9] | [4, 5] | [9] |
| `1 2 - 4 5 - - - 9` | 24 | 10 | [1, 9] | [1, 4, 5] | [1, 9] |
| `1 2 - 4 5 - - - 9` | 24 | 11 | [2, 9] | [2, 4, 5] | [2, 9] |
| `1 2 - 4 5 - - - 9` | 24 | 12 | [1, 2, 9] | [1, 2, 4, 5] | [1, 2, 9] |
| `1 2 3 - - - 7 8 -` | 24 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 - - - 7 8 -` | 24 | 8 | [8] | [1, 7] | [8] |
| `1 2 3 - - - 7 8 -` | 24 | 9 | [1, 8] | [2, 7] | [1, 8] |
| `1 2 3 - - - 7 8 -` | 24 | 10 | [2, 8] | [1, 2, 7] | [2, 8] |
| `1 2 3 - - - 7 8 -` | 24 | 11 | [3, 8] | [1, 3, 7] | [3, 8] |
| `1 2 3 - - - 7 8 -` | 24 | 12 | [1, 3, 8] | [2, 3, 7] | [1, 3, 8] |
| `1 2 3 - - 6 - - 9` | 24 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 - - 6 - - 9` | 24 | 6 | [6] | [1, 2, 3] | [6] |
| `1 2 3 - - 6 - - 9` | 24 | 9 | [9] | [1, 2, 6] | [9] |
| `1 2 3 - - 6 - - 9` | 24 | 10 | [1, 9] | [1, 3, 6] | [1, 9] |
| `1 2 3 - - 6 - - 9` | 24 | 11 | [2, 9] | [2, 3, 6] | [2, 9] |
| `1 2 3 - - 6 - - 9` | 24 | 12 | [3, 9] | [1, 2, 3, 6] | [3, 9] |
| `1 2 3 4 5 6 - - -` | 24 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 4 5 6 - - -` | 24 | 4 | [4] | [1, 3] | [4] |
| `1 2 3 4 5 6 - - -` | 24 | 5 | [5] | [2, 3] | [5] |
| `1 2 3 4 5 6 - - -` | 24 | 6 | [6] | [1, 2, 3] | [6] |
| `1 2 3 4 5 6 - - -` | 24 | 7 | [1, 6] | [1, 2, 4] | [1, 6] |
| `1 2 3 4 5 6 - - -` | 24 | 8 | [3, 5] | [1, 2, 5] | [2, 6] |
| `1 2 3 4 5 6 - - -` | 24 | 9 | [3, 6] | [1, 3, 5] | [3, 6] |
| `1 2 3 4 5 6 - - -` | 24 | 10 | [4, 6] | [1, 2, 3, 4] | [4, 6] |
| `1 2 3 4 5 6 - - -` | 24 | 11 | [5, 6] | [1, 2, 3, 5] | [5, 6] |
| `1 2 3 4 5 6 - - -` | 24 | 12 | [3, 4, 5] | [1, 2, 3, 6] | [1, 5, 6] |
| `- - 3 4 - 6 7 - -` | 25 | 7 | [7] | [3, 4] | [7] |
| `- - 3 4 - 6 7 - -` | 25 | 10 | [4, 6] | [3, 7] | [3, 7] |
| `- - 3 4 5 - - 8 -` | 25 | 8 | [8] | [3, 5] | [8] |
| `- - 3 4 5 - - 8 -` | 25 | 12 | [4, 8] | [3, 4, 5] | [4, 8] |
| `- 2 - - 5 6 7 - -` | 25 | 7 | [7] | [2, 5] | [7] |
| `- 2 - 4 - 6 - 8 -` | 25 | 6 | [6] | [2, 4] | [6] |
| `- 2 - 4 - 6 - 8 -` | 25 | 8 | [8] | [2, 6] | [8] |
| `- 2 - 4 - 6 - 8 -` | 25 | 10 | [2, 8] | [4, 6] | [2, 8] |
| `- 2 - 4 - 6 - 8 -` | 25 | 12 | [4, 8] | [2, 4, 6] | [4, 8] |
| `- 2 - 4 5 - - - 9` | 25 | 9 | [9] | [4, 5] | [9] |
| `- 2 - 4 5 - - - 9` | 25 | 11 | [2, 9] | [2, 4, 5] | [2, 9] |
| `- 2 3 - - - 7 8 -` | 25 | 10 | [2, 8] | [3, 7] | [2, 8] |
| `- 2 3 - - 6 - - 9` | 25 | 9 | [9] | [3, 6] | [9] |
| `- 2 3 - - 6 - - 9` | 25 | 11 | [2, 9] | [2, 3, 6] | [2, 9] |
| `- 2 3 4 5 6 - - -` | 25 | 5 | [5] | [2, 3] | [5] |
| `- 2 3 4 5 6 - - -` | 25 | 6 | [6] | [2, 4] | [6] |
| `- 2 3 4 5 6 - - -` | 25 | 7 | [3, 4] | [2, 5] | [2, 5] |
| `- 2 3 4 5 6 - - -` | 25 | 8 | [2, 6] | [3, 5] | [2, 6] |
| `- 2 3 4 5 6 - - -` | 25 | 9 | [3, 6] | [2, 3, 4] | [3, 6] |
| `- 2 3 4 5 6 - - -` | 25 | 10 | [4, 6] | [2, 3, 5] | [4, 6] |
| `- 2 3 4 5 6 - - -` | 25 | 11 | [5, 6] | [2, 3, 6] | [5, 6] |
| `- 2 3 4 5 6 - - -` | 25 | 12 | [3, 4, 5] | [2, 4, 6] | [2, 4, 6] |
| `1 - - - 5 6 - 8 -` | 25 | 6 | [6] | [1, 5] | [6] |
| `1 - - 4 - - 7 8 -` | 25 | 8 | [8] | [1, 7] | [8] |
| `1 - - 4 - - 7 8 -` | 25 | 12 | [4, 8] | [1, 4, 7] | [4, 8] |
| `1 - - 4 - 6 - - 9` | 25 | 10 | [1, 9] | [4, 6] | [1, 9] |
| `1 - 3 - - - 7 - 9` | 25 | 10 | [1, 9] | [3, 7] | [1, 9] |
| `1 - 3 4 5 - 7 - -` | 25 | 4 | [4] | [1, 3] | [4] |
| `1 - 3 4 5 - 7 - -` | 25 | 5 | [5] | [1, 4] | [5] |
| `1 - 3 4 5 - 7 - -` | 25 | 7 | [7] | [3, 4] | [7] |
| `1 - 3 4 5 - 7 - -` | 25 | 8 | [1, 7] | [1, 3, 4] | [1, 7] |
| `1 - 3 4 5 - 7 - -` | 25 | 9 | [4, 5] | [1, 3, 5] | [4, 5] |
| `1 - 3 4 5 - 7 - -` | 25 | 10 | [3, 7] | [1, 4, 5] | [3, 7] |
| `1 - 3 4 5 - 7 - -` | 25 | 11 | [4, 7] | [1, 3, 7] | [4, 7] |
| `1 - 3 4 5 - 7 - -` | 25 | 12 | [5, 7] | [1, 4, 7] | [5, 7] |
| `1 2 - - - - - 8 9` | 25 | 9 | [9] | [1, 8] | [9] |
| `1 2 - - - - - 8 9` | 25 | 10 | [1, 9] | [2, 8] | [1, 9] |
| `1 2 - - - - - 8 9` | 25 | 11 | [2, 9] | [1, 2, 8] | [2, 9] |
| `1 2 - 4 - 6 7 - -` | 25 | 6 | [6] | [2, 4] | [6] |
| `1 2 - 4 - 6 7 - -` | 25 | 7 | [7] | [1, 2, 4] | [7] |
| `1 2 - 4 - 6 7 - -` | 25 | 8 | [2, 6] | [1, 7] | [1, 7] |
| `1 2 - 4 - 6 7 - -` | 25 | 9 | [2, 7] | [1, 2, 6] | [2, 7] |
| `1 2 - 4 - 6 7 - -` | 25 | 10 | [4, 6] | [1, 2, 7] | [4, 6] |
| `1 2 - 4 - 6 7 - -` | 25 | 11 | [4, 7] | [1, 4, 6] | [4, 7] |
| `1 2 - 4 - 6 7 - -` | 25 | 12 | [2, 4, 6] | [1, 4, 7] | [1, 4, 7] |
| `1 2 - 4 5 - - 8 -` | 25 | 5 | [5] | [1, 4] | [5] |
| `1 2 - 4 5 - - 8 -` | 25 | 6 | [2, 4] | [1, 5] | [1, 5] |
| `1 2 - 4 5 - - 8 -` | 25 | 7 | [2, 5] | [1, 2, 4] | [2, 5] |
| `1 2 - 4 5 - - 8 -` | 25 | 8 | [8] | [1, 2, 5] | [8] |
| `1 2 - 4 5 - - 8 -` | 25 | 9 | [1, 8] | [4, 5] | [1, 8] |
| `1 2 - 4 5 - - 8 -` | 25 | 10 | [2, 8] | [1, 4, 5] | [2, 8] |
| `1 2 - 4 5 - - 8 -` | 25 | 11 | [2, 4, 5] | [1, 2, 8] | [1, 2, 8] |
| `1 2 - 4 5 - - 8 -` | 25 | 12 | [4, 8] | [1, 2, 4, 5] | [4, 8] |
| `1 2 3 - - 6 - 8 -` | 25 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 - - 6 - 8 -` | 25 | 6 | [6] | [1, 2, 3] | [6] |
| `1 2 3 - - 6 - 8 -` | 25 | 8 | [8] | [2, 6] | [8] |
| `1 2 3 - - 6 - 8 -` | 25 | 9 | [1, 8] | [1, 2, 6] | [1, 8] |
| `1 2 3 - - 6 - 8 -` | 25 | 10 | [2, 8] | [1, 3, 6] | [2, 8] |
| `1 2 3 - - 6 - 8 -` | 25 | 11 | [3, 8] | [1, 2, 8] | [3, 8] |
| `1 2 3 - - 6 - 8 -` | 25 | 12 | [1, 3, 8] | [1, 2, 3, 6] | [1, 3, 8] |
| `1 2 3 - 5 - - - 9` | 25 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 - 5 - - - 9` | 25 | 5 | [5] | [2, 3] | [5] |
| `1 2 3 - 5 - - - 9` | 25 | 6 | [1, 5] | [1, 2, 3] | [1, 5] |
| `1 2 3 - 5 - - - 9` | 25 | 8 | [3, 5] | [1, 2, 5] | [3, 5] |
| `1 2 3 - 5 - - - 9` | 25 | 9 | [9] | [1, 3, 5] | [9] |
| `1 2 3 - 5 - - - 9` | 25 | 10 | [1, 9] | [2, 3, 5] | [1, 9] |
| `1 2 3 - 5 - - - 9` | 25 | 11 | [2, 9] | [1, 2, 3, 5] | [2, 9] |
| `1 2 3 - 5 - - - 9` | 25 | 12 | [3, 9] | [1, 2, 9] | [3, 9] |
| `- - 3 4 5 - 7 - -` | 26 | 7 | [7] | [3, 4] | [7] |
| `- - 3 4 5 - 7 - -` | 26 | 12 | [5, 7] | [3, 4, 5] | [5, 7] |
| `- 2 - 4 - 6 7 - -` | 26 | 6 | [6] | [2, 4] | [6] |
| `- 2 3 - - 6 - 8 -` | 26 | 8 | [8] | [2, 6] | [8] |
| `- 2 3 - - 6 - 8 -` | 26 | 11 | [3, 8] | [2, 3, 6] | [3, 8] |
| `- 2 3 - 5 - - - 9` | 26 | 5 | [5] | [2, 3] | [5] |
| `1 - - - 5 6 7 - -` | 26 | 6 | [6] | [1, 5] | [6] |
| `1 - - - 5 6 7 - -` | 26 | 7 | [7] | [1, 6] | [7] |
| `1 - - - 5 6 7 - -` | 26 | 12 | [5, 7] | [1, 5, 6] | [5, 7] |
| `1 - - 4 5 - - - 9` | 26 | 5 | [5] | [1, 4] | [5] |
| `1 - - 4 5 - - - 9` | 26 | 9 | [9] | [4, 5] | [9] |
| `1 - - 4 5 - - - 9` | 26 | 10 | [1, 9] | [1, 4, 5] | [1, 9] |
| `1 - 3 - - - 7 8 -` | 26 | 8 | [8] | [1, 7] | [8] |
| `1 - 3 - - - 7 8 -` | 26 | 11 | [3, 8] | [1, 3, 7] | [3, 8] |
| `1 - 3 - - 6 - - 9` | 26 | 9 | [9] | [3, 6] | [9] |
| `1 - 3 - - 6 - - 9` | 26 | 10 | [1, 9] | [1, 3, 6] | [1, 9] |
| `1 - 3 4 5 6 - - -` | 26 | 4 | [4] | [1, 3] | [4] |
| `1 - 3 4 5 6 - - -` | 26 | 5 | [5] | [1, 4] | [5] |
| `1 - 3 4 5 6 - - -` | 26 | 6 | [6] | [1, 5] | [6] |
| `1 - 3 4 5 6 - - -` | 26 | 7 | [1, 6] | [3, 4] | [1, 6] |
| `1 - 3 4 5 6 - - -` | 26 | 8 | [3, 5] | [1, 3, 4] | [3, 5] |
| `1 - 3 4 5 6 - - -` | 26 | 9 | [4, 5] | [1, 3, 5] | [3, 6] |
| `1 - 3 4 5 6 - - -` | 26 | 10 | [4, 6] | [1, 3, 6] | [4, 6] |
| `1 - 3 4 5 6 - - -` | 26 | 11 | [5, 6] | [1, 4, 6] | [5, 6] |
| `1 - 3 4 5 6 - - -` | 26 | 12 | [3, 4, 5] | [1, 5, 6] | [1, 5, 6] |
| `1 2 - - - - 7 - 9` | 26 | 9 | [9] | [2, 7] | [9] |
| `1 2 - - - - 7 - 9` | 26 | 10 | [1, 9] | [1, 2, 7] | [1, 9] |
| `1 2 - 4 5 - 7 - -` | 26 | 5 | [5] | [1, 4] | [5] |
| `1 2 - 4 5 - 7 - -` | 26 | 6 | [2, 4] | [1, 5] | [1, 5] |
| `1 2 - 4 5 - 7 - -` | 26 | 7 | [7] | [1, 2, 4] | [7] |
| `1 2 - 4 5 - 7 - -` | 26 | 8 | [1, 7] | [1, 2, 5] | [1, 7] |
| `1 2 - 4 5 - 7 - -` | 26 | 9 | [4, 5] | [2, 7] | [2, 7] |
| `1 2 - 4 5 - 7 - -` | 26 | 10 | [1, 4, 5] | [1, 2, 7] | [1, 2, 7] |
| `1 2 - 4 5 - 7 - -` | 26 | 11 | [4, 7] | [2, 4, 5] | [4, 7] |
| `1 2 - 4 5 - 7 - -` | 26 | 12 | [5, 7] | [1, 2, 4, 5] | [5, 7] |
| `1 2 3 - - 6 7 - -` | 26 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 - - 6 7 - -` | 26 | 6 | [6] | [1, 2, 3] | [6] |
| `1 2 3 - - 6 7 - -` | 26 | 7 | [7] | [1, 6] | [7] |
| `1 2 3 - - 6 7 - -` | 26 | 8 | [1, 7] | [2, 6] | [1, 7] |
| `1 2 3 - - 6 7 - -` | 26 | 9 | [2, 7] | [1, 2, 6] | [2, 7] |
| `1 2 3 - - 6 7 - -` | 26 | 10 | [3, 7] | [1, 2, 7] | [3, 7] |
| `1 2 3 - - 6 7 - -` | 26 | 11 | [2, 3, 6] | [1, 3, 7] | [1, 3, 7] |
| `1 2 3 - - 6 7 - -` | 26 | 12 | [2, 3, 7] | [1, 2, 3, 6] | [2, 3, 7] |
| `1 2 3 - 5 - - 8 -` | 26 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 - 5 - - 8 -` | 26 | 5 | [5] | [2, 3] | [5] |
| `1 2 3 - 5 - - 8 -` | 26 | 6 | [1, 5] | [1, 2, 3] | [1, 5] |
| `1 2 3 - 5 - - 8 -` | 26 | 8 | [8] | [1, 2, 5] | [8] |
| `1 2 3 - 5 - - 8 -` | 26 | 9 | [1, 8] | [1, 3, 5] | [1, 8] |
| `1 2 3 - 5 - - 8 -` | 26 | 10 | [2, 8] | [2, 3, 5] | [2, 8] |
| `1 2 3 - 5 - - 8 -` | 26 | 11 | [3, 8] | [1, 2, 3, 5] | [3, 8] |
| `1 2 3 4 - - - - 9` | 26 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 4 - - - - 9` | 26 | 4 | [4] | [1, 3] | [4] |
| `1 2 3 4 - - - - 9` | 26 | 5 | [2, 3] | [1, 4] | [1, 4] |
| `1 2 3 4 - - - - 9` | 26 | 6 | [2, 4] | [1, 2, 3] | [2, 4] |
| `1 2 3 4 - - - - 9` | 26 | 7 | [3, 4] | [1, 2, 4] | [3, 4] |
| `1 2 3 4 - - - - 9` | 26 | 9 | [9] | [2, 3, 4] | [9] |
| `1 2 3 4 - - - - 9` | 26 | 10 | [1, 9] | [1, 2, 3, 4] | [1, 9] |
| `1 2 3 4 - - - - 9` | 26 | 12 | [3, 9] | [1, 2, 9] | [3, 9] |
| `- - - 4 5 - - - 9` | 27 | 9 | [9] | [4, 5] | [9] |
| `- - 3 - - 6 - - 9` | 27 | 9 | [9] | [3, 6] | [9] |
| `- - 3 4 5 6 - - -` | 27 | 9 | [4, 5] | [3, 6] | [3, 6] |
| `- 2 - - - - 7 - 9` | 27 | 9 | [9] | [2, 7] | [9] |
| `- 2 - 4 5 - 7 - -` | 27 | 7 | [7] | [2, 5] | [7] |
| `- 2 - 4 5 - 7 - -` | 27 | 9 | [4, 5] | [2, 7] | [2, 7] |
| `- 2 - 4 5 - 7 - -` | 27 | 11 | [4, 7] | [2, 4, 5] | [4, 7] |
| `- 2 3 - - 6 7 - -` | 27 | 9 | [3, 6] | [2, 7] | [2, 7] |
| `- 2 3 - 5 - - 8 -` | 27 | 5 | [5] | [2, 3] | [5] |
| `- 2 3 - 5 - - 8 -` | 27 | 8 | [8] | [3, 5] | [8] |
| `- 2 3 - 5 - - 8 -` | 27 | 10 | [2, 8] | [2, 3, 5] | [2, 8] |
| `- 2 3 4 - - - - 9` | 27 | 9 | [9] | [2, 3, 4] | [9] |
| `1 - - - - - - 8 9` | 27 | 9 | [9] | [1, 8] | [9] |
| `1 - - 4 - 6 7 - -` | 27 | 7 | [7] | [1, 6] | [7] |
| `1 - - 4 - 6 7 - -` | 27 | 11 | [4, 7] | [1, 4, 6] | [4, 7] |
| `1 - - 4 5 - - 8 -` | 27 | 5 | [5] | [1, 4] | [5] |
| `1 - - 4 5 - - 8 -` | 27 | 9 | [4, 5] | [1, 8] | [1, 8] |
| `1 - 3 - - 6 - 8 -` | 27 | 9 | [3, 6] | [1, 8] | [1, 8] |
| `1 - 3 - 5 - - - 9` | 27 | 9 | [9] | [1, 3, 5] | [9] |
| `1 2 - - - - 7 8 -` | 27 | 8 | [8] | [1, 7] | [8] |
| `1 2 - - - - 7 8 -` | 27 | 9 | [1, 8] | [2, 7] | [1, 8] |
| `1 2 - - - - 7 8 -` | 27 | 10 | [2, 8] | [1, 2, 7] | [2, 8] |
| `1 2 - - - 6 - - 9` | 27 | 9 | [9] | [1, 2, 6] | [9] |
| `1 2 - 4 5 6 - - -` | 27 | 5 | [5] | [1, 4] | [5] |
| `1 2 - 4 5 6 - - -` | 27 | 6 | [6] | [1, 5] | [6] |
| `1 2 - 4 5 6 - - -` | 27 | 7 | [1, 6] | [1, 2, 4] | [1, 6] |
| `1 2 - 4 5 6 - - -` | 27 | 8 | [2, 6] | [1, 2, 5] | [2, 6] |
| `1 2 - 4 5 6 - - -` | 27 | 9 | [4, 5] | [1, 2, 6] | [4, 5] |
| `1 2 - 4 5 6 - - -` | 27 | 10 | [4, 6] | [1, 4, 5] | [4, 6] |
| `1 2 - 4 5 6 - - -` | 27 | 11 | [5, 6] | [1, 4, 6] | [5, 6] |
| `1 2 - 4 5 6 - - -` | 27 | 12 | [2, 4, 6] | [1, 2, 4, 5] | [1, 5, 6] |
| `1 2 3 - 5 - 7 - -` | 27 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 - 5 - 7 - -` | 27 | 5 | [5] | [2, 3] | [5] |
| `1 2 3 - 5 - 7 - -` | 27 | 6 | [1, 5] | [1, 2, 3] | [1, 5] |
| `1 2 3 - 5 - 7 - -` | 27 | 7 | [7] | [2, 5] | [7] |
| `1 2 3 - 5 - 7 - -` | 27 | 8 | [3, 5] | [1, 2, 5] | [1, 7] |
| `1 2 3 - 5 - 7 - -` | 27 | 9 | [2, 7] | [1, 3, 5] | [2, 7] |
| `1 2 3 - 5 - 7 - -` | 27 | 10 | [3, 7] | [1, 2, 7] | [3, 7] |
| `1 2 3 - 5 - 7 - -` | 27 | 11 | [1, 3, 7] | [1, 2, 3, 5] | [1, 3, 7] |
| `1 2 3 - 5 - 7 - -` | 27 | 12 | [5, 7] | [2, 3, 7] | [5, 7] |
| `1 2 3 4 - - - 8 -` | 27 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 4 - - - 8 -` | 27 | 4 | [4] | [1, 3] | [4] |
| `1 2 3 4 - - - 8 -` | 27 | 5 | [2, 3] | [1, 4] | [1, 4] |
| `1 2 3 4 - - - 8 -` | 27 | 6 | [2, 4] | [1, 2, 3] | [2, 4] |
| `1 2 3 4 - - - 8 -` | 27 | 7 | [3, 4] | [1, 2, 4] | [3, 4] |
| `1 2 3 4 - - - 8 -` | 27 | 8 | [8] | [1, 3, 4] | [8] |
| `1 2 3 4 - - - 8 -` | 27 | 9 | [1, 8] | [2, 3, 4] | [1, 8] |
| `1 2 3 4 - - - 8 -` | 27 | 10 | [2, 8] | [1, 2, 3, 4] | [2, 8] |
| `1 2 3 4 - - - 8 -` | 27 | 11 | [3, 8] | [1, 2, 8] | [3, 8] |
| `1 2 3 4 - - - 8 -` | 27 | 12 | [4, 8] | [1, 3, 8] | [4, 8] |
| `- 2 - 4 5 6 - - -` | 28 | 6 | [6] | [2, 4] | [6] |
| `- 2 - 4 5 6 - - -` | 28 | 11 | [5, 6] | [2, 4, 5] | [5, 6] |
| `- 2 3 - 5 - 7 - -` | 28 | 5 | [5] | [2, 3] | [5] |
| `- 2 3 - 5 - 7 - -` | 28 | 7 | [7] | [2, 5] | [7] |
| `- 2 3 - 5 - 7 - -` | 28 | 10 | [3, 7] | [2, 3, 5] | [3, 7] |
| `- 2 3 - 5 - 7 - -` | 28 | 12 | [5, 7] | [2, 3, 7] | [5, 7] |
| `1 - - 4 5 - 7 - -` | 28 | 5 | [5] | [1, 4] | [5] |
| `1 - - 4 5 - 7 - -` | 28 | 12 | [5, 7] | [1, 4, 7] | [5, 7] |
| `1 - 3 - - 6 7 - -` | 28 | 7 | [7] | [1, 6] | [7] |
| `1 - 3 - - 6 7 - -` | 28 | 10 | [3, 7] | [1, 3, 6] | [3, 7] |
| `1 - 3 - 5 - - 8 -` | 28 | 8 | [8] | [3, 5] | [8] |
| `1 - 3 - 5 - - 8 -` | 28 | 9 | [1, 8] | [1, 3, 5] | [1, 8] |
| `1 - 3 4 - - - - 9` | 28 | 4 | [4] | [1, 3] | [4] |
| `1 2 - - - 6 - 8 -` | 28 | 8 | [8] | [2, 6] | [8] |
| `1 2 - - - 6 - 8 -` | 28 | 9 | [1, 8] | [1, 2, 6] | [1, 8] |
| `1 2 3 - 5 6 - - -` | 28 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 - 5 6 - - -` | 28 | 5 | [5] | [2, 3] | [5] |
| `1 2 3 - 5 6 - - -` | 28 | 6 | [6] | [1, 2, 3] | [6] |
| `1 2 3 - 5 6 - - -` | 28 | 7 | [2, 5] | [1, 6] | [1, 6] |
| `1 2 3 - 5 6 - - -` | 28 | 8 | [3, 5] | [1, 2, 5] | [2, 6] |
| `1 2 3 - 5 6 - - -` | 28 | 9 | [3, 6] | [1, 2, 6] | [3, 6] |
| `1 2 3 - 5 6 - - -` | 28 | 10 | [2, 3, 5] | [1, 3, 6] | [1, 3, 6] |
| `1 2 3 - 5 6 - - -` | 28 | 11 | [5, 6] | [1, 2, 3, 5] | [5, 6] |
| `1 2 3 - 5 6 - - -` | 28 | 12 | [1, 5, 6] | [1, 2, 3, 6] | [1, 5, 6] |
| `1 2 3 4 - - 7 - -` | 28 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 4 - - 7 - -` | 28 | 4 | [4] | [1, 3] | [4] |
| `1 2 3 4 - - 7 - -` | 28 | 5 | [2, 3] | [1, 4] | [1, 4] |
| `1 2 3 4 - - 7 - -` | 28 | 6 | [2, 4] | [1, 2, 3] | [2, 4] |
| `1 2 3 4 - - 7 - -` | 28 | 7 | [7] | [1, 2, 4] | [7] |
| `1 2 3 4 - - 7 - -` | 28 | 8 | [1, 7] | [1, 3, 4] | [1, 7] |
| `1 2 3 4 - - 7 - -` | 28 | 9 | [2, 7] | [2, 3, 4] | [2, 7] |
| `1 2 3 4 - - 7 - -` | 28 | 10 | [3, 7] | [1, 2, 3, 4] | [3, 7] |
| `1 2 3 4 - - 7 - -` | 28 | 11 | [4, 7] | [1, 3, 7] | [4, 7] |
| `1 2 3 4 - - 7 - -` | 28 | 12 | [2, 3, 7] | [1, 4, 7] | [1, 4, 7] |
| `- - 3 - 5 - - 8 -` | 29 | 8 | [8] | [3, 5] | [8] |
| `- 2 - - - 6 - 8 -` | 29 | 8 | [8] | [2, 6] | [8] |
| `- 2 3 - 5 6 - - -` | 29 | 5 | [5] | [2, 3] | [5] |
| `- 2 3 - 5 6 - - -` | 29 | 8 | [3, 5] | [2, 6] | [2, 6] |
| `- 2 3 - 5 6 - - -` | 29 | 11 | [5, 6] | [2, 3, 6] | [5, 6] |
| `- 2 3 4 - - 7 - -` | 29 | 7 | [7] | [3, 4] | [7] |
| `- 2 3 4 - - 7 - -` | 29 | 9 | [2, 7] | [2, 3, 4] | [2, 7] |
| `1 - - - - - 7 8 -` | 29 | 8 | [8] | [1, 7] | [8] |
| `1 - - 4 5 6 - - -` | 29 | 5 | [5] | [1, 4] | [5] |
| `1 - - 4 5 6 - - -` | 29 | 6 | [6] | [1, 5] | [6] |
| `1 - - 4 5 6 - - -` | 29 | 10 | [4, 6] | [1, 4, 5] | [4, 6] |
| `1 - - 4 5 6 - - -` | 29 | 11 | [5, 6] | [1, 4, 6] | [5, 6] |
| `1 - 3 - 5 - 7 - -` | 29 | 8 | [3, 5] | [1, 7] | [1, 7] |
| `1 - 3 4 - - - 8 -` | 29 | 4 | [4] | [1, 3] | [4] |
| `1 - 3 4 - - - 8 -` | 29 | 8 | [8] | [1, 3, 4] | [8] |
| `1 - 3 4 - - - 8 -` | 29 | 12 | [4, 8] | [1, 3, 8] | [4, 8] |
| `1 2 - - - 6 7 - -` | 29 | 7 | [7] | [1, 6] | [7] |
| `1 2 - - - 6 7 - -` | 29 | 8 | [2, 6] | [1, 7] | [1, 7] |
| `1 2 - - - 6 7 - -` | 29 | 9 | [2, 7] | [1, 2, 6] | [2, 7] |
| `1 2 - - 5 - - 8 -` | 29 | 8 | [8] | [1, 2, 5] | [8] |
| `1 2 3 4 - 6 - - -` | 29 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 4 - 6 - - -` | 29 | 4 | [4] | [1, 3] | [4] |
| `1 2 3 4 - 6 - - -` | 29 | 5 | [2, 3] | [1, 4] | [1, 4] |
| `1 2 3 4 - 6 - - -` | 29 | 6 | [6] | [1, 2, 3] | [6] |
| `1 2 3 4 - 6 - - -` | 29 | 7 | [3, 4] | [1, 2, 4] | [1, 6] |
| `1 2 3 4 - 6 - - -` | 29 | 8 | [2, 6] | [1, 3, 4] | [2, 6] |
| `1 2 3 4 - 6 - - -` | 29 | 9 | [3, 6] | [1, 2, 6] | [3, 6] |
| `1 2 3 4 - 6 - - -` | 29 | 10 | [4, 6] | [1, 2, 3, 4] | [4, 6] |
| `1 2 3 4 - 6 - - -` | 29 | 11 | [2, 3, 6] | [1, 4, 6] | [1, 4, 6] |
| `1 2 3 4 - 6 - - -` | 29 | 12 | [2, 4, 6] | [1, 2, 3, 6] | [2, 4, 6] |
| `- 2 3 4 - 6 - - -` | 30 | 6 | [6] | [2, 4] | [6] |
| `- 2 3 4 - 6 - - -` | 30 | 9 | [3, 6] | [2, 3, 4] | [3, 6] |
| `1 - 3 - 5 6 - - -` | 30 | 6 | [6] | [1, 5] | [6] |
| `1 - 3 - 5 6 - - -` | 30 | 9 | [3, 6] | [1, 3, 5] | [3, 6] |
| `1 - 3 4 - - 7 - -` | 30 | 4 | [4] | [1, 3] | [4] |
| `1 - 3 4 - - 7 - -` | 30 | 7 | [7] | [3, 4] | [7] |
| `1 - 3 4 - - 7 - -` | 30 | 8 | [1, 7] | [1, 3, 4] | [1, 7] |
| `1 - 3 4 - - 7 - -` | 30 | 11 | [4, 7] | [1, 3, 7] | [4, 7] |
| `1 2 - - 5 - 7 - -` | 30 | 7 | [7] | [2, 5] | [7] |
| `1 2 - - 5 - 7 - -` | 30 | 8 | [1, 7] | [1, 2, 5] | [1, 7] |
| `1 2 3 - - - - - 9` | 30 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 - - - - - 9` | 30 | 12 | [3, 9] | [1, 2, 9] | [3, 9] |
| `1 2 3 4 5 - - - -` | 30 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 4 5 - - - -` | 30 | 4 | [4] | [1, 3] | [4] |
| `1 2 3 4 5 - - - -` | 30 | 5 | [5] | [2, 3] | [5] |
| `1 2 3 4 5 - - - -` | 30 | 6 | [1, 5] | [1, 2, 3] | [1, 5] |
| `1 2 3 4 5 - - - -` | 30 | 7 | [3, 4] | [1, 2, 4] | [2, 5] |
| `1 2 3 4 5 - - - -` | 30 | 8 | [3, 5] | [1, 2, 5] | [3, 5] |
| `1 2 3 4 5 - - - -` | 30 | 9 | [4, 5] | [1, 3, 5] | [4, 5] |
| `1 2 3 4 5 - - - -` | 30 | 10 | [2, 3, 5] | [1, 2, 3, 4] | [1, 4, 5] |
| `1 2 3 4 5 - - - -` | 30 | 11 | [2, 4, 5] | [1, 2, 3, 5] | [2, 4, 5] |
| `1 2 3 4 5 - - - -` | 30 | 12 | [3, 4, 5] | [1, 2, 4, 5] | [3, 4, 5] |
| `- - 3 4 - - 7 - -` | 31 | 7 | [7] | [3, 4] | [7] |
| `- 2 - - 5 - 7 - -` | 31 | 7 | [7] | [2, 5] | [7] |
| `- 2 3 4 5 - - - -` | 31 | 5 | [5] | [2, 3] | [5] |
| `- 2 3 4 5 - - - -` | 31 | 7 | [3, 4] | [2, 5] | [2, 5] |
| `- 2 3 4 5 - - - -` | 31 | 9 | [4, 5] | [2, 3, 4] | [4, 5] |
| `1 - - - - 6 7 - -` | 31 | 7 | [7] | [1, 6] | [7] |
| `1 - 3 4 - 6 - - -` | 31 | 4 | [4] | [1, 3] | [4] |
| `1 - 3 4 - 6 - - -` | 31 | 7 | [3, 4] | [1, 6] | [1, 6] |
| `1 - 3 4 - 6 - - -` | 31 | 10 | [4, 6] | [1, 3, 6] | [4, 6] |
| `1 2 - - 5 6 - - -` | 31 | 6 | [6] | [1, 5] | [6] |
| `1 2 - - 5 6 - - -` | 31 | 7 | [2, 5] | [1, 6] | [1, 6] |
| `1 2 - - 5 6 - - -` | 31 | 8 | [2, 6] | [1, 2, 5] | [2, 6] |
| `1 2 - 4 - - 7 - -` | 31 | 7 | [7] | [1, 2, 4] | [7] |
| `1 2 3 - - - - 8 -` | 31 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 - - - - 8 -` | 31 | 11 | [3, 8] | [1, 2, 8] | [3, 8] |
| `1 - 3 4 5 - - - -` | 32 | 4 | [4] | [1, 3] | [4] |
| `1 - 3 4 5 - - - -` | 32 | 5 | [5] | [1, 4] | [5] |
| `1 - 3 4 5 - - - -` | 32 | 8 | [3, 5] | [1, 3, 4] | [3, 5] |
| `1 - 3 4 5 - - - -` | 32 | 9 | [4, 5] | [1, 3, 5] | [4, 5] |
| `1 2 - 4 - 6 - - -` | 32 | 6 | [6] | [2, 4] | [6] |
| `1 2 - 4 - 6 - - -` | 32 | 7 | [1, 6] | [1, 2, 4] | [1, 6] |
| `1 2 3 - - - 7 - -` | 32 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 - - - 7 - -` | 32 | 10 | [3, 7] | [1, 2, 7] | [3, 7] |
| `- 2 - 4 - 6 - - -` | 33 | 6 | [6] | [2, 4] | [6] |
| `1 - - - 5 6 - - -` | 33 | 6 | [6] | [1, 5] | [6] |
| `1 2 - 4 5 - - - -` | 33 | 5 | [5] | [1, 4] | [5] |
| `1 2 - 4 5 - - - -` | 33 | 6 | [2, 4] | [1, 5] | [1, 5] |
| `1 2 - 4 5 - - - -` | 33 | 7 | [2, 5] | [1, 2, 4] | [2, 5] |
| `1 2 3 - - 6 - - -` | 33 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 - - 6 - - -` | 33 | 6 | [6] | [1, 2, 3] | [6] |
| `1 2 3 - - 6 - - -` | 33 | 9 | [3, 6] | [1, 2, 6] | [3, 6] |
| `1 2 3 - 5 - - - -` | 34 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 - 5 - - - -` | 34 | 5 | [5] | [2, 3] | [5] |
| `1 2 3 - 5 - - - -` | 34 | 6 | [1, 5] | [1, 2, 3] | [1, 5] |
| `1 2 3 - 5 - - - -` | 34 | 8 | [3, 5] | [1, 2, 5] | [3, 5] |
| `- 2 3 - 5 - - - -` | 35 | 5 | [5] | [2, 3] | [5] |
| `1 - - 4 5 - - - -` | 35 | 5 | [5] | [1, 4] | [5] |
| `1 2 3 4 - - - - -` | 35 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 4 - - - - -` | 35 | 4 | [4] | [1, 3] | [4] |
| `1 2 3 4 - - - - -` | 35 | 5 | [2, 3] | [1, 4] | [1, 4] |
| `1 2 3 4 - - - - -` | 35 | 6 | [2, 4] | [1, 2, 3] | [2, 4] |
| `1 2 3 4 - - - - -` | 35 | 7 | [3, 4] | [1, 2, 4] | [3, 4] |
| `1 - 3 4 - - - - -` | 37 | 4 | [4] | [1, 3] | [4] |
| `1 2 3 - - - - - -` | 39 | 3 | [3] | [1, 2] | [3] |


### Exhibit E: Optimal Moves for All or Nothing Scoring System

Note: Only showing results where there is a difference between the optimal strategy and the worst strategy

In [14]:
def exhibit_e(results: dict) -> Table:
    """returns a table for optimal moves of the all or nothing scoring system"""
    exhibit_table = []
    results_subtable = results['all_or_nothing_move_table']
    for record in results_subtable:
        if record['Max Move'] == record['Min Move']:
            continue
        exhibit_table.append({
            'Board (Remaining)': f'`{record["Board"]}`',
            'Board Sum': record['Sum'],
            'Roll': record['Roll'],
            'Max Move': record['Max Move'],
            'Min Move': record['Min Move'],
            'Highest Tile Move': record['Highest Tile Move'],
        })
    # sort by board sum ascending
    exhibit_table.sort(key=lambda record: (record['Board Sum'], record['Board (Remaining)'], record['Roll']))
    return exhibit_table


display_table(exhibit_e(full_results))

| Board (Remaining) | Board Sum | Roll | Max Move | Min Move | Highest Tile Move |
| --- | --- | --- | --- | --- | --- |
| `1 2 3 4 5 6 7 8 9` | 0 | 3 | [3] | [1, 2], [3] | [3] |
| `1 2 3 4 5 6 7 8 9` | 0 | 4 | [4] | [1, 3] | [4] |
| `1 2 3 4 5 6 7 8 9` | 0 | 5 | [5] | [2, 3] | [5] |
| `1 2 3 4 5 6 7 8 9` | 0 | 6 | [6] | [1, 2, 3] | [6] |
| `1 2 3 4 5 6 7 8 9` | 0 | 7 | [7] | [1, 2, 4] | [7] |
| `1 2 3 4 5 6 7 8 9` | 0 | 8 | [8] | [1, 3, 4] | [8] |
| `1 2 3 4 5 6 7 8 9` | 0 | 9 | [9] | [2, 3, 4] | [9] |
| `1 2 3 4 5 6 7 8 9` | 0 | 10 | [1, 9] | [1, 2, 3, 4] | [1, 9] |
| `1 2 3 4 5 6 7 8 9` | 0 | 11 | [2, 9] | [2, 3, 6] | [2, 9] |
| `1 2 3 4 5 6 7 8 9` | 0 | 12 | [3, 9] | [1, 2, 3, 6] | [3, 9] |
| `- 2 3 4 5 6 7 8 9` | 1 | 5 | [5] | [2, 3] | [5] |
| `- 2 3 4 5 6 7 8 9` | 1 | 6 | [6] | [2, 4] | [6] |
| `- 2 3 4 5 6 7 8 9` | 1 | 7 | [7] | [3, 4] | [7] |
| `- 2 3 4 5 6 7 8 9` | 1 | 8 | [8] | [2, 6] | [8] |
| `- 2 3 4 5 6 7 8 9` | 1 | 9 | [9] | [2, 3, 4] | [9] |
| `- 2 3 4 5 6 7 8 9` | 1 | 10 | [2, 8] | [4, 6] | [2, 8] |
| `- 2 3 4 5 6 7 8 9` | 1 | 11 | [2, 9] | [2, 3, 6] | [2, 9] |
| `- 2 3 4 5 6 7 8 9` | 1 | 12 | [3, 9] | [2, 3, 7] | [3, 9] |
| `1 - 3 4 5 6 7 8 9` | 2 | 4 | [4] | [1, 3] | [4] |
| `1 - 3 4 5 6 7 8 9` | 2 | 5 | [5] | [1, 4] | [5] |
| `1 - 3 4 5 6 7 8 9` | 2 | 6 | [6] | [1, 5] | [6] |
| `1 - 3 4 5 6 7 8 9` | 2 | 7 | [7] | [3, 4] | [7] |
| `1 - 3 4 5 6 7 8 9` | 2 | 8 | [8] | [1, 3, 4] | [8] |
| `1 - 3 4 5 6 7 8 9` | 2 | 9 | [9] | [3, 6] | [9] |
| `1 - 3 4 5 6 7 8 9` | 2 | 10 | [1, 9] | [1, 3, 6] | [1, 9] |
| `1 - 3 4 5 6 7 8 9` | 2 | 11 | [3, 8] | [1, 3, 7] | [3, 8] |
| `1 - 3 4 5 6 7 8 9` | 2 | 12 | [3, 9] | [1, 4, 7] | [3, 9] |
| `- - 3 4 5 6 7 8 9` | 3 | 7 | [7] | [3, 4] | [7] |
| `- - 3 4 5 6 7 8 9` | 3 | 8 | [8] | [3, 5] | [8] |
| `- - 3 4 5 6 7 8 9` | 3 | 9 | [9] | [3, 6] | [9] |
| `- - 3 4 5 6 7 8 9` | 3 | 10 | [4, 6] | [3, 7] | [3, 7] |
| `- - 3 4 5 6 7 8 9` | 3 | 11 | [3, 8] | [4, 7] | [3, 8] |
| `- - 3 4 5 6 7 8 9` | 3 | 12 | [3, 9] | [3, 4, 5] | [3, 9] |
| `1 2 - 4 5 6 7 8 9` | 3 | 5 | [5] | [1, 4] | [5] |
| `1 2 - 4 5 6 7 8 9` | 3 | 6 | [6] | [2, 4] | [6] |
| `1 2 - 4 5 6 7 8 9` | 3 | 7 | [7] | [1, 2, 4] | [7] |
| `1 2 - 4 5 6 7 8 9` | 3 | 8 | [8] | [2, 6] | [8] |
| `1 2 - 4 5 6 7 8 9` | 3 | 9 | [9] | [1, 2, 6] | [9] |
| `1 2 - 4 5 6 7 8 9` | 3 | 10 | [1, 9] | [1, 2, 7] | [1, 9] |
| `1 2 - 4 5 6 7 8 9` | 3 | 11 | [2, 9] | [2, 4, 5] | [2, 9] |
| `1 2 - 4 5 6 7 8 9` | 3 | 12 | [4, 8] | [1, 2, 4, 5] | [4, 8] |
| `- 2 - 4 5 6 7 8 9` | 4 | 6 | [6] | [2, 4] | [6] |
| `- 2 - 4 5 6 7 8 9` | 4 | 8 | [8] | [2, 6] | [8] |
| `- 2 - 4 5 6 7 8 9` | 4 | 9 | [9] | [2, 7] | [9] |
| `- 2 - 4 5 6 7 8 9` | 4 | 10 | [4, 6] | [2, 8], [4, 6] | [2, 8] |
| `- 2 - 4 5 6 7 8 9` | 4 | 11 | [2, 9] | [2, 4, 5] | [2, 9] |
| `- 2 - 4 5 6 7 8 9` | 4 | 12 | [4, 8] | [2, 4, 6] | [4, 8] |
| `1 2 3 - 5 6 7 8 9` | 4 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 - 5 6 7 8 9` | 4 | 5 | [5] | [2, 3] | [5] |
| `1 2 3 - 5 6 7 8 9` | 4 | 6 | [6] | [1, 2, 3] | [6] |
| `1 2 3 - 5 6 7 8 9` | 4 | 7 | [7] | [1, 6] | [7] |
| `1 2 3 - 5 6 7 8 9` | 4 | 8 | [8] | [1, 2, 5] | [8] |
| `1 2 3 - 5 6 7 8 9` | 4 | 9 | [9] | [1, 3, 5] | [9] |
| `1 2 3 - 5 6 7 8 9` | 4 | 10 | [1, 9] | [1, 2, 7] | [1, 9] |
| `1 2 3 - 5 6 7 8 9` | 4 | 11 | [2, 9] | [1, 2, 3, 5] | [2, 9] |
| `1 2 3 - 5 6 7 8 9` | 4 | 12 | [3, 9] | [1, 2, 3, 6] | [3, 9] |
| `- 2 3 - 5 6 7 8 9` | 5 | 5 | [5] | [2, 3] | [5] |
| `- 2 3 - 5 6 7 8 9` | 5 | 7 | [7] | [2, 5] | [7] |
| `- 2 3 - 5 6 7 8 9` | 5 | 8 | [8] | [3, 5] | [8] |
| `- 2 3 - 5 6 7 8 9` | 5 | 9 | [9] | [2, 7] | [9] |
| `- 2 3 - 5 6 7 8 9` | 5 | 10 | [2, 8] | [2, 3, 5] | [2, 8] |
| `- 2 3 - 5 6 7 8 9` | 5 | 11 | [2, 9] | [2, 3, 6] | [2, 9] |
| `- 2 3 - 5 6 7 8 9` | 5 | 12 | [3, 9] | [2, 3, 7] | [3, 9] |
| `1 - - 4 5 6 7 8 9` | 5 | 5 | [5] | [1, 4] | [5] |
| `1 - - 4 5 6 7 8 9` | 5 | 7 | [7] | [1, 6] | [7] |
| `1 - - 4 5 6 7 8 9` | 5 | 8 | [8] | [1, 7] | [8] |
| `1 - - 4 5 6 7 8 9` | 5 | 9 | [9] | [4, 5] | [9] |
| `1 - - 4 5 6 7 8 9` | 5 | 10 | [1, 9] | [1, 4, 5] | [1, 9] |
| `1 - - 4 5 6 7 8 9` | 5 | 11 | [5, 6] | [1, 4, 6] | [4, 7] |
| `1 - - 4 5 6 7 8 9` | 5 | 12 | [4, 8] | [1, 4, 7] | [4, 8] |
| `1 2 3 4 - 6 7 8 9` | 5 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 4 - 6 7 8 9` | 5 | 4 | [4] | [1, 3] | [4] |
| `1 2 3 4 - 6 7 8 9` | 5 | 6 | [6] | [1, 2, 3] | [6] |
| `1 2 3 4 - 6 7 8 9` | 5 | 7 | [7] | [1, 2, 4] | [7] |
| `1 2 3 4 - 6 7 8 9` | 5 | 8 | [8] | [1, 3, 4] | [8] |
| `1 2 3 4 - 6 7 8 9` | 5 | 9 | [9] | [2, 3, 4] | [9] |
| `1 2 3 4 - 6 7 8 9` | 5 | 10 | [1, 9] | [1, 2, 3, 4] | [1, 9] |
| `1 2 3 4 - 6 7 8 9` | 5 | 11 | [2, 9] | [1, 3, 7] | [2, 9] |
| `1 2 3 4 - 6 7 8 9` | 5 | 12 | [3, 9] | [1, 2, 3, 6] | [3, 9] |
| `- - - 4 5 6 7 8 9` | 6 | 9 | [9] | [4, 5] | [9] |
| `- - - 4 5 6 7 8 9` | 6 | 11 | [5, 6] | [4, 7] | [4, 7] |
| `- - - 4 5 6 7 8 9` | 6 | 12 | [4, 8] | [5, 7] | [4, 8] |
| `- 2 3 4 - 6 7 8 9` | 6 | 6 | [6] | [2, 4] | [6] |
| `- 2 3 4 - 6 7 8 9` | 6 | 7 | [7] | [3, 4] | [7] |
| `- 2 3 4 - 6 7 8 9` | 6 | 8 | [8] | [2, 6] | [8] |
| `- 2 3 4 - 6 7 8 9` | 6 | 9 | [9] | [2, 3, 4] | [9] |
| `- 2 3 4 - 6 7 8 9` | 6 | 10 | [4, 6] | [3, 7] | [2, 8] |
| `- 2 3 4 - 6 7 8 9` | 6 | 11 | [2, 9] | [2, 3, 6] | [2, 9] |
| `- 2 3 4 - 6 7 8 9` | 6 | 12 | [3, 9] | [2, 3, 7] | [3, 9] |
| `1 - 3 - 5 6 7 8 9` | 6 | 6 | [6] | [1, 5] | [6] |
| `1 - 3 - 5 6 7 8 9` | 6 | 7 | [7] | [1, 6] | [7] |
| `1 - 3 - 5 6 7 8 9` | 6 | 8 | [8] | [1, 7] | [8] |
| `1 - 3 - 5 6 7 8 9` | 6 | 9 | [9] | [1, 3, 5] | [9] |
| `1 - 3 - 5 6 7 8 9` | 6 | 10 | [1, 9] | [1, 3, 6] | [1, 9] |
| `1 - 3 - 5 6 7 8 9` | 6 | 11 | [3, 8] | [1, 3, 7] | [3, 8] |
| `1 - 3 - 5 6 7 8 9` | 6 | 12 | [3, 9] | [1, 3, 8] | [3, 9] |
| `1 2 3 4 5 - 7 8 9` | 6 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 4 5 - 7 8 9` | 6 | 4 | [4] | [1, 3] | [4] |
| `1 2 3 4 5 - 7 8 9` | 6 | 5 | [5] | [2, 3] | [5] |
| `1 2 3 4 5 - 7 8 9` | 6 | 6 | [2, 4] | [1, 2, 3] | [1, 5], [2, 4] |
| `1 2 3 4 5 - 7 8 9` | 6 | 7 | [7] | [1, 2, 4] | [7] |
| `1 2 3 4 5 - 7 8 9` | 6 | 8 | [8] | [1, 3, 4] | [8] |
| `1 2 3 4 5 - 7 8 9` | 6 | 9 | [9] | [2, 3, 4] | [9] |
| `1 2 3 4 5 - 7 8 9` | 6 | 10 | [1, 9] | [1, 2, 3, 4] | [1, 9] |
| `1 2 3 4 5 - 7 8 9` | 6 | 11 | [2, 9] | [1, 2, 3, 5] | [2, 9] |
| `1 2 3 4 5 - 7 8 9` | 6 | 12 | [3, 9] | [2, 3, 7] | [3, 9] |
| `- - 3 - 5 6 7 8 9` | 7 | 8 | [8] | [3, 5] | [8] |
| `- - 3 - 5 6 7 8 9` | 7 | 9 | [9] | [3, 6] | [9] |
| `- - 3 - 5 6 7 8 9` | 7 | 11 | [5, 6] | [3, 8] | [3, 8] |
| `- - 3 - 5 6 7 8 9` | 7 | 12 | [3, 9] | [5, 7] | [3, 9] |
| `- 2 3 4 5 - 7 8 9` | 7 | 5 | [5] | [2, 3] | [5] |
| `- 2 3 4 5 - 7 8 9` | 7 | 7 | [7] | [3, 4] | [7] |
| `- 2 3 4 5 - 7 8 9` | 7 | 8 | [8] | [3, 5] | [8] |
| `- 2 3 4 5 - 7 8 9` | 7 | 9 | [9] | [2, 3, 4] | [9] |
| `- 2 3 4 5 - 7 8 9` | 7 | 10 | [2, 8] | [2, 3, 5] | [2, 8] |
| `- 2 3 4 5 - 7 8 9` | 7 | 11 | [2, 9] | [3, 8] | [2, 9] |
| `- 2 3 4 5 - 7 8 9` | 7 | 12 | [3, 9] | [2, 3, 7] | [3, 9] |
| `1 - 3 4 - 6 7 8 9` | 7 | 4 | [4] | [1, 3] | [4] |
| `1 - 3 4 - 6 7 8 9` | 7 | 7 | [7] | [3, 4] | [7] |
| `1 - 3 4 - 6 7 8 9` | 7 | 8 | [8] | [1, 3, 4] | [8] |
| `1 - 3 4 - 6 7 8 9` | 7 | 9 | [9] | [1, 8] | [9] |
| `1 - 3 4 - 6 7 8 9` | 7 | 10 | [1, 9] | [1, 3, 6] | [1, 9] |
| `1 - 3 4 - 6 7 8 9` | 7 | 11 | [3, 8] | [1, 3, 7] | [3, 8] |
| `1 - 3 4 - 6 7 8 9` | 7 | 12 | [3, 9] | [1, 4, 7] | [3, 9] |
| `1 2 - - 5 6 7 8 9` | 7 | 6 | [6] | [1, 5] | [6] |
| `1 2 - - 5 6 7 8 9` | 7 | 7 | [7] | [2, 5] | [7] |
| `1 2 - - 5 6 7 8 9` | 7 | 8 | [8] | [1, 2, 5] | [8] |
| `1 2 - - 5 6 7 8 9` | 7 | 9 | [9] | [1, 2, 6] | [9] |
| `1 2 - - 5 6 7 8 9` | 7 | 10 | [1, 9] | [1, 2, 7] | [1, 9] |
| `1 2 - - 5 6 7 8 9` | 7 | 11 | [2, 9] | [1, 2, 8] | [2, 9] |
| `1 2 - - 5 6 7 8 9` | 7 | 12 | [5, 7] | [1, 2, 9] | [5, 7] |
| `1 2 3 4 5 6 - 8 9` | 7 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 4 5 6 - 8 9` | 7 | 4 | [4] | [1, 3] | [4] |
| `1 2 3 4 5 6 - 8 9` | 7 | 5 | [5] | [2, 3] | [5] |
| `1 2 3 4 5 6 - 8 9` | 7 | 6 | [6] | [1, 2, 3] | [6] |
| `1 2 3 4 5 6 - 8 9` | 7 | 7 | [2, 5] | [1, 2, 4] | [1, 6] |
| `1 2 3 4 5 6 - 8 9` | 7 | 8 | [8] | [1, 2, 5] | [8] |
| `1 2 3 4 5 6 - 8 9` | 7 | 9 | [9] | [1, 2, 6] | [9] |
| `1 2 3 4 5 6 - 8 9` | 7 | 10 | [1, 9] | [1, 2, 3, 4] | [1, 9] |
| `1 2 3 4 5 6 - 8 9` | 7 | 11 | [2, 9] | [2, 3, 6] | [2, 9] |
| `1 2 3 4 5 6 - 8 9` | 7 | 12 | [3, 9] | [1, 2, 3, 6] | [3, 9] |
| `- - 3 4 - 6 7 8 9` | 8 | 7 | [7] | [3, 4] | [7] |
| `- - 3 4 - 6 7 8 9` | 8 | 9 | [9] | [3, 6] | [9] |
| `- - 3 4 - 6 7 8 9` | 8 | 10 | [4, 6] | [3, 7] | [3, 7] |
| `- - 3 4 - 6 7 8 9` | 8 | 11 | [3, 8] | [4, 7] | [3, 8] |
| `- - 3 4 - 6 7 8 9` | 8 | 12 | [3, 9] | [4, 8] | [3, 9] |
| `- 2 - - 5 6 7 8 9` | 8 | 7 | [7] | [2, 5] | [7] |
| `- 2 - - 5 6 7 8 9` | 8 | 8 | [8] | [2, 6] | [8] |
| `- 2 - - 5 6 7 8 9` | 8 | 9 | [9] | [2, 7] | [9] |
| `- 2 - - 5 6 7 8 9` | 8 | 11 | [5, 6] | [2, 9] | [2, 9] |
| `- 2 3 4 5 6 - 8 9` | 8 | 5 | [5] | [2, 3] | [5] |
| `- 2 3 4 5 6 - 8 9` | 8 | 6 | [6] | [2, 4] | [6] |
| `- 2 3 4 5 6 - 8 9` | 8 | 7 | [3, 4] | [2, 5] | [2, 5] |
| `- 2 3 4 5 6 - 8 9` | 8 | 8 | [8] | [2, 6] | [8] |
| `- 2 3 4 5 6 - 8 9` | 8 | 9 | [9] | [2, 3, 4] | [9] |
| `- 2 3 4 5 6 - 8 9` | 8 | 10 | [2, 8] | [2, 3, 5] | [2, 8] |
| `- 2 3 4 5 6 - 8 9` | 8 | 11 | [2, 9] | [2, 3, 6] | [2, 9] |
| `- 2 3 4 5 6 - 8 9` | 8 | 12 | [3, 9] | [2, 4, 6] | [3, 9] |
| `1 - 3 4 5 - 7 8 9` | 8 | 4 | [4] | [1, 3] | [4] |
| `1 - 3 4 5 - 7 8 9` | 8 | 5 | [5] | [1, 4] | [5] |
| `1 - 3 4 5 - 7 8 9` | 8 | 7 | [7] | [3, 4] | [7] |
| `1 - 3 4 5 - 7 8 9` | 8 | 8 | [8] | [1, 3, 4] | [8] |
| `1 - 3 4 5 - 7 8 9` | 8 | 9 | [9] | [1, 3, 5] | [9] |
| `1 - 3 4 5 - 7 8 9` | 8 | 10 | [1, 9] | [3, 7] | [1, 9] |
| `1 - 3 4 5 - 7 8 9` | 8 | 11 | [3, 8] | [1, 3, 7] | [3, 8] |
| `1 - 3 4 5 - 7 8 9` | 8 | 12 | [3, 9] | [1, 3, 8] | [3, 9] |
| `1 2 - 4 - 6 7 8 9` | 8 | 6 | [6] | [2, 4] | [6] |
| `1 2 - 4 - 6 7 8 9` | 8 | 7 | [7] | [1, 2, 4] | [7] |
| `1 2 - 4 - 6 7 8 9` | 8 | 8 | [8] | [1, 7] | [8] |
| `1 2 - 4 - 6 7 8 9` | 8 | 9 | [9] | [1, 2, 6] | [9] |
| `1 2 - 4 - 6 7 8 9` | 8 | 10 | [1, 9] | [1, 2, 7] | [1, 9] |
| `1 2 - 4 - 6 7 8 9` | 8 | 11 | [2, 9] | [1, 2, 8] | [2, 9] |
| `1 2 - 4 - 6 7 8 9` | 8 | 12 | [4, 8] | [2, 4, 6] | [4, 8] |
| `1 2 3 4 5 6 7 - 9` | 8 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 4 5 6 7 - 9` | 8 | 4 | [4] | [1, 3] | [4] |
| `1 2 3 4 5 6 7 - 9` | 8 | 5 | [5] | [2, 3] | [5] |
| `1 2 3 4 5 6 7 - 9` | 8 | 6 | [6] | [1, 2, 3] | [6] |
| `1 2 3 4 5 6 7 - 9` | 8 | 7 | [7] | [1, 2, 4] | [7] |
| `1 2 3 4 5 6 7 - 9` | 8 | 8 | [3, 5] | [1, 2, 5] | [1, 7] |
| `1 2 3 4 5 6 7 - 9` | 8 | 9 | [9] | [1, 2, 6] | [9] |
| `1 2 3 4 5 6 7 - 9` | 8 | 10 | [1, 9] | [1, 3, 6] | [1, 9] |
| `1 2 3 4 5 6 7 - 9` | 8 | 11 | [2, 9] | [1, 2, 3, 5] | [2, 9] |
| `1 2 3 4 5 6 7 - 9` | 8 | 12 | [3, 9] | [1, 2, 3, 6] | [3, 9] |
| `- - 3 4 5 - 7 8 9` | 9 | 7 | [7] | [3, 4] | [7] |
| `- - 3 4 5 - 7 8 9` | 9 | 8 | [8] | [3, 5] | [8] |
| `- - 3 4 5 - 7 8 9` | 9 | 9 | [9] | [4, 5] | [9] |
| `- - 3 4 5 - 7 8 9` | 9 | 12 | [3, 9] | [3, 4, 5] | [3, 9] |
| `- 2 - 4 - 6 7 8 9` | 9 | 6 | [6] | [2, 4] | [6] |
| `- 2 - 4 - 6 7 8 9` | 9 | 8 | [8] | [2, 6] | [8] |
| `- 2 - 4 - 6 7 8 9` | 9 | 9 | [9] | [2, 7] | [9] |
| `- 2 - 4 - 6 7 8 9` | 9 | 10 | [4, 6] | [2, 8] | [2, 8] |
| `- 2 - 4 - 6 7 8 9` | 9 | 11 | [2, 9] | [4, 7] | [2, 9] |
| `- 2 - 4 - 6 7 8 9` | 9 | 12 | [4, 8] | [2, 4, 6] | [4, 8] |
| `- 2 3 4 5 6 7 - 9` | 9 | 5 | [5] | [2, 3] | [5] |
| `- 2 3 4 5 6 7 - 9` | 9 | 6 | [6] | [2, 4] | [6] |
| `- 2 3 4 5 6 7 - 9` | 9 | 7 | [7] | [2, 5] | [7] |
| `- 2 3 4 5 6 7 - 9` | 9 | 8 | [3, 5] | [2, 6] | [2, 6] |
| `- 2 3 4 5 6 7 - 9` | 9 | 9 | [9] | [3, 6] | [9] |
| `- 2 3 4 5 6 7 - 9` | 9 | 10 | [3, 7] | [2, 3, 5] | [3, 7] |
| `- 2 3 4 5 6 7 - 9` | 9 | 11 | [2, 9] | [2, 3, 6] | [2, 9] |
| `- 2 3 4 5 6 7 - 9` | 9 | 12 | [3, 9] | [2, 3, 7] | [3, 9] |
| `1 - - - 5 6 7 8 9` | 9 | 6 | [6] | [1, 5] | [6] |
| `1 - - - 5 6 7 8 9` | 9 | 7 | [7] | [1, 6] | [7] |
| `1 - - - 5 6 7 8 9` | 9 | 8 | [8] | [1, 7] | [8] |
| `1 - - - 5 6 7 8 9` | 9 | 9 | [9] | [1, 8] | [9] |
| `1 - - - 5 6 7 8 9` | 9 | 12 | [5, 7] | [1, 5, 6] | [5, 7] |
| `1 - 3 4 5 6 - 8 9` | 9 | 4 | [4] | [1, 3] | [4] |
| `1 - 3 4 5 6 - 8 9` | 9 | 5 | [5] | [1, 4] | [5] |
| `1 - 3 4 5 6 - 8 9` | 9 | 6 | [6] | [1, 5] | [6] |
| `1 - 3 4 5 6 - 8 9` | 9 | 7 | [3, 4] | [1, 6] | [1, 6] |
| `1 - 3 4 5 6 - 8 9` | 9 | 8 | [8] | [1, 3, 4] | [8] |
| `1 - 3 4 5 6 - 8 9` | 9 | 9 | [9] | [3, 6] | [9] |
| `1 - 3 4 5 6 - 8 9` | 9 | 10 | [1, 9] | [1, 3, 6] | [1, 9] |
| `1 - 3 4 5 6 - 8 9` | 9 | 11 | [5, 6] | [1, 4, 6] | [3, 8] |
| `1 - 3 4 5 6 - 8 9` | 9 | 12 | [3, 9] | [1, 3, 8] | [3, 9] |
| `1 2 - 4 5 - 7 8 9` | 9 | 5 | [5] | [1, 4] | [5] |
| `1 2 - 4 5 - 7 8 9` | 9 | 6 | [1, 5] | [2, 4] | [1, 5] |
| `1 2 - 4 5 - 7 8 9` | 9 | 7 | [7] | [1, 2, 4] | [7] |
| `1 2 - 4 5 - 7 8 9` | 9 | 8 | [8] | [1, 2, 5] | [8] |
| `1 2 - 4 5 - 7 8 9` | 9 | 9 | [9] | [2, 7] | [9] |
| `1 2 - 4 5 - 7 8 9` | 9 | 10 | [1, 9] | [1, 2, 7] | [1, 9] |
| `1 2 - 4 5 - 7 8 9` | 9 | 11 | [2, 9] | [1, 2, 8] | [2, 9] |
| `1 2 - 4 5 - 7 8 9` | 9 | 12 | [4, 8] | [1, 2, 4, 5] | [4, 8] |
| `1 2 3 - - 6 7 8 9` | 9 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 - - 6 7 8 9` | 9 | 6 | [6] | [1, 2, 3] | [6] |
| `1 2 3 - - 6 7 8 9` | 9 | 7 | [7] | [1, 6] | [7] |
| `1 2 3 - - 6 7 8 9` | 9 | 8 | [8] | [1, 7] | [8] |
| `1 2 3 - - 6 7 8 9` | 9 | 9 | [9] | [1, 2, 6] | [9] |
| `1 2 3 - - 6 7 8 9` | 9 | 10 | [1, 9] | [1, 2, 7] | [1, 9] |
| `1 2 3 - - 6 7 8 9` | 9 | 11 | [2, 9] | [1, 2, 8] | [2, 9] |
| `1 2 3 - - 6 7 8 9` | 9 | 12 | [3, 9] | [1, 2, 3, 6] | [3, 9] |
| `1 2 3 4 5 6 7 8 -` | 9 | 4 | [4] | [1, 3] | [4] |
| `1 2 3 4 5 6 7 8 -` | 9 | 5 | [5] | [2, 3] | [5] |
| `1 2 3 4 5 6 7 8 -` | 9 | 6 | [6] | [2, 4] | [6] |
| `1 2 3 4 5 6 7 8 -` | 9 | 7 | [7] | [1, 2, 4] | [7] |
| `1 2 3 4 5 6 7 8 -` | 9 | 8 | [8] | [1, 3, 4] | [8] |
| `1 2 3 4 5 6 7 8 -` | 9 | 9 | [1, 8] | [2, 3, 4] | [1, 8] |
| `1 2 3 4 5 6 7 8 -` | 9 | 10 | [2, 8] | [1, 2, 3, 4] | [2, 8] |
| `1 2 3 4 5 6 7 8 -` | 9 | 11 | [3, 8] | [1, 2, 3, 5] | [3, 8] |
| `1 2 3 4 5 6 7 8 -` | 9 | 12 | [4, 8] | [1, 2, 3, 6] | [4, 8] |
| `- - 3 4 5 6 - 8 9` | 10 | 8 | [8] | [3, 5] | [8] |
| `- - 3 4 5 6 - 8 9` | 10 | 9 | [9] | [3, 6] | [9] |
| `- - 3 4 5 6 - 8 9` | 10 | 11 | [5, 6] | [3, 8] | [3, 8] |
| `- - 3 4 5 6 - 8 9` | 10 | 12 | [3, 9] | [3, 4, 5] | [3, 9] |
| `- 2 - 4 5 - 7 8 9` | 10 | 7 | [7] | [2, 5] | [7] |
| `- 2 - 4 5 - 7 8 9` | 10 | 9 | [9] | [2, 7] | [9] |
| `- 2 - 4 5 - 7 8 9` | 10 | 11 | [2, 9] | [2, 4, 5] | [2, 9] |
| `- 2 - 4 5 - 7 8 9` | 10 | 12 | [5, 7] | [4, 8] | [4, 8] |
| `- 2 3 - - 6 7 8 9` | 10 | 8 | [8] | [2, 6] | [8] |
| `- 2 3 - - 6 7 8 9` | 10 | 9 | [9] | [2, 7] | [9] |
| `- 2 3 - - 6 7 8 9` | 10 | 10 | [3, 7] | [2, 8] | [2, 8] |
| `- 2 3 - - 6 7 8 9` | 10 | 11 | [2, 9] | [2, 3, 6] | [2, 9] |
| `- 2 3 - - 6 7 8 9` | 10 | 12 | [3, 9] | [2, 3, 7] | [3, 9] |
| `- 2 3 4 5 6 7 8 -` | 10 | 5 | [5] | [2, 3] | [5] |
| `- 2 3 4 5 6 7 8 -` | 10 | 6 | [6] | [2, 4] | [6] |
| `- 2 3 4 5 6 7 8 -` | 10 | 7 | [7] | [3, 4] | [7] |
| `- 2 3 4 5 6 7 8 -` | 10 | 8 | [8] | [3, 5] | [8] |
| `- 2 3 4 5 6 7 8 -` | 10 | 9 | [4, 5] | [2, 3, 4] | [2, 7] |
| `- 2 3 4 5 6 7 8 -` | 10 | 10 | [2, 8] | [2, 3, 5] | [2, 8] |
| `- 2 3 4 5 6 7 8 -` | 10 | 11 | [3, 8] | [2, 3, 6] | [3, 8] |
| `- 2 3 4 5 6 7 8 -` | 10 | 12 | [4, 8] | [2, 4, 6] | [4, 8] |
| `1 - - 4 - 6 7 8 9` | 10 | 7 | [7] | [1, 6] | [7] |
| `1 - - 4 - 6 7 8 9` | 10 | 8 | [8] | [1, 7] | [8] |
| `1 - - 4 - 6 7 8 9` | 10 | 9 | [9] | [1, 8] | [9] |
| `1 - - 4 - 6 7 8 9` | 10 | 10 | [1, 9] | [4, 6] | [1, 9] |
| `1 - - 4 - 6 7 8 9` | 10 | 11 | [4, 7] | [1, 4, 6] | [4, 7] |
| `1 - - 4 - 6 7 8 9` | 10 | 12 | [4, 8] | [1, 4, 7] | [4, 8] |
| `1 - 3 4 5 6 7 - 9` | 10 | 4 | [4] | [1, 3] | [4] |
| `1 - 3 4 5 6 7 - 9` | 10 | 5 | [5] | [1, 4] | [5] |
| `1 - 3 4 5 6 7 - 9` | 10 | 6 | [6] | [1, 5] | [6] |
| `1 - 3 4 5 6 7 - 9` | 10 | 7 | [7] | [1, 6] | [7] |
| `1 - 3 4 5 6 7 - 9` | 10 | 8 | [3, 5] | [1, 3, 4] | [1, 7] |
| `1 - 3 4 5 6 7 - 9` | 10 | 9 | [9] | [1, 3, 5] | [9] |
| `1 - 3 4 5 6 7 - 9` | 10 | 10 | [1, 9] | [1, 3, 6] | [1, 9] |
| `1 - 3 4 5 6 7 - 9` | 10 | 11 | [4, 7] | [1, 3, 7] | [4, 7] |
| `1 - 3 4 5 6 7 - 9` | 10 | 12 | [3, 9] | [1, 4, 7] | [3, 9] |
| `1 2 - 4 5 6 - 8 9` | 10 | 5 | [5] | [1, 4] | [5] |
| `1 2 - 4 5 6 - 8 9` | 10 | 6 | [6] | [1, 5] | [6] |
| `1 2 - 4 5 6 - 8 9` | 10 | 7 | [2, 5] | [1, 2, 4] | [1, 6] |
| `1 2 - 4 5 6 - 8 9` | 10 | 8 | [8] | [2, 6] | [8] |
| `1 2 - 4 5 6 - 8 9` | 10 | 9 | [9] | [1, 2, 6] | [9] |
| `1 2 - 4 5 6 - 8 9` | 10 | 10 | [1, 9] | [1, 4, 5] | [1, 9] |
| `1 2 - 4 5 6 - 8 9` | 10 | 11 | [2, 9] | [1, 2, 8] | [2, 9] |
| `1 2 - 4 5 6 - 8 9` | 10 | 12 | [4, 8] | [1, 2, 4, 5] | [4, 8] |
| `1 2 3 - 5 - 7 8 9` | 10 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 - 5 - 7 8 9` | 10 | 5 | [5] | [2, 3] | [5] |
| `1 2 3 - 5 - 7 8 9` | 10 | 6 | [1, 5] | [1, 2, 3] | [1, 5] |
| `1 2 3 - 5 - 7 8 9` | 10 | 7 | [7] | [2, 5] | [7] |
| `1 2 3 - 5 - 7 8 9` | 10 | 8 | [8] | [1, 2, 5] | [8] |
| `1 2 3 - 5 - 7 8 9` | 10 | 9 | [9] | [1, 3, 5] | [9] |
| `1 2 3 - 5 - 7 8 9` | 10 | 10 | [2, 8] | [1, 2, 7] | [1, 9] |
| `1 2 3 - 5 - 7 8 9` | 10 | 11 | [2, 9] | [1, 2, 3, 5] | [2, 9] |
| `1 2 3 - 5 - 7 8 9` | 10 | 12 | [3, 9] | [1, 3, 8] | [3, 9] |
| `- - 3 4 5 6 7 - 9` | 11 | 7 | [7] | [3, 4] | [7] |
| `- - 3 4 5 6 7 - 9` | 11 | 9 | [9] | [3, 6] | [9] |
| `- - 3 4 5 6 7 - 9` | 11 | 10 | [4, 6] | [3, 7] | [3, 7] |
| `- - 3 4 5 6 7 - 9` | 11 | 12 | [3, 9] | [3, 4, 5] | [3, 9] |
| `- 2 - 4 5 6 - 8 9` | 11 | 6 | [6] | [2, 4] | [6] |
| `- 2 - 4 5 6 - 8 9` | 11 | 8 | [8] | [2, 6] | [8] |
| `- 2 - 4 5 6 - 8 9` | 11 | 9 | [9] | [4, 5] | [9] |
| `- 2 - 4 5 6 - 8 9` | 11 | 10 | [4, 6] | [2, 8] | [2, 8] |
| `- 2 - 4 5 6 - 8 9` | 11 | 11 | [2, 9] | [2, 4, 5] | [2, 9] |
| `- 2 - 4 5 6 - 8 9` | 11 | 12 | [4, 8] | [2, 4, 6] | [4, 8] |
| `- 2 3 - 5 - 7 8 9` | 11 | 5 | [5] | [2, 3] | [5] |
| `- 2 3 - 5 - 7 8 9` | 11 | 7 | [7] | [2, 5] | [7] |
| `- 2 3 - 5 - 7 8 9` | 11 | 8 | [8] | [3, 5] | [8] |
| `- 2 3 - 5 - 7 8 9` | 11 | 9 | [9] | [2, 7] | [9] |
| `- 2 3 - 5 - 7 8 9` | 11 | 10 | [2, 8] | [2, 3, 5] | [2, 8] |
| `- 2 3 - 5 - 7 8 9` | 11 | 11 | [2, 9] | [3, 8] | [2, 9] |
| `- 2 3 - 5 - 7 8 9` | 11 | 12 | [3, 9] | [2, 3, 7] | [3, 9] |
| `1 - - 4 5 - 7 8 9` | 11 | 5 | [5] | [1, 4] | [5] |
| `1 - - 4 5 - 7 8 9` | 11 | 8 | [8] | [1, 7] | [8] |
| `1 - - 4 5 - 7 8 9` | 11 | 9 | [9] | [1, 8] | [9] |
| `1 - - 4 5 - 7 8 9` | 11 | 10 | [1, 9] | [1, 4, 5] | [1, 9] |
| `1 - - 4 5 - 7 8 9` | 11 | 12 | [4, 8] | [1, 4, 7] | [4, 8] |
| `1 - 3 - - 6 7 8 9` | 11 | 7 | [7] | [1, 6] | [7] |
| `1 - 3 - - 6 7 8 9` | 11 | 8 | [8] | [1, 7] | [8] |
| `1 - 3 - - 6 7 8 9` | 11 | 9 | [9] | [1, 8] | [9] |
| `1 - 3 - - 6 7 8 9` | 11 | 10 | [1, 9] | [1, 3, 6] | [1, 9] |
| `1 - 3 - - 6 7 8 9` | 11 | 11 | [3, 8] | [1, 3, 7] | [3, 8] |
| `1 - 3 - - 6 7 8 9` | 11 | 12 | [3, 9] | [1, 3, 8] | [3, 9] |
| `1 - 3 4 5 6 7 8 -` | 11 | 5 | [5] | [1, 4] | [5] |
| `1 - 3 4 5 6 7 8 -` | 11 | 6 | [6] | [1, 5] | [6] |
| `1 - 3 4 5 6 7 8 -` | 11 | 7 | [7] | [3, 4] | [7] |
| `1 - 3 4 5 6 7 8 -` | 11 | 8 | [8] | [1, 3, 4] | [8] |
| `1 - 3 4 5 6 7 8 -` | 11 | 9 | [1, 8] | [1, 3, 5] | [1, 8] |
| `1 - 3 4 5 6 7 8 -` | 11 | 10 | [4, 6] | [1, 3, 6] | [3, 7] |
| `1 - 3 4 5 6 7 8 -` | 11 | 11 | [3, 8] | [1, 4, 6] | [3, 8] |
| `1 - 3 4 5 6 7 8 -` | 11 | 12 | [4, 8] | [1, 4, 7] | [4, 8] |
| `1 2 - 4 5 6 7 - 9` | 11 | 5 | [5] | [1, 4] | [5] |
| `1 2 - 4 5 6 7 - 9` | 11 | 6 | [6] | [1, 5] | [6] |
| `1 2 - 4 5 6 7 - 9` | 11 | 7 | [7] | [1, 6] | [7] |
| `1 2 - 4 5 6 7 - 9` | 11 | 8 | [1, 7] | [1, 2, 5] | [1, 7] |
| `1 2 - 4 5 6 7 - 9` | 11 | 9 | [9] | [1, 2, 6] | [9] |
| `1 2 - 4 5 6 7 - 9` | 11 | 10 | [1, 9] | [1, 2, 7] | [1, 9] |
| `1 2 - 4 5 6 7 - 9` | 11 | 11 | [2, 9] | [1, 4, 6] | [2, 9] |
| `1 2 - 4 5 6 7 - 9` | 11 | 12 | [5, 7] | [1, 2, 4, 5] | [5, 7] |
| `1 2 3 - 5 6 - 8 9` | 11 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 - 5 6 - 8 9` | 11 | 5 | [5] | [2, 3] | [5] |
| `1 2 3 - 5 6 - 8 9` | 11 | 6 | [6] | [1, 2, 3] | [6] |
| `1 2 3 - 5 6 - 8 9` | 11 | 7 | [2, 5] | [1, 6] | [1, 6] |
| `1 2 3 - 5 6 - 8 9` | 11 | 8 | [8] | [1, 2, 5] | [8] |
| `1 2 3 - 5 6 - 8 9` | 11 | 9 | [9] | [1, 2, 6] | [9] |
| `1 2 3 - 5 6 - 8 9` | 11 | 10 | [1, 9] | [2, 3, 5] | [1, 9] |
| `1 2 3 - 5 6 - 8 9` | 11 | 11 | [2, 9] | [1, 2, 3, 5] | [2, 9] |
| `1 2 3 - 5 6 - 8 9` | 11 | 12 | [3, 9] | [1, 2, 3, 6] | [3, 9] |
| `1 2 3 4 - - 7 8 9` | 11 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 4 - - 7 8 9` | 11 | 4 | [4] | [1, 3] | [4] |
| `1 2 3 4 - - 7 8 9` | 11 | 5 | [1, 4] | [2, 3] | [1, 4] |
| `1 2 3 4 - - 7 8 9` | 11 | 6 | [2, 4] | [1, 2, 3] | [2, 4] |
| `1 2 3 4 - - 7 8 9` | 11 | 7 | [7] | [1, 2, 4] | [7] |
| `1 2 3 4 - - 7 8 9` | 11 | 8 | [8] | [1, 3, 4] | [8] |
| `1 2 3 4 - - 7 8 9` | 11 | 9 | [9] | [2, 3, 4] | [9] |
| `1 2 3 4 - - 7 8 9` | 11 | 10 | [1, 9] | [1, 2, 3, 4] | [1, 9] |
| `1 2 3 4 - - 7 8 9` | 11 | 11 | [3, 8] | [1, 2, 8] | [2, 9] |
| `1 2 3 4 - - 7 8 9` | 11 | 12 | [3, 9] | [2, 3, 7] | [3, 9] |
| `- - - 4 5 - 7 8 9` | 12 | 9 | [9] | [4, 5] | [9] |
| `- - - 4 5 - 7 8 9` | 12 | 12 | [4, 8] | [5, 7] | [4, 8] |
| `- - 3 - - 6 7 8 9` | 12 | 9 | [9] | [3, 6] | [9] |
| `- - 3 4 5 6 7 8 -` | 12 | 7 | [7] | [3, 4] | [7] |
| `- - 3 4 5 6 7 8 -` | 12 | 8 | [8] | [3, 5] | [8] |
| `- - 3 4 5 6 7 8 -` | 12 | 9 | [4, 5] | [3, 6] | [3, 6] |
| `- - 3 4 5 6 7 8 -` | 12 | 10 | [3, 7] | [4, 6] | [3, 7] |
| `- - 3 4 5 6 7 8 -` | 12 | 11 | [3, 8] | [4, 7] | [3, 8] |
| `- - 3 4 5 6 7 8 -` | 12 | 12 | [4, 8] | [3, 4, 5] | [4, 8] |
| `- 2 - 4 5 6 7 - 9` | 12 | 7 | [7] | [2, 5] | [7] |
| `- 2 - 4 5 6 7 - 9` | 12 | 9 | [9] | [2, 7] | [9] |
| `- 2 - 4 5 6 7 - 9` | 12 | 11 | [2, 9] | [2, 4, 5] | [2, 9] |
| `- 2 - 4 5 6 7 - 9` | 12 | 12 | [5, 7] | [2, 4, 6] | [5, 7] |
| `- 2 3 - 5 6 - 8 9` | 12 | 5 | [5] | [2, 3] | [5] |
| `- 2 3 - 5 6 - 8 9` | 12 | 8 | [8] | [2, 6] | [8] |
| `- 2 3 - 5 6 - 8 9` | 12 | 9 | [9] | [3, 6] | [9] |
| `- 2 3 - 5 6 - 8 9` | 12 | 10 | [2, 8] | [2, 3, 5] | [2, 8] |
| `- 2 3 - 5 6 - 8 9` | 12 | 11 | [3, 8] | [2, 3, 6] | [2, 9] |
| `- 2 3 4 - - 7 8 9` | 12 | 7 | [7] | [3, 4] | [7] |
| `- 2 3 4 - - 7 8 9` | 12 | 9 | [9] | [2, 3, 4] | [9] |
| `- 2 3 4 - - 7 8 9` | 12 | 10 | [3, 7] | [2, 8] | [2, 8] |
| `- 2 3 4 - - 7 8 9` | 12 | 11 | [2, 9] | [3, 8] | [2, 9] |
| `- 2 3 4 - - 7 8 9` | 12 | 12 | [3, 9] | [2, 3, 7] | [3, 9] |
| `1 - - 4 5 6 - 8 9` | 12 | 5 | [5] | [1, 4] | [5] |
| `1 - - 4 5 6 - 8 9` | 12 | 9 | [9] | [1, 8] | [9] |
| `1 - - 4 5 6 - 8 9` | 12 | 10 | [1, 9] | [1, 4, 5] | [1, 9] |
| `1 - - 4 5 6 - 8 9` | 12 | 11 | [5, 6] | [1, 4, 6] | [5, 6] |
| `1 - - 4 5 6 - 8 9` | 12 | 12 | [4, 8] | [1, 5, 6] | [4, 8] |
| `1 - 3 - 5 - 7 8 9` | 12 | 8 | [8] | [1, 7] | [8] |
| `1 - 3 - 5 - 7 8 9` | 12 | 9 | [9] | [1, 3, 5] | [9] |
| `1 - 3 - 5 - 7 8 9` | 12 | 10 | [1, 9] | [3, 7] | [1, 9] |
| `1 - 3 - 5 - 7 8 9` | 12 | 11 | [3, 8] | [1, 3, 7] | [3, 8] |
| `1 - 3 - 5 - 7 8 9` | 12 | 12 | [3, 9] | [1, 3, 8] | [3, 9] |
| `1 2 - - - 6 7 8 9` | 12 | 7 | [7] | [1, 6] | [7] |
| `1 2 - - - 6 7 8 9` | 12 | 8 | [8] | [2, 6] | [8] |
| `1 2 - - - 6 7 8 9` | 12 | 9 | [9] | [1, 2, 6] | [9] |
| `1 2 - - - 6 7 8 9` | 12 | 10 | [1, 9] | [1, 2, 7] | [1, 9] |
| `1 2 - - - 6 7 8 9` | 12 | 11 | [2, 9] | [1, 2, 8] | [2, 9] |
| `1 2 - 4 5 6 7 8 -` | 12 | 5 | [5] | [1, 4] | [5] |
| `1 2 - 4 5 6 7 8 -` | 12 | 6 | [6] | [2, 4] | [6] |
| `1 2 - 4 5 6 7 8 -` | 12 | 7 | [7] | [1, 2, 4] | [7] |
| `1 2 - 4 5 6 7 8 -` | 12 | 8 | [8] | [1, 2, 5] | [8] |
| `1 2 - 4 5 6 7 8 -` | 12 | 9 | [1, 8] | [1, 2, 6] | [1, 8] |
| `1 2 - 4 5 6 7 8 -` | 12 | 10 | [2, 8] | [1, 4, 5] | [2, 8] |
| `1 2 - 4 5 6 7 8 -` | 12 | 11 | [5, 6] | [2, 4, 5] | [4, 7] |
| `1 2 - 4 5 6 7 8 -` | 12 | 12 | [5, 7] | [1, 2, 4, 5] | [4, 8] |
| `1 2 3 - 5 6 7 - 9` | 12 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 - 5 6 7 - 9` | 12 | 5 | [5] | [2, 3] | [5] |
| `1 2 3 - 5 6 7 - 9` | 12 | 6 | [6] | [1, 2, 3] | [6] |
| `1 2 3 - 5 6 7 - 9` | 12 | 7 | [7] | [1, 6] | [7] |
| `1 2 3 - 5 6 7 - 9` | 12 | 8 | [2, 6] | [1, 2, 5] | [1, 7] |
| `1 2 3 - 5 6 7 - 9` | 12 | 9 | [9] | [1, 3, 5] | [9] |
| `1 2 3 - 5 6 7 - 9` | 12 | 10 | [1, 9] | [1, 3, 6] | [1, 9] |
| `1 2 3 - 5 6 7 - 9` | 12 | 11 | [2, 9] | [1, 2, 3, 5] | [2, 9] |
| `1 2 3 - 5 6 7 - 9` | 12 | 12 | [3, 9] | [1, 2, 3, 6] | [3, 9] |
| `1 2 3 4 - 6 - 8 9` | 12 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 4 - 6 - 8 9` | 12 | 4 | [4] | [1, 3] | [4] |
| `1 2 3 4 - 6 - 8 9` | 12 | 5 | [2, 3] | [1, 4] | [1, 4] |
| `1 2 3 4 - 6 - 8 9` | 12 | 6 | [6] | [1, 2, 3] | [6] |
| `1 2 3 4 - 6 - 8 9` | 12 | 7 | [1, 6] | [1, 2, 4] | [1, 6] |
| `1 2 3 4 - 6 - 8 9` | 12 | 8 | [8] | [1, 3, 4] | [8] |
| `1 2 3 4 - 6 - 8 9` | 12 | 9 | [9] | [1, 2, 6] | [9] |
| `1 2 3 4 - 6 - 8 9` | 12 | 10 | [1, 9] | [1, 2, 3, 4] | [1, 9] |
| `1 2 3 4 - 6 - 8 9` | 12 | 11 | [2, 9] | [2, 3, 6] | [2, 9] |
| `1 2 3 4 - 6 - 8 9` | 12 | 12 | [3, 9] | [1, 2, 3, 6] | [3, 9] |
| `- - - 4 5 6 - 8 9` | 13 | 9 | [9] | [4, 5] | [9] |
| `- - 3 - 5 - 7 8 9` | 13 | 8 | [8] | [3, 5] | [8] |
| `- - 3 - 5 - 7 8 9` | 13 | 12 | [5, 7] | [3, 9] | [3, 9] |
| `- 2 - - - 6 7 8 9` | 13 | 8 | [8] | [2, 6] | [8] |
| `- 2 - - - 6 7 8 9` | 13 | 9 | [9] | [2, 7] | [9] |
| `- 2 - 4 5 6 7 8 -` | 13 | 6 | [6] | [2, 4] | [6] |
| `- 2 - 4 5 6 7 8 -` | 13 | 7 | [7] | [2, 5] | [7] |
| `- 2 - 4 5 6 7 8 -` | 13 | 8 | [8] | [2, 6] | [8] |
| `- 2 - 4 5 6 7 8 -` | 13 | 10 | [2, 8] | [4, 6] | [2, 8] |
| `- 2 - 4 5 6 7 8 -` | 13 | 11 | [5, 6] | [2, 4, 5] | [4, 7] |
| `- 2 - 4 5 6 7 8 -` | 13 | 12 | [4, 8] | [2, 4, 6] | [4, 8] |
| `- 2 3 - 5 6 7 - 9` | 13 | 5 | [5] | [2, 3] | [5] |
| `- 2 3 - 5 6 7 - 9` | 13 | 7 | [7] | [2, 5] | [7] |
| `- 2 3 - 5 6 7 - 9` | 13 | 9 | [9] | [3, 6] | [9] |
| `- 2 3 - 5 6 7 - 9` | 13 | 10 | [3, 7] | [2, 3, 5] | [3, 7] |
| `- 2 3 - 5 6 7 - 9` | 13 | 11 | [2, 9] | [2, 3, 6] | [2, 9] |
| `- 2 3 - 5 6 7 - 9` | 13 | 12 | [3, 9] | [2, 3, 7] | [3, 9] |
| `- 2 3 4 - 6 - 8 9` | 13 | 6 | [6] | [2, 4] | [6] |
| `- 2 3 4 - 6 - 8 9` | 13 | 8 | [8] | [2, 6] | [8] |
| `- 2 3 4 - 6 - 8 9` | 13 | 9 | [9] | [2, 3, 4] | [9] |
| `- 2 3 4 - 6 - 8 9` | 13 | 11 | [2, 9] | [2, 3, 6] | [2, 9] |
| `- 2 3 4 - 6 - 8 9` | 13 | 12 | [4, 8] | [2, 4, 6] | [3, 9] |
| `1 - - 4 5 6 7 - 9` | 13 | 5 | [5] | [1, 4] | [5] |
| `1 - - 4 5 6 7 - 9` | 13 | 6 | [6] | [1, 5] | [6] |
| `1 - - 4 5 6 7 - 9` | 13 | 7 | [7] | [1, 6] | [7] |
| `1 - - 4 5 6 7 - 9` | 13 | 9 | [9] | [4, 5] | [9] |
| `1 - - 4 5 6 7 - 9` | 13 | 10 | [1, 9] | [1, 4, 5] | [1, 9] |
| `1 - - 4 5 6 7 - 9` | 13 | 11 | [5, 6] | [1, 4, 6] | [4, 7] |
| `1 - - 4 5 6 7 - 9` | 13 | 12 | [5, 7] | [1, 5, 6] | [5, 7] |
| `1 - 3 - 5 6 - 8 9` | 13 | 6 | [6] | [1, 5] | [6] |
| `1 - 3 - 5 6 - 8 9` | 13 | 8 | [8] | [3, 5] | [8] |
| `1 - 3 - 5 6 - 8 9` | 13 | 9 | [9] | [1, 3, 5] | [9] |
| `1 - 3 - 5 6 - 8 9` | 13 | 10 | [1, 9] | [1, 3, 6] | [1, 9] |
| `1 - 3 - 5 6 - 8 9` | 13 | 11 | [3, 8] | [5, 6] | [3, 8] |
| `1 - 3 - 5 6 - 8 9` | 13 | 12 | [3, 9] | [1, 3, 8] | [3, 9] |
| `1 - 3 4 - - 7 8 9` | 13 | 4 | [4] | [1, 3] | [4] |
| `1 - 3 4 - - 7 8 9` | 13 | 7 | [7] | [3, 4] | [7] |
| `1 - 3 4 - - 7 8 9` | 13 | 8 | [8] | [1, 3, 4] | [8] |
| `1 - 3 4 - - 7 8 9` | 13 | 9 | [9] | [1, 8] | [9] |
| `1 - 3 4 - - 7 8 9` | 13 | 10 | [1, 9] | [3, 7] | [1, 9] |
| `1 - 3 4 - - 7 8 9` | 13 | 11 | [3, 8] | [1, 3, 7] | [3, 8] |
| `1 - 3 4 - - 7 8 9` | 13 | 12 | [3, 9] | [1, 3, 8] | [3, 9] |
| `1 2 - - 5 - 7 8 9` | 13 | 7 | [7] | [2, 5] | [7] |
| `1 2 - - 5 - 7 8 9` | 13 | 8 | [8] | [1, 2, 5] | [8] |
| `1 2 - - 5 - 7 8 9` | 13 | 9 | [9] | [1, 8] | [9] |
| `1 2 - - 5 - 7 8 9` | 13 | 10 | [2, 8] | [1, 2, 7] | [1, 9] |
| `1 2 - - 5 - 7 8 9` | 13 | 11 | [2, 9] | [1, 2, 8] | [2, 9] |
| `1 2 - - 5 - 7 8 9` | 13 | 12 | [5, 7] | [1, 2, 9] | [5, 7] |
| `1 2 3 - 5 6 7 8 -` | 13 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 - 5 6 7 8 -` | 13 | 5 | [5] | [2, 3] | [5] |
| `1 2 3 - 5 6 7 8 -` | 13 | 6 | [6] | [1, 2, 3] | [6] |
| `1 2 3 - 5 6 7 8 -` | 13 | 7 | [7] | [1, 6] | [7] |
| `1 2 3 - 5 6 7 8 -` | 13 | 8 | [8] | [1, 2, 5] | [8] |
| `1 2 3 - 5 6 7 8 -` | 13 | 9 | [1, 8] | [1, 2, 6] | [1, 8] |
| `1 2 3 - 5 6 7 8 -` | 13 | 10 | [2, 8] | [1, 2, 7] | [2, 8] |
| `1 2 3 - 5 6 7 8 -` | 13 | 11 | [3, 8] | [1, 2, 3, 5] | [3, 8] |
| `1 2 3 - 5 6 7 8 -` | 13 | 12 | [5, 7] | [1, 2, 3, 6] | [5, 7] |
| `1 2 3 4 - 6 7 - 9` | 13 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 4 - 6 7 - 9` | 13 | 4 | [4] | [1, 3] | [4] |
| `1 2 3 4 - 6 7 - 9` | 13 | 6 | [6] | [1, 2, 3] | [6] |
| `1 2 3 4 - 6 7 - 9` | 13 | 7 | [7] | [1, 2, 4] | [7] |
| `1 2 3 4 - 6 7 - 9` | 13 | 8 | [1, 7] | [1, 3, 4] | [1, 7] |
| `1 2 3 4 - 6 7 - 9` | 13 | 9 | [9] | [1, 2, 6] | [9] |
| `1 2 3 4 - 6 7 - 9` | 13 | 10 | [1, 9] | [1, 2, 3, 4] | [1, 9] |
| `1 2 3 4 - 6 7 - 9` | 13 | 11 | [2, 9] | [1, 4, 6] | [2, 9] |
| `1 2 3 4 - 6 7 - 9` | 13 | 12 | [3, 9] | [1, 2, 3, 6] | [3, 9] |
| `1 2 3 4 5 - - 8 9` | 13 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 4 5 - - 8 9` | 13 | 4 | [4] | [1, 3] | [4] |
| `1 2 3 4 5 - - 8 9` | 13 | 5 | [5] | [2, 3] | [5] |
| `1 2 3 4 5 - - 8 9` | 13 | 6 | [1, 5] | [1, 2, 3] | [1, 5] |
| `1 2 3 4 5 - - 8 9` | 13 | 7 | [3, 4] | [1, 2, 4] | [2, 5] |
| `1 2 3 4 5 - - 8 9` | 13 | 8 | [8] | [1, 2, 5] | [8] |
| `1 2 3 4 5 - - 8 9` | 13 | 9 | [9] | [2, 3, 4] | [9] |
| `1 2 3 4 5 - - 8 9` | 13 | 10 | [1, 9] | [1, 2, 3, 4] | [1, 9] |
| `1 2 3 4 5 - - 8 9` | 13 | 11 | [2, 9] | [1, 2, 3, 5] | [2, 9] |
| `1 2 3 4 5 - - 8 9` | 13 | 12 | [3, 9] | [1, 2, 4, 5] | [3, 9] |
| `- - - 4 5 6 7 - 9` | 14 | 9 | [9] | [4, 5] | [9] |
| `- - - 4 5 6 7 - 9` | 14 | 11 | [4, 7] | [5, 6] | [4, 7] |
| `- - 3 - 5 6 - 8 9` | 14 | 8 | [8] | [3, 5] | [8] |
| `- - 3 - 5 6 - 8 9` | 14 | 9 | [9] | [3, 6] | [9] |
| `- - 3 - 5 6 - 8 9` | 14 | 11 | [5, 6] | [3, 8] | [3, 8] |
| `- - 3 4 - - 7 8 9` | 14 | 7 | [7] | [3, 4] | [7] |
| `- - 3 4 - - 7 8 9` | 14 | 11 | [4, 7] | [3, 8] | [3, 8] |
| `- - 3 4 - - 7 8 9` | 14 | 12 | [3, 9] | [4, 8] | [3, 9] |
| `- 2 - - 5 - 7 8 9` | 14 | 7 | [7] | [2, 5] | [7] |
| `- 2 - - 5 - 7 8 9` | 14 | 9 | [9] | [2, 7] | [9] |
| `- 2 3 - 5 6 7 8 -` | 14 | 5 | [5] | [2, 3] | [5] |
| `- 2 3 - 5 6 7 8 -` | 14 | 7 | [7] | [2, 5] | [7] |
| `- 2 3 - 5 6 7 8 -` | 14 | 8 | [8] | [2, 6] | [8] |
| `- 2 3 - 5 6 7 8 -` | 14 | 9 | [3, 6] | [2, 7] | [2, 7] |
| `- 2 3 - 5 6 7 8 -` | 14 | 10 | [2, 8] | [2, 3, 5] | [2, 8] |
| `- 2 3 - 5 6 7 8 -` | 14 | 11 | [3, 8] | [2, 3, 6] | [3, 8] |
| `- 2 3 - 5 6 7 8 -` | 14 | 12 | [5, 7] | [2, 3, 7] | [5, 7] |
| `- 2 3 4 - 6 7 - 9` | 14 | 6 | [6] | [2, 4] | [6] |
| `- 2 3 4 - 6 7 - 9` | 14 | 7 | [7] | [3, 4] | [7] |
| `- 2 3 4 - 6 7 - 9` | 14 | 9 | [9] | [2, 3, 4] | [9] |
| `- 2 3 4 - 6 7 - 9` | 14 | 10 | [3, 7], [4, 6] | [4, 6] | [3, 7], [4, 6] |
| `- 2 3 4 - 6 7 - 9` | 14 | 11 | [2, 9] | [2, 3, 6] | [2, 9] |
| `- 2 3 4 - 6 7 - 9` | 14 | 12 | [3, 9] | [2, 3, 7] | [3, 9] |
| `- 2 3 4 5 - - 8 9` | 14 | 5 | [5] | [2, 3] | [5] |
| `- 2 3 4 5 - - 8 9` | 14 | 7 | [3, 4] | [2, 5] | [2, 5] |
| `- 2 3 4 5 - - 8 9` | 14 | 8 | [8] | [3, 5] | [8] |
| `- 2 3 4 5 - - 8 9` | 14 | 9 | [9] | [2, 3, 4] | [9] |
| `- 2 3 4 5 - - 8 9` | 14 | 10 | [2, 8] | [2, 3, 5] | [2, 8] |
| `- 2 3 4 5 - - 8 9` | 14 | 11 | [2, 9] | [2, 4, 5] | [2, 9] |
| `- 2 3 4 5 - - 8 9` | 14 | 12 | [3, 9] | [3, 4, 5] | [3, 9] |
| `1 - - - - 6 7 8 9` | 14 | 7 | [7] | [1, 6] | [7] |
| `1 - - - - 6 7 8 9` | 14 | 8 | [8] | [1, 7] | [8] |
| `1 - - - - 6 7 8 9` | 14 | 9 | [9] | [1, 8] | [9] |
| `1 - - 4 5 6 7 8 -` | 14 | 5 | [5] | [1, 4] | [5] |
| `1 - - 4 5 6 7 8 -` | 14 | 6 | [6] | [1, 5] | [6] |
| `1 - - 4 5 6 7 8 -` | 14 | 7 | [7] | [1, 6] | [7] |
| `1 - - 4 5 6 7 8 -` | 14 | 8 | [8] | [1, 7] | [8] |
| `1 - - 4 5 6 7 8 -` | 14 | 9 | [1, 8] | [4, 5] | [1, 8] |
| `1 - - 4 5 6 7 8 -` | 14 | 10 | [4, 6] | [1, 4, 5] | [4, 6] |
| `1 - - 4 5 6 7 8 -` | 14 | 11 | [4, 7] | [1, 4, 6] | [4, 7] |
| `1 - - 4 5 6 7 8 -` | 14 | 12 | [5, 7] | [1, 4, 7] | [4, 8] |
| `1 - 3 - 5 6 7 - 9` | 14 | 6 | [6] | [1, 5] | [6] |
| `1 - 3 - 5 6 7 - 9` | 14 | 7 | [7] | [1, 6] | [7] |
| `1 - 3 - 5 6 7 - 9` | 14 | 8 | [3, 5] | [1, 7] | [1, 7] |
| `1 - 3 - 5 6 7 - 9` | 14 | 9 | [9] | [1, 3, 5] | [9] |
| `1 - 3 - 5 6 7 - 9` | 14 | 10 | [1, 9] | [1, 3, 6] | [1, 9] |
| `1 - 3 - 5 6 7 - 9` | 14 | 11 | [5, 6] | [1, 3, 7] | [5, 6] |
| `1 - 3 - 5 6 7 - 9` | 14 | 12 | [3, 9] | [1, 5, 6] | [3, 9] |
| `1 - 3 4 - 6 - 8 9` | 14 | 4 | [4] | [1, 3] | [4] |
| `1 - 3 4 - 6 - 8 9` | 14 | 7 | [3, 4] | [1, 6] | [1, 6] |
| `1 - 3 4 - 6 - 8 9` | 14 | 8 | [8] | [1, 3, 4] | [8] |
| `1 - 3 4 - 6 - 8 9` | 14 | 9 | [9] | [3, 6] | [9] |
| `1 - 3 4 - 6 - 8 9` | 14 | 10 | [1, 9] | [1, 3, 6] | [1, 9] |
| `1 - 3 4 - 6 - 8 9` | 14 | 11 | [3, 8] | [1, 4, 6] | [3, 8] |
| `1 - 3 4 - 6 - 8 9` | 14 | 12 | [3, 9] | [1, 3, 8] | [3, 9] |
| `1 2 - - 5 6 - 8 9` | 14 | 6 | [6] | [1, 5] | [6] |
| `1 2 - - 5 6 - 8 9` | 14 | 8 | [8] | [1, 2, 5] | [8] |
| `1 2 - - 5 6 - 8 9` | 14 | 9 | [9] | [1, 2, 6] | [9] |
| `1 2 - - 5 6 - 8 9` | 14 | 11 | [2, 9] | [1, 2, 8] | [2, 9] |
| `1 2 - - 5 6 - 8 9` | 14 | 12 | [1, 5, 6] | [1, 2, 9] | [1, 2, 9] |
| `1 2 - 4 - - 7 8 9` | 14 | 7 | [7] | [1, 2, 4] | [7] |
| `1 2 - 4 - - 7 8 9` | 14 | 8 | [8] | [1, 7] | [8] |
| `1 2 - 4 - - 7 8 9` | 14 | 9 | [9] | [2, 7] | [9] |
| `1 2 - 4 - - 7 8 9` | 14 | 10 | [1, 9] | [1, 2, 7] | [1, 9] |
| `1 2 - 4 - - 7 8 9` | 14 | 11 | [2, 9] | [1, 2, 8] | [2, 9] |
| `1 2 - 4 - - 7 8 9` | 14 | 12 | [4, 8] | [1, 2, 9] | [4, 8] |
| `1 2 3 4 - 6 7 8 -` | 14 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 4 - 6 7 8 -` | 14 | 4 | [4] | [1, 3] | [4] |
| `1 2 3 4 - 6 7 8 -` | 14 | 5 | [1, 4] | [2, 3] | [1, 4] |
| `1 2 3 4 - 6 7 8 -` | 14 | 6 | [6] | [1, 2, 3] | [6] |
| `1 2 3 4 - 6 7 8 -` | 14 | 7 | [7] | [1, 2, 4] | [7] |
| `1 2 3 4 - 6 7 8 -` | 14 | 8 | [8] | [1, 3, 4] | [8] |
| `1 2 3 4 - 6 7 8 -` | 14 | 9 | [1, 8] | [2, 3, 4] | [1, 8] |
| `1 2 3 4 - 6 7 8 -` | 14 | 10 | [2, 8] | [1, 2, 3, 4] | [2, 8] |
| `1 2 3 4 - 6 7 8 -` | 14 | 11 | [3, 8] | [1, 3, 7] | [3, 8] |
| `1 2 3 4 - 6 7 8 -` | 14 | 12 | [4, 8] | [1, 2, 3, 6] | [4, 8] |
| `1 2 3 4 5 - 7 - 9` | 14 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 4 5 - 7 - 9` | 14 | 4 | [4] | [1, 3] | [4] |
| `1 2 3 4 5 - 7 - 9` | 14 | 5 | [5] | [1, 4] | [5] |
| `1 2 3 4 5 - 7 - 9` | 14 | 6 | [2, 4] | [1, 2, 3] | [1, 5] |
| `1 2 3 4 5 - 7 - 9` | 14 | 7 | [7] | [1, 2, 4] | [7] |
| `1 2 3 4 5 - 7 - 9` | 14 | 8 | [3, 5] | [1, 3, 4] | [1, 7] |
| `1 2 3 4 5 - 7 - 9` | 14 | 9 | [9] | [1, 3, 5] | [9] |
| `1 2 3 4 5 - 7 - 9` | 14 | 10 | [1, 9] | [1, 2, 3, 4] | [1, 9] |
| `1 2 3 4 5 - 7 - 9` | 14 | 11 | [2, 9] | [1, 2, 3, 5] | [2, 9] |
| `1 2 3 4 5 - 7 - 9` | 14 | 12 | [3, 9] | [2, 3, 7] | [3, 9] |
| `- - - 4 5 6 7 8 -` | 15 | 11 | [5, 6] | [4, 7] | [4, 7] |
| `- - - 4 5 6 7 8 -` | 15 | 12 | [4, 8] | [5, 7] | [4, 8] |
| `- - 3 - 5 6 7 - 9` | 15 | 9 | [9] | [3, 6] | [9] |
| `- - 3 - 5 6 7 - 9` | 15 | 12 | [3, 9] | [5, 7] | [3, 9] |
| `- - 3 4 - 6 - 8 9` | 15 | 9 | [9] | [3, 6] | [9] |
| `- - 3 4 - 6 - 8 9` | 15 | 12 | [3, 9] | [4, 8] | [3, 9] |
| `- 2 - - 5 6 - 8 9` | 15 | 8 | [8] | [2, 6] | [8] |
| `- 2 - - 5 6 - 8 9` | 15 | 11 | [5, 6] | [2, 9] | [2, 9] |
| `- 2 - 4 - - 7 8 9` | 15 | 9 | [9] | [2, 7] | [9] |
| `- 2 - 4 - - 7 8 9` | 15 | 11 | [4, 7] | [2, 9] | [2, 9] |
| `- 2 3 4 - 6 7 8 -` | 15 | 6 | [6] | [2, 4] | [6] |
| `- 2 3 4 - 6 7 8 -` | 15 | 7 | [7] | [3, 4] | [7] |
| `- 2 3 4 - 6 7 8 -` | 15 | 8 | [8] | [2, 6] | [8] |
| `- 2 3 4 - 6 7 8 -` | 15 | 9 | [2, 7] | [2, 3, 4] | [2, 7] |
| `- 2 3 4 - 6 7 8 -` | 15 | 10 | [4, 6] | [3, 7] | [2, 8] |
| `- 2 3 4 - 6 7 8 -` | 15 | 11 | [3, 8] | [2, 3, 6] | [3, 8] |
| `- 2 3 4 - 6 7 8 -` | 15 | 12 | [4, 8] | [2, 3, 7] | [4, 8] |
| `- 2 3 4 5 - 7 - 9` | 15 | 5 | [5] | [2, 3] | [5] |
| `- 2 3 4 5 - 7 - 9` | 15 | 7 | [7] | [3, 4] | [7] |
| `- 2 3 4 5 - 7 - 9` | 15 | 9 | [9] | [2, 3, 4] | [9] |
| `- 2 3 4 5 - 7 - 9` | 15 | 10 | [3, 7] | [2, 3, 5] | [3, 7] |
| `- 2 3 4 5 - 7 - 9` | 15 | 11 | [2, 9] | [2, 4, 5] | [2, 9] |
| `- 2 3 4 5 - 7 - 9` | 15 | 12 | [3, 9] | [2, 3, 7] | [3, 9] |
| `1 - - - 5 - 7 8 9` | 15 | 8 | [8] | [1, 7] | [8] |
| `1 - - - 5 - 7 8 9` | 15 | 9 | [9] | [1, 8] | [9] |
| `1 - 3 - 5 6 7 8 -` | 15 | 6 | [6] | [1, 5] | [6] |
| `1 - 3 - 5 6 7 8 -` | 15 | 7 | [7] | [1, 6] | [7] |
| `1 - 3 - 5 6 7 8 -` | 15 | 8 | [8] | [1, 7] | [8] |
| `1 - 3 - 5 6 7 8 -` | 15 | 9 | [1, 8] | [1, 3, 5] | [1, 8] |
| `1 - 3 - 5 6 7 8 -` | 15 | 10 | [3, 7] | [1, 3, 6] | [3, 7] |
| `1 - 3 - 5 6 7 8 -` | 15 | 11 | [5, 6] | [1, 3, 7] | [3, 8] |
| `1 - 3 - 5 6 7 8 -` | 15 | 12 | [5, 7] | [1, 3, 8] | [5, 7] |
| `1 - 3 4 - 6 7 - 9` | 15 | 4 | [4] | [1, 3] | [4] |
| `1 - 3 4 - 6 7 - 9` | 15 | 7 | [7] | [1, 6] | [7] |
| `1 - 3 4 - 6 7 - 9` | 15 | 8 | [1, 7] | [1, 3, 4] | [1, 7] |
| `1 - 3 4 - 6 7 - 9` | 15 | 9 | [9] | [3, 6] | [9] |
| `1 - 3 4 - 6 7 - 9` | 15 | 10 | [1, 9] | [1, 3, 6] | [1, 9] |
| `1 - 3 4 - 6 7 - 9` | 15 | 11 | [4, 7] | [1, 3, 7] | [4, 7] |
| `1 - 3 4 - 6 7 - 9` | 15 | 12 | [3, 9] | [1, 4, 7] | [3, 9] |
| `1 - 3 4 5 - - 8 9` | 15 | 4 | [4] | [1, 3] | [4] |
| `1 - 3 4 5 - - 8 9` | 15 | 5 | [5] | [1, 4] | [5] |
| `1 - 3 4 5 - - 8 9` | 15 | 8 | [8] | [1, 3, 4] | [8] |
| `1 - 3 4 5 - - 8 9` | 15 | 9 | [9] | [1, 3, 5] | [9] |
| `1 - 3 4 5 - - 8 9` | 15 | 10 | [1, 9] | [1, 4, 5] | [1, 9] |
| `1 - 3 4 5 - - 8 9` | 15 | 12 | [3, 9] | [1, 3, 8] | [3, 9] |
| `1 2 - - 5 6 7 - 9` | 15 | 6 | [6] | [1, 5] | [6] |
| `1 2 - - 5 6 7 - 9` | 15 | 7 | [7] | [1, 6] | [7] |
| `1 2 - - 5 6 7 - 9` | 15 | 8 | [2, 6] | [1, 2, 5] | [1, 7] |
| `1 2 - - 5 6 7 - 9` | 15 | 9 | [9] | [1, 2, 6] | [9] |
| `1 2 - - 5 6 7 - 9` | 15 | 10 | [1, 9] | [1, 2, 7] | [1, 9] |
| `1 2 - - 5 6 7 - 9` | 15 | 12 | [5, 7] | [1, 5, 6] | [5, 7] |
| `1 2 - 4 - 6 - 8 9` | 15 | 6 | [6] | [2, 4] | [6] |
| `1 2 - 4 - 6 - 8 9` | 15 | 7 | [1, 6] | [1, 2, 4] | [1, 6] |
| `1 2 - 4 - 6 - 8 9` | 15 | 8 | [8] | [2, 6] | [8] |
| `1 2 - 4 - 6 - 8 9` | 15 | 9 | [9] | [1, 2, 6] | [9] |
| `1 2 - 4 - 6 - 8 9` | 15 | 10 | [2, 8] | [1, 9] | [1, 9] |
| `1 2 - 4 - 6 - 8 9` | 15 | 11 | [2, 9] | [1, 2, 8] | [2, 9] |
| `1 2 - 4 - 6 - 8 9` | 15 | 12 | [4, 8] | [2, 4, 6] | [4, 8] |
| `1 2 3 - - - 7 8 9` | 15 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 - - - 7 8 9` | 15 | 8 | [8] | [1, 7] | [8] |
| `1 2 3 - - - 7 8 9` | 15 | 9 | [9] | [2, 7] | [9] |
| `1 2 3 - - - 7 8 9` | 15 | 10 | [1, 9] | [1, 2, 7] | [1, 9] |
| `1 2 3 - - - 7 8 9` | 15 | 11 | [2, 9] | [1, 2, 8] | [2, 9] |
| `1 2 3 - - - 7 8 9` | 15 | 12 | [3, 9] | [1, 3, 8] | [3, 9] |
| `1 2 3 4 5 - 7 8 -` | 15 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 4 5 - 7 8 -` | 15 | 4 | [4] | [1, 3] | [4] |
| `1 2 3 4 5 - 7 8 -` | 15 | 5 | [5] | [1, 4] | [5] |
| `1 2 3 4 5 - 7 8 -` | 15 | 6 | [1, 5] | [1, 2, 3] | [1, 5] |
| `1 2 3 4 5 - 7 8 -` | 15 | 7 | [7] | [1, 2, 4] | [7] |
| `1 2 3 4 5 - 7 8 -` | 15 | 8 | [8] | [1, 2, 5] | [8] |
| `1 2 3 4 5 - 7 8 -` | 15 | 9 | [1, 8] | [2, 3, 4] | [1, 8] |
| `1 2 3 4 5 - 7 8 -` | 15 | 10 | [2, 8] | [1, 2, 3, 4] | [2, 8] |
| `1 2 3 4 5 - 7 8 -` | 15 | 11 | [3, 8] | [1, 2, 3, 5] | [3, 8] |
| `1 2 3 4 5 - 7 8 -` | 15 | 12 | [4, 8] | [1, 2, 4, 5] | [4, 8] |
| `1 2 3 4 5 6 - - 9` | 15 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 4 5 6 - - 9` | 15 | 4 | [4] | [1, 3] | [4] |
| `1 2 3 4 5 6 - - 9` | 15 | 5 | [5] | [2, 3] | [5] |
| `1 2 3 4 5 6 - - 9` | 15 | 6 | [6] | [1, 2, 3] | [6] |
| `1 2 3 4 5 6 - - 9` | 15 | 7 | [3, 4] | [1, 2, 4] | [1, 6] |
| `1 2 3 4 5 6 - - 9` | 15 | 8 | [3, 5] | [1, 2, 5] | [2, 6] |
| `1 2 3 4 5 6 - - 9` | 15 | 9 | [9] | [1, 2, 6] | [9] |
| `1 2 3 4 5 6 - - 9` | 15 | 10 | [1, 9] | [1, 2, 3, 4] | [1, 9] |
| `1 2 3 4 5 6 - - 9` | 15 | 11 | [2, 9] | [1, 2, 3, 5] | [2, 9] |
| `1 2 3 4 5 6 - - 9` | 15 | 12 | [3, 9] | [1, 2, 3, 6] | [3, 9] |
| `- - 3 - 5 6 7 8 -` | 16 | 8 | [8] | [3, 5] | [8] |
| `- - 3 - 5 6 7 8 -` | 16 | 11 | [5, 6] | [3, 8] | [3, 8] |
| `- - 3 4 - 6 7 - 9` | 16 | 7 | [7] | [3, 4] | [7] |
| `- - 3 4 - 6 7 - 9` | 16 | 9 | [9] | [3, 6] | [9] |
| `- - 3 4 - 6 7 - 9` | 16 | 10 | [4, 6] | [3, 7] | [3, 7] |
| `- - 3 4 5 - - 8 9` | 16 | 8 | [8] | [3, 5] | [8] |
| `- - 3 4 5 - - 8 9` | 16 | 9 | [9] | [4, 5] | [9] |
| `- - 3 4 5 - - 8 9` | 16 | 12 | [3, 9] | [3, 4, 5] | [3, 9] |
| `- 2 - - 5 6 7 - 9` | 16 | 7 | [7] | [2, 5] | [7] |
| `- 2 - - 5 6 7 - 9` | 16 | 9 | [9] | [2, 7] | [9] |
| `- 2 - - 5 6 7 - 9` | 16 | 11 | [2, 9] | [5, 6] | [2, 9] |
| `- 2 - 4 - 6 - 8 9` | 16 | 6 | [6] | [2, 4] | [6] |
| `- 2 - 4 - 6 - 8 9` | 16 | 8 | [8] | [2, 6] | [8] |
| `- 2 - 4 - 6 - 8 9` | 16 | 10 | [4, 6] | [2, 8] | [2, 8] |
| `- 2 - 4 - 6 - 8 9` | 16 | 12 | [4, 8] | [2, 4, 6] | [4, 8] |
| `- 2 3 - - - 7 8 9` | 16 | 9 | [9] | [2, 7] | [9] |
| `- 2 3 - - - 7 8 9` | 16 | 10 | [3, 7] | [2, 8] | [2, 8] |
| `- 2 3 - - - 7 8 9` | 16 | 11 | [2, 9] | [3, 8] | [2, 9] |
| `- 2 3 - - - 7 8 9` | 16 | 12 | [3, 9] | [2, 3, 7] | [3, 9] |
| `- 2 3 4 5 - 7 8 -` | 16 | 5 | [5] | [2, 3] | [5] |
| `- 2 3 4 5 - 7 8 -` | 16 | 7 | [7] | [2, 5] | [7] |
| `- 2 3 4 5 - 7 8 -` | 16 | 8 | [8] | [3, 5] | [8] |
| `- 2 3 4 5 - 7 8 -` | 16 | 9 | [4, 5] | [2, 3, 4] | [2, 7] |
| `- 2 3 4 5 - 7 8 -` | 16 | 10 | [2, 8] | [2, 3, 5] | [2, 8] |
| `- 2 3 4 5 - 7 8 -` | 16 | 11 | [3, 8] | [2, 4, 5] | [3, 8] |
| `- 2 3 4 5 - 7 8 -` | 16 | 12 | [5, 7] | [2, 3, 7] | [4, 8] |
| `- 2 3 4 5 6 - - 9` | 16 | 5 | [5] | [2, 3] | [5] |
| `- 2 3 4 5 6 - - 9` | 16 | 6 | [6] | [2, 4] | [6] |
| `- 2 3 4 5 6 - - 9` | 16 | 7 | [3, 4] | [2, 5] | [2, 5] |
| `- 2 3 4 5 6 - - 9` | 16 | 8 | [3, 5] | [2, 6] | [2, 6] |
| `- 2 3 4 5 6 - - 9` | 16 | 9 | [9] | [2, 3, 4] | [9] |
| `- 2 3 4 5 6 - - 9` | 16 | 10 | [4, 6] | [2, 3, 5] | [4, 6] |
| `- 2 3 4 5 6 - - 9` | 16 | 11 | [2, 9] | [2, 3, 6] | [2, 9] |
| `- 2 3 4 5 6 - - 9` | 16 | 12 | [3, 9] | [2, 4, 6] | [3, 9] |
| `1 - - - 5 6 - 8 9` | 16 | 6 | [6] | [1, 5] | [6] |
| `1 - - - 5 6 - 8 9` | 16 | 9 | [9] | [1, 8] | [9] |
| `1 - - 4 - - 7 8 9` | 16 | 8 | [8] | [1, 7] | [8] |
| `1 - - 4 - - 7 8 9` | 16 | 9 | [9] | [1, 8] | [9] |
| `1 - - 4 - - 7 8 9` | 16 | 12 | [4, 8] | [1, 4, 7] | [4, 8] |
| `1 - 3 4 - 6 7 8 -` | 16 | 4 | [4] | [1, 3] | [4] |
| `1 - 3 4 - 6 7 8 -` | 16 | 7 | [7] | [3, 4] | [7] |
| `1 - 3 4 - 6 7 8 -` | 16 | 8 | [8] | [1, 3, 4] | [8] |
| `1 - 3 4 - 6 7 8 -` | 16 | 9 | [1, 8] | [3, 6] | [1, 8] |
| `1 - 3 4 - 6 7 8 -` | 16 | 10 | [3, 7] | [1, 3, 6] | [3, 7] |
| `1 - 3 4 - 6 7 8 -` | 16 | 11 | [4, 7] | [1, 3, 7] | [3, 8] |
| `1 - 3 4 - 6 7 8 -` | 16 | 12 | [4, 8] | [1, 4, 7] | [4, 8] |
| `1 - 3 4 5 - 7 - 9` | 16 | 4 | [4] | [1, 3] | [4] |
| `1 - 3 4 5 - 7 - 9` | 16 | 5 | [5] | [1, 4] | [5] |
| `1 - 3 4 5 - 7 - 9` | 16 | 8 | [3, 5] | [1, 3, 4] | [1, 7] |
| `1 - 3 4 5 - 7 - 9` | 16 | 9 | [9] | [1, 3, 5] | [9] |
| `1 - 3 4 5 - 7 - 9` | 16 | 10 | [1, 9] | [3, 7] | [1, 9] |
| `1 - 3 4 5 - 7 - 9` | 16 | 11 | [4, 7] | [1, 3, 7] | [4, 7] |
| `1 - 3 4 5 - 7 - 9` | 16 | 12 | [3, 9] | [1, 4, 7] | [3, 9] |
| `1 2 - - 5 6 7 8 -` | 16 | 6 | [6] | [1, 5] | [6] |
| `1 2 - - 5 6 7 8 -` | 16 | 7 | [7] | [2, 5] | [7] |
| `1 2 - - 5 6 7 8 -` | 16 | 8 | [8] | [1, 2, 5] | [8] |
| `1 2 - - 5 6 7 8 -` | 16 | 9 | [1, 8] | [1, 2, 6] | [1, 8] |
| `1 2 - - 5 6 7 8 -` | 16 | 10 | [2, 8] | [1, 2, 7] | [2, 8] |
| `1 2 - - 5 6 7 8 -` | 16 | 11 | [5, 6] | [1, 2, 8] | [5, 6] |
| `1 2 - - 5 6 7 8 -` | 16 | 12 | [5, 7] | [1, 5, 6] | [5, 7] |
| `1 2 - 4 - 6 7 - 9` | 16 | 6 | [6] | [2, 4] | [6] |
| `1 2 - 4 - 6 7 - 9` | 16 | 7 | [7] | [1, 2, 4] | [7] |
| `1 2 - 4 - 6 7 - 9` | 16 | 8 | [2, 6] | [1, 7] | [1, 7] |
| `1 2 - 4 - 6 7 - 9` | 16 | 9 | [9] | [1, 2, 6] | [9] |
| `1 2 - 4 - 6 7 - 9` | 16 | 10 | [1, 9] | [1, 2, 7] | [1, 9] |
| `1 2 - 4 - 6 7 - 9` | 16 | 11 | [2, 9] | [1, 4, 6] | [2, 9] |
| `1 2 - 4 - 6 7 - 9` | 16 | 12 | [2, 4, 6] | [1, 4, 7] | [1, 2, 9] |
| `1 2 - 4 5 - - 8 9` | 16 | 5 | [5] | [1, 4] | [5] |
| `1 2 - 4 5 - - 8 9` | 16 | 6 | [1, 5] | [2, 4] | [1, 5] |
| `1 2 - 4 5 - - 8 9` | 16 | 7 | [2, 5] | [1, 2, 4] | [2, 5] |
| `1 2 - 4 5 - - 8 9` | 16 | 8 | [8] | [1, 2, 5] | [8] |
| `1 2 - 4 5 - - 8 9` | 16 | 9 | [9] | [1, 8] | [9] |
| `1 2 - 4 5 - - 8 9` | 16 | 10 | [1, 9] | [2, 8] | [1, 9] |
| `1 2 - 4 5 - - 8 9` | 16 | 11 | [2, 9] | [1, 2, 8] | [2, 9] |
| `1 2 - 4 5 - - 8 9` | 16 | 12 | [4, 8] | [1, 2, 4, 5] | [4, 8] |
| `1 2 3 - - 6 - 8 9` | 16 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 - - 6 - 8 9` | 16 | 6 | [6] | [1, 2, 3] | [6] |
| `1 2 3 - - 6 - 8 9` | 16 | 8 | [8] | [2, 6] | [8] |
| `1 2 3 - - 6 - 8 9` | 16 | 9 | [9] | [1, 2, 6] | [9] |
| `1 2 3 - - 6 - 8 9` | 16 | 10 | [1, 9] | [1, 3, 6] | [1, 9] |
| `1 2 3 - - 6 - 8 9` | 16 | 11 | [2, 9] | [1, 2, 8] | [2, 9] |
| `1 2 3 - - 6 - 8 9` | 16 | 12 | [3, 9] | [1, 2, 3, 6] | [3, 9] |
| `1 2 3 4 5 6 - 8 -` | 16 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 4 5 6 - 8 -` | 16 | 4 | [4] | [1, 3] | [4] |
| `1 2 3 4 5 6 - 8 -` | 16 | 5 | [5] | [1, 4] | [5] |
| `1 2 3 4 5 6 - 8 -` | 16 | 6 | [6] | [1, 2, 3] | [6] |
| `1 2 3 4 5 6 - 8 -` | 16 | 7 | [2, 5] | [1, 2, 4] | [1, 6] |
| `1 2 3 4 5 6 - 8 -` | 16 | 8 | [8] | [1, 3, 4] | [8] |
| `1 2 3 4 5 6 - 8 -` | 16 | 9 | [1, 8] | [1, 3, 5] | [1, 8] |
| `1 2 3 4 5 6 - 8 -` | 16 | 10 | [2, 8] | [1, 2, 3, 4] | [2, 8] |
| `1 2 3 4 5 6 - 8 -` | 16 | 11 | [3, 8] | [1, 2, 3, 5] | [3, 8] |
| `1 2 3 4 5 6 - 8 -` | 16 | 12 | [4, 8] | [1, 2, 3, 6] | [4, 8] |
| `- - 3 4 - 6 7 8 -` | 17 | 7 | [7] | [3, 4] | [7] |
| `- - 3 4 - 6 7 8 -` | 17 | 10 | [4, 6] | [3, 7] | [3, 7] |
| `- - 3 4 - 6 7 8 -` | 17 | 11 | [3, 8] | [4, 7] | [3, 8] |
| `- - 3 4 5 - 7 - 9` | 17 | 7 | [7] | [3, 4] | [7] |
| `- - 3 4 5 - 7 - 9` | 17 | 9 | [9] | [4, 5] | [9] |
| `- - 3 4 5 - 7 - 9` | 17 | 12 | [3, 9] | [3, 4, 5] | [3, 9] |
| `- 2 - - 5 6 7 8 -` | 17 | 7 | [7] | [2, 5] | [7] |
| `- 2 - - 5 6 7 8 -` | 17 | 8 | [8] | [2, 6] | [8] |
| `- 2 - 4 - 6 7 - 9` | 17 | 6 | [6] | [2, 4] | [6] |
| `- 2 - 4 - 6 7 - 9` | 17 | 9 | [9] | [2, 7] | [9] |
| `- 2 - 4 - 6 7 - 9` | 17 | 11 | [2, 9] | [4, 7] | [2, 9] |
| `- 2 - 4 5 - - 8 9` | 17 | 9 | [9] | [4, 5] | [9] |
| `- 2 - 4 5 - - 8 9` | 17 | 11 | [2, 9] | [2, 4, 5] | [2, 9] |
| `- 2 3 - - 6 - 8 9` | 17 | 8 | [8] | [2, 6] | [8] |
| `- 2 3 - - 6 - 8 9` | 17 | 9 | [9] | [3, 6] | [9] |
| `- 2 3 - - 6 - 8 9` | 17 | 11 | [2, 9] | [2, 3, 6] | [2, 9] |
| `- 2 3 4 5 6 - 8 -` | 17 | 5 | [5] | [2, 3] | [5] |
| `- 2 3 4 5 6 - 8 -` | 17 | 6 | [6] | [2, 4] | [6] |
| `- 2 3 4 5 6 - 8 -` | 17 | 7 | [2, 5] | [3, 4] | [2, 5] |
| `- 2 3 4 5 6 - 8 -` | 17 | 8 | [8] | [3, 5] | [8] |
| `- 2 3 4 5 6 - 8 -` | 17 | 9 | [3, 6] | [2, 3, 4] | [3, 6] |
| `- 2 3 4 5 6 - 8 -` | 17 | 10 | [2, 8] | [2, 3, 5] | [2, 8] |
| `- 2 3 4 5 6 - 8 -` | 17 | 11 | [3, 8] | [2, 3, 6] | [3, 8] |
| `- 2 3 4 5 6 - 8 -` | 17 | 12 | [4, 8] | [2, 4, 6] | [4, 8] |
| `1 - - - 5 6 7 - 9` | 17 | 6 | [6] | [1, 5] | [6] |
| `1 - - - 5 6 7 - 9` | 17 | 7 | [7] | [1, 6] | [7] |
| `1 - - - 5 6 7 - 9` | 17 | 12 | [5, 7] | [1, 5, 6] | [5, 7] |
| `1 - - 4 - 6 - 8 9` | 17 | 9 | [9] | [1, 8] | [9] |
| `1 - - 4 - 6 - 8 9` | 17 | 10 | [1, 9] | [4, 6] | [1, 9] |
| `1 - 3 - - - 7 8 9` | 17 | 8 | [8] | [1, 7] | [8] |
| `1 - 3 - - - 7 8 9` | 17 | 9 | [9] | [1, 8] | [9] |
| `1 - 3 - - - 7 8 9` | 17 | 10 | [1, 9] | [3, 7] | [1, 9] |
| `1 - 3 - - - 7 8 9` | 17 | 11 | [3, 8] | [1, 3, 7] | [3, 8] |
| `1 - 3 - - - 7 8 9` | 17 | 12 | [3, 9] | [1, 3, 8] | [3, 9] |
| `1 - 3 4 5 - 7 8 -` | 17 | 4 | [4] | [1, 3] | [4] |
| `1 - 3 4 5 - 7 8 -` | 17 | 5 | [5] | [1, 4] | [5] |
| `1 - 3 4 5 - 7 8 -` | 17 | 7 | [7] | [3, 4] | [7] |
| `1 - 3 4 5 - 7 8 -` | 17 | 8 | [8] | [1, 3, 4] | [8] |
| `1 - 3 4 5 - 7 8 -` | 17 | 9 | [1, 8] | [1, 3, 5] | [1, 8] |
| `1 - 3 4 5 - 7 8 -` | 17 | 10 | [3, 7] | [1, 4, 5] | [3, 7] |
| `1 - 3 4 5 - 7 8 -` | 17 | 11 | [3, 8] | [1, 3, 7] | [3, 8] |
| `1 - 3 4 5 - 7 8 -` | 17 | 12 | [4, 8] | [1, 4, 7] | [4, 8] |
| `1 - 3 4 5 6 - - 9` | 17 | 4 | [4] | [1, 3] | [4] |
| `1 - 3 4 5 6 - - 9` | 17 | 5 | [5] | [1, 4] | [5] |
| `1 - 3 4 5 6 - - 9` | 17 | 6 | [6] | [1, 5] | [6] |
| `1 - 3 4 5 6 - - 9` | 17 | 7 | [3, 4] | [1, 6] | [1, 6] |
| `1 - 3 4 5 6 - - 9` | 17 | 8 | [3, 5] | [1, 3, 4] | [3, 5] |
| `1 - 3 4 5 6 - - 9` | 17 | 9 | [9] | [1, 3, 5] | [9] |
| `1 - 3 4 5 6 - - 9` | 17 | 10 | [1, 9] | [1, 3, 6] | [1, 9] |
| `1 - 3 4 5 6 - - 9` | 17 | 11 | [5, 6] | [1, 4, 6] | [5, 6] |
| `1 - 3 4 5 6 - - 9` | 17 | 12 | [3, 9] | [1, 5, 6] | [3, 9] |
| `1 2 - 4 - 6 7 8 -` | 17 | 6 | [6] | [2, 4] | [6] |
| `1 2 - 4 - 6 7 8 -` | 17 | 7 | [7] | [1, 2, 4] | [7] |
| `1 2 - 4 - 6 7 8 -` | 17 | 8 | [8] | [1, 7] | [8] |
| `1 2 - 4 - 6 7 8 -` | 17 | 9 | [1, 8] | [1, 2, 6] | [1, 8] |
| `1 2 - 4 - 6 7 8 -` | 17 | 10 | [2, 8] | [1, 2, 7] | [2, 8] |
| `1 2 - 4 - 6 7 8 -` | 17 | 11 | [4, 7] | [1, 2, 8] | [4, 7] |
| `1 2 - 4 - 6 7 8 -` | 17 | 12 | [4, 8] | [1, 4, 7] | [4, 8] |
| `1 2 - 4 5 - 7 - 9` | 17 | 5 | [5] | [1, 4] | [5] |
| `1 2 - 4 5 - 7 - 9` | 17 | 6 | [2, 4] | [1, 5] | [1, 5] |
| `1 2 - 4 5 - 7 - 9` | 17 | 7 | [7] | [1, 2, 4] | [7] |
| `1 2 - 4 5 - 7 - 9` | 17 | 8 | [1, 7] | [1, 2, 5] | [1, 7] |
| `1 2 - 4 5 - 7 - 9` | 17 | 9 | [9] | [2, 7] | [9] |
| `1 2 - 4 5 - 7 - 9` | 17 | 10 | [1, 9] | [1, 2, 7] | [1, 9] |
| `1 2 - 4 5 - 7 - 9` | 17 | 11 | [2, 9] | [2, 4, 5] | [2, 9] |
| `1 2 - 4 5 - 7 - 9` | 17 | 12 | [5, 7] | [1, 2, 4, 5] | [5, 7] |
| `1 2 3 - - 6 7 - 9` | 17 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 - - 6 7 - 9` | 17 | 6 | [6] | [1, 2, 3] | [6] |
| `1 2 3 - - 6 7 - 9` | 17 | 7 | [7] | [1, 6] | [7] |
| `1 2 3 - - 6 7 - 9` | 17 | 8 | [2, 6] | [1, 7] | [1, 7] |
| `1 2 3 - - 6 7 - 9` | 17 | 9 | [9] | [1, 2, 6] | [9] |
| `1 2 3 - - 6 7 - 9` | 17 | 10 | [1, 9] | [1, 2, 7] | [1, 9] |
| `1 2 3 - - 6 7 - 9` | 17 | 11 | [2, 9] | [1, 3, 7] | [2, 9] |
| `1 2 3 - - 6 7 - 9` | 17 | 12 | [3, 9] | [1, 2, 3, 6] | [3, 9] |
| `1 2 3 - 5 - - 8 9` | 17 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 - 5 - - 8 9` | 17 | 5 | [5] | [2, 3] | [5] |
| `1 2 3 - 5 - - 8 9` | 17 | 6 | [1, 5] | [1, 2, 3] | [1, 5] |
| `1 2 3 - 5 - - 8 9` | 17 | 8 | [8] | [1, 2, 5] | [8] |
| `1 2 3 - 5 - - 8 9` | 17 | 9 | [9] | [1, 3, 5] | [9] |
| `1 2 3 - 5 - - 8 9` | 17 | 10 | [1, 9] | [2, 3, 5] | [1, 9] |
| `1 2 3 - 5 - - 8 9` | 17 | 11 | [3, 8] | [1, 2, 3, 5] | [2, 9] |
| `1 2 3 - 5 - - 8 9` | 17 | 12 | [3, 9] | [1, 2, 9] | [3, 9] |
| `1 2 3 4 5 6 7 - -` | 17 | 4 | [4] | [1, 3] | [4] |
| `1 2 3 4 5 6 7 - -` | 17 | 5 | [5] | [2, 3] | [5] |
| `1 2 3 4 5 6 7 - -` | 17 | 6 | [6] | [1, 2, 3] | [6] |
| `1 2 3 4 5 6 7 - -` | 17 | 7 | [7] | [1, 2, 4] | [7] |
| `1 2 3 4 5 6 7 - -` | 17 | 8 | [1, 7] | [1, 2, 5] | [1, 7] |
| `1 2 3 4 5 6 7 - -` | 17 | 9 | [2, 7] | [2, 3, 4] | [2, 7] |
| `1 2 3 4 5 6 7 - -` | 17 | 10 | [3, 7] | [1, 2, 3, 4] | [3, 7] |
| `1 2 3 4 5 6 7 - -` | 17 | 11 | [4, 7] | [1, 2, 3, 5] | [4, 7] |
| `1 2 3 4 5 6 7 - -` | 17 | 12 | [5, 7] | [1, 2, 3, 6] | [5, 7] |
| `- - 3 4 5 - 7 8 -` | 18 | 7 | [7] | [3, 4] | [7] |
| `- - 3 4 5 - 7 8 -` | 18 | 8 | [8] | [3, 5] | [8] |
| `- - 3 4 5 - 7 8 -` | 18 | 11 | [3, 8] | [4, 7] | [3, 8] |
| `- - 3 4 5 - 7 8 -` | 18 | 12 | [4, 8] | [3, 4, 5] | [4, 8] |
| `- - 3 4 5 6 - - 9` | 18 | 9 | [9] | [3, 6] | [9] |
| `- - 3 4 5 6 - - 9` | 18 | 12 | [3, 9] | [3, 4, 5] | [3, 9] |
| `- 2 - 4 - 6 7 8 -` | 18 | 6 | [6] | [2, 4] | [6] |
| `- 2 - 4 - 6 7 8 -` | 18 | 8 | [8] | [2, 6] | [8] |
| `- 2 - 4 - 6 7 8 -` | 18 | 10 | [4, 6] | [2, 8] | [2, 8] |
| `- 2 - 4 - 6 7 8 -` | 18 | 12 | [4, 8] | [2, 4, 6] | [4, 8] |
| `- 2 - 4 5 - 7 - 9` | 18 | 7 | [7] | [2, 5] | [7] |
| `- 2 - 4 5 - 7 - 9` | 18 | 9 | [9] | [2, 7] | [9] |
| `- 2 - 4 5 - 7 - 9` | 18 | 11 | [2, 9] | [2, 4, 5] | [2, 9] |
| `- 2 3 - - 6 7 - 9` | 18 | 9 | [9] | [2, 7] | [9] |
| `- 2 3 - - 6 7 - 9` | 18 | 11 | [2, 9] | [2, 3, 6] | [2, 9] |
| `- 2 3 - - 6 7 - 9` | 18 | 12 | [3, 9] | [2, 3, 7] | [3, 9] |
| `- 2 3 - 5 - - 8 9` | 18 | 5 | [5] | [2, 3] | [5] |
| `- 2 3 - 5 - - 8 9` | 18 | 8 | [8] | [3, 5] | [8] |
| `- 2 3 - 5 - - 8 9` | 18 | 10 | [2, 8] | [2, 3, 5] | [2, 8] |
| `- 2 3 - 5 - - 8 9` | 18 | 11 | [3, 8] | [2, 9] | [2, 9] |
| `- 2 3 4 5 6 7 - -` | 18 | 5 | [5] | [2, 3] | [5] |
| `- 2 3 4 5 6 7 - -` | 18 | 6 | [6] | [2, 4] | [6] |
| `- 2 3 4 5 6 7 - -` | 18 | 7 | [7] | [2, 5] | [7] |
| `- 2 3 4 5 6 7 - -` | 18 | 9 | [2, 7] | [2, 3, 4] | [2, 7] |
| `- 2 3 4 5 6 7 - -` | 18 | 10 | [3, 7] | [2, 3, 5] | [3, 7] |
| `- 2 3 4 5 6 7 - -` | 18 | 11 | [4, 7] | [2, 3, 6] | [4, 7] |
| `- 2 3 4 5 6 7 - -` | 18 | 12 | [5, 7] | [2, 3, 7] | [5, 7] |
| `1 - - - 5 6 7 8 -` | 18 | 6 | [6] | [1, 5] | [6] |
| `1 - - - 5 6 7 8 -` | 18 | 7 | [7] | [1, 6] | [7] |
| `1 - - - 5 6 7 8 -` | 18 | 8 | [8] | [1, 7] | [8] |
| `1 - - - 5 6 7 8 -` | 18 | 12 | [5, 7] | [1, 5, 6] | [5, 7] |
| `1 - - 4 - 6 7 - 9` | 18 | 7 | [7] | [1, 6] | [7] |
| `1 - - 4 - 6 7 - 9` | 18 | 10 | [4, 6] | [1, 9] | [1, 9] |
| `1 - - 4 - 6 7 - 9` | 18 | 11 | [4, 7] | [1, 4, 6] | [4, 7] |
| `1 - - 4 5 - - 8 9` | 18 | 5 | [5] | [1, 4] | [5] |
| `1 - - 4 5 - - 8 9` | 18 | 9 | [9] | [1, 8] | [9] |
| `1 - - 4 5 - - 8 9` | 18 | 10 | [1, 9] | [1, 4, 5] | [1, 9] |
| `1 - 3 - - 6 - 8 9` | 18 | 9 | [9] | [1, 8] | [9] |
| `1 - 3 - - 6 - 8 9` | 18 | 10 | [1, 9] | [1, 3, 6] | [1, 9] |
| `1 - 3 - - 6 - 8 9` | 18 | 12 | [3, 9] | [1, 3, 8] | [3, 9] |
| `1 - 3 4 5 6 - 8 -` | 18 | 4 | [4] | [1, 3] | [4] |
| `1 - 3 4 5 6 - 8 -` | 18 | 5 | [5] | [1, 4] | [5] |
| `1 - 3 4 5 6 - 8 -` | 18 | 6 | [6] | [1, 5] | [6] |
| `1 - 3 4 5 6 - 8 -` | 18 | 7 | [3, 4] | [1, 6] | [1, 6] |
| `1 - 3 4 5 6 - 8 -` | 18 | 8 | [8] | [1, 3, 4] | [8] |
| `1 - 3 4 5 6 - 8 -` | 18 | 9 | [1, 8] | [1, 3, 5] | [1, 8] |
| `1 - 3 4 5 6 - 8 -` | 18 | 10 | [4, 6] | [1, 3, 6] | [4, 6] |
| `1 - 3 4 5 6 - 8 -` | 18 | 11 | [3, 8] | [1, 4, 6] | [3, 8] |
| `1 - 3 4 5 6 - 8 -` | 18 | 12 | [4, 8] | [1, 5, 6] | [4, 8] |
| `1 2 - - - - 7 8 9` | 18 | 8 | [8] | [1, 7] | [8] |
| `1 2 - - - - 7 8 9` | 18 | 9 | [9] | [1, 8] | [9] |
| `1 2 - - - - 7 8 9` | 18 | 10 | [1, 9] | [1, 2, 7] | [1, 9] |
| `1 2 - - - - 7 8 9` | 18 | 11 | [2, 9] | [1, 2, 8] | [2, 9] |
| `1 2 - 4 5 - 7 8 -` | 18 | 5 | [5] | [1, 4] | [5] |
| `1 2 - 4 5 - 7 8 -` | 18 | 6 | [1, 5] | [2, 4] | [1, 5] |
| `1 2 - 4 5 - 7 8 -` | 18 | 7 | [7] | [1, 2, 4] | [7] |
| `1 2 - 4 5 - 7 8 -` | 18 | 8 | [8] | [1, 2, 5] | [8] |
| `1 2 - 4 5 - 7 8 -` | 18 | 9 | [1, 8] | [2, 7] | [1, 8] |
| `1 2 - 4 5 - 7 8 -` | 18 | 10 | [2, 8] | [1, 2, 7] | [2, 8] |
| `1 2 - 4 5 - 7 8 -` | 18 | 11 | [4, 7] | [2, 4, 5] | [4, 7] |
| `1 2 - 4 5 - 7 8 -` | 18 | 12 | [5, 7] | [1, 2, 4, 5] | [4, 8] |
| `1 2 - 4 5 6 - - 9` | 18 | 5 | [5] | [1, 4] | [5] |
| `1 2 - 4 5 6 - - 9` | 18 | 6 | [6] | [1, 5] | [6] |
| `1 2 - 4 5 6 - - 9` | 18 | 7 | [2, 5] | [1, 2, 4] | [1, 6] |
| `1 2 - 4 5 6 - - 9` | 18 | 8 | [2, 6] | [1, 2, 5] | [2, 6] |
| `1 2 - 4 5 6 - - 9` | 18 | 9 | [9] | [1, 2, 6] | [9] |
| `1 2 - 4 5 6 - - 9` | 18 | 10 | [1, 9] | [1, 4, 5] | [1, 9] |
| `1 2 - 4 5 6 - - 9` | 18 | 11 | [2, 9] | [1, 4, 6] | [2, 9] |
| `1 2 - 4 5 6 - - 9` | 18 | 12 | [1, 2, 9] | [1, 2, 4, 5] | [1, 2, 9] |
| `1 2 3 - - 6 7 8 -` | 18 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 - - 6 7 8 -` | 18 | 6 | [6] | [1, 2, 3] | [6] |
| `1 2 3 - - 6 7 8 -` | 18 | 7 | [7] | [1, 6] | [7] |
| `1 2 3 - - 6 7 8 -` | 18 | 8 | [8] | [1, 7] | [8] |
| `1 2 3 - - 6 7 8 -` | 18 | 9 | [1, 8] | [1, 2, 6] | [1, 8] |
| `1 2 3 - - 6 7 8 -` | 18 | 10 | [2, 8] | [1, 2, 7] | [2, 8] |
| `1 2 3 - - 6 7 8 -` | 18 | 11 | [3, 8] | [1, 3, 7] | [3, 8] |
| `1 2 3 - - 6 7 8 -` | 18 | 12 | [1, 3, 8] | [1, 2, 3, 6] | [1, 3, 8] |
| `1 2 3 - 5 - 7 - 9` | 18 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 - 5 - 7 - 9` | 18 | 5 | [5] | [2, 3] | [5] |
| `1 2 3 - 5 - 7 - 9` | 18 | 6 | [1, 5] | [1, 2, 3] | [1, 5] |
| `1 2 3 - 5 - 7 - 9` | 18 | 7 | [7] | [2, 5] | [7] |
| `1 2 3 - 5 - 7 - 9` | 18 | 8 | [1, 7] | [1, 2, 5] | [1, 7] |
| `1 2 3 - 5 - 7 - 9` | 18 | 9 | [9] | [1, 3, 5] | [9] |
| `1 2 3 - 5 - 7 - 9` | 18 | 10 | [3, 7] | [1, 2, 7] | [1, 9] |
| `1 2 3 - 5 - 7 - 9` | 18 | 11 | [2, 9] | [1, 2, 3, 5] | [2, 9] |
| `1 2 3 - 5 - 7 - 9` | 18 | 12 | [3, 9] | [2, 3, 7] | [3, 9] |
| `1 2 3 4 - - - 8 9` | 18 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 4 - - - 8 9` | 18 | 4 | [4] | [1, 3] | [4] |
| `1 2 3 4 - - - 8 9` | 18 | 5 | [1, 4] | [2, 3] | [1, 4] |
| `1 2 3 4 - - - 8 9` | 18 | 6 | [2, 4] | [1, 2, 3] | [2, 4] |
| `1 2 3 4 - - - 8 9` | 18 | 7 | [3, 4] | [1, 2, 4] | [3, 4] |
| `1 2 3 4 - - - 8 9` | 18 | 8 | [8] | [1, 3, 4] | [8] |
| `1 2 3 4 - - - 8 9` | 18 | 9 | [9] | [2, 3, 4] | [9] |
| `1 2 3 4 - - - 8 9` | 18 | 10 | [1, 9] | [1, 2, 3, 4] | [1, 9] |
| `1 2 3 4 - - - 8 9` | 18 | 11 | [2, 9] | [1, 2, 8] | [2, 9] |
| `1 2 3 4 - - - 8 9` | 18 | 12 | [3, 9] | [1, 3, 8] | [3, 9] |
| `- - - 4 5 - - 8 9` | 19 | 9 | [9] | [4, 5] | [9] |
| `- - 3 - - 6 - 8 9` | 19 | 9 | [9] | [3, 6] | [9] |
| `- - 3 4 5 6 - 8 -` | 19 | 8 | [8] | [3, 5] | [8] |
| `- - 3 4 5 6 - 8 -` | 19 | 9 | [4, 5] | [3, 6] | [3, 6] |
| `- - 3 4 5 6 - 8 -` | 19 | 11 | [3, 8] | [5, 6] | [3, 8] |
| `- - 3 4 5 6 - 8 -` | 19 | 12 | [4, 8] | [3, 4, 5] | [4, 8] |
| `- 2 - - - - 7 8 9` | 19 | 9 | [9] | [2, 7] | [9] |
| `- 2 - 4 5 - 7 8 -` | 19 | 7 | [7] | [2, 5] | [7] |
| `- 2 - 4 5 - 7 8 -` | 19 | 9 | [4, 5] | [2, 7] | [2, 7] |
| `- 2 - 4 5 - 7 8 -` | 19 | 11 | [4, 7] | [2, 4, 5] | [4, 7] |
| `- 2 - 4 5 - 7 8 -` | 19 | 12 | [4, 8] | [5, 7] | [4, 8] |
| `- 2 - 4 5 6 - - 9` | 19 | 6 | [6] | [2, 4] | [6] |
| `- 2 - 4 5 6 - - 9` | 19 | 9 | [9] | [4, 5] | [9] |
| `- 2 - 4 5 6 - - 9` | 19 | 11 | [2, 9] | [2, 4, 5] | [2, 9] |
| `- 2 3 - - 6 7 8 -` | 19 | 8 | [8] | [2, 6] | [8] |
| `- 2 3 - - 6 7 8 -` | 19 | 9 | [3, 6] | [2, 7] | [2, 7] |
| `- 2 3 - - 6 7 8 -` | 19 | 10 | [2, 8] | [3, 7] | [2, 8] |
| `- 2 3 - - 6 7 8 -` | 19 | 11 | [3, 8] | [2, 3, 6] | [3, 8] |
| `- 2 3 - 5 - 7 - 9` | 19 | 5 | [5] | [2, 3] | [5] |
| `- 2 3 - 5 - 7 - 9` | 19 | 7 | [7] | [2, 5] | [7] |
| `- 2 3 - 5 - 7 - 9` | 19 | 9 | [9] | [2, 7] | [9] |
| `- 2 3 - 5 - 7 - 9` | 19 | 10 | [3, 7] | [2, 3, 5] | [3, 7] |
| `- 2 3 - 5 - 7 - 9` | 19 | 12 | [3, 9] | [2, 3, 7] | [3, 9] |
| `- 2 3 4 - - - 8 9` | 19 | 9 | [9] | [2, 3, 4] | [9] |
| `- 2 3 4 - - - 8 9` | 19 | 11 | [2, 9] | [3, 8] | [2, 9] |
| `- 2 3 4 - - - 8 9` | 19 | 12 | [3, 9] | [4, 8] | [3, 9] |
| `1 - - 4 - 6 7 8 -` | 19 | 7 | [7] | [1, 6] | [7] |
| `1 - - 4 - 6 7 8 -` | 19 | 8 | [8] | [1, 7] | [8] |
| `1 - - 4 - 6 7 8 -` | 19 | 11 | [4, 7] | [1, 4, 6] | [4, 7] |
| `1 - - 4 - 6 7 8 -` | 19 | 12 | [4, 8] | [1, 4, 7] | [4, 8] |
| `1 - - 4 5 - 7 - 9` | 19 | 5 | [5] | [1, 4] | [5] |
| `1 - - 4 5 - 7 - 9` | 19 | 9 | [9] | [4, 5] | [9] |
| `1 - - 4 5 - 7 - 9` | 19 | 10 | [1, 9] | [1, 4, 5] | [1, 9] |
| `1 - - 4 5 - 7 - 9` | 19 | 12 | [5, 7] | [1, 4, 7] | [5, 7] |
| `1 - 3 - - 6 7 - 9` | 19 | 7 | [7] | [1, 6] | [7] |
| `1 - 3 - - 6 7 - 9` | 19 | 9 | [9] | [3, 6] | [9] |
| `1 - 3 - - 6 7 - 9` | 19 | 10 | [1, 9] | [1, 3, 6] | [1, 9] |
| `1 - 3 - 5 - - 8 9` | 19 | 8 | [8] | [3, 5] | [8] |
| `1 - 3 - 5 - - 8 9` | 19 | 9 | [9] | [1, 3, 5] | [9] |
| `1 - 3 - 5 - - 8 9` | 19 | 12 | [3, 9] | [1, 3, 8] | [3, 9] |
| `1 - 3 4 5 6 7 - -` | 19 | 4 | [4] | [1, 3] | [4] |
| `1 - 3 4 5 6 7 - -` | 19 | 5 | [5] | [1, 4] | [5] |
| `1 - 3 4 5 6 7 - -` | 19 | 6 | [6] | [1, 5] | [6] |
| `1 - 3 4 5 6 7 - -` | 19 | 7 | [7] | [3, 4] | [7] |
| `1 - 3 4 5 6 7 - -` | 19 | 8 | [1, 7] | [1, 3, 4] | [1, 7] |
| `1 - 3 4 5 6 7 - -` | 19 | 9 | [3, 6] | [1, 3, 5] | [3, 6] |
| `1 - 3 4 5 6 7 - -` | 19 | 10 | [4, 6] | [1, 3, 6] | [3, 7] |
| `1 - 3 4 5 6 7 - -` | 19 | 11 | [4, 7] | [1, 3, 7] | [4, 7] |
| `1 - 3 4 5 6 7 - -` | 19 | 12 | [5, 7] | [1, 5, 6] | [5, 7] |
| `1 2 - - - 6 - 8 9` | 19 | 8 | [8] | [2, 6] | [8] |
| `1 2 - - - 6 - 8 9` | 19 | 9 | [9] | [1, 2, 6] | [9] |
| `1 2 - - - 6 - 8 9` | 19 | 10 | [2, 8] | [1, 9] | [1, 9] |
| `1 2 - - - 6 - 8 9` | 19 | 11 | [2, 9] | [1, 2, 8] | [2, 9] |
| `1 2 - 4 5 6 - 8 -` | 19 | 5 | [5] | [1, 4] | [5] |
| `1 2 - 4 5 6 - 8 -` | 19 | 6 | [6] | [1, 5] | [6] |
| `1 2 - 4 5 6 - 8 -` | 19 | 7 | [1, 6] | [1, 2, 4] | [1, 6] |
| `1 2 - 4 5 6 - 8 -` | 19 | 8 | [8] | [1, 2, 5] | [8] |
| `1 2 - 4 5 6 - 8 -` | 19 | 9 | [1, 8] | [1, 2, 6] | [1, 8] |
| `1 2 - 4 5 6 - 8 -` | 19 | 10 | [4, 6] | [1, 4, 5] | [2, 8] |
| `1 2 - 4 5 6 - 8 -` | 19 | 11 | [5, 6] | [1, 2, 8] | [5, 6] |
| `1 2 - 4 5 6 - 8 -` | 19 | 12 | [4, 8] | [1, 2, 4, 5] | [4, 8] |
| `1 2 3 - 5 - 7 8 -` | 19 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 - 5 - 7 8 -` | 19 | 5 | [5] | [2, 3] | [5] |
| `1 2 3 - 5 - 7 8 -` | 19 | 6 | [1, 5] | [1, 2, 3] | [1, 5] |
| `1 2 3 - 5 - 7 8 -` | 19 | 7 | [7] | [2, 5] | [7] |
| `1 2 3 - 5 - 7 8 -` | 19 | 8 | [8] | [1, 2, 5] | [8] |
| `1 2 3 - 5 - 7 8 -` | 19 | 9 | [1, 8] | [1, 3, 5] | [1, 8] |
| `1 2 3 - 5 - 7 8 -` | 19 | 10 | [2, 8] | [1, 2, 7] | [2, 8] |
| `1 2 3 - 5 - 7 8 -` | 19 | 11 | [3, 8] | [1, 2, 3, 5] | [3, 8] |
| `1 2 3 - 5 - 7 8 -` | 19 | 12 | [5, 7] | [1, 3, 8] | [5, 7] |
| `1 2 3 - 5 6 - - 9` | 19 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 - 5 6 - - 9` | 19 | 5 | [5] | [2, 3] | [5] |
| `1 2 3 - 5 6 - - 9` | 19 | 6 | [6] | [1, 2, 3] | [6] |
| `1 2 3 - 5 6 - - 9` | 19 | 7 | [1, 6] | [2, 5] | [1, 6] |
| `1 2 3 - 5 6 - - 9` | 19 | 8 | [3, 5] | [1, 2, 5] | [2, 6] |
| `1 2 3 - 5 6 - - 9` | 19 | 9 | [9] | [1, 2, 6] | [9] |
| `1 2 3 - 5 6 - - 9` | 19 | 10 | [1, 9] | [1, 3, 6] | [1, 9] |
| `1 2 3 - 5 6 - - 9` | 19 | 11 | [2, 9] | [1, 2, 3, 5] | [2, 9] |
| `1 2 3 - 5 6 - - 9` | 19 | 12 | [3, 9] | [1, 2, 3, 6] | [3, 9] |
| `1 2 3 4 - - 7 - 9` | 19 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 4 - - 7 - 9` | 19 | 4 | [4] | [1, 3] | [4] |
| `1 2 3 4 - - 7 - 9` | 19 | 5 | [2, 3] | [1, 4] | [1, 4] |
| `1 2 3 4 - - 7 - 9` | 19 | 6 | [2, 4] | [1, 2, 3] | [2, 4] |
| `1 2 3 4 - - 7 - 9` | 19 | 7 | [7] | [1, 2, 4] | [7] |
| `1 2 3 4 - - 7 - 9` | 19 | 8 | [1, 7] | [1, 3, 4] | [1, 7] |
| `1 2 3 4 - - 7 - 9` | 19 | 9 | [9] | [2, 3, 4] | [9] |
| `1 2 3 4 - - 7 - 9` | 19 | 10 | [1, 9] | [1, 2, 3, 4] | [1, 9] |
| `1 2 3 4 - - 7 - 9` | 19 | 11 | [2, 9] | [1, 3, 7] | [2, 9] |
| `1 2 3 4 - - 7 - 9` | 19 | 12 | [3, 9] | [1, 4, 7] | [3, 9] |
| `- - - 4 5 - 7 - 9` | 20 | 9 | [9] | [4, 5] | [9] |
| `- - 3 - - 6 7 - 9` | 20 | 9 | [9] | [3, 6] | [9] |
| `- - 3 - 5 - - 8 9` | 20 | 8 | [8] | [3, 5] | [8] |
| `- - 3 4 5 6 7 - -` | 20 | 7 | [7] | [3, 4] | [7] |
| `- - 3 4 5 6 7 - -` | 20 | 9 | [4, 5] | [3, 6] | [3, 6] |
| `- - 3 4 5 6 7 - -` | 20 | 10 | [4, 6] | [3, 7] | [3, 7] |
| `- - 3 4 5 6 7 - -` | 20 | 11 | [4, 7] | [5, 6] | [4, 7] |
| `- - 3 4 5 6 7 - -` | 20 | 12 | [5, 7] | [3, 4, 5] | [5, 7] |
| `- 2 - - - 6 - 8 9` | 20 | 8 | [8] | [2, 6] | [8] |
| `- 2 - 4 5 6 - 8 -` | 20 | 6 | [6] | [2, 4] | [6] |
| `- 2 - 4 5 6 - 8 -` | 20 | 8 | [8] | [2, 6] | [8] |
| `- 2 - 4 5 6 - 8 -` | 20 | 10 | [4, 6] | [2, 8] | [2, 8] |
| `- 2 - 4 5 6 - 8 -` | 20 | 11 | [5, 6] | [2, 4, 5] | [5, 6] |
| `- 2 - 4 5 6 - 8 -` | 20 | 12 | [4, 8] | [2, 4, 6] | [4, 8] |
| `- 2 3 - 5 - 7 8 -` | 20 | 5 | [5] | [2, 3] | [5] |
| `- 2 3 - 5 - 7 8 -` | 20 | 7 | [7] | [2, 5] | [7] |
| `- 2 3 - 5 - 7 8 -` | 20 | 8 | [8] | [3, 5] | [8] |
| `- 2 3 - 5 - 7 8 -` | 20 | 10 | [2, 8] | [2, 3, 5] | [2, 8] |
| `- 2 3 - 5 - 7 8 -` | 20 | 12 | [5, 7] | [2, 3, 7] | [5, 7] |
| `- 2 3 - 5 6 - - 9` | 20 | 5 | [5] | [2, 3] | [5] |
| `- 2 3 - 5 6 - - 9` | 20 | 8 | [3, 5] | [2, 6] | [2, 6] |
| `- 2 3 - 5 6 - - 9` | 20 | 9 | [9] | [3, 6] | [9] |
| `- 2 3 - 5 6 - - 9` | 20 | 11 | [2, 9] | [2, 3, 6] | [2, 9] |
| `- 2 3 4 - - 7 - 9` | 20 | 7 | [7] | [3, 4] | [7] |
| `- 2 3 4 - - 7 - 9` | 20 | 9 | [9] | [2, 3, 4] | [9] |
| `- 2 3 4 - - 7 - 9` | 20 | 11 | [2, 9] | [4, 7] | [2, 9] |
| `- 2 3 4 - - 7 - 9` | 20 | 12 | [3, 9] | [2, 3, 7] | [3, 9] |
| `1 - - - - - 7 8 9` | 20 | 8 | [8] | [1, 7] | [8] |
| `1 - - - - - 7 8 9` | 20 | 9 | [9] | [1, 8] | [9] |
| `1 - - 4 5 - 7 8 -` | 20 | 5 | [5] | [1, 4] | [5] |
| `1 - - 4 5 - 7 8 -` | 20 | 8 | [8] | [1, 7] | [8] |
| `1 - - 4 5 - 7 8 -` | 20 | 9 | [1, 8] | [4, 5] | [1, 8] |
| `1 - - 4 5 - 7 8 -` | 20 | 12 | [4, 8] | [1, 4, 7] | [4, 8] |
| `1 - - 4 5 6 - - 9` | 20 | 5 | [5] | [1, 4] | [5] |
| `1 - - 4 5 6 - - 9` | 20 | 6 | [6] | [1, 5] | [6] |
| `1 - - 4 5 6 - - 9` | 20 | 9 | [9] | [4, 5] | [9] |
| `1 - - 4 5 6 - - 9` | 20 | 10 | [1, 9] | [1, 4, 5] | [1, 9] |
| `1 - - 4 5 6 - - 9` | 20 | 11 | [5, 6] | [1, 4, 6] | [5, 6] |
| `1 - 3 - - 6 7 8 -` | 20 | 7 | [7] | [1, 6] | [7] |
| `1 - 3 - - 6 7 8 -` | 20 | 8 | [8] | [1, 7] | [8] |
| `1 - 3 - - 6 7 8 -` | 20 | 9 | [1, 8] | [3, 6] | [1, 8] |
| `1 - 3 - - 6 7 8 -` | 20 | 10 | [3, 7] | [1, 3, 6] | [3, 7] |
| `1 - 3 - - 6 7 8 -` | 20 | 11 | [3, 8] | [1, 3, 7] | [3, 8] |
| `1 - 3 - 5 - 7 - 9` | 20 | 8 | [3, 5] | [1, 7] | [1, 7] |
| `1 - 3 - 5 - 7 - 9` | 20 | 9 | [9] | [1, 3, 5] | [9] |
| `1 - 3 - 5 - 7 - 9` | 20 | 10 | [1, 9] | [3, 7] | [1, 9] |
| `1 - 3 - 5 - 7 - 9` | 20 | 12 | [3, 9] | [5, 7] | [3, 9] |
| `1 - 3 4 - - - 8 9` | 20 | 4 | [4] | [1, 3] | [4] |
| `1 - 3 4 - - - 8 9` | 20 | 8 | [8] | [1, 3, 4] | [8] |
| `1 - 3 4 - - - 8 9` | 20 | 9 | [9] | [1, 8] | [9] |
| `1 - 3 4 - - - 8 9` | 20 | 12 | [3, 9] | [1, 3, 8] | [3, 9] |
| `1 2 - - - 6 7 - 9` | 20 | 7 | [7] | [1, 6] | [7] |
| `1 2 - - - 6 7 - 9` | 20 | 8 | [2, 6] | [1, 7] | [1, 7] |
| `1 2 - - - 6 7 - 9` | 20 | 9 | [9] | [1, 2, 6] | [9] |
| `1 2 - - - 6 7 - 9` | 20 | 10 | [1, 9] | [1, 2, 7] | [1, 9] |
| `1 2 - - 5 - - 8 9` | 20 | 8 | [8] | [1, 2, 5] | [8] |
| `1 2 - - 5 - - 8 9` | 20 | 9 | [9] | [1, 8] | [9] |
| `1 2 - - 5 - - 8 9` | 20 | 10 | [1, 9] | [2, 8] | [1, 9] |
| `1 2 - - 5 - - 8 9` | 20 | 11 | [2, 9] | [1, 2, 8] | [2, 9] |
| `1 2 - 4 5 6 7 - -` | 20 | 5 | [5] | [1, 4] | [5] |
| `1 2 - 4 5 6 7 - -` | 20 | 6 | [6] | [2, 4] | [6] |
| `1 2 - 4 5 6 7 - -` | 20 | 7 | [7] | [1, 2, 4] | [7] |
| `1 2 - 4 5 6 7 - -` | 20 | 8 | [1, 7] | [1, 2, 5] | [1, 7] |
| `1 2 - 4 5 6 7 - -` | 20 | 9 | [2, 7] | [1, 2, 6] | [2, 7] |
| `1 2 - 4 5 6 7 - -` | 20 | 10 | [4, 6] | [1, 2, 7] | [4, 6] |
| `1 2 - 4 5 6 7 - -` | 20 | 11 | [5, 6] | [1, 4, 6] | [4, 7] |
| `1 2 - 4 5 6 7 - -` | 20 | 12 | [5, 7] | [1, 2, 4, 5] | [5, 7] |
| `1 2 3 - 5 6 - 8 -` | 20 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 - 5 6 - 8 -` | 20 | 5 | [5] | [2, 3] | [5] |
| `1 2 3 - 5 6 - 8 -` | 20 | 6 | [6] | [1, 2, 3] | [6] |
| `1 2 3 - 5 6 - 8 -` | 20 | 7 | [2, 5] | [1, 6] | [1, 6] |
| `1 2 3 - 5 6 - 8 -` | 20 | 8 | [8] | [1, 2, 5] | [8] |
| `1 2 3 - 5 6 - 8 -` | 20 | 9 | [1, 8] | [1, 2, 6] | [1, 8] |
| `1 2 3 - 5 6 - 8 -` | 20 | 10 | [2, 8] | [1, 3, 6] | [2, 8] |
| `1 2 3 - 5 6 - 8 -` | 20 | 11 | [3, 8] | [1, 2, 3, 5] | [3, 8] |
| `1 2 3 - 5 6 - 8 -` | 20 | 12 | [1, 3, 8] | [1, 2, 3, 6] | [1, 3, 8] |
| `1 2 3 4 - - 7 8 -` | 20 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 4 - - 7 8 -` | 20 | 4 | [4] | [1, 3] | [4] |
| `1 2 3 4 - - 7 8 -` | 20 | 5 | [1, 4] | [2, 3] | [1, 4] |
| `1 2 3 4 - - 7 8 -` | 20 | 6 | [2, 4] | [1, 2, 3] | [2, 4] |
| `1 2 3 4 - - 7 8 -` | 20 | 7 | [7] | [1, 2, 4] | [7] |
| `1 2 3 4 - - 7 8 -` | 20 | 8 | [8] | [1, 3, 4] | [8] |
| `1 2 3 4 - - 7 8 -` | 20 | 9 | [1, 8] | [2, 3, 4] | [1, 8] |
| `1 2 3 4 - - 7 8 -` | 20 | 10 | [3, 7] | [1, 2, 3, 4] | [2, 8] |
| `1 2 3 4 - - 7 8 -` | 20 | 11 | [3, 8] | [1, 2, 8] | [3, 8] |
| `1 2 3 4 - - 7 8 -` | 20 | 12 | [4, 8] | [1, 4, 7] | [4, 8] |
| `1 2 3 4 - 6 - - 9` | 20 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 4 - 6 - - 9` | 20 | 4 | [4] | [1, 3] | [4] |
| `1 2 3 4 - 6 - - 9` | 20 | 5 | [2, 3] | [1, 4] | [1, 4] |
| `1 2 3 4 - 6 - - 9` | 20 | 6 | [6] | [1, 2, 3] | [6] |
| `1 2 3 4 - 6 - - 9` | 20 | 7 | [3, 4] | [1, 2, 4] | [1, 6] |
| `1 2 3 4 - 6 - - 9` | 20 | 8 | [2, 6] | [1, 3, 4] | [2, 6] |
| `1 2 3 4 - 6 - - 9` | 20 | 9 | [9] | [1, 2, 6] | [9] |
| `1 2 3 4 - 6 - - 9` | 20 | 10 | [1, 9] | [1, 2, 3, 4] | [1, 9] |
| `1 2 3 4 - 6 - - 9` | 20 | 11 | [2, 9] | [1, 4, 6] | [2, 9] |
| `1 2 3 4 - 6 - - 9` | 20 | 12 | [3, 9] | [1, 2, 3, 6] | [3, 9] |
| `- - - 4 5 - 7 8 -` | 21 | 12 | [4, 8] | [5, 7] | [4, 8] |
| `- - - 4 5 6 - - 9` | 21 | 9 | [9] | [4, 5] | [9] |
| `- - 3 - 5 - 7 - 9` | 21 | 12 | [3, 9] | [5, 7] | [3, 9] |
| `- - 3 4 - - - 8 9` | 21 | 12 | [3, 9] | [4, 8] | [3, 9] |
| `- 2 - - - 6 7 - 9` | 21 | 9 | [9] | [2, 7] | [9] |
| `- 2 - 4 5 6 7 - -` | 21 | 6 | [6] | [2, 4] | [6] |
| `- 2 - 4 5 6 7 - -` | 21 | 7 | [7] | [2, 5] | [7] |
| `- 2 - 4 5 6 7 - -` | 21 | 9 | [4, 5] | [2, 7] | [2, 7] |
| `- 2 - 4 5 6 7 - -` | 21 | 11 | [4, 7] | [2, 4, 5] | [4, 7] |
| `- 2 - 4 5 6 7 - -` | 21 | 12 | [5, 7] | [2, 4, 6] | [5, 7] |
| `- 2 3 - 5 6 - 8 -` | 21 | 5 | [5] | [2, 3] | [5] |
| `- 2 3 - 5 6 - 8 -` | 21 | 8 | [8] | [2, 6] | [8] |
| `- 2 3 - 5 6 - 8 -` | 21 | 10 | [2, 8] | [2, 3, 5] | [2, 8] |
| `- 2 3 - 5 6 - 8 -` | 21 | 11 | [3, 8] | [2, 3, 6] | [3, 8] |
| `- 2 3 4 - - 7 8 -` | 21 | 7 | [7] | [3, 4] | [7] |
| `- 2 3 4 - - 7 8 -` | 21 | 9 | [2, 7] | [2, 3, 4] | [2, 7] |
| `- 2 3 4 - - 7 8 -` | 21 | 10 | [3, 7] | [2, 8] | [2, 8] |
| `- 2 3 4 - - 7 8 -` | 21 | 11 | [3, 8] | [4, 7] | [3, 8] |
| `- 2 3 4 - - 7 8 -` | 21 | 12 | [4, 8] | [2, 3, 7] | [4, 8] |
| `- 2 3 4 - 6 - - 9` | 21 | 6 | [6] | [2, 4] | [6] |
| `- 2 3 4 - 6 - - 9` | 21 | 9 | [9] | [2, 3, 4] | [9] |
| `- 2 3 4 - 6 - - 9` | 21 | 11 | [2, 9] | [2, 3, 6] | [2, 9] |
| `- 2 3 4 - 6 - - 9` | 21 | 12 | [3, 9] | [2, 4, 6] | [3, 9] |
| `1 - - - - 6 - 8 9` | 21 | 9 | [9] | [1, 8] | [9] |
| `1 - - 4 5 6 - 8 -` | 21 | 5 | [5] | [1, 4] | [5] |
| `1 - - 4 5 6 - 8 -` | 21 | 6 | [6] | [1, 5] | [6] |
| `1 - - 4 5 6 - 8 -` | 21 | 9 | [4, 5] | [1, 8] | [1, 8] |
| `1 - - 4 5 6 - 8 -` | 21 | 10 | [4, 6] | [1, 4, 5] | [4, 6] |
| `1 - - 4 5 6 - 8 -` | 21 | 11 | [5, 6] | [1, 4, 6] | [5, 6] |
| `1 - - 4 5 6 - 8 -` | 21 | 12 | [4, 8] | [1, 5, 6] | [4, 8] |
| `1 - 3 - 5 - 7 8 -` | 21 | 8 | [8] | [1, 7] | [8] |
| `1 - 3 - 5 - 7 8 -` | 21 | 9 | [1, 8] | [1, 3, 5] | [1, 8] |
| `1 - 3 - 5 - 7 8 -` | 21 | 11 | [3, 8] | [1, 3, 7] | [3, 8] |
| `1 - 3 - 5 - 7 8 -` | 21 | 12 | [1, 3, 8] | [5, 7] | [5, 7] |
| `1 - 3 - 5 6 - - 9` | 21 | 6 | [6] | [1, 5] | [6] |
| `1 - 3 - 5 6 - - 9` | 21 | 9 | [9] | [1, 3, 5] | [9] |
| `1 - 3 - 5 6 - - 9` | 21 | 10 | [1, 9] | [1, 3, 6] | [1, 9] |
| `1 - 3 - 5 6 - - 9` | 21 | 12 | [3, 9] | [1, 5, 6] | [3, 9] |
| `1 - 3 4 - - 7 - 9` | 21 | 4 | [4] | [1, 3] | [4] |
| `1 - 3 4 - - 7 - 9` | 21 | 7 | [7] | [3, 4] | [7] |
| `1 - 3 4 - - 7 - 9` | 21 | 8 | [1, 7] | [1, 3, 4] | [1, 7] |
| `1 - 3 4 - - 7 - 9` | 21 | 10 | [1, 9] | [3, 7] | [1, 9] |
| `1 - 3 4 - - 7 - 9` | 21 | 11 | [4, 7] | [1, 3, 7] | [4, 7] |
| `1 - 3 4 - - 7 - 9` | 21 | 12 | [3, 9] | [1, 4, 7] | [3, 9] |
| `1 2 - - - 6 7 8 -` | 21 | 7 | [7] | [1, 6] | [7] |
| `1 2 - - - 6 7 8 -` | 21 | 8 | [8] | [1, 7] | [8] |
| `1 2 - - - 6 7 8 -` | 21 | 9 | [1, 8] | [1, 2, 6] | [1, 8] |
| `1 2 - - - 6 7 8 -` | 21 | 10 | [2, 8] | [1, 2, 7] | [2, 8] |
| `1 2 - - 5 - 7 - 9` | 21 | 7 | [7] | [2, 5] | [7] |
| `1 2 - - 5 - 7 - 9` | 21 | 8 | [1, 7] | [1, 2, 5] | [1, 7] |
| `1 2 - - 5 - 7 - 9` | 21 | 9 | [9] | [2, 7] | [9] |
| `1 2 - - 5 - 7 - 9` | 21 | 10 | [1, 9] | [1, 2, 7] | [1, 9] |
| `1 2 - - 5 - 7 - 9` | 21 | 12 | [1, 2, 9] | [5, 7] | [5, 7] |
| `1 2 - 4 - - - 8 9` | 21 | 9 | [9] | [1, 8] | [9] |
| `1 2 - 4 - - - 8 9` | 21 | 10 | [1, 9] | [2, 8] | [1, 9] |
| `1 2 - 4 - - - 8 9` | 21 | 11 | [2, 9] | [1, 2, 8] | [2, 9] |
| `1 2 - 4 - - - 8 9` | 21 | 12 | [1, 2, 9] | [4, 8] | [4, 8] |
| `1 2 3 - 5 6 7 - -` | 21 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 - 5 6 7 - -` | 21 | 5 | [5] | [2, 3] | [5] |
| `1 2 3 - 5 6 7 - -` | 21 | 6 | [6] | [1, 2, 3] | [6] |
| `1 2 3 - 5 6 7 - -` | 21 | 7 | [7] | [1, 6] | [7] |
| `1 2 3 - 5 6 7 - -` | 21 | 8 | [2, 6] | [1, 2, 5] | [1, 7] |
| `1 2 3 - 5 6 7 - -` | 21 | 9 | [2, 7] | [1, 2, 6] | [2, 7] |
| `1 2 3 - 5 6 7 - -` | 21 | 10 | [3, 7] | [1, 3, 6] | [3, 7] |
| `1 2 3 - 5 6 7 - -` | 21 | 11 | [5, 6] | [1, 2, 3, 5] | [5, 6] |
| `1 2 3 - 5 6 7 - -` | 21 | 12 | [5, 7] | [1, 2, 3, 6] | [5, 7] |
| `1 2 3 4 - 6 - 8 -` | 21 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 4 - 6 - 8 -` | 21 | 4 | [4] | [1, 3] | [4] |
| `1 2 3 4 - 6 - 8 -` | 21 | 5 | [2, 3] | [1, 4] | [1, 4] |
| `1 2 3 4 - 6 - 8 -` | 21 | 6 | [6] | [1, 2, 3] | [6] |
| `1 2 3 4 - 6 - 8 -` | 21 | 7 | [1, 6] | [1, 2, 4] | [1, 6] |
| `1 2 3 4 - 6 - 8 -` | 21 | 8 | [8] | [1, 3, 4] | [8] |
| `1 2 3 4 - 6 - 8 -` | 21 | 9 | [1, 8] | [1, 2, 6] | [1, 8] |
| `1 2 3 4 - 6 - 8 -` | 21 | 10 | [2, 8] | [1, 2, 3, 4] | [2, 8] |
| `1 2 3 4 - 6 - 8 -` | 21 | 11 | [3, 8] | [1, 4, 6] | [3, 8] |
| `1 2 3 4 - 6 - 8 -` | 21 | 12 | [4, 8] | [1, 2, 3, 6] | [4, 8] |
| `1 2 3 4 5 - - - 9` | 21 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 4 5 - - - 9` | 21 | 4 | [4] | [1, 3] | [4] |
| `1 2 3 4 5 - - - 9` | 21 | 5 | [5] | [2, 3] | [5] |
| `1 2 3 4 5 - - - 9` | 21 | 6 | [2, 4] | [1, 2, 3] | [1, 5] |
| `1 2 3 4 5 - - - 9` | 21 | 7 | [3, 4] | [1, 2, 4] | [2, 5] |
| `1 2 3 4 5 - - - 9` | 21 | 8 | [3, 5] | [1, 2, 5] | [3, 5] |
| `1 2 3 4 5 - - - 9` | 21 | 9 | [9] | [1, 3, 5] | [9] |
| `1 2 3 4 5 - - - 9` | 21 | 10 | [1, 9] | [1, 2, 3, 4] | [1, 9] |
| `1 2 3 4 5 - - - 9` | 21 | 11 | [2, 9] | [1, 2, 3, 5] | [2, 9] |
| `1 2 3 4 5 - - - 9` | 21 | 12 | [3, 9] | [1, 2, 4, 5] | [3, 9] |
| `- - 3 - 5 - 7 8 -` | 22 | 8 | [8] | [3, 5] | [8] |
| `- - 3 - 5 6 - - 9` | 22 | 9 | [9] | [3, 6] | [9] |
| `- - 3 4 - - 7 - 9` | 22 | 7 | [7] | [3, 4] | [7] |
| `- 2 - - - 6 7 8 -` | 22 | 8 | [8] | [2, 6] | [8] |
| `- 2 - - 5 - 7 - 9` | 22 | 7 | [7] | [2, 5] | [7] |
| `- 2 - - 5 - 7 - 9` | 22 | 9 | [9] | [2, 7] | [9] |
| `- 2 3 - 5 6 7 - -` | 22 | 5 | [5] | [2, 3] | [5] |
| `- 2 3 - 5 6 7 - -` | 22 | 7 | [7] | [2, 5] | [7] |
| `- 2 3 - 5 6 7 - -` | 22 | 8 | [3, 5] | [2, 6] | [2, 6] |
| `- 2 3 - 5 6 7 - -` | 22 | 9 | [2, 7] | [3, 6] | [2, 7] |
| `- 2 3 - 5 6 7 - -` | 22 | 10 | [3, 7] | [2, 3, 5] | [3, 7] |
| `- 2 3 - 5 6 7 - -` | 22 | 11 | [5, 6] | [2, 3, 6] | [5, 6] |
| `- 2 3 - 5 6 7 - -` | 22 | 12 | [5, 7] | [2, 3, 7] | [5, 7] |
| `- 2 3 4 - 6 - 8 -` | 22 | 6 | [6] | [2, 4] | [6] |
| `- 2 3 4 - 6 - 8 -` | 22 | 8 | [8] | [2, 6] | [8] |
| `- 2 3 4 - 6 - 8 -` | 22 | 9 | [3, 6] | [2, 3, 4] | [3, 6] |
| `- 2 3 4 - 6 - 8 -` | 22 | 10 | [2, 8] | [4, 6] | [2, 8] |
| `- 2 3 4 - 6 - 8 -` | 22 | 11 | [3, 8] | [2, 3, 6] | [3, 8] |
| `- 2 3 4 - 6 - 8 -` | 22 | 12 | [4, 8] | [2, 4, 6] | [4, 8] |
| `- 2 3 4 5 - - - 9` | 22 | 5 | [5] | [2, 3] | [5] |
| `- 2 3 4 5 - - - 9` | 22 | 7 | [3, 4] | [2, 5] | [2, 5] |
| `- 2 3 4 5 - - - 9` | 22 | 9 | [9] | [2, 3, 4] | [9] |
| `- 2 3 4 5 - - - 9` | 22 | 11 | [2, 9] | [2, 4, 5] | [2, 9] |
| `- 2 3 4 5 - - - 9` | 22 | 12 | [3, 9] | [3, 4, 5] | [3, 9] |
| `1 - - - - 6 7 - 9` | 22 | 7 | [7] | [1, 6] | [7] |
| `1 - - - 5 - - 8 9` | 22 | 9 | [9] | [1, 8] | [9] |
| `1 - - 4 5 6 7 - -` | 22 | 5 | [5] | [1, 4] | [5] |
| `1 - - 4 5 6 7 - -` | 22 | 6 | [6] | [1, 5] | [6] |
| `1 - - 4 5 6 7 - -` | 22 | 7 | [7] | [1, 6] | [7] |
| `1 - - 4 5 6 7 - -` | 22 | 10 | [4, 6] | [1, 4, 5] | [4, 6] |
| `1 - - 4 5 6 7 - -` | 22 | 11 | [5, 6] | [1, 4, 6] | [4, 7] |
| `1 - - 4 5 6 7 - -` | 22 | 12 | [5, 7] | [1, 5, 6] | [5, 7] |
| `1 - 3 - 5 6 - 8 -` | 22 | 6 | [6] | [1, 5] | [6] |
| `1 - 3 - 5 6 - 8 -` | 22 | 8 | [8] | [3, 5] | [8] |
| `1 - 3 - 5 6 - 8 -` | 22 | 9 | [1, 8] | [1, 3, 5] | [1, 8] |
| `1 - 3 - 5 6 - 8 -` | 22 | 11 | [3, 8] | [5, 6] | [3, 8] |
| `1 - 3 - 5 6 - 8 -` | 22 | 12 | [1, 3, 8] | [1, 5, 6] | [1, 3, 8] |
| `1 - 3 4 - - 7 8 -` | 22 | 4 | [4] | [1, 3] | [4] |
| `1 - 3 4 - - 7 8 -` | 22 | 7 | [7] | [3, 4] | [7] |
| `1 - 3 4 - - 7 8 -` | 22 | 8 | [8] | [1, 3, 4] | [8] |
| `1 - 3 4 - - 7 8 -` | 22 | 11 | [3, 8] | [1, 3, 7] | [3, 8] |
| `1 - 3 4 - - 7 8 -` | 22 | 12 | [4, 8] | [1, 4, 7] | [4, 8] |
| `1 - 3 4 - 6 - - 9` | 22 | 4 | [4] | [1, 3] | [4] |
| `1 - 3 4 - 6 - - 9` | 22 | 7 | [3, 4] | [1, 6] | [1, 6] |
| `1 - 3 4 - 6 - - 9` | 22 | 9 | [9] | [3, 6] | [9] |
| `1 - 3 4 - 6 - - 9` | 22 | 10 | [1, 9] | [1, 3, 6] | [1, 9] |
| `1 2 - - 5 - 7 8 -` | 22 | 7 | [7] | [2, 5] | [7] |
| `1 2 - - 5 - 7 8 -` | 22 | 8 | [8] | [1, 2, 5] | [8] |
| `1 2 - - 5 - 7 8 -` | 22 | 9 | [2, 7] | [1, 8] | [1, 8] |
| `1 2 - - 5 - 7 8 -` | 22 | 10 | [2, 8] | [1, 2, 7] | [2, 8] |
| `1 2 - - 5 6 - - 9` | 22 | 6 | [6] | [1, 5] | [6] |
| `1 2 - - 5 6 - - 9` | 22 | 7 | [2, 5] | [1, 6] | [1, 6] |
| `1 2 - - 5 6 - - 9` | 22 | 8 | [2, 6] | [1, 2, 5] | [2, 6] |
| `1 2 - - 5 6 - - 9` | 22 | 9 | [9] | [1, 2, 6] | [9] |
| `1 2 - - 5 6 - - 9` | 22 | 11 | [2, 9] | [5, 6] | [2, 9] |
| `1 2 - - 5 6 - - 9` | 22 | 12 | [1, 2, 9] | [1, 5, 6] | [1, 2, 9] |
| `1 2 - 4 - - 7 - 9` | 22 | 7 | [7] | [1, 2, 4] | [7] |
| `1 2 - 4 - - 7 - 9` | 22 | 9 | [9] | [2, 7] | [9] |
| `1 2 - 4 - - 7 - 9` | 22 | 10 | [1, 9] | [1, 2, 7] | [1, 9] |
| `1 2 - 4 - - 7 - 9` | 22 | 11 | [2, 9] | [4, 7] | [2, 9] |
| `1 2 - 4 - - 7 - 9` | 22 | 12 | [1, 2, 9] | [1, 4, 7] | [1, 2, 9] |
| `1 2 3 - - - - 8 9` | 22 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 - - - - 8 9` | 22 | 9 | [9] | [1, 8] | [9] |
| `1 2 3 - - - - 8 9` | 22 | 10 | [1, 9] | [2, 8] | [1, 9] |
| `1 2 3 - - - - 8 9` | 22 | 11 | [2, 9] | [1, 2, 8] | [2, 9] |
| `1 2 3 - - - - 8 9` | 22 | 12 | [3, 9] | [1, 3, 8] | [3, 9] |
| `1 2 3 4 - 6 7 - -` | 22 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 4 - 6 7 - -` | 22 | 4 | [4] | [1, 3] | [4] |
| `1 2 3 4 - 6 7 - -` | 22 | 5 | [1, 4] | [2, 3] | [1, 4] |
| `1 2 3 4 - 6 7 - -` | 22 | 6 | [6] | [1, 2, 3] | [6] |
| `1 2 3 4 - 6 7 - -` | 22 | 7 | [7] | [1, 2, 4] | [7] |
| `1 2 3 4 - 6 7 - -` | 22 | 8 | [1, 7] | [1, 3, 4] | [1, 7] |
| `1 2 3 4 - 6 7 - -` | 22 | 9 | [2, 7] | [1, 2, 6] | [2, 7] |
| `1 2 3 4 - 6 7 - -` | 22 | 10 | [3, 7] | [1, 2, 3, 4] | [3, 7] |
| `1 2 3 4 - 6 7 - -` | 22 | 11 | [4, 7] | [1, 3, 7] | [4, 7] |
| `1 2 3 4 - 6 7 - -` | 22 | 12 | [2, 3, 7] | [1, 2, 3, 6] | [1, 4, 7] |
| `1 2 3 4 5 - - 8 -` | 22 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 4 5 - - 8 -` | 22 | 5 | [5] | [1, 4] | [5] |
| `1 2 3 4 5 - - 8 -` | 22 | 6 | [1, 5] | [1, 2, 3] | [1, 5] |
| `1 2 3 4 5 - - 8 -` | 22 | 7 | [3, 4] | [1, 2, 4] | [2, 5] |
| `1 2 3 4 5 - - 8 -` | 22 | 8 | [8] | [1, 2, 5] | [8] |
| `1 2 3 4 5 - - 8 -` | 22 | 9 | [1, 8] | [1, 3, 5] | [1, 8] |
| `1 2 3 4 5 - - 8 -` | 22 | 10 | [2, 8] | [1, 2, 3, 4] | [2, 8] |
| `1 2 3 4 5 - - 8 -` | 22 | 11 | [3, 8] | [1, 2, 3, 5] | [3, 8] |
| `1 2 3 4 5 - - 8 -` | 22 | 12 | [4, 8] | [1, 2, 4, 5] | [4, 8] |
| `- - - 4 5 6 7 - -` | 23 | 11 | [4, 7] | [5, 6] | [4, 7] |
| `- - 3 - 5 6 - 8 -` | 23 | 8 | [8] | [3, 5] | [8] |
| `- - 3 - 5 6 - 8 -` | 23 | 11 | [3, 8] | [5, 6] | [3, 8] |
| `- - 3 4 - - 7 8 -` | 23 | 7 | [7] | [3, 4] | [7] |
| `- - 3 4 - - 7 8 -` | 23 | 11 | [3, 8] | [4, 7] | [3, 8] |
| `- - 3 4 - 6 - - 9` | 23 | 9 | [9] | [3, 6] | [9] |
| `- 2 - - 5 - 7 8 -` | 23 | 7 | [7] | [2, 5] | [7] |
| `- 2 - - 5 6 - - 9` | 23 | 11 | [2, 9] | [5, 6] | [2, 9] |
| `- 2 - 4 - - 7 - 9` | 23 | 9 | [9] | [2, 7] | [9] |
| `- 2 - 4 - - 7 - 9` | 23 | 11 | [2, 9] | [4, 7] | [2, 9] |
| `- 2 3 - - - - 8 9` | 23 | 11 | [2, 9] | [3, 8] | [2, 9] |
| `- 2 3 4 - 6 7 - -` | 23 | 6 | [6] | [2, 4] | [6] |
| `- 2 3 4 - 6 7 - -` | 23 | 7 | [7] | [3, 4] | [7] |
| `- 2 3 4 - 6 7 - -` | 23 | 9 | [2, 7] | [2, 3, 4] | [2, 7] |
| `- 2 3 4 - 6 7 - -` | 23 | 10 | [4, 6] | [3, 7] | [3, 7] |
| `- 2 3 4 - 6 7 - -` | 23 | 11 | [4, 7] | [2, 3, 6] | [4, 7] |
| `- 2 3 4 - 6 7 - -` | 23 | 12 | [2, 3, 7] | [2, 4, 6] | [2, 3, 7] |
| `- 2 3 4 5 - - 8 -` | 23 | 5 | [5] | [2, 3] | [5] |
| `- 2 3 4 5 - - 8 -` | 23 | 7 | [3, 4] | [2, 5] | [2, 5] |
| `- 2 3 4 5 - - 8 -` | 23 | 8 | [8] | [3, 5] | [8] |
| `- 2 3 4 5 - - 8 -` | 23 | 9 | [4, 5] | [2, 3, 4] | [4, 5] |
| `- 2 3 4 5 - - 8 -` | 23 | 10 | [2, 8] | [2, 3, 5] | [2, 8] |
| `- 2 3 4 5 - - 8 -` | 23 | 11 | [3, 8] | [2, 4, 5] | [3, 8] |
| `- 2 3 4 5 - - 8 -` | 23 | 12 | [4, 8] | [3, 4, 5] | [4, 8] |
| `1 - - - - 6 7 8 -` | 23 | 7 | [7] | [1, 6] | [7] |
| `1 - - - - 6 7 8 -` | 23 | 8 | [8] | [1, 7] | [8] |
| `1 - - 4 - - - 8 9` | 23 | 9 | [9] | [1, 8] | [9] |
| `1 - 3 - 5 6 7 - -` | 23 | 6 | [6] | [1, 5] | [6] |
| `1 - 3 - 5 6 7 - -` | 23 | 7 | [7] | [1, 6] | [7] |
| `1 - 3 - 5 6 7 - -` | 23 | 8 | [1, 7] | [3, 5] | [1, 7] |
| `1 - 3 - 5 6 7 - -` | 23 | 9 | [3, 6] | [1, 3, 5] | [3, 6] |
| `1 - 3 - 5 6 7 - -` | 23 | 10 | [3, 7] | [1, 3, 6] | [3, 7] |
| `1 - 3 - 5 6 7 - -` | 23 | 11 | [5, 6] | [1, 3, 7] | [5, 6] |
| `1 - 3 - 5 6 7 - -` | 23 | 12 | [5, 7] | [1, 5, 6] | [5, 7] |
| `1 - 3 4 - 6 - 8 -` | 23 | 4 | [4] | [1, 3] | [4] |
| `1 - 3 4 - 6 - 8 -` | 23 | 7 | [3, 4] | [1, 6] | [1, 6] |
| `1 - 3 4 - 6 - 8 -` | 23 | 8 | [8] | [1, 3, 4] | [8] |
| `1 - 3 4 - 6 - 8 -` | 23 | 9 | [1, 8] | [3, 6] | [1, 8] |
| `1 - 3 4 - 6 - 8 -` | 23 | 10 | [4, 6] | [1, 3, 6] | [4, 6] |
| `1 - 3 4 - 6 - 8 -` | 23 | 11 | [3, 8] | [1, 4, 6] | [3, 8] |
| `1 - 3 4 - 6 - 8 -` | 23 | 12 | [4, 8] | [1, 3, 8] | [4, 8] |
| `1 - 3 4 5 - - - 9` | 23 | 4 | [4] | [1, 3] | [4] |
| `1 - 3 4 5 - - - 9` | 23 | 5 | [5] | [1, 4] | [5] |
| `1 - 3 4 5 - - - 9` | 23 | 8 | [3, 5] | [1, 3, 4] | [3, 5] |
| `1 - 3 4 5 - - - 9` | 23 | 9 | [9] | [1, 3, 5] | [9] |
| `1 - 3 4 5 - - - 9` | 23 | 10 | [1, 9] | [1, 4, 5] | [1, 9] |
| `1 - 3 4 5 - - - 9` | 23 | 12 | [3, 9] | [3, 4, 5] | [3, 9] |
| `1 2 - - 5 6 - 8 -` | 23 | 6 | [6] | [1, 5] | [6] |
| `1 2 - - 5 6 - 8 -` | 23 | 7 | [2, 5] | [1, 6] | [1, 6] |
| `1 2 - - 5 6 - 8 -` | 23 | 8 | [8] | [1, 2, 5] | [8] |
| `1 2 - - 5 6 - 8 -` | 23 | 9 | [1, 8] | [1, 2, 6] | [1, 8] |
| `1 2 - - 5 6 - 8 -` | 23 | 11 | [1, 2, 8] | [5, 6] | [5, 6] |
| `1 2 - 4 - - 7 8 -` | 23 | 7 | [7] | [1, 2, 4] | [7] |
| `1 2 - 4 - - 7 8 -` | 23 | 8 | [8] | [1, 7] | [8] |
| `1 2 - 4 - - 7 8 -` | 23 | 9 | [1, 8] | [2, 7] | [1, 8] |
| `1 2 - 4 - - 7 8 -` | 23 | 10 | [2, 8] | [1, 2, 7] | [2, 8] |
| `1 2 - 4 - - 7 8 -` | 23 | 11 | [1, 2, 8] | [4, 7] | [4, 7] |
| `1 2 - 4 - - 7 8 -` | 23 | 12 | [4, 8] | [1, 4, 7] | [4, 8] |
| `1 2 - 4 - 6 - - 9` | 23 | 6 | [6] | [2, 4] | [6] |
| `1 2 - 4 - 6 - - 9` | 23 | 7 | [1, 6] | [1, 2, 4] | [1, 6] |
| `1 2 - 4 - 6 - - 9` | 23 | 9 | [9] | [1, 2, 6] | [9] |
| `1 2 - 4 - 6 - - 9` | 23 | 10 | [1, 9] | [4, 6] | [1, 9] |
| `1 2 - 4 - 6 - - 9` | 23 | 11 | [2, 9] | [1, 4, 6] | [2, 9] |
| `1 2 - 4 - 6 - - 9` | 23 | 12 | [1, 2, 9] | [2, 4, 6] | [1, 2, 9] |
| `1 2 3 - - - 7 - 9` | 23 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 - - - 7 - 9` | 23 | 9 | [9] | [2, 7] | [9] |
| `1 2 3 - - - 7 - 9` | 23 | 10 | [1, 9] | [1, 2, 7] | [1, 9] |
| `1 2 3 - - - 7 - 9` | 23 | 11 | [2, 9] | [1, 3, 7] | [2, 9] |
| `1 2 3 - - - 7 - 9` | 23 | 12 | [3, 9] | [2, 3, 7] | [3, 9] |
| `1 2 3 4 5 - 7 - -` | 23 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 4 5 - 7 - -` | 23 | 4 | [4] | [1, 3] | [4] |
| `1 2 3 4 5 - 7 - -` | 23 | 5 | [5] | [1, 4] | [5] |
| `1 2 3 4 5 - 7 - -` | 23 | 6 | [2, 4] | [1, 2, 3] | [1, 5] |
| `1 2 3 4 5 - 7 - -` | 23 | 7 | [7] | [1, 2, 4] | [7] |
| `1 2 3 4 5 - 7 - -` | 23 | 8 | [1, 7] | [1, 2, 5] | [1, 7] |
| `1 2 3 4 5 - 7 - -` | 23 | 9 | [2, 7], [4, 5] | [1, 3, 5] | [2, 7], [4, 5] |
| `1 2 3 4 5 - 7 - -` | 23 | 10 | [3, 7] | [1, 2, 3, 4] | [3, 7] |
| `1 2 3 4 5 - 7 - -` | 23 | 11 | [4, 7] | [1, 2, 3, 5] | [4, 7] |
| `1 2 3 4 5 - 7 - -` | 23 | 12 | [5, 7] | [1, 2, 4, 5] | [5, 7] |
| `- - 3 4 5 - - - 9` | 24 | 9 | [9] | [4, 5] | [9] |
| `- - 3 4 5 - - - 9` | 24 | 12 | [3, 9] | [3, 4, 5] | [3, 9] |
| `- 2 - - 5 6 - 8 -` | 24 | 8 | [8] | [2, 6] | [8] |
| `- 2 - 4 - 6 - - 9` | 24 | 6 | [6] | [2, 4] | [6] |
| `- 2 3 - - - 7 - 9` | 24 | 9 | [9] | [2, 7] | [9] |
| `- 2 3 - - - 7 - 9` | 24 | 12 | [3, 9] | [2, 3, 7] | [3, 9] |
| `- 2 3 4 5 - 7 - -` | 24 | 5 | [5] | [2, 3] | [5] |
| `- 2 3 4 5 - 7 - -` | 24 | 7 | [7] | [2, 5] | [7] |
| `- 2 3 4 5 - 7 - -` | 24 | 9 | [2, 7] | [2, 3, 4] | [2, 7] |
| `- 2 3 4 5 - 7 - -` | 24 | 10 | [3, 7] | [2, 3, 5] | [3, 7] |
| `- 2 3 4 5 - 7 - -` | 24 | 11 | [4, 7] | [2, 4, 5] | [4, 7] |
| `- 2 3 4 5 - 7 - -` | 24 | 12 | [5, 7] | [3, 4, 5] | [5, 7] |
| `1 - - - 5 - 7 8 -` | 24 | 8 | [8] | [1, 7] | [8] |
| `1 - - - 5 6 - - 9` | 24 | 6 | [6] | [1, 5] | [6] |
| `1 - 3 - - - - 8 9` | 24 | 9 | [9] | [1, 8] | [9] |
| `1 - 3 4 - 6 7 - -` | 24 | 4 | [4] | [1, 3] | [4] |
| `1 - 3 4 - 6 7 - -` | 24 | 7 | [7] | [1, 6] | [7] |
| `1 - 3 4 - 6 7 - -` | 24 | 8 | [1, 7] | [1, 3, 4] | [1, 7] |
| `1 - 3 4 - 6 7 - -` | 24 | 10 | [3, 7] | [1, 3, 6] | [3, 7] |
| `1 - 3 4 - 6 7 - -` | 24 | 11 | [4, 7] | [1, 4, 6] | [4, 7] |
| `1 - 3 4 5 - - 8 -` | 24 | 4 | [4] | [1, 3] | [4] |
| `1 - 3 4 5 - - 8 -` | 24 | 5 | [5] | [1, 4] | [5] |
| `1 - 3 4 5 - - 8 -` | 24 | 8 | [8] | [1, 3, 4] | [8] |
| `1 - 3 4 5 - - 8 -` | 24 | 9 | [1, 8] | [1, 3, 5] | [1, 8] |
| `1 - 3 4 5 - - 8 -` | 24 | 12 | [4, 8] | [3, 4, 5] | [4, 8] |
| `1 2 - - 5 6 7 - -` | 24 | 6 | [6] | [1, 5] | [6] |
| `1 2 - - 5 6 7 - -` | 24 | 7 | [7] | [1, 6] | [7] |
| `1 2 - - 5 6 7 - -` | 24 | 8 | [1, 7] | [1, 2, 5] | [1, 7] |
| `1 2 - - 5 6 7 - -` | 24 | 9 | [2, 7] | [1, 2, 6] | [2, 7] |
| `1 2 - - 5 6 7 - -` | 24 | 12 | [5, 7] | [1, 5, 6] | [5, 7] |
| `1 2 - 4 - 6 - 8 -` | 24 | 6 | [6] | [2, 4] | [6] |
| `1 2 - 4 - 6 - 8 -` | 24 | 7 | [1, 6] | [1, 2, 4] | [1, 6] |
| `1 2 - 4 - 6 - 8 -` | 24 | 8 | [8] | [2, 6] | [8] |
| `1 2 - 4 - 6 - 8 -` | 24 | 9 | [1, 8] | [1, 2, 6] | [1, 8] |
| `1 2 - 4 - 6 - 8 -` | 24 | 10 | [2, 8] | [4, 6] | [2, 8] |
| `1 2 - 4 - 6 - 8 -` | 24 | 11 | [1, 2, 8] | [1, 4, 6] | [1, 2, 8] |
| `1 2 - 4 - 6 - 8 -` | 24 | 12 | [4, 8] | [2, 4, 6] | [4, 8] |
| `1 2 - 4 5 - - - 9` | 24 | 5 | [5] | [1, 4] | [5] |
| `1 2 - 4 5 - - - 9` | 24 | 6 | [2, 4] | [1, 5] | [1, 5] |
| `1 2 - 4 5 - - - 9` | 24 | 7 | [2, 5] | [1, 2, 4] | [2, 5] |
| `1 2 - 4 5 - - - 9` | 24 | 9 | [9] | [4, 5] | [9] |
| `1 2 - 4 5 - - - 9` | 24 | 10 | [1, 9] | [1, 4, 5] | [1, 9] |
| `1 2 - 4 5 - - - 9` | 24 | 11 | [2, 9] | [2, 4, 5] | [2, 9] |
| `1 2 - 4 5 - - - 9` | 24 | 12 | [1, 2, 9] | [1, 2, 4, 5] | [1, 2, 9] |
| `1 2 3 - - - 7 8 -` | 24 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 - - - 7 8 -` | 24 | 8 | [8] | [1, 7] | [8] |
| `1 2 3 - - - 7 8 -` | 24 | 9 | [1, 8] | [2, 7] | [1, 8] |
| `1 2 3 - - - 7 8 -` | 24 | 10 | [2, 8] | [1, 2, 7] | [2, 8] |
| `1 2 3 - - - 7 8 -` | 24 | 11 | [3, 8] | [1, 3, 7] | [3, 8] |
| `1 2 3 - - - 7 8 -` | 24 | 12 | [1, 3, 8] | [2, 3, 7] | [1, 3, 8] |
| `1 2 3 - - 6 - - 9` | 24 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 - - 6 - - 9` | 24 | 6 | [6] | [1, 2, 3] | [6] |
| `1 2 3 - - 6 - - 9` | 24 | 9 | [9] | [1, 2, 6] | [9] |
| `1 2 3 - - 6 - - 9` | 24 | 10 | [1, 9] | [1, 3, 6] | [1, 9] |
| `1 2 3 - - 6 - - 9` | 24 | 11 | [2, 9] | [2, 3, 6] | [2, 9] |
| `1 2 3 - - 6 - - 9` | 24 | 12 | [3, 9] | [1, 2, 3, 6] | [3, 9] |
| `1 2 3 4 5 6 - - -` | 24 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 4 5 6 - - -` | 24 | 4 | [4] | [1, 3] | [4] |
| `1 2 3 4 5 6 - - -` | 24 | 5 | [5] | [2, 3] | [5] |
| `1 2 3 4 5 6 - - -` | 24 | 6 | [6] | [1, 2, 3] | [6] |
| `1 2 3 4 5 6 - - -` | 24 | 7 | [1, 6] | [1, 2, 4] | [1, 6] |
| `1 2 3 4 5 6 - - -` | 24 | 8 | [3, 5] | [1, 2, 5] | [2, 6] |
| `1 2 3 4 5 6 - - -` | 24 | 9 | [3, 6] | [1, 3, 5] | [3, 6] |
| `1 2 3 4 5 6 - - -` | 24 | 10 | [4, 6] | [1, 2, 3, 4] | [4, 6] |
| `1 2 3 4 5 6 - - -` | 24 | 11 | [5, 6] | [1, 2, 3, 5] | [5, 6] |
| `1 2 3 4 5 6 - - -` | 24 | 12 | [1, 5, 6] | [1, 2, 4, 5] | [1, 5, 6] |
| `- - 3 4 - 6 7 - -` | 25 | 7 | [7] | [3, 4] | [7] |
| `- - 3 4 - 6 7 - -` | 25 | 10 | [3, 7] | [4, 6] | [3, 7] |
| `- - 3 4 5 - - 8 -` | 25 | 8 | [8] | [3, 5] | [8] |
| `- - 3 4 5 - - 8 -` | 25 | 12 | [4, 8] | [3, 4, 5] | [4, 8] |
| `- 2 - - 5 6 7 - -` | 25 | 7 | [7] | [2, 5] | [7] |
| `- 2 - 4 - 6 - 8 -` | 25 | 6 | [6] | [2, 4] | [6] |
| `- 2 - 4 - 6 - 8 -` | 25 | 8 | [8] | [2, 6] | [8] |
| `- 2 - 4 - 6 - 8 -` | 25 | 10 | [2, 8] | [4, 6] | [2, 8] |
| `- 2 - 4 - 6 - 8 -` | 25 | 12 | [4, 8] | [2, 4, 6] | [4, 8] |
| `- 2 - 4 5 - - - 9` | 25 | 9 | [9] | [4, 5] | [9] |
| `- 2 - 4 5 - - - 9` | 25 | 11 | [2, 9] | [2, 4, 5] | [2, 9] |
| `- 2 3 - - - 7 8 -` | 25 | 10 | [2, 8] | [3, 7] | [2, 8] |
| `- 2 3 - - 6 - - 9` | 25 | 9 | [9] | [3, 6] | [9] |
| `- 2 3 - - 6 - - 9` | 25 | 11 | [2, 9] | [2, 3, 6] | [2, 9] |
| `- 2 3 4 5 6 - - -` | 25 | 5 | [5] | [2, 3] | [5] |
| `- 2 3 4 5 6 - - -` | 25 | 6 | [6] | [2, 4] | [6] |
| `- 2 3 4 5 6 - - -` | 25 | 7 | [3, 4] | [2, 5] | [2, 5] |
| `- 2 3 4 5 6 - - -` | 25 | 8 | [2, 6] | [3, 5] | [2, 6] |
| `- 2 3 4 5 6 - - -` | 25 | 9 | [3, 6] | [2, 3, 4] | [3, 6] |
| `- 2 3 4 5 6 - - -` | 25 | 10 | [4, 6] | [2, 3, 5] | [4, 6] |
| `- 2 3 4 5 6 - - -` | 25 | 11 | [5, 6] | [2, 4, 5] | [5, 6] |
| `- 2 3 4 5 6 - - -` | 25 | 12 | [2, 4, 6] | [3, 4, 5] | [2, 4, 6] |
| `1 - - - 5 6 - 8 -` | 25 | 6 | [6] | [1, 5] | [6] |
| `1 - - 4 - - 7 8 -` | 25 | 8 | [8] | [1, 7] | [8] |
| `1 - - 4 - 6 - - 9` | 25 | 10 | [1, 9] | [4, 6] | [1, 9] |
| `1 - 3 - - - 7 - 9` | 25 | 10 | [1, 9] | [3, 7] | [1, 9] |
| `1 - 3 4 5 - 7 - -` | 25 | 4 | [4] | [1, 3] | [4] |
| `1 - 3 4 5 - 7 - -` | 25 | 5 | [5] | [1, 4] | [5] |
| `1 - 3 4 5 - 7 - -` | 25 | 7 | [7] | [3, 4] | [7] |
| `1 - 3 4 5 - 7 - -` | 25 | 8 | [1, 7] | [1, 3, 4] | [1, 7] |
| `1 - 3 4 5 - 7 - -` | 25 | 9 | [4, 5] | [1, 3, 5] | [4, 5] |
| `1 - 3 4 5 - 7 - -` | 25 | 10 | [3, 7] | [1, 4, 5] | [3, 7] |
| `1 - 3 4 5 - 7 - -` | 25 | 11 | [4, 7] | [1, 3, 7] | [4, 7] |
| `1 - 3 4 5 - 7 - -` | 25 | 12 | [5, 7] | [3, 4, 5] | [5, 7] |
| `1 2 - - - - - 8 9` | 25 | 9 | [9] | [1, 8] | [9] |
| `1 2 - - - - - 8 9` | 25 | 10 | [1, 9] | [2, 8] | [1, 9] |
| `1 2 - 4 - 6 7 - -` | 25 | 6 | [6] | [2, 4] | [6] |
| `1 2 - 4 - 6 7 - -` | 25 | 7 | [7] | [1, 2, 4] | [7] |
| `1 2 - 4 - 6 7 - -` | 25 | 8 | [2, 6] | [1, 7] | [1, 7] |
| `1 2 - 4 - 6 7 - -` | 25 | 9 | [2, 7] | [1, 2, 6] | [2, 7] |
| `1 2 - 4 - 6 7 - -` | 25 | 10 | [4, 6] | [1, 2, 7] | [4, 6] |
| `1 2 - 4 - 6 7 - -` | 25 | 11 | [4, 7] | [1, 4, 6] | [4, 7] |
| `1 2 - 4 - 6 7 - -` | 25 | 12 | [1, 4, 7] | [2, 4, 6] | [1, 4, 7] |
| `1 2 - 4 5 - - 8 -` | 25 | 5 | [5] | [1, 4] | [5] |
| `1 2 - 4 5 - - 8 -` | 25 | 6 | [2, 4] | [1, 5] | [1, 5] |
| `1 2 - 4 5 - - 8 -` | 25 | 7 | [2, 5] | [1, 2, 4] | [2, 5] |
| `1 2 - 4 5 - - 8 -` | 25 | 8 | [8] | [1, 2, 5] | [8] |
| `1 2 - 4 5 - - 8 -` | 25 | 9 | [1, 8] | [4, 5] | [1, 8] |
| `1 2 - 4 5 - - 8 -` | 25 | 10 | [2, 8] | [1, 4, 5] | [2, 8] |
| `1 2 - 4 5 - - 8 -` | 25 | 11 | [1, 2, 8] | [2, 4, 5] | [1, 2, 8] |
| `1 2 - 4 5 - - 8 -` | 25 | 12 | [4, 8] | [1, 2, 4, 5] | [4, 8] |
| `1 2 3 - - 6 - 8 -` | 25 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 - - 6 - 8 -` | 25 | 6 | [6] | [1, 2, 3] | [6] |
| `1 2 3 - - 6 - 8 -` | 25 | 8 | [8] | [2, 6] | [8] |
| `1 2 3 - - 6 - 8 -` | 25 | 9 | [1, 8] | [1, 2, 6] | [1, 8] |
| `1 2 3 - - 6 - 8 -` | 25 | 10 | [2, 8] | [1, 3, 6] | [2, 8] |
| `1 2 3 - - 6 - 8 -` | 25 | 11 | [3, 8] | [2, 3, 6] | [3, 8] |
| `1 2 3 - - 6 - 8 -` | 25 | 12 | [1, 3, 8] | [1, 2, 3, 6] | [1, 3, 8] |
| `1 2 3 - 5 - - - 9` | 25 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 - 5 - - - 9` | 25 | 5 | [5] | [2, 3] | [5] |
| `1 2 3 - 5 - - - 9` | 25 | 6 | [1, 5] | [1, 2, 3] | [1, 5] |
| `1 2 3 - 5 - - - 9` | 25 | 8 | [3, 5] | [1, 2, 5] | [3, 5] |
| `1 2 3 - 5 - - - 9` | 25 | 9 | [9] | [1, 3, 5] | [9] |
| `1 2 3 - 5 - - - 9` | 25 | 10 | [1, 9] | [2, 3, 5] | [1, 9] |
| `1 2 3 - 5 - - - 9` | 25 | 11 | [2, 9] | [1, 2, 3, 5] | [2, 9] |
| `1 2 3 - 5 - - - 9` | 25 | 12 | [3, 9] | [1, 2, 9] | [3, 9] |
| `- - 3 4 5 - 7 - -` | 26 | 7 | [7] | [3, 4] | [7] |
| `- - 3 4 5 - 7 - -` | 26 | 12 | [5, 7] | [3, 4, 5] | [5, 7] |
| `- 2 - 4 - 6 7 - -` | 26 | 6 | [6] | [2, 4] | [6] |
| `- 2 3 - - 6 - 8 -` | 26 | 8 | [8] | [2, 6] | [8] |
| `- 2 3 - - 6 - 8 -` | 26 | 11 | [3, 8] | [2, 3, 6] | [3, 8] |
| `- 2 3 - 5 - - - 9` | 26 | 5 | [5] | [2, 3] | [5] |
| `1 - - - 5 6 7 - -` | 26 | 6 | [6] | [1, 5] | [6] |
| `1 - - - 5 6 7 - -` | 26 | 7 | [7] | [1, 6] | [7] |
| `1 - - 4 5 - - - 9` | 26 | 5 | [5] | [1, 4] | [5] |
| `1 - - 4 5 - - - 9` | 26 | 9 | [9] | [4, 5] | [9] |
| `1 - - 4 5 - - - 9` | 26 | 10 | [1, 9] | [1, 4, 5] | [1, 9] |
| `1 - 3 - - - 7 8 -` | 26 | 8 | [8] | [1, 7] | [8] |
| `1 - 3 - - 6 - - 9` | 26 | 9 | [9] | [3, 6] | [9] |
| `1 - 3 - - 6 - - 9` | 26 | 10 | [1, 9] | [1, 3, 6] | [1, 9] |
| `1 - 3 4 5 6 - - -` | 26 | 4 | [4] | [1, 3] | [4] |
| `1 - 3 4 5 6 - - -` | 26 | 5 | [5] | [1, 4] | [5] |
| `1 - 3 4 5 6 - - -` | 26 | 6 | [6] | [1, 5] | [6] |
| `1 - 3 4 5 6 - - -` | 26 | 7 | [1, 6] | [3, 4] | [1, 6] |
| `1 - 3 4 5 6 - - -` | 26 | 8 | [3, 5] | [1, 3, 4] | [3, 5] |
| `1 - 3 4 5 6 - - -` | 26 | 9 | [4, 5] | [1, 3, 5] | [3, 6] |
| `1 - 3 4 5 6 - - -` | 26 | 10 | [4, 6] | [1, 4, 5] | [4, 6] |
| `1 - 3 4 5 6 - - -` | 26 | 11 | [5, 6] | [1, 4, 6] | [5, 6] |
| `1 - 3 4 5 6 - - -` | 26 | 12 | [1, 5, 6] | [3, 4, 5] | [1, 5, 6] |
| `1 2 - - - - 7 - 9` | 26 | 9 | [9] | [2, 7] | [9] |
| `1 2 - - - - 7 - 9` | 26 | 10 | [1, 9] | [1, 2, 7] | [1, 9] |
| `1 2 - 4 5 - 7 - -` | 26 | 5 | [5] | [1, 4] | [5] |
| `1 2 - 4 5 - 7 - -` | 26 | 6 | [2, 4] | [1, 5] | [1, 5] |
| `1 2 - 4 5 - 7 - -` | 26 | 7 | [7] | [1, 2, 4] | [7] |
| `1 2 - 4 5 - 7 - -` | 26 | 8 | [1, 7] | [1, 2, 5] | [1, 7] |
| `1 2 - 4 5 - 7 - -` | 26 | 9 | [2, 7] | [4, 5] | [2, 7] |
| `1 2 - 4 5 - 7 - -` | 26 | 10 | [1, 2, 7] | [1, 4, 5] | [1, 2, 7] |
| `1 2 - 4 5 - 7 - -` | 26 | 11 | [4, 7] | [2, 4, 5] | [4, 7] |
| `1 2 - 4 5 - 7 - -` | 26 | 12 | [5, 7] | [1, 2, 4, 5] | [5, 7] |
| `1 2 3 - - 6 7 - -` | 26 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 - - 6 7 - -` | 26 | 6 | [6] | [1, 2, 3] | [6] |
| `1 2 3 - - 6 7 - -` | 26 | 7 | [7] | [1, 6] | [7] |
| `1 2 3 - - 6 7 - -` | 26 | 8 | [1, 7] | [2, 6] | [1, 7] |
| `1 2 3 - - 6 7 - -` | 26 | 9 | [2, 7] | [1, 2, 6] | [2, 7] |
| `1 2 3 - - 6 7 - -` | 26 | 10 | [3, 7] | [1, 3, 6] | [3, 7] |
| `1 2 3 - - 6 7 - -` | 26 | 11 | [1, 3, 7] | [2, 3, 6] | [1, 3, 7] |
| `1 2 3 - 5 - - 8 -` | 26 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 - 5 - - 8 -` | 26 | 5 | [5] | [2, 3] | [5] |
| `1 2 3 - 5 - - 8 -` | 26 | 6 | [1, 5] | [1, 2, 3] | [1, 5] |
| `1 2 3 - 5 - - 8 -` | 26 | 8 | [8] | [1, 2, 5] | [8] |
| `1 2 3 - 5 - - 8 -` | 26 | 9 | [1, 8] | [1, 3, 5] | [1, 8] |
| `1 2 3 - 5 - - 8 -` | 26 | 10 | [2, 8] | [2, 3, 5] | [2, 8] |
| `1 2 3 - 5 - - 8 -` | 26 | 11 | [3, 8] | [1, 2, 3, 5] | [3, 8] |
| `1 2 3 4 - - - - 9` | 26 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 4 - - - - 9` | 26 | 4 | [4] | [1, 3] | [4] |
| `1 2 3 4 - - - - 9` | 26 | 5 | [2, 3] | [1, 4] | [1, 4] |
| `1 2 3 4 - - - - 9` | 26 | 6 | [2, 4] | [1, 2, 3] | [2, 4] |
| `1 2 3 4 - - - - 9` | 26 | 7 | [3, 4] | [1, 2, 4] | [3, 4] |
| `1 2 3 4 - - - - 9` | 26 | 9 | [9] | [2, 3, 4] | [9] |
| `1 2 3 4 - - - - 9` | 26 | 10 | [1, 9] | [1, 2, 3, 4] | [1, 9] |
| `1 2 3 4 - - - - 9` | 26 | 12 | [3, 9] | [1, 2, 9] | [3, 9] |
| `- - - 4 5 - - - 9` | 27 | 9 | [9] | [4, 5] | [9] |
| `- - 3 - - 6 - - 9` | 27 | 9 | [9] | [3, 6] | [9] |
| `- - 3 4 5 6 - - -` | 27 | 9 | [3, 6] | [4, 5] | [3, 6] |
| `- 2 - - - - 7 - 9` | 27 | 9 | [9] | [2, 7] | [9] |
| `- 2 - 4 5 - 7 - -` | 27 | 7 | [7] | [2, 5] | [7] |
| `- 2 - 4 5 - 7 - -` | 27 | 9 | [2, 7] | [4, 5] | [2, 7] |
| `- 2 - 4 5 - 7 - -` | 27 | 11 | [4, 7] | [2, 4, 5] | [4, 7] |
| `- 2 3 - - 6 7 - -` | 27 | 9 | [2, 7] | [3, 6] | [2, 7] |
| `- 2 3 - 5 - - 8 -` | 27 | 5 | [5] | [2, 3] | [5] |
| `- 2 3 - 5 - - 8 -` | 27 | 8 | [8] | [3, 5] | [8] |
| `- 2 3 - 5 - - 8 -` | 27 | 10 | [2, 8] | [2, 3, 5] | [2, 8] |
| `- 2 3 4 - - - - 9` | 27 | 9 | [9] | [2, 3, 4] | [9] |
| `1 - - 4 - 6 7 - -` | 27 | 7 | [7] | [1, 6] | [7] |
| `1 - - 4 5 - - 8 -` | 27 | 5 | [5] | [1, 4] | [5] |
| `1 - - 4 5 - - 8 -` | 27 | 9 | [1, 8] | [4, 5] | [1, 8] |
| `1 - 3 - - 6 - 8 -` | 27 | 9 | [1, 8] | [3, 6] | [1, 8] |
| `1 - 3 - 5 - - - 9` | 27 | 9 | [9] | [1, 3, 5] | [9] |
| `1 2 - - - - 7 8 -` | 27 | 8 | [8] | [1, 7] | [8] |
| `1 2 - - - - 7 8 -` | 27 | 9 | [1, 8] | [2, 7] | [1, 8] |
| `1 2 - - - 6 - - 9` | 27 | 9 | [9] | [1, 2, 6] | [9] |
| `1 2 - 4 5 6 - - -` | 27 | 5 | [5] | [1, 4] | [5] |
| `1 2 - 4 5 6 - - -` | 27 | 6 | [6] | [1, 5] | [6] |
| `1 2 - 4 5 6 - - -` | 27 | 7 | [1, 6] | [1, 2, 4] | [1, 6] |
| `1 2 - 4 5 6 - - -` | 27 | 8 | [2, 6] | [1, 2, 5] | [2, 6] |
| `1 2 - 4 5 6 - - -` | 27 | 9 | [4, 5] | [1, 2, 6] | [4, 5] |
| `1 2 - 4 5 6 - - -` | 27 | 10 | [4, 6] | [1, 4, 5] | [4, 6] |
| `1 2 - 4 5 6 - - -` | 27 | 11 | [5, 6] | [2, 4, 5] | [5, 6] |
| `1 2 - 4 5 6 - - -` | 27 | 12 | [1, 5, 6] | [1, 2, 4, 5], [2, 4, 6] | [1, 5, 6] |
| `1 2 3 - 5 - 7 - -` | 27 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 - 5 - 7 - -` | 27 | 5 | [5] | [2, 3] | [5] |
| `1 2 3 - 5 - 7 - -` | 27 | 6 | [1, 5] | [1, 2, 3] | [1, 5] |
| `1 2 3 - 5 - 7 - -` | 27 | 7 | [7] | [2, 5] | [7] |
| `1 2 3 - 5 - 7 - -` | 27 | 8 | [1, 7] | [1, 2, 5] | [1, 7] |
| `1 2 3 - 5 - 7 - -` | 27 | 9 | [2, 7] | [1, 3, 5] | [2, 7] |
| `1 2 3 - 5 - 7 - -` | 27 | 10 | [3, 7] | [2, 3, 5] | [3, 7] |
| `1 2 3 - 5 - 7 - -` | 27 | 11 | [1, 3, 7] | [1, 2, 3, 5] | [1, 3, 7] |
| `1 2 3 - 5 - 7 - -` | 27 | 12 | [5, 7] | [2, 3, 7] | [5, 7] |
| `1 2 3 4 - - - 8 -` | 27 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 4 - - - 8 -` | 27 | 4 | [4] | [1, 3] | [4] |
| `1 2 3 4 - - - 8 -` | 27 | 5 | [2, 3] | [1, 4] | [1, 4] |
| `1 2 3 4 - - - 8 -` | 27 | 6 | [2, 4] | [1, 2, 3] | [2, 4] |
| `1 2 3 4 - - - 8 -` | 27 | 7 | [3, 4] | [1, 2, 4] | [3, 4] |
| `1 2 3 4 - - - 8 -` | 27 | 8 | [8] | [1, 3, 4] | [8] |
| `1 2 3 4 - - - 8 -` | 27 | 9 | [1, 8] | [2, 3, 4] | [1, 8] |
| `1 2 3 4 - - - 8 -` | 27 | 10 | [2, 8] | [1, 2, 3, 4] | [2, 8] |
| `1 2 3 4 - - - 8 -` | 27 | 11 | [3, 8] | [1, 2, 8] | [3, 8] |
| `1 2 3 4 - - - 8 -` | 27 | 12 | [4, 8] | [1, 3, 8] | [4, 8] |
| `- 2 - 4 5 6 - - -` | 28 | 6 | [6] | [2, 4] | [6] |
| `- 2 - 4 5 6 - - -` | 28 | 11 | [5, 6] | [2, 4, 5] | [5, 6] |
| `- 2 3 - 5 - 7 - -` | 28 | 5 | [5] | [2, 3] | [5] |
| `- 2 3 - 5 - 7 - -` | 28 | 7 | [7] | [2, 5] | [7] |
| `- 2 3 - 5 - 7 - -` | 28 | 10 | [3, 7] | [2, 3, 5] | [3, 7] |
| `- 2 3 - 5 - 7 - -` | 28 | 12 | [5, 7] | [2, 3, 7] | [5, 7] |
| `1 - - 4 5 - 7 - -` | 28 | 5 | [5] | [1, 4] | [5] |
| `1 - 3 - - 6 7 - -` | 28 | 7 | [7] | [1, 6] | [7] |
| `1 - 3 - 5 - - 8 -` | 28 | 8 | [8] | [3, 5] | [8] |
| `1 - 3 - 5 - - 8 -` | 28 | 9 | [1, 8] | [1, 3, 5] | [1, 8] |
| `1 - 3 4 - - - - 9` | 28 | 4 | [4] | [1, 3] | [4] |
| `1 2 - - - 6 - 8 -` | 28 | 8 | [8] | [2, 6] | [8] |
| `1 2 - - - 6 - 8 -` | 28 | 9 | [1, 8] | [1, 2, 6] | [1, 8] |
| `1 2 3 - 5 6 - - -` | 28 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 - 5 6 - - -` | 28 | 5 | [5] | [2, 3] | [5] |
| `1 2 3 - 5 6 - - -` | 28 | 6 | [6] | [1, 2, 3] | [6] |
| `1 2 3 - 5 6 - - -` | 28 | 7 | [2, 5] | [1, 6] | [1, 6] |
| `1 2 3 - 5 6 - - -` | 28 | 8 | [2, 6] | [1, 2, 5] | [2, 6] |
| `1 2 3 - 5 6 - - -` | 28 | 9 | [3, 6] | [1, 3, 5] | [3, 6] |
| `1 2 3 - 5 6 - - -` | 28 | 10 | [1, 3, 6] | [2, 3, 5] | [1, 3, 6] |
| `1 2 3 - 5 6 - - -` | 28 | 11 | [5, 6] | [1, 2, 3, 5], [2, 3, 6] | [5, 6] |
| `1 2 3 - 5 6 - - -` | 28 | 12 | [1, 5, 6] | [1, 2, 3, 6] | [1, 5, 6] |
| `1 2 3 4 - - 7 - -` | 28 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 4 - - 7 - -` | 28 | 4 | [4] | [1, 3] | [4] |
| `1 2 3 4 - - 7 - -` | 28 | 5 | [2, 3] | [1, 4] | [1, 4] |
| `1 2 3 4 - - 7 - -` | 28 | 6 | [2, 4] | [1, 2, 3] | [2, 4] |
| `1 2 3 4 - - 7 - -` | 28 | 7 | [7] | [1, 2, 4] | [7] |
| `1 2 3 4 - - 7 - -` | 28 | 8 | [1, 7] | [1, 3, 4] | [1, 7] |
| `1 2 3 4 - - 7 - -` | 28 | 9 | [2, 7] | [2, 3, 4] | [2, 7] |
| `1 2 3 4 - - 7 - -` | 28 | 10 | [3, 7] | [1, 2, 3, 4] | [3, 7] |
| `1 2 3 4 - - 7 - -` | 28 | 11 | [4, 7] | [1, 3, 7] | [4, 7] |
| `1 2 3 4 - - 7 - -` | 28 | 12 | [1, 4, 7] | [2, 3, 7] | [1, 4, 7] |
| `- - 3 - 5 - - 8 -` | 29 | 8 | [8] | [3, 5] | [8] |
| `- 2 - - - 6 - 8 -` | 29 | 8 | [8] | [2, 6] | [8] |
| `- 2 3 - 5 6 - - -` | 29 | 5 | [5] | [2, 3] | [5] |
| `- 2 3 - 5 6 - - -` | 29 | 8 | [2, 6] | [3, 5] | [2, 6] |
| `- 2 3 - 5 6 - - -` | 29 | 11 | [5, 6] | [2, 3, 6] | [5, 6] |
| `- 2 3 4 - - 7 - -` | 29 | 7 | [7] | [3, 4] | [7] |
| `- 2 3 4 - - 7 - -` | 29 | 9 | [2, 7] | [2, 3, 4] | [2, 7] |
| `1 - - 4 5 6 - - -` | 29 | 5 | [5] | [1, 4] | [5] |
| `1 - - 4 5 6 - - -` | 29 | 6 | [6] | [1, 5] | [6] |
| `1 - 3 - 5 - 7 - -` | 29 | 8 | [1, 7] | [3, 5] | [1, 7] |
| `1 - 3 4 - - - 8 -` | 29 | 4 | [4] | [1, 3] | [4] |
| `1 - 3 4 - - - 8 -` | 29 | 8 | [8] | [1, 3, 4] | [8] |
| `1 2 - - - 6 7 - -` | 29 | 7 | [7] | [1, 6] | [7] |
| `1 2 - - - 6 7 - -` | 29 | 8 | [1, 7] | [2, 6] | [1, 7] |
| `1 2 - - 5 - - 8 -` | 29 | 8 | [8] | [1, 2, 5] | [8] |
| `1 2 3 4 - 6 - - -` | 29 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 4 - 6 - - -` | 29 | 4 | [4] | [1, 3] | [4] |
| `1 2 3 4 - 6 - - -` | 29 | 5 | [2, 3] | [1, 4] | [1, 4] |
| `1 2 3 4 - 6 - - -` | 29 | 6 | [6] | [1, 2, 3] | [6] |
| `1 2 3 4 - 6 - - -` | 29 | 7 | [1, 6] | [1, 2, 4] | [1, 6] |
| `1 2 3 4 - 6 - - -` | 29 | 8 | [2, 6] | [1, 3, 4] | [2, 6] |
| `1 2 3 4 - 6 - - -` | 29 | 9 | [3, 6] | [2, 3, 4] | [3, 6] |
| `1 2 3 4 - 6 - - -` | 29 | 10 | [4, 6] | [1, 2, 3, 4] | [4, 6] |
| `1 2 3 4 - 6 - - -` | 29 | 11 | [1, 4, 6] | [2, 3, 6] | [1, 4, 6] |
| `- 2 3 4 - 6 - - -` | 30 | 6 | [6] | [2, 4] | [6] |
| `- 2 3 4 - 6 - - -` | 30 | 9 | [3, 6] | [2, 3, 4] | [3, 6] |
| `1 - 3 - 5 6 - - -` | 30 | 6 | [6] | [1, 5] | [6] |
| `1 - 3 4 - - 7 - -` | 30 | 4 | [4] | [1, 3] | [4] |
| `1 - 3 4 - - 7 - -` | 30 | 7 | [7] | [3, 4] | [7] |
| `1 - 3 4 - - 7 - -` | 30 | 8 | [1, 7] | [1, 3, 4] | [1, 7] |
| `1 2 - - 5 - 7 - -` | 30 | 7 | [7] | [2, 5] | [7] |
| `1 2 - - 5 - 7 - -` | 30 | 8 | [1, 7] | [1, 2, 5] | [1, 7] |
| `1 2 3 - - - - - 9` | 30 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 4 5 - - - -` | 30 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 4 5 - - - -` | 30 | 4 | [4] | [1, 3] | [4] |
| `1 2 3 4 5 - - - -` | 30 | 5 | [5] | [2, 3] | [5] |
| `1 2 3 4 5 - - - -` | 30 | 6 | [1, 5] | [1, 2, 3] | [1, 5] |
| `1 2 3 4 5 - - - -` | 30 | 7 | [3, 4] | [1, 2, 4] | [2, 5] |
| `1 2 3 4 5 - - - -` | 30 | 8 | [3, 5] | [1, 3, 4] | [3, 5] |
| `1 2 3 4 5 - - - -` | 30 | 9 | [4, 5] | [2, 3, 4] | [4, 5] |
| `1 2 3 4 5 - - - -` | 30 | 10 | [1, 4, 5] | [1, 2, 3, 4], [2, 3, 5] | [1, 4, 5] |
| `- - 3 4 - - 7 - -` | 31 | 7 | [7] | [3, 4] | [7] |
| `- 2 - - 5 - 7 - -` | 31 | 7 | [7] | [2, 5] | [7] |
| `- 2 3 4 5 - - - -` | 31 | 5 | [5] | [2, 3] | [5] |
| `- 2 3 4 5 - - - -` | 31 | 7 | [2, 5] | [3, 4] | [2, 5] |
| `- 2 3 4 5 - - - -` | 31 | 9 | [4, 5] | [2, 3, 4] | [4, 5] |
| `1 - 3 4 - 6 - - -` | 31 | 4 | [4] | [1, 3] | [4] |
| `1 - 3 4 - 6 - - -` | 31 | 7 | [1, 6] | [3, 4] | [1, 6] |
| `1 2 - - 5 6 - - -` | 31 | 6 | [6] | [1, 5] | [6] |
| `1 2 - - 5 6 - - -` | 31 | 7 | [1, 6] | [2, 5] | [1, 6] |
| `1 2 - 4 - - 7 - -` | 31 | 7 | [7] | [1, 2, 4] | [7] |
| `1 2 3 - - - - 8 -` | 31 | 3 | [3] | [1, 2] | [3] |
| `1 - 3 4 5 - - - -` | 32 | 4 | [4] | [1, 3] | [4] |
| `1 - 3 4 5 - - - -` | 32 | 5 | [5] | [1, 4] | [5] |
| `1 2 - 4 - 6 - - -` | 32 | 6 | [6] | [2, 4] | [6] |
| `1 2 - 4 - 6 - - -` | 32 | 7 | [1, 6] | [1, 2, 4] | [1, 6] |
| `1 2 3 - - - 7 - -` | 32 | 3 | [3] | [1, 2] | [3] |
| `- 2 - 4 - 6 - - -` | 33 | 6 | [6] | [2, 4] | [6] |
| `1 2 - 4 5 - - - -` | 33 | 5 | [5] | [1, 4] | [5] |
| `1 2 - 4 5 - - - -` | 33 | 6 | [1, 5] | [2, 4] | [1, 5] |
| `1 2 3 - - 6 - - -` | 33 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 - - 6 - - -` | 33 | 6 | [6] | [1, 2, 3] | [6] |
| `1 2 3 - 5 - - - -` | 34 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 - 5 - - - -` | 34 | 5 | [5] | [2, 3] | [5] |
| `1 2 3 - 5 - - - -` | 34 | 6 | [1, 5] | [1, 2, 3] | [1, 5] |
| `- 2 3 - 5 - - - -` | 35 | 5 | [5] | [2, 3] | [5] |
| `1 2 3 4 - - - - -` | 35 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 4 - - - - -` | 35 | 4 | [4] | [1, 3] | [4] |
| `1 2 3 4 - - - - -` | 35 | 5 | [1, 4] | [2, 3] | [1, 4] |


### Exhibit F: Optimal Moves for Flipped Scoring System

Note: Only showing results where there is a difference between the optimal strategy and the worst strategy

In [15]:
def exhibit_f(results: dict) -> Table:
    """returns a table for optimal moves of the flipped count scoring system"""
    exhibit_table = []
    results_subtable = results['flipped_count_move_table']
    for record in results_subtable:
        if record['Max Move'] == record['Min Move']:
            continue
        exhibit_table.append({
            'Board (Remaining)': f'`{record["Board"]}`',
            'Board Sum': record['Sum'],
            'Roll': record['Roll'],
            'Max Move': record['Max Move'],
            'Min Move': record['Min Move'],
            'Highest Tile Move': record['Highest Tile Move'],
        })
    # sort by board sum ascending
    exhibit_table.sort(key=lambda record: (record['Board Sum'], record['Board (Remaining)'], record['Roll']))
    return exhibit_table


display_table(exhibit_f(full_results))

| Board (Remaining) | Board Sum | Roll | Max Move | Min Move | Highest Tile Move |
| --- | --- | --- | --- | --- | --- |
| `1 2 3 4 5 6 7 8 9` | 0 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 4 5 6 7 8 9` | 0 | 4 | [4] | [1, 3] | [4] |
| `1 2 3 4 5 6 7 8 9` | 0 | 5 | [5] | [2, 3] | [5] |
| `1 2 3 4 5 6 7 8 9` | 0 | 6 | [6] | [1, 2, 3] | [6] |
| `1 2 3 4 5 6 7 8 9` | 0 | 7 | [7] | [3, 4] | [7] |
| `1 2 3 4 5 6 7 8 9` | 0 | 8 | [8] | [1, 2, 5] | [8] |
| `1 2 3 4 5 6 7 8 9` | 0 | 9 | [9] | [1, 2, 6] | [9] |
| `1 2 3 4 5 6 7 8 9` | 0 | 10 | [1, 9] | [1, 2, 3, 4] | [1, 9] |
| `1 2 3 4 5 6 7 8 9` | 0 | 11 | [3, 8] | [1, 4, 6] | [2, 9] |
| `1 2 3 4 5 6 7 8 9` | 0 | 12 | [3, 9] | [3, 4, 5] | [3, 9] |
| `- 2 3 4 5 6 7 8 9` | 1 | 5 | [5] | [2, 3] | [5] |
| `- 2 3 4 5 6 7 8 9` | 1 | 6 | [6] | [2, 4] | [6] |
| `- 2 3 4 5 6 7 8 9` | 1 | 7 | [7] | [2, 5] | [7] |
| `- 2 3 4 5 6 7 8 9` | 1 | 8 | [8] | [2, 6] | [8] |
| `- 2 3 4 5 6 7 8 9` | 1 | 9 | [9] | [2, 3, 4] | [9] |
| `- 2 3 4 5 6 7 8 9` | 1 | 10 | [2, 8] | [4, 6] | [2, 8] |
| `- 2 3 4 5 6 7 8 9` | 1 | 11 | [2, 9] | [2, 3, 6] | [2, 9] |
| `- 2 3 4 5 6 7 8 9` | 1 | 12 | [5, 7] | [2, 3, 7] | [3, 9] |
| `1 - 3 4 5 6 7 8 9` | 2 | 4 | [4] | [1, 3] | [4] |
| `1 - 3 4 5 6 7 8 9` | 2 | 5 | [5] | [1, 4] | [5] |
| `1 - 3 4 5 6 7 8 9` | 2 | 6 | [6] | [1, 5] | [6] |
| `1 - 3 4 5 6 7 8 9` | 2 | 7 | [7] | [1, 6] | [7] |
| `1 - 3 4 5 6 7 8 9` | 2 | 8 | [8] | [1, 3, 4] | [8] |
| `1 - 3 4 5 6 7 8 9` | 2 | 9 | [9] | [4, 5] | [9] |
| `1 - 3 4 5 6 7 8 9` | 2 | 10 | [1, 9] | [1, 3, 6] | [1, 9] |
| `1 - 3 4 5 6 7 8 9` | 2 | 11 | [4, 7] | [5, 6] | [3, 8] |
| `1 - 3 4 5 6 7 8 9` | 2 | 12 | [4, 8] | [1, 4, 7] | [3, 9] |
| `- - 3 4 5 6 7 8 9` | 3 | 7 | [7] | [3, 4] | [7] |
| `- - 3 4 5 6 7 8 9` | 3 | 8 | [8] | [3, 5] | [8] |
| `- - 3 4 5 6 7 8 9` | 3 | 9 | [9] | [3, 6] | [9] |
| `- - 3 4 5 6 7 8 9` | 3 | 10 | [4, 6] | [3, 7] | [3, 7] |
| `- - 3 4 5 6 7 8 9` | 3 | 11 | [3, 8] | [4, 7] | [3, 8] |
| `- - 3 4 5 6 7 8 9` | 3 | 12 | [3, 9] | [3, 4, 5] | [3, 9] |
| `1 2 - 4 5 6 7 8 9` | 3 | 5 | [5] | [1, 4] | [5] |
| `1 2 - 4 5 6 7 8 9` | 3 | 6 | [6] | [2, 4] | [6] |
| `1 2 - 4 5 6 7 8 9` | 3 | 7 | [7] | [1, 2, 4] | [7] |
| `1 2 - 4 5 6 7 8 9` | 3 | 8 | [8] | [2, 6] | [8] |
| `1 2 - 4 5 6 7 8 9` | 3 | 9 | [9] | [4, 5] | [9] |
| `1 2 - 4 5 6 7 8 9` | 3 | 10 | [1, 9] | [1, 2, 7] | [1, 9] |
| `1 2 - 4 5 6 7 8 9` | 3 | 11 | [5, 6] | [2, 4, 5] | [2, 9] |
| `1 2 - 4 5 6 7 8 9` | 3 | 12 | [5, 7] | [2, 4, 6] | [4, 8] |
| `- 2 - 4 5 6 7 8 9` | 4 | 6 | [6] | [2, 4] | [6] |
| `- 2 - 4 5 6 7 8 9` | 4 | 7 | [7] | [2, 5] | [7] |
| `- 2 - 4 5 6 7 8 9` | 4 | 8 | [8] | [2, 6] | [8] |
| `- 2 - 4 5 6 7 8 9` | 4 | 9 | [9] | [2, 7] | [9] |
| `- 2 - 4 5 6 7 8 9` | 4 | 10 | [2, 8] | [4, 6] | [2, 8] |
| `- 2 - 4 5 6 7 8 9` | 4 | 11 | [2, 9] | [2, 4, 5] | [2, 9] |
| `- 2 - 4 5 6 7 8 9` | 4 | 12 | [4, 8] | [5, 7] | [4, 8] |
| `1 2 3 - 5 6 7 8 9` | 4 | 5 | [5] | [2, 3] | [5] |
| `1 2 3 - 5 6 7 8 9` | 4 | 6 | [6] | [1, 2, 3] | [6] |
| `1 2 3 - 5 6 7 8 9` | 4 | 7 | [7] | [1, 6] | [7] |
| `1 2 3 - 5 6 7 8 9` | 4 | 8 | [8] | [3, 5] | [8] |
| `1 2 3 - 5 6 7 8 9` | 4 | 9 | [9] | [1, 3, 5] | [9] |
| `1 2 3 - 5 6 7 8 9` | 4 | 10 | [2, 8] | [1, 2, 7] | [1, 9] |
| `1 2 3 - 5 6 7 8 9` | 4 | 11 | [2, 9] | [2, 3, 6] | [2, 9] |
| `1 2 3 - 5 6 7 8 9` | 4 | 12 | [5, 7] | [1, 3, 8] | [3, 9] |
| `- 2 3 - 5 6 7 8 9` | 5 | 5 | [5] | [2, 3] | [5] |
| `- 2 3 - 5 6 7 8 9` | 5 | 7 | [7] | [2, 5] | [7] |
| `- 2 3 - 5 6 7 8 9` | 5 | 8 | [8] | [3, 5] | [8] |
| `- 2 3 - 5 6 7 8 9` | 5 | 9 | [9] | [2, 7] | [9] |
| `- 2 3 - 5 6 7 8 9` | 5 | 10 | [2, 8] | [2, 3, 5] | [2, 8] |
| `- 2 3 - 5 6 7 8 9` | 5 | 11 | [2, 9] | [3, 8] | [2, 9] |
| `- 2 3 - 5 6 7 8 9` | 5 | 12 | [3, 9] | [5, 7] | [3, 9] |
| `1 - - 4 5 6 7 8 9` | 5 | 5 | [5] | [1, 4] | [5] |
| `1 - - 4 5 6 7 8 9` | 5 | 7 | [7] | [1, 6] | [7] |
| `1 - - 4 5 6 7 8 9` | 5 | 8 | [8] | [1, 7] | [8] |
| `1 - - 4 5 6 7 8 9` | 5 | 9 | [9] | [4, 5] | [9] |
| `1 - - 4 5 6 7 8 9` | 5 | 10 | [1, 9] | [4, 6] | [1, 9] |
| `1 - - 4 5 6 7 8 9` | 5 | 11 | [4, 7] | [5, 6] | [4, 7] |
| `1 - - 4 5 6 7 8 9` | 5 | 12 | [5, 7] | [1, 4, 7] | [4, 8] |
| `1 2 3 4 - 6 7 8 9` | 5 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 4 - 6 7 8 9` | 5 | 4 | [4] | [1, 3] | [4] |
| `1 2 3 4 - 6 7 8 9` | 5 | 5 | [2, 3] | [1, 4] | [1, 4] |
| `1 2 3 4 - 6 7 8 9` | 5 | 6 | [6] | [2, 4] | [6] |
| `1 2 3 4 - 6 7 8 9` | 5 | 7 | [7] | [3, 4] | [7] |
| `1 2 3 4 - 6 7 8 9` | 5 | 8 | [8] | [2, 6] | [8] |
| `1 2 3 4 - 6 7 8 9` | 5 | 9 | [9] | [2, 3, 4] | [9] |
| `1 2 3 4 - 6 7 8 9` | 5 | 10 | [1, 9] | [1, 2, 3, 4] | [1, 9] |
| `1 2 3 4 - 6 7 8 9` | 5 | 11 | [3, 8] | [1, 3, 7] | [2, 9] |
| `1 2 3 4 - 6 7 8 9` | 5 | 12 | [3, 9] | [2, 4, 6] | [3, 9] |
| `- - - 4 5 6 7 8 9` | 6 | 9 | [9] | [4, 5] | [9] |
| `- - - 4 5 6 7 8 9` | 6 | 11 | [5, 6] | [4, 7] | [4, 7] |
| `- - - 4 5 6 7 8 9` | 6 | 12 | [4, 8] | [5, 7] | [4, 8] |
| `- 2 3 4 - 6 7 8 9` | 6 | 6 | [6] | [2, 4] | [6] |
| `- 2 3 4 - 6 7 8 9` | 6 | 7 | [7] | [3, 4] | [7] |
| `- 2 3 4 - 6 7 8 9` | 6 | 8 | [8] | [2, 6] | [8] |
| `- 2 3 4 - 6 7 8 9` | 6 | 9 | [9] | [2, 3, 4] | [9] |
| `- 2 3 4 - 6 7 8 9` | 6 | 10 | [4, 6] | [3, 7] | [2, 8] |
| `- 2 3 4 - 6 7 8 9` | 6 | 11 | [2, 9] | [4, 7] | [2, 9] |
| `- 2 3 4 - 6 7 8 9` | 6 | 12 | [4, 8] | [2, 3, 7] | [3, 9] |
| `1 - 3 - 5 6 7 8 9` | 6 | 6 | [6] | [1, 5] | [6] |
| `1 - 3 - 5 6 7 8 9` | 6 | 7 | [7] | [1, 6] | [7] |
| `1 - 3 - 5 6 7 8 9` | 6 | 8 | [8] | [1, 7] | [8] |
| `1 - 3 - 5 6 7 8 9` | 6 | 9 | [9] | [3, 6] | [9] |
| `1 - 3 - 5 6 7 8 9` | 6 | 10 | [1, 9] | [1, 3, 6] | [1, 9] |
| `1 - 3 - 5 6 7 8 9` | 6 | 11 | [3, 8] | [5, 6] | [3, 8] |
| `1 - 3 - 5 6 7 8 9` | 6 | 12 | [5, 7] | [3, 9] | [3, 9] |
| `1 2 3 4 5 - 7 8 9` | 6 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 4 5 - 7 8 9` | 6 | 4 | [4] | [1, 3] | [4] |
| `1 2 3 4 5 - 7 8 9` | 6 | 5 | [5] | [1, 4] | [5] |
| `1 2 3 4 5 - 7 8 9` | 6 | 6 | [1, 5] | [1, 2, 3] | [1, 5] |
| `1 2 3 4 5 - 7 8 9` | 6 | 7 | [7] | [2, 5] | [7] |
| `1 2 3 4 5 - 7 8 9` | 6 | 8 | [8] | [1, 2, 5] | [8] |
| `1 2 3 4 5 - 7 8 9` | 6 | 9 | [9] | [2, 3, 4] | [9] |
| `1 2 3 4 5 - 7 8 9` | 6 | 10 | [1, 9] | [1, 2, 7] | [1, 9] |
| `1 2 3 4 5 - 7 8 9` | 6 | 11 | [3, 8] | [2, 4, 5] | [2, 9] |
| `1 2 3 4 5 - 7 8 9` | 6 | 12 | [3, 9] | [2, 3, 7] | [3, 9] |
| `- - 3 - 5 6 7 8 9` | 7 | 8 | [8] | [3, 5] | [8] |
| `- - 3 - 5 6 7 8 9` | 7 | 9 | [9] | [3, 6] | [9] |
| `- - 3 - 5 6 7 8 9` | 7 | 11 | [5, 6] | [3, 8] | [3, 8] |
| `- - 3 - 5 6 7 8 9` | 7 | 12 | [3, 9] | [5, 7] | [3, 9] |
| `- 2 3 4 5 - 7 8 9` | 7 | 5 | [5] | [2, 3] | [5] |
| `- 2 3 4 5 - 7 8 9` | 7 | 7 | [7] | [2, 5] | [7] |
| `- 2 3 4 5 - 7 8 9` | 7 | 8 | [8] | [3, 5] | [8] |
| `- 2 3 4 5 - 7 8 9` | 7 | 9 | [9] | [2, 7] | [9] |
| `- 2 3 4 5 - 7 8 9` | 7 | 10 | [3, 7] | [2, 3, 5] | [2, 8] |
| `- 2 3 4 5 - 7 8 9` | 7 | 11 | [2, 9] | [3, 8] | [2, 9] |
| `- 2 3 4 5 - 7 8 9` | 7 | 12 | [5, 7] | [2, 3, 7] | [3, 9] |
| `1 - 3 4 - 6 7 8 9` | 7 | 7 | [7] | [3, 4] | [7] |
| `1 - 3 4 - 6 7 8 9` | 7 | 8 | [8] | [1, 3, 4] | [8] |
| `1 - 3 4 - 6 7 8 9` | 7 | 9 | [9] | [1, 8] | [9] |
| `1 - 3 4 - 6 7 8 9` | 7 | 10 | [3, 7] | [4, 6] | [1, 9] |
| `1 - 3 4 - 6 7 8 9` | 7 | 11 | [3, 8] | [1, 3, 7] | [3, 8] |
| `1 - 3 4 - 6 7 8 9` | 7 | 12 | [3, 9] | [1, 4, 7] | [3, 9] |
| `1 2 - - 5 6 7 8 9` | 7 | 6 | [6] | [1, 5] | [6] |
| `1 2 - - 5 6 7 8 9` | 7 | 7 | [7] | [2, 5] | [7] |
| `1 2 - - 5 6 7 8 9` | 7 | 8 | [8] | [2, 6] | [8] |
| `1 2 - - 5 6 7 8 9` | 7 | 9 | [9] | [1, 8] | [9] |
| `1 2 - - 5 6 7 8 9` | 7 | 10 | [1, 9] | [1, 2, 7] | [1, 9] |
| `1 2 - - 5 6 7 8 9` | 7 | 11 | [1, 2, 8] | [5, 6] | [2, 9] |
| `1 2 3 4 5 6 - 8 9` | 7 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 4 5 6 - 8 9` | 7 | 4 | [4] | [1, 3] | [4] |
| `1 2 3 4 5 6 - 8 9` | 7 | 5 | [5] | [2, 3] | [5] |
| `1 2 3 4 5 6 - 8 9` | 7 | 6 | [6] | [1, 2, 3] | [6] |
| `1 2 3 4 5 6 - 8 9` | 7 | 7 | [1, 6] | [1, 2, 4] | [1, 6] |
| `1 2 3 4 5 6 - 8 9` | 7 | 8 | [8] | [2, 6] | [8] |
| `1 2 3 4 5 6 - 8 9` | 7 | 9 | [9] | [1, 3, 5] | [9] |
| `1 2 3 4 5 6 - 8 9` | 7 | 10 | [1, 9] | [1, 4, 5] | [1, 9] |
| `1 2 3 4 5 6 - 8 9` | 7 | 11 | [3, 8] | [2, 3, 6] | [2, 9] |
| `1 2 3 4 5 6 - 8 9` | 7 | 12 | [3, 9] | [1, 2, 3, 6] | [3, 9] |
| `- - 3 4 - 6 7 8 9` | 8 | 7 | [7] | [3, 4] | [7] |
| `- - 3 4 - 6 7 8 9` | 8 | 9 | [9] | [3, 6] | [9] |
| `- - 3 4 - 6 7 8 9` | 8 | 10 | [4, 6] | [3, 7] | [3, 7] |
| `- - 3 4 - 6 7 8 9` | 8 | 11 | [3, 8] | [4, 7] | [3, 8] |
| `- - 3 4 - 6 7 8 9` | 8 | 12 | [3, 9] | [4, 8] | [3, 9] |
| `- 2 - - 5 6 7 8 9` | 8 | 7 | [7] | [2, 5] | [7] |
| `- 2 - - 5 6 7 8 9` | 8 | 9 | [9] | [2, 7] | [9] |
| `- 2 3 4 5 6 - 8 9` | 8 | 5 | [5] | [2, 3] | [5] |
| `- 2 3 4 5 6 - 8 9` | 8 | 6 | [6] | [2, 4] | [6] |
| `- 2 3 4 5 6 - 8 9` | 8 | 7 | [2, 5] | [3, 4] | [2, 5] |
| `- 2 3 4 5 6 - 8 9` | 8 | 8 | [8] | [3, 5] | [8] |
| `- 2 3 4 5 6 - 8 9` | 8 | 9 | [9] | [4, 5] | [9] |
| `- 2 3 4 5 6 - 8 9` | 8 | 10 | [2, 8] | [4, 6] | [2, 8] |
| `- 2 3 4 5 6 - 8 9` | 8 | 11 | [2, 9] | [2, 3, 6] | [2, 9] |
| `- 2 3 4 5 6 - 8 9` | 8 | 12 | [3, 9] | [2, 4, 6] | [3, 9] |
| `1 - 3 4 5 - 7 8 9` | 8 | 4 | [4] | [1, 3] | [4] |
| `1 - 3 4 5 - 7 8 9` | 8 | 7 | [7] | [3, 4] | [7] |
| `1 - 3 4 5 - 7 8 9` | 8 | 8 | [8] | [1, 7] | [8] |
| `1 - 3 4 5 - 7 8 9` | 8 | 9 | [9] | [4, 5] | [9] |
| `1 - 3 4 5 - 7 8 9` | 8 | 10 | [1, 9] | [3, 7] | [1, 9] |
| `1 - 3 4 5 - 7 8 9` | 8 | 11 | [3, 8] | [1, 3, 7] | [3, 8] |
| `1 - 3 4 5 - 7 8 9` | 8 | 12 | [4, 8] | [1, 4, 7] | [3, 9] |
| `1 2 - 4 - 6 7 8 9` | 8 | 6 | [6] | [2, 4] | [6] |
| `1 2 - 4 - 6 7 8 9` | 8 | 7 | [7] | [1, 2, 4] | [7] |
| `1 2 - 4 - 6 7 8 9` | 8 | 8 | [8] | [1, 7] | [8] |
| `1 2 - 4 - 6 7 8 9` | 8 | 9 | [9] | [1, 2, 6] | [9] |
| `1 2 - 4 - 6 7 8 9` | 8 | 10 | [2, 8] | [4, 6] | [1, 9] |
| `1 2 - 4 - 6 7 8 9` | 8 | 11 | [2, 9] | [4, 7] | [2, 9] |
| `1 2 - 4 - 6 7 8 9` | 8 | 12 | [4, 8] | [2, 4, 6] | [4, 8] |
| `1 2 3 4 5 6 7 - 9` | 8 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 4 5 6 7 - 9` | 8 | 4 | [4] | [1, 3] | [4] |
| `1 2 3 4 5 6 7 - 9` | 8 | 5 | [5] | [1, 4] | [5] |
| `1 2 3 4 5 6 7 - 9` | 8 | 6 | [6] | [1, 2, 3] | [6] |
| `1 2 3 4 5 6 7 - 9` | 8 | 7 | [7] | [3, 4] | [7] |
| `1 2 3 4 5 6 7 - 9` | 8 | 8 | [1, 7] | [1, 2, 5] | [1, 7] |
| `1 2 3 4 5 6 7 - 9` | 8 | 9 | [9] | [1, 2, 6] | [9] |
| `1 2 3 4 5 6 7 - 9` | 8 | 10 | [1, 9] | [1, 3, 6] | [1, 9] |
| `1 2 3 4 5 6 7 - 9` | 8 | 11 | [2, 9] | [1, 4, 6] | [2, 9] |
| `1 2 3 4 5 6 7 - 9` | 8 | 12 | [3, 9] | [1, 2, 3, 6] | [3, 9] |
| `- - 3 4 5 - 7 8 9` | 9 | 7 | [3, 4] | [7] | [7] |
| `- - 3 4 5 - 7 8 9` | 9 | 8 | [8] | [3, 5] | [8] |
| `- - 3 4 5 - 7 8 9` | 9 | 11 | [3, 8] | [4, 7] | [3, 8] |
| `- - 3 4 5 - 7 8 9` | 9 | 12 | [3, 4, 5] | [5, 7] | [3, 9] |
| `- 2 - 4 - 6 7 8 9` | 9 | 6 | [6] | [2, 4] | [6] |
| `- 2 - 4 - 6 7 8 9` | 9 | 8 | [8] | [2, 6] | [8] |
| `- 2 - 4 - 6 7 8 9` | 9 | 9 | [9] | [2, 7] | [9] |
| `- 2 - 4 - 6 7 8 9` | 9 | 10 | [2, 8] | [4, 6] | [2, 8] |
| `- 2 - 4 - 6 7 8 9` | 9 | 11 | [2, 9] | [4, 7] | [2, 9] |
| `- 2 - 4 - 6 7 8 9` | 9 | 12 | [2, 4, 6] | [4, 8] | [4, 8] |
| `- 2 3 4 5 6 7 - 9` | 9 | 5 | [5] | [2, 3] | [5] |
| `- 2 3 4 5 6 7 - 9` | 9 | 6 | [6] | [2, 4] | [6] |
| `- 2 3 4 5 6 7 - 9` | 9 | 7 | [7] | [2, 5] | [7] |
| `- 2 3 4 5 6 7 - 9` | 9 | 8 | [3, 5] | [2, 6] | [2, 6] |
| `- 2 3 4 5 6 7 - 9` | 9 | 9 | [9] | [3, 6] | [9] |
| `- 2 3 4 5 6 7 - 9` | 9 | 10 | [3, 7] | [4, 6] | [3, 7] |
| `- 2 3 4 5 6 7 - 9` | 9 | 11 | [2, 9] | [2, 3, 6] | [2, 9] |
| `- 2 3 4 5 6 7 - 9` | 9 | 12 | [3, 9] | [2, 3, 7] | [3, 9] |
| `1 - - - 5 6 7 8 9` | 9 | 7 | [7] | [1, 6] | [7] |
| `1 - - - 5 6 7 8 9` | 9 | 8 | [8] | [1, 7] | [8] |
| `1 - - - 5 6 7 8 9` | 9 | 9 | [1, 8] | [9] | [9] |
| `1 - - - 5 6 7 8 9` | 9 | 12 | [1, 5, 6] | [5, 7] | [5, 7] |
| `1 - 3 4 5 6 - 8 9` | 9 | 4 | [4] | [1, 3] | [4] |
| `1 - 3 4 5 6 - 8 9` | 9 | 5 | [5] | [1, 4] | [5] |
| `1 - 3 4 5 6 - 8 9` | 9 | 7 | [3, 4] | [1, 6] | [1, 6] |
| `1 - 3 4 5 6 - 8 9` | 9 | 8 | [8] | [1, 3, 4] | [8] |
| `1 - 3 4 5 6 - 8 9` | 9 | 9 | [9] | [3, 6] | [9] |
| `1 - 3 4 5 6 - 8 9` | 9 | 10 | [1, 9] | [1, 3, 6] | [1, 9] |
| `1 - 3 4 5 6 - 8 9` | 9 | 11 | [5, 6] | [1, 4, 6] | [3, 8] |
| `1 - 3 4 5 6 - 8 9` | 9 | 12 | [4, 8] | [1, 3, 8] | [3, 9] |
| `1 2 - 4 5 - 7 8 9` | 9 | 5 | [5] | [1, 4] | [5] |
| `1 2 - 4 5 - 7 8 9` | 9 | 6 | [1, 5] | [2, 4] | [1, 5] |
| `1 2 - 4 5 - 7 8 9` | 9 | 7 | [7] | [2, 5] | [7] |
| `1 2 - 4 5 - 7 8 9` | 9 | 8 | [8] | [1, 2, 5] | [8] |
| `1 2 - 4 5 - 7 8 9` | 9 | 9 | [9] | [2, 7] | [9] |
| `1 2 - 4 5 - 7 8 9` | 9 | 10 | [1, 9] | [1, 2, 7] | [1, 9] |
| `1 2 - 4 5 - 7 8 9` | 9 | 11 | [2, 9] | [2, 4, 5] | [2, 9] |
| `1 2 - 4 5 - 7 8 9` | 9 | 12 | [5, 7] | [1, 2, 9] | [4, 8] |
| `1 2 3 - - 6 7 8 9` | 9 | 3 | [1, 2] | [3] | [3] |
| `1 2 3 - - 6 7 8 9` | 9 | 6 | [6] | [1, 2, 3] | [6] |
| `1 2 3 - - 6 7 8 9` | 9 | 7 | [7] | [1, 6] | [7] |
| `1 2 3 - - 6 7 8 9` | 9 | 8 | [8] | [2, 6] | [8] |
| `1 2 3 - - 6 7 8 9` | 9 | 9 | [9] | [3, 6] | [9] |
| `1 2 3 - - 6 7 8 9` | 9 | 10 | [1, 9] | [1, 2, 7] | [1, 9] |
| `1 2 3 - - 6 7 8 9` | 9 | 11 | [2, 9] | [2, 3, 6] | [2, 9] |
| `1 2 3 - - 6 7 8 9` | 9 | 12 | [1, 2, 3, 6] | [3, 9] | [3, 9] |
| `1 2 3 4 5 6 7 8 -` | 9 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 4 5 6 7 8 -` | 9 | 4 | [4] | [1, 3] | [4] |
| `1 2 3 4 5 6 7 8 -` | 9 | 5 | [5] | [1, 4] | [5] |
| `1 2 3 4 5 6 7 8 -` | 9 | 6 | [6] | [2, 4] | [6] |
| `1 2 3 4 5 6 7 8 -` | 9 | 7 | [7] | [3, 4] | [7] |
| `1 2 3 4 5 6 7 8 -` | 9 | 8 | [8] | [1, 3, 4] | [8] |
| `1 2 3 4 5 6 7 8 -` | 9 | 9 | [1, 8] | [2, 3, 4] | [1, 8] |
| `1 2 3 4 5 6 7 8 -` | 9 | 10 | [3, 7] | [1, 2, 3, 4] | [2, 8] |
| `1 2 3 4 5 6 7 8 -` | 9 | 11 | [3, 8] | [1, 4, 6] | [3, 8] |
| `1 2 3 4 5 6 7 8 -` | 9 | 12 | [5, 7] | [3, 4, 5] | [4, 8] |
| `- - 3 4 5 6 - 8 9` | 10 | 8 | [8] | [3, 5] | [8] |
| `- - 3 4 5 6 - 8 9` | 10 | 9 | [9] | [3, 6] | [9] |
| `- - 3 4 5 6 - 8 9` | 10 | 11 | [5, 6] | [3, 8] | [3, 8] |
| `- - 3 4 5 6 - 8 9` | 10 | 12 | [3, 4, 5] | [4, 8] | [3, 9] |
| `- 2 - 4 5 - 7 8 9` | 10 | 7 | [7] | [2, 5] | [7] |
| `- 2 - 4 5 - 7 8 9` | 10 | 9 | [9] | [2, 7] | [9] |
| `- 2 - 4 5 - 7 8 9` | 10 | 11 | [2, 4, 5] | [2, 9] | [2, 9] |
| `- 2 - 4 5 - 7 8 9` | 10 | 12 | [5, 7] | [4, 8] | [4, 8] |
| `- 2 3 - - 6 7 8 9` | 10 | 8 | [8] | [2, 6] | [8] |
| `- 2 3 - - 6 7 8 9` | 10 | 9 | [9] | [2, 7] | [9] |
| `- 2 3 - - 6 7 8 9` | 10 | 11 | [2, 3, 6] | [3, 8] | [2, 9] |
| `- 2 3 - - 6 7 8 9` | 10 | 12 | [2, 3, 7] | [3, 9] | [3, 9] |
| `- 2 3 4 5 6 7 8 -` | 10 | 5 | [5] | [2, 3] | [5] |
| `- 2 3 4 5 6 7 8 -` | 10 | 6 | [6] | [2, 4] | [6] |
| `- 2 3 4 5 6 7 8 -` | 10 | 7 | [7] | [3, 4] | [7] |
| `- 2 3 4 5 6 7 8 -` | 10 | 8 | [8] | [3, 5] | [8] |
| `- 2 3 4 5 6 7 8 -` | 10 | 9 | [2, 7] | [2, 3, 4] | [2, 7] |
| `- 2 3 4 5 6 7 8 -` | 10 | 10 | [2, 8] | [4, 6] | [2, 8] |
| `- 2 3 4 5 6 7 8 -` | 10 | 11 | [5, 6] | [2, 3, 6] | [3, 8] |
| `- 2 3 4 5 6 7 8 -` | 10 | 12 | [5, 7] | [2, 4, 6] | [4, 8] |
| `1 - - 4 - 6 7 8 9` | 10 | 7 | [7] | [1, 6] | [7] |
| `1 - - 4 - 6 7 8 9` | 10 | 8 | [8] | [1, 7] | [8] |
| `1 - - 4 - 6 7 8 9` | 10 | 9 | [9] | [1, 8] | [9] |
| `1 - - 4 - 6 7 8 9` | 10 | 10 | [1, 9] | [4, 6] | [1, 9] |
| `1 - - 4 - 6 7 8 9` | 10 | 11 | [1, 4, 6] | [4, 7] | [4, 7] |
| `1 - - 4 - 6 7 8 9` | 10 | 12 | [1, 4, 7] | [4, 8] | [4, 8] |
| `1 - 3 4 5 6 7 - 9` | 10 | 4 | [4] | [1, 3] | [4] |
| `1 - 3 4 5 6 7 - 9` | 10 | 5 | [5] | [1, 4] | [5] |
| `1 - 3 4 5 6 7 - 9` | 10 | 6 | [6] | [1, 5] | [6] |
| `1 - 3 4 5 6 7 - 9` | 10 | 7 | [7] | [1, 6] | [7] |
| `1 - 3 4 5 6 7 - 9` | 10 | 8 | [1, 7] | [1, 3, 4] | [1, 7] |
| `1 - 3 4 5 6 7 - 9` | 10 | 9 | [9] | [1, 3, 5] | [9] |
| `1 - 3 4 5 6 7 - 9` | 10 | 10 | [1, 9] | [1, 3, 6] | [1, 9] |
| `1 - 3 4 5 6 7 - 9` | 10 | 11 | [4, 7] | [1, 3, 7] | [4, 7] |
| `1 - 3 4 5 6 7 - 9` | 10 | 12 | [5, 7] | [1, 5, 6] | [3, 9] |
| `1 2 - 4 5 6 - 8 9` | 10 | 5 | [5] | [1, 4] | [5] |
| `1 2 - 4 5 6 - 8 9` | 10 | 6 | [6] | [1, 5] | [6] |
| `1 2 - 4 5 6 - 8 9` | 10 | 7 | [1, 6] | [1, 2, 4] | [1, 6] |
| `1 2 - 4 5 6 - 8 9` | 10 | 8 | [8] | [2, 6] | [8] |
| `1 2 - 4 5 6 - 8 9` | 10 | 9 | [9] | [1, 2, 6] | [9] |
| `1 2 - 4 5 6 - 8 9` | 10 | 10 | [1, 9] | [2, 8] | [1, 9] |
| `1 2 - 4 5 6 - 8 9` | 10 | 11 | [5, 6] | [1, 2, 8] | [2, 9] |
| `1 2 - 4 5 6 - 8 9` | 10 | 12 | [4, 8] | [2, 4, 6] | [4, 8] |
| `1 2 3 - 5 - 7 8 9` | 10 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 - 5 - 7 8 9` | 10 | 5 | [5] | [2, 3] | [5] |
| `1 2 3 - 5 - 7 8 9` | 10 | 7 | [7] | [2, 5] | [7] |
| `1 2 3 - 5 - 7 8 9` | 10 | 8 | [8] | [3, 5] | [8] |
| `1 2 3 - 5 - 7 8 9` | 10 | 9 | [9] | [1, 8] | [9] |
| `1 2 3 - 5 - 7 8 9` | 10 | 10 | [2, 8] | [1, 2, 7] | [1, 9] |
| `1 2 3 - 5 - 7 8 9` | 10 | 11 | [2, 9] | [1, 2, 8] | [2, 9] |
| `1 2 3 - 5 - 7 8 9` | 10 | 12 | [3, 9] | [1, 3, 8] | [3, 9] |
| `- - 3 4 5 6 7 - 9` | 11 | 7 | [7] | [3, 4] | [7] |
| `- - 3 4 5 6 7 - 9` | 11 | 9 | [9] | [3, 6] | [9] |
| `- - 3 4 5 6 7 - 9` | 11 | 10 | [4, 6] | [3, 7] | [3, 7] |
| `- - 3 4 5 6 7 - 9` | 11 | 11 | [4, 7] | [5, 6] | [4, 7] |
| `- - 3 4 5 6 7 - 9` | 11 | 12 | [3, 9] | [5, 7] | [3, 9] |
| `- 2 - 4 5 6 - 8 9` | 11 | 6 | [6] | [2, 4] | [6] |
| `- 2 - 4 5 6 - 8 9` | 11 | 8 | [8] | [2, 6] | [8] |
| `- 2 - 4 5 6 - 8 9` | 11 | 9 | [9] | [4, 5] | [9] |
| `- 2 - 4 5 6 - 8 9` | 11 | 10 | [4, 6] | [2, 8] | [2, 8] |
| `- 2 - 4 5 6 - 8 9` | 11 | 11 | [2, 4, 5] | [5, 6] | [2, 9] |
| `- 2 - 4 5 6 - 8 9` | 11 | 12 | [4, 8] | [2, 4, 6] | [4, 8] |
| `- 2 3 - 5 - 7 8 9` | 11 | 7 | [2, 5] | [7] | [7] |
| `- 2 3 - 5 - 7 8 9` | 11 | 9 | [9] | [2, 7] | [9] |
| `- 2 3 - 5 - 7 8 9` | 11 | 10 | [2, 3, 5] | [2, 8] | [2, 8] |
| `- 2 3 - 5 - 7 8 9` | 11 | 11 | [2, 9] | [3, 8] | [2, 9] |
| `- 2 3 - 5 - 7 8 9` | 11 | 12 | [2, 3, 7] | [5, 7] | [3, 9] |
| `1 - - 4 5 - 7 8 9` | 11 | 8 | [8] | [1, 7] | [8] |
| `1 - - 4 5 - 7 8 9` | 11 | 9 | [9] | [4, 5] | [9] |
| `1 - - 4 5 - 7 8 9` | 11 | 10 | [1, 4, 5] | [1, 9] | [1, 9] |
| `1 - - 4 5 - 7 8 9` | 11 | 12 | [4, 8] | [5, 7] | [4, 8] |
| `1 - 3 - - 6 7 8 9` | 11 | 8 | [8] | [1, 7] | [8] |
| `1 - 3 - - 6 7 8 9` | 11 | 9 | [9] | [3, 6] | [9] |
| `1 - 3 - - 6 7 8 9` | 11 | 10 | [1, 3, 6] | [3, 7] | [1, 9] |
| `1 - 3 - - 6 7 8 9` | 11 | 11 | [1, 3, 7] | [3, 8] | [3, 8] |
| `1 - 3 - - 6 7 8 9` | 11 | 12 | [1, 3, 8] | [3, 9] | [3, 9] |
| `1 - 3 4 5 6 7 8 -` | 11 | 5 | [5] | [1, 4] | [5] |
| `1 - 3 4 5 6 7 8 -` | 11 | 6 | [6] | [1, 5] | [6] |
| `1 - 3 4 5 6 7 8 -` | 11 | 7 | [7] | [3, 4] | [7] |
| `1 - 3 4 5 6 7 8 -` | 11 | 8 | [8] | [1, 3, 4] | [8] |
| `1 - 3 4 5 6 7 8 -` | 11 | 9 | [1, 8] | [1, 3, 5] | [1, 8] |
| `1 - 3 4 5 6 7 8 -` | 11 | 10 | [4, 6] | [1, 3, 6] | [3, 7] |
| `1 - 3 4 5 6 7 8 -` | 11 | 11 | [4, 7] | [1, 4, 6] | [3, 8] |
| `1 - 3 4 5 6 7 8 -` | 11 | 12 | [4, 8] | [3, 4, 5] | [4, 8] |
| `1 2 - 4 5 6 7 - 9` | 11 | 5 | [5] | [1, 4] | [5] |
| `1 2 - 4 5 6 7 - 9` | 11 | 6 | [6] | [1, 5] | [6] |
| `1 2 - 4 5 6 7 - 9` | 11 | 7 | [7] | [1, 6] | [7] |
| `1 2 - 4 5 6 7 - 9` | 11 | 8 | [1, 7] | [1, 2, 5] | [1, 7] |
| `1 2 - 4 5 6 7 - 9` | 11 | 9 | [9] | [1, 2, 6] | [9] |
| `1 2 - 4 5 6 7 - 9` | 11 | 10 | [1, 9] | [1, 2, 7] | [1, 9] |
| `1 2 - 4 5 6 7 - 9` | 11 | 11 | [5, 6] | [1, 4, 6] | [2, 9] |
| `1 2 - 4 5 6 7 - 9` | 11 | 12 | [5, 7] | [1, 5, 6] | [5, 7] |
| `1 2 3 - 5 6 - 8 9` | 11 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 - 5 6 - 8 9` | 11 | 5 | [5] | [2, 3] | [5] |
| `1 2 3 - 5 6 - 8 9` | 11 | 6 | [6] | [1, 5] | [6] |
| `1 2 3 - 5 6 - 8 9` | 11 | 7 | [2, 5] | [1, 6] | [1, 6] |
| `1 2 3 - 5 6 - 8 9` | 11 | 8 | [8] | [1, 2, 5] | [8] |
| `1 2 3 - 5 6 - 8 9` | 11 | 9 | [9] | [1, 2, 6] | [9] |
| `1 2 3 - 5 6 - 8 9` | 11 | 10 | [2, 8] | [2, 3, 5] | [1, 9] |
| `1 2 3 - 5 6 - 8 9` | 11 | 11 | [2, 9] | [2, 3, 6] | [2, 9] |
| `1 2 3 - 5 6 - 8 9` | 11 | 12 | [3, 9] | [1, 2, 9] | [3, 9] |
| `1 2 3 4 - - 7 8 9` | 11 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 4 - - 7 8 9` | 11 | 4 | [4] | [1, 3] | [4] |
| `1 2 3 4 - - 7 8 9` | 11 | 6 | [1, 2, 3] | [2, 4] | [2, 4] |
| `1 2 3 4 - - 7 8 9` | 11 | 7 | [7] | [3, 4] | [7] |
| `1 2 3 4 - - 7 8 9` | 11 | 8 | [8] | [1, 3, 4] | [8] |
| `1 2 3 4 - - 7 8 9` | 11 | 9 | [9] | [2, 3, 4] | [9] |
| `1 2 3 4 - - 7 8 9` | 11 | 10 | [1, 9] | [1, 2, 7] | [1, 9] |
| `1 2 3 4 - - 7 8 9` | 11 | 11 | [3, 8] | [1, 2, 8] | [2, 9] |
| `1 2 3 4 - - 7 8 9` | 11 | 12 | [3, 9] | [2, 3, 7] | [3, 9] |
| `- - - 4 5 - 7 8 9` | 12 | 9 | [4, 5] | [9] | [9] |
| `- - - 4 5 - 7 8 9` | 12 | 12 | [4, 8] | [5, 7] | [4, 8] |
| `- - 3 - - 6 7 8 9` | 12 | 9 | [3, 6] | [9] | [9] |
| `- - 3 4 5 6 7 8 -` | 12 | 7 | [7] | [3, 4] | [7] |
| `- - 3 4 5 6 7 8 -` | 12 | 8 | [8] | [3, 5] | [8] |
| `- - 3 4 5 6 7 8 -` | 12 | 9 | [4, 5] | [3, 6] | [3, 6] |
| `- - 3 4 5 6 7 8 -` | 12 | 10 | [3, 7] | [4, 6] | [3, 7] |
| `- - 3 4 5 6 7 8 -` | 12 | 11 | [3, 8] | [4, 7] | [3, 8] |
| `- - 3 4 5 6 7 8 -` | 12 | 12 | [4, 8] | [3, 4, 5] | [4, 8] |
| `- 2 - 4 5 6 7 - 9` | 12 | 6 | [2, 4] | [6] | [6] |
| `- 2 - 4 5 6 7 - 9` | 12 | 7 | [7] | [2, 5] | [7] |
| `- 2 - 4 5 6 7 - 9` | 12 | 9 | [9] | [2, 7] | [9] |
| `- 2 - 4 5 6 7 - 9` | 12 | 11 | [2, 9] | [5, 6] | [2, 9] |
| `- 2 - 4 5 6 7 - 9` | 12 | 12 | [2, 4, 6] | [5, 7] | [5, 7] |
| `- 2 3 - 5 6 - 8 9` | 12 | 5 | [2, 3] | [5] | [5] |
| `- 2 3 - 5 6 - 8 9` | 12 | 8 | [8] | [2, 6] | [8] |
| `- 2 3 - 5 6 - 8 9` | 12 | 10 | [2, 3, 5] | [2, 8] | [2, 8] |
| `- 2 3 - 5 6 - 8 9` | 12 | 11 | [3, 8] | [2, 9] | [2, 9] |
| `- 2 3 4 - - 7 8 9` | 12 | 7 | [7] | [3, 4] | [7] |
| `- 2 3 4 - - 7 8 9` | 12 | 9 | [9] | [2, 7] | [9] |
| `- 2 3 4 - - 7 8 9` | 12 | 10 | [3, 7] | [2, 8] | [2, 8] |
| `- 2 3 4 - - 7 8 9` | 12 | 11 | [2, 9] | [3, 8] | [2, 9] |
| `- 2 3 4 - - 7 8 9` | 12 | 12 | [3, 9] | [4, 8] | [3, 9] |
| `1 - - 4 5 6 - 8 9` | 12 | 5 | [5] | [1, 4] | [5] |
| `1 - - 4 5 6 - 8 9` | 12 | 6 | [1, 5] | [6] | [6] |
| `1 - - 4 5 6 - 8 9` | 12 | 9 | [9] | [1, 8] | [9] |
| `1 - - 4 5 6 - 8 9` | 12 | 10 | [1, 4, 5] | [4, 6] | [1, 9] |
| `1 - - 4 5 6 - 8 9` | 12 | 11 | [1, 4, 6] | [5, 6] | [5, 6] |
| `1 - 3 - 5 - 7 8 9` | 12 | 8 | [8] | [1, 7] | [8] |
| `1 - 3 - 5 - 7 8 9` | 12 | 9 | [9] | [1, 8] | [9] |
| `1 - 3 - 5 - 7 8 9` | 12 | 10 | [1, 9] | [3, 7] | [1, 9] |
| `1 - 3 - 5 - 7 8 9` | 12 | 11 | [3, 8] | [1, 3, 7] | [3, 8] |
| `1 - 3 - 5 - 7 8 9` | 12 | 12 | [1, 3, 8] | [5, 7] | [3, 9] |
| `1 2 - - - 6 7 8 9` | 12 | 8 | [8] | [2, 6] | [8] |
| `1 2 - - - 6 7 8 9` | 12 | 9 | [1, 2, 6] | [9] | [9] |
| `1 2 - - - 6 7 8 9` | 12 | 10 | [1, 2, 7] | [2, 8] | [1, 9] |
| `1 2 - - - 6 7 8 9` | 12 | 11 | [1, 2, 8] | [2, 9] | [2, 9] |
| `1 2 - 4 5 6 7 8 -` | 12 | 5 | [5] | [1, 4] | [5] |
| `1 2 - 4 5 6 7 8 -` | 12 | 6 | [6] | [2, 4] | [6] |
| `1 2 - 4 5 6 7 8 -` | 12 | 7 | [7] | [1, 2, 4] | [7] |
| `1 2 - 4 5 6 7 8 -` | 12 | 8 | [8] | [1, 2, 5] | [8] |
| `1 2 - 4 5 6 7 8 -` | 12 | 9 | [1, 8] | [4, 5] | [1, 8] |
| `1 2 - 4 5 6 7 8 -` | 12 | 10 | [2, 8] | [1, 4, 5] | [2, 8] |
| `1 2 - 4 5 6 7 8 -` | 12 | 11 | [5, 6] | [2, 4, 5] | [4, 7] |
| `1 2 - 4 5 6 7 8 -` | 12 | 12 | [5, 7] | [2, 4, 6] | [4, 8] |
| `1 2 3 - 5 6 7 - 9` | 12 | 5 | [5] | [2, 3] | [5] |
| `1 2 3 - 5 6 7 - 9` | 12 | 6 | [6] | [1, 2, 3] | [6] |
| `1 2 3 - 5 6 7 - 9` | 12 | 7 | [7] | [1, 6] | [7] |
| `1 2 3 - 5 6 7 - 9` | 12 | 8 | [2, 6] | [1, 2, 5] | [1, 7] |
| `1 2 3 - 5 6 7 - 9` | 12 | 9 | [9] | [1, 3, 5] | [9] |
| `1 2 3 - 5 6 7 - 9` | 12 | 10 | [1, 9] | [1, 3, 6] | [1, 9] |
| `1 2 3 - 5 6 7 - 9` | 12 | 11 | [2, 9] | [1, 2, 3, 5] | [2, 9] |
| `1 2 3 - 5 6 7 - 9` | 12 | 12 | [5, 7] | [1, 5, 6] | [3, 9] |
| `1 2 3 4 - 6 - 8 9` | 12 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 4 - 6 - 8 9` | 12 | 4 | [4] | [1, 3] | [4] |
| `1 2 3 4 - 6 - 8 9` | 12 | 5 | [2, 3] | [1, 4] | [1, 4] |
| `1 2 3 4 - 6 - 8 9` | 12 | 6 | [6] | [1, 2, 3] | [6] |
| `1 2 3 4 - 6 - 8 9` | 12 | 7 | [1, 6] | [1, 2, 4] | [1, 6] |
| `1 2 3 4 - 6 - 8 9` | 12 | 8 | [8] | [1, 3, 4] | [8] |
| `1 2 3 4 - 6 - 8 9` | 12 | 9 | [9] | [1, 2, 6] | [9] |
| `1 2 3 4 - 6 - 8 9` | 12 | 10 | [1, 9] | [4, 6] | [1, 9] |
| `1 2 3 4 - 6 - 8 9` | 12 | 11 | [3, 8] | [2, 3, 6] | [2, 9] |
| `1 2 3 4 - 6 - 8 9` | 12 | 12 | [3, 9] | [2, 4, 6] | [3, 9] |
| `- - - 4 5 6 - 8 9` | 13 | 9 | [4, 5] | [9] | [9] |
| `- - 3 - 5 - 7 8 9` | 13 | 8 | [3, 5] | [8] | [8] |
| `- - 3 - 5 - 7 8 9` | 13 | 12 | [3, 9] | [5, 7] | [3, 9] |
| `- 2 - - - 6 7 8 9` | 13 | 8 | [2, 6] | [8] | [8] |
| `- 2 - - - 6 7 8 9` | 13 | 9 | [2, 7] | [9] | [9] |
| `- 2 - 4 5 6 7 8 -` | 13 | 6 | [6] | [2, 4] | [6] |
| `- 2 - 4 5 6 7 8 -` | 13 | 7 | [7] | [2, 5] | [7] |
| `- 2 - 4 5 6 7 8 -` | 13 | 8 | [8] | [2, 6] | [8] |
| `- 2 - 4 5 6 7 8 -` | 13 | 9 | [2, 7] | [4, 5] | [2, 7] |
| `- 2 - 4 5 6 7 8 -` | 13 | 10 | [2, 8] | [4, 6] | [2, 8] |
| `- 2 - 4 5 6 7 8 -` | 13 | 11 | [5, 6] | [4, 7] | [4, 7] |
| `- 2 - 4 5 6 7 8 -` | 13 | 12 | [4, 8] | [5, 7] | [4, 8] |
| `- 2 3 - 5 6 7 - 9` | 13 | 5 | [5] | [2, 3] | [5] |
| `- 2 3 - 5 6 7 - 9` | 13 | 7 | [7] | [2, 5] | [7] |
| `- 2 3 - 5 6 7 - 9` | 13 | 8 | [2, 6] | [3, 5] | [2, 6] |
| `- 2 3 - 5 6 7 - 9` | 13 | 9 | [9] | [3, 6] | [9] |
| `- 2 3 - 5 6 7 - 9` | 13 | 10 | [3, 7] | [2, 3, 5] | [3, 7] |
| `- 2 3 - 5 6 7 - 9` | 13 | 11 | [2, 9] | [5, 6] | [2, 9] |
| `- 2 3 - 5 6 7 - 9` | 13 | 12 | [3, 9] | [5, 7] | [3, 9] |
| `- 2 3 4 - 6 - 8 9` | 13 | 6 | [6] | [2, 4] | [6] |
| `- 2 3 4 - 6 - 8 9` | 13 | 8 | [8] | [2, 6] | [8] |
| `- 2 3 4 - 6 - 8 9` | 13 | 9 | [9] | [3, 6] | [9] |
| `- 2 3 4 - 6 - 8 9` | 13 | 10 | [2, 8] | [4, 6] | [2, 8] |
| `- 2 3 4 - 6 - 8 9` | 13 | 11 | [2, 9] | [3, 8] | [2, 9] |
| `- 2 3 4 - 6 - 8 9` | 13 | 12 | [2, 4, 6] | [3, 9] | [3, 9] |
| `1 - - 4 5 6 7 - 9` | 13 | 5 | [5] | [1, 4] | [5] |
| `1 - - 4 5 6 7 - 9` | 13 | 6 | [6] | [1, 5] | [6] |
| `1 - - 4 5 6 7 - 9` | 13 | 7 | [7] | [1, 6] | [7] |
| `1 - - 4 5 6 7 - 9` | 13 | 9 | [9] | [4, 5] | [9] |
| `1 - - 4 5 6 7 - 9` | 13 | 10 | [4, 6] | [1, 4, 5] | [1, 9] |
| `1 - - 4 5 6 7 - 9` | 13 | 11 | [5, 6] | [4, 7] | [4, 7] |
| `1 - - 4 5 6 7 - 9` | 13 | 12 | [5, 7] | [1, 5, 6] | [5, 7] |
| `1 - 3 - 5 6 - 8 9` | 13 | 6 | [6] | [1, 5] | [6] |
| `1 - 3 - 5 6 - 8 9` | 13 | 8 | [8] | [3, 5] | [8] |
| `1 - 3 - 5 6 - 8 9` | 13 | 9 | [9] | [3, 6] | [9] |
| `1 - 3 - 5 6 - 8 9` | 13 | 10 | [1, 3, 6] | [1, 9] | [1, 9] |
| `1 - 3 - 5 6 - 8 9` | 13 | 11 | [3, 8] | [5, 6] | [3, 8] |
| `1 - 3 - 5 6 - 8 9` | 13 | 12 | [3, 9] | [1, 5, 6] | [3, 9] |
| `1 - 3 4 - - 7 8 9` | 13 | 4 | [1, 3] | [4] | [4] |
| `1 - 3 4 - - 7 8 9` | 13 | 7 | [7] | [3, 4] | [7] |
| `1 - 3 4 - - 7 8 9` | 13 | 8 | [1, 3, 4] | [1, 7] | [8] |
| `1 - 3 4 - - 7 8 9` | 13 | 9 | [9] | [1, 8] | [9] |
| `1 - 3 4 - - 7 8 9` | 13 | 10 | [1, 9] | [3, 7] | [1, 9] |
| `1 - 3 4 - - 7 8 9` | 13 | 11 | [3, 8] | [4, 7] | [3, 8] |
| `1 - 3 4 - - 7 8 9` | 13 | 12 | [1, 3, 8] | [3, 9] | [3, 9] |
| `1 2 - - 5 - 7 8 9` | 13 | 7 | [7] | [2, 5] | [7] |
| `1 2 - - 5 - 7 8 9` | 13 | 8 | [8] | [1, 7] | [8] |
| `1 2 - - 5 - 7 8 9` | 13 | 9 | [9] | [1, 8] | [9] |
| `1 2 - - 5 - 7 8 9` | 13 | 10 | [2, 8] | [1, 9] | [1, 9] |
| `1 2 - - 5 - 7 8 9` | 13 | 11 | [1, 2, 8] | [2, 9] | [2, 9] |
| `1 2 - - 5 - 7 8 9` | 13 | 12 | [1, 2, 9] | [5, 7] | [5, 7] |
| `1 2 3 - 5 6 7 8 -` | 13 | 5 | [5] | [2, 3] | [5] |
| `1 2 3 - 5 6 7 8 -` | 13 | 6 | [6] | [1, 2, 3] | [6] |
| `1 2 3 - 5 6 7 8 -` | 13 | 7 | [7] | [1, 6] | [7] |
| `1 2 3 - 5 6 7 8 -` | 13 | 8 | [8] | [3, 5] | [8] |
| `1 2 3 - 5 6 7 8 -` | 13 | 9 | [2, 7] | [1, 2, 6] | [1, 8] |
| `1 2 3 - 5 6 7 8 -` | 13 | 10 | [2, 8] | [2, 3, 5] | [2, 8] |
| `1 2 3 - 5 6 7 8 -` | 13 | 11 | [5, 6] | [2, 3, 6] | [3, 8] |
| `1 2 3 - 5 6 7 8 -` | 13 | 12 | [5, 7] | [2, 3, 7] | [5, 7] |
| `1 2 3 4 - 6 7 - 9` | 13 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 4 - 6 7 - 9` | 13 | 4 | [4] | [1, 3] | [4] |
| `1 2 3 4 - 6 7 - 9` | 13 | 5 | [2, 3] | [1, 4] | [1, 4] |
| `1 2 3 4 - 6 7 - 9` | 13 | 6 | [6] | [1, 2, 3] | [6] |
| `1 2 3 4 - 6 7 - 9` | 13 | 7 | [7] | [1, 2, 4] | [7] |
| `1 2 3 4 - 6 7 - 9` | 13 | 8 | [1, 7] | [1, 3, 4] | [1, 7] |
| `1 2 3 4 - 6 7 - 9` | 13 | 9 | [9] | [1, 2, 6] | [9] |
| `1 2 3 4 - 6 7 - 9` | 13 | 10 | [1, 9] | [1, 3, 6] | [1, 9] |
| `1 2 3 4 - 6 7 - 9` | 13 | 11 | [2, 9] | [1, 4, 6] | [2, 9] |
| `1 2 3 4 - 6 7 - 9` | 13 | 12 | [3, 9] | [1, 4, 7] | [3, 9] |
| `1 2 3 4 5 - - 8 9` | 13 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 4 5 - - 8 9` | 13 | 5 | [5] | [2, 3] | [5] |
| `1 2 3 4 5 - - 8 9` | 13 | 6 | [1, 5] | [1, 2, 3] | [1, 5] |
| `1 2 3 4 5 - - 8 9` | 13 | 7 | [3, 4] | [1, 2, 4] | [2, 5] |
| `1 2 3 4 5 - - 8 9` | 13 | 8 | [8] | [1, 2, 5] | [8] |
| `1 2 3 4 5 - - 8 9` | 13 | 9 | [9] | [2, 3, 4] | [9] |
| `1 2 3 4 5 - - 8 9` | 13 | 10 | [1, 9] | [2, 3, 5] | [1, 9] |
| `1 2 3 4 5 - - 8 9` | 13 | 11 | [3, 8] | [2, 4, 5] | [2, 9] |
| `1 2 3 4 5 - - 8 9` | 13 | 12 | [3, 9] | [3, 4, 5] | [3, 9] |
| `- - - 4 5 6 7 - 9` | 14 | 9 | [9] | [4, 5] | [9] |
| `- - - 4 5 6 7 - 9` | 14 | 11 | [4, 7] | [5, 6] | [4, 7] |
| `- - 3 - 5 6 - 8 9` | 14 | 8 | [3, 5] | [8] | [8] |
| `- - 3 - 5 6 - 8 9` | 14 | 9 | [3, 6] | [9] | [9] |
| `- - 3 - 5 6 - 8 9` | 14 | 11 | [3, 8] | [5, 6] | [3, 8] |
| `- - 3 4 - - 7 8 9` | 14 | 7 | [3, 4] | [7] | [7] |
| `- - 3 4 - - 7 8 9` | 14 | 11 | [3, 8] | [4, 7] | [3, 8] |
| `- - 3 4 - - 7 8 9` | 14 | 12 | [3, 9] | [4, 8] | [3, 9] |
| `- 2 - - 5 - 7 8 9` | 14 | 7 | [2, 5] | [7] | [7] |
| `- 2 - - 5 - 7 8 9` | 14 | 9 | [2, 7] | [9] | [9] |
| `- 2 3 - 5 6 7 8 -` | 14 | 5 | [5] | [2, 3] | [5] |
| `- 2 3 - 5 6 7 8 -` | 14 | 8 | [8] | [2, 6] | [8] |
| `- 2 3 - 5 6 7 8 -` | 14 | 9 | [3, 6] | [2, 7] | [2, 7] |
| `- 2 3 - 5 6 7 8 -` | 14 | 10 | [2, 8] | [3, 7] | [2, 8] |
| `- 2 3 - 5 6 7 8 -` | 14 | 11 | [3, 8] | [2, 3, 6] | [3, 8] |
| `- 2 3 - 5 6 7 8 -` | 14 | 12 | [2, 3, 7] | [5, 7] | [5, 7] |
| `- 2 3 4 - 6 7 - 9` | 14 | 6 | [6] | [2, 4] | [6] |
| `- 2 3 4 - 6 7 - 9` | 14 | 7 | [7] | [3, 4] | [7] |
| `- 2 3 4 - 6 7 - 9` | 14 | 9 | [9] | [3, 6] | [9] |
| `- 2 3 4 - 6 7 - 9` | 14 | 10 | [3, 7] | [4, 6] | [3, 7] |
| `- 2 3 4 - 6 7 - 9` | 14 | 11 | [2, 3, 6] | [4, 7] | [2, 9] |
| `- 2 3 4 - 6 7 - 9` | 14 | 12 | [3, 9] | [2, 3, 7] | [3, 9] |
| `- 2 3 4 5 - - 8 9` | 14 | 5 | [5] | [2, 3] | [5] |
| `- 2 3 4 5 - - 8 9` | 14 | 7 | [3, 4] | [2, 5] | [2, 5] |
| `- 2 3 4 5 - - 8 9` | 14 | 8 | [8] | [3, 5] | [8] |
| `- 2 3 4 5 - - 8 9` | 14 | 9 | [9] | [4, 5] | [9] |
| `- 2 3 4 5 - - 8 9` | 14 | 10 | [2, 3, 5] | [2, 8] | [2, 8] |
| `- 2 3 4 5 - - 8 9` | 14 | 11 | [2, 9] | [3, 8] | [2, 9] |
| `- 2 3 4 5 - - 8 9` | 14 | 12 | [3, 9] | [3, 4, 5] | [3, 9] |
| `1 - - - - 6 7 8 9` | 14 | 7 | [1, 6] | [7] | [7] |
| `1 - - - - 6 7 8 9` | 14 | 8 | [1, 7] | [8] | [8] |
| `1 - - - - 6 7 8 9` | 14 | 9 | [1, 8] | [9] | [9] |
| `1 - - 4 5 6 7 8 -` | 14 | 5 | [5] | [1, 4] | [5] |
| `1 - - 4 5 6 7 8 -` | 14 | 6 | [6] | [1, 5] | [6] |
| `1 - - 4 5 6 7 8 -` | 14 | 7 | [7] | [1, 6] | [7] |
| `1 - - 4 5 6 7 8 -` | 14 | 8 | [8] | [1, 7] | [8] |
| `1 - - 4 5 6 7 8 -` | 14 | 9 | [1, 8] | [4, 5] | [1, 8] |
| `1 - - 4 5 6 7 8 -` | 14 | 10 | [1, 4, 5] | [4, 6] | [4, 6] |
| `1 - - 4 5 6 7 8 -` | 14 | 11 | [4, 7] | [5, 6] | [4, 7] |
| `1 - - 4 5 6 7 8 -` | 14 | 12 | [5, 7] | [4, 8] | [4, 8] |
| `1 - 3 - 5 6 7 - 9` | 14 | 6 | [6] | [1, 5] | [6] |
| `1 - 3 - 5 6 7 - 9` | 14 | 7 | [7] | [1, 6] | [7] |
| `1 - 3 - 5 6 7 - 9` | 14 | 8 | [3, 5] | [1, 7] | [1, 7] |
| `1 - 3 - 5 6 7 - 9` | 14 | 9 | [9] | [1, 3, 5] | [9] |
| `1 - 3 - 5 6 7 - 9` | 14 | 10 | [1, 9] | [3, 7] | [1, 9] |
| `1 - 3 - 5 6 7 - 9` | 14 | 11 | [1, 3, 7] | [5, 6] | [5, 6] |
| `1 - 3 - 5 6 7 - 9` | 14 | 12 | [1, 5, 6] | [3, 9] | [3, 9] |
| `1 - 3 4 - 6 - 8 9` | 14 | 4 | [4] | [1, 3] | [4] |
| `1 - 3 4 - 6 - 8 9` | 14 | 7 | [3, 4] | [1, 6] | [1, 6] |
| `1 - 3 4 - 6 - 8 9` | 14 | 8 | [8] | [1, 3, 4] | [8] |
| `1 - 3 4 - 6 - 8 9` | 14 | 9 | [9] | [3, 6] | [9] |
| `1 - 3 4 - 6 - 8 9` | 14 | 10 | [1, 9] | [4, 6] | [1, 9] |
| `1 - 3 4 - 6 - 8 9` | 14 | 11 | [3, 8] | [1, 4, 6] | [3, 8] |
| `1 - 3 4 - 6 - 8 9` | 14 | 12 | [3, 9] | [4, 8] | [3, 9] |
| `1 2 - - 5 6 - 8 9` | 14 | 6 | [6] | [1, 5] | [6] |
| `1 2 - - 5 6 - 8 9` | 14 | 7 | [1, 6] | [2, 5] | [1, 6] |
| `1 2 - - 5 6 - 8 9` | 14 | 8 | [8] | [2, 6] | [8] |
| `1 2 - - 5 6 - 8 9` | 14 | 9 | [9] | [1, 2, 6] | [9] |
| `1 2 - - 5 6 - 8 9` | 14 | 11 | [2, 9] | [5, 6] | [2, 9] |
| `1 2 - - 5 6 - 8 9` | 14 | 12 | [1, 5, 6] | [1, 2, 9] | [1, 2, 9] |
| `1 2 - 4 - - 7 8 9` | 14 | 7 | [7] | [1, 2, 4] | [7] |
| `1 2 - 4 - - 7 8 9` | 14 | 8 | [8] | [1, 7] | [8] |
| `1 2 - 4 - - 7 8 9` | 14 | 9 | [9] | [2, 7] | [9] |
| `1 2 - 4 - - 7 8 9` | 14 | 10 | [1, 9] | [1, 2, 7] | [1, 9] |
| `1 2 - 4 - - 7 8 9` | 14 | 11 | [1, 2, 8] | [4, 7] | [2, 9] |
| `1 2 - 4 - - 7 8 9` | 14 | 12 | [1, 2, 9] | [4, 8] | [4, 8] |
| `1 2 3 4 - 6 7 8 -` | 14 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 4 - 6 7 8 -` | 14 | 4 | [4] | [1, 3] | [4] |
| `1 2 3 4 - 6 7 8 -` | 14 | 5 | [1, 4] | [2, 3] | [1, 4] |
| `1 2 3 4 - 6 7 8 -` | 14 | 6 | [6] | [1, 2, 3] | [6] |
| `1 2 3 4 - 6 7 8 -` | 14 | 7 | [7] | [3, 4] | [7] |
| `1 2 3 4 - 6 7 8 -` | 14 | 8 | [8] | [2, 6] | [8] |
| `1 2 3 4 - 6 7 8 -` | 14 | 9 | [1, 8] | [2, 3, 4] | [1, 8] |
| `1 2 3 4 - 6 7 8 -` | 14 | 10 | [3, 7] | [1, 2, 3, 4] | [2, 8] |
| `1 2 3 4 - 6 7 8 -` | 14 | 11 | [3, 8] | [1, 3, 7] | [3, 8] |
| `1 2 3 4 - 6 7 8 -` | 14 | 12 | [4, 8] | [1, 4, 7] | [4, 8] |
| `1 2 3 4 5 - 7 - 9` | 14 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 4 5 - 7 - 9` | 14 | 4 | [4] | [1, 3] | [4] |
| `1 2 3 4 5 - 7 - 9` | 14 | 5 | [5] | [1, 4] | [5] |
| `1 2 3 4 5 - 7 - 9` | 14 | 6 | [2, 4] | [1, 2, 3] | [1, 5] |
| `1 2 3 4 5 - 7 - 9` | 14 | 7 | [7] | [2, 5] | [7] |
| `1 2 3 4 5 - 7 - 9` | 14 | 8 | [3, 5] | [1, 3, 4] | [1, 7] |
| `1 2 3 4 5 - 7 - 9` | 14 | 9 | [9] | [1, 3, 5] | [9] |
| `1 2 3 4 5 - 7 - 9` | 14 | 10 | [1, 9] | [1, 4, 5] | [1, 9] |
| `1 2 3 4 5 - 7 - 9` | 14 | 11 | [2, 9] | [1, 3, 7] | [2, 9] |
| `1 2 3 4 5 - 7 - 9` | 14 | 12 | [3, 9] | [2, 3, 7] | [3, 9] |
| `- - - 4 5 6 7 8 -` | 15 | 11 | [5, 6] | [4, 7] | [4, 7] |
| `- - - 4 5 6 7 8 -` | 15 | 12 | [4, 8] | [5, 7] | [4, 8] |
| `- - 3 - 5 6 7 - 9` | 15 | 9 | [9] | [3, 6] | [9] |
| `- - 3 - 5 6 7 - 9` | 15 | 12 | [3, 9] | [5, 7] | [3, 9] |
| `- - 3 4 - 6 - 8 9` | 15 | 9 | [9] | [3, 6] | [9] |
| `- - 3 4 - 6 - 8 9` | 15 | 12 | [3, 9] | [4, 8] | [3, 9] |
| `- 2 - - 5 6 - 8 9` | 15 | 8 | [8] | [2, 6] | [8] |
| `- 2 - - 5 6 - 8 9` | 15 | 11 | [5, 6] | [2, 9] | [2, 9] |
| `- 2 - 4 - - 7 8 9` | 15 | 9 | [9] | [2, 7] | [9] |
| `- 2 - 4 - - 7 8 9` | 15 | 11 | [2, 9] | [4, 7] | [2, 9] |
| `- 2 3 4 - 6 7 8 -` | 15 | 6 | [6] | [2, 4] | [6] |
| `- 2 3 4 - 6 7 8 -` | 15 | 7 | [7] | [3, 4] | [7] |
| `- 2 3 4 - 6 7 8 -` | 15 | 8 | [8] | [2, 6] | [8] |
| `- 2 3 4 - 6 7 8 -` | 15 | 9 | [2, 7] | [2, 3, 4] | [2, 7] |
| `- 2 3 4 - 6 7 8 -` | 15 | 10 | [4, 6] | [3, 7] | [2, 8] |
| `- 2 3 4 - 6 7 8 -` | 15 | 11 | [3, 8] | [4, 7] | [3, 8] |
| `- 2 3 4 - 6 7 8 -` | 15 | 12 | [4, 8] | [2, 3, 7] | [4, 8] |
| `- 2 3 4 5 - 7 - 9` | 15 | 5 | [5] | [2, 3] | [5] |
| `- 2 3 4 5 - 7 - 9` | 15 | 7 | [7] | [3, 4] | [7] |
| `- 2 3 4 5 - 7 - 9` | 15 | 9 | [9] | [4, 5] | [9] |
| `- 2 3 4 5 - 7 - 9` | 15 | 10 | [2, 3, 5] | [3, 7] | [3, 7] |
| `- 2 3 4 5 - 7 - 9` | 15 | 11 | [2, 9] | [2, 4, 5] | [2, 9] |
| `- 2 3 4 5 - 7 - 9` | 15 | 12 | [5, 7] | [2, 3, 7] | [3, 9] |
| `1 - - - 5 - 7 8 9` | 15 | 8 | [8] | [1, 7] | [8] |
| `1 - - - 5 - 7 8 9` | 15 | 9 | [1, 8] | [9] | [9] |
| `1 - 3 - 5 6 7 8 -` | 15 | 6 | [6] | [1, 5] | [6] |
| `1 - 3 - 5 6 7 8 -` | 15 | 7 | [7] | [1, 6] | [7] |
| `1 - 3 - 5 6 7 8 -` | 15 | 8 | [8] | [3, 5] | [8] |
| `1 - 3 - 5 6 7 8 -` | 15 | 9 | [1, 8] | [3, 6] | [1, 8] |
| `1 - 3 - 5 6 7 8 -` | 15 | 11 | [5, 6] | [3, 8] | [3, 8] |
| `1 - 3 - 5 6 7 8 -` | 15 | 12 | [5, 7] | [1, 3, 8] | [5, 7] |
| `1 - 3 4 - 6 7 - 9` | 15 | 4 | [4] | [1, 3] | [4] |
| `1 - 3 4 - 6 7 - 9` | 15 | 7 | [7] | [1, 6] | [7] |
| `1 - 3 4 - 6 7 - 9` | 15 | 8 | [1, 3, 4] | [1, 7] | [1, 7] |
| `1 - 3 4 - 6 7 - 9` | 15 | 9 | [9] | [3, 6] | [9] |
| `1 - 3 4 - 6 7 - 9` | 15 | 10 | [3, 7] | [1, 9] | [1, 9] |
| `1 - 3 4 - 6 7 - 9` | 15 | 11 | [1, 4, 6] | [4, 7] | [4, 7] |
| `1 - 3 4 - 6 7 - 9` | 15 | 12 | [3, 9] | [1, 4, 7] | [3, 9] |
| `1 - 3 4 5 - - 8 9` | 15 | 4 | [4] | [1, 3] | [4] |
| `1 - 3 4 5 - - 8 9` | 15 | 5 | [5] | [1, 4] | [5] |
| `1 - 3 4 5 - - 8 9` | 15 | 8 | [8] | [3, 5] | [8] |
| `1 - 3 4 5 - - 8 9` | 15 | 9 | [9] | [4, 5] | [9] |
| `1 - 3 4 5 - - 8 9` | 15 | 10 | [1, 9] | [1, 4, 5] | [1, 9] |
| `1 - 3 4 5 - - 8 9` | 15 | 12 | [4, 8] | [1, 3, 8] | [3, 9] |
| `1 2 - - 5 6 7 - 9` | 15 | 6 | [6] | [1, 5] | [6] |
| `1 2 - - 5 6 7 - 9` | 15 | 7 | [7] | [1, 6] | [7] |
| `1 2 - - 5 6 7 - 9` | 15 | 8 | [2, 6] | [1, 2, 5] | [1, 7] |
| `1 2 - - 5 6 7 - 9` | 15 | 9 | [9] | [2, 7] | [9] |
| `1 2 - - 5 6 7 - 9` | 15 | 10 | [1, 9] | [1, 2, 7] | [1, 9] |
| `1 2 - - 5 6 7 - 9` | 15 | 11 | [5, 6] | [2, 9] | [2, 9] |
| `1 2 - - 5 6 7 - 9` | 15 | 12 | [5, 7] | [1, 5, 6] | [5, 7] |
| `1 2 - 4 - 6 - 8 9` | 15 | 6 | [6] | [2, 4] | [6] |
| `1 2 - 4 - 6 - 8 9` | 15 | 7 | [1, 2, 4] | [1, 6] | [1, 6] |
| `1 2 - 4 - 6 - 8 9` | 15 | 8 | [8] | [2, 6] | [8] |
| `1 2 - 4 - 6 - 8 9` | 15 | 9 | [9] | [1, 8] | [9] |
| `1 2 - 4 - 6 - 8 9` | 15 | 10 | [2, 8] | [1, 9] | [1, 9] |
| `1 2 - 4 - 6 - 8 9` | 15 | 11 | [2, 9] | [1, 2, 8] | [2, 9] |
| `1 2 - 4 - 6 - 8 9` | 15 | 12 | [4, 8] | [2, 4, 6] | [4, 8] |
| `1 2 3 - - - 7 8 9` | 15 | 3 | [1, 2] | [3] | [3] |
| `1 2 3 - - - 7 8 9` | 15 | 8 | [8] | [1, 7] | [8] |
| `1 2 3 - - - 7 8 9` | 15 | 9 | [9] | [2, 7] | [9] |
| `1 2 3 - - - 7 8 9` | 15 | 10 | [1, 9] | [3, 7] | [1, 9] |
| `1 2 3 - - - 7 8 9` | 15 | 11 | [1, 2, 8] | [3, 8] | [2, 9] |
| `1 2 3 - - - 7 8 9` | 15 | 12 | [1, 2, 9] | [3, 9] | [3, 9] |
| `1 2 3 4 5 - 7 8 -` | 15 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 4 5 - 7 8 -` | 15 | 5 | [5] | [1, 4] | [5] |
| `1 2 3 4 5 - 7 8 -` | 15 | 6 | [1, 5] | [1, 2, 3] | [1, 5] |
| `1 2 3 4 5 - 7 8 -` | 15 | 7 | [7] | [1, 2, 4] | [7] |
| `1 2 3 4 5 - 7 8 -` | 15 | 8 | [8] | [1, 2, 5] | [8] |
| `1 2 3 4 5 - 7 8 -` | 15 | 9 | [1, 8] | [2, 3, 4] | [1, 8] |
| `1 2 3 4 5 - 7 8 -` | 15 | 10 | [3, 7] | [2, 3, 5] | [2, 8] |
| `1 2 3 4 5 - 7 8 -` | 15 | 11 | [3, 8] | [2, 4, 5] | [3, 8] |
| `1 2 3 4 5 - 7 8 -` | 15 | 12 | [4, 8] | [1, 4, 7] | [4, 8] |
| `1 2 3 4 5 6 - - 9` | 15 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 4 5 6 - - 9` | 15 | 4 | [4] | [1, 3] | [4] |
| `1 2 3 4 5 6 - - 9` | 15 | 5 | [5] | [2, 3] | [5] |
| `1 2 3 4 5 6 - - 9` | 15 | 6 | [6] | [1, 2, 3] | [6] |
| `1 2 3 4 5 6 - - 9` | 15 | 7 | [1, 6] | [1, 2, 4] | [1, 6] |
| `1 2 3 4 5 6 - - 9` | 15 | 8 | [3, 5] | [1, 2, 5] | [2, 6] |
| `1 2 3 4 5 6 - - 9` | 15 | 9 | [9] | [1, 3, 5] | [9] |
| `1 2 3 4 5 6 - - 9` | 15 | 10 | [1, 9] | [1, 4, 5] | [1, 9] |
| `1 2 3 4 5 6 - - 9` | 15 | 11 | [2, 9] | [2, 3, 6] | [2, 9] |
| `1 2 3 4 5 6 - - 9` | 15 | 12 | [3, 9] | [1, 2, 3, 6] | [3, 9] |
| `- - 3 - 5 6 7 8 -` | 16 | 8 | [8] | [3, 5] | [8] |
| `- - 3 - 5 6 7 8 -` | 16 | 11 | [5, 6] | [3, 8] | [3, 8] |
| `- - 3 4 - 6 7 - 9` | 16 | 7 | [3, 4] | [7] | [7] |
| `- - 3 4 - 6 7 - 9` | 16 | 9 | [3, 6] | [9] | [9] |
| `- - 3 4 - 6 7 - 9` | 16 | 10 | [4, 6] | [3, 7] | [3, 7] |
| `- - 3 4 5 - - 8 9` | 16 | 8 | [3, 5] | [8] | [8] |
| `- - 3 4 5 - - 8 9` | 16 | 9 | [9] | [4, 5] | [9] |
| `- - 3 4 5 - - 8 9` | 16 | 12 | [3, 4, 5] | [4, 8] | [3, 9] |
| `- 2 - - 5 6 7 - 9` | 16 | 7 | [7] | [2, 5] | [7] |
| `- 2 - - 5 6 7 - 9` | 16 | 9 | [9] | [2, 7] | [9] |
| `- 2 - - 5 6 7 - 9` | 16 | 11 | [2, 9] | [5, 6] | [2, 9] |
| `- 2 - 4 - 6 - 8 9` | 16 | 6 | [2, 4] | [6] | [6] |
| `- 2 - 4 - 6 - 8 9` | 16 | 8 | [2, 6] | [8] | [8] |
| `- 2 - 4 - 6 - 8 9` | 16 | 10 | [4, 6] | [2, 8] | [2, 8] |
| `- 2 - 4 - 6 - 8 9` | 16 | 12 | [2, 4, 6] | [4, 8] | [4, 8] |
| `- 2 3 - - - 7 8 9` | 16 | 9 | [9] | [2, 7] | [9] |
| `- 2 3 - - - 7 8 9` | 16 | 10 | [2, 8] | [3, 7] | [2, 8] |
| `- 2 3 - - - 7 8 9` | 16 | 11 | [2, 9] | [3, 8] | [2, 9] |
| `- 2 3 - - - 7 8 9` | 16 | 12 | [2, 3, 7] | [3, 9] | [3, 9] |
| `- 2 3 4 5 - 7 8 -` | 16 | 5 | [5] | [2, 3] | [5] |
| `- 2 3 4 5 - 7 8 -` | 16 | 7 | [7] | [2, 5] | [7] |
| `- 2 3 4 5 - 7 8 -` | 16 | 8 | [8] | [3, 5] | [8] |
| `- 2 3 4 5 - 7 8 -` | 16 | 9 | [4, 5] | [2, 7] | [2, 7] |
| `- 2 3 4 5 - 7 8 -` | 16 | 10 | [3, 7] | [2, 3, 5] | [2, 8] |
| `- 2 3 4 5 - 7 8 -` | 16 | 11 | [2, 4, 5] | [4, 7] | [3, 8] |
| `- 2 3 4 5 - 7 8 -` | 16 | 12 | [5, 7] | [4, 8] | [4, 8] |
| `- 2 3 4 5 6 - - 9` | 16 | 5 | [5] | [2, 3] | [5] |
| `- 2 3 4 5 6 - - 9` | 16 | 6 | [6] | [2, 4] | [6] |
| `- 2 3 4 5 6 - - 9` | 16 | 7 | [3, 4] | [2, 5] | [2, 5] |
| `- 2 3 4 5 6 - - 9` | 16 | 9 | [9] | [4, 5] | [9] |
| `- 2 3 4 5 6 - - 9` | 16 | 10 | [4, 6] | [2, 3, 5] | [4, 6] |
| `- 2 3 4 5 6 - - 9` | 16 | 11 | [2, 9] | [2, 3, 6] | [2, 9] |
| `- 2 3 4 5 6 - - 9` | 16 | 12 | [3, 9] | [2, 4, 6] | [3, 9] |
| `1 - - - 5 6 - 8 9` | 16 | 6 | [1, 5] | [6] | [6] |
| `1 - - - 5 6 - 8 9` | 16 | 9 | [9] | [1, 8] | [9] |
| `1 - - 4 - - 7 8 9` | 16 | 8 | [8] | [1, 7] | [8] |
| `1 - - 4 - - 7 8 9` | 16 | 9 | [1, 8] | [9] | [9] |
| `1 - - 4 - - 7 8 9` | 16 | 12 | [1, 4, 7] | [4, 8] | [4, 8] |
| `1 - 3 4 - 6 7 8 -` | 16 | 4 | [4] | [1, 3] | [4] |
| `1 - 3 4 - 6 7 8 -` | 16 | 7 | [7] | [3, 4] | [7] |
| `1 - 3 4 - 6 7 8 -` | 16 | 8 | [8] | [1, 3, 4] | [8] |
| `1 - 3 4 - 6 7 8 -` | 16 | 10 | [3, 7] | [4, 6] | [3, 7] |
| `1 - 3 4 - 6 7 8 -` | 16 | 11 | [4, 7] | [1, 3, 7] | [3, 8] |
| `1 - 3 4 - 6 7 8 -` | 16 | 12 | [4, 8] | [1, 4, 7] | [4, 8] |
| `1 - 3 4 5 - 7 - 9` | 16 | 4 | [4] | [1, 3] | [4] |
| `1 - 3 4 5 - 7 - 9` | 16 | 8 | [3, 5] | [1, 7] | [1, 7] |
| `1 - 3 4 5 - 7 - 9` | 16 | 9 | [9] | [4, 5] | [9] |
| `1 - 3 4 5 - 7 - 9` | 16 | 10 | [1, 9] | [3, 7] | [1, 9] |
| `1 - 3 4 5 - 7 - 9` | 16 | 11 | [4, 7] | [1, 3, 7] | [4, 7] |
| `1 - 3 4 5 - 7 - 9` | 16 | 12 | [3, 9] | [1, 4, 7] | [3, 9] |
| `1 2 - - 5 6 7 8 -` | 16 | 6 | [6] | [1, 5] | [6] |
| `1 2 - - 5 6 7 8 -` | 16 | 7 | [7] | [2, 5] | [7] |
| `1 2 - - 5 6 7 8 -` | 16 | 8 | [8] | [2, 6] | [8] |
| `1 2 - - 5 6 7 8 -` | 16 | 9 | [1, 8] | [2, 7] | [1, 8] |
| `1 2 - - 5 6 7 8 -` | 16 | 10 | [1, 2, 7] | [2, 8] | [2, 8] |
| `1 2 - - 5 6 7 8 -` | 16 | 11 | [1, 2, 8] | [5, 6] | [5, 6] |
| `1 2 - 4 - 6 7 - 9` | 16 | 6 | [6] | [2, 4] | [6] |
| `1 2 - 4 - 6 7 - 9` | 16 | 7 | [7] | [1, 6] | [7] |
| `1 2 - 4 - 6 7 - 9` | 16 | 8 | [2, 6] | [1, 7] | [1, 7] |
| `1 2 - 4 - 6 7 - 9` | 16 | 9 | [9] | [1, 2, 6] | [9] |
| `1 2 - 4 - 6 7 - 9` | 16 | 10 | [1, 9] | [4, 6] | [1, 9] |
| `1 2 - 4 - 6 7 - 9` | 16 | 11 | [2, 9] | [1, 4, 6] | [2, 9] |
| `1 2 - 4 - 6 7 - 9` | 16 | 12 | [2, 4, 6] | [1, 4, 7] | [1, 2, 9] |
| `1 2 - 4 5 - - 8 9` | 16 | 5 | [5] | [1, 4] | [5] |
| `1 2 - 4 5 - - 8 9` | 16 | 6 | [1, 5] | [2, 4] | [1, 5] |
| `1 2 - 4 5 - - 8 9` | 16 | 7 | [1, 2, 4] | [2, 5] | [2, 5] |
| `1 2 - 4 5 - - 8 9` | 16 | 8 | [8] | [1, 2, 5] | [8] |
| `1 2 - 4 5 - - 8 9` | 16 | 9 | [9] | [4, 5] | [9] |
| `1 2 - 4 5 - - 8 9` | 16 | 10 | [1, 9] | [2, 8] | [1, 9] |
| `1 2 - 4 5 - - 8 9` | 16 | 11 | [2, 9] | [1, 2, 8] | [2, 9] |
| `1 2 - 4 5 - - 8 9` | 16 | 12 | [1, 2, 4, 5] | [1, 2, 9] | [4, 8] |
| `1 2 3 - - 6 - 8 9` | 16 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 - - 6 - 8 9` | 16 | 6 | [1, 2, 3] | [6] | [6] |
| `1 2 3 - - 6 - 8 9` | 16 | 8 | [8] | [2, 6] | [8] |
| `1 2 3 - - 6 - 8 9` | 16 | 9 | [9] | [3, 6] | [9] |
| `1 2 3 - - 6 - 8 9` | 16 | 10 | [1, 3, 6] | [2, 8] | [1, 9] |
| `1 2 3 - - 6 - 8 9` | 16 | 11 | [2, 9] | [1, 2, 8] | [2, 9] |
| `1 2 3 - - 6 - 8 9` | 16 | 12 | [1, 2, 3, 6] | [1, 3, 8] | [3, 9] |
| `1 2 3 4 5 6 - 8 -` | 16 | 4 | [4] | [1, 3] | [4] |
| `1 2 3 4 5 6 - 8 -` | 16 | 5 | [5] | [1, 4] | [5] |
| `1 2 3 4 5 6 - 8 -` | 16 | 6 | [6] | [1, 2, 3] | [6] |
| `1 2 3 4 5 6 - 8 -` | 16 | 7 | [1, 6] | [1, 2, 4] | [1, 6] |
| `1 2 3 4 5 6 - 8 -` | 16 | 8 | [8] | [1, 3, 4] | [8] |
| `1 2 3 4 5 6 - 8 -` | 16 | 9 | [1, 8] | [1, 3, 5] | [1, 8] |
| `1 2 3 4 5 6 - 8 -` | 16 | 10 | [2, 8] | [1, 4, 5] | [2, 8] |
| `1 2 3 4 5 6 - 8 -` | 16 | 11 | [3, 8] | [1, 4, 6] | [3, 8] |
| `1 2 3 4 5 6 - 8 -` | 16 | 12 | [4, 8] | [2, 4, 6] | [4, 8] |
| `- - 3 4 - 6 7 8 -` | 17 | 7 | [7] | [3, 4] | [7] |
| `- - 3 4 - 6 7 8 -` | 17 | 10 | [4, 6] | [3, 7] | [3, 7] |
| `- - 3 4 - 6 7 8 -` | 17 | 11 | [3, 8] | [4, 7] | [3, 8] |
| `- - 3 4 5 - 7 - 9` | 17 | 7 | [3, 4] | [7] | [7] |
| `- - 3 4 5 - 7 - 9` | 17 | 9 | [9] | [4, 5] | [9] |
| `- - 3 4 5 - 7 - 9` | 17 | 12 | [3, 4, 5] | [5, 7] | [3, 9] |
| `- 2 - - 5 6 7 8 -` | 17 | 7 | [2, 5] | [7] | [7] |
| `- 2 - - 5 6 7 8 -` | 17 | 8 | [8] | [2, 6] | [8] |
| `- 2 - 4 - 6 7 - 9` | 17 | 6 | [2, 4] | [6] | [6] |
| `- 2 - 4 - 6 7 - 9` | 17 | 9 | [9] | [2, 7] | [9] |
| `- 2 - 4 - 6 7 - 9` | 17 | 11 | [2, 9] | [4, 7] | [2, 9] |
| `- 2 - 4 5 - - 8 9` | 17 | 9 | [9] | [4, 5] | [9] |
| `- 2 - 4 5 - - 8 9` | 17 | 11 | [2, 4, 5] | [2, 9] | [2, 9] |
| `- 2 3 - - 6 - 8 9` | 17 | 8 | [2, 6] | [8] | [8] |
| `- 2 3 - - 6 - 8 9` | 17 | 9 | [3, 6] | [9] | [9] |
| `- 2 3 - - 6 - 8 9` | 17 | 11 | [2, 3, 6] | [3, 8] | [2, 9] |
| `- 2 3 4 5 6 - 8 -` | 17 | 5 | [5] | [2, 3] | [5] |
| `- 2 3 4 5 6 - 8 -` | 17 | 6 | [6] | [2, 4] | [6] |
| `- 2 3 4 5 6 - 8 -` | 17 | 7 | [2, 5] | [3, 4] | [2, 5] |
| `- 2 3 4 5 6 - 8 -` | 17 | 8 | [8] | [3, 5] | [8] |
| `- 2 3 4 5 6 - 8 -` | 17 | 9 | [3, 6] | [4, 5] | [3, 6] |
| `- 2 3 4 5 6 - 8 -` | 17 | 10 | [2, 8] | [4, 6] | [2, 8] |
| `- 2 3 4 5 6 - 8 -` | 17 | 11 | [5, 6] | [2, 3, 6] | [3, 8] |
| `- 2 3 4 5 6 - 8 -` | 17 | 12 | [4, 8] | [2, 4, 6] | [4, 8] |
| `1 - - - 5 6 7 - 9` | 17 | 6 | [6] | [1, 5] | [6] |
| `1 - - - 5 6 7 - 9` | 17 | 7 | [1, 6] | [7] | [7] |
| `1 - - - 5 6 7 - 9` | 17 | 12 | [1, 5, 6] | [5, 7] | [5, 7] |
| `1 - - 4 - 6 - 8 9` | 17 | 9 | [9] | [1, 8] | [9] |
| `1 - - 4 - 6 - 8 9` | 17 | 10 | [1, 9] | [4, 6] | [1, 9] |
| `1 - 3 - - - 7 8 9` | 17 | 8 | [1, 7] | [8] | [8] |
| `1 - 3 - - - 7 8 9` | 17 | 9 | [1, 8] | [9] | [9] |
| `1 - 3 - - - 7 8 9` | 17 | 10 | [1, 9] | [3, 7] | [1, 9] |
| `1 - 3 - - - 7 8 9` | 17 | 11 | [1, 3, 7] | [3, 8] | [3, 8] |
| `1 - 3 - - - 7 8 9` | 17 | 12 | [1, 3, 8] | [3, 9] | [3, 9] |
| `1 - 3 4 5 - 7 8 -` | 17 | 4 | [4] | [1, 3] | [4] |
| `1 - 3 4 5 - 7 8 -` | 17 | 5 | [5] | [1, 4] | [5] |
| `1 - 3 4 5 - 7 8 -` | 17 | 7 | [7] | [3, 4] | [7] |
| `1 - 3 4 5 - 7 8 -` | 17 | 8 | [8] | [3, 5] | [8] |
| `1 - 3 4 5 - 7 8 -` | 17 | 9 | [1, 8] | [4, 5] | [1, 8] |
| `1 - 3 4 5 - 7 8 -` | 17 | 10 | [1, 4, 5] | [3, 7] | [3, 7] |
| `1 - 3 4 5 - 7 8 -` | 17 | 11 | [3, 8] | [4, 7] | [3, 8] |
| `1 - 3 4 5 - 7 8 -` | 17 | 12 | [4, 8] | [1, 4, 7] | [4, 8] |
| `1 - 3 4 5 6 - - 9` | 17 | 4 | [4] | [1, 3] | [4] |
| `1 - 3 4 5 6 - - 9` | 17 | 5 | [5] | [1, 4] | [5] |
| `1 - 3 4 5 6 - - 9` | 17 | 6 | [6] | [1, 5] | [6] |
| `1 - 3 4 5 6 - - 9` | 17 | 7 | [3, 4] | [1, 6] | [1, 6] |
| `1 - 3 4 5 6 - - 9` | 17 | 8 | [3, 5] | [1, 3, 4] | [3, 5] |
| `1 - 3 4 5 6 - - 9` | 17 | 9 | [9] | [3, 6] | [9] |
| `1 - 3 4 5 6 - - 9` | 17 | 10 | [1, 9] | [1, 3, 6] | [1, 9] |
| `1 - 3 4 5 6 - - 9` | 17 | 11 | [5, 6] | [1, 4, 6] | [5, 6] |
| `1 - 3 4 5 6 - - 9` | 17 | 12 | [3, 9] | [1, 5, 6] | [3, 9] |
| `1 2 - 4 - 6 7 8 -` | 17 | 6 | [6] | [2, 4] | [6] |
| `1 2 - 4 - 6 7 8 -` | 17 | 7 | [7] | [1, 2, 4] | [7] |
| `1 2 - 4 - 6 7 8 -` | 17 | 8 | [8] | [1, 7] | [8] |
| `1 2 - 4 - 6 7 8 -` | 17 | 9 | [2, 7] | [1, 2, 6] | [1, 8] |
| `1 2 - 4 - 6 7 8 -` | 17 | 10 | [2, 8] | [4, 6] | [2, 8] |
| `1 2 - 4 - 6 7 8 -` | 17 | 12 | [4, 8] | [1, 4, 7] | [4, 8] |
| `1 2 - 4 5 - 7 - 9` | 17 | 5 | [5] | [1, 4] | [5] |
| `1 2 - 4 5 - 7 - 9` | 17 | 6 | [2, 4] | [1, 5] | [1, 5] |
| `1 2 - 4 5 - 7 - 9` | 17 | 7 | [7] | [1, 2, 4] | [7] |
| `1 2 - 4 5 - 7 - 9` | 17 | 8 | [1, 2, 5] | [1, 7] | [1, 7] |
| `1 2 - 4 5 - 7 - 9` | 17 | 9 | [9] | [2, 7] | [9] |
| `1 2 - 4 5 - 7 - 9` | 17 | 10 | [1, 9] | [1, 2, 7] | [1, 9] |
| `1 2 - 4 5 - 7 - 9` | 17 | 11 | [2, 9] | [2, 4, 5] | [2, 9] |
| `1 2 - 4 5 - 7 - 9` | 17 | 12 | [1, 2, 4, 5] | [1, 4, 7] | [5, 7] |
| `1 2 3 - - 6 7 - 9` | 17 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 - - 6 7 - 9` | 17 | 6 | [6] | [1, 2, 3] | [6] |
| `1 2 3 - - 6 7 - 9` | 17 | 7 | [7] | [1, 6] | [7] |
| `1 2 3 - - 6 7 - 9` | 17 | 8 | [2, 6] | [1, 7] | [1, 7] |
| `1 2 3 - - 6 7 - 9` | 17 | 9 | [9] | [2, 7] | [9] |
| `1 2 3 - - 6 7 - 9` | 17 | 10 | [1, 9] | [1, 2, 7] | [1, 9] |
| `1 2 3 - - 6 7 - 9` | 17 | 11 | [2, 9] | [1, 3, 7] | [2, 9] |
| `1 2 3 - - 6 7 - 9` | 17 | 12 | [1, 2, 3, 6] | [3, 9] | [3, 9] |
| `1 2 3 - 5 - - 8 9` | 17 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 - 5 - - 8 9` | 17 | 5 | [5] | [2, 3] | [5] |
| `1 2 3 - 5 - - 8 9` | 17 | 6 | [1, 2, 3] | [1, 5] | [1, 5] |
| `1 2 3 - 5 - - 8 9` | 17 | 8 | [8] | [3, 5] | [8] |
| `1 2 3 - 5 - - 8 9` | 17 | 9 | [9] | [1, 3, 5] | [9] |
| `1 2 3 - 5 - - 8 9` | 17 | 10 | [2, 8] | [1, 9] | [1, 9] |
| `1 2 3 - 5 - - 8 9` | 17 | 11 | [1, 2, 3, 5] | [2, 9] | [2, 9] |
| `1 2 3 - 5 - - 8 9` | 17 | 12 | [3, 9] | [1, 2, 9] | [3, 9] |
| `1 2 3 4 5 6 7 - -` | 17 | 4 | [4] | [1, 3] | [4] |
| `1 2 3 4 5 6 7 - -` | 17 | 5 | [5] | [2, 3] | [5] |
| `1 2 3 4 5 6 7 - -` | 17 | 6 | [6] | [1, 2, 3] | [6] |
| `1 2 3 4 5 6 7 - -` | 17 | 7 | [7] | [3, 4] | [7] |
| `1 2 3 4 5 6 7 - -` | 17 | 8 | [1, 7] | [1, 2, 5] | [1, 7] |
| `1 2 3 4 5 6 7 - -` | 17 | 9 | [3, 6] | [2, 3, 4] | [2, 7] |
| `1 2 3 4 5 6 7 - -` | 17 | 10 | [4, 6] | [1, 3, 6] | [3, 7] |
| `1 2 3 4 5 6 7 - -` | 17 | 11 | [4, 7] | [1, 4, 6] | [4, 7] |
| `1 2 3 4 5 6 7 - -` | 17 | 12 | [5, 7] | [3, 4, 5] | [5, 7] |
| `- - 3 4 5 - 7 8 -` | 18 | 8 | [8] | [3, 5] | [8] |
| `- - 3 4 5 - 7 8 -` | 18 | 11 | [3, 8] | [4, 7] | [3, 8] |
| `- - 3 4 5 - 7 8 -` | 18 | 12 | [3, 4, 5] | [5, 7] | [4, 8] |
| `- - 3 4 5 6 - - 9` | 18 | 9 | [9] | [3, 6] | [9] |
| `- - 3 4 5 6 - - 9` | 18 | 12 | [3, 4, 5] | [3, 9] | [3, 9] |
| `- 2 - 4 - 6 7 8 -` | 18 | 6 | [6] | [2, 4] | [6] |
| `- 2 - 4 - 6 7 8 -` | 18 | 8 | [8] | [2, 6] | [8] |
| `- 2 - 4 - 6 7 8 -` | 18 | 10 | [4, 6] | [2, 8] | [2, 8] |
| `- 2 - 4 - 6 7 8 -` | 18 | 12 | [2, 4, 6] | [4, 8] | [4, 8] |
| `- 2 - 4 5 - 7 - 9` | 18 | 7 | [2, 5] | [7] | [7] |
| `- 2 - 4 5 - 7 - 9` | 18 | 9 | [9] | [2, 7] | [9] |
| `- 2 - 4 5 - 7 - 9` | 18 | 11 | [2, 4, 5] | [4, 7] | [2, 9] |
| `- 2 3 - - 6 7 - 9` | 18 | 9 | [9] | [2, 7] | [9] |
| `- 2 3 - - 6 7 - 9` | 18 | 11 | [2, 3, 6] | [2, 9] | [2, 9] |
| `- 2 3 - - 6 7 - 9` | 18 | 12 | [2, 3, 7] | [3, 9] | [3, 9] |
| `- 2 3 - 5 - - 8 9` | 18 | 5 | [2, 3] | [5] | [5] |
| `- 2 3 - 5 - - 8 9` | 18 | 8 | [8] | [3, 5] | [8] |
| `- 2 3 - 5 - - 8 9` | 18 | 10 | [2, 3, 5] | [2, 8] | [2, 8] |
| `- 2 3 - 5 - - 8 9` | 18 | 11 | [3, 8] | [2, 9] | [2, 9] |
| `- 2 3 4 5 6 7 - -` | 18 | 5 | [5] | [2, 3] | [5] |
| `- 2 3 4 5 6 7 - -` | 18 | 6 | [6] | [2, 4] | [6] |
| `- 2 3 4 5 6 7 - -` | 18 | 7 | [7] | [2, 5] | [7] |
| `- 2 3 4 5 6 7 - -` | 18 | 9 | [2, 7] | [3, 6] | [2, 7] |
| `- 2 3 4 5 6 7 - -` | 18 | 10 | [3, 7] | [4, 6] | [3, 7] |
| `- 2 3 4 5 6 7 - -` | 18 | 11 | [4, 7] | [2, 3, 6] | [4, 7] |
| `- 2 3 4 5 6 7 - -` | 18 | 12 | [5, 7] | [2, 3, 7] | [5, 7] |
| `1 - - - 5 6 7 8 -` | 18 | 6 | [1, 5] | [6] | [6] |
| `1 - - - 5 6 7 8 -` | 18 | 8 | [1, 7] | [8] | [8] |
| `1 - - - 5 6 7 8 -` | 18 | 12 | [1, 5, 6] | [5, 7] | [5, 7] |
| `1 - - 4 - 6 7 - 9` | 18 | 7 | [7] | [1, 6] | [7] |
| `1 - - 4 - 6 7 - 9` | 18 | 10 | [4, 6] | [1, 9] | [1, 9] |
| `1 - - 4 - 6 7 - 9` | 18 | 11 | [1, 4, 6] | [4, 7] | [4, 7] |
| `1 - - 4 5 - - 8 9` | 18 | 5 | [1, 4] | [5] | [5] |
| `1 - - 4 5 - - 8 9` | 18 | 9 | [9] | [1, 8] | [9] |
| `1 - - 4 5 - - 8 9` | 18 | 10 | [1, 4, 5] | [1, 9] | [1, 9] |
| `1 - 3 - - 6 - 8 9` | 18 | 9 | [9] | [1, 8] | [9] |
| `1 - 3 - - 6 - 8 9` | 18 | 10 | [1, 3, 6] | [1, 9] | [1, 9] |
| `1 - 3 - - 6 - 8 9` | 18 | 12 | [1, 3, 8] | [3, 9] | [3, 9] |
| `1 - 3 4 5 6 - 8 -` | 18 | 4 | [4] | [1, 3] | [4] |
| `1 - 3 4 5 6 - 8 -` | 18 | 5 | [5] | [1, 4] | [5] |
| `1 - 3 4 5 6 - 8 -` | 18 | 7 | [3, 4] | [1, 6] | [1, 6] |
| `1 - 3 4 5 6 - 8 -` | 18 | 8 | [8] | [1, 3, 4] | [8] |
| `1 - 3 4 5 6 - 8 -` | 18 | 9 | [1, 8] | [3, 6] | [1, 8] |
| `1 - 3 4 5 6 - 8 -` | 18 | 10 | [1, 4, 5] | [4, 6] | [4, 6] |
| `1 - 3 4 5 6 - 8 -` | 18 | 11 | [3, 8] | [1, 4, 6] | [3, 8] |
| `1 - 3 4 5 6 - 8 -` | 18 | 12 | [4, 8] | [1, 5, 6] | [4, 8] |
| `1 2 - - - - 7 8 9` | 18 | 8 | [1, 7] | [8] | [8] |
| `1 2 - - - - 7 8 9` | 18 | 9 | [1, 8] | [9] | [9] |
| `1 2 - - - - 7 8 9` | 18 | 10 | [1, 2, 7] | [2, 8] | [1, 9] |
| `1 2 - - - - 7 8 9` | 18 | 11 | [1, 2, 8] | [2, 9] | [2, 9] |
| `1 2 - 4 5 - 7 8 -` | 18 | 5 | [5] | [1, 4] | [5] |
| `1 2 - 4 5 - 7 8 -` | 18 | 6 | [1, 5] | [2, 4] | [1, 5] |
| `1 2 - 4 5 - 7 8 -` | 18 | 7 | [7] | [2, 5] | [7] |
| `1 2 - 4 5 - 7 8 -` | 18 | 8 | [8] | [1, 2, 5] | [8] |
| `1 2 - 4 5 - 7 8 -` | 18 | 9 | [1, 8] | [4, 5] | [1, 8] |
| `1 2 - 4 5 - 7 8 -` | 18 | 10 | [2, 8] | [1, 2, 7] | [2, 8] |
| `1 2 - 4 5 - 7 8 -` | 18 | 11 | [4, 7] | [2, 4, 5] | [4, 7] |
| `1 2 - 4 5 - 7 8 -` | 18 | 12 | [5, 7] | [1, 4, 7] | [4, 8] |
| `1 2 - 4 5 6 - - 9` | 18 | 5 | [5] | [1, 4] | [5] |
| `1 2 - 4 5 6 - - 9` | 18 | 6 | [6] | [1, 5] | [6] |
| `1 2 - 4 5 6 - - 9` | 18 | 7 | [2, 5] | [1, 6] | [1, 6] |
| `1 2 - 4 5 6 - - 9` | 18 | 8 | [1, 2, 5] | [2, 6] | [2, 6] |
| `1 2 - 4 5 6 - - 9` | 18 | 9 | [9] | [1, 2, 6] | [9] |
| `1 2 - 4 5 6 - - 9` | 18 | 10 | [1, 9] | [1, 4, 5] | [1, 9] |
| `1 2 - 4 5 6 - - 9` | 18 | 11 | [5, 6] | [2, 9] | [2, 9] |
| `1 2 - 4 5 6 - - 9` | 18 | 12 | [1, 2, 4, 5] | [1, 5, 6] | [1, 2, 9] |
| `1 2 3 - - 6 7 8 -` | 18 | 3 | [1, 2] | [3] | [3] |
| `1 2 3 - - 6 7 8 -` | 18 | 6 | [6] | [1, 2, 3] | [6] |
| `1 2 3 - - 6 7 8 -` | 18 | 7 | [7] | [1, 6] | [7] |
| `1 2 3 - - 6 7 8 -` | 18 | 8 | [8] | [1, 7] | [8] |
| `1 2 3 - - 6 7 8 -` | 18 | 9 | [1, 8] | [3, 6] | [1, 8] |
| `1 2 3 - - 6 7 8 -` | 18 | 10 | [2, 8] | [3, 7] | [2, 8] |
| `1 2 3 - - 6 7 8 -` | 18 | 11 | [1, 2, 8] | [1, 3, 7] | [3, 8] |
| `1 2 3 - - 6 7 8 -` | 18 | 12 | [1, 2, 3, 6] | [2, 3, 7] | [1, 3, 8] |
| `1 2 3 - 5 - 7 - 9` | 18 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 - 5 - 7 - 9` | 18 | 5 | [5] | [2, 3] | [5] |
| `1 2 3 - 5 - 7 - 9` | 18 | 6 | [1, 2, 3] | [1, 5] | [1, 5] |
| `1 2 3 - 5 - 7 - 9` | 18 | 7 | [7] | [2, 5] | [7] |
| `1 2 3 - 5 - 7 - 9` | 18 | 8 | [1, 7] | [3, 5] | [1, 7] |
| `1 2 3 - 5 - 7 - 9` | 18 | 9 | [9] | [1, 3, 5] | [9] |
| `1 2 3 - 5 - 7 - 9` | 18 | 10 | [3, 7] | [1, 9] | [1, 9] |
| `1 2 3 - 5 - 7 - 9` | 18 | 11 | [2, 9] | [1, 3, 7] | [2, 9] |
| `1 2 3 - 5 - 7 - 9` | 18 | 12 | [3, 9] | [5, 7] | [3, 9] |
| `1 2 3 4 - - - 8 9` | 18 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 4 - - - 8 9` | 18 | 5 | [1, 4] | [2, 3] | [1, 4] |
| `1 2 3 4 - - - 8 9` | 18 | 6 | [1, 2, 3] | [2, 4] | [2, 4] |
| `1 2 3 4 - - - 8 9` | 18 | 7 | [1, 2, 4] | [3, 4] | [3, 4] |
| `1 2 3 4 - - - 8 9` | 18 | 8 | [8] | [1, 3, 4] | [8] |
| `1 2 3 4 - - - 8 9` | 18 | 9 | [9] | [2, 3, 4] | [9] |
| `1 2 3 4 - - - 8 9` | 18 | 10 | [1, 9] | [2, 8] | [1, 9] |
| `1 2 3 4 - - - 8 9` | 18 | 11 | [3, 8] | [2, 9] | [2, 9] |
| `1 2 3 4 - - - 8 9` | 18 | 12 | [3, 9] | [4, 8] | [3, 9] |
| `- - - 4 5 - - 8 9` | 19 | 9 | [4, 5] | [9] | [9] |
| `- - 3 - - 6 - 8 9` | 19 | 9 | [3, 6] | [9] | [9] |
| `- - 3 4 5 6 - 8 -` | 19 | 8 | [8] | [3, 5] | [8] |
| `- - 3 4 5 6 - 8 -` | 19 | 9 | [4, 5] | [3, 6] | [3, 6] |
| `- - 3 4 5 6 - 8 -` | 19 | 11 | [3, 8] | [5, 6] | [3, 8] |
| `- - 3 4 5 6 - 8 -` | 19 | 12 | [3, 4, 5] | [4, 8] | [4, 8] |
| `- 2 - - - - 7 8 9` | 19 | 9 | [2, 7] | [9] | [9] |
| `- 2 - 4 5 - 7 8 -` | 19 | 7 | [7] | [2, 5] | [7] |
| `- 2 - 4 5 - 7 8 -` | 19 | 9 | [4, 5] | [2, 7] | [2, 7] |
| `- 2 - 4 5 - 7 8 -` | 19 | 11 | [2, 4, 5] | [4, 7] | [4, 7] |
| `- 2 - 4 5 - 7 8 -` | 19 | 12 | [5, 7] | [4, 8] | [4, 8] |
| `- 2 - 4 5 6 - - 9` | 19 | 6 | [2, 4] | [6] | [6] |
| `- 2 - 4 5 6 - - 9` | 19 | 9 | [9] | [4, 5] | [9] |
| `- 2 - 4 5 6 - - 9` | 19 | 11 | [2, 4, 5] | [5, 6] | [2, 9] |
| `- 2 3 - - 6 7 8 -` | 19 | 8 | [8] | [2, 6] | [8] |
| `- 2 3 - - 6 7 8 -` | 19 | 9 | [3, 6] | [2, 7] | [2, 7] |
| `- 2 3 - - 6 7 8 -` | 19 | 10 | [2, 8] | [3, 7] | [2, 8] |
| `- 2 3 - - 6 7 8 -` | 19 | 11 | [2, 3, 6] | [3, 8] | [3, 8] |
| `- 2 3 - 5 - 7 - 9` | 19 | 5 | [2, 3] | [5] | [5] |
| `- 2 3 - 5 - 7 - 9` | 19 | 7 | [7] | [2, 5] | [7] |
| `- 2 3 - 5 - 7 - 9` | 19 | 10 | [2, 3, 5] | [3, 7] | [3, 7] |
| `- 2 3 - 5 - 7 - 9` | 19 | 12 | [2, 3, 7] | [5, 7] | [3, 9] |
| `- 2 3 4 - - - 8 9` | 19 | 9 | [9] | [2, 3, 4] | [9] |
| `- 2 3 4 - - - 8 9` | 19 | 11 | [2, 9] | [3, 8] | [2, 9] |
| `- 2 3 4 - - - 8 9` | 19 | 12 | [3, 9] | [4, 8] | [3, 9] |
| `1 - - 4 - 6 7 8 -` | 19 | 7 | [7] | [1, 6] | [7] |
| `1 - - 4 - 6 7 8 -` | 19 | 8 | [8] | [1, 7] | [8] |
| `1 - - 4 - 6 7 8 -` | 19 | 11 | [1, 4, 6] | [4, 7] | [4, 7] |
| `1 - - 4 - 6 7 8 -` | 19 | 12 | [1, 4, 7] | [4, 8] | [4, 8] |
| `1 - - 4 5 - 7 - 9` | 19 | 5 | [5] | [1, 4] | [5] |
| `1 - - 4 5 - 7 - 9` | 19 | 9 | [9] | [4, 5] | [9] |
| `1 - - 4 5 - 7 - 9` | 19 | 10 | [1, 4, 5] | [1, 9] | [1, 9] |
| `1 - - 4 5 - 7 - 9` | 19 | 12 | [1, 4, 7] | [5, 7] | [5, 7] |
| `1 - 3 - - 6 7 - 9` | 19 | 7 | [1, 6] | [7] | [7] |
| `1 - 3 - - 6 7 - 9` | 19 | 9 | [9] | [3, 6] | [9] |
| `1 - 3 - - 6 7 - 9` | 19 | 10 | [1, 3, 6] | [3, 7] | [1, 9] |
| `1 - 3 - 5 - - 8 9` | 19 | 8 | [8] | [3, 5] | [8] |
| `1 - 3 - 5 - - 8 9` | 19 | 9 | [1, 3, 5] | [9] | [9] |
| `1 - 3 - 5 - - 8 9` | 19 | 12 | [1, 3, 8] | [3, 9] | [3, 9] |
| `1 - 3 4 5 6 7 - -` | 19 | 4 | [4] | [1, 3] | [4] |
| `1 - 3 4 5 6 7 - -` | 19 | 6 | [6] | [1, 5] | [6] |
| `1 - 3 4 5 6 7 - -` | 19 | 7 | [7] | [3, 4] | [7] |
| `1 - 3 4 5 6 7 - -` | 19 | 8 | [1, 7] | [1, 3, 4] | [1, 7] |
| `1 - 3 4 5 6 7 - -` | 19 | 9 | [3, 6] | [1, 3, 5] | [3, 6] |
| `1 - 3 4 5 6 7 - -` | 19 | 10 | [4, 6] | [3, 7] | [3, 7] |
| `1 - 3 4 5 6 7 - -` | 19 | 11 | [4, 7] | [5, 6] | [4, 7] |
| `1 - 3 4 5 6 7 - -` | 19 | 12 | [5, 7] | [1, 5, 6] | [5, 7] |
| `1 2 - - - 6 - 8 9` | 19 | 8 | [8] | [2, 6] | [8] |
| `1 2 - - - 6 - 8 9` | 19 | 9 | [1, 2, 6] | [1, 8] | [9] |
| `1 2 - - - 6 - 8 9` | 19 | 10 | [2, 8] | [1, 9] | [1, 9] |
| `1 2 - - - 6 - 8 9` | 19 | 11 | [1, 2, 8] | [2, 9] | [2, 9] |
| `1 2 - 4 5 6 - 8 -` | 19 | 5 | [5] | [1, 4] | [5] |
| `1 2 - 4 5 6 - 8 -` | 19 | 6 | [6] | [1, 5] | [6] |
| `1 2 - 4 5 6 - 8 -` | 19 | 7 | [1, 6] | [1, 2, 4] | [1, 6] |
| `1 2 - 4 5 6 - 8 -` | 19 | 8 | [8] | [2, 6] | [8] |
| `1 2 - 4 5 6 - 8 -` | 19 | 9 | [1, 8] | [1, 2, 6] | [1, 8] |
| `1 2 - 4 5 6 - 8 -` | 19 | 10 | [4, 6] | [1, 4, 5] | [2, 8] |
| `1 2 - 4 5 6 - 8 -` | 19 | 11 | [5, 6] | [1, 4, 6] | [5, 6] |
| `1 2 - 4 5 6 - 8 -` | 19 | 12 | [1, 2, 4, 5] | [1, 5, 6] | [4, 8] |
| `1 2 3 - 5 - 7 8 -` | 19 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 - 5 - 7 8 -` | 19 | 5 | [5] | [2, 3] | [5] |
| `1 2 3 - 5 - 7 8 -` | 19 | 6 | [1, 5] | [1, 2, 3] | [1, 5] |
| `1 2 3 - 5 - 7 8 -` | 19 | 7 | [7] | [2, 5] | [7] |
| `1 2 3 - 5 - 7 8 -` | 19 | 8 | [8] | [1, 7] | [8] |
| `1 2 3 - 5 - 7 8 -` | 19 | 9 | [1, 3, 5] | [1, 8] | [1, 8] |
| `1 2 3 - 5 - 7 8 -` | 19 | 10 | [2, 8] | [1, 2, 7] | [2, 8] |
| `1 2 3 - 5 - 7 8 -` | 19 | 11 | [1, 2, 3, 5] | [1, 3, 7] | [3, 8] |
| `1 2 3 - 5 - 7 8 -` | 19 | 12 | [5, 7] | [1, 3, 8] | [5, 7] |
| `1 2 3 - 5 6 - - 9` | 19 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 - 5 6 - - 9` | 19 | 5 | [5] | [2, 3] | [5] |
| `1 2 3 - 5 6 - - 9` | 19 | 6 | [6] | [1, 5] | [6] |
| `1 2 3 - 5 6 - - 9` | 19 | 7 | [1, 6] | [2, 5] | [1, 6] |
| `1 2 3 - 5 6 - - 9` | 19 | 8 | [3, 5] | [1, 2, 5] | [2, 6] |
| `1 2 3 - 5 6 - - 9` | 19 | 9 | [9] | [1, 2, 6] | [9] |
| `1 2 3 - 5 6 - - 9` | 19 | 10 | [1, 9] | [1, 3, 6] | [1, 9] |
| `1 2 3 - 5 6 - - 9` | 19 | 11 | [2, 9] | [5, 6] | [2, 9] |
| `1 2 3 - 5 6 - - 9` | 19 | 12 | [1, 2, 3, 6] | [1, 5, 6] | [3, 9] |
| `1 2 3 4 - - 7 - 9` | 19 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 4 - - 7 - 9` | 19 | 4 | [4] | [1, 3] | [4] |
| `1 2 3 4 - - 7 - 9` | 19 | 5 | [2, 3] | [1, 4] | [1, 4] |
| `1 2 3 4 - - 7 - 9` | 19 | 6 | [1, 2, 3] | [2, 4] | [2, 4] |
| `1 2 3 4 - - 7 - 9` | 19 | 7 | [7] | [3, 4] | [7] |
| `1 2 3 4 - - 7 - 9` | 19 | 8 | [1, 7] | [1, 3, 4] | [1, 7] |
| `1 2 3 4 - - 7 - 9` | 19 | 9 | [9] | [2, 7] | [9] |
| `1 2 3 4 - - 7 - 9` | 19 | 10 | [1, 2, 3, 4] | [1, 2, 7] | [1, 9] |
| `1 2 3 4 - - 7 - 9` | 19 | 11 | [2, 9] | [4, 7] | [2, 9] |
| `1 2 3 4 - - 7 - 9` | 19 | 12 | [3, 9] | [1, 4, 7] | [3, 9] |
| `- - - 4 5 - 7 - 9` | 20 | 9 | [4, 5] | [9] | [9] |
| `- - 3 - - 6 7 - 9` | 20 | 9 | [3, 6] | [9] | [9] |
| `- - 3 - 5 - - 8 9` | 20 | 8 | [3, 5] | [8] | [8] |
| `- - 3 4 5 6 7 - -` | 20 | 7 | [7] | [3, 4] | [7] |
| `- - 3 4 5 6 7 - -` | 20 | 9 | [4, 5] | [3, 6] | [3, 6] |
| `- - 3 4 5 6 7 - -` | 20 | 10 | [4, 6] | [3, 7] | [3, 7] |
| `- - 3 4 5 6 7 - -` | 20 | 11 | [4, 7] | [5, 6] | [4, 7] |
| `- - 3 4 5 6 7 - -` | 20 | 12 | [3, 4, 5] | [5, 7] | [5, 7] |
| `- 2 - - - 6 - 8 9` | 20 | 8 | [2, 6] | [8] | [8] |
| `- 2 - 4 5 6 - 8 -` | 20 | 6 | [6] | [2, 4] | [6] |
| `- 2 - 4 5 6 - 8 -` | 20 | 8 | [8] | [2, 6] | [8] |
| `- 2 - 4 5 6 - 8 -` | 20 | 10 | [2, 8] | [4, 6] | [2, 8] |
| `- 2 - 4 5 6 - 8 -` | 20 | 11 | [2, 4, 5] | [5, 6] | [5, 6] |
| `- 2 - 4 5 6 - 8 -` | 20 | 12 | [2, 4, 6] | [4, 8] | [4, 8] |
| `- 2 3 - 5 - 7 8 -` | 20 | 5 | [5] | [2, 3] | [5] |
| `- 2 3 - 5 - 7 8 -` | 20 | 7 | [2, 5] | [7] | [7] |
| `- 2 3 - 5 - 7 8 -` | 20 | 8 | [3, 5] | [8] | [8] |
| `- 2 3 - 5 - 7 8 -` | 20 | 10 | [2, 3, 5] | [3, 7] | [2, 8] |
| `- 2 3 - 5 - 7 8 -` | 20 | 12 | [2, 3, 7] | [5, 7] | [5, 7] |
| `- 2 3 - 5 6 - - 9` | 20 | 5 | [2, 3] | [5] | [5] |
| `- 2 3 - 5 6 - - 9` | 20 | 8 | [3, 5] | [2, 6] | [2, 6] |
| `- 2 3 - 5 6 - - 9` | 20 | 9 | [9] | [3, 6] | [9] |
| `- 2 3 - 5 6 - - 9` | 20 | 11 | [2, 3, 6] | [5, 6] | [2, 9] |
| `- 2 3 4 - - 7 - 9` | 20 | 7 | [7] | [3, 4] | [7] |
| `- 2 3 4 - - 7 - 9` | 20 | 9 | [2, 3, 4] | [2, 7] | [9] |
| `- 2 3 4 - - 7 - 9` | 20 | 11 | [2, 9] | [4, 7] | [2, 9] |
| `- 2 3 4 - - 7 - 9` | 20 | 12 | [2, 3, 7] | [3, 9] | [3, 9] |
| `1 - - - - - 7 8 9` | 20 | 8 | [1, 7] | [8] | [8] |
| `1 - - - - - 7 8 9` | 20 | 9 | [1, 8] | [9] | [9] |
| `1 - - 4 5 - 7 8 -` | 20 | 5 | [1, 4] | [5] | [5] |
| `1 - - 4 5 - 7 8 -` | 20 | 8 | [8] | [1, 7] | [8] |
| `1 - - 4 5 - 7 8 -` | 20 | 9 | [1, 8] | [4, 5] | [1, 8] |
| `1 - - 4 5 - 7 8 -` | 20 | 12 | [1, 4, 7] | [5, 7] | [4, 8] |
| `1 - - 4 5 6 - - 9` | 20 | 5 | [5] | [1, 4] | [5] |
| `1 - - 4 5 6 - - 9` | 20 | 6 | [1, 5] | [6] | [6] |
| `1 - - 4 5 6 - - 9` | 20 | 10 | [1, 4, 5] | [4, 6] | [1, 9] |
| `1 - - 4 5 6 - - 9` | 20 | 11 | [1, 4, 6] | [5, 6] | [5, 6] |
| `1 - 3 - - 6 7 8 -` | 20 | 7 | [7] | [1, 6] | [7] |
| `1 - 3 - - 6 7 8 -` | 20 | 8 | [8] | [1, 7] | [8] |
| `1 - 3 - - 6 7 8 -` | 20 | 9 | [1, 8] | [3, 6] | [1, 8] |
| `1 - 3 - - 6 7 8 -` | 20 | 10 | [1, 3, 6] | [3, 7] | [3, 7] |
| `1 - 3 - - 6 7 8 -` | 20 | 11 | [1, 3, 7] | [3, 8] | [3, 8] |
| `1 - 3 - 5 - 7 - 9` | 20 | 8 | [3, 5] | [1, 7] | [1, 7] |
| `1 - 3 - 5 - 7 - 9` | 20 | 9 | [9] | [1, 3, 5] | [9] |
| `1 - 3 - 5 - 7 - 9` | 20 | 10 | [1, 9] | [3, 7] | [1, 9] |
| `1 - 3 - 5 - 7 - 9` | 20 | 12 | [3, 9] | [5, 7] | [3, 9] |
| `1 - 3 4 - - - 8 9` | 20 | 4 | [1, 3] | [4] | [4] |
| `1 - 3 4 - - - 8 9` | 20 | 8 | [1, 3, 4] | [8] | [8] |
| `1 - 3 4 - - - 8 9` | 20 | 12 | [1, 3, 8] | [4, 8] | [3, 9] |
| `1 2 - - - 6 7 - 9` | 20 | 7 | [7] | [1, 6] | [7] |
| `1 2 - - - 6 7 - 9` | 20 | 8 | [2, 6] | [1, 7] | [1, 7] |
| `1 2 - - - 6 7 - 9` | 20 | 9 | [1, 2, 6] | [9] | [9] |
| `1 2 - - - 6 7 - 9` | 20 | 10 | [1, 2, 7] | [1, 9] | [1, 9] |
| `1 2 - - 5 - - 8 9` | 20 | 8 | [1, 2, 5] | [8] | [8] |
| `1 2 - - 5 - - 8 9` | 20 | 9 | [9] | [1, 8] | [9] |
| `1 2 - - 5 - - 8 9` | 20 | 10 | [1, 9] | [2, 8] | [1, 9] |
| `1 2 - - 5 - - 8 9` | 20 | 11 | [1, 2, 8] | [2, 9] | [2, 9] |
| `1 2 - 4 5 6 7 - -` | 20 | 5 | [5] | [1, 4] | [5] |
| `1 2 - 4 5 6 7 - -` | 20 | 6 | [6] | [2, 4] | [6] |
| `1 2 - 4 5 6 7 - -` | 20 | 7 | [7] | [1, 6] | [7] |
| `1 2 - 4 5 6 7 - -` | 20 | 8 | [1, 7] | [1, 2, 5] | [1, 7] |
| `1 2 - 4 5 6 7 - -` | 20 | 9 | [2, 7] | [4, 5] | [2, 7] |
| `1 2 - 4 5 6 7 - -` | 20 | 10 | [4, 6] | [1, 2, 7] | [4, 6] |
| `1 2 - 4 5 6 7 - -` | 20 | 11 | [5, 6] | [1, 4, 6] | [4, 7] |
| `1 2 - 4 5 6 7 - -` | 20 | 12 | [5, 7] | [1, 5, 6] | [5, 7] |
| `1 2 3 - 5 6 - 8 -` | 20 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 - 5 6 - 8 -` | 20 | 5 | [5] | [2, 3] | [5] |
| `1 2 3 - 5 6 - 8 -` | 20 | 6 | [6] | [1, 5] | [6] |
| `1 2 3 - 5 6 - 8 -` | 20 | 7 | [2, 5] | [1, 6] | [1, 6] |
| `1 2 3 - 5 6 - 8 -` | 20 | 8 | [8] | [2, 6] | [8] |
| `1 2 3 - 5 6 - 8 -` | 20 | 9 | [1, 8] | [1, 2, 6] | [1, 8] |
| `1 2 3 - 5 6 - 8 -` | 20 | 10 | [2, 8] | [1, 3, 6] | [2, 8] |
| `1 2 3 - 5 6 - 8 -` | 20 | 11 | [1, 2, 3, 5] | [2, 3, 6] | [3, 8] |
| `1 2 3 - 5 6 - 8 -` | 20 | 12 | [1, 2, 3, 6] | [1, 5, 6] | [1, 3, 8] |
| `1 2 3 4 - - 7 8 -` | 20 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 4 - - 7 8 -` | 20 | 4 | [4] | [1, 3] | [4] |
| `1 2 3 4 - - 7 8 -` | 20 | 5 | [1, 4] | [2, 3] | [1, 4] |
| `1 2 3 4 - - 7 8 -` | 20 | 6 | [1, 2, 3] | [2, 4] | [2, 4] |
| `1 2 3 4 - - 7 8 -` | 20 | 7 | [7] | [3, 4] | [7] |
| `1 2 3 4 - - 7 8 -` | 20 | 8 | [8] | [1, 3, 4] | [8] |
| `1 2 3 4 - - 7 8 -` | 20 | 9 | [1, 8] | [2, 3, 4] | [1, 8] |
| `1 2 3 4 - - 7 8 -` | 20 | 10 | [3, 7] | [2, 8] | [2, 8] |
| `1 2 3 4 - - 7 8 -` | 20 | 11 | [3, 8] | [1, 2, 8] | [3, 8] |
| `1 2 3 4 - - 7 8 -` | 20 | 12 | [4, 8] | [1, 4, 7] | [4, 8] |
| `1 2 3 4 - 6 - - 9` | 20 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 4 - 6 - - 9` | 20 | 4 | [4] | [1, 3] | [4] |
| `1 2 3 4 - 6 - - 9` | 20 | 5 | [2, 3] | [1, 4] | [1, 4] |
| `1 2 3 4 - 6 - - 9` | 20 | 6 | [6] | [2, 4] | [6] |
| `1 2 3 4 - 6 - - 9` | 20 | 7 | [3, 4] | [1, 2, 4] | [1, 6] |
| `1 2 3 4 - 6 - - 9` | 20 | 8 | [2, 6] | [1, 3, 4] | [2, 6] |
| `1 2 3 4 - 6 - - 9` | 20 | 9 | [9] | [1, 2, 6] | [9] |
| `1 2 3 4 - 6 - - 9` | 20 | 10 | [1, 9] | [4, 6] | [1, 9] |
| `1 2 3 4 - 6 - - 9` | 20 | 11 | [2, 9] | [1, 4, 6] | [2, 9] |
| `1 2 3 4 - 6 - - 9` | 20 | 12 | [3, 9] | [2, 4, 6] | [3, 9] |
| `- - - 4 5 - 7 8 -` | 21 | 12 | [4, 8] | [5, 7] | [4, 8] |
| `- - - 4 5 6 - - 9` | 21 | 9 | [4, 5] | [9] | [9] |
| `- - 3 - 5 - 7 - 9` | 21 | 12 | [3, 9] | [5, 7] | [3, 9] |
| `- - 3 4 - - - 8 9` | 21 | 12 | [3, 9] | [4, 8] | [3, 9] |
| `- 2 - - - 6 7 - 9` | 21 | 9 | [2, 7] | [9] | [9] |
| `- 2 - 4 5 6 7 - -` | 21 | 6 | [2, 4] | [6] | [6] |
| `- 2 - 4 5 6 7 - -` | 21 | 7 | [7] | [2, 5] | [7] |
| `- 2 - 4 5 6 7 - -` | 21 | 9 | [4, 5] | [2, 7] | [2, 7] |
| `- 2 - 4 5 6 7 - -` | 21 | 11 | [2, 4, 5] | [5, 6] | [4, 7] |
| `- 2 - 4 5 6 7 - -` | 21 | 12 | [2, 4, 6] | [5, 7] | [5, 7] |
| `- 2 3 - 5 6 - 8 -` | 21 | 5 | [2, 3] | [5] | [5] |
| `- 2 3 - 5 6 - 8 -` | 21 | 8 | [8] | [2, 6] | [8] |
| `- 2 3 - 5 6 - 8 -` | 21 | 10 | [2, 3, 5] | [2, 8] | [2, 8] |
| `- 2 3 - 5 6 - 8 -` | 21 | 11 | [2, 3, 6] | [5, 6] | [3, 8] |
| `- 2 3 4 - - 7 8 -` | 21 | 7 | [7] | [3, 4] | [7] |
| `- 2 3 4 - - 7 8 -` | 21 | 9 | [2, 3, 4] | [2, 7] | [2, 7] |
| `- 2 3 4 - - 7 8 -` | 21 | 10 | [3, 7] | [2, 8] | [2, 8] |
| `- 2 3 4 - - 7 8 -` | 21 | 11 | [3, 8] | [4, 7] | [3, 8] |
| `- 2 3 4 - - 7 8 -` | 21 | 12 | [2, 3, 7] | [4, 8] | [4, 8] |
| `- 2 3 4 - 6 - - 9` | 21 | 6 | [6] | [2, 4] | [6] |
| `- 2 3 4 - 6 - - 9` | 21 | 9 | [9] | [3, 6] | [9] |
| `- 2 3 4 - 6 - - 9` | 21 | 11 | [2, 3, 6] | [2, 9] | [2, 9] |
| `- 2 3 4 - 6 - - 9` | 21 | 12 | [2, 4, 6] | [3, 9] | [3, 9] |
| `1 - - - - 6 - 8 9` | 21 | 9 | [1, 8] | [9] | [9] |
| `1 - - 4 5 6 - 8 -` | 21 | 5 | [5] | [1, 4] | [5] |
| `1 - - 4 5 6 - 8 -` | 21 | 6 | [1, 5] | [6] | [6] |
| `1 - - 4 5 6 - 8 -` | 21 | 9 | [1, 8] | [4, 5] | [1, 8] |
| `1 - - 4 5 6 - 8 -` | 21 | 10 | [1, 4, 5] | [4, 6] | [4, 6] |
| `1 - - 4 5 6 - 8 -` | 21 | 11 | [1, 4, 6] | [5, 6] | [5, 6] |
| `1 - - 4 5 6 - 8 -` | 21 | 12 | [1, 5, 6] | [4, 8] | [4, 8] |
| `1 - 3 - 5 - 7 8 -` | 21 | 8 | [8] | [1, 7] | [8] |
| `1 - 3 - 5 - 7 8 -` | 21 | 9 | [1, 3, 5] | [1, 8] | [1, 8] |
| `1 - 3 - 5 - 7 8 -` | 21 | 11 | [1, 3, 7] | [3, 8] | [3, 8] |
| `1 - 3 - 5 - 7 8 -` | 21 | 12 | [1, 3, 8] | [5, 7] | [5, 7] |
| `1 - 3 - 5 6 - - 9` | 21 | 6 | [6] | [1, 5] | [6] |
| `1 - 3 - 5 6 - - 9` | 21 | 9 | [9] | [3, 6] | [9] |
| `1 - 3 - 5 6 - - 9` | 21 | 10 | [1, 3, 6] | [1, 9] | [1, 9] |
| `1 - 3 - 5 6 - - 9` | 21 | 12 | [1, 5, 6] | [3, 9] | [3, 9] |
| `1 - 3 4 - - 7 - 9` | 21 | 4 | [1, 3] | [4] | [4] |
| `1 - 3 4 - - 7 - 9` | 21 | 8 | [1, 3, 4] | [1, 7] | [1, 7] |
| `1 - 3 4 - - 7 - 9` | 21 | 10 | [1, 9] | [3, 7] | [1, 9] |
| `1 - 3 4 - - 7 - 9` | 21 | 11 | [1, 3, 7] | [4, 7] | [4, 7] |
| `1 - 3 4 - - 7 - 9` | 21 | 12 | [1, 4, 7] | [3, 9] | [3, 9] |
| `1 2 - - - 6 7 8 -` | 21 | 8 | [8] | [1, 7] | [8] |
| `1 2 - - - 6 7 8 -` | 21 | 9 | [1, 2, 6] | [2, 7] | [1, 8] |
| `1 2 - - - 6 7 8 -` | 21 | 10 | [1, 2, 7] | [2, 8] | [2, 8] |
| `1 2 - - 5 - 7 - 9` | 21 | 7 | [7] | [2, 5] | [7] |
| `1 2 - - 5 - 7 - 9` | 21 | 8 | [1, 2, 5] | [1, 7] | [1, 7] |
| `1 2 - - 5 - 7 - 9` | 21 | 9 | [9] | [2, 7] | [9] |
| `1 2 - - 5 - 7 - 9` | 21 | 10 | [1, 2, 7] | [1, 9] | [1, 9] |
| `1 2 - - 5 - 7 - 9` | 21 | 12 | [1, 2, 9] | [5, 7] | [5, 7] |
| `1 2 - 4 - - - 8 9` | 21 | 9 | [9] | [1, 8] | [9] |
| `1 2 - 4 - - - 8 9` | 21 | 10 | [1, 9] | [2, 8] | [1, 9] |
| `1 2 - 4 - - - 8 9` | 21 | 11 | [1, 2, 8] | [2, 9] | [2, 9] |
| `1 2 - 4 - - - 8 9` | 21 | 12 | [1, 2, 9] | [4, 8] | [4, 8] |
| `1 2 3 - 5 6 7 - -` | 21 | 3 | [1, 2] | [3] | [3] |
| `1 2 3 - 5 6 7 - -` | 21 | 5 | [5] | [2, 3] | [5] |
| `1 2 3 - 5 6 7 - -` | 21 | 6 | [6] | [1, 2, 3] | [6] |
| `1 2 3 - 5 6 7 - -` | 21 | 7 | [7] | [1, 6] | [7] |
| `1 2 3 - 5 6 7 - -` | 21 | 8 | [2, 6] | [3, 5] | [1, 7] |
| `1 2 3 - 5 6 7 - -` | 21 | 9 | [2, 7] | [1, 2, 6] | [2, 7] |
| `1 2 3 - 5 6 7 - -` | 21 | 10 | [3, 7] | [1, 3, 6] | [3, 7] |
| `1 2 3 - 5 6 7 - -` | 21 | 11 | [5, 6] | [2, 3, 6] | [5, 6] |
| `1 2 3 - 5 6 7 - -` | 21 | 12 | [5, 7] | [2, 3, 7] | [5, 7] |
| `1 2 3 4 - 6 - 8 -` | 21 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 4 - 6 - 8 -` | 21 | 4 | [4] | [1, 3] | [4] |
| `1 2 3 4 - 6 - 8 -` | 21 | 5 | [2, 3] | [1, 4] | [1, 4] |
| `1 2 3 4 - 6 - 8 -` | 21 | 6 | [6] | [1, 2, 3] | [6] |
| `1 2 3 4 - 6 - 8 -` | 21 | 7 | [1, 6] | [3, 4] | [1, 6] |
| `1 2 3 4 - 6 - 8 -` | 21 | 8 | [8] | [1, 3, 4] | [8] |
| `1 2 3 4 - 6 - 8 -` | 21 | 9 | [1, 8] | [1, 2, 6] | [1, 8] |
| `1 2 3 4 - 6 - 8 -` | 21 | 10 | [2, 8] | [1, 3, 6] | [2, 8] |
| `1 2 3 4 - 6 - 8 -` | 21 | 11 | [3, 8] | [1, 4, 6] | [3, 8] |
| `1 2 3 4 - 6 - 8 -` | 21 | 12 | [4, 8] | [2, 4, 6] | [4, 8] |
| `1 2 3 4 5 - - - 9` | 21 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 4 5 - - - 9` | 21 | 4 | [4] | [1, 3] | [4] |
| `1 2 3 4 5 - - - 9` | 21 | 5 | [5] | [2, 3] | [5] |
| `1 2 3 4 5 - - - 9` | 21 | 6 | [1, 5] | [1, 2, 3] | [1, 5] |
| `1 2 3 4 5 - - - 9` | 21 | 7 | [3, 4] | [1, 2, 4] | [2, 5] |
| `1 2 3 4 5 - - - 9` | 21 | 8 | [3, 5] | [1, 2, 5] | [3, 5] |
| `1 2 3 4 5 - - - 9` | 21 | 9 | [9] | [4, 5] | [9] |
| `1 2 3 4 5 - - - 9` | 21 | 10 | [1, 9] | [1, 4, 5] | [1, 9] |
| `1 2 3 4 5 - - - 9` | 21 | 11 | [2, 9] | [2, 4, 5] | [2, 9] |
| `1 2 3 4 5 - - - 9` | 21 | 12 | [3, 9] | [3, 4, 5] | [3, 9] |
| `- - 3 - 5 - 7 8 -` | 22 | 8 | [3, 5] | [8] | [8] |
| `- - 3 - 5 6 - - 9` | 22 | 9 | [3, 6] | [9] | [9] |
| `- - 3 4 - - 7 - 9` | 22 | 7 | [3, 4] | [7] | [7] |
| `- 2 - - - 6 7 8 -` | 22 | 8 | [2, 6] | [8] | [8] |
| `- 2 - - 5 - 7 - 9` | 22 | 7 | [2, 5] | [7] | [7] |
| `- 2 - - 5 - 7 - 9` | 22 | 9 | [2, 7] | [9] | [9] |
| `- 2 3 - 5 6 7 - -` | 22 | 5 | [5] | [2, 3] | [5] |
| `- 2 3 - 5 6 7 - -` | 22 | 7 | [7] | [2, 5] | [7] |
| `- 2 3 - 5 6 7 - -` | 22 | 8 | [3, 5] | [2, 6] | [2, 6] |
| `- 2 3 - 5 6 7 - -` | 22 | 9 | [2, 7] | [3, 6] | [2, 7] |
| `- 2 3 - 5 6 7 - -` | 22 | 10 | [2, 3, 5] | [3, 7] | [3, 7] |
| `- 2 3 - 5 6 7 - -` | 22 | 11 | [2, 3, 6] | [5, 6] | [5, 6] |
| `- 2 3 - 5 6 7 - -` | 22 | 12 | [2, 3, 7] | [5, 7] | [5, 7] |
| `- 2 3 4 - 6 - 8 -` | 22 | 6 | [6] | [2, 4] | [6] |
| `- 2 3 4 - 6 - 8 -` | 22 | 8 | [8] | [2, 6] | [8] |
| `- 2 3 4 - 6 - 8 -` | 22 | 9 | [2, 3, 4] | [3, 6] | [3, 6] |
| `- 2 3 4 - 6 - 8 -` | 22 | 10 | [2, 8] | [4, 6] | [2, 8] |
| `- 2 3 4 - 6 - 8 -` | 22 | 11 | [2, 3, 6] | [3, 8] | [3, 8] |
| `- 2 3 4 - 6 - 8 -` | 22 | 12 | [2, 4, 6] | [4, 8] | [4, 8] |
| `- 2 3 4 5 - - - 9` | 22 | 5 | [5] | [2, 3] | [5] |
| `- 2 3 4 5 - - - 9` | 22 | 7 | [3, 4] | [2, 5] | [2, 5] |
| `- 2 3 4 5 - - - 9` | 22 | 9 | [9] | [4, 5] | [9] |
| `- 2 3 4 5 - - - 9` | 22 | 11 | [2, 9] | [2, 4, 5] | [2, 9] |
| `- 2 3 4 5 - - - 9` | 22 | 12 | [3, 9] | [3, 4, 5] | [3, 9] |
| `1 - - - - 6 7 - 9` | 22 | 7 | [1, 6] | [7] | [7] |
| `1 - - - 5 - - 8 9` | 22 | 9 | [1, 8] | [9] | [9] |
| `1 - - 4 5 6 7 - -` | 22 | 5 | [5] | [1, 4] | [5] |
| `1 - - 4 5 6 7 - -` | 22 | 6 | [6] | [1, 5] | [6] |
| `1 - - 4 5 6 7 - -` | 22 | 10 | [1, 4, 5] | [4, 6] | [4, 6] |
| `1 - - 4 5 6 7 - -` | 22 | 11 | [1, 4, 6] | [4, 7] | [4, 7] |
| `1 - - 4 5 6 7 - -` | 22 | 12 | [1, 4, 7] | [5, 7] | [5, 7] |
| `1 - 3 - 5 6 - 8 -` | 22 | 6 | [1, 5] | [6] | [6] |
| `1 - 3 - 5 6 - 8 -` | 22 | 8 | [8] | [3, 5] | [8] |
| `1 - 3 - 5 6 - 8 -` | 22 | 9 | [1, 3, 5] | [3, 6] | [1, 8] |
| `1 - 3 - 5 6 - 8 -` | 22 | 11 | [3, 8] | [5, 6] | [3, 8] |
| `1 - 3 - 5 6 - 8 -` | 22 | 12 | [1, 3, 8] | [1, 5, 6] | [1, 3, 8] |
| `1 - 3 4 - - 7 8 -` | 22 | 4 | [1, 3] | [4] | [4] |
| `1 - 3 4 - - 7 8 -` | 22 | 7 | [7] | [3, 4] | [7] |
| `1 - 3 4 - - 7 8 -` | 22 | 8 | [1, 3, 4] | [8] | [8] |
| `1 - 3 4 - - 7 8 -` | 22 | 11 | [1, 3, 7] | [4, 7] | [3, 8] |
| `1 - 3 4 - - 7 8 -` | 22 | 12 | [1, 3, 8] | [4, 8] | [4, 8] |
| `1 - 3 4 - 6 - - 9` | 22 | 4 | [1, 3] | [4] | [4] |
| `1 - 3 4 - 6 - - 9` | 22 | 7 | [3, 4] | [1, 6] | [1, 6] |
| `1 - 3 4 - 6 - - 9` | 22 | 9 | [9] | [3, 6] | [9] |
| `1 - 3 4 - 6 - - 9` | 22 | 10 | [1, 3, 6] | [4, 6] | [1, 9] |
| `1 2 - - 5 - 7 8 -` | 22 | 7 | [7] | [2, 5] | [7] |
| `1 2 - - 5 - 7 8 -` | 22 | 8 | [1, 2, 5] | [1, 7] | [8] |
| `1 2 - - 5 - 7 8 -` | 22 | 9 | [2, 7] | [1, 8] | [1, 8] |
| `1 2 - - 5 - 7 8 -` | 22 | 10 | [1, 2, 7] | [2, 8] | [2, 8] |
| `1 2 - - 5 6 - - 9` | 22 | 6 | [6] | [1, 5] | [6] |
| `1 2 - - 5 6 - - 9` | 22 | 7 | [2, 5] | [1, 6] | [1, 6] |
| `1 2 - - 5 6 - - 9` | 22 | 8 | [1, 2, 5] | [2, 6] | [2, 6] |
| `1 2 - - 5 6 - - 9` | 22 | 9 | [1, 2, 6] | [9] | [9] |
| `1 2 - - 5 6 - - 9` | 22 | 11 | [2, 9] | [5, 6] | [2, 9] |
| `1 2 - - 5 6 - - 9` | 22 | 12 | [1, 2, 9] | [1, 5, 6] | [1, 2, 9] |
| `1 2 - 4 - - 7 - 9` | 22 | 7 | [1, 2, 4] | [7] | [7] |
| `1 2 - 4 - - 7 - 9` | 22 | 9 | [9] | [2, 7] | [9] |
| `1 2 - 4 - - 7 - 9` | 22 | 10 | [1, 2, 7] | [1, 9] | [1, 9] |
| `1 2 - 4 - - 7 - 9` | 22 | 11 | [2, 9] | [4, 7] | [2, 9] |
| `1 2 - 4 - - 7 - 9` | 22 | 12 | [1, 2, 9] | [1, 4, 7] | [1, 2, 9] |
| `1 2 3 - - - - 8 9` | 22 | 3 | [1, 2] | [3] | [3] |
| `1 2 3 - - - - 8 9` | 22 | 9 | [9] | [1, 8] | [9] |
| `1 2 3 - - - - 8 9` | 22 | 10 | [1, 9] | [2, 8] | [1, 9] |
| `1 2 3 - - - - 8 9` | 22 | 11 | [1, 2, 8] | [3, 8] | [2, 9] |
| `1 2 3 - - - - 8 9` | 22 | 12 | [1, 2, 9] | [3, 9] | [3, 9] |
| `1 2 3 4 - 6 7 - -` | 22 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 4 - 6 7 - -` | 22 | 4 | [4] | [1, 3] | [4] |
| `1 2 3 4 - 6 7 - -` | 22 | 5 | [1, 4] | [2, 3] | [1, 4] |
| `1 2 3 4 - 6 7 - -` | 22 | 6 | [6] | [1, 2, 3] | [6] |
| `1 2 3 4 - 6 7 - -` | 22 | 7 | [7] | [3, 4] | [7] |
| `1 2 3 4 - 6 7 - -` | 22 | 8 | [1, 7] | [2, 6] | [1, 7] |
| `1 2 3 4 - 6 7 - -` | 22 | 9 | [3, 6] | [1, 2, 6] | [2, 7] |
| `1 2 3 4 - 6 7 - -` | 22 | 10 | [3, 7] | [1, 3, 6] | [3, 7] |
| `1 2 3 4 - 6 7 - -` | 22 | 11 | [4, 7] | [1, 3, 7] | [4, 7] |
| `1 2 3 4 - 6 7 - -` | 22 | 12 | [1, 2, 3, 6] | [2, 4, 6] | [1, 4, 7] |
| `1 2 3 4 5 - - 8 -` | 22 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 4 5 - - 8 -` | 22 | 5 | [5] | [1, 4] | [5] |
| `1 2 3 4 5 - - 8 -` | 22 | 6 | [1, 5] | [2, 4] | [1, 5] |
| `1 2 3 4 5 - - 8 -` | 22 | 7 | [3, 4] | [1, 2, 4] | [2, 5] |
| `1 2 3 4 5 - - 8 -` | 22 | 8 | [8] | [1, 2, 5] | [8] |
| `1 2 3 4 5 - - 8 -` | 22 | 9 | [1, 8] | [1, 3, 5] | [1, 8] |
| `1 2 3 4 5 - - 8 -` | 22 | 10 | [2, 8] | [1, 4, 5] | [2, 8] |
| `1 2 3 4 5 - - 8 -` | 22 | 11 | [3, 8] | [2, 4, 5] | [3, 8] |
| `1 2 3 4 5 - - 8 -` | 22 | 12 | [4, 8] | [3, 4, 5] | [4, 8] |
| `- - - 4 5 6 7 - -` | 23 | 11 | [4, 7] | [5, 6] | [4, 7] |
| `- - 3 - 5 6 - 8 -` | 23 | 8 | [3, 5] | [8] | [8] |
| `- - 3 - 5 6 - 8 -` | 23 | 11 | [3, 8] | [5, 6] | [3, 8] |
| `- - 3 4 - - 7 8 -` | 23 | 7 | [3, 4] | [7] | [7] |
| `- - 3 4 - - 7 8 -` | 23 | 11 | [3, 8] | [4, 7] | [3, 8] |
| `- - 3 4 - 6 - - 9` | 23 | 9 | [3, 6] | [9] | [9] |
| `- 2 - - 5 - 7 8 -` | 23 | 7 | [2, 5] | [7] | [7] |
| `- 2 - - 5 6 - - 9` | 23 | 11 | [2, 9] | [5, 6] | [2, 9] |
| `- 2 - 4 - - 7 - 9` | 23 | 9 | [2, 7] | [9] | [9] |
| `- 2 - 4 - - 7 - 9` | 23 | 11 | [2, 9] | [4, 7] | [2, 9] |
| `- 2 3 - - - - 8 9` | 23 | 11 | [2, 9] | [3, 8] | [2, 9] |
| `- 2 3 4 - 6 7 - -` | 23 | 6 | [6] | [2, 4] | [6] |
| `- 2 3 4 - 6 7 - -` | 23 | 7 | [7] | [3, 4] | [7] |
| `- 2 3 4 - 6 7 - -` | 23 | 9 | [2, 3, 4] | [3, 6] | [2, 7] |
| `- 2 3 4 - 6 7 - -` | 23 | 10 | [4, 6] | [3, 7] | [3, 7] |
| `- 2 3 4 - 6 7 - -` | 23 | 11 | [2, 3, 6] | [4, 7] | [4, 7] |
| `- 2 3 4 - 6 7 - -` | 23 | 12 | [2, 3, 7] | [2, 4, 6] | [2, 3, 7] |
| `- 2 3 4 5 - - 8 -` | 23 | 5 | [5] | [2, 3] | [5] |
| `- 2 3 4 5 - - 8 -` | 23 | 7 | [3, 4] | [2, 5] | [2, 5] |
| `- 2 3 4 5 - - 8 -` | 23 | 8 | [8] | [3, 5] | [8] |
| `- 2 3 4 5 - - 8 -` | 23 | 9 | [2, 3, 4] | [4, 5] | [4, 5] |
| `- 2 3 4 5 - - 8 -` | 23 | 10 | [2, 8] | [2, 3, 5] | [2, 8] |
| `- 2 3 4 5 - - 8 -` | 23 | 11 | [3, 8] | [2, 4, 5] | [3, 8] |
| `- 2 3 4 5 - - 8 -` | 23 | 12 | [3, 4, 5] | [4, 8] | [4, 8] |
| `1 - - - - 6 7 8 -` | 23 | 7 | [1, 6] | [7] | [7] |
| `1 - - - - 6 7 8 -` | 23 | 8 | [1, 7] | [8] | [8] |
| `1 - - 4 - - - 8 9` | 23 | 9 | [1, 8] | [9] | [9] |
| `1 - 3 - 5 6 7 - -` | 23 | 6 | [6] | [1, 5] | [6] |
| `1 - 3 - 5 6 7 - -` | 23 | 7 | [7] | [1, 6] | [7] |
| `1 - 3 - 5 6 7 - -` | 23 | 8 | [1, 7] | [3, 5] | [1, 7] |
| `1 - 3 - 5 6 7 - -` | 23 | 9 | [1, 3, 5] | [3, 6] | [3, 6] |
| `1 - 3 - 5 6 7 - -` | 23 | 10 | [1, 3, 6] | [3, 7] | [3, 7] |
| `1 - 3 - 5 6 7 - -` | 23 | 11 | [1, 3, 7] | [5, 6] | [5, 6] |
| `1 - 3 - 5 6 7 - -` | 23 | 12 | [1, 5, 6] | [5, 7] | [5, 7] |
| `1 - 3 4 - 6 - 8 -` | 23 | 4 | [4] | [1, 3] | [4] |
| `1 - 3 4 - 6 - 8 -` | 23 | 7 | [3, 4] | [1, 6] | [1, 6] |
| `1 - 3 4 - 6 - 8 -` | 23 | 8 | [8] | [1, 3, 4] | [8] |
| `1 - 3 4 - 6 - 8 -` | 23 | 9 | [1, 8] | [3, 6] | [1, 8] |
| `1 - 3 4 - 6 - 8 -` | 23 | 10 | [1, 3, 6] | [4, 6] | [4, 6] |
| `1 - 3 4 - 6 - 8 -` | 23 | 11 | [1, 4, 6] | [3, 8] | [3, 8] |
| `1 - 3 4 - 6 - 8 -` | 23 | 12 | [1, 3, 8] | [4, 8] | [4, 8] |
| `1 - 3 4 5 - - - 9` | 23 | 4 | [4] | [1, 3] | [4] |
| `1 - 3 4 5 - - - 9` | 23 | 5 | [5] | [1, 4] | [5] |
| `1 - 3 4 5 - - - 9` | 23 | 8 | [1, 3, 4] | [3, 5] | [3, 5] |
| `1 - 3 4 5 - - - 9` | 23 | 9 | [9] | [4, 5] | [9] |
| `1 - 3 4 5 - - - 9` | 23 | 10 | [1, 9] | [1, 4, 5] | [1, 9] |
| `1 - 3 4 5 - - - 9` | 23 | 12 | [3, 4, 5] | [3, 9] | [3, 9] |
| `1 2 - - 5 6 - 8 -` | 23 | 6 | [6] | [1, 5] | [6] |
| `1 2 - - 5 6 - 8 -` | 23 | 7 | [2, 5] | [1, 6] | [1, 6] |
| `1 2 - - 5 6 - 8 -` | 23 | 8 | [1, 2, 5] | [2, 6] | [8] |
| `1 2 - - 5 6 - 8 -` | 23 | 9 | [1, 2, 6] | [1, 8] | [1, 8] |
| `1 2 - - 5 6 - 8 -` | 23 | 11 | [1, 2, 8] | [5, 6] | [5, 6] |
| `1 2 - 4 - - 7 8 -` | 23 | 7 | [7] | [1, 2, 4] | [7] |
| `1 2 - 4 - - 7 8 -` | 23 | 8 | [8] | [1, 7] | [8] |
| `1 2 - 4 - - 7 8 -` | 23 | 9 | [1, 8] | [2, 7] | [1, 8] |
| `1 2 - 4 - - 7 8 -` | 23 | 10 | [1, 2, 7] | [2, 8] | [2, 8] |
| `1 2 - 4 - - 7 8 -` | 23 | 11 | [1, 2, 8] | [4, 7] | [4, 7] |
| `1 2 - 4 - - 7 8 -` | 23 | 12 | [1, 4, 7] | [4, 8] | [4, 8] |
| `1 2 - 4 - 6 - - 9` | 23 | 6 | [6] | [2, 4] | [6] |
| `1 2 - 4 - 6 - - 9` | 23 | 7 | [1, 2, 4] | [1, 6] | [1, 6] |
| `1 2 - 4 - 6 - - 9` | 23 | 9 | [9] | [1, 2, 6] | [9] |
| `1 2 - 4 - 6 - - 9` | 23 | 10 | [1, 9] | [4, 6] | [1, 9] |
| `1 2 - 4 - 6 - - 9` | 23 | 11 | [1, 4, 6] | [2, 9] | [2, 9] |
| `1 2 - 4 - 6 - - 9` | 23 | 12 | [1, 2, 9] | [2, 4, 6] | [1, 2, 9] |
| `1 2 3 - - - 7 - 9` | 23 | 3 | [1, 2] | [3] | [3] |
| `1 2 3 - - - 7 - 9` | 23 | 9 | [9] | [2, 7] | [9] |
| `1 2 3 - - - 7 - 9` | 23 | 10 | [1, 2, 7] | [3, 7] | [1, 9] |
| `1 2 3 - - - 7 - 9` | 23 | 11 | [1, 3, 7] | [2, 9] | [2, 9] |
| `1 2 3 - - - 7 - 9` | 23 | 12 | [1, 2, 9] | [3, 9] | [3, 9] |
| `1 2 3 4 5 - 7 - -` | 23 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 4 5 - 7 - -` | 23 | 4 | [4] | [1, 3] | [4] |
| `1 2 3 4 5 - 7 - -` | 23 | 5 | [5] | [1, 4] | [5] |
| `1 2 3 4 5 - 7 - -` | 23 | 6 | [2, 4] | [1, 2, 3] | [1, 5] |
| `1 2 3 4 5 - 7 - -` | 23 | 7 | [7] | [2, 5] | [7] |
| `1 2 3 4 5 - 7 - -` | 23 | 8 | [1, 7] | [1, 2, 5] | [1, 7] |
| `1 2 3 4 5 - 7 - -` | 23 | 9 | [4, 5] | [1, 3, 5] | [2, 7] |
| `1 2 3 4 5 - 7 - -` | 23 | 10 | [3, 7] | [1, 4, 5] | [3, 7] |
| `1 2 3 4 5 - 7 - -` | 23 | 11 | [4, 7] | [2, 4, 5] | [4, 7] |
| `1 2 3 4 5 - 7 - -` | 23 | 12 | [5, 7] | [2, 3, 7] | [5, 7] |
| `- - 3 4 5 - - - 9` | 24 | 9 | [9] | [4, 5] | [9] |
| `- - 3 4 5 - - - 9` | 24 | 12 | [3, 4, 5] | [3, 9] | [3, 9] |
| `- 2 - - 5 6 - 8 -` | 24 | 8 | [2, 6] | [8] | [8] |
| `- 2 - 4 - 6 - - 9` | 24 | 6 | [2, 4] | [6] | [6] |
| `- 2 3 - - - 7 - 9` | 24 | 9 | [2, 7] | [9] | [9] |
| `- 2 3 - - - 7 - 9` | 24 | 12 | [2, 3, 7] | [3, 9] | [3, 9] |
| `- 2 3 4 5 - 7 - -` | 24 | 5 | [5] | [2, 3] | [5] |
| `- 2 3 4 5 - 7 - -` | 24 | 7 | [7] | [2, 5] | [7] |
| `- 2 3 4 5 - 7 - -` | 24 | 9 | [2, 3, 4] | [4, 5] | [2, 7] |
| `- 2 3 4 5 - 7 - -` | 24 | 10 | [2, 3, 5] | [3, 7] | [3, 7] |
| `- 2 3 4 5 - 7 - -` | 24 | 11 | [2, 4, 5] | [4, 7] | [4, 7] |
| `- 2 3 4 5 - 7 - -` | 24 | 12 | [2, 3, 7] | [5, 7] | [5, 7] |
| `1 - - - 5 - 7 8 -` | 24 | 8 | [1, 7] | [8] | [8] |
| `1 - - - 5 6 - - 9` | 24 | 6 | [1, 5] | [6] | [6] |
| `1 - 3 - - - - 8 9` | 24 | 9 | [1, 8] | [9] | [9] |
| `1 - 3 - - - - 8 9` | 24 | 12 | [1, 3, 8] | [3, 9] | [3, 9] |
| `1 - 3 4 - 6 7 - -` | 24 | 4 | [4] | [1, 3] | [4] |
| `1 - 3 4 - 6 7 - -` | 24 | 7 | [7] | [1, 6] | [7] |
| `1 - 3 4 - 6 7 - -` | 24 | 8 | [1, 3, 4] | [1, 7] | [1, 7] |
| `1 - 3 4 - 6 7 - -` | 24 | 10 | [1, 3, 6] | [4, 6] | [3, 7] |
| `1 - 3 4 - 6 7 - -` | 24 | 11 | [1, 3, 7] | [4, 7] | [4, 7] |
| `1 - 3 4 5 - - 8 -` | 24 | 4 | [1, 3] | [4] | [4] |
| `1 - 3 4 5 - - 8 -` | 24 | 5 | [5] | [1, 4] | [5] |
| `1 - 3 4 5 - - 8 -` | 24 | 8 | [8] | [3, 5] | [8] |
| `1 - 3 4 5 - - 8 -` | 24 | 9 | [1, 8] | [4, 5] | [1, 8] |
| `1 - 3 4 5 - - 8 -` | 24 | 12 | [1, 3, 8] | [4, 8] | [4, 8] |
| `1 2 - - 5 6 7 - -` | 24 | 6 | [6] | [1, 5] | [6] |
| `1 2 - - 5 6 7 - -` | 24 | 7 | [7] | [1, 6] | [7] |
| `1 2 - - 5 6 7 - -` | 24 | 8 | [1, 2, 5] | [2, 6] | [1, 7] |
| `1 2 - - 5 6 7 - -` | 24 | 9 | [1, 2, 6] | [2, 7] | [2, 7] |
| `1 2 - - 5 6 7 - -` | 24 | 12 | [1, 5, 6] | [5, 7] | [5, 7] |
| `1 2 - 4 - 6 - 8 -` | 24 | 6 | [6] | [2, 4] | [6] |
| `1 2 - 4 - 6 - 8 -` | 24 | 7 | [1, 2, 4] | [1, 6] | [1, 6] |
| `1 2 - 4 - 6 - 8 -` | 24 | 8 | [8] | [2, 6] | [8] |
| `1 2 - 4 - 6 - 8 -` | 24 | 9 | [1, 2, 6] | [1, 8] | [1, 8] |
| `1 2 - 4 - 6 - 8 -` | 24 | 10 | [2, 8] | [4, 6] | [2, 8] |
| `1 2 - 4 - 6 - 8 -` | 24 | 11 | [1, 2, 8] | [1, 4, 6] | [1, 2, 8] |
| `1 2 - 4 - 6 - 8 -` | 24 | 12 | [2, 4, 6] | [4, 8] | [4, 8] |
| `1 2 - 4 5 - - - 9` | 24 | 5 | [5] | [1, 4] | [5] |
| `1 2 - 4 5 - - - 9` | 24 | 6 | [2, 4] | [1, 5] | [1, 5] |
| `1 2 - 4 5 - - - 9` | 24 | 7 | [1, 2, 4] | [2, 5] | [2, 5] |
| `1 2 - 4 5 - - - 9` | 24 | 9 | [9] | [4, 5] | [9] |
| `1 2 - 4 5 - - - 9` | 24 | 10 | [1, 9] | [1, 4, 5] | [1, 9] |
| `1 2 - 4 5 - - - 9` | 24 | 11 | [2, 4, 5] | [2, 9] | [2, 9] |
| `1 2 - 4 5 - - - 9` | 24 | 12 | [1, 2, 4, 5] | [1, 2, 9] | [1, 2, 9] |
| `1 2 3 - - - 7 8 -` | 24 | 3 | [1, 2] | [3] | [3] |
| `1 2 3 - - - 7 8 -` | 24 | 8 | [8] | [1, 7] | [8] |
| `1 2 3 - - - 7 8 -` | 24 | 9 | [1, 8] | [2, 7] | [1, 8] |
| `1 2 3 - - - 7 8 -` | 24 | 10 | [1, 2, 7] | [3, 7] | [2, 8] |
| `1 2 3 - - - 7 8 -` | 24 | 11 | [1, 2, 8] | [3, 8] | [3, 8] |
| `1 2 3 - - - 7 8 -` | 24 | 12 | [1, 3, 8] | [2, 3, 7] | [1, 3, 8] |
| `1 2 3 - - 6 - - 9` | 24 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 - - 6 - - 9` | 24 | 6 | [1, 2, 3] | [6] | [6] |
| `1 2 3 - - 6 - - 9` | 24 | 9 | [9] | [3, 6] | [9] |
| `1 2 3 - - 6 - - 9` | 24 | 10 | [1, 3, 6] | [1, 9] | [1, 9] |
| `1 2 3 - - 6 - - 9` | 24 | 11 | [2, 3, 6] | [2, 9] | [2, 9] |
| `1 2 3 - - 6 - - 9` | 24 | 12 | [1, 2, 3, 6] | [3, 9] | [3, 9] |
| `1 2 3 4 5 6 - - -` | 24 | 5 | [5] | [2, 3] | [5] |
| `1 2 3 4 5 6 - - -` | 24 | 6 | [6] | [1, 2, 3] | [6] |
| `1 2 3 4 5 6 - - -` | 24 | 7 | [1, 6] | [3, 4] | [1, 6] |
| `1 2 3 4 5 6 - - -` | 24 | 8 | [3, 5] | [1, 2, 5] | [2, 6] |
| `1 2 3 4 5 6 - - -` | 24 | 9 | [3, 6] | [1, 3, 5] | [3, 6] |
| `1 2 3 4 5 6 - - -` | 24 | 10 | [4, 6] | [1, 4, 5] | [4, 6] |
| `1 2 3 4 5 6 - - -` | 24 | 11 | [5, 6] | [2, 3, 6] | [5, 6] |
| `1 2 3 4 5 6 - - -` | 24 | 12 | [1, 2, 3, 6] | [3, 4, 5] | [1, 5, 6] |
| `- - 3 4 - 6 7 - -` | 25 | 7 | [3, 4] | [7] | [7] |
| `- - 3 4 - 6 7 - -` | 25 | 10 | [3, 7] | [4, 6] | [3, 7] |
| `- - 3 4 5 - - 8 -` | 25 | 8 | [8] | [3, 5] | [8] |
| `- - 3 4 5 - - 8 -` | 25 | 12 | [3, 4, 5] | [4, 8] | [4, 8] |
| `- 2 - - 5 6 7 - -` | 25 | 7 | [2, 5] | [7] | [7] |
| `- 2 - 4 - 6 - 8 -` | 25 | 6 | [2, 4] | [6] | [6] |
| `- 2 - 4 - 6 - 8 -` | 25 | 8 | [2, 6] | [8] | [8] |
| `- 2 - 4 - 6 - 8 -` | 25 | 10 | [2, 8] | [4, 6] | [2, 8] |
| `- 2 - 4 - 6 - 8 -` | 25 | 12 | [2, 4, 6] | [4, 8] | [4, 8] |
| `- 2 - 4 5 - - - 9` | 25 | 9 | [9] | [4, 5] | [9] |
| `- 2 - 4 5 - - - 9` | 25 | 11 | [2, 4, 5] | [2, 9] | [2, 9] |
| `- 2 3 - - - 7 8 -` | 25 | 10 | [2, 8] | [3, 7] | [2, 8] |
| `- 2 3 - - 6 - - 9` | 25 | 9 | [3, 6] | [9] | [9] |
| `- 2 3 - - 6 - - 9` | 25 | 11 | [2, 3, 6] | [2, 9] | [2, 9] |
| `- 2 3 4 5 6 - - -` | 25 | 5 | [5] | [2, 3] | [5] |
| `- 2 3 4 5 6 - - -` | 25 | 6 | [6] | [2, 4] | [6] |
| `- 2 3 4 5 6 - - -` | 25 | 7 | [3, 4] | [2, 5] | [2, 5] |
| `- 2 3 4 5 6 - - -` | 25 | 8 | [2, 6] | [3, 5] | [2, 6] |
| `- 2 3 4 5 6 - - -` | 25 | 9 | [2, 3, 4] | [4, 5] | [3, 6] |
| `- 2 3 4 5 6 - - -` | 25 | 10 | [2, 3, 5] | [4, 6] | [4, 6] |
| `- 2 3 4 5 6 - - -` | 25 | 11 | [2, 3, 6] | [5, 6] | [5, 6] |
| `- 2 3 4 5 6 - - -` | 25 | 12 | [2, 4, 6] | [3, 4, 5] | [2, 4, 6] |
| `1 - - - 5 6 - 8 -` | 25 | 6 | [1, 5] | [6] | [6] |
| `1 - - 4 - - 7 8 -` | 25 | 8 | [1, 7] | [8] | [8] |
| `1 - - 4 - - 7 8 -` | 25 | 12 | [1, 4, 7] | [4, 8] | [4, 8] |
| `1 - - 4 - 6 - - 9` | 25 | 10 | [1, 9] | [4, 6] | [1, 9] |
| `1 - 3 - - - 7 - 9` | 25 | 10 | [1, 9] | [3, 7] | [1, 9] |
| `1 - 3 4 5 - 7 - -` | 25 | 4 | [4] | [1, 3] | [4] |
| `1 - 3 4 5 - 7 - -` | 25 | 5 | [1, 4] | [5] | [5] |
| `1 - 3 4 5 - 7 - -` | 25 | 7 | [7] | [3, 4] | [7] |
| `1 - 3 4 5 - 7 - -` | 25 | 8 | [1, 3, 4] | [3, 5] | [1, 7] |
| `1 - 3 4 5 - 7 - -` | 25 | 9 | [1, 3, 5] | [4, 5] | [4, 5] |
| `1 - 3 4 5 - 7 - -` | 25 | 10 | [1, 4, 5] | [3, 7] | [3, 7] |
| `1 - 3 4 5 - 7 - -` | 25 | 11 | [1, 3, 7] | [4, 7] | [4, 7] |
| `1 - 3 4 5 - 7 - -` | 25 | 12 | [1, 4, 7] | [5, 7] | [5, 7] |
| `1 2 - - - - - 8 9` | 25 | 9 | [1, 8] | [9] | [9] |
| `1 2 - - - - - 8 9` | 25 | 10 | [1, 9] | [2, 8] | [1, 9] |
| `1 2 - - - - - 8 9` | 25 | 11 | [1, 2, 8] | [2, 9] | [2, 9] |
| `1 2 - 4 - 6 7 - -` | 25 | 6 | [6] | [2, 4] | [6] |
| `1 2 - 4 - 6 7 - -` | 25 | 7 | [7] | [1, 6] | [7] |
| `1 2 - 4 - 6 7 - -` | 25 | 8 | [2, 6] | [1, 7] | [1, 7] |
| `1 2 - 4 - 6 7 - -` | 25 | 9 | [1, 2, 6] | [2, 7] | [2, 7] |
| `1 2 - 4 - 6 7 - -` | 25 | 10 | [1, 2, 7] | [4, 6] | [4, 6] |
| `1 2 - 4 - 6 7 - -` | 25 | 11 | [1, 4, 6] | [4, 7] | [4, 7] |
| `1 2 - 4 - 6 7 - -` | 25 | 12 | [1, 4, 7] | [2, 4, 6] | [1, 4, 7] |
| `1 2 - 4 5 - - 8 -` | 25 | 5 | [5] | [1, 4] | [5] |
| `1 2 - 4 5 - - 8 -` | 25 | 6 | [2, 4] | [1, 5] | [1, 5] |
| `1 2 - 4 5 - - 8 -` | 25 | 7 | [1, 2, 4] | [2, 5] | [2, 5] |
| `1 2 - 4 5 - - 8 -` | 25 | 8 | [8] | [1, 2, 5] | [8] |
| `1 2 - 4 5 - - 8 -` | 25 | 9 | [1, 8] | [4, 5] | [1, 8] |
| `1 2 - 4 5 - - 8 -` | 25 | 10 | [1, 4, 5] | [2, 8] | [2, 8] |
| `1 2 - 4 5 - - 8 -` | 25 | 11 | [1, 2, 8] | [2, 4, 5] | [1, 2, 8] |
| `1 2 - 4 5 - - 8 -` | 25 | 12 | [1, 2, 4, 5] | [4, 8] | [4, 8] |
| `1 2 3 - - 6 - 8 -` | 25 | 6 | [1, 2, 3] | [6] | [6] |
| `1 2 3 - - 6 - 8 -` | 25 | 8 | [8] | [2, 6] | [8] |
| `1 2 3 - - 6 - 8 -` | 25 | 9 | [1, 2, 6] | [3, 6] | [1, 8] |
| `1 2 3 - - 6 - 8 -` | 25 | 10 | [1, 3, 6] | [2, 8] | [2, 8] |
| `1 2 3 - - 6 - 8 -` | 25 | 11 | [1, 2, 8] | [3, 8] | [3, 8] |
| `1 2 3 - - 6 - 8 -` | 25 | 12 | [1, 2, 3, 6] | [1, 3, 8] | [1, 3, 8] |
| `1 2 3 - 5 - - - 9` | 25 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 - 5 - - - 9` | 25 | 5 | [2, 3] | [5] | [5] |
| `1 2 3 - 5 - - - 9` | 25 | 6 | [1, 2, 3] | [1, 5] | [1, 5] |
| `1 2 3 - 5 - - - 9` | 25 | 8 | [1, 2, 5] | [3, 5] | [3, 5] |
| `1 2 3 - 5 - - - 9` | 25 | 9 | [9] | [1, 3, 5] | [9] |
| `1 2 3 - 5 - - - 9` | 25 | 10 | [2, 3, 5] | [1, 9] | [1, 9] |
| `1 2 3 - 5 - - - 9` | 25 | 11 | [1, 2, 3, 5] | [2, 9] | [2, 9] |
| `1 2 3 - 5 - - - 9` | 25 | 12 | [1, 2, 9] | [3, 9] | [3, 9] |
| `- - 3 4 5 - 7 - -` | 26 | 7 | [3, 4] | [7] | [7] |
| `- - 3 4 5 - 7 - -` | 26 | 12 | [3, 4, 5] | [5, 7] | [5, 7] |
| `- 2 - 4 - 6 7 - -` | 26 | 6 | [2, 4] | [6] | [6] |
| `- 2 3 - - 6 - 8 -` | 26 | 8 | [2, 6] | [8] | [8] |
| `- 2 3 - - 6 - 8 -` | 26 | 11 | [2, 3, 6] | [3, 8] | [3, 8] |
| `- 2 3 - 5 - - - 9` | 26 | 5 | [2, 3] | [5] | [5] |
| `1 - - - 5 6 7 - -` | 26 | 6 | [1, 5] | [6] | [6] |
| `1 - - - 5 6 7 - -` | 26 | 7 | [1, 6] | [7] | [7] |
| `1 - - - 5 6 7 - -` | 26 | 12 | [1, 5, 6] | [5, 7] | [5, 7] |
| `1 - - 4 5 - - - 9` | 26 | 5 | [1, 4] | [5] | [5] |
| `1 - - 4 5 - - - 9` | 26 | 9 | [4, 5] | [9] | [9] |
| `1 - - 4 5 - - - 9` | 26 | 10 | [1, 4, 5] | [1, 9] | [1, 9] |
| `1 - 3 - - - 7 8 -` | 26 | 8 | [1, 7] | [8] | [8] |
| `1 - 3 - - - 7 8 -` | 26 | 11 | [1, 3, 7] | [3, 8] | [3, 8] |
| `1 - 3 - - 6 - - 9` | 26 | 9 | [3, 6] | [9] | [9] |
| `1 - 3 - - 6 - - 9` | 26 | 10 | [1, 3, 6] | [1, 9] | [1, 9] |
| `1 - 3 4 5 6 - - -` | 26 | 4 | [4] | [1, 3] | [4] |
| `1 - 3 4 5 6 - - -` | 26 | 5 | [5] | [1, 4] | [5] |
| `1 - 3 4 5 6 - - -` | 26 | 6 | [6] | [1, 5] | [6] |
| `1 - 3 4 5 6 - - -` | 26 | 7 | [1, 6] | [3, 4] | [1, 6] |
| `1 - 3 4 5 6 - - -` | 26 | 8 | [1, 3, 4] | [3, 5] | [3, 5] |
| `1 - 3 4 5 6 - - -` | 26 | 9 | [1, 3, 5] | [3, 6] | [3, 6] |
| `1 - 3 4 5 6 - - -` | 26 | 10 | [1, 3, 6] | [4, 6] | [4, 6] |
| `1 - 3 4 5 6 - - -` | 26 | 11 | [1, 4, 6] | [5, 6] | [5, 6] |
| `1 - 3 4 5 6 - - -` | 26 | 12 | [1, 5, 6] | [3, 4, 5] | [1, 5, 6] |
| `1 2 - - - - 7 - 9` | 26 | 9 | [2, 7] | [9] | [9] |
| `1 2 - - - - 7 - 9` | 26 | 10 | [1, 2, 7] | [1, 9] | [1, 9] |
| `1 2 - 4 5 - 7 - -` | 26 | 5 | [5] | [1, 4] | [5] |
| `1 2 - 4 5 - 7 - -` | 26 | 6 | [2, 4] | [1, 5] | [1, 5] |
| `1 2 - 4 5 - 7 - -` | 26 | 7 | [7] | [2, 5] | [7] |
| `1 2 - 4 5 - 7 - -` | 26 | 8 | [1, 2, 5] | [1, 7] | [1, 7] |
| `1 2 - 4 5 - 7 - -` | 26 | 10 | [1, 2, 7] | [1, 4, 5] | [1, 2, 7] |
| `1 2 - 4 5 - 7 - -` | 26 | 11 | [2, 4, 5] | [4, 7] | [4, 7] |
| `1 2 - 4 5 - 7 - -` | 26 | 12 | [1, 2, 4, 5] | [5, 7] | [5, 7] |
| `1 2 3 - - 6 7 - -` | 26 | 3 | [1, 2] | [3] | [3] |
| `1 2 3 - - 6 7 - -` | 26 | 6 | [6] | [1, 2, 3] | [6] |
| `1 2 3 - - 6 7 - -` | 26 | 7 | [7] | [1, 6] | [7] |
| `1 2 3 - - 6 7 - -` | 26 | 8 | [1, 7] | [2, 6] | [1, 7] |
| `1 2 3 - - 6 7 - -` | 26 | 9 | [1, 2, 6] | [3, 6] | [2, 7] |
| `1 2 3 - - 6 7 - -` | 26 | 10 | [1, 2, 7] | [3, 7] | [3, 7] |
| `1 2 3 - - 6 7 - -` | 26 | 11 | [1, 3, 7] | [2, 3, 6] | [1, 3, 7] |
| `1 2 3 - - 6 7 - -` | 26 | 12 | [1, 2, 3, 6] | [2, 3, 7] | [2, 3, 7] |
| `1 2 3 - 5 - - 8 -` | 26 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 - 5 - - 8 -` | 26 | 5 | [5] | [2, 3] | [5] |
| `1 2 3 - 5 - - 8 -` | 26 | 6 | [1, 2, 3] | [1, 5] | [1, 5] |
| `1 2 3 - 5 - - 8 -` | 26 | 8 | [8] | [3, 5] | [8] |
| `1 2 3 - 5 - - 8 -` | 26 | 9 | [1, 3, 5] | [1, 8] | [1, 8] |
| `1 2 3 - 5 - - 8 -` | 26 | 10 | [2, 3, 5] | [2, 8] | [2, 8] |
| `1 2 3 - 5 - - 8 -` | 26 | 11 | [1, 2, 3, 5] | [3, 8] | [3, 8] |
| `1 2 3 4 - - - - 9` | 26 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 4 - - - - 9` | 26 | 4 | [1, 3] | [4] | [4] |
| `1 2 3 4 - - - - 9` | 26 | 5 | [2, 3] | [1, 4] | [1, 4] |
| `1 2 3 4 - - - - 9` | 26 | 6 | [1, 2, 3] | [2, 4] | [2, 4] |
| `1 2 3 4 - - - - 9` | 26 | 7 | [1, 2, 4] | [3, 4] | [3, 4] |
| `1 2 3 4 - - - - 9` | 26 | 9 | [9] | [2, 3, 4] | [9] |
| `1 2 3 4 - - - - 9` | 26 | 10 | [1, 2, 3, 4] | [1, 9] | [1, 9] |
| `1 2 3 4 - - - - 9` | 26 | 12 | [1, 2, 9] | [3, 9] | [3, 9] |
| `- - - 4 5 - - - 9` | 27 | 9 | [4, 5] | [9] | [9] |
| `- - 3 - - 6 - - 9` | 27 | 9 | [3, 6] | [9] | [9] |
| `- - 3 4 5 6 - - -` | 27 | 9 | [3, 6] | [4, 5] | [3, 6] |
| `- 2 - - - - 7 - 9` | 27 | 9 | [2, 7] | [9] | [9] |
| `- 2 - 4 5 - 7 - -` | 27 | 7 | [2, 5] | [7] | [7] |
| `- 2 - 4 5 - 7 - -` | 27 | 9 | [2, 7] | [4, 5] | [2, 7] |
| `- 2 - 4 5 - 7 - -` | 27 | 11 | [2, 4, 5] | [4, 7] | [4, 7] |
| `- 2 3 - - 6 7 - -` | 27 | 9 | [2, 7] | [3, 6] | [2, 7] |
| `- 2 3 - 5 - - 8 -` | 27 | 5 | [2, 3] | [5] | [5] |
| `- 2 3 - 5 - - 8 -` | 27 | 8 | [3, 5] | [8] | [8] |
| `- 2 3 - 5 - - 8 -` | 27 | 10 | [2, 3, 5] | [2, 8] | [2, 8] |
| `- 2 3 4 - - - - 9` | 27 | 9 | [2, 3, 4] | [9] | [9] |
| `1 - - - - - - 8 9` | 27 | 9 | [1, 8] | [9] | [9] |
| `1 - - 4 - 6 7 - -` | 27 | 7 | [1, 6] | [7] | [7] |
| `1 - - 4 - 6 7 - -` | 27 | 11 | [1, 4, 6] | [4, 7] | [4, 7] |
| `1 - - 4 5 - - 8 -` | 27 | 5 | [1, 4] | [5] | [5] |
| `1 - - 4 5 - - 8 -` | 27 | 9 | [1, 8] | [4, 5] | [1, 8] |
| `1 - 3 - - 6 - 8 -` | 27 | 9 | [1, 8] | [3, 6] | [1, 8] |
| `1 - 3 - 5 - - - 9` | 27 | 9 | [1, 3, 5] | [9] | [9] |
| `1 2 - - - - 7 8 -` | 27 | 8 | [1, 7] | [8] | [8] |
| `1 2 - - - - 7 8 -` | 27 | 9 | [1, 8] | [2, 7] | [1, 8] |
| `1 2 - - - - 7 8 -` | 27 | 10 | [1, 2, 7] | [2, 8] | [2, 8] |
| `1 2 - - - 6 - - 9` | 27 | 9 | [1, 2, 6] | [9] | [9] |
| `1 2 - 4 5 6 - - -` | 27 | 5 | [5] | [1, 4] | [5] |
| `1 2 - 4 5 6 - - -` | 27 | 6 | [6] | [1, 5] | [6] |
| `1 2 - 4 5 6 - - -` | 27 | 7 | [1, 2, 4] | [2, 5] | [1, 6] |
| `1 2 - 4 5 6 - - -` | 27 | 8 | [1, 2, 5] | [2, 6] | [2, 6] |
| `1 2 - 4 5 6 - - -` | 27 | 9 | [1, 2, 6] | [4, 5] | [4, 5] |
| `1 2 - 4 5 6 - - -` | 27 | 10 | [1, 4, 5] | [4, 6] | [4, 6] |
| `1 2 - 4 5 6 - - -` | 27 | 11 | [1, 4, 6] | [5, 6] | [5, 6] |
| `1 2 - 4 5 6 - - -` | 27 | 12 | [1, 2, 4, 5] | [2, 4, 6] | [1, 5, 6] |
| `1 2 3 - 5 - 7 - -` | 27 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 - 5 - 7 - -` | 27 | 5 | [5] | [2, 3] | [5] |
| `1 2 3 - 5 - 7 - -` | 27 | 6 | [1, 2, 3] | [1, 5] | [1, 5] |
| `1 2 3 - 5 - 7 - -` | 27 | 7 | [7] | [2, 5] | [7] |
| `1 2 3 - 5 - 7 - -` | 27 | 8 | [1, 2, 5] | [3, 5] | [1, 7] |
| `1 2 3 - 5 - 7 - -` | 27 | 9 | [1, 3, 5] | [2, 7] | [2, 7] |
| `1 2 3 - 5 - 7 - -` | 27 | 10 | [1, 2, 7] | [3, 7] | [3, 7] |
| `1 2 3 - 5 - 7 - -` | 27 | 11 | [1, 2, 3, 5] | [1, 3, 7] | [1, 3, 7] |
| `1 2 3 - 5 - 7 - -` | 27 | 12 | [2, 3, 7] | [5, 7] | [5, 7] |
| `1 2 3 4 - - - 8 -` | 27 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 4 - - - 8 -` | 27 | 4 | [4] | [1, 3] | [4] |
| `1 2 3 4 - - - 8 -` | 27 | 5 | [2, 3] | [1, 4] | [1, 4] |
| `1 2 3 4 - - - 8 -` | 27 | 6 | [1, 2, 3] | [2, 4] | [2, 4] |
| `1 2 3 4 - - - 8 -` | 27 | 7 | [1, 2, 4] | [3, 4] | [3, 4] |
| `1 2 3 4 - - - 8 -` | 27 | 8 | [8] | [1, 3, 4] | [8] |
| `1 2 3 4 - - - 8 -` | 27 | 9 | [1, 8] | [2, 3, 4] | [1, 8] |
| `1 2 3 4 - - - 8 -` | 27 | 10 | [1, 2, 3, 4] | [2, 8] | [2, 8] |
| `1 2 3 4 - - - 8 -` | 27 | 11 | [1, 2, 8] | [3, 8] | [3, 8] |
| `1 2 3 4 - - - 8 -` | 27 | 12 | [1, 3, 8] | [4, 8] | [4, 8] |
| `- 2 - 4 5 6 - - -` | 28 | 6 | [2, 4] | [6] | [6] |
| `- 2 - 4 5 6 - - -` | 28 | 11 | [2, 4, 5] | [5, 6] | [5, 6] |
| `- 2 3 - 5 - 7 - -` | 28 | 5 | [2, 3] | [5] | [5] |
| `- 2 3 - 5 - 7 - -` | 28 | 7 | [2, 5] | [7] | [7] |
| `- 2 3 - 5 - 7 - -` | 28 | 10 | [2, 3, 5] | [3, 7] | [3, 7] |
| `- 2 3 - 5 - 7 - -` | 28 | 12 | [2, 3, 7] | [5, 7] | [5, 7] |
| `1 - - 4 5 - 7 - -` | 28 | 5 | [1, 4] | [5] | [5] |
| `1 - - 4 5 - 7 - -` | 28 | 12 | [1, 4, 7] | [5, 7] | [5, 7] |
| `1 - 3 - - 6 7 - -` | 28 | 7 | [1, 6] | [7] | [7] |
| `1 - 3 - - 6 7 - -` | 28 | 10 | [1, 3, 6] | [3, 7] | [3, 7] |
| `1 - 3 - 5 - - 8 -` | 28 | 8 | [3, 5] | [8] | [8] |
| `1 - 3 - 5 - - 8 -` | 28 | 9 | [1, 3, 5] | [1, 8] | [1, 8] |
| `1 - 3 4 - - - - 9` | 28 | 4 | [1, 3] | [4] | [4] |
| `1 2 - - - 6 - 8 -` | 28 | 8 | [2, 6] | [8] | [8] |
| `1 2 - - - 6 - 8 -` | 28 | 9 | [1, 2, 6] | [1, 8] | [1, 8] |
| `1 2 3 - 5 6 - - -` | 28 | 5 | [5] | [2, 3] | [5] |
| `1 2 3 - 5 6 - - -` | 28 | 6 | [6] | [1, 5] | [6] |
| `1 2 3 - 5 6 - - -` | 28 | 7 | [2, 5] | [1, 6] | [1, 6] |
| `1 2 3 - 5 6 - - -` | 28 | 8 | [1, 2, 5] | [3, 5] | [2, 6] |
| `1 2 3 - 5 6 - - -` | 28 | 9 | [1, 2, 6] | [3, 6] | [3, 6] |
| `1 2 3 - 5 6 - - -` | 28 | 10 | [1, 3, 6] | [2, 3, 5] | [1, 3, 6] |
| `1 2 3 - 5 6 - - -` | 28 | 11 | [1, 2, 3, 5] | [5, 6] | [5, 6] |
| `1 2 3 - 5 6 - - -` | 28 | 12 | [1, 2, 3, 6] | [1, 5, 6] | [1, 5, 6] |
| `1 2 3 4 - - 7 - -` | 28 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 4 - - 7 - -` | 28 | 4 | [4] | [1, 3] | [4] |
| `1 2 3 4 - - 7 - -` | 28 | 5 | [2, 3] | [1, 4] | [1, 4] |
| `1 2 3 4 - - 7 - -` | 28 | 6 | [1, 2, 3] | [2, 4] | [2, 4] |
| `1 2 3 4 - - 7 - -` | 28 | 7 | [7] | [3, 4] | [7] |
| `1 2 3 4 - - 7 - -` | 28 | 8 | [1, 3, 4] | [1, 7] | [1, 7] |
| `1 2 3 4 - - 7 - -` | 28 | 9 | [2, 3, 4] | [2, 7] | [2, 7] |
| `1 2 3 4 - - 7 - -` | 28 | 10 | [1, 2, 3, 4] | [3, 7] | [3, 7] |
| `1 2 3 4 - - 7 - -` | 28 | 11 | [1, 3, 7] | [4, 7] | [4, 7] |
| `1 2 3 4 - - 7 - -` | 28 | 12 | [1, 4, 7] | [2, 3, 7] | [1, 4, 7] |
| `- - 3 - 5 - - 8 -` | 29 | 8 | [3, 5] | [8] | [8] |
| `- 2 - - - 6 - 8 -` | 29 | 8 | [2, 6] | [8] | [8] |
| `- 2 3 - 5 6 - - -` | 29 | 5 | [2, 3] | [5] | [5] |
| `- 2 3 - 5 6 - - -` | 29 | 8 | [2, 6] | [3, 5] | [2, 6] |
| `- 2 3 - 5 6 - - -` | 29 | 11 | [2, 3, 6] | [5, 6] | [5, 6] |
| `- 2 3 4 - - 7 - -` | 29 | 7 | [3, 4] | [7] | [7] |
| `- 2 3 4 - - 7 - -` | 29 | 9 | [2, 3, 4] | [2, 7] | [2, 7] |
| `1 - - - - - 7 8 -` | 29 | 8 | [1, 7] | [8] | [8] |
| `1 - - 4 5 6 - - -` | 29 | 5 | [1, 4] | [5] | [5] |
| `1 - - 4 5 6 - - -` | 29 | 6 | [1, 5] | [6] | [6] |
| `1 - - 4 5 6 - - -` | 29 | 10 | [1, 4, 5] | [4, 6] | [4, 6] |
| `1 - - 4 5 6 - - -` | 29 | 11 | [1, 4, 6] | [5, 6] | [5, 6] |
| `1 - 3 - 5 - 7 - -` | 29 | 8 | [1, 7] | [3, 5] | [1, 7] |
| `1 - 3 4 - - - 8 -` | 29 | 4 | [1, 3] | [4] | [4] |
| `1 - 3 4 - - - 8 -` | 29 | 8 | [1, 3, 4] | [8] | [8] |
| `1 - 3 4 - - - 8 -` | 29 | 12 | [1, 3, 8] | [4, 8] | [4, 8] |
| `1 2 - - - 6 7 - -` | 29 | 7 | [1, 6] | [7] | [7] |
| `1 2 - - - 6 7 - -` | 29 | 8 | [1, 7] | [2, 6] | [1, 7] |
| `1 2 - - - 6 7 - -` | 29 | 9 | [1, 2, 6] | [2, 7] | [2, 7] |
| `1 2 - - 5 - - 8 -` | 29 | 8 | [1, 2, 5] | [8] | [8] |
| `1 2 3 4 - 6 - - -` | 29 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 4 - 6 - - -` | 29 | 4 | [4] | [1, 3] | [4] |
| `1 2 3 4 - 6 - - -` | 29 | 5 | [2, 3] | [1, 4] | [1, 4] |
| `1 2 3 4 - 6 - - -` | 29 | 6 | [6] | [2, 4] | [6] |
| `1 2 3 4 - 6 - - -` | 29 | 7 | [1, 2, 4] | [3, 4] | [1, 6] |
| `1 2 3 4 - 6 - - -` | 29 | 8 | [1, 3, 4] | [2, 6] | [2, 6] |
| `1 2 3 4 - 6 - - -` | 29 | 9 | [1, 2, 6] | [3, 6] | [3, 6] |
| `1 2 3 4 - 6 - - -` | 29 | 10 | [1, 2, 3, 4] | [4, 6] | [4, 6] |
| `1 2 3 4 - 6 - - -` | 29 | 11 | [1, 4, 6] | [2, 3, 6] | [1, 4, 6] |
| `1 2 3 4 - 6 - - -` | 29 | 12 | [1, 2, 3, 6] | [2, 4, 6] | [2, 4, 6] |
| `- 2 3 4 - 6 - - -` | 30 | 6 | [2, 4] | [6] | [6] |
| `- 2 3 4 - 6 - - -` | 30 | 9 | [2, 3, 4] | [3, 6] | [3, 6] |
| `1 - 3 - 5 6 - - -` | 30 | 6 | [1, 5] | [6] | [6] |
| `1 - 3 - 5 6 - - -` | 30 | 9 | [1, 3, 5] | [3, 6] | [3, 6] |
| `1 - 3 4 - - 7 - -` | 30 | 4 | [1, 3] | [4] | [4] |
| `1 - 3 4 - - 7 - -` | 30 | 7 | [3, 4] | [7] | [7] |
| `1 - 3 4 - - 7 - -` | 30 | 8 | [1, 3, 4] | [1, 7] | [1, 7] |
| `1 - 3 4 - - 7 - -` | 30 | 11 | [1, 3, 7] | [4, 7] | [4, 7] |
| `1 2 - - 5 - 7 - -` | 30 | 7 | [2, 5] | [7] | [7] |
| `1 2 - - 5 - 7 - -` | 30 | 8 | [1, 2, 5] | [1, 7] | [1, 7] |
| `1 2 3 - - - - - 9` | 30 | 3 | [1, 2] | [3] | [3] |
| `1 2 3 - - - - - 9` | 30 | 12 | [1, 2, 9] | [3, 9] | [3, 9] |
| `1 2 3 4 5 - - - -` | 30 | 3 | [3] | [1, 2] | [3] |
| `1 2 3 4 5 - - - -` | 30 | 4 | [4] | [1, 3] | [4] |
| `1 2 3 4 5 - - - -` | 30 | 5 | [5] | [2, 3] | [5] |
| `1 2 3 4 5 - - - -` | 30 | 6 | [1, 2, 3] | [2, 4] | [1, 5] |
| `1 2 3 4 5 - - - -` | 30 | 7 | [1, 2, 4] | [2, 5] | [2, 5] |
| `1 2 3 4 5 - - - -` | 30 | 8 | [1, 2, 5] | [3, 5] | [3, 5] |
| `1 2 3 4 5 - - - -` | 30 | 9 | [1, 3, 5] | [4, 5] | [4, 5] |
| `1 2 3 4 5 - - - -` | 30 | 10 | [1, 2, 3, 4] | [2, 3, 5] | [1, 4, 5] |
| `1 2 3 4 5 - - - -` | 30 | 11 | [1, 2, 3, 5] | [2, 4, 5] | [2, 4, 5] |
| `1 2 3 4 5 - - - -` | 30 | 12 | [1, 2, 4, 5] | [3, 4, 5] | [3, 4, 5] |
| `- - 3 4 - - 7 - -` | 31 | 7 | [3, 4] | [7] | [7] |
| `- 2 - - 5 - 7 - -` | 31 | 7 | [2, 5] | [7] | [7] |
| `- 2 3 4 5 - - - -` | 31 | 5 | [2, 3] | [5] | [5] |
| `- 2 3 4 5 - - - -` | 31 | 7 | [2, 5] | [3, 4] | [2, 5] |
| `- 2 3 4 5 - - - -` | 31 | 9 | [2, 3, 4] | [4, 5] | [4, 5] |
| `1 - - - - 6 7 - -` | 31 | 7 | [1, 6] | [7] | [7] |
| `1 - 3 4 - 6 - - -` | 31 | 4 | [1, 3] | [4] | [4] |
| `1 - 3 4 - 6 - - -` | 31 | 7 | [1, 6] | [3, 4] | [1, 6] |
| `1 - 3 4 - 6 - - -` | 31 | 10 | [1, 3, 6] | [4, 6] | [4, 6] |
| `1 2 - - 5 6 - - -` | 31 | 6 | [1, 5] | [6] | [6] |
| `1 2 - - 5 6 - - -` | 31 | 7 | [1, 6] | [2, 5] | [1, 6] |
| `1 2 - - 5 6 - - -` | 31 | 8 | [1, 2, 5] | [2, 6] | [2, 6] |
| `1 2 - 4 - - 7 - -` | 31 | 7 | [1, 2, 4] | [7] | [7] |
| `1 2 3 - - - - 8 -` | 31 | 3 | [1, 2] | [3] | [3] |
| `1 2 3 - - - - 8 -` | 31 | 11 | [1, 2, 8] | [3, 8] | [3, 8] |
| `1 - 3 4 5 - - - -` | 32 | 4 | [1, 3] | [4] | [4] |
| `1 - 3 4 5 - - - -` | 32 | 5 | [1, 4] | [5] | [5] |
| `1 - 3 4 5 - - - -` | 32 | 8 | [1, 3, 4] | [3, 5] | [3, 5] |
| `1 - 3 4 5 - - - -` | 32 | 9 | [1, 3, 5] | [4, 5] | [4, 5] |
| `1 2 - 4 - 6 - - -` | 32 | 6 | [2, 4] | [6] | [6] |
| `1 2 - 4 - 6 - - -` | 32 | 7 | [1, 2, 4] | [1, 6] | [1, 6] |
| `1 2 3 - - - 7 - -` | 32 | 3 | [1, 2] | [3] | [3] |
| `1 2 3 - - - 7 - -` | 32 | 10 | [1, 2, 7] | [3, 7] | [3, 7] |
| `- 2 - 4 - 6 - - -` | 33 | 6 | [2, 4] | [6] | [6] |
| `1 - - - 5 6 - - -` | 33 | 6 | [1, 5] | [6] | [6] |
| `1 2 - 4 5 - - - -` | 33 | 5 | [1, 4] | [5] | [5] |
| `1 2 - 4 5 - - - -` | 33 | 6 | [1, 5] | [2, 4] | [1, 5] |
| `1 2 - 4 5 - - - -` | 33 | 7 | [1, 2, 4] | [2, 5] | [2, 5] |
| `1 2 3 - - 6 - - -` | 33 | 3 | [1, 2] | [3] | [3] |
| `1 2 3 - - 6 - - -` | 33 | 6 | [1, 2, 3] | [6] | [6] |
| `1 2 3 - - 6 - - -` | 33 | 9 | [1, 2, 6] | [3, 6] | [3, 6] |
| `1 2 3 - 5 - - - -` | 34 | 3 | [1, 2] | [3] | [3] |
| `1 2 3 - 5 - - - -` | 34 | 5 | [2, 3] | [5] | [5] |
| `1 2 3 - 5 - - - -` | 34 | 6 | [1, 2, 3] | [1, 5] | [1, 5] |
| `1 2 3 - 5 - - - -` | 34 | 8 | [1, 2, 5] | [3, 5] | [3, 5] |
| `- 2 3 - 5 - - - -` | 35 | 5 | [2, 3] | [5] | [5] |
| `1 - - 4 5 - - - -` | 35 | 5 | [1, 4] | [5] | [5] |
| `1 2 3 4 - - - - -` | 35 | 3 | [1, 2] | [3] | [3] |
| `1 2 3 4 - - - - -` | 35 | 4 | [1, 3] | [4] | [4] |
| `1 2 3 4 - - - - -` | 35 | 5 | [1, 4] | [2, 3] | [1, 4] |
| `1 2 3 4 - - - - -` | 35 | 6 | [1, 2, 3] | [2, 4] | [2, 4] |
| `1 2 3 4 - - - - -` | 35 | 7 | [1, 2, 4] | [3, 4] | [3, 4] |
| `1 - 3 4 - - - - -` | 37 | 4 | [1, 3] | [4] | [4] |
| `1 2 3 - - - - - -` | 39 | 3 | [1, 2] | [3] | [3] |


### Exhibit G: Full Comparison of Moves for Optimal Strategy and Highest Tile Strategy

Note: traditional scoring system is used for this comparison

In [16]:
def exhibit_g(results: dict) -> Table:
    """a comparison of the optimal strategy and the highest tile strategy for the traditional scoring system"""
    exhibit_table = []
    results_subtable = results['traditional_move_table']
    es_max = results['es_traditional_max']
    es_highest_tile = results['es_traditional_highest_tile']
    for record in results_subtable:
        board = Board(record['Board Id'])
        exhibit_table.append({
            'Board (Remaining)': f'`{record["Board"]}`',
            'Board Sum': record['Sum'],
            'Roll': record['Roll'],
            'Max Move': record['Max Move'],
            'Max ES': f'{es_max[board]:.3f}',
            'Highest Tile Move': record['Highest Tile Move'],
            'Highest Tile ES': f'{es_highest_tile[board]:.3f}'
        })
    # sort by board sum ascending
    exhibit_table.sort(key=lambda record: (record['Board Sum'], record['Board (Remaining)'], record['Roll']))
    return exhibit_table


display_table(exhibit_g(full_results))

| Board (Remaining) | Board Sum | Roll | Max Move | Max ES | Highest Tile Move | Highest Tile ES |
| --- | --- | --- | --- | --- | --- | --- |
| `1 2 3 4 5 6 7 8 9` | 0 | 2 | [2] | 33.842 | [2] | 33.677 |
| `1 2 3 4 5 6 7 8 9` | 0 | 3 | [3] | 33.842 | [3] | 33.677 |
| `1 2 3 4 5 6 7 8 9` | 0 | 4 | [4] | 33.842 | [4] | 33.677 |
| `1 2 3 4 5 6 7 8 9` | 0 | 5 | [5] | 33.842 | [5] | 33.677 |
| `1 2 3 4 5 6 7 8 9` | 0 | 6 | [6] | 33.842 | [6] | 33.677 |
| `1 2 3 4 5 6 7 8 9` | 0 | 7 | [7] | 33.842 | [7] | 33.677 |
| `1 2 3 4 5 6 7 8 9` | 0 | 8 | [8] | 33.842 | [8] | 33.677 |
| `1 2 3 4 5 6 7 8 9` | 0 | 9 | [9] | 33.842 | [9] | 33.677 |
| `1 2 3 4 5 6 7 8 9` | 0 | 10 | [1, 9] | 33.842 | [1, 9] | 33.677 |
| `1 2 3 4 5 6 7 8 9` | 0 | 11 | [2, 9] | 33.842 | [2, 9] | 33.677 |
| `1 2 3 4 5 6 7 8 9` | 0 | 12 | [3, 9] | 33.842 | [3, 9] | 33.677 |
| `- 2 3 4 5 6 7 8 9` | 1 | 2 | [2] | 29.366 | [2] | 29.269 |
| `- 2 3 4 5 6 7 8 9` | 1 | 3 | [3] | 29.366 | [3] | 29.269 |
| `- 2 3 4 5 6 7 8 9` | 1 | 4 | [4] | 29.366 | [4] | 29.269 |
| `- 2 3 4 5 6 7 8 9` | 1 | 5 | [5] | 29.366 | [5] | 29.269 |
| `- 2 3 4 5 6 7 8 9` | 1 | 6 | [6] | 29.366 | [6] | 29.269 |
| `- 2 3 4 5 6 7 8 9` | 1 | 7 | [7] | 29.366 | [7] | 29.269 |
| `- 2 3 4 5 6 7 8 9` | 1 | 8 | [8] | 29.366 | [8] | 29.269 |
| `- 2 3 4 5 6 7 8 9` | 1 | 9 | [9] | 29.366 | [9] | 29.269 |
| `- 2 3 4 5 6 7 8 9` | 1 | 10 | [2, 8] | 29.366 | [2, 8] | 29.269 |
| `- 2 3 4 5 6 7 8 9` | 1 | 11 | [2, 9] | 29.366 | [2, 9] | 29.269 |
| `- 2 3 4 5 6 7 8 9` | 1 | 12 | [3, 9] | 29.366 | [3, 9] | 29.269 |
| `1 - 3 4 5 6 7 8 9` | 2 | 3 | [3] | 29.161 | [3] | 29.036 |
| `1 - 3 4 5 6 7 8 9` | 2 | 4 | [4] | 29.161 | [4] | 29.036 |
| `1 - 3 4 5 6 7 8 9` | 2 | 5 | [5] | 29.161 | [5] | 29.036 |
| `1 - 3 4 5 6 7 8 9` | 2 | 6 | [6] | 29.161 | [6] | 29.036 |
| `1 - 3 4 5 6 7 8 9` | 2 | 7 | [7] | 29.161 | [7] | 29.036 |
| `1 - 3 4 5 6 7 8 9` | 2 | 8 | [8] | 29.161 | [8] | 29.036 |
| `1 - 3 4 5 6 7 8 9` | 2 | 9 | [9] | 29.161 | [9] | 29.036 |
| `1 - 3 4 5 6 7 8 9` | 2 | 10 | [1, 9] | 29.161 | [1, 9] | 29.036 |
| `1 - 3 4 5 6 7 8 9` | 2 | 11 | [3, 8] | 29.161 | [3, 8] | 29.036 |
| `1 - 3 4 5 6 7 8 9` | 2 | 12 | [4, 8] | 29.161 | [3, 9] | 29.036 |
| `- - 3 4 5 6 7 8 9` | 3 | 3 | [3] | 24.603 | [3] | 24.502 |
| `- - 3 4 5 6 7 8 9` | 3 | 4 | [4] | 24.603 | [4] | 24.502 |
| `- - 3 4 5 6 7 8 9` | 3 | 5 | [5] | 24.603 | [5] | 24.502 |
| `- - 3 4 5 6 7 8 9` | 3 | 6 | [6] | 24.603 | [6] | 24.502 |
| `- - 3 4 5 6 7 8 9` | 3 | 7 | [7] | 24.603 | [7] | 24.502 |
| `- - 3 4 5 6 7 8 9` | 3 | 8 | [8] | 24.603 | [8] | 24.502 |
| `- - 3 4 5 6 7 8 9` | 3 | 9 | [9] | 24.603 | [9] | 24.502 |
| `- - 3 4 5 6 7 8 9` | 3 | 10 | [4, 6] | 24.603 | [3, 7] | 24.502 |
| `- - 3 4 5 6 7 8 9` | 3 | 11 | [3, 8] | 24.603 | [3, 8] | 24.502 |
| `- - 3 4 5 6 7 8 9` | 3 | 12 | [3, 9] | 24.603 | [3, 9] | 24.502 |
| `1 2 - 4 5 6 7 8 9` | 3 | 2 | [2] | 30.686 | [2] | 30.584 |
| `1 2 - 4 5 6 7 8 9` | 3 | 3 | [1, 2] | 30.686 | [1, 2] | 30.584 |
| `1 2 - 4 5 6 7 8 9` | 3 | 4 | [4] | 30.686 | [4] | 30.584 |
| `1 2 - 4 5 6 7 8 9` | 3 | 5 | [5] | 30.686 | [5] | 30.584 |
| `1 2 - 4 5 6 7 8 9` | 3 | 6 | [6] | 30.686 | [6] | 30.584 |
| `1 2 - 4 5 6 7 8 9` | 3 | 7 | [7] | 30.686 | [7] | 30.584 |
| `1 2 - 4 5 6 7 8 9` | 3 | 8 | [8] | 30.686 | [8] | 30.584 |
| `1 2 - 4 5 6 7 8 9` | 3 | 9 | [9] | 30.686 | [9] | 30.584 |
| `1 2 - 4 5 6 7 8 9` | 3 | 10 | [1, 9] | 30.686 | [1, 9] | 30.584 |
| `1 2 - 4 5 6 7 8 9` | 3 | 11 | [2, 9] | 30.686 | [2, 9] | 30.584 |
| `1 2 - 4 5 6 7 8 9` | 3 | 12 | [5, 7] | 30.686 | [4, 8] | 30.584 |
| `- 2 - 4 5 6 7 8 9` | 4 | 2 | [2] | 25.143 | [2] | 25.095 |
| `- 2 - 4 5 6 7 8 9` | 4 | 4 | [4] | 25.143 | [4] | 25.095 |
| `- 2 - 4 5 6 7 8 9` | 4 | 5 | [5] | 25.143 | [5] | 25.095 |
| `- 2 - 4 5 6 7 8 9` | 4 | 6 | [6] | 25.143 | [6] | 25.095 |
| `- 2 - 4 5 6 7 8 9` | 4 | 7 | [7] | 25.143 | [7] | 25.095 |
| `- 2 - 4 5 6 7 8 9` | 4 | 8 | [8] | 25.143 | [8] | 25.095 |
| `- 2 - 4 5 6 7 8 9` | 4 | 9 | [9] | 25.143 | [9] | 25.095 |
| `- 2 - 4 5 6 7 8 9` | 4 | 10 | [2, 8] | 25.143 | [2, 8] | 25.095 |
| `- 2 - 4 5 6 7 8 9` | 4 | 11 | [2, 9] | 25.143 | [2, 9] | 25.095 |
| `- 2 - 4 5 6 7 8 9` | 4 | 12 | [4, 8] | 25.143 | [4, 8] | 25.095 |
| `1 2 3 - 5 6 7 8 9` | 4 | 2 | [2] | 31.294 | [2] | 31.103 |
| `1 2 3 - 5 6 7 8 9` | 4 | 3 | [3] | 31.294 | [3] | 31.103 |
| `1 2 3 - 5 6 7 8 9` | 4 | 4 | [1, 3] | 31.294 | [1, 3] | 31.103 |
| `1 2 3 - 5 6 7 8 9` | 4 | 5 | [5] | 31.294 | [5] | 31.103 |
| `1 2 3 - 5 6 7 8 9` | 4 | 6 | [6] | 31.294 | [6] | 31.103 |
| `1 2 3 - 5 6 7 8 9` | 4 | 7 | [7] | 31.294 | [7] | 31.103 |
| `1 2 3 - 5 6 7 8 9` | 4 | 8 | [8] | 31.294 | [8] | 31.103 |
| `1 2 3 - 5 6 7 8 9` | 4 | 9 | [9] | 31.294 | [9] | 31.103 |
| `1 2 3 - 5 6 7 8 9` | 4 | 10 | [2, 8] | 31.294 | [1, 9] | 31.103 |
| `1 2 3 - 5 6 7 8 9` | 4 | 11 | [2, 9] | 31.294 | [2, 9] | 31.103 |
| `1 2 3 - 5 6 7 8 9` | 4 | 12 | [3, 9] | 31.294 | [3, 9] | 31.103 |
| `- 2 3 - 5 6 7 8 9` | 5 | 2 | [2] | 25.242 | [2] | 25.206 |
| `- 2 3 - 5 6 7 8 9` | 5 | 3 | [3] | 25.242 | [3] | 25.206 |
| `- 2 3 - 5 6 7 8 9` | 5 | 5 | [5] | 25.242 | [5] | 25.206 |
| `- 2 3 - 5 6 7 8 9` | 5 | 6 | [6] | 25.242 | [6] | 25.206 |
| `- 2 3 - 5 6 7 8 9` | 5 | 7 | [7] | 25.242 | [7] | 25.206 |
| `- 2 3 - 5 6 7 8 9` | 5 | 8 | [8] | 25.242 | [8] | 25.206 |
| `- 2 3 - 5 6 7 8 9` | 5 | 9 | [9] | 25.242 | [9] | 25.206 |
| `- 2 3 - 5 6 7 8 9` | 5 | 10 | [2, 8] | 25.242 | [2, 8] | 25.206 |
| `- 2 3 - 5 6 7 8 9` | 5 | 11 | [2, 9] | 25.242 | [2, 9] | 25.206 |
| `- 2 3 - 5 6 7 8 9` | 5 | 12 | [3, 9] | 25.242 | [3, 9] | 25.206 |
| `1 - - 4 5 6 7 8 9` | 5 | 4 | [4] | 25.583 | [4] | 25.560 |
| `1 - - 4 5 6 7 8 9` | 5 | 5 | [5] | 25.583 | [5] | 25.560 |
| `1 - - 4 5 6 7 8 9` | 5 | 6 | [6] | 25.583 | [6] | 25.560 |
| `1 - - 4 5 6 7 8 9` | 5 | 7 | [7] | 25.583 | [7] | 25.560 |
| `1 - - 4 5 6 7 8 9` | 5 | 8 | [8] | 25.583 | [8] | 25.560 |
| `1 - - 4 5 6 7 8 9` | 5 | 9 | [9] | 25.583 | [9] | 25.560 |
| `1 - - 4 5 6 7 8 9` | 5 | 10 | [1, 9] | 25.583 | [1, 9] | 25.560 |
| `1 - - 4 5 6 7 8 9` | 5 | 11 | [4, 7] | 25.583 | [4, 7] | 25.560 |
| `1 - - 4 5 6 7 8 9` | 5 | 12 | [5, 7] | 25.583 | [4, 8] | 25.560 |
| `1 2 3 4 - 6 7 8 9` | 5 | 2 | [2] | 32.485 | [2] | 32.311 |
| `1 2 3 4 - 6 7 8 9` | 5 | 3 | [3] | 32.485 | [3] | 32.311 |
| `1 2 3 4 - 6 7 8 9` | 5 | 4 | [4] | 32.485 | [4] | 32.311 |
| `1 2 3 4 - 6 7 8 9` | 5 | 5 | [2, 3] | 32.485 | [1, 4] | 32.311 |
| `1 2 3 4 - 6 7 8 9` | 5 | 6 | [6] | 32.485 | [6] | 32.311 |
| `1 2 3 4 - 6 7 8 9` | 5 | 7 | [7] | 32.485 | [7] | 32.311 |
| `1 2 3 4 - 6 7 8 9` | 5 | 8 | [8] | 32.485 | [8] | 32.311 |
| `1 2 3 4 - 6 7 8 9` | 5 | 9 | [9] | 32.485 | [9] | 32.311 |
| `1 2 3 4 - 6 7 8 9` | 5 | 10 | [1, 9] | 32.485 | [1, 9] | 32.311 |
| `1 2 3 4 - 6 7 8 9` | 5 | 11 | [2, 9] | 32.485 | [2, 9] | 32.311 |
| `1 2 3 4 - 6 7 8 9` | 5 | 12 | [3, 9] | 32.485 | [3, 9] | 32.311 |
| `- - - 4 5 6 7 8 9` | 6 | 4 | [4] | 21.798 | [4] | 21.759 |
| `- - - 4 5 6 7 8 9` | 6 | 5 | [5] | 21.798 | [5] | 21.759 |
| `- - - 4 5 6 7 8 9` | 6 | 6 | [6] | 21.798 | [6] | 21.759 |
| `- - - 4 5 6 7 8 9` | 6 | 7 | [7] | 21.798 | [7] | 21.759 |
| `- - - 4 5 6 7 8 9` | 6 | 8 | [8] | 21.798 | [8] | 21.759 |
| `- - - 4 5 6 7 8 9` | 6 | 9 | [9] | 21.798 | [9] | 21.759 |
| `- - - 4 5 6 7 8 9` | 6 | 10 | [4, 6] | 21.798 | [4, 6] | 21.759 |
| `- - - 4 5 6 7 8 9` | 6 | 11 | [5, 6] | 21.798 | [4, 7] | 21.759 |
| `- - - 4 5 6 7 8 9` | 6 | 12 | [4, 8] | 21.798 | [4, 8] | 21.759 |
| `- 2 3 4 - 6 7 8 9` | 6 | 2 | [2] | 28.199 | [2] | 28.073 |
| `- 2 3 4 - 6 7 8 9` | 6 | 3 | [3] | 28.199 | [3] | 28.073 |
| `- 2 3 4 - 6 7 8 9` | 6 | 4 | [4] | 28.199 | [4] | 28.073 |
| `- 2 3 4 - 6 7 8 9` | 6 | 5 | [2, 3] | 28.199 | [2, 3] | 28.073 |
| `- 2 3 4 - 6 7 8 9` | 6 | 6 | [6] | 28.199 | [6] | 28.073 |
| `- 2 3 4 - 6 7 8 9` | 6 | 7 | [7] | 28.199 | [7] | 28.073 |
| `- 2 3 4 - 6 7 8 9` | 6 | 8 | [8] | 28.199 | [8] | 28.073 |
| `- 2 3 4 - 6 7 8 9` | 6 | 9 | [9] | 28.199 | [9] | 28.073 |
| `- 2 3 4 - 6 7 8 9` | 6 | 10 | [4, 6] | 28.199 | [2, 8] | 28.073 |
| `- 2 3 4 - 6 7 8 9` | 6 | 11 | [2, 9] | 28.199 | [2, 9] | 28.073 |
| `- 2 3 4 - 6 7 8 9` | 6 | 12 | [4, 8] | 28.199 | [3, 9] | 28.073 |
| `1 - 3 - 5 6 7 8 9` | 6 | 3 | [3] | 27.551 | [3] | 27.504 |
| `1 - 3 - 5 6 7 8 9` | 6 | 4 | [1, 3] | 27.551 | [1, 3] | 27.504 |
| `1 - 3 - 5 6 7 8 9` | 6 | 5 | [5] | 27.551 | [5] | 27.504 |
| `1 - 3 - 5 6 7 8 9` | 6 | 6 | [6] | 27.551 | [6] | 27.504 |
| `1 - 3 - 5 6 7 8 9` | 6 | 7 | [7] | 27.551 | [7] | 27.504 |
| `1 - 3 - 5 6 7 8 9` | 6 | 8 | [8] | 27.551 | [8] | 27.504 |
| `1 - 3 - 5 6 7 8 9` | 6 | 9 | [9] | 27.551 | [9] | 27.504 |
| `1 - 3 - 5 6 7 8 9` | 6 | 10 | [1, 9] | 27.551 | [1, 9] | 27.504 |
| `1 - 3 - 5 6 7 8 9` | 6 | 11 | [3, 8] | 27.551 | [3, 8] | 27.504 |
| `1 - 3 - 5 6 7 8 9` | 6 | 12 | [5, 7] | 27.551 | [3, 9] | 27.504 |
| `1 2 3 4 5 - 7 8 9` | 6 | 2 | [2] | 33.273 | [2] | 33.079 |
| `1 2 3 4 5 - 7 8 9` | 6 | 3 | [3] | 33.273 | [3] | 33.079 |
| `1 2 3 4 5 - 7 8 9` | 6 | 4 | [4] | 33.273 | [4] | 33.079 |
| `1 2 3 4 5 - 7 8 9` | 6 | 5 | [5] | 33.273 | [5] | 33.079 |
| `1 2 3 4 5 - 7 8 9` | 6 | 6 | [1, 5] | 33.273 | [1, 5] | 33.079 |
| `1 2 3 4 5 - 7 8 9` | 6 | 7 | [7] | 33.273 | [7] | 33.079 |
| `1 2 3 4 5 - 7 8 9` | 6 | 8 | [8] | 33.273 | [8] | 33.079 |
| `1 2 3 4 5 - 7 8 9` | 6 | 9 | [9] | 33.273 | [9] | 33.079 |
| `1 2 3 4 5 - 7 8 9` | 6 | 10 | [1, 9] | 33.273 | [1, 9] | 33.079 |
| `1 2 3 4 5 - 7 8 9` | 6 | 11 | [2, 9] | 33.273 | [2, 9] | 33.079 |
| `1 2 3 4 5 - 7 8 9` | 6 | 12 | [3, 9] | 33.273 | [3, 9] | 33.079 |
| `- - 3 - 5 6 7 8 9` | 7 | 3 | [3] | 22.449 | [3] | 22.399 |
| `- - 3 - 5 6 7 8 9` | 7 | 5 | [5] | 22.449 | [5] | 22.399 |
| `- - 3 - 5 6 7 8 9` | 7 | 6 | [6] | 22.449 | [6] | 22.399 |
| `- - 3 - 5 6 7 8 9` | 7 | 7 | [7] | 22.449 | [7] | 22.399 |
| `- - 3 - 5 6 7 8 9` | 7 | 8 | [8] | 22.449 | [8] | 22.399 |
| `- - 3 - 5 6 7 8 9` | 7 | 9 | [9] | 22.449 | [9] | 22.399 |
| `- - 3 - 5 6 7 8 9` | 7 | 10 | [3, 7] | 22.449 | [3, 7] | 22.399 |
| `- - 3 - 5 6 7 8 9` | 7 | 11 | [5, 6] | 22.449 | [3, 8] | 22.399 |
| `- - 3 - 5 6 7 8 9` | 7 | 12 | [3, 9] | 22.449 | [3, 9] | 22.399 |
| `- 2 3 4 5 - 7 8 9` | 7 | 2 | [2] | 28.744 | [2] | 28.599 |
| `- 2 3 4 5 - 7 8 9` | 7 | 3 | [3] | 28.744 | [3] | 28.599 |
| `- 2 3 4 5 - 7 8 9` | 7 | 4 | [4] | 28.744 | [4] | 28.599 |
| `- 2 3 4 5 - 7 8 9` | 7 | 5 | [5] | 28.744 | [5] | 28.599 |
| `- 2 3 4 5 - 7 8 9` | 7 | 6 | [2, 4] | 28.744 | [2, 4] | 28.599 |
| `- 2 3 4 5 - 7 8 9` | 7 | 7 | [7] | 28.744 | [7] | 28.599 |
| `- 2 3 4 5 - 7 8 9` | 7 | 8 | [8] | 28.744 | [8] | 28.599 |
| `- 2 3 4 5 - 7 8 9` | 7 | 9 | [9] | 28.744 | [9] | 28.599 |
| `- 2 3 4 5 - 7 8 9` | 7 | 10 | [3, 7] | 28.744 | [2, 8] | 28.599 |
| `- 2 3 4 5 - 7 8 9` | 7 | 11 | [2, 9] | 28.744 | [2, 9] | 28.599 |
| `- 2 3 4 5 - 7 8 9` | 7 | 12 | [3, 9] | 28.744 | [3, 9] | 28.599 |
| `1 - 3 4 - 6 7 8 9` | 7 | 3 | [3] | 28.263 | [3] | 28.231 |
| `1 - 3 4 - 6 7 8 9` | 7 | 4 | [4] | 28.263 | [4] | 28.231 |
| `1 - 3 4 - 6 7 8 9` | 7 | 5 | [1, 4] | 28.263 | [1, 4] | 28.231 |
| `1 - 3 4 - 6 7 8 9` | 7 | 6 | [6] | 28.263 | [6] | 28.231 |
| `1 - 3 4 - 6 7 8 9` | 7 | 7 | [7] | 28.263 | [7] | 28.231 |
| `1 - 3 4 - 6 7 8 9` | 7 | 8 | [8] | 28.263 | [8] | 28.231 |
| `1 - 3 4 - 6 7 8 9` | 7 | 9 | [9] | 28.263 | [9] | 28.231 |
| `1 - 3 4 - 6 7 8 9` | 7 | 10 | [1, 9] | 28.263 | [1, 9] | 28.231 |
| `1 - 3 4 - 6 7 8 9` | 7 | 11 | [3, 8] | 28.263 | [3, 8] | 28.231 |
| `1 - 3 4 - 6 7 8 9` | 7 | 12 | [3, 9] | 28.263 | [3, 9] | 28.231 |
| `1 2 - - 5 6 7 8 9` | 7 | 2 | [2] | 26.837 | [2] | 26.794 |
| `1 2 - - 5 6 7 8 9` | 7 | 3 | [1, 2] | 26.837 | [1, 2] | 26.794 |
| `1 2 - - 5 6 7 8 9` | 7 | 5 | [5] | 26.837 | [5] | 26.794 |
| `1 2 - - 5 6 7 8 9` | 7 | 6 | [6] | 26.837 | [6] | 26.794 |
| `1 2 - - 5 6 7 8 9` | 7 | 7 | [7] | 26.837 | [7] | 26.794 |
| `1 2 - - 5 6 7 8 9` | 7 | 8 | [8] | 26.837 | [8] | 26.794 |
| `1 2 - - 5 6 7 8 9` | 7 | 9 | [9] | 26.837 | [9] | 26.794 |
| `1 2 - - 5 6 7 8 9` | 7 | 10 | [1, 9] | 26.837 | [1, 9] | 26.794 |
| `1 2 - - 5 6 7 8 9` | 7 | 11 | [2, 9] | 26.837 | [2, 9] | 26.794 |
| `1 2 - - 5 6 7 8 9` | 7 | 12 | [5, 7] | 26.837 | [5, 7] | 26.794 |
| `1 2 3 4 5 6 - 8 9` | 7 | 2 | [2] | 34.175 | [2] | 33.964 |
| `1 2 3 4 5 6 - 8 9` | 7 | 3 | [3] | 34.175 | [3] | 33.964 |
| `1 2 3 4 5 6 - 8 9` | 7 | 4 | [4] | 34.175 | [4] | 33.964 |
| `1 2 3 4 5 6 - 8 9` | 7 | 5 | [5] | 34.175 | [5] | 33.964 |
| `1 2 3 4 5 6 - 8 9` | 7 | 6 | [6] | 34.175 | [6] | 33.964 |
| `1 2 3 4 5 6 - 8 9` | 7 | 7 | [1, 6] | 34.175 | [1, 6] | 33.964 |
| `1 2 3 4 5 6 - 8 9` | 7 | 8 | [8] | 34.175 | [8] | 33.964 |
| `1 2 3 4 5 6 - 8 9` | 7 | 9 | [9] | 34.175 | [9] | 33.964 |
| `1 2 3 4 5 6 - 8 9` | 7 | 10 | [1, 9] | 34.175 | [1, 9] | 33.964 |
| `1 2 3 4 5 6 - 8 9` | 7 | 11 | [2, 9] | 34.175 | [2, 9] | 33.964 |
| `1 2 3 4 5 6 - 8 9` | 7 | 12 | [3, 9] | 34.175 | [3, 9] | 33.964 |
| `- - 3 4 - 6 7 8 9` | 8 | 3 | [3] | 23.194 | [3] | 23.085 |
| `- - 3 4 - 6 7 8 9` | 8 | 4 | [4] | 23.194 | [4] | 23.085 |
| `- - 3 4 - 6 7 8 9` | 8 | 6 | [6] | 23.194 | [6] | 23.085 |
| `- - 3 4 - 6 7 8 9` | 8 | 7 | [7] | 23.194 | [7] | 23.085 |
| `- - 3 4 - 6 7 8 9` | 8 | 8 | [8] | 23.194 | [8] | 23.085 |
| `- - 3 4 - 6 7 8 9` | 8 | 9 | [9] | 23.194 | [9] | 23.085 |
| `- - 3 4 - 6 7 8 9` | 8 | 10 | [4, 6] | 23.194 | [3, 7] | 23.085 |
| `- - 3 4 - 6 7 8 9` | 8 | 11 | [3, 8] | 23.194 | [3, 8] | 23.085 |
| `- - 3 4 - 6 7 8 9` | 8 | 12 | [3, 9] | 23.194 | [3, 9] | 23.085 |
| `- 2 - - 5 6 7 8 9` | 8 | 2 | [2] | 23.170 | [2] | 23.167 |
| `- 2 - - 5 6 7 8 9` | 8 | 5 | [5] | 23.170 | [5] | 23.167 |
| `- 2 - - 5 6 7 8 9` | 8 | 6 | [6] | 23.170 | [6] | 23.167 |
| `- 2 - - 5 6 7 8 9` | 8 | 7 | [7] | 23.170 | [7] | 23.167 |
| `- 2 - - 5 6 7 8 9` | 8 | 8 | [8] | 23.170 | [8] | 23.167 |
| `- 2 - - 5 6 7 8 9` | 8 | 9 | [9] | 23.170 | [9] | 23.167 |
| `- 2 - - 5 6 7 8 9` | 8 | 10 | [2, 8] | 23.170 | [2, 8] | 23.167 |
| `- 2 - - 5 6 7 8 9` | 8 | 11 | [2, 9] | 23.170 | [2, 9] | 23.167 |
| `- 2 - - 5 6 7 8 9` | 8 | 12 | [5, 7] | 23.170 | [5, 7] | 23.167 |
| `- 2 3 4 5 6 - 8 9` | 8 | 2 | [2] | 30.233 | [2] | 30.154 |
| `- 2 3 4 5 6 - 8 9` | 8 | 3 | [3] | 30.233 | [3] | 30.154 |
| `- 2 3 4 5 6 - 8 9` | 8 | 4 | [4] | 30.233 | [4] | 30.154 |
| `- 2 3 4 5 6 - 8 9` | 8 | 5 | [5] | 30.233 | [5] | 30.154 |
| `- 2 3 4 5 6 - 8 9` | 8 | 6 | [6] | 30.233 | [6] | 30.154 |
| `- 2 3 4 5 6 - 8 9` | 8 | 7 | [2, 5] | 30.233 | [2, 5] | 30.154 |
| `- 2 3 4 5 6 - 8 9` | 8 | 8 | [8] | 30.233 | [8] | 30.154 |
| `- 2 3 4 5 6 - 8 9` | 8 | 9 | [9] | 30.233 | [9] | 30.154 |
| `- 2 3 4 5 6 - 8 9` | 8 | 10 | [2, 8] | 30.233 | [2, 8] | 30.154 |
| `- 2 3 4 5 6 - 8 9` | 8 | 11 | [2, 9] | 30.233 | [2, 9] | 30.154 |
| `- 2 3 4 5 6 - 8 9` | 8 | 12 | [3, 9] | 30.233 | [3, 9] | 30.154 |
| `1 - 3 4 5 - 7 8 9` | 8 | 3 | [3] | 28.841 | [3] | 28.798 |
| `1 - 3 4 5 - 7 8 9` | 8 | 4 | [4] | 28.841 | [4] | 28.798 |
| `1 - 3 4 5 - 7 8 9` | 8 | 5 | [5] | 28.841 | [5] | 28.798 |
| `1 - 3 4 5 - 7 8 9` | 8 | 6 | [1, 5] | 28.841 | [1, 5] | 28.798 |
| `1 - 3 4 5 - 7 8 9` | 8 | 7 | [7] | 28.841 | [7] | 28.798 |
| `1 - 3 4 5 - 7 8 9` | 8 | 8 | [8] | 28.841 | [8] | 28.798 |
| `1 - 3 4 5 - 7 8 9` | 8 | 9 | [9] | 28.841 | [9] | 28.798 |
| `1 - 3 4 5 - 7 8 9` | 8 | 10 | [1, 9] | 28.841 | [1, 9] | 28.798 |
| `1 - 3 4 5 - 7 8 9` | 8 | 11 | [3, 8] | 28.841 | [3, 8] | 28.798 |
| `1 - 3 4 5 - 7 8 9` | 8 | 12 | [4, 8] | 28.841 | [3, 9] | 28.798 |
| `1 2 - 4 - 6 7 8 9` | 8 | 2 | [2] | 29.820 | [2] | 29.699 |
| `1 2 - 4 - 6 7 8 9` | 8 | 3 | [1, 2] | 29.820 | [1, 2] | 29.699 |
| `1 2 - 4 - 6 7 8 9` | 8 | 4 | [4] | 29.820 | [4] | 29.699 |
| `1 2 - 4 - 6 7 8 9` | 8 | 5 | [1, 4] | 29.820 | [1, 4] | 29.699 |
| `1 2 - 4 - 6 7 8 9` | 8 | 6 | [6] | 29.820 | [6] | 29.699 |
| `1 2 - 4 - 6 7 8 9` | 8 | 7 | [7] | 29.820 | [7] | 29.699 |
| `1 2 - 4 - 6 7 8 9` | 8 | 8 | [8] | 29.820 | [8] | 29.699 |
| `1 2 - 4 - 6 7 8 9` | 8 | 9 | [9] | 29.820 | [9] | 29.699 |
| `1 2 - 4 - 6 7 8 9` | 8 | 10 | [2, 8] | 29.820 | [1, 9] | 29.699 |
| `1 2 - 4 - 6 7 8 9` | 8 | 11 | [2, 9] | 29.820 | [2, 9] | 29.699 |
| `1 2 - 4 - 6 7 8 9` | 8 | 12 | [4, 8] | 29.820 | [4, 8] | 29.699 |
| `1 2 3 4 5 6 7 - 9` | 8 | 2 | [2] | 35.759 | [2] | 35.535 |
| `1 2 3 4 5 6 7 - 9` | 8 | 3 | [3] | 35.759 | [3] | 35.535 |
| `1 2 3 4 5 6 7 - 9` | 8 | 4 | [4] | 35.759 | [4] | 35.535 |
| `1 2 3 4 5 6 7 - 9` | 8 | 5 | [5] | 35.759 | [5] | 35.535 |
| `1 2 3 4 5 6 7 - 9` | 8 | 6 | [6] | 35.759 | [6] | 35.535 |
| `1 2 3 4 5 6 7 - 9` | 8 | 7 | [7] | 35.759 | [7] | 35.535 |
| `1 2 3 4 5 6 7 - 9` | 8 | 8 | [1, 7] | 35.759 | [1, 7] | 35.535 |
| `1 2 3 4 5 6 7 - 9` | 8 | 9 | [9] | 35.759 | [9] | 35.535 |
| `1 2 3 4 5 6 7 - 9` | 8 | 10 | [1, 9] | 35.759 | [1, 9] | 35.535 |
| `1 2 3 4 5 6 7 - 9` | 8 | 11 | [2, 9] | 35.759 | [2, 9] | 35.535 |
| `1 2 3 4 5 6 7 - 9` | 8 | 12 | [3, 9] | 35.759 | [3, 9] | 35.535 |
| `- - 3 4 5 - 7 8 9` | 9 | 3 | [3] | 22.911 | [3] | 22.910 |
| `- - 3 4 5 - 7 8 9` | 9 | 4 | [4] | 22.911 | [4] | 22.910 |
| `- - 3 4 5 - 7 8 9` | 9 | 5 | [5] | 22.911 | [5] | 22.910 |
| `- - 3 4 5 - 7 8 9` | 9 | 7 | [7] | 22.911 | [7] | 22.910 |
| `- - 3 4 5 - 7 8 9` | 9 | 8 | [8] | 22.911 | [8] | 22.910 |
| `- - 3 4 5 - 7 8 9` | 9 | 9 | [9] | 22.911 | [9] | 22.910 |
| `- - 3 4 5 - 7 8 9` | 9 | 10 | [3, 7] | 22.911 | [3, 7] | 22.910 |
| `- - 3 4 5 - 7 8 9` | 9 | 11 | [3, 8] | 22.911 | [3, 8] | 22.910 |
| `- - 3 4 5 - 7 8 9` | 9 | 12 | [3, 9] | 22.911 | [3, 9] | 22.910 |
| `- 2 - 4 - 6 7 8 9` | 9 | 2 | [2] | 23.631 | [2] | 23.620 |
| `- 2 - 4 - 6 7 8 9` | 9 | 4 | [4] | 23.631 | [4] | 23.620 |
| `- 2 - 4 - 6 7 8 9` | 9 | 6 | [6] | 23.631 | [6] | 23.620 |
| `- 2 - 4 - 6 7 8 9` | 9 | 7 | [7] | 23.631 | [7] | 23.620 |
| `- 2 - 4 - 6 7 8 9` | 9 | 8 | [8] | 23.631 | [8] | 23.620 |
| `- 2 - 4 - 6 7 8 9` | 9 | 9 | [9] | 23.631 | [9] | 23.620 |
| `- 2 - 4 - 6 7 8 9` | 9 | 10 | [2, 8] | 23.631 | [2, 8] | 23.620 |
| `- 2 - 4 - 6 7 8 9` | 9 | 11 | [2, 9] | 23.631 | [2, 9] | 23.620 |
| `- 2 - 4 - 6 7 8 9` | 9 | 12 | [4, 8] | 23.631 | [4, 8] | 23.620 |
| `- 2 3 4 5 6 7 - 9` | 9 | 2 | [2] | 31.565 | [2] | 31.478 |
| `- 2 3 4 5 6 7 - 9` | 9 | 3 | [3] | 31.565 | [3] | 31.478 |
| `- 2 3 4 5 6 7 - 9` | 9 | 4 | [4] | 31.565 | [4] | 31.478 |
| `- 2 3 4 5 6 7 - 9` | 9 | 5 | [5] | 31.565 | [5] | 31.478 |
| `- 2 3 4 5 6 7 - 9` | 9 | 6 | [6] | 31.565 | [6] | 31.478 |
| `- 2 3 4 5 6 7 - 9` | 9 | 7 | [7] | 31.565 | [7] | 31.478 |
| `- 2 3 4 5 6 7 - 9` | 9 | 8 | [3, 5] | 31.565 | [2, 6] | 31.478 |
| `- 2 3 4 5 6 7 - 9` | 9 | 9 | [9] | 31.565 | [9] | 31.478 |
| `- 2 3 4 5 6 7 - 9` | 9 | 10 | [3, 7] | 31.565 | [3, 7] | 31.478 |
| `- 2 3 4 5 6 7 - 9` | 9 | 11 | [2, 9] | 31.565 | [2, 9] | 31.478 |
| `- 2 3 4 5 6 7 - 9` | 9 | 12 | [3, 9] | 31.565 | [3, 9] | 31.478 |
| `1 - - - 5 6 7 8 9` | 9 | 5 | [5] | 23.704 | [5] | 23.704 |
| `1 - - - 5 6 7 8 9` | 9 | 6 | [6] | 23.704 | [6] | 23.704 |
| `1 - - - 5 6 7 8 9` | 9 | 7 | [7] | 23.704 | [7] | 23.704 |
| `1 - - - 5 6 7 8 9` | 9 | 8 | [8] | 23.704 | [8] | 23.704 |
| `1 - - - 5 6 7 8 9` | 9 | 9 | [9] | 23.704 | [9] | 23.704 |
| `1 - - - 5 6 7 8 9` | 9 | 10 | [1, 9] | 23.704 | [1, 9] | 23.704 |
| `1 - - - 5 6 7 8 9` | 9 | 11 | [5, 6] | 23.704 | [5, 6] | 23.704 |
| `1 - - - 5 6 7 8 9` | 9 | 12 | [5, 7] | 23.704 | [5, 7] | 23.704 |
| `1 - 3 4 5 6 - 8 9` | 9 | 3 | [3] | 30.157 | [3] | 29.765 |
| `1 - 3 4 5 6 - 8 9` | 9 | 4 | [4] | 30.157 | [4] | 29.765 |
| `1 - 3 4 5 6 - 8 9` | 9 | 5 | [5] | 30.157 | [5] | 29.765 |
| `1 - 3 4 5 6 - 8 9` | 9 | 6 | [6] | 30.157 | [6] | 29.765 |
| `1 - 3 4 5 6 - 8 9` | 9 | 7 | [3, 4] | 30.157 | [1, 6] | 29.765 |
| `1 - 3 4 5 6 - 8 9` | 9 | 8 | [8] | 30.157 | [8] | 29.765 |
| `1 - 3 4 5 6 - 8 9` | 9 | 9 | [9] | 30.157 | [9] | 29.765 |
| `1 - 3 4 5 6 - 8 9` | 9 | 10 | [1, 9] | 30.157 | [1, 9] | 29.765 |
| `1 - 3 4 5 6 - 8 9` | 9 | 11 | [5, 6] | 30.157 | [3, 8] | 29.765 |
| `1 - 3 4 5 6 - 8 9` | 9 | 12 | [3, 9] | 30.157 | [3, 9] | 29.765 |
| `1 2 - 4 5 - 7 8 9` | 9 | 2 | [2] | 30.779 | [2] | 30.705 |
| `1 2 - 4 5 - 7 8 9` | 9 | 3 | [1, 2] | 30.779 | [1, 2] | 30.705 |
| `1 2 - 4 5 - 7 8 9` | 9 | 4 | [4] | 30.779 | [4] | 30.705 |
| `1 2 - 4 5 - 7 8 9` | 9 | 5 | [5] | 30.779 | [5] | 30.705 |
| `1 2 - 4 5 - 7 8 9` | 9 | 6 | [1, 5] | 30.779 | [1, 5] | 30.705 |
| `1 2 - 4 5 - 7 8 9` | 9 | 7 | [7] | 30.779 | [7] | 30.705 |
| `1 2 - 4 5 - 7 8 9` | 9 | 8 | [8] | 30.779 | [8] | 30.705 |
| `1 2 - 4 5 - 7 8 9` | 9 | 9 | [9] | 30.779 | [9] | 30.705 |
| `1 2 - 4 5 - 7 8 9` | 9 | 10 | [1, 9] | 30.779 | [1, 9] | 30.705 |
| `1 2 - 4 5 - 7 8 9` | 9 | 11 | [2, 9] | 30.779 | [2, 9] | 30.705 |
| `1 2 - 4 5 - 7 8 9` | 9 | 12 | [4, 8] | 30.779 | [4, 8] | 30.705 |
| `1 2 3 - - 6 7 8 9` | 9 | 2 | [2] | 30.156 | [2] | 30.127 |
| `1 2 3 - - 6 7 8 9` | 9 | 3 | [3] | 30.156 | [3] | 30.127 |
| `1 2 3 - - 6 7 8 9` | 9 | 4 | [1, 3] | 30.156 | [1, 3] | 30.127 |
| `1 2 3 - - 6 7 8 9` | 9 | 5 | [2, 3] | 30.156 | [2, 3] | 30.127 |
| `1 2 3 - - 6 7 8 9` | 9 | 6 | [6] | 30.156 | [6] | 30.127 |
| `1 2 3 - - 6 7 8 9` | 9 | 7 | [7] | 30.156 | [7] | 30.127 |
| `1 2 3 - - 6 7 8 9` | 9 | 8 | [8] | 30.156 | [8] | 30.127 |
| `1 2 3 - - 6 7 8 9` | 9 | 9 | [9] | 30.156 | [9] | 30.127 |
| `1 2 3 - - 6 7 8 9` | 9 | 10 | [1, 9] | 30.156 | [1, 9] | 30.127 |
| `1 2 3 - - 6 7 8 9` | 9 | 11 | [2, 9] | 30.156 | [2, 9] | 30.127 |
| `1 2 3 - - 6 7 8 9` | 9 | 12 | [3, 9] | 30.156 | [3, 9] | 30.127 |
| `1 2 3 4 5 6 7 8 -` | 9 | 2 | [2] | 37.376 | [2] | 37.263 |
| `1 2 3 4 5 6 7 8 -` | 9 | 3 | [3] | 37.376 | [3] | 37.263 |
| `1 2 3 4 5 6 7 8 -` | 9 | 4 | [4] | 37.376 | [4] | 37.263 |
| `1 2 3 4 5 6 7 8 -` | 9 | 5 | [5] | 37.376 | [5] | 37.263 |
| `1 2 3 4 5 6 7 8 -` | 9 | 6 | [6] | 37.376 | [6] | 37.263 |
| `1 2 3 4 5 6 7 8 -` | 9 | 7 | [7] | 37.376 | [7] | 37.263 |
| `1 2 3 4 5 6 7 8 -` | 9 | 8 | [8] | 37.376 | [8] | 37.263 |
| `1 2 3 4 5 6 7 8 -` | 9 | 9 | [1, 8] | 37.376 | [1, 8] | 37.263 |
| `1 2 3 4 5 6 7 8 -` | 9 | 10 | [2, 8] | 37.376 | [2, 8] | 37.263 |
| `1 2 3 4 5 6 7 8 -` | 9 | 11 | [3, 8] | 37.376 | [3, 8] | 37.263 |
| `1 2 3 4 5 6 7 8 -` | 9 | 12 | [4, 8] | 37.376 | [4, 8] | 37.263 |
| `- - - - 5 6 7 8 9` | 10 | 5 | [5] | 20.289 | [5] | 20.289 |
| `- - - - 5 6 7 8 9` | 10 | 6 | [6] | 20.289 | [6] | 20.289 |
| `- - - - 5 6 7 8 9` | 10 | 7 | [7] | 20.289 | [7] | 20.289 |
| `- - - - 5 6 7 8 9` | 10 | 8 | [8] | 20.289 | [8] | 20.289 |
| `- - - - 5 6 7 8 9` | 10 | 9 | [9] | 20.289 | [9] | 20.289 |
| `- - - - 5 6 7 8 9` | 10 | 11 | [5, 6] | 20.289 | [5, 6] | 20.289 |
| `- - - - 5 6 7 8 9` | 10 | 12 | [5, 7] | 20.289 | [5, 7] | 20.289 |
| `- - 3 4 5 6 - 8 9` | 10 | 3 | [3] | 26.024 | [3] | 25.976 |
| `- - 3 4 5 6 - 8 9` | 10 | 4 | [4] | 26.024 | [4] | 25.976 |
| `- - 3 4 5 6 - 8 9` | 10 | 5 | [5] | 26.024 | [5] | 25.976 |
| `- - 3 4 5 6 - 8 9` | 10 | 6 | [6] | 26.024 | [6] | 25.976 |
| `- - 3 4 5 6 - 8 9` | 10 | 7 | [3, 4] | 26.024 | [3, 4] | 25.976 |
| `- - 3 4 5 6 - 8 9` | 10 | 8 | [8] | 26.024 | [8] | 25.976 |
| `- - 3 4 5 6 - 8 9` | 10 | 9 | [9] | 26.024 | [9] | 25.976 |
| `- - 3 4 5 6 - 8 9` | 10 | 10 | [4, 6] | 26.024 | [4, 6] | 25.976 |
| `- - 3 4 5 6 - 8 9` | 10 | 11 | [5, 6] | 26.024 | [3, 8] | 25.976 |
| `- - 3 4 5 6 - 8 9` | 10 | 12 | [3, 9] | 26.024 | [3, 9] | 25.976 |
| `- 2 - 4 5 - 7 8 9` | 10 | 2 | [2] | 25.884 | [2] | 25.825 |
| `- 2 - 4 5 - 7 8 9` | 10 | 4 | [4] | 25.884 | [4] | 25.825 |
| `- 2 - 4 5 - 7 8 9` | 10 | 5 | [5] | 25.884 | [5] | 25.825 |
| `- 2 - 4 5 - 7 8 9` | 10 | 6 | [2, 4] | 25.884 | [2, 4] | 25.825 |
| `- 2 - 4 5 - 7 8 9` | 10 | 7 | [7] | 25.884 | [7] | 25.825 |
| `- 2 - 4 5 - 7 8 9` | 10 | 8 | [8] | 25.884 | [8] | 25.825 |
| `- 2 - 4 5 - 7 8 9` | 10 | 9 | [9] | 25.884 | [9] | 25.825 |
| `- 2 - 4 5 - 7 8 9` | 10 | 10 | [2, 8] | 25.884 | [2, 8] | 25.825 |
| `- 2 - 4 5 - 7 8 9` | 10 | 11 | [2, 9] | 25.884 | [2, 9] | 25.825 |
| `- 2 - 4 5 - 7 8 9` | 10 | 12 | [5, 7] | 25.884 | [4, 8] | 25.825 |
| `- 2 3 - - 6 7 8 9` | 10 | 2 | [2] | 25.672 | [2] | 25.663 |
| `- 2 3 - - 6 7 8 9` | 10 | 3 | [3] | 25.672 | [3] | 25.663 |
| `- 2 3 - - 6 7 8 9` | 10 | 5 | [2, 3] | 25.672 | [2, 3] | 25.663 |
| `- 2 3 - - 6 7 8 9` | 10 | 6 | [6] | 25.672 | [6] | 25.663 |
| `- 2 3 - - 6 7 8 9` | 10 | 7 | [7] | 25.672 | [7] | 25.663 |
| `- 2 3 - - 6 7 8 9` | 10 | 8 | [8] | 25.672 | [8] | 25.663 |
| `- 2 3 - - 6 7 8 9` | 10 | 9 | [9] | 25.672 | [9] | 25.663 |
| `- 2 3 - - 6 7 8 9` | 10 | 10 | [3, 7] | 25.672 | [2, 8] | 25.663 |
| `- 2 3 - - 6 7 8 9` | 10 | 11 | [2, 9] | 25.672 | [2, 9] | 25.663 |
| `- 2 3 - - 6 7 8 9` | 10 | 12 | [3, 9] | 25.672 | [3, 9] | 25.663 |
| `- 2 3 4 5 6 7 8 -` | 10 | 2 | [2] | 33.860 | [2] | 33.785 |
| `- 2 3 4 5 6 7 8 -` | 10 | 3 | [3] | 33.860 | [3] | 33.785 |
| `- 2 3 4 5 6 7 8 -` | 10 | 4 | [4] | 33.860 | [4] | 33.785 |
| `- 2 3 4 5 6 7 8 -` | 10 | 5 | [5] | 33.860 | [5] | 33.785 |
| `- 2 3 4 5 6 7 8 -` | 10 | 6 | [6] | 33.860 | [6] | 33.785 |
| `- 2 3 4 5 6 7 8 -` | 10 | 7 | [7] | 33.860 | [7] | 33.785 |
| `- 2 3 4 5 6 7 8 -` | 10 | 8 | [8] | 33.860 | [8] | 33.785 |
| `- 2 3 4 5 6 7 8 -` | 10 | 9 | [2, 7] | 33.860 | [2, 7] | 33.785 |
| `- 2 3 4 5 6 7 8 -` | 10 | 10 | [2, 8] | 33.860 | [2, 8] | 33.785 |
| `- 2 3 4 5 6 7 8 -` | 10 | 11 | [5, 6] | 33.860 | [3, 8] | 33.785 |
| `- 2 3 4 5 6 7 8 -` | 10 | 12 | [5, 7] | 33.860 | [4, 8] | 33.785 |
| `1 - - 4 - 6 7 8 9` | 10 | 4 | [4] | 26.089 | [4] | 26.087 |
| `1 - - 4 - 6 7 8 9` | 10 | 5 | [1, 4] | 26.089 | [1, 4] | 26.087 |
| `1 - - 4 - 6 7 8 9` | 10 | 6 | [6] | 26.089 | [6] | 26.087 |
| `1 - - 4 - 6 7 8 9` | 10 | 7 | [7] | 26.089 | [7] | 26.087 |
| `1 - - 4 - 6 7 8 9` | 10 | 8 | [8] | 26.089 | [8] | 26.087 |
| `1 - - 4 - 6 7 8 9` | 10 | 9 | [9] | 26.089 | [9] | 26.087 |
| `1 - - 4 - 6 7 8 9` | 10 | 10 | [1, 9] | 26.089 | [1, 9] | 26.087 |
| `1 - - 4 - 6 7 8 9` | 10 | 11 | [4, 7] | 26.089 | [4, 7] | 26.087 |
| `1 - - 4 - 6 7 8 9` | 10 | 12 | [4, 8] | 26.089 | [4, 8] | 26.087 |
| `1 - 3 4 5 6 7 - 9` | 10 | 3 | [3] | 32.078 | [3] | 31.984 |
| `1 - 3 4 5 6 7 - 9` | 10 | 4 | [4] | 32.078 | [4] | 31.984 |
| `1 - 3 4 5 6 7 - 9` | 10 | 5 | [5] | 32.078 | [5] | 31.984 |
| `1 - 3 4 5 6 7 - 9` | 10 | 6 | [6] | 32.078 | [6] | 31.984 |
| `1 - 3 4 5 6 7 - 9` | 10 | 7 | [7] | 32.078 | [7] | 31.984 |
| `1 - 3 4 5 6 7 - 9` | 10 | 8 | [1, 7] | 32.078 | [1, 7] | 31.984 |
| `1 - 3 4 5 6 7 - 9` | 10 | 9 | [9] | 32.078 | [9] | 31.984 |
| `1 - 3 4 5 6 7 - 9` | 10 | 10 | [1, 9] | 32.078 | [1, 9] | 31.984 |
| `1 - 3 4 5 6 7 - 9` | 10 | 11 | [4, 7] | 32.078 | [4, 7] | 31.984 |
| `1 - 3 4 5 6 7 - 9` | 10 | 12 | [3, 9] | 32.078 | [3, 9] | 31.984 |
| `1 2 - 4 5 6 - 8 9` | 10 | 2 | [2] | 31.684 | [2] | 31.624 |
| `1 2 - 4 5 6 - 8 9` | 10 | 3 | [1, 2] | 31.684 | [1, 2] | 31.624 |
| `1 2 - 4 5 6 - 8 9` | 10 | 4 | [4] | 31.684 | [4] | 31.624 |
| `1 2 - 4 5 6 - 8 9` | 10 | 5 | [5] | 31.684 | [5] | 31.624 |
| `1 2 - 4 5 6 - 8 9` | 10 | 6 | [6] | 31.684 | [6] | 31.624 |
| `1 2 - 4 5 6 - 8 9` | 10 | 7 | [1, 6] | 31.684 | [1, 6] | 31.624 |
| `1 2 - 4 5 6 - 8 9` | 10 | 8 | [8] | 31.684 | [8] | 31.624 |
| `1 2 - 4 5 6 - 8 9` | 10 | 9 | [9] | 31.684 | [9] | 31.624 |
| `1 2 - 4 5 6 - 8 9` | 10 | 10 | [1, 9] | 31.684 | [1, 9] | 31.624 |
| `1 2 - 4 5 6 - 8 9` | 10 | 11 | [2, 9] | 31.684 | [2, 9] | 31.624 |
| `1 2 - 4 5 6 - 8 9` | 10 | 12 | [4, 8] | 31.684 | [4, 8] | 31.624 |
| `1 2 3 - 5 - 7 8 9` | 10 | 2 | [2] | 31.333 | [2] | 31.107 |
| `1 2 3 - 5 - 7 8 9` | 10 | 3 | [3] | 31.333 | [3] | 31.107 |
| `1 2 3 - 5 - 7 8 9` | 10 | 4 | [1, 3] | 31.333 | [1, 3] | 31.107 |
| `1 2 3 - 5 - 7 8 9` | 10 | 5 | [5] | 31.333 | [5] | 31.107 |
| `1 2 3 - 5 - 7 8 9` | 10 | 6 | [1, 5] | 31.333 | [1, 5] | 31.107 |
| `1 2 3 - 5 - 7 8 9` | 10 | 7 | [7] | 31.333 | [7] | 31.107 |
| `1 2 3 - 5 - 7 8 9` | 10 | 8 | [8] | 31.333 | [8] | 31.107 |
| `1 2 3 - 5 - 7 8 9` | 10 | 9 | [9] | 31.333 | [9] | 31.107 |
| `1 2 3 - 5 - 7 8 9` | 10 | 10 | [2, 8] | 31.333 | [1, 9] | 31.107 |
| `1 2 3 - 5 - 7 8 9` | 10 | 11 | [2, 9] | 31.333 | [2, 9] | 31.107 |
| `1 2 3 - 5 - 7 8 9` | 10 | 12 | [3, 9] | 31.333 | [3, 9] | 31.107 |
| `- - - 4 - 6 7 8 9` | 11 | 4 | [4] | 22.320 | [4] | 22.320 |
| `- - - 4 - 6 7 8 9` | 11 | 6 | [6] | 22.320 | [6] | 22.320 |
| `- - - 4 - 6 7 8 9` | 11 | 7 | [7] | 22.320 | [7] | 22.320 |
| `- - - 4 - 6 7 8 9` | 11 | 8 | [8] | 22.320 | [8] | 22.320 |
| `- - - 4 - 6 7 8 9` | 11 | 9 | [9] | 22.320 | [9] | 22.320 |
| `- - - 4 - 6 7 8 9` | 11 | 10 | [4, 6] | 22.320 | [4, 6] | 22.320 |
| `- - - 4 - 6 7 8 9` | 11 | 11 | [4, 7] | 22.320 | [4, 7] | 22.320 |
| `- - - 4 - 6 7 8 9` | 11 | 12 | [4, 8] | 22.320 | [4, 8] | 22.320 |
| `- - 3 4 5 6 7 - 9` | 11 | 3 | [3] | 28.009 | [3] | 27.869 |
| `- - 3 4 5 6 7 - 9` | 11 | 4 | [4] | 28.009 | [4] | 27.869 |
| `- - 3 4 5 6 7 - 9` | 11 | 5 | [5] | 28.009 | [5] | 27.869 |
| `- - 3 4 5 6 7 - 9` | 11 | 6 | [6] | 28.009 | [6] | 27.869 |
| `- - 3 4 5 6 7 - 9` | 11 | 7 | [7] | 28.009 | [7] | 27.869 |
| `- - 3 4 5 6 7 - 9` | 11 | 8 | [3, 5] | 28.009 | [3, 5] | 27.869 |
| `- - 3 4 5 6 7 - 9` | 11 | 9 | [9] | 28.009 | [9] | 27.869 |
| `- - 3 4 5 6 7 - 9` | 11 | 10 | [4, 6] | 28.009 | [3, 7] | 27.869 |
| `- - 3 4 5 6 7 - 9` | 11 | 11 | [4, 7] | 28.009 | [4, 7] | 27.869 |
| `- - 3 4 5 6 7 - 9` | 11 | 12 | [3, 9] | 28.009 | [3, 9] | 27.869 |
| `- 2 - 4 5 6 - 8 9` | 11 | 2 | [2] | 27.189 | [2] | 27.049 |
| `- 2 - 4 5 6 - 8 9` | 11 | 4 | [4] | 27.189 | [4] | 27.049 |
| `- 2 - 4 5 6 - 8 9` | 11 | 5 | [5] | 27.189 | [5] | 27.049 |
| `- 2 - 4 5 6 - 8 9` | 11 | 6 | [6] | 27.189 | [6] | 27.049 |
| `- 2 - 4 5 6 - 8 9` | 11 | 7 | [2, 5] | 27.189 | [2, 5] | 27.049 |
| `- 2 - 4 5 6 - 8 9` | 11 | 8 | [8] | 27.189 | [8] | 27.049 |
| `- 2 - 4 5 6 - 8 9` | 11 | 9 | [9] | 27.189 | [9] | 27.049 |
| `- 2 - 4 5 6 - 8 9` | 11 | 10 | [4, 6] | 27.189 | [2, 8] | 27.049 |
| `- 2 - 4 5 6 - 8 9` | 11 | 11 | [2, 9] | 27.189 | [2, 9] | 27.049 |
| `- 2 - 4 5 6 - 8 9` | 11 | 12 | [4, 8] | 27.189 | [4, 8] | 27.049 |
| `- 2 3 - 5 - 7 8 9` | 11 | 2 | [2] | 24.420 | [2] | 24.410 |
| `- 2 3 - 5 - 7 8 9` | 11 | 3 | [3] | 24.420 | [3] | 24.410 |
| `- 2 3 - 5 - 7 8 9` | 11 | 5 | [5] | 24.420 | [5] | 24.410 |
| `- 2 3 - 5 - 7 8 9` | 11 | 7 | [7] | 24.420 | [7] | 24.410 |
| `- 2 3 - 5 - 7 8 9` | 11 | 8 | [8] | 24.420 | [8] | 24.410 |
| `- 2 3 - 5 - 7 8 9` | 11 | 9 | [9] | 24.420 | [9] | 24.410 |
| `- 2 3 - 5 - 7 8 9` | 11 | 10 | [2, 8] | 24.420 | [2, 8] | 24.410 |
| `- 2 3 - 5 - 7 8 9` | 11 | 11 | [2, 9] | 24.420 | [2, 9] | 24.410 |
| `- 2 3 - 5 - 7 8 9` | 11 | 12 | [3, 9] | 24.420 | [3, 9] | 24.410 |
| `1 - - 4 5 - 7 8 9` | 11 | 4 | [4] | 26.441 | [4] | 26.440 |
| `1 - - 4 5 - 7 8 9` | 11 | 5 | [5] | 26.441 | [5] | 26.440 |
| `1 - - 4 5 - 7 8 9` | 11 | 6 | [1, 5] | 26.441 | [1, 5] | 26.440 |
| `1 - - 4 5 - 7 8 9` | 11 | 7 | [7] | 26.441 | [7] | 26.440 |
| `1 - - 4 5 - 7 8 9` | 11 | 8 | [8] | 26.441 | [8] | 26.440 |
| `1 - - 4 5 - 7 8 9` | 11 | 9 | [9] | 26.441 | [9] | 26.440 |
| `1 - - 4 5 - 7 8 9` | 11 | 10 | [1, 9] | 26.441 | [1, 9] | 26.440 |
| `1 - - 4 5 - 7 8 9` | 11 | 11 | [4, 7] | 26.441 | [4, 7] | 26.440 |
| `1 - - 4 5 - 7 8 9` | 11 | 12 | [4, 8] | 26.441 | [4, 8] | 26.440 |
| `1 - 3 - - 6 7 8 9` | 11 | 3 | [3] | 25.885 | [3] | 25.885 |
| `1 - 3 - - 6 7 8 9` | 11 | 4 | [1, 3] | 25.885 | [1, 3] | 25.885 |
| `1 - 3 - - 6 7 8 9` | 11 | 6 | [6] | 25.885 | [6] | 25.885 |
| `1 - 3 - - 6 7 8 9` | 11 | 7 | [7] | 25.885 | [7] | 25.885 |
| `1 - 3 - - 6 7 8 9` | 11 | 8 | [8] | 25.885 | [8] | 25.885 |
| `1 - 3 - - 6 7 8 9` | 11 | 9 | [9] | 25.885 | [9] | 25.885 |
| `1 - 3 - - 6 7 8 9` | 11 | 10 | [1, 9] | 25.885 | [1, 9] | 25.885 |
| `1 - 3 - - 6 7 8 9` | 11 | 11 | [3, 8] | 25.885 | [3, 8] | 25.885 |
| `1 - 3 - - 6 7 8 9` | 11 | 12 | [3, 9] | 25.885 | [3, 9] | 25.885 |
| `1 - 3 4 5 6 7 8 -` | 11 | 3 | [3] | 33.806 | [3] | 33.705 |
| `1 - 3 4 5 6 7 8 -` | 11 | 4 | [4] | 33.806 | [4] | 33.705 |
| `1 - 3 4 5 6 7 8 -` | 11 | 5 | [5] | 33.806 | [5] | 33.705 |
| `1 - 3 4 5 6 7 8 -` | 11 | 6 | [6] | 33.806 | [6] | 33.705 |
| `1 - 3 4 5 6 7 8 -` | 11 | 7 | [7] | 33.806 | [7] | 33.705 |
| `1 - 3 4 5 6 7 8 -` | 11 | 8 | [8] | 33.806 | [8] | 33.705 |
| `1 - 3 4 5 6 7 8 -` | 11 | 9 | [1, 8] | 33.806 | [1, 8] | 33.705 |
| `1 - 3 4 5 6 7 8 -` | 11 | 10 | [4, 6] | 33.806 | [3, 7] | 33.705 |
| `1 - 3 4 5 6 7 8 -` | 11 | 11 | [3, 8] | 33.806 | [3, 8] | 33.705 |
| `1 - 3 4 5 6 7 8 -` | 11 | 12 | [4, 8] | 33.806 | [4, 8] | 33.705 |
| `1 2 - 4 5 6 7 - 9` | 11 | 2 | [2] | 33.146 | [2] | 33.019 |
| `1 2 - 4 5 6 7 - 9` | 11 | 3 | [1, 2] | 33.146 | [1, 2] | 33.019 |
| `1 2 - 4 5 6 7 - 9` | 11 | 4 | [4] | 33.146 | [4] | 33.019 |
| `1 2 - 4 5 6 7 - 9` | 11 | 5 | [5] | 33.146 | [5] | 33.019 |
| `1 2 - 4 5 6 7 - 9` | 11 | 6 | [6] | 33.146 | [6] | 33.019 |
| `1 2 - 4 5 6 7 - 9` | 11 | 7 | [7] | 33.146 | [7] | 33.019 |
| `1 2 - 4 5 6 7 - 9` | 11 | 8 | [1, 7] | 33.146 | [1, 7] | 33.019 |
| `1 2 - 4 5 6 7 - 9` | 11 | 9 | [9] | 33.146 | [9] | 33.019 |
| `1 2 - 4 5 6 7 - 9` | 11 | 10 | [1, 9] | 33.146 | [1, 9] | 33.019 |
| `1 2 - 4 5 6 7 - 9` | 11 | 11 | [2, 9] | 33.146 | [2, 9] | 33.019 |
| `1 2 - 4 5 6 7 - 9` | 11 | 12 | [5, 7] | 33.146 | [5, 7] | 33.019 |
| `1 2 3 - 5 6 - 8 9` | 11 | 2 | [2] | 32.395 | [2] | 32.075 |
| `1 2 3 - 5 6 - 8 9` | 11 | 3 | [3] | 32.395 | [3] | 32.075 |
| `1 2 3 - 5 6 - 8 9` | 11 | 4 | [1, 3] | 32.395 | [1, 3] | 32.075 |
| `1 2 3 - 5 6 - 8 9` | 11 | 5 | [5] | 32.395 | [5] | 32.075 |
| `1 2 3 - 5 6 - 8 9` | 11 | 6 | [6] | 32.395 | [6] | 32.075 |
| `1 2 3 - 5 6 - 8 9` | 11 | 7 | [2, 5] | 32.395 | [1, 6] | 32.075 |
| `1 2 3 - 5 6 - 8 9` | 11 | 8 | [8] | 32.395 | [8] | 32.075 |
| `1 2 3 - 5 6 - 8 9` | 11 | 9 | [9] | 32.395 | [9] | 32.075 |
| `1 2 3 - 5 6 - 8 9` | 11 | 10 | [2, 8] | 32.395 | [1, 9] | 32.075 |
| `1 2 3 - 5 6 - 8 9` | 11 | 11 | [2, 9] | 32.395 | [2, 9] | 32.075 |
| `1 2 3 - 5 6 - 8 9` | 11 | 12 | [3, 9] | 32.395 | [3, 9] | 32.075 |
| `1 2 3 4 - - 7 8 9` | 11 | 2 | [2] | 32.123 | [2] | 31.989 |
| `1 2 3 4 - - 7 8 9` | 11 | 3 | [3] | 32.123 | [3] | 31.989 |
| `1 2 3 4 - - 7 8 9` | 11 | 4 | [4] | 32.123 | [4] | 31.989 |
| `1 2 3 4 - - 7 8 9` | 11 | 5 | [1, 4] | 32.123 | [1, 4] | 31.989 |
| `1 2 3 4 - - 7 8 9` | 11 | 6 | [2, 4] | 32.123 | [2, 4] | 31.989 |
| `1 2 3 4 - - 7 8 9` | 11 | 7 | [7] | 32.123 | [7] | 31.989 |
| `1 2 3 4 - - 7 8 9` | 11 | 8 | [8] | 32.123 | [8] | 31.989 |
| `1 2 3 4 - - 7 8 9` | 11 | 9 | [9] | 32.123 | [9] | 31.989 |
| `1 2 3 4 - - 7 8 9` | 11 | 10 | [1, 9] | 32.123 | [1, 9] | 31.989 |
| `1 2 3 4 - - 7 8 9` | 11 | 11 | [3, 8] | 32.123 | [2, 9] | 31.989 |
| `1 2 3 4 - - 7 8 9` | 11 | 12 | [3, 9] | 32.123 | [3, 9] | 31.989 |
| `- - - 4 5 - 7 8 9` | 12 | 4 | [4] | 21.159 | [4] | 21.159 |
| `- - - 4 5 - 7 8 9` | 12 | 5 | [5] | 21.159 | [5] | 21.159 |
| `- - - 4 5 - 7 8 9` | 12 | 7 | [7] | 21.159 | [7] | 21.159 |
| `- - - 4 5 - 7 8 9` | 12 | 8 | [8] | 21.159 | [8] | 21.159 |
| `- - - 4 5 - 7 8 9` | 12 | 9 | [9] | 21.159 | [9] | 21.159 |
| `- - - 4 5 - 7 8 9` | 12 | 11 | [4, 7] | 21.159 | [4, 7] | 21.159 |
| `- - - 4 5 - 7 8 9` | 12 | 12 | [4, 8] | 21.159 | [4, 8] | 21.159 |
| `- - 3 - - 6 7 8 9` | 12 | 3 | [3] | 22.962 | [3] | 22.962 |
| `- - 3 - - 6 7 8 9` | 12 | 6 | [6] | 22.962 | [6] | 22.962 |
| `- - 3 - - 6 7 8 9` | 12 | 7 | [7] | 22.962 | [7] | 22.962 |
| `- - 3 - - 6 7 8 9` | 12 | 8 | [8] | 22.962 | [8] | 22.962 |
| `- - 3 - - 6 7 8 9` | 12 | 9 | [9] | 22.962 | [9] | 22.962 |
| `- - 3 - - 6 7 8 9` | 12 | 10 | [3, 7] | 22.962 | [3, 7] | 22.962 |
| `- - 3 - - 6 7 8 9` | 12 | 11 | [3, 8] | 22.962 | [3, 8] | 22.962 |
| `- - 3 - - 6 7 8 9` | 12 | 12 | [3, 9] | 22.962 | [3, 9] | 22.962 |
| `- - 3 4 5 6 7 8 -` | 12 | 3 | [3] | 30.311 | [3] | 30.192 |
| `- - 3 4 5 6 7 8 -` | 12 | 4 | [4] | 30.311 | [4] | 30.192 |
| `- - 3 4 5 6 7 8 -` | 12 | 5 | [5] | 30.311 | [5] | 30.192 |
| `- - 3 4 5 6 7 8 -` | 12 | 6 | [6] | 30.311 | [6] | 30.192 |
| `- - 3 4 5 6 7 8 -` | 12 | 7 | [7] | 30.311 | [7] | 30.192 |
| `- - 3 4 5 6 7 8 -` | 12 | 8 | [8] | 30.311 | [8] | 30.192 |
| `- - 3 4 5 6 7 8 -` | 12 | 9 | [4, 5] | 30.311 | [3, 6] | 30.192 |
| `- - 3 4 5 6 7 8 -` | 12 | 10 | [3, 7] | 30.311 | [3, 7] | 30.192 |
| `- - 3 4 5 6 7 8 -` | 12 | 11 | [3, 8] | 30.311 | [3, 8] | 30.192 |
| `- - 3 4 5 6 7 8 -` | 12 | 12 | [4, 8] | 30.311 | [4, 8] | 30.192 |
| `- 2 - 4 5 6 7 - 9` | 12 | 2 | [2] | 27.930 | [2] | 27.927 |
| `- 2 - 4 5 6 7 - 9` | 12 | 4 | [4] | 27.930 | [4] | 27.927 |
| `- 2 - 4 5 6 7 - 9` | 12 | 5 | [5] | 27.930 | [5] | 27.927 |
| `- 2 - 4 5 6 7 - 9` | 12 | 6 | [6] | 27.930 | [6] | 27.927 |
| `- 2 - 4 5 6 7 - 9` | 12 | 7 | [7] | 27.930 | [7] | 27.927 |
| `- 2 - 4 5 6 7 - 9` | 12 | 8 | [2, 6] | 27.930 | [2, 6] | 27.927 |
| `- 2 - 4 5 6 7 - 9` | 12 | 9 | [9] | 27.930 | [9] | 27.927 |
| `- 2 - 4 5 6 7 - 9` | 12 | 10 | [4, 6] | 27.930 | [4, 6] | 27.927 |
| `- 2 - 4 5 6 7 - 9` | 12 | 11 | [2, 9] | 27.930 | [2, 9] | 27.927 |
| `- 2 - 4 5 6 7 - 9` | 12 | 12 | [5, 7] | 27.930 | [5, 7] | 27.927 |
| `- 2 3 - 5 6 - 8 9` | 12 | 2 | [2] | 27.142 | [2] | 27.052 |
| `- 2 3 - 5 6 - 8 9` | 12 | 3 | [3] | 27.142 | [3] | 27.052 |
| `- 2 3 - 5 6 - 8 9` | 12 | 5 | [5] | 27.142 | [5] | 27.052 |
| `- 2 3 - 5 6 - 8 9` | 12 | 6 | [6] | 27.142 | [6] | 27.052 |
| `- 2 3 - 5 6 - 8 9` | 12 | 7 | [2, 5] | 27.142 | [2, 5] | 27.052 |
| `- 2 3 - 5 6 - 8 9` | 12 | 8 | [8] | 27.142 | [8] | 27.052 |
| `- 2 3 - 5 6 - 8 9` | 12 | 9 | [9] | 27.142 | [9] | 27.052 |
| `- 2 3 - 5 6 - 8 9` | 12 | 10 | [2, 8] | 27.142 | [2, 8] | 27.052 |
| `- 2 3 - 5 6 - 8 9` | 12 | 11 | [3, 8] | 27.142 | [2, 9] | 27.052 |
| `- 2 3 - 5 6 - 8 9` | 12 | 12 | [3, 9] | 27.142 | [3, 9] | 27.052 |
| `- 2 3 4 - - 7 8 9` | 12 | 2 | [2] | 28.868 | [2] | 28.800 |
| `- 2 3 4 - - 7 8 9` | 12 | 3 | [3] | 28.868 | [3] | 28.800 |
| `- 2 3 4 - - 7 8 9` | 12 | 4 | [4] | 28.868 | [4] | 28.800 |
| `- 2 3 4 - - 7 8 9` | 12 | 5 | [2, 3] | 28.868 | [2, 3] | 28.800 |
| `- 2 3 4 - - 7 8 9` | 12 | 6 | [2, 4] | 28.868 | [2, 4] | 28.800 |
| `- 2 3 4 - - 7 8 9` | 12 | 7 | [7] | 28.868 | [7] | 28.800 |
| `- 2 3 4 - - 7 8 9` | 12 | 8 | [8] | 28.868 | [8] | 28.800 |
| `- 2 3 4 - - 7 8 9` | 12 | 9 | [9] | 28.868 | [9] | 28.800 |
| `- 2 3 4 - - 7 8 9` | 12 | 10 | [3, 7] | 28.868 | [2, 8] | 28.800 |
| `- 2 3 4 - - 7 8 9` | 12 | 11 | [2, 9] | 28.868 | [2, 9] | 28.800 |
| `- 2 3 4 - - 7 8 9` | 12 | 12 | [3, 9] | 28.868 | [3, 9] | 28.800 |
| `1 - - 4 5 6 - 8 9` | 12 | 4 | [4] | 26.967 | [4] | 26.966 |
| `1 - - 4 5 6 - 8 9` | 12 | 5 | [5] | 26.967 | [5] | 26.966 |
| `1 - - 4 5 6 - 8 9` | 12 | 6 | [6] | 26.967 | [6] | 26.966 |
| `1 - - 4 5 6 - 8 9` | 12 | 7 | [1, 6] | 26.967 | [1, 6] | 26.966 |
| `1 - - 4 5 6 - 8 9` | 12 | 8 | [8] | 26.967 | [8] | 26.966 |
| `1 - - 4 5 6 - 8 9` | 12 | 9 | [9] | 26.967 | [9] | 26.966 |
| `1 - - 4 5 6 - 8 9` | 12 | 10 | [1, 9] | 26.967 | [1, 9] | 26.966 |
| `1 - - 4 5 6 - 8 9` | 12 | 11 | [5, 6] | 26.967 | [5, 6] | 26.966 |
| `1 - - 4 5 6 - 8 9` | 12 | 12 | [4, 8] | 26.967 | [4, 8] | 26.966 |
| `1 - 3 - 5 - 7 8 9` | 12 | 3 | [3] | 28.349 | [3] | 28.326 |
| `1 - 3 - 5 - 7 8 9` | 12 | 4 | [1, 3] | 28.349 | [1, 3] | 28.326 |
| `1 - 3 - 5 - 7 8 9` | 12 | 5 | [5] | 28.349 | [5] | 28.326 |
| `1 - 3 - 5 - 7 8 9` | 12 | 6 | [1, 5] | 28.349 | [1, 5] | 28.326 |
| `1 - 3 - 5 - 7 8 9` | 12 | 7 | [7] | 28.349 | [7] | 28.326 |
| `1 - 3 - 5 - 7 8 9` | 12 | 8 | [8] | 28.349 | [8] | 28.326 |
| `1 - 3 - 5 - 7 8 9` | 12 | 9 | [9] | 28.349 | [9] | 28.326 |
| `1 - 3 - 5 - 7 8 9` | 12 | 10 | [1, 9] | 28.349 | [1, 9] | 28.326 |
| `1 - 3 - 5 - 7 8 9` | 12 | 11 | [3, 8] | 28.349 | [3, 8] | 28.326 |
| `1 - 3 - 5 - 7 8 9` | 12 | 12 | [3, 9] | 28.349 | [3, 9] | 28.326 |
| `1 2 - - - 6 7 8 9` | 12 | 2 | [2] | 25.576 | [2] | 25.549 |
| `1 2 - - - 6 7 8 9` | 12 | 3 | [1, 2] | 25.576 | [1, 2] | 25.549 |
| `1 2 - - - 6 7 8 9` | 12 | 6 | [6] | 25.576 | [6] | 25.549 |
| `1 2 - - - 6 7 8 9` | 12 | 7 | [7] | 25.576 | [7] | 25.549 |
| `1 2 - - - 6 7 8 9` | 12 | 8 | [8] | 25.576 | [8] | 25.549 |
| `1 2 - - - 6 7 8 9` | 12 | 9 | [9] | 25.576 | [9] | 25.549 |
| `1 2 - - - 6 7 8 9` | 12 | 10 | [1, 9] | 25.576 | [1, 9] | 25.549 |
| `1 2 - - - 6 7 8 9` | 12 | 11 | [2, 9] | 25.576 | [2, 9] | 25.549 |
| `1 2 - - - 6 7 8 9` | 12 | 12 | [1, 2, 9] | 25.576 | [1, 2, 9] | 25.549 |
| `1 2 - 4 5 6 7 8 -` | 12 | 2 | [2] | 35.064 | [2] | 34.938 |
| `1 2 - 4 5 6 7 8 -` | 12 | 3 | [1, 2] | 35.064 | [1, 2] | 34.938 |
| `1 2 - 4 5 6 7 8 -` | 12 | 4 | [4] | 35.064 | [4] | 34.938 |
| `1 2 - 4 5 6 7 8 -` | 12 | 5 | [5] | 35.064 | [5] | 34.938 |
| `1 2 - 4 5 6 7 8 -` | 12 | 6 | [6] | 35.064 | [6] | 34.938 |
| `1 2 - 4 5 6 7 8 -` | 12 | 7 | [7] | 35.064 | [7] | 34.938 |
| `1 2 - 4 5 6 7 8 -` | 12 | 8 | [8] | 35.064 | [8] | 34.938 |
| `1 2 - 4 5 6 7 8 -` | 12 | 9 | [1, 8] | 35.064 | [1, 8] | 34.938 |
| `1 2 - 4 5 6 7 8 -` | 12 | 10 | [2, 8] | 35.064 | [2, 8] | 34.938 |
| `1 2 - 4 5 6 7 8 -` | 12 | 11 | [5, 6] | 35.064 | [4, 7] | 34.938 |
| `1 2 - 4 5 6 7 8 -` | 12 | 12 | [5, 7] | 35.064 | [4, 8] | 34.938 |
| `1 2 3 - 5 6 7 - 9` | 12 | 2 | [2] | 34.134 | [2] | 33.900 |
| `1 2 3 - 5 6 7 - 9` | 12 | 3 | [3] | 34.134 | [3] | 33.900 |
| `1 2 3 - 5 6 7 - 9` | 12 | 4 | [1, 3] | 34.134 | [1, 3] | 33.900 |
| `1 2 3 - 5 6 7 - 9` | 12 | 5 | [5] | 34.134 | [5] | 33.900 |
| `1 2 3 - 5 6 7 - 9` | 12 | 6 | [6] | 34.134 | [6] | 33.900 |
| `1 2 3 - 5 6 7 - 9` | 12 | 7 | [7] | 34.134 | [7] | 33.900 |
| `1 2 3 - 5 6 7 - 9` | 12 | 8 | [2, 6] | 34.134 | [1, 7] | 33.900 |
| `1 2 3 - 5 6 7 - 9` | 12 | 9 | [9] | 34.134 | [9] | 33.900 |
| `1 2 3 - 5 6 7 - 9` | 12 | 10 | [1, 9] | 34.134 | [1, 9] | 33.900 |
| `1 2 3 - 5 6 7 - 9` | 12 | 11 | [2, 9] | 34.134 | [2, 9] | 33.900 |
| `1 2 3 - 5 6 7 - 9` | 12 | 12 | [3, 9] | 34.134 | [3, 9] | 33.900 |
| `1 2 3 4 - 6 - 8 9` | 12 | 2 | [2] | 33.649 | [2] | 33.313 |
| `1 2 3 4 - 6 - 8 9` | 12 | 3 | [3] | 33.649 | [3] | 33.313 |
| `1 2 3 4 - 6 - 8 9` | 12 | 4 | [4] | 33.649 | [4] | 33.313 |
| `1 2 3 4 - 6 - 8 9` | 12 | 5 | [2, 3] | 33.649 | [1, 4] | 33.313 |
| `1 2 3 4 - 6 - 8 9` | 12 | 6 | [6] | 33.649 | [6] | 33.313 |
| `1 2 3 4 - 6 - 8 9` | 12 | 7 | [1, 6] | 33.649 | [1, 6] | 33.313 |
| `1 2 3 4 - 6 - 8 9` | 12 | 8 | [8] | 33.649 | [8] | 33.313 |
| `1 2 3 4 - 6 - 8 9` | 12 | 9 | [9] | 33.649 | [9] | 33.313 |
| `1 2 3 4 - 6 - 8 9` | 12 | 10 | [1, 9] | 33.649 | [1, 9] | 33.313 |
| `1 2 3 4 - 6 - 8 9` | 12 | 11 | [2, 9] | 33.649 | [2, 9] | 33.313 |
| `1 2 3 4 - 6 - 8 9` | 12 | 12 | [3, 9] | 33.649 | [3, 9] | 33.313 |
| `- - - 4 5 6 - 8 9` | 13 | 4 | [4] | 22.839 | [4] | 22.839 |
| `- - - 4 5 6 - 8 9` | 13 | 5 | [5] | 22.839 | [5] | 22.839 |
| `- - - 4 5 6 - 8 9` | 13 | 6 | [6] | 22.839 | [6] | 22.839 |
| `- - - 4 5 6 - 8 9` | 13 | 8 | [8] | 22.839 | [8] | 22.839 |
| `- - - 4 5 6 - 8 9` | 13 | 9 | [9] | 22.839 | [9] | 22.839 |
| `- - - 4 5 6 - 8 9` | 13 | 10 | [4, 6] | 22.839 | [4, 6] | 22.839 |
| `- - - 4 5 6 - 8 9` | 13 | 11 | [5, 6] | 22.839 | [5, 6] | 22.839 |
| `- - - 4 5 6 - 8 9` | 13 | 12 | [4, 8] | 22.839 | [4, 8] | 22.839 |
| `- - 3 - 5 - 7 8 9` | 13 | 3 | [3] | 23.335 | [3] | 23.335 |
| `- - 3 - 5 - 7 8 9` | 13 | 5 | [5] | 23.335 | [5] | 23.335 |
| `- - 3 - 5 - 7 8 9` | 13 | 7 | [7] | 23.335 | [7] | 23.335 |
| `- - 3 - 5 - 7 8 9` | 13 | 8 | [8] | 23.335 | [8] | 23.335 |
| `- - 3 - 5 - 7 8 9` | 13 | 9 | [9] | 23.335 | [9] | 23.335 |
| `- - 3 - 5 - 7 8 9` | 13 | 10 | [3, 7] | 23.335 | [3, 7] | 23.335 |
| `- - 3 - 5 - 7 8 9` | 13 | 11 | [3, 8] | 23.335 | [3, 8] | 23.335 |
| `- - 3 - 5 - 7 8 9` | 13 | 12 | [3, 9] | 23.335 | [3, 9] | 23.335 |
| `- 2 - - - 6 7 8 9` | 13 | 2 | [2] | 23.225 | [2] | 23.225 |
| `- 2 - - - 6 7 8 9` | 13 | 6 | [6] | 23.225 | [6] | 23.225 |
| `- 2 - - - 6 7 8 9` | 13 | 7 | [7] | 23.225 | [7] | 23.225 |
| `- 2 - - - 6 7 8 9` | 13 | 8 | [8] | 23.225 | [8] | 23.225 |
| `- 2 - - - 6 7 8 9` | 13 | 9 | [9] | 23.225 | [9] | 23.225 |
| `- 2 - - - 6 7 8 9` | 13 | 10 | [2, 8] | 23.225 | [2, 8] | 23.225 |
| `- 2 - - - 6 7 8 9` | 13 | 11 | [2, 9] | 23.225 | [2, 9] | 23.225 |
| `- 2 - 4 5 6 7 8 -` | 13 | 2 | [2] | 30.745 | [2] | 30.641 |
| `- 2 - 4 5 6 7 8 -` | 13 | 4 | [4] | 30.745 | [4] | 30.641 |
| `- 2 - 4 5 6 7 8 -` | 13 | 5 | [5] | 30.745 | [5] | 30.641 |
| `- 2 - 4 5 6 7 8 -` | 13 | 6 | [6] | 30.745 | [6] | 30.641 |
| `- 2 - 4 5 6 7 8 -` | 13 | 7 | [7] | 30.745 | [7] | 30.641 |
| `- 2 - 4 5 6 7 8 -` | 13 | 8 | [8] | 30.745 | [8] | 30.641 |
| `- 2 - 4 5 6 7 8 -` | 13 | 9 | [2, 7] | 30.745 | [2, 7] | 30.641 |
| `- 2 - 4 5 6 7 8 -` | 13 | 10 | [2, 8] | 30.745 | [2, 8] | 30.641 |
| `- 2 - 4 5 6 7 8 -` | 13 | 11 | [5, 6] | 30.745 | [4, 7] | 30.641 |
| `- 2 - 4 5 6 7 8 -` | 13 | 12 | [4, 8] | 30.745 | [4, 8] | 30.641 |
| `- 2 3 - 5 6 7 - 9` | 13 | 2 | [2] | 29.283 | [2] | 29.268 |
| `- 2 3 - 5 6 7 - 9` | 13 | 3 | [3] | 29.283 | [3] | 29.268 |
| `- 2 3 - 5 6 7 - 9` | 13 | 5 | [5] | 29.283 | [5] | 29.268 |
| `- 2 3 - 5 6 7 - 9` | 13 | 6 | [6] | 29.283 | [6] | 29.268 |
| `- 2 3 - 5 6 7 - 9` | 13 | 7 | [7] | 29.283 | [7] | 29.268 |
| `- 2 3 - 5 6 7 - 9` | 13 | 8 | [2, 6] | 29.283 | [2, 6] | 29.268 |
| `- 2 3 - 5 6 7 - 9` | 13 | 9 | [9] | 29.283 | [9] | 29.268 |
| `- 2 3 - 5 6 7 - 9` | 13 | 10 | [3, 7] | 29.283 | [3, 7] | 29.268 |
| `- 2 3 - 5 6 7 - 9` | 13 | 11 | [2, 9] | 29.283 | [2, 9] | 29.268 |
| `- 2 3 - 5 6 7 - 9` | 13 | 12 | [3, 9] | 29.283 | [3, 9] | 29.268 |
| `- 2 3 4 - 6 - 8 9` | 13 | 2 | [2] | 29.572 | [2] | 29.533 |
| `- 2 3 4 - 6 - 8 9` | 13 | 3 | [3] | 29.572 | [3] | 29.533 |
| `- 2 3 4 - 6 - 8 9` | 13 | 4 | [4] | 29.572 | [4] | 29.533 |
| `- 2 3 4 - 6 - 8 9` | 13 | 5 | [2, 3] | 29.572 | [2, 3] | 29.533 |
| `- 2 3 4 - 6 - 8 9` | 13 | 6 | [6] | 29.572 | [6] | 29.533 |
| `- 2 3 4 - 6 - 8 9` | 13 | 7 | [3, 4] | 29.572 | [3, 4] | 29.533 |
| `- 2 3 4 - 6 - 8 9` | 13 | 8 | [8] | 29.572 | [8] | 29.533 |
| `- 2 3 4 - 6 - 8 9` | 13 | 9 | [9] | 29.572 | [9] | 29.533 |
| `- 2 3 4 - 6 - 8 9` | 13 | 10 | [2, 8] | 29.572 | [2, 8] | 29.533 |
| `- 2 3 4 - 6 - 8 9` | 13 | 11 | [2, 9] | 29.572 | [2, 9] | 29.533 |
| `- 2 3 4 - 6 - 8 9` | 13 | 12 | [4, 8] | 29.572 | [3, 9] | 29.533 |
| `1 - - 4 5 6 7 - 9` | 13 | 4 | [4] | 29.029 | [4] | 28.980 |
| `1 - - 4 5 6 7 - 9` | 13 | 5 | [5] | 29.029 | [5] | 28.980 |
| `1 - - 4 5 6 7 - 9` | 13 | 6 | [6] | 29.029 | [6] | 28.980 |
| `1 - - 4 5 6 7 - 9` | 13 | 7 | [7] | 29.029 | [7] | 28.980 |
| `1 - - 4 5 6 7 - 9` | 13 | 8 | [1, 7] | 29.029 | [1, 7] | 28.980 |
| `1 - - 4 5 6 7 - 9` | 13 | 9 | [9] | 29.029 | [9] | 28.980 |
| `1 - - 4 5 6 7 - 9` | 13 | 10 | [1, 9] | 29.029 | [1, 9] | 28.980 |
| `1 - - 4 5 6 7 - 9` | 13 | 11 | [5, 6] | 29.029 | [4, 7] | 28.980 |
| `1 - - 4 5 6 7 - 9` | 13 | 12 | [5, 7] | 29.029 | [5, 7] | 28.980 |
| `1 - 3 - 5 6 - 8 9` | 13 | 3 | [3] | 28.874 | [3] | 28.874 |
| `1 - 3 - 5 6 - 8 9` | 13 | 4 | [1, 3] | 28.874 | [1, 3] | 28.874 |
| `1 - 3 - 5 6 - 8 9` | 13 | 5 | [5] | 28.874 | [5] | 28.874 |
| `1 - 3 - 5 6 - 8 9` | 13 | 6 | [6] | 28.874 | [6] | 28.874 |
| `1 - 3 - 5 6 - 8 9` | 13 | 7 | [1, 6] | 28.874 | [1, 6] | 28.874 |
| `1 - 3 - 5 6 - 8 9` | 13 | 8 | [8] | 28.874 | [8] | 28.874 |
| `1 - 3 - 5 6 - 8 9` | 13 | 9 | [9] | 28.874 | [9] | 28.874 |
| `1 - 3 - 5 6 - 8 9` | 13 | 10 | [1, 9] | 28.874 | [1, 9] | 28.874 |
| `1 - 3 - 5 6 - 8 9` | 13 | 11 | [3, 8] | 28.874 | [3, 8] | 28.874 |
| `1 - 3 - 5 6 - 8 9` | 13 | 12 | [3, 9] | 28.874 | [3, 9] | 28.874 |
| `1 - 3 4 - - 7 8 9` | 13 | 3 | [3] | 27.045 | [3] | 27.043 |
| `1 - 3 4 - - 7 8 9` | 13 | 4 | [4] | 27.045 | [4] | 27.043 |
| `1 - 3 4 - - 7 8 9` | 13 | 5 | [1, 4] | 27.045 | [1, 4] | 27.043 |
| `1 - 3 4 - - 7 8 9` | 13 | 7 | [7] | 27.045 | [7] | 27.043 |
| `1 - 3 4 - - 7 8 9` | 13 | 8 | [8] | 27.045 | [8] | 27.043 |
| `1 - 3 4 - - 7 8 9` | 13 | 9 | [9] | 27.045 | [9] | 27.043 |
| `1 - 3 4 - - 7 8 9` | 13 | 10 | [1, 9] | 27.045 | [1, 9] | 27.043 |
| `1 - 3 4 - - 7 8 9` | 13 | 11 | [3, 8] | 27.045 | [3, 8] | 27.043 |
| `1 - 3 4 - - 7 8 9` | 13 | 12 | [4, 8] | 27.045 | [3, 9] | 27.043 |
| `1 2 - - 5 - 7 8 9` | 13 | 2 | [2] | 28.519 | [2] | 28.441 |
| `1 2 - - 5 - 7 8 9` | 13 | 3 | [1, 2] | 28.519 | [1, 2] | 28.441 |
| `1 2 - - 5 - 7 8 9` | 13 | 5 | [5] | 28.519 | [5] | 28.441 |
| `1 2 - - 5 - 7 8 9` | 13 | 6 | [1, 5] | 28.519 | [1, 5] | 28.441 |
| `1 2 - - 5 - 7 8 9` | 13 | 7 | [7] | 28.519 | [7] | 28.441 |
| `1 2 - - 5 - 7 8 9` | 13 | 8 | [8] | 28.519 | [8] | 28.441 |
| `1 2 - - 5 - 7 8 9` | 13 | 9 | [9] | 28.519 | [9] | 28.441 |
| `1 2 - - 5 - 7 8 9` | 13 | 10 | [2, 8] | 28.519 | [1, 9] | 28.441 |
| `1 2 - - 5 - 7 8 9` | 13 | 11 | [2, 9] | 28.519 | [2, 9] | 28.441 |
| `1 2 - - 5 - 7 8 9` | 13 | 12 | [5, 7] | 28.519 | [5, 7] | 28.441 |
| `1 2 3 - 5 6 7 8 -` | 13 | 2 | [2] | 35.639 | [2] | 35.521 |
| `1 2 3 - 5 6 7 8 -` | 13 | 3 | [3] | 35.639 | [3] | 35.521 |
| `1 2 3 - 5 6 7 8 -` | 13 | 4 | [1, 3] | 35.639 | [1, 3] | 35.521 |
| `1 2 3 - 5 6 7 8 -` | 13 | 5 | [5] | 35.639 | [5] | 35.521 |
| `1 2 3 - 5 6 7 8 -` | 13 | 6 | [6] | 35.639 | [6] | 35.521 |
| `1 2 3 - 5 6 7 8 -` | 13 | 7 | [7] | 35.639 | [7] | 35.521 |
| `1 2 3 - 5 6 7 8 -` | 13 | 8 | [8] | 35.639 | [8] | 35.521 |
| `1 2 3 - 5 6 7 8 -` | 13 | 9 | [1, 8] | 35.639 | [1, 8] | 35.521 |
| `1 2 3 - 5 6 7 8 -` | 13 | 10 | [2, 8] | 35.639 | [2, 8] | 35.521 |
| `1 2 3 - 5 6 7 8 -` | 13 | 11 | [5, 6] | 35.639 | [3, 8] | 35.521 |
| `1 2 3 - 5 6 7 8 -` | 13 | 12 | [5, 7] | 35.639 | [5, 7] | 35.521 |
| `1 2 3 4 - 6 7 - 9` | 13 | 2 | [2] | 35.097 | [2] | 34.956 |
| `1 2 3 4 - 6 7 - 9` | 13 | 3 | [3] | 35.097 | [3] | 34.956 |
| `1 2 3 4 - 6 7 - 9` | 13 | 4 | [4] | 35.097 | [4] | 34.956 |
| `1 2 3 4 - 6 7 - 9` | 13 | 5 | [2, 3] | 35.097 | [1, 4] | 34.956 |
| `1 2 3 4 - 6 7 - 9` | 13 | 6 | [6] | 35.097 | [6] | 34.956 |
| `1 2 3 4 - 6 7 - 9` | 13 | 7 | [7] | 35.097 | [7] | 34.956 |
| `1 2 3 4 - 6 7 - 9` | 13 | 8 | [1, 7] | 35.097 | [1, 7] | 34.956 |
| `1 2 3 4 - 6 7 - 9` | 13 | 9 | [9] | 35.097 | [9] | 34.956 |
| `1 2 3 4 - 6 7 - 9` | 13 | 10 | [1, 9] | 35.097 | [1, 9] | 34.956 |
| `1 2 3 4 - 6 7 - 9` | 13 | 11 | [2, 9] | 35.097 | [2, 9] | 34.956 |
| `1 2 3 4 - 6 7 - 9` | 13 | 12 | [3, 9] | 35.097 | [3, 9] | 34.956 |
| `1 2 3 4 5 - - 8 9` | 13 | 2 | [2] | 34.274 | [2] | 33.982 |
| `1 2 3 4 5 - - 8 9` | 13 | 3 | [3] | 34.274 | [3] | 33.982 |
| `1 2 3 4 5 - - 8 9` | 13 | 4 | [4] | 34.274 | [4] | 33.982 |
| `1 2 3 4 5 - - 8 9` | 13 | 5 | [5] | 34.274 | [5] | 33.982 |
| `1 2 3 4 5 - - 8 9` | 13 | 6 | [1, 5] | 34.274 | [1, 5] | 33.982 |
| `1 2 3 4 5 - - 8 9` | 13 | 7 | [3, 4] | 34.274 | [2, 5] | 33.982 |
| `1 2 3 4 5 - - 8 9` | 13 | 8 | [8] | 34.274 | [8] | 33.982 |
| `1 2 3 4 5 - - 8 9` | 13 | 9 | [9] | 34.274 | [9] | 33.982 |
| `1 2 3 4 5 - - 8 9` | 13 | 10 | [1, 9] | 34.274 | [1, 9] | 33.982 |
| `1 2 3 4 5 - - 8 9` | 13 | 11 | [2, 9] | 34.274 | [2, 9] | 33.982 |
| `1 2 3 4 5 - - 8 9` | 13 | 12 | [3, 9] | 34.274 | [3, 9] | 33.982 |
| `- - - 4 5 6 7 - 9` | 14 | 4 | [4] | 24.448 | [4] | 24.442 |
| `- - - 4 5 6 7 - 9` | 14 | 5 | [5] | 24.448 | [5] | 24.442 |
| `- - - 4 5 6 7 - 9` | 14 | 6 | [6] | 24.448 | [6] | 24.442 |
| `- - - 4 5 6 7 - 9` | 14 | 7 | [7] | 24.448 | [7] | 24.442 |
| `- - - 4 5 6 7 - 9` | 14 | 9 | [9] | 24.448 | [9] | 24.442 |
| `- - - 4 5 6 7 - 9` | 14 | 10 | [4, 6] | 24.448 | [4, 6] | 24.442 |
| `- - - 4 5 6 7 - 9` | 14 | 11 | [5, 6] | 24.448 | [4, 7] | 24.442 |
| `- - - 4 5 6 7 - 9` | 14 | 12 | [5, 7] | 24.448 | [5, 7] | 24.442 |
| `- - 3 - 5 6 - 8 9` | 14 | 3 | [3] | 22.328 | [3] | 22.322 |
| `- - 3 - 5 6 - 8 9` | 14 | 5 | [5] | 22.328 | [5] | 22.322 |
| `- - 3 - 5 6 - 8 9` | 14 | 6 | [6] | 22.328 | [6] | 22.322 |
| `- - 3 - 5 6 - 8 9` | 14 | 8 | [8] | 22.328 | [8] | 22.322 |
| `- - 3 - 5 6 - 8 9` | 14 | 9 | [9] | 22.328 | [9] | 22.322 |
| `- - 3 - 5 6 - 8 9` | 14 | 11 | [5, 6] | 22.328 | [3, 8] | 22.322 |
| `- - 3 - 5 6 - 8 9` | 14 | 12 | [3, 9] | 22.328 | [3, 9] | 22.322 |
| `- - 3 4 - - 7 8 9` | 14 | 3 | [3] | 23.949 | [3] | 23.949 |
| `- - 3 4 - - 7 8 9` | 14 | 4 | [4] | 23.949 | [4] | 23.949 |
| `- - 3 4 - - 7 8 9` | 14 | 7 | [7] | 23.949 | [7] | 23.949 |
| `- - 3 4 - - 7 8 9` | 14 | 8 | [8] | 23.949 | [8] | 23.949 |
| `- - 3 4 - - 7 8 9` | 14 | 9 | [9] | 23.949 | [9] | 23.949 |
| `- - 3 4 - - 7 8 9` | 14 | 10 | [3, 7] | 23.949 | [3, 7] | 23.949 |
| `- - 3 4 - - 7 8 9` | 14 | 11 | [4, 7] | 23.949 | [3, 8] | 23.949 |
| `- - 3 4 - - 7 8 9` | 14 | 12 | [3, 9] | 23.949 | [3, 9] | 23.949 |
| `- 2 - - 5 - 7 8 9` | 14 | 2 | [2] | 24.152 | [2] | 24.152 |
| `- 2 - - 5 - 7 8 9` | 14 | 5 | [5] | 24.152 | [5] | 24.152 |
| `- 2 - - 5 - 7 8 9` | 14 | 7 | [7] | 24.152 | [7] | 24.152 |
| `- 2 - - 5 - 7 8 9` | 14 | 8 | [8] | 24.152 | [8] | 24.152 |
| `- 2 - - 5 - 7 8 9` | 14 | 9 | [9] | 24.152 | [9] | 24.152 |
| `- 2 - - 5 - 7 8 9` | 14 | 10 | [2, 8] | 24.152 | [2, 8] | 24.152 |
| `- 2 - - 5 - 7 8 9` | 14 | 11 | [2, 9] | 24.152 | [2, 9] | 24.152 |
| `- 2 - - 5 - 7 8 9` | 14 | 12 | [5, 7] | 24.152 | [5, 7] | 24.152 |
| `- 2 3 - 5 6 7 8 -` | 14 | 2 | [2] | 30.712 | [2] | 30.591 |
| `- 2 3 - 5 6 7 8 -` | 14 | 3 | [3] | 30.712 | [3] | 30.591 |
| `- 2 3 - 5 6 7 8 -` | 14 | 5 | [5] | 30.712 | [5] | 30.591 |
| `- 2 3 - 5 6 7 8 -` | 14 | 6 | [6] | 30.712 | [6] | 30.591 |
| `- 2 3 - 5 6 7 8 -` | 14 | 7 | [7] | 30.712 | [7] | 30.591 |
| `- 2 3 - 5 6 7 8 -` | 14 | 8 | [8] | 30.712 | [8] | 30.591 |
| `- 2 3 - 5 6 7 8 -` | 14 | 9 | [3, 6] | 30.712 | [2, 7] | 30.591 |
| `- 2 3 - 5 6 7 8 -` | 14 | 10 | [2, 8] | 30.712 | [2, 8] | 30.591 |
| `- 2 3 - 5 6 7 8 -` | 14 | 11 | [3, 8] | 30.712 | [3, 8] | 30.591 |
| `- 2 3 - 5 6 7 8 -` | 14 | 12 | [5, 7] | 30.712 | [5, 7] | 30.591 |
| `- 2 3 4 - 6 7 - 9` | 14 | 2 | [2] | 30.867 | [2] | 30.859 |
| `- 2 3 4 - 6 7 - 9` | 14 | 3 | [3] | 30.867 | [3] | 30.859 |
| `- 2 3 4 - 6 7 - 9` | 14 | 4 | [4] | 30.867 | [4] | 30.859 |
| `- 2 3 4 - 6 7 - 9` | 14 | 5 | [2, 3] | 30.867 | [2, 3] | 30.859 |
| `- 2 3 4 - 6 7 - 9` | 14 | 6 | [6] | 30.867 | [6] | 30.859 |
| `- 2 3 4 - 6 7 - 9` | 14 | 7 | [7] | 30.867 | [7] | 30.859 |
| `- 2 3 4 - 6 7 - 9` | 14 | 8 | [2, 6] | 30.867 | [2, 6] | 30.859 |
| `- 2 3 4 - 6 7 - 9` | 14 | 9 | [9] | 30.867 | [9] | 30.859 |
| `- 2 3 4 - 6 7 - 9` | 14 | 10 | [3, 7] | 30.867 | [3, 7] | 30.859 |
| `- 2 3 4 - 6 7 - 9` | 14 | 11 | [2, 9] | 30.867 | [2, 9] | 30.859 |
| `- 2 3 4 - 6 7 - 9` | 14 | 12 | [3, 9] | 30.867 | [3, 9] | 30.859 |
| `- 2 3 4 5 - - 8 9` | 14 | 2 | [2] | 30.555 | [2] | 30.397 |
| `- 2 3 4 5 - - 8 9` | 14 | 3 | [3] | 30.555 | [3] | 30.397 |
| `- 2 3 4 5 - - 8 9` | 14 | 4 | [4] | 30.555 | [4] | 30.397 |
| `- 2 3 4 5 - - 8 9` | 14 | 5 | [5] | 30.555 | [5] | 30.397 |
| `- 2 3 4 5 - - 8 9` | 14 | 6 | [2, 4] | 30.555 | [2, 4] | 30.397 |
| `- 2 3 4 5 - - 8 9` | 14 | 7 | [3, 4] | 30.555 | [2, 5] | 30.397 |
| `- 2 3 4 5 - - 8 9` | 14 | 8 | [8] | 30.555 | [8] | 30.397 |
| `- 2 3 4 5 - - 8 9` | 14 | 9 | [9] | 30.555 | [9] | 30.397 |
| `- 2 3 4 5 - - 8 9` | 14 | 10 | [2, 8] | 30.555 | [2, 8] | 30.397 |
| `- 2 3 4 5 - - 8 9` | 14 | 11 | [2, 9] | 30.555 | [2, 9] | 30.397 |
| `- 2 3 4 5 - - 8 9` | 14 | 12 | [3, 9] | 30.555 | [3, 9] | 30.397 |
| `1 - - - - 6 7 8 9` | 14 | 6 | [6] | 23.044 | [6] | 23.044 |
| `1 - - - - 6 7 8 9` | 14 | 7 | [7] | 23.044 | [7] | 23.044 |
| `1 - - - - 6 7 8 9` | 14 | 8 | [8] | 23.044 | [8] | 23.044 |
| `1 - - - - 6 7 8 9` | 14 | 9 | [9] | 23.044 | [9] | 23.044 |
| `1 - - - - 6 7 8 9` | 14 | 10 | [1, 9] | 23.044 | [1, 9] | 23.044 |
| `1 - - 4 5 6 7 8 -` | 14 | 4 | [4] | 30.954 | [4] | 30.875 |
| `1 - - 4 5 6 7 8 -` | 14 | 5 | [5] | 30.954 | [5] | 30.875 |
| `1 - - 4 5 6 7 8 -` | 14 | 6 | [6] | 30.954 | [6] | 30.875 |
| `1 - - 4 5 6 7 8 -` | 14 | 7 | [7] | 30.954 | [7] | 30.875 |
| `1 - - 4 5 6 7 8 -` | 14 | 8 | [8] | 30.954 | [8] | 30.875 |
| `1 - - 4 5 6 7 8 -` | 14 | 9 | [1, 8] | 30.954 | [1, 8] | 30.875 |
| `1 - - 4 5 6 7 8 -` | 14 | 10 | [4, 6] | 30.954 | [4, 6] | 30.875 |
| `1 - - 4 5 6 7 8 -` | 14 | 11 | [4, 7] | 30.954 | [4, 7] | 30.875 |
| `1 - - 4 5 6 7 8 -` | 14 | 12 | [5, 7] | 30.954 | [4, 8] | 30.875 |
| `1 - 3 - 5 6 7 - 9` | 14 | 3 | [3] | 31.182 | [3] | 31.073 |
| `1 - 3 - 5 6 7 - 9` | 14 | 4 | [1, 3] | 31.182 | [1, 3] | 31.073 |
| `1 - 3 - 5 6 7 - 9` | 14 | 5 | [5] | 31.182 | [5] | 31.073 |
| `1 - 3 - 5 6 7 - 9` | 14 | 6 | [6] | 31.182 | [6] | 31.073 |
| `1 - 3 - 5 6 7 - 9` | 14 | 7 | [7] | 31.182 | [7] | 31.073 |
| `1 - 3 - 5 6 7 - 9` | 14 | 8 | [3, 5] | 31.182 | [1, 7] | 31.073 |
| `1 - 3 - 5 6 7 - 9` | 14 | 9 | [9] | 31.182 | [9] | 31.073 |
| `1 - 3 - 5 6 7 - 9` | 14 | 10 | [1, 9] | 31.182 | [1, 9] | 31.073 |
| `1 - 3 - 5 6 7 - 9` | 14 | 11 | [5, 6] | 31.182 | [5, 6] | 31.073 |
| `1 - 3 - 5 6 7 - 9` | 14 | 12 | [3, 9] | 31.182 | [3, 9] | 31.073 |
| `1 - 3 4 - 6 - 8 9` | 14 | 3 | [3] | 30.367 | [3] | 30.259 |
| `1 - 3 4 - 6 - 8 9` | 14 | 4 | [4] | 30.367 | [4] | 30.259 |
| `1 - 3 4 - 6 - 8 9` | 14 | 5 | [1, 4] | 30.367 | [1, 4] | 30.259 |
| `1 - 3 4 - 6 - 8 9` | 14 | 6 | [6] | 30.367 | [6] | 30.259 |
| `1 - 3 4 - 6 - 8 9` | 14 | 7 | [3, 4] | 30.367 | [1, 6] | 30.259 |
| `1 - 3 4 - 6 - 8 9` | 14 | 8 | [8] | 30.367 | [8] | 30.259 |
| `1 - 3 4 - 6 - 8 9` | 14 | 9 | [9] | 30.367 | [9] | 30.259 |
| `1 - 3 4 - 6 - 8 9` | 14 | 10 | [1, 9] | 30.367 | [1, 9] | 30.259 |
| `1 - 3 4 - 6 - 8 9` | 14 | 11 | [3, 8] | 30.367 | [3, 8] | 30.259 |
| `1 - 3 4 - 6 - 8 9` | 14 | 12 | [3, 9] | 30.367 | [3, 9] | 30.259 |
| `1 2 - - 5 6 - 8 9` | 14 | 2 | [2] | 29.623 | [2] | 29.583 |
| `1 2 - - 5 6 - 8 9` | 14 | 3 | [1, 2] | 29.623 | [1, 2] | 29.583 |
| `1 2 - - 5 6 - 8 9` | 14 | 5 | [5] | 29.623 | [5] | 29.583 |
| `1 2 - - 5 6 - 8 9` | 14 | 6 | [6] | 29.623 | [6] | 29.583 |
| `1 2 - - 5 6 - 8 9` | 14 | 7 | [1, 6] | 29.623 | [1, 6] | 29.583 |
| `1 2 - - 5 6 - 8 9` | 14 | 8 | [8] | 29.623 | [8] | 29.583 |
| `1 2 - - 5 6 - 8 9` | 14 | 9 | [9] | 29.623 | [9] | 29.583 |
| `1 2 - - 5 6 - 8 9` | 14 | 10 | [1, 9] | 29.623 | [1, 9] | 29.583 |
| `1 2 - - 5 6 - 8 9` | 14 | 11 | [2, 9] | 29.623 | [2, 9] | 29.583 |
| `1 2 - - 5 6 - 8 9` | 14 | 12 | [1, 5, 6] | 29.623 | [1, 2, 9] | 29.583 |
| `1 2 - 4 - - 7 8 9` | 14 | 2 | [2] | 30.591 | [2] | 30.591 |
| `1 2 - 4 - - 7 8 9` | 14 | 3 | [1, 2] | 30.591 | [1, 2] | 30.591 |
| `1 2 - 4 - - 7 8 9` | 14 | 4 | [4] | 30.591 | [4] | 30.591 |
| `1 2 - 4 - - 7 8 9` | 14 | 5 | [1, 4] | 30.591 | [1, 4] | 30.591 |
| `1 2 - 4 - - 7 8 9` | 14 | 6 | [2, 4] | 30.591 | [2, 4] | 30.591 |
| `1 2 - 4 - - 7 8 9` | 14 | 7 | [7] | 30.591 | [7] | 30.591 |
| `1 2 - 4 - - 7 8 9` | 14 | 8 | [8] | 30.591 | [8] | 30.591 |
| `1 2 - 4 - - 7 8 9` | 14 | 9 | [9] | 30.591 | [9] | 30.591 |
| `1 2 - 4 - - 7 8 9` | 14 | 10 | [1, 9] | 30.591 | [1, 9] | 30.591 |
| `1 2 - 4 - - 7 8 9` | 14 | 11 | [2, 9] | 30.591 | [2, 9] | 30.591 |
| `1 2 - 4 - - 7 8 9` | 14 | 12 | [4, 8] | 30.591 | [4, 8] | 30.591 |
| `1 2 3 4 - 6 7 8 -` | 14 | 2 | [2] | 36.729 | [2] | 36.629 |
| `1 2 3 4 - 6 7 8 -` | 14 | 3 | [3] | 36.729 | [3] | 36.629 |
| `1 2 3 4 - 6 7 8 -` | 14 | 4 | [4] | 36.729 | [4] | 36.629 |
| `1 2 3 4 - 6 7 8 -` | 14 | 5 | [1, 4] | 36.729 | [1, 4] | 36.629 |
| `1 2 3 4 - 6 7 8 -` | 14 | 6 | [6] | 36.729 | [6] | 36.629 |
| `1 2 3 4 - 6 7 8 -` | 14 | 7 | [7] | 36.729 | [7] | 36.629 |
| `1 2 3 4 - 6 7 8 -` | 14 | 8 | [8] | 36.729 | [8] | 36.629 |
| `1 2 3 4 - 6 7 8 -` | 14 | 9 | [1, 8] | 36.729 | [1, 8] | 36.629 |
| `1 2 3 4 - 6 7 8 -` | 14 | 10 | [2, 8] | 36.729 | [2, 8] | 36.629 |
| `1 2 3 4 - 6 7 8 -` | 14 | 11 | [3, 8] | 36.729 | [3, 8] | 36.629 |
| `1 2 3 4 - 6 7 8 -` | 14 | 12 | [4, 8] | 36.729 | [4, 8] | 36.629 |
| `1 2 3 4 5 - 7 - 9` | 14 | 2 | [2] | 35.899 | [2] | 35.418 |
| `1 2 3 4 5 - 7 - 9` | 14 | 3 | [3] | 35.899 | [3] | 35.418 |
| `1 2 3 4 5 - 7 - 9` | 14 | 4 | [4] | 35.899 | [4] | 35.418 |
| `1 2 3 4 5 - 7 - 9` | 14 | 5 | [5] | 35.899 | [5] | 35.418 |
| `1 2 3 4 5 - 7 - 9` | 14 | 6 | [2, 4] | 35.899 | [1, 5] | 35.418 |
| `1 2 3 4 5 - 7 - 9` | 14 | 7 | [7] | 35.899 | [7] | 35.418 |
| `1 2 3 4 5 - 7 - 9` | 14 | 8 | [3, 5] | 35.899 | [1, 7] | 35.418 |
| `1 2 3 4 5 - 7 - 9` | 14 | 9 | [9] | 35.899 | [9] | 35.418 |
| `1 2 3 4 5 - 7 - 9` | 14 | 10 | [1, 9] | 35.899 | [1, 9] | 35.418 |
| `1 2 3 4 5 - 7 - 9` | 14 | 11 | [2, 9] | 35.899 | [2, 9] | 35.418 |
| `1 2 3 4 5 - 7 - 9` | 14 | 12 | [3, 9] | 35.899 | [3, 9] | 35.418 |
| `- - - - - 6 7 8 9` | 15 | 6 | [6] | 21.354 | [6] | 21.354 |
| `- - - - - 6 7 8 9` | 15 | 7 | [7] | 21.354 | [7] | 21.354 |
| `- - - - - 6 7 8 9` | 15 | 8 | [8] | 21.354 | [8] | 21.354 |
| `- - - - - 6 7 8 9` | 15 | 9 | [9] | 21.354 | [9] | 21.354 |
| `- - - 4 5 6 7 8 -` | 15 | 4 | [4] | 28.111 | [4] | 28.081 |
| `- - - 4 5 6 7 8 -` | 15 | 5 | [5] | 28.111 | [5] | 28.081 |
| `- - - 4 5 6 7 8 -` | 15 | 6 | [6] | 28.111 | [6] | 28.081 |
| `- - - 4 5 6 7 8 -` | 15 | 7 | [7] | 28.111 | [7] | 28.081 |
| `- - - 4 5 6 7 8 -` | 15 | 8 | [8] | 28.111 | [8] | 28.081 |
| `- - - 4 5 6 7 8 -` | 15 | 9 | [4, 5] | 28.111 | [4, 5] | 28.081 |
| `- - - 4 5 6 7 8 -` | 15 | 10 | [4, 6] | 28.111 | [4, 6] | 28.081 |
| `- - - 4 5 6 7 8 -` | 15 | 11 | [5, 6] | 28.111 | [4, 7] | 28.081 |
| `- - - 4 5 6 7 8 -` | 15 | 12 | [4, 8] | 28.111 | [4, 8] | 28.081 |
| `- - 3 - 5 6 7 - 9` | 15 | 3 | [3] | 27.195 | [3] | 27.195 |
| `- - 3 - 5 6 7 - 9` | 15 | 5 | [5] | 27.195 | [5] | 27.195 |
| `- - 3 - 5 6 7 - 9` | 15 | 6 | [6] | 27.195 | [6] | 27.195 |
| `- - 3 - 5 6 7 - 9` | 15 | 7 | [7] | 27.195 | [7] | 27.195 |
| `- - 3 - 5 6 7 - 9` | 15 | 8 | [3, 5] | 27.195 | [3, 5] | 27.195 |
| `- - 3 - 5 6 7 - 9` | 15 | 9 | [9] | 27.195 | [9] | 27.195 |
| `- - 3 - 5 6 7 - 9` | 15 | 10 | [3, 7] | 27.195 | [3, 7] | 27.195 |
| `- - 3 - 5 6 7 - 9` | 15 | 11 | [5, 6] | 27.195 | [5, 6] | 27.195 |
| `- - 3 - 5 6 7 - 9` | 15 | 12 | [3, 9] | 27.195 | [3, 9] | 27.195 |
| `- - 3 4 - 6 - 8 9` | 15 | 3 | [3] | 26.420 | [3] | 26.420 |
| `- - 3 4 - 6 - 8 9` | 15 | 4 | [4] | 26.420 | [4] | 26.420 |
| `- - 3 4 - 6 - 8 9` | 15 | 6 | [6] | 26.420 | [6] | 26.420 |
| `- - 3 4 - 6 - 8 9` | 15 | 7 | [3, 4] | 26.420 | [3, 4] | 26.420 |
| `- - 3 4 - 6 - 8 9` | 15 | 8 | [8] | 26.420 | [8] | 26.420 |
| `- - 3 4 - 6 - 8 9` | 15 | 9 | [9] | 26.420 | [9] | 26.420 |
| `- - 3 4 - 6 - 8 9` | 15 | 10 | [4, 6] | 26.420 | [4, 6] | 26.420 |
| `- - 3 4 - 6 - 8 9` | 15 | 11 | [3, 8] | 26.420 | [3, 8] | 26.420 |
| `- - 3 4 - 6 - 8 9` | 15 | 12 | [3, 9] | 26.420 | [3, 9] | 26.420 |
| `- 2 - - 5 6 - 8 9` | 15 | 2 | [2] | 26.220 | [2] | 26.203 |
| `- 2 - - 5 6 - 8 9` | 15 | 5 | [5] | 26.220 | [5] | 26.203 |
| `- 2 - - 5 6 - 8 9` | 15 | 6 | [6] | 26.220 | [6] | 26.203 |
| `- 2 - - 5 6 - 8 9` | 15 | 7 | [2, 5] | 26.220 | [2, 5] | 26.203 |
| `- 2 - - 5 6 - 8 9` | 15 | 8 | [8] | 26.220 | [8] | 26.203 |
| `- 2 - - 5 6 - 8 9` | 15 | 9 | [9] | 26.220 | [9] | 26.203 |
| `- 2 - - 5 6 - 8 9` | 15 | 10 | [2, 8] | 26.220 | [2, 8] | 26.203 |
| `- 2 - - 5 6 - 8 9` | 15 | 11 | [5, 6] | 26.220 | [2, 9] | 26.203 |
| `- 2 - 4 - - 7 8 9` | 15 | 2 | [2] | 26.333 | [2] | 26.333 |
| `- 2 - 4 - - 7 8 9` | 15 | 4 | [4] | 26.333 | [4] | 26.333 |
| `- 2 - 4 - - 7 8 9` | 15 | 6 | [2, 4] | 26.333 | [2, 4] | 26.333 |
| `- 2 - 4 - - 7 8 9` | 15 | 7 | [7] | 26.333 | [7] | 26.333 |
| `- 2 - 4 - - 7 8 9` | 15 | 8 | [8] | 26.333 | [8] | 26.333 |
| `- 2 - 4 - - 7 8 9` | 15 | 9 | [9] | 26.333 | [9] | 26.333 |
| `- 2 - 4 - - 7 8 9` | 15 | 10 | [2, 8] | 26.333 | [2, 8] | 26.333 |
| `- 2 - 4 - - 7 8 9` | 15 | 11 | [2, 9] | 26.333 | [2, 9] | 26.333 |
| `- 2 - 4 - - 7 8 9` | 15 | 12 | [4, 8] | 26.333 | [4, 8] | 26.333 |
| `- 2 3 4 - 6 7 8 -` | 15 | 2 | [2] | 33.386 | [2] | 33.295 |
| `- 2 3 4 - 6 7 8 -` | 15 | 3 | [3] | 33.386 | [3] | 33.295 |
| `- 2 3 4 - 6 7 8 -` | 15 | 4 | [4] | 33.386 | [4] | 33.295 |
| `- 2 3 4 - 6 7 8 -` | 15 | 5 | [2, 3] | 33.386 | [2, 3] | 33.295 |
| `- 2 3 4 - 6 7 8 -` | 15 | 6 | [6] | 33.386 | [6] | 33.295 |
| `- 2 3 4 - 6 7 8 -` | 15 | 7 | [7] | 33.386 | [7] | 33.295 |
| `- 2 3 4 - 6 7 8 -` | 15 | 8 | [8] | 33.386 | [8] | 33.295 |
| `- 2 3 4 - 6 7 8 -` | 15 | 9 | [2, 7] | 33.386 | [2, 7] | 33.295 |
| `- 2 3 4 - 6 7 8 -` | 15 | 10 | [4, 6] | 33.386 | [2, 8] | 33.295 |
| `- 2 3 4 - 6 7 8 -` | 15 | 11 | [3, 8] | 33.386 | [3, 8] | 33.295 |
| `- 2 3 4 - 6 7 8 -` | 15 | 12 | [4, 8] | 33.386 | [4, 8] | 33.295 |
| `- 2 3 4 5 - 7 - 9` | 15 | 2 | [2] | 31.501 | [2] | 31.487 |
| `- 2 3 4 5 - 7 - 9` | 15 | 3 | [3] | 31.501 | [3] | 31.487 |
| `- 2 3 4 5 - 7 - 9` | 15 | 4 | [4] | 31.501 | [4] | 31.487 |
| `- 2 3 4 5 - 7 - 9` | 15 | 5 | [5] | 31.501 | [5] | 31.487 |
| `- 2 3 4 5 - 7 - 9` | 15 | 6 | [2, 4] | 31.501 | [2, 4] | 31.487 |
| `- 2 3 4 5 - 7 - 9` | 15 | 7 | [7] | 31.501 | [7] | 31.487 |
| `- 2 3 4 5 - 7 - 9` | 15 | 8 | [3, 5] | 31.501 | [3, 5] | 31.487 |
| `- 2 3 4 5 - 7 - 9` | 15 | 9 | [9] | 31.501 | [9] | 31.487 |
| `- 2 3 4 5 - 7 - 9` | 15 | 10 | [3, 7] | 31.501 | [3, 7] | 31.487 |
| `- 2 3 4 5 - 7 - 9` | 15 | 11 | [2, 9] | 31.501 | [2, 9] | 31.487 |
| `- 2 3 4 5 - 7 - 9` | 15 | 12 | [3, 9] | 31.501 | [3, 9] | 31.487 |
| `1 - - - 5 - 7 8 9` | 15 | 5 | [5] | 25.568 | [5] | 25.568 |
| `1 - - - 5 - 7 8 9` | 15 | 6 | [1, 5] | 25.568 | [1, 5] | 25.568 |
| `1 - - - 5 - 7 8 9` | 15 | 7 | [7] | 25.568 | [7] | 25.568 |
| `1 - - - 5 - 7 8 9` | 15 | 8 | [8] | 25.568 | [8] | 25.568 |
| `1 - - - 5 - 7 8 9` | 15 | 9 | [9] | 25.568 | [9] | 25.568 |
| `1 - - - 5 - 7 8 9` | 15 | 10 | [1, 9] | 25.568 | [1, 9] | 25.568 |
| `1 - - - 5 - 7 8 9` | 15 | 12 | [5, 7] | 25.568 | [5, 7] | 25.568 |
| `1 - 3 - 5 6 7 8 -` | 15 | 3 | [3] | 32.668 | [3] | 32.610 |
| `1 - 3 - 5 6 7 8 -` | 15 | 4 | [1, 3] | 32.668 | [1, 3] | 32.610 |
| `1 - 3 - 5 6 7 8 -` | 15 | 5 | [5] | 32.668 | [5] | 32.610 |
| `1 - 3 - 5 6 7 8 -` | 15 | 6 | [6] | 32.668 | [6] | 32.610 |
| `1 - 3 - 5 6 7 8 -` | 15 | 7 | [7] | 32.668 | [7] | 32.610 |
| `1 - 3 - 5 6 7 8 -` | 15 | 8 | [8] | 32.668 | [8] | 32.610 |
| `1 - 3 - 5 6 7 8 -` | 15 | 9 | [1, 8] | 32.668 | [1, 8] | 32.610 |
| `1 - 3 - 5 6 7 8 -` | 15 | 10 | [3, 7] | 32.668 | [3, 7] | 32.610 |
| `1 - 3 - 5 6 7 8 -` | 15 | 11 | [5, 6] | 32.668 | [3, 8] | 32.610 |
| `1 - 3 - 5 6 7 8 -` | 15 | 12 | [5, 7] | 32.668 | [5, 7] | 32.610 |
| `1 - 3 4 - 6 7 - 9` | 15 | 3 | [3] | 31.344 | [3] | 31.313 |
| `1 - 3 4 - 6 7 - 9` | 15 | 4 | [4] | 31.344 | [4] | 31.313 |
| `1 - 3 4 - 6 7 - 9` | 15 | 5 | [1, 4] | 31.344 | [1, 4] | 31.313 |
| `1 - 3 4 - 6 7 - 9` | 15 | 6 | [6] | 31.344 | [6] | 31.313 |
| `1 - 3 4 - 6 7 - 9` | 15 | 7 | [7] | 31.344 | [7] | 31.313 |
| `1 - 3 4 - 6 7 - 9` | 15 | 8 | [1, 7] | 31.344 | [1, 7] | 31.313 |
| `1 - 3 4 - 6 7 - 9` | 15 | 9 | [9] | 31.344 | [9] | 31.313 |
| `1 - 3 4 - 6 7 - 9` | 15 | 10 | [4, 6] | 31.344 | [1, 9] | 31.313 |
| `1 - 3 4 - 6 7 - 9` | 15 | 11 | [4, 7] | 31.344 | [4, 7] | 31.313 |
| `1 - 3 4 - 6 7 - 9` | 15 | 12 | [3, 9] | 31.344 | [3, 9] | 31.313 |
| `1 - 3 4 5 - - 8 9` | 15 | 3 | [3] | 30.357 | [3] | 30.355 |
| `1 - 3 4 5 - - 8 9` | 15 | 4 | [4] | 30.357 | [4] | 30.355 |
| `1 - 3 4 5 - - 8 9` | 15 | 5 | [5] | 30.357 | [5] | 30.355 |
| `1 - 3 4 5 - - 8 9` | 15 | 6 | [1, 5] | 30.357 | [1, 5] | 30.355 |
| `1 - 3 4 5 - - 8 9` | 15 | 7 | [3, 4] | 30.357 | [3, 4] | 30.355 |
| `1 - 3 4 5 - - 8 9` | 15 | 8 | [8] | 30.357 | [8] | 30.355 |
| `1 - 3 4 5 - - 8 9` | 15 | 9 | [9] | 30.357 | [9] | 30.355 |
| `1 - 3 4 5 - - 8 9` | 15 | 10 | [1, 9] | 30.357 | [1, 9] | 30.355 |
| `1 - 3 4 5 - - 8 9` | 15 | 11 | [3, 8] | 30.357 | [3, 8] | 30.355 |
| `1 - 3 4 5 - - 8 9` | 15 | 12 | [4, 8] | 30.357 | [3, 9] | 30.355 |
| `1 2 - - 5 6 7 - 9` | 15 | 2 | [2] | 31.099 | [2] | 30.972 |
| `1 2 - - 5 6 7 - 9` | 15 | 3 | [1, 2] | 31.099 | [1, 2] | 30.972 |
| `1 2 - - 5 6 7 - 9` | 15 | 5 | [5] | 31.099 | [5] | 30.972 |
| `1 2 - - 5 6 7 - 9` | 15 | 6 | [6] | 31.099 | [6] | 30.972 |
| `1 2 - - 5 6 7 - 9` | 15 | 7 | [7] | 31.099 | [7] | 30.972 |
| `1 2 - - 5 6 7 - 9` | 15 | 8 | [2, 6] | 31.099 | [1, 7] | 30.972 |
| `1 2 - - 5 6 7 - 9` | 15 | 9 | [9] | 31.099 | [9] | 30.972 |
| `1 2 - - 5 6 7 - 9` | 15 | 10 | [1, 9] | 31.099 | [1, 9] | 30.972 |
| `1 2 - - 5 6 7 - 9` | 15 | 11 | [5, 6] | 31.099 | [2, 9] | 30.972 |
| `1 2 - - 5 6 7 - 9` | 15 | 12 | [5, 7] | 31.099 | [5, 7] | 30.972 |
| `1 2 - 4 - 6 - 8 9` | 15 | 2 | [2] | 31.603 | [2] | 31.419 |
| `1 2 - 4 - 6 - 8 9` | 15 | 3 | [1, 2] | 31.603 | [1, 2] | 31.419 |
| `1 2 - 4 - 6 - 8 9` | 15 | 4 | [4] | 31.603 | [4] | 31.419 |
| `1 2 - 4 - 6 - 8 9` | 15 | 5 | [1, 4] | 31.603 | [1, 4] | 31.419 |
| `1 2 - 4 - 6 - 8 9` | 15 | 6 | [6] | 31.603 | [6] | 31.419 |
| `1 2 - 4 - 6 - 8 9` | 15 | 7 | [1, 6] | 31.603 | [1, 6] | 31.419 |
| `1 2 - 4 - 6 - 8 9` | 15 | 8 | [8] | 31.603 | [8] | 31.419 |
| `1 2 - 4 - 6 - 8 9` | 15 | 9 | [9] | 31.603 | [9] | 31.419 |
| `1 2 - 4 - 6 - 8 9` | 15 | 10 | [2, 8] | 31.603 | [1, 9] | 31.419 |
| `1 2 - 4 - 6 - 8 9` | 15 | 11 | [2, 9] | 31.603 | [2, 9] | 31.419 |
| `1 2 - 4 - 6 - 8 9` | 15 | 12 | [4, 8] | 31.603 | [4, 8] | 31.419 |
| `1 2 3 - - - 7 8 9` | 15 | 2 | [2] | 30.599 | [2] | 30.599 |
| `1 2 3 - - - 7 8 9` | 15 | 3 | [3] | 30.599 | [3] | 30.599 |
| `1 2 3 - - - 7 8 9` | 15 | 4 | [1, 3] | 30.599 | [1, 3] | 30.599 |
| `1 2 3 - - - 7 8 9` | 15 | 5 | [2, 3] | 30.599 | [2, 3] | 30.599 |
| `1 2 3 - - - 7 8 9` | 15 | 6 | [1, 2, 3] | 30.599 | [1, 2, 3] | 30.599 |
| `1 2 3 - - - 7 8 9` | 15 | 7 | [7] | 30.599 | [7] | 30.599 |
| `1 2 3 - - - 7 8 9` | 15 | 8 | [8] | 30.599 | [8] | 30.599 |
| `1 2 3 - - - 7 8 9` | 15 | 9 | [9] | 30.599 | [9] | 30.599 |
| `1 2 3 - - - 7 8 9` | 15 | 10 | [1, 9] | 30.599 | [1, 9] | 30.599 |
| `1 2 3 - - - 7 8 9` | 15 | 11 | [2, 9] | 30.599 | [2, 9] | 30.599 |
| `1 2 3 - - - 7 8 9` | 15 | 12 | [3, 9] | 30.599 | [3, 9] | 30.599 |
| `1 2 3 4 5 - 7 8 -` | 15 | 2 | [2] | 37.510 | [2] | 37.396 |
| `1 2 3 4 5 - 7 8 -` | 15 | 3 | [3] | 37.510 | [3] | 37.396 |
| `1 2 3 4 5 - 7 8 -` | 15 | 4 | [4] | 37.510 | [4] | 37.396 |
| `1 2 3 4 5 - 7 8 -` | 15 | 5 | [5] | 37.510 | [5] | 37.396 |
| `1 2 3 4 5 - 7 8 -` | 15 | 6 | [1, 5] | 37.510 | [1, 5] | 37.396 |
| `1 2 3 4 5 - 7 8 -` | 15 | 7 | [7] | 37.510 | [7] | 37.396 |
| `1 2 3 4 5 - 7 8 -` | 15 | 8 | [8] | 37.510 | [8] | 37.396 |
| `1 2 3 4 5 - 7 8 -` | 15 | 9 | [1, 8] | 37.510 | [1, 8] | 37.396 |
| `1 2 3 4 5 - 7 8 -` | 15 | 10 | [2, 8] | 37.510 | [2, 8] | 37.396 |
| `1 2 3 4 5 - 7 8 -` | 15 | 11 | [3, 8] | 37.510 | [3, 8] | 37.396 |
| `1 2 3 4 5 - 7 8 -` | 15 | 12 | [4, 8] | 37.510 | [4, 8] | 37.396 |
| `1 2 3 4 5 6 - - 9` | 15 | 2 | [2] | 36.633 | [2] | 36.322 |
| `1 2 3 4 5 6 - - 9` | 15 | 3 | [3] | 36.633 | [3] | 36.322 |
| `1 2 3 4 5 6 - - 9` | 15 | 4 | [4] | 36.633 | [4] | 36.322 |
| `1 2 3 4 5 6 - - 9` | 15 | 5 | [5] | 36.633 | [5] | 36.322 |
| `1 2 3 4 5 6 - - 9` | 15 | 6 | [6] | 36.633 | [6] | 36.322 |
| `1 2 3 4 5 6 - - 9` | 15 | 7 | [1, 6] | 36.633 | [1, 6] | 36.322 |
| `1 2 3 4 5 6 - - 9` | 15 | 8 | [3, 5] | 36.633 | [2, 6] | 36.322 |
| `1 2 3 4 5 6 - - 9` | 15 | 9 | [9] | 36.633 | [9] | 36.322 |
| `1 2 3 4 5 6 - - 9` | 15 | 10 | [1, 9] | 36.633 | [1, 9] | 36.322 |
| `1 2 3 4 5 6 - - 9` | 15 | 11 | [2, 9] | 36.633 | [2, 9] | 36.322 |
| `1 2 3 4 5 6 - - 9` | 15 | 12 | [3, 9] | 36.633 | [3, 9] | 36.322 |
| `- - - - 5 - 7 8 9` | 16 | 5 | [5] | 22.273 | [5] | 22.273 |
| `- - - - 5 - 7 8 9` | 16 | 7 | [7] | 22.273 | [7] | 22.273 |
| `- - - - 5 - 7 8 9` | 16 | 8 | [8] | 22.273 | [8] | 22.273 |
| `- - - - 5 - 7 8 9` | 16 | 9 | [9] | 22.273 | [9] | 22.273 |
| `- - - - 5 - 7 8 9` | 16 | 12 | [5, 7] | 22.273 | [5, 7] | 22.273 |
| `- - 3 - 5 6 7 8 -` | 16 | 3 | [3] | 28.596 | [3] | 28.579 |
| `- - 3 - 5 6 7 8 -` | 16 | 5 | [5] | 28.596 | [5] | 28.579 |
| `- - 3 - 5 6 7 8 -` | 16 | 6 | [6] | 28.596 | [6] | 28.579 |
| `- - 3 - 5 6 7 8 -` | 16 | 7 | [7] | 28.596 | [7] | 28.579 |
| `- - 3 - 5 6 7 8 -` | 16 | 8 | [8] | 28.596 | [8] | 28.579 |
| `- - 3 - 5 6 7 8 -` | 16 | 9 | [3, 6] | 28.596 | [3, 6] | 28.579 |
| `- - 3 - 5 6 7 8 -` | 16 | 10 | [3, 7] | 28.596 | [3, 7] | 28.579 |
| `- - 3 - 5 6 7 8 -` | 16 | 11 | [5, 6] | 28.596 | [3, 8] | 28.579 |
| `- - 3 - 5 6 7 8 -` | 16 | 12 | [5, 7] | 28.596 | [5, 7] | 28.579 |
| `- - 3 4 - 6 7 - 9` | 16 | 3 | [3] | 25.689 | [3] | 25.640 |
| `- - 3 4 - 6 7 - 9` | 16 | 4 | [4] | 25.689 | [4] | 25.640 |
| `- - 3 4 - 6 7 - 9` | 16 | 6 | [6] | 25.689 | [6] | 25.640 |
| `- - 3 4 - 6 7 - 9` | 16 | 7 | [7] | 25.689 | [7] | 25.640 |
| `- - 3 4 - 6 7 - 9` | 16 | 9 | [9] | 25.689 | [9] | 25.640 |
| `- - 3 4 - 6 7 - 9` | 16 | 10 | [4, 6] | 25.689 | [3, 7] | 25.640 |
| `- - 3 4 - 6 7 - 9` | 16 | 11 | [4, 7] | 25.689 | [4, 7] | 25.640 |
| `- - 3 4 - 6 7 - 9` | 16 | 12 | [3, 9] | 25.689 | [3, 9] | 25.640 |
| `- - 3 4 5 - - 8 9` | 16 | 3 | [3] | 25.336 | [3] | 25.336 |
| `- - 3 4 5 - - 8 9` | 16 | 4 | [4] | 25.336 | [4] | 25.336 |
| `- - 3 4 5 - - 8 9` | 16 | 5 | [5] | 25.336 | [5] | 25.336 |
| `- - 3 4 5 - - 8 9` | 16 | 7 | [3, 4] | 25.336 | [3, 4] | 25.336 |
| `- - 3 4 5 - - 8 9` | 16 | 8 | [8] | 25.336 | [8] | 25.336 |
| `- - 3 4 5 - - 8 9` | 16 | 9 | [9] | 25.336 | [9] | 25.336 |
| `- - 3 4 5 - - 8 9` | 16 | 11 | [3, 8] | 25.336 | [3, 8] | 25.336 |
| `- - 3 4 5 - - 8 9` | 16 | 12 | [3, 9] | 25.336 | [3, 9] | 25.336 |
| `- 2 - - 5 6 7 - 9` | 16 | 2 | [2] | 26.556 | [2] | 26.556 |
| `- 2 - - 5 6 7 - 9` | 16 | 5 | [5] | 26.556 | [5] | 26.556 |
| `- 2 - - 5 6 7 - 9` | 16 | 6 | [6] | 26.556 | [6] | 26.556 |
| `- 2 - - 5 6 7 - 9` | 16 | 7 | [7] | 26.556 | [7] | 26.556 |
| `- 2 - - 5 6 7 - 9` | 16 | 8 | [2, 6] | 26.556 | [2, 6] | 26.556 |
| `- 2 - - 5 6 7 - 9` | 16 | 9 | [9] | 26.556 | [9] | 26.556 |
| `- 2 - - 5 6 7 - 9` | 16 | 11 | [2, 9] | 26.556 | [2, 9] | 26.556 |
| `- 2 - - 5 6 7 - 9` | 16 | 12 | [5, 7] | 26.556 | [5, 7] | 26.556 |
| `- 2 - 4 - 6 - 8 9` | 16 | 2 | [2] | 25.078 | [2] | 25.029 |
| `- 2 - 4 - 6 - 8 9` | 16 | 4 | [4] | 25.078 | [4] | 25.029 |
| `- 2 - 4 - 6 - 8 9` | 16 | 6 | [6] | 25.078 | [6] | 25.029 |
| `- 2 - 4 - 6 - 8 9` | 16 | 8 | [8] | 25.078 | [8] | 25.029 |
| `- 2 - 4 - 6 - 8 9` | 16 | 9 | [9] | 25.078 | [9] | 25.029 |
| `- 2 - 4 - 6 - 8 9` | 16 | 10 | [4, 6] | 25.078 | [2, 8] | 25.029 |
| `- 2 - 4 - 6 - 8 9` | 16 | 11 | [2, 9] | 25.078 | [2, 9] | 25.029 |
| `- 2 - 4 - 6 - 8 9` | 16 | 12 | [4, 8] | 25.078 | [4, 8] | 25.029 |
| `- 2 3 - - - 7 8 9` | 16 | 2 | [2] | 26.606 | [2] | 26.605 |
| `- 2 3 - - - 7 8 9` | 16 | 3 | [3] | 26.606 | [3] | 26.605 |
| `- 2 3 - - - 7 8 9` | 16 | 5 | [2, 3] | 26.606 | [2, 3] | 26.605 |
| `- 2 3 - - - 7 8 9` | 16 | 7 | [7] | 26.606 | [7] | 26.605 |
| `- 2 3 - - - 7 8 9` | 16 | 8 | [8] | 26.606 | [8] | 26.605 |
| `- 2 3 - - - 7 8 9` | 16 | 9 | [9] | 26.606 | [9] | 26.605 |
| `- 2 3 - - - 7 8 9` | 16 | 10 | [3, 7] | 26.606 | [2, 8] | 26.605 |
| `- 2 3 - - - 7 8 9` | 16 | 11 | [2, 9] | 26.606 | [2, 9] | 26.605 |
| `- 2 3 - - - 7 8 9` | 16 | 12 | [3, 9] | 26.606 | [3, 9] | 26.605 |
| `- 2 3 4 5 - 7 8 -` | 16 | 2 | [2] | 33.861 | [2] | 33.668 |
| `- 2 3 4 5 - 7 8 -` | 16 | 3 | [3] | 33.861 | [3] | 33.668 |
| `- 2 3 4 5 - 7 8 -` | 16 | 4 | [4] | 33.861 | [4] | 33.668 |
| `- 2 3 4 5 - 7 8 -` | 16 | 5 | [5] | 33.861 | [5] | 33.668 |
| `- 2 3 4 5 - 7 8 -` | 16 | 6 | [2, 4] | 33.861 | [2, 4] | 33.668 |
| `- 2 3 4 5 - 7 8 -` | 16 | 7 | [7] | 33.861 | [7] | 33.668 |
| `- 2 3 4 5 - 7 8 -` | 16 | 8 | [8] | 33.861 | [8] | 33.668 |
| `- 2 3 4 5 - 7 8 -` | 16 | 9 | [4, 5] | 33.861 | [2, 7] | 33.668 |
| `- 2 3 4 5 - 7 8 -` | 16 | 10 | [3, 7] | 33.861 | [2, 8] | 33.668 |
| `- 2 3 4 5 - 7 8 -` | 16 | 11 | [3, 8] | 33.861 | [3, 8] | 33.668 |
| `- 2 3 4 5 - 7 8 -` | 16 | 12 | [5, 7] | 33.861 | [4, 8] | 33.668 |
| `- 2 3 4 5 6 - - 9` | 16 | 2 | [2] | 33.302 | [2] | 33.066 |
| `- 2 3 4 5 6 - - 9` | 16 | 3 | [3] | 33.302 | [3] | 33.066 |
| `- 2 3 4 5 6 - - 9` | 16 | 4 | [4] | 33.302 | [4] | 33.066 |
| `- 2 3 4 5 6 - - 9` | 16 | 5 | [5] | 33.302 | [5] | 33.066 |
| `- 2 3 4 5 6 - - 9` | 16 | 6 | [6] | 33.302 | [6] | 33.066 |
| `- 2 3 4 5 6 - - 9` | 16 | 7 | [3, 4] | 33.302 | [2, 5] | 33.066 |
| `- 2 3 4 5 6 - - 9` | 16 | 8 | [3, 5] | 33.302 | [2, 6] | 33.066 |
| `- 2 3 4 5 6 - - 9` | 16 | 9 | [9] | 33.302 | [9] | 33.066 |
| `- 2 3 4 5 6 - - 9` | 16 | 10 | [4, 6] | 33.302 | [4, 6] | 33.066 |
| `- 2 3 4 5 6 - - 9` | 16 | 11 | [2, 9] | 33.302 | [2, 9] | 33.066 |
| `- 2 3 4 5 6 - - 9` | 16 | 12 | [3, 9] | 33.302 | [3, 9] | 33.066 |
| `1 - - - 5 6 - 8 9` | 16 | 5 | [5] | 27.178 | [5] | 27.178 |
| `1 - - - 5 6 - 8 9` | 16 | 6 | [6] | 27.178 | [6] | 27.178 |
| `1 - - - 5 6 - 8 9` | 16 | 7 | [1, 6] | 27.178 | [1, 6] | 27.178 |
| `1 - - - 5 6 - 8 9` | 16 | 8 | [8] | 27.178 | [8] | 27.178 |
| `1 - - - 5 6 - 8 9` | 16 | 9 | [9] | 27.178 | [9] | 27.178 |
| `1 - - - 5 6 - 8 9` | 16 | 10 | [1, 9] | 27.178 | [1, 9] | 27.178 |
| `1 - - - 5 6 - 8 9` | 16 | 11 | [5, 6] | 27.178 | [5, 6] | 27.178 |
| `1 - - - 5 6 - 8 9` | 16 | 12 | [1, 5, 6] | 27.178 | [1, 5, 6] | 27.178 |
| `1 - - 4 - - 7 8 9` | 16 | 4 | [4] | 26.563 | [4] | 26.563 |
| `1 - - 4 - - 7 8 9` | 16 | 5 | [1, 4] | 26.563 | [1, 4] | 26.563 |
| `1 - - 4 - - 7 8 9` | 16 | 7 | [7] | 26.563 | [7] | 26.563 |
| `1 - - 4 - - 7 8 9` | 16 | 8 | [8] | 26.563 | [8] | 26.563 |
| `1 - - 4 - - 7 8 9` | 16 | 9 | [9] | 26.563 | [9] | 26.563 |
| `1 - - 4 - - 7 8 9` | 16 | 10 | [1, 9] | 26.563 | [1, 9] | 26.563 |
| `1 - - 4 - - 7 8 9` | 16 | 11 | [4, 7] | 26.563 | [4, 7] | 26.563 |
| `1 - - 4 - - 7 8 9` | 16 | 12 | [4, 8] | 26.563 | [4, 8] | 26.563 |
| `1 - 3 4 - 6 7 8 -` | 16 | 3 | [3] | 33.480 | [3] | 33.440 |
| `1 - 3 4 - 6 7 8 -` | 16 | 4 | [4] | 33.480 | [4] | 33.440 |
| `1 - 3 4 - 6 7 8 -` | 16 | 5 | [1, 4] | 33.480 | [1, 4] | 33.440 |
| `1 - 3 4 - 6 7 8 -` | 16 | 6 | [6] | 33.480 | [6] | 33.440 |
| `1 - 3 4 - 6 7 8 -` | 16 | 7 | [7] | 33.480 | [7] | 33.440 |
| `1 - 3 4 - 6 7 8 -` | 16 | 8 | [8] | 33.480 | [8] | 33.440 |
| `1 - 3 4 - 6 7 8 -` | 16 | 9 | [1, 8] | 33.480 | [1, 8] | 33.440 |
| `1 - 3 4 - 6 7 8 -` | 16 | 10 | [3, 7] | 33.480 | [3, 7] | 33.440 |
| `1 - 3 4 - 6 7 8 -` | 16 | 11 | [4, 7] | 33.480 | [3, 8] | 33.440 |
| `1 - 3 4 - 6 7 8 -` | 16 | 12 | [4, 8] | 33.480 | [4, 8] | 33.440 |
| `1 - 3 4 5 - 7 - 9` | 16 | 3 | [3] | 32.505 | [3] | 32.278 |
| `1 - 3 4 5 - 7 - 9` | 16 | 4 | [4] | 32.505 | [4] | 32.278 |
| `1 - 3 4 5 - 7 - 9` | 16 | 5 | [5] | 32.505 | [5] | 32.278 |
| `1 - 3 4 5 - 7 - 9` | 16 | 6 | [1, 5] | 32.505 | [1, 5] | 32.278 |
| `1 - 3 4 5 - 7 - 9` | 16 | 7 | [7] | 32.505 | [7] | 32.278 |
| `1 - 3 4 5 - 7 - 9` | 16 | 8 | [3, 5] | 32.505 | [1, 7] | 32.278 |
| `1 - 3 4 5 - 7 - 9` | 16 | 9 | [9] | 32.505 | [9] | 32.278 |
| `1 - 3 4 5 - 7 - 9` | 16 | 10 | [1, 9] | 32.505 | [1, 9] | 32.278 |
| `1 - 3 4 5 - 7 - 9` | 16 | 11 | [4, 7] | 32.505 | [4, 7] | 32.278 |
| `1 - 3 4 5 - 7 - 9` | 16 | 12 | [3, 9] | 32.505 | [3, 9] | 32.278 |
| `1 2 - - 5 6 7 8 -` | 16 | 2 | [2] | 32.068 | [2] | 32.047 |
| `1 2 - - 5 6 7 8 -` | 16 | 3 | [1, 2] | 32.068 | [1, 2] | 32.047 |
| `1 2 - - 5 6 7 8 -` | 16 | 5 | [5] | 32.068 | [5] | 32.047 |
| `1 2 - - 5 6 7 8 -` | 16 | 6 | [6] | 32.068 | [6] | 32.047 |
| `1 2 - - 5 6 7 8 -` | 16 | 7 | [7] | 32.068 | [7] | 32.047 |
| `1 2 - - 5 6 7 8 -` | 16 | 8 | [8] | 32.068 | [8] | 32.047 |
| `1 2 - - 5 6 7 8 -` | 16 | 9 | [1, 8] | 32.068 | [1, 8] | 32.047 |
| `1 2 - - 5 6 7 8 -` | 16 | 10 | [2, 8] | 32.068 | [2, 8] | 32.047 |
| `1 2 - - 5 6 7 8 -` | 16 | 11 | [5, 6] | 32.068 | [5, 6] | 32.047 |
| `1 2 - - 5 6 7 8 -` | 16 | 12 | [5, 7] | 32.068 | [5, 7] | 32.047 |
| `1 2 - 4 - 6 7 - 9` | 16 | 2 | [2] | 33.192 | [2] | 33.000 |
| `1 2 - 4 - 6 7 - 9` | 16 | 3 | [1, 2] | 33.192 | [1, 2] | 33.000 |
| `1 2 - 4 - 6 7 - 9` | 16 | 4 | [4] | 33.192 | [4] | 33.000 |
| `1 2 - 4 - 6 7 - 9` | 16 | 5 | [1, 4] | 33.192 | [1, 4] | 33.000 |
| `1 2 - 4 - 6 7 - 9` | 16 | 6 | [6] | 33.192 | [6] | 33.000 |
| `1 2 - 4 - 6 7 - 9` | 16 | 7 | [7] | 33.192 | [7] | 33.000 |
| `1 2 - 4 - 6 7 - 9` | 16 | 8 | [2, 6] | 33.192 | [1, 7] | 33.000 |
| `1 2 - 4 - 6 7 - 9` | 16 | 9 | [9] | 33.192 | [9] | 33.000 |
| `1 2 - 4 - 6 7 - 9` | 16 | 10 | [1, 9] | 33.192 | [1, 9] | 33.000 |
| `1 2 - 4 - 6 7 - 9` | 16 | 11 | [2, 9] | 33.192 | [2, 9] | 33.000 |
| `1 2 - 4 - 6 7 - 9` | 16 | 12 | [2, 4, 6] | 33.192 | [1, 2, 9] | 33.000 |
| `1 2 - 4 5 - - 8 9` | 16 | 2 | [2] | 32.177 | [2] | 32.165 |
| `1 2 - 4 5 - - 8 9` | 16 | 3 | [1, 2] | 32.177 | [1, 2] | 32.165 |
| `1 2 - 4 5 - - 8 9` | 16 | 4 | [4] | 32.177 | [4] | 32.165 |
| `1 2 - 4 5 - - 8 9` | 16 | 5 | [5] | 32.177 | [5] | 32.165 |
| `1 2 - 4 5 - - 8 9` | 16 | 6 | [1, 5] | 32.177 | [1, 5] | 32.165 |
| `1 2 - 4 5 - - 8 9` | 16 | 7 | [2, 5] | 32.177 | [2, 5] | 32.165 |
| `1 2 - 4 5 - - 8 9` | 16 | 8 | [8] | 32.177 | [8] | 32.165 |
| `1 2 - 4 5 - - 8 9` | 16 | 9 | [9] | 32.177 | [9] | 32.165 |
| `1 2 - 4 5 - - 8 9` | 16 | 10 | [1, 9] | 32.177 | [1, 9] | 32.165 |
| `1 2 - 4 5 - - 8 9` | 16 | 11 | [2, 9] | 32.177 | [2, 9] | 32.165 |
| `1 2 - 4 5 - - 8 9` | 16 | 12 | [4, 8] | 32.177 | [4, 8] | 32.165 |
| `1 2 3 - - 6 - 8 9` | 16 | 2 | [2] | 32.024 | [2] | 32.019 |
| `1 2 3 - - 6 - 8 9` | 16 | 3 | [3] | 32.024 | [3] | 32.019 |
| `1 2 3 - - 6 - 8 9` | 16 | 4 | [1, 3] | 32.024 | [1, 3] | 32.019 |
| `1 2 3 - - 6 - 8 9` | 16 | 5 | [2, 3] | 32.024 | [2, 3] | 32.019 |
| `1 2 3 - - 6 - 8 9` | 16 | 6 | [6] | 32.024 | [6] | 32.019 |
| `1 2 3 - - 6 - 8 9` | 16 | 7 | [1, 6] | 32.024 | [1, 6] | 32.019 |
| `1 2 3 - - 6 - 8 9` | 16 | 8 | [8] | 32.024 | [8] | 32.019 |
| `1 2 3 - - 6 - 8 9` | 16 | 9 | [9] | 32.024 | [9] | 32.019 |
| `1 2 3 - - 6 - 8 9` | 16 | 10 | [1, 9] | 32.024 | [1, 9] | 32.019 |
| `1 2 3 - - 6 - 8 9` | 16 | 11 | [2, 9] | 32.024 | [2, 9] | 32.019 |
| `1 2 3 - - 6 - 8 9` | 16 | 12 | [3, 9] | 32.024 | [3, 9] | 32.019 |
| `1 2 3 4 5 6 - 8 -` | 16 | 2 | [2] | 38.329 | [2] | 38.189 |
| `1 2 3 4 5 6 - 8 -` | 16 | 3 | [3] | 38.329 | [3] | 38.189 |
| `1 2 3 4 5 6 - 8 -` | 16 | 4 | [4] | 38.329 | [4] | 38.189 |
| `1 2 3 4 5 6 - 8 -` | 16 | 5 | [5] | 38.329 | [5] | 38.189 |
| `1 2 3 4 5 6 - 8 -` | 16 | 6 | [6] | 38.329 | [6] | 38.189 |
| `1 2 3 4 5 6 - 8 -` | 16 | 7 | [1, 6] | 38.329 | [1, 6] | 38.189 |
| `1 2 3 4 5 6 - 8 -` | 16 | 8 | [8] | 38.329 | [8] | 38.189 |
| `1 2 3 4 5 6 - 8 -` | 16 | 9 | [1, 8] | 38.329 | [1, 8] | 38.189 |
| `1 2 3 4 5 6 - 8 -` | 16 | 10 | [2, 8] | 38.329 | [2, 8] | 38.189 |
| `1 2 3 4 5 6 - 8 -` | 16 | 11 | [3, 8] | 38.329 | [3, 8] | 38.189 |
| `1 2 3 4 5 6 - 8 -` | 16 | 12 | [4, 8] | 38.329 | [4, 8] | 38.189 |
| `- - - - 5 6 - 8 9` | 17 | 5 | [5] | 23.100 | [5] | 23.100 |
| `- - - - 5 6 - 8 9` | 17 | 6 | [6] | 23.100 | [6] | 23.100 |
| `- - - - 5 6 - 8 9` | 17 | 8 | [8] | 23.100 | [8] | 23.100 |
| `- - - - 5 6 - 8 9` | 17 | 9 | [9] | 23.100 | [9] | 23.100 |
| `- - - - 5 6 - 8 9` | 17 | 11 | [5, 6] | 23.100 | [5, 6] | 23.100 |
| `- - - 4 - - 7 8 9` | 17 | 4 | [4] | 23.722 | [4] | 23.722 |
| `- - - 4 - - 7 8 9` | 17 | 7 | [7] | 23.722 | [7] | 23.722 |
| `- - - 4 - - 7 8 9` | 17 | 8 | [8] | 23.722 | [8] | 23.722 |
| `- - - 4 - - 7 8 9` | 17 | 9 | [9] | 23.722 | [9] | 23.722 |
| `- - - 4 - - 7 8 9` | 17 | 11 | [4, 7] | 23.722 | [4, 7] | 23.722 |
| `- - - 4 - - 7 8 9` | 17 | 12 | [4, 8] | 23.722 | [4, 8] | 23.722 |
| `- - 3 4 - 6 7 8 -` | 17 | 3 | [3] | 29.399 | [3] | 29.353 |
| `- - 3 4 - 6 7 8 -` | 17 | 4 | [4] | 29.399 | [4] | 29.353 |
| `- - 3 4 - 6 7 8 -` | 17 | 6 | [6] | 29.399 | [6] | 29.353 |
| `- - 3 4 - 6 7 8 -` | 17 | 7 | [7] | 29.399 | [7] | 29.353 |
| `- - 3 4 - 6 7 8 -` | 17 | 8 | [8] | 29.399 | [8] | 29.353 |
| `- - 3 4 - 6 7 8 -` | 17 | 9 | [3, 6] | 29.399 | [3, 6] | 29.353 |
| `- - 3 4 - 6 7 8 -` | 17 | 10 | [4, 6] | 29.399 | [3, 7] | 29.353 |
| `- - 3 4 - 6 7 8 -` | 17 | 11 | [3, 8] | 29.399 | [3, 8] | 29.353 |
| `- - 3 4 - 6 7 8 -` | 17 | 12 | [4, 8] | 29.399 | [4, 8] | 29.353 |
| `- - 3 4 5 - 7 - 9` | 17 | 3 | [3] | 27.962 | [3] | 27.962 |
| `- - 3 4 5 - 7 - 9` | 17 | 4 | [4] | 27.962 | [4] | 27.962 |
| `- - 3 4 5 - 7 - 9` | 17 | 5 | [5] | 27.962 | [5] | 27.962 |
| `- - 3 4 5 - 7 - 9` | 17 | 7 | [7] | 27.962 | [7] | 27.962 |
| `- - 3 4 5 - 7 - 9` | 17 | 8 | [3, 5] | 27.962 | [3, 5] | 27.962 |
| `- - 3 4 5 - 7 - 9` | 17 | 9 | [9] | 27.962 | [9] | 27.962 |
| `- - 3 4 5 - 7 - 9` | 17 | 10 | [3, 7] | 27.962 | [3, 7] | 27.962 |
| `- - 3 4 5 - 7 - 9` | 17 | 11 | [4, 7] | 27.962 | [4, 7] | 27.962 |
| `- - 3 4 5 - 7 - 9` | 17 | 12 | [3, 9] | 27.962 | [3, 9] | 27.962 |
| `- 2 - - 5 6 7 8 -` | 17 | 2 | [2] | 29.231 | [2] | 29.231 |
| `- 2 - - 5 6 7 8 -` | 17 | 5 | [5] | 29.231 | [5] | 29.231 |
| `- 2 - - 5 6 7 8 -` | 17 | 6 | [6] | 29.231 | [6] | 29.231 |
| `- 2 - - 5 6 7 8 -` | 17 | 7 | [7] | 29.231 | [7] | 29.231 |
| `- 2 - - 5 6 7 8 -` | 17 | 8 | [8] | 29.231 | [8] | 29.231 |
| `- 2 - - 5 6 7 8 -` | 17 | 9 | [2, 7] | 29.231 | [2, 7] | 29.231 |
| `- 2 - - 5 6 7 8 -` | 17 | 10 | [2, 8] | 29.231 | [2, 8] | 29.231 |
| `- 2 - - 5 6 7 8 -` | 17 | 11 | [5, 6] | 29.231 | [5, 6] | 29.231 |
| `- 2 - - 5 6 7 8 -` | 17 | 12 | [5, 7] | 29.231 | [5, 7] | 29.231 |
| `- 2 - 4 - 6 7 - 9` | 17 | 2 | [2] | 28.220 | [2] | 28.220 |
| `- 2 - 4 - 6 7 - 9` | 17 | 4 | [4] | 28.220 | [4] | 28.220 |
| `- 2 - 4 - 6 7 - 9` | 17 | 6 | [6] | 28.220 | [6] | 28.220 |
| `- 2 - 4 - 6 7 - 9` | 17 | 7 | [7] | 28.220 | [7] | 28.220 |
| `- 2 - 4 - 6 7 - 9` | 17 | 8 | [2, 6] | 28.220 | [2, 6] | 28.220 |
| `- 2 - 4 - 6 7 - 9` | 17 | 9 | [9] | 28.220 | [9] | 28.220 |
| `- 2 - 4 - 6 7 - 9` | 17 | 10 | [4, 6] | 28.220 | [4, 6] | 28.220 |
| `- 2 - 4 - 6 7 - 9` | 17 | 11 | [2, 9] | 28.220 | [2, 9] | 28.220 |
| `- 2 - 4 - 6 7 - 9` | 17 | 12 | [2, 4, 6] | 28.220 | [2, 4, 6] | 28.220 |
| `- 2 - 4 5 - - 8 9` | 17 | 2 | [2] | 28.930 | [2] | 28.930 |
| `- 2 - 4 5 - - 8 9` | 17 | 4 | [4] | 28.930 | [4] | 28.930 |
| `- 2 - 4 5 - - 8 9` | 17 | 5 | [5] | 28.930 | [5] | 28.930 |
| `- 2 - 4 5 - - 8 9` | 17 | 6 | [2, 4] | 28.930 | [2, 4] | 28.930 |
| `- 2 - 4 5 - - 8 9` | 17 | 7 | [2, 5] | 28.930 | [2, 5] | 28.930 |
| `- 2 - 4 5 - - 8 9` | 17 | 8 | [8] | 28.930 | [8] | 28.930 |
| `- 2 - 4 5 - - 8 9` | 17 | 9 | [9] | 28.930 | [9] | 28.930 |
| `- 2 - 4 5 - - 8 9` | 17 | 10 | [2, 8] | 28.930 | [2, 8] | 28.930 |
| `- 2 - 4 5 - - 8 9` | 17 | 11 | [2, 9] | 28.930 | [2, 9] | 28.930 |
| `- 2 - 4 5 - - 8 9` | 17 | 12 | [4, 8] | 28.930 | [4, 8] | 28.930 |
| `- 2 3 - - 6 - 8 9` | 17 | 2 | [2] | 26.667 | [2] | 26.667 |
| `- 2 3 - - 6 - 8 9` | 17 | 3 | [3] | 26.667 | [3] | 26.667 |
| `- 2 3 - - 6 - 8 9` | 17 | 5 | [2, 3] | 26.667 | [2, 3] | 26.667 |
| `- 2 3 - - 6 - 8 9` | 17 | 6 | [6] | 26.667 | [6] | 26.667 |
| `- 2 3 - - 6 - 8 9` | 17 | 8 | [8] | 26.667 | [8] | 26.667 |
| `- 2 3 - - 6 - 8 9` | 17 | 9 | [9] | 26.667 | [9] | 26.667 |
| `- 2 3 - - 6 - 8 9` | 17 | 10 | [2, 8] | 26.667 | [2, 8] | 26.667 |
| `- 2 3 - - 6 - 8 9` | 17 | 11 | [2, 9] | 26.667 | [2, 9] | 26.667 |
| `- 2 3 - - 6 - 8 9` | 17 | 12 | [3, 9] | 26.667 | [3, 9] | 26.667 |
| `- 2 3 4 5 6 - 8 -` | 17 | 2 | [2] | 35.450 | [2] | 35.443 |
| `- 2 3 4 5 6 - 8 -` | 17 | 3 | [3] | 35.450 | [3] | 35.443 |
| `- 2 3 4 5 6 - 8 -` | 17 | 4 | [4] | 35.450 | [4] | 35.443 |
| `- 2 3 4 5 6 - 8 -` | 17 | 5 | [5] | 35.450 | [5] | 35.443 |
| `- 2 3 4 5 6 - 8 -` | 17 | 6 | [6] | 35.450 | [6] | 35.443 |
| `- 2 3 4 5 6 - 8 -` | 17 | 7 | [2, 5] | 35.450 | [2, 5] | 35.443 |
| `- 2 3 4 5 6 - 8 -` | 17 | 8 | [8] | 35.450 | [8] | 35.443 |
| `- 2 3 4 5 6 - 8 -` | 17 | 9 | [3, 6] | 35.450 | [3, 6] | 35.443 |
| `- 2 3 4 5 6 - 8 -` | 17 | 10 | [2, 8] | 35.450 | [2, 8] | 35.443 |
| `- 2 3 4 5 6 - 8 -` | 17 | 11 | [3, 8] | 35.450 | [3, 8] | 35.443 |
| `- 2 3 4 5 6 - 8 -` | 17 | 12 | [4, 8] | 35.450 | [4, 8] | 35.443 |
| `1 - - - 5 6 7 - 9` | 17 | 5 | [5] | 28.709 | [5] | 28.709 |
| `1 - - - 5 6 7 - 9` | 17 | 6 | [6] | 28.709 | [6] | 28.709 |
| `1 - - - 5 6 7 - 9` | 17 | 7 | [7] | 28.709 | [7] | 28.709 |
| `1 - - - 5 6 7 - 9` | 17 | 8 | [1, 7] | 28.709 | [1, 7] | 28.709 |
| `1 - - - 5 6 7 - 9` | 17 | 9 | [9] | 28.709 | [9] | 28.709 |
| `1 - - - 5 6 7 - 9` | 17 | 10 | [1, 9] | 28.709 | [1, 9] | 28.709 |
| `1 - - - 5 6 7 - 9` | 17 | 11 | [5, 6] | 28.709 | [5, 6] | 28.709 |
| `1 - - - 5 6 7 - 9` | 17 | 12 | [5, 7] | 28.709 | [5, 7] | 28.709 |
| `1 - - 4 - 6 - 8 9` | 17 | 4 | [4] | 28.869 | [4] | 28.869 |
| `1 - - 4 - 6 - 8 9` | 17 | 5 | [1, 4] | 28.869 | [1, 4] | 28.869 |
| `1 - - 4 - 6 - 8 9` | 17 | 6 | [6] | 28.869 | [6] | 28.869 |
| `1 - - 4 - 6 - 8 9` | 17 | 7 | [1, 6] | 28.869 | [1, 6] | 28.869 |
| `1 - - 4 - 6 - 8 9` | 17 | 8 | [8] | 28.869 | [8] | 28.869 |
| `1 - - 4 - 6 - 8 9` | 17 | 9 | [9] | 28.869 | [9] | 28.869 |
| `1 - - 4 - 6 - 8 9` | 17 | 10 | [1, 9] | 28.869 | [1, 9] | 28.869 |
| `1 - - 4 - 6 - 8 9` | 17 | 11 | [1, 4, 6] | 28.869 | [1, 4, 6] | 28.869 |
| `1 - - 4 - 6 - 8 9` | 17 | 12 | [4, 8] | 28.869 | [4, 8] | 28.869 |
| `1 - 3 - - - 7 8 9` | 17 | 3 | [3] | 26.952 | [3] | 26.952 |
| `1 - 3 - - - 7 8 9` | 17 | 4 | [1, 3] | 26.952 | [1, 3] | 26.952 |
| `1 - 3 - - - 7 8 9` | 17 | 7 | [7] | 26.952 | [7] | 26.952 |
| `1 - 3 - - - 7 8 9` | 17 | 8 | [8] | 26.952 | [8] | 26.952 |
| `1 - 3 - - - 7 8 9` | 17 | 9 | [9] | 26.952 | [9] | 26.952 |
| `1 - 3 - - - 7 8 9` | 17 | 10 | [1, 9] | 26.952 | [1, 9] | 26.952 |
| `1 - 3 - - - 7 8 9` | 17 | 11 | [3, 8] | 26.952 | [3, 8] | 26.952 |
| `1 - 3 - - - 7 8 9` | 17 | 12 | [3, 9] | 26.952 | [3, 9] | 26.952 |
| `1 - 3 4 5 - 7 8 -` | 17 | 3 | [3] | 34.048 | [3] | 34.044 |
| `1 - 3 4 5 - 7 8 -` | 17 | 4 | [4] | 34.048 | [4] | 34.044 |
| `1 - 3 4 5 - 7 8 -` | 17 | 5 | [5] | 34.048 | [5] | 34.044 |
| `1 - 3 4 5 - 7 8 -` | 17 | 6 | [1, 5] | 34.048 | [1, 5] | 34.044 |
| `1 - 3 4 5 - 7 8 -` | 17 | 7 | [7] | 34.048 | [7] | 34.044 |
| `1 - 3 4 5 - 7 8 -` | 17 | 8 | [8] | 34.048 | [8] | 34.044 |
| `1 - 3 4 5 - 7 8 -` | 17 | 9 | [1, 8] | 34.048 | [1, 8] | 34.044 |
| `1 - 3 4 5 - 7 8 -` | 17 | 10 | [3, 7] | 34.048 | [3, 7] | 34.044 |
| `1 - 3 4 5 - 7 8 -` | 17 | 11 | [3, 8] | 34.048 | [3, 8] | 34.044 |
| `1 - 3 4 5 - 7 8 -` | 17 | 12 | [4, 8] | 34.048 | [4, 8] | 34.044 |
| `1 - 3 4 5 6 - - 9` | 17 | 3 | [3] | 33.490 | [3] | 33.333 |
| `1 - 3 4 5 6 - - 9` | 17 | 4 | [4] | 33.490 | [4] | 33.333 |
| `1 - 3 4 5 6 - - 9` | 17 | 5 | [5] | 33.490 | [5] | 33.333 |
| `1 - 3 4 5 6 - - 9` | 17 | 6 | [6] | 33.490 | [6] | 33.333 |
| `1 - 3 4 5 6 - - 9` | 17 | 7 | [3, 4] | 33.490 | [1, 6] | 33.333 |
| `1 - 3 4 5 6 - - 9` | 17 | 8 | [3, 5] | 33.490 | [3, 5] | 33.333 |
| `1 - 3 4 5 6 - - 9` | 17 | 9 | [9] | 33.490 | [9] | 33.333 |
| `1 - 3 4 5 6 - - 9` | 17 | 10 | [1, 9] | 33.490 | [1, 9] | 33.333 |
| `1 - 3 4 5 6 - - 9` | 17 | 11 | [5, 6] | 33.490 | [5, 6] | 33.333 |
| `1 - 3 4 5 6 - - 9` | 17 | 12 | [3, 9] | 33.490 | [3, 9] | 33.333 |
| `1 2 - 4 - 6 7 8 -` | 17 | 2 | [2] | 34.715 | [2] | 34.650 |
| `1 2 - 4 - 6 7 8 -` | 17 | 3 | [1, 2] | 34.715 | [1, 2] | 34.650 |
| `1 2 - 4 - 6 7 8 -` | 17 | 4 | [4] | 34.715 | [4] | 34.650 |
| `1 2 - 4 - 6 7 8 -` | 17 | 5 | [1, 4] | 34.715 | [1, 4] | 34.650 |
| `1 2 - 4 - 6 7 8 -` | 17 | 6 | [6] | 34.715 | [6] | 34.650 |
| `1 2 - 4 - 6 7 8 -` | 17 | 7 | [7] | 34.715 | [7] | 34.650 |
| `1 2 - 4 - 6 7 8 -` | 17 | 8 | [8] | 34.715 | [8] | 34.650 |
| `1 2 - 4 - 6 7 8 -` | 17 | 9 | [2, 7] | 34.715 | [1, 8] | 34.650 |
| `1 2 - 4 - 6 7 8 -` | 17 | 10 | [2, 8] | 34.715 | [2, 8] | 34.650 |
| `1 2 - 4 - 6 7 8 -` | 17 | 11 | [4, 7] | 34.715 | [4, 7] | 34.650 |
| `1 2 - 4 - 6 7 8 -` | 17 | 12 | [4, 8] | 34.715 | [4, 8] | 34.650 |
| `1 2 - 4 5 - 7 - 9` | 17 | 2 | [2] | 33.831 | [2] | 33.469 |
| `1 2 - 4 5 - 7 - 9` | 17 | 3 | [1, 2] | 33.831 | [1, 2] | 33.469 |
| `1 2 - 4 5 - 7 - 9` | 17 | 4 | [4] | 33.831 | [4] | 33.469 |
| `1 2 - 4 5 - 7 - 9` | 17 | 5 | [5] | 33.831 | [5] | 33.469 |
| `1 2 - 4 5 - 7 - 9` | 17 | 6 | [2, 4] | 33.831 | [1, 5] | 33.469 |
| `1 2 - 4 5 - 7 - 9` | 17 | 7 | [7] | 33.831 | [7] | 33.469 |
| `1 2 - 4 5 - 7 - 9` | 17 | 8 | [1, 7] | 33.831 | [1, 7] | 33.469 |
| `1 2 - 4 5 - 7 - 9` | 17 | 9 | [9] | 33.831 | [9] | 33.469 |
| `1 2 - 4 5 - 7 - 9` | 17 | 10 | [1, 9] | 33.831 | [1, 9] | 33.469 |
| `1 2 - 4 5 - 7 - 9` | 17 | 11 | [2, 9] | 33.831 | [2, 9] | 33.469 |
| `1 2 - 4 5 - 7 - 9` | 17 | 12 | [5, 7] | 33.831 | [5, 7] | 33.469 |
| `1 2 3 - - 6 7 - 9` | 17 | 2 | [2] | 33.922 | [2] | 33.775 |
| `1 2 3 - - 6 7 - 9` | 17 | 3 | [3] | 33.922 | [3] | 33.775 |
| `1 2 3 - - 6 7 - 9` | 17 | 4 | [1, 3] | 33.922 | [1, 3] | 33.775 |
| `1 2 3 - - 6 7 - 9` | 17 | 5 | [2, 3] | 33.922 | [2, 3] | 33.775 |
| `1 2 3 - - 6 7 - 9` | 17 | 6 | [6] | 33.922 | [6] | 33.775 |
| `1 2 3 - - 6 7 - 9` | 17 | 7 | [7] | 33.922 | [7] | 33.775 |
| `1 2 3 - - 6 7 - 9` | 17 | 8 | [2, 6] | 33.922 | [1, 7] | 33.775 |
| `1 2 3 - - 6 7 - 9` | 17 | 9 | [9] | 33.922 | [9] | 33.775 |
| `1 2 3 - - 6 7 - 9` | 17 | 10 | [1, 9] | 33.922 | [1, 9] | 33.775 |
| `1 2 3 - - 6 7 - 9` | 17 | 11 | [2, 9] | 33.922 | [2, 9] | 33.775 |
| `1 2 3 - - 6 7 - 9` | 17 | 12 | [3, 9] | 33.922 | [3, 9] | 33.775 |
| `1 2 3 - 5 - - 8 9` | 17 | 2 | [2] | 32.750 | [2] | 32.611 |
| `1 2 3 - 5 - - 8 9` | 17 | 3 | [3] | 32.750 | [3] | 32.611 |
| `1 2 3 - 5 - - 8 9` | 17 | 4 | [1, 3] | 32.750 | [1, 3] | 32.611 |
| `1 2 3 - 5 - - 8 9` | 17 | 5 | [5] | 32.750 | [5] | 32.611 |
| `1 2 3 - 5 - - 8 9` | 17 | 6 | [1, 5] | 32.750 | [1, 5] | 32.611 |
| `1 2 3 - 5 - - 8 9` | 17 | 7 | [2, 5] | 32.750 | [2, 5] | 32.611 |
| `1 2 3 - 5 - - 8 9` | 17 | 8 | [8] | 32.750 | [8] | 32.611 |
| `1 2 3 - 5 - - 8 9` | 17 | 9 | [9] | 32.750 | [9] | 32.611 |
| `1 2 3 - 5 - - 8 9` | 17 | 10 | [2, 8] | 32.750 | [1, 9] | 32.611 |
| `1 2 3 - 5 - - 8 9` | 17 | 11 | [3, 8] | 32.750 | [2, 9] | 32.611 |
| `1 2 3 - 5 - - 8 9` | 17 | 12 | [3, 9] | 32.750 | [3, 9] | 32.611 |
| `1 2 3 4 5 6 7 - -` | 17 | 2 | [2] | 39.466 | [2] | 39.339 |
| `1 2 3 4 5 6 7 - -` | 17 | 3 | [3] | 39.466 | [3] | 39.339 |
| `1 2 3 4 5 6 7 - -` | 17 | 4 | [4] | 39.466 | [4] | 39.339 |
| `1 2 3 4 5 6 7 - -` | 17 | 5 | [5] | 39.466 | [5] | 39.339 |
| `1 2 3 4 5 6 7 - -` | 17 | 6 | [6] | 39.466 | [6] | 39.339 |
| `1 2 3 4 5 6 7 - -` | 17 | 7 | [7] | 39.466 | [7] | 39.339 |
| `1 2 3 4 5 6 7 - -` | 17 | 8 | [1, 7] | 39.466 | [1, 7] | 39.339 |
| `1 2 3 4 5 6 7 - -` | 17 | 9 | [3, 6] | 39.466 | [2, 7] | 39.339 |
| `1 2 3 4 5 6 7 - -` | 17 | 10 | [3, 7] | 39.466 | [3, 7] | 39.339 |
| `1 2 3 4 5 6 7 - -` | 17 | 11 | [4, 7] | 39.466 | [4, 7] | 39.339 |
| `1 2 3 4 5 6 7 - -` | 17 | 12 | [5, 7] | 39.466 | [5, 7] | 39.339 |
| `- - - - 5 6 7 - 9` | 18 | 5 | [5] | 24.804 | [5] | 24.804 |
| `- - - - 5 6 7 - 9` | 18 | 6 | [6] | 24.804 | [6] | 24.804 |
| `- - - - 5 6 7 - 9` | 18 | 7 | [7] | 24.804 | [7] | 24.804 |
| `- - - - 5 6 7 - 9` | 18 | 9 | [9] | 24.804 | [9] | 24.804 |
| `- - - - 5 6 7 - 9` | 18 | 11 | [5, 6] | 24.804 | [5, 6] | 24.804 |
| `- - - - 5 6 7 - 9` | 18 | 12 | [5, 7] | 24.804 | [5, 7] | 24.804 |
| `- - - 4 - 6 - 8 9` | 18 | 4 | [4] | 24.483 | [4] | 24.483 |
| `- - - 4 - 6 - 8 9` | 18 | 6 | [6] | 24.483 | [6] | 24.483 |
| `- - - 4 - 6 - 8 9` | 18 | 8 | [8] | 24.483 | [8] | 24.483 |
| `- - - 4 - 6 - 8 9` | 18 | 9 | [9] | 24.483 | [9] | 24.483 |
| `- - - 4 - 6 - 8 9` | 18 | 10 | [4, 6] | 24.483 | [4, 6] | 24.483 |
| `- - - 4 - 6 - 8 9` | 18 | 12 | [4, 8] | 24.483 | [4, 8] | 24.483 |
| `- - 3 - - - 7 8 9` | 18 | 3 | [3] | 25.645 | [3] | 25.645 |
| `- - 3 - - - 7 8 9` | 18 | 7 | [7] | 25.645 | [7] | 25.645 |
| `- - 3 - - - 7 8 9` | 18 | 8 | [8] | 25.645 | [8] | 25.645 |
| `- - 3 - - - 7 8 9` | 18 | 9 | [9] | 25.645 | [9] | 25.645 |
| `- - 3 - - - 7 8 9` | 18 | 10 | [3, 7] | 25.645 | [3, 7] | 25.645 |
| `- - 3 - - - 7 8 9` | 18 | 11 | [3, 8] | 25.645 | [3, 8] | 25.645 |
| `- - 3 - - - 7 8 9` | 18 | 12 | [3, 9] | 25.645 | [3, 9] | 25.645 |
| `- - 3 4 5 - 7 8 -` | 18 | 3 | [3] | 29.716 | [3] | 29.716 |
| `- - 3 4 5 - 7 8 -` | 18 | 4 | [4] | 29.716 | [4] | 29.716 |
| `- - 3 4 5 - 7 8 -` | 18 | 5 | [5] | 29.716 | [5] | 29.716 |
| `- - 3 4 5 - 7 8 -` | 18 | 7 | [7] | 29.716 | [7] | 29.716 |
| `- - 3 4 5 - 7 8 -` | 18 | 8 | [8] | 29.716 | [8] | 29.716 |
| `- - 3 4 5 - 7 8 -` | 18 | 9 | [4, 5] | 29.716 | [4, 5] | 29.716 |
| `- - 3 4 5 - 7 8 -` | 18 | 10 | [3, 7] | 29.716 | [3, 7] | 29.716 |
| `- - 3 4 5 - 7 8 -` | 18 | 11 | [3, 8] | 29.716 | [3, 8] | 29.716 |
| `- - 3 4 5 - 7 8 -` | 18 | 12 | [4, 8] | 29.716 | [4, 8] | 29.716 |
| `- - 3 4 5 6 - - 9` | 18 | 3 | [3] | 30.420 | [3] | 30.419 |
| `- - 3 4 5 6 - - 9` | 18 | 4 | [4] | 30.420 | [4] | 30.419 |
| `- - 3 4 5 6 - - 9` | 18 | 5 | [5] | 30.420 | [5] | 30.419 |
| `- - 3 4 5 6 - - 9` | 18 | 6 | [6] | 30.420 | [6] | 30.419 |
| `- - 3 4 5 6 - - 9` | 18 | 7 | [3, 4] | 30.420 | [3, 4] | 30.419 |
| `- - 3 4 5 6 - - 9` | 18 | 8 | [3, 5] | 30.420 | [3, 5] | 30.419 |
| `- - 3 4 5 6 - - 9` | 18 | 9 | [9] | 30.420 | [9] | 30.419 |
| `- - 3 4 5 6 - - 9` | 18 | 10 | [4, 6] | 30.420 | [4, 6] | 30.419 |
| `- - 3 4 5 6 - - 9` | 18 | 11 | [5, 6] | 30.420 | [5, 6] | 30.419 |
| `- - 3 4 5 6 - - 9` | 18 | 12 | [3, 9] | 30.420 | [3, 9] | 30.419 |
| `- 2 - 4 - 6 7 8 -` | 18 | 2 | [2] | 29.768 | [2] | 29.740 |
| `- 2 - 4 - 6 7 8 -` | 18 | 4 | [4] | 29.768 | [4] | 29.740 |
| `- 2 - 4 - 6 7 8 -` | 18 | 6 | [6] | 29.768 | [6] | 29.740 |
| `- 2 - 4 - 6 7 8 -` | 18 | 7 | [7] | 29.768 | [7] | 29.740 |
| `- 2 - 4 - 6 7 8 -` | 18 | 8 | [8] | 29.768 | [8] | 29.740 |
| `- 2 - 4 - 6 7 8 -` | 18 | 9 | [2, 7] | 29.768 | [2, 7] | 29.740 |
| `- 2 - 4 - 6 7 8 -` | 18 | 10 | [4, 6] | 29.768 | [2, 8] | 29.740 |
| `- 2 - 4 - 6 7 8 -` | 18 | 11 | [4, 7] | 29.768 | [4, 7] | 29.740 |
| `- 2 - 4 - 6 7 8 -` | 18 | 12 | [4, 8] | 29.768 | [4, 8] | 29.740 |
| `- 2 - 4 5 - 7 - 9` | 18 | 2 | [2] | 26.978 | [2] | 26.974 |
| `- 2 - 4 5 - 7 - 9` | 18 | 4 | [4] | 26.978 | [4] | 26.974 |
| `- 2 - 4 5 - 7 - 9` | 18 | 5 | [5] | 26.978 | [5] | 26.974 |
| `- 2 - 4 5 - 7 - 9` | 18 | 6 | [2, 4] | 26.978 | [2, 4] | 26.974 |
| `- 2 - 4 5 - 7 - 9` | 18 | 7 | [7] | 26.978 | [7] | 26.974 |
| `- 2 - 4 5 - 7 - 9` | 18 | 9 | [9] | 26.978 | [9] | 26.974 |
| `- 2 - 4 5 - 7 - 9` | 18 | 11 | [2, 9] | 26.978 | [2, 9] | 26.974 |
| `- 2 - 4 5 - 7 - 9` | 18 | 12 | [5, 7] | 26.978 | [5, 7] | 26.974 |
| `- 2 3 - - 6 7 - 9` | 18 | 2 | [2] | 30.135 | [2] | 30.133 |
| `- 2 3 - - 6 7 - 9` | 18 | 3 | [3] | 30.135 | [3] | 30.133 |
| `- 2 3 - - 6 7 - 9` | 18 | 5 | [2, 3] | 30.135 | [2, 3] | 30.133 |
| `- 2 3 - - 6 7 - 9` | 18 | 6 | [6] | 30.135 | [6] | 30.133 |
| `- 2 3 - - 6 7 - 9` | 18 | 7 | [7] | 30.135 | [7] | 30.133 |
| `- 2 3 - - 6 7 - 9` | 18 | 8 | [2, 6] | 30.135 | [2, 6] | 30.133 |
| `- 2 3 - - 6 7 - 9` | 18 | 9 | [9] | 30.135 | [9] | 30.133 |
| `- 2 3 - - 6 7 - 9` | 18 | 10 | [3, 7] | 30.135 | [3, 7] | 30.133 |
| `- 2 3 - - 6 7 - 9` | 18 | 11 | [2, 9] | 30.135 | [2, 9] | 30.133 |
| `- 2 3 - - 6 7 - 9` | 18 | 12 | [3, 9] | 30.135 | [3, 9] | 30.133 |
| `- 2 3 - 5 - - 8 9` | 18 | 2 | [2] | 28.213 | [2] | 28.159 |
| `- 2 3 - 5 - - 8 9` | 18 | 3 | [3] | 28.213 | [3] | 28.159 |
| `- 2 3 - 5 - - 8 9` | 18 | 5 | [5] | 28.213 | [5] | 28.159 |
| `- 2 3 - 5 - - 8 9` | 18 | 7 | [2, 5] | 28.213 | [2, 5] | 28.159 |
| `- 2 3 - 5 - - 8 9` | 18 | 8 | [8] | 28.213 | [8] | 28.159 |
| `- 2 3 - 5 - - 8 9` | 18 | 9 | [9] | 28.213 | [9] | 28.159 |
| `- 2 3 - 5 - - 8 9` | 18 | 10 | [2, 8] | 28.213 | [2, 8] | 28.159 |
| `- 2 3 - 5 - - 8 9` | 18 | 11 | [3, 8] | 28.213 | [2, 9] | 28.159 |
| `- 2 3 - 5 - - 8 9` | 18 | 12 | [3, 9] | 28.213 | [3, 9] | 28.159 |
| `- 2 3 4 5 6 7 - -` | 18 | 2 | [2] | 36.797 | [2] | 36.776 |
| `- 2 3 4 5 6 7 - -` | 18 | 3 | [3] | 36.797 | [3] | 36.776 |
| `- 2 3 4 5 6 7 - -` | 18 | 4 | [4] | 36.797 | [4] | 36.776 |
| `- 2 3 4 5 6 7 - -` | 18 | 5 | [5] | 36.797 | [5] | 36.776 |
| `- 2 3 4 5 6 7 - -` | 18 | 6 | [6] | 36.797 | [6] | 36.776 |
| `- 2 3 4 5 6 7 - -` | 18 | 7 | [7] | 36.797 | [7] | 36.776 |
| `- 2 3 4 5 6 7 - -` | 18 | 8 | [3, 5] | 36.797 | [2, 6] | 36.776 |
| `- 2 3 4 5 6 7 - -` | 18 | 9 | [2, 7] | 36.797 | [2, 7] | 36.776 |
| `- 2 3 4 5 6 7 - -` | 18 | 10 | [3, 7] | 36.797 | [3, 7] | 36.776 |
| `- 2 3 4 5 6 7 - -` | 18 | 11 | [4, 7] | 36.797 | [4, 7] | 36.776 |
| `- 2 3 4 5 6 7 - -` | 18 | 12 | [5, 7] | 36.797 | [5, 7] | 36.776 |
| `1 - - - 5 6 7 8 -` | 18 | 5 | [5] | 28.500 | [5] | 28.500 |
| `1 - - - 5 6 7 8 -` | 18 | 6 | [6] | 28.500 | [6] | 28.500 |
| `1 - - - 5 6 7 8 -` | 18 | 7 | [7] | 28.500 | [7] | 28.500 |
| `1 - - - 5 6 7 8 -` | 18 | 8 | [8] | 28.500 | [8] | 28.500 |
| `1 - - - 5 6 7 8 -` | 18 | 9 | [1, 8] | 28.500 | [1, 8] | 28.500 |
| `1 - - - 5 6 7 8 -` | 18 | 11 | [5, 6] | 28.500 | [5, 6] | 28.500 |
| `1 - - - 5 6 7 8 -` | 18 | 12 | [5, 7] | 28.500 | [5, 7] | 28.500 |
| `1 - - 4 - 6 7 - 9` | 18 | 4 | [4] | 30.356 | [4] | 30.345 |
| `1 - - 4 - 6 7 - 9` | 18 | 5 | [1, 4] | 30.356 | [1, 4] | 30.345 |
| `1 - - 4 - 6 7 - 9` | 18 | 6 | [6] | 30.356 | [6] | 30.345 |
| `1 - - 4 - 6 7 - 9` | 18 | 7 | [7] | 30.356 | [7] | 30.345 |
| `1 - - 4 - 6 7 - 9` | 18 | 8 | [1, 7] | 30.356 | [1, 7] | 30.345 |
| `1 - - 4 - 6 7 - 9` | 18 | 9 | [9] | 30.356 | [9] | 30.345 |
| `1 - - 4 - 6 7 - 9` | 18 | 10 | [4, 6] | 30.356 | [1, 9] | 30.345 |
| `1 - - 4 - 6 7 - 9` | 18 | 11 | [4, 7] | 30.356 | [4, 7] | 30.345 |
| `1 - - 4 - 6 7 - 9` | 18 | 12 | [1, 4, 7] | 30.356 | [1, 4, 7] | 30.345 |
| `1 - - 4 5 - - 8 9` | 18 | 4 | [4] | 26.547 | [4] | 26.545 |
| `1 - - 4 5 - - 8 9` | 18 | 5 | [5] | 26.547 | [5] | 26.545 |
| `1 - - 4 5 - - 8 9` | 18 | 6 | [1, 5] | 26.547 | [1, 5] | 26.545 |
| `1 - - 4 5 - - 8 9` | 18 | 8 | [8] | 26.547 | [8] | 26.545 |
| `1 - - 4 5 - - 8 9` | 18 | 9 | [9] | 26.547 | [9] | 26.545 |
| `1 - - 4 5 - - 8 9` | 18 | 10 | [1, 9] | 26.547 | [1, 9] | 26.545 |
| `1 - - 4 5 - - 8 9` | 18 | 12 | [4, 8] | 26.547 | [4, 8] | 26.545 |
| `1 - 3 - - 6 - 8 9` | 18 | 3 | [3] | 29.231 | [3] | 29.231 |
| `1 - 3 - - 6 - 8 9` | 18 | 4 | [1, 3] | 29.231 | [1, 3] | 29.231 |
| `1 - 3 - - 6 - 8 9` | 18 | 6 | [6] | 29.231 | [6] | 29.231 |
| `1 - 3 - - 6 - 8 9` | 18 | 7 | [1, 6] | 29.231 | [1, 6] | 29.231 |
| `1 - 3 - - 6 - 8 9` | 18 | 8 | [8] | 29.231 | [8] | 29.231 |
| `1 - 3 - - 6 - 8 9` | 18 | 9 | [9] | 29.231 | [9] | 29.231 |
| `1 - 3 - - 6 - 8 9` | 18 | 10 | [1, 9] | 29.231 | [1, 9] | 29.231 |
| `1 - 3 - - 6 - 8 9` | 18 | 11 | [3, 8] | 29.231 | [3, 8] | 29.231 |
| `1 - 3 - - 6 - 8 9` | 18 | 12 | [3, 9] | 29.231 | [3, 9] | 29.231 |
| `1 - 3 4 5 6 - 8 -` | 18 | 3 | [3] | 35.317 | [3] | 35.239 |
| `1 - 3 4 5 6 - 8 -` | 18 | 4 | [4] | 35.317 | [4] | 35.239 |
| `1 - 3 4 5 6 - 8 -` | 18 | 5 | [5] | 35.317 | [5] | 35.239 |
| `1 - 3 4 5 6 - 8 -` | 18 | 6 | [6] | 35.317 | [6] | 35.239 |
| `1 - 3 4 5 6 - 8 -` | 18 | 7 | [3, 4] | 35.317 | [1, 6] | 35.239 |
| `1 - 3 4 5 6 - 8 -` | 18 | 8 | [8] | 35.317 | [8] | 35.239 |
| `1 - 3 4 5 6 - 8 -` | 18 | 9 | [1, 8] | 35.317 | [1, 8] | 35.239 |
| `1 - 3 4 5 6 - 8 -` | 18 | 10 | [4, 6] | 35.317 | [4, 6] | 35.239 |
| `1 - 3 4 5 6 - 8 -` | 18 | 11 | [3, 8] | 35.317 | [3, 8] | 35.239 |
| `1 - 3 4 5 6 - 8 -` | 18 | 12 | [4, 8] | 35.317 | [4, 8] | 35.239 |
| `1 2 - - - - 7 8 9` | 18 | 2 | [2] | 27.104 | [2] | 27.104 |
| `1 2 - - - - 7 8 9` | 18 | 3 | [1, 2] | 27.104 | [1, 2] | 27.104 |
| `1 2 - - - - 7 8 9` | 18 | 7 | [7] | 27.104 | [7] | 27.104 |
| `1 2 - - - - 7 8 9` | 18 | 8 | [8] | 27.104 | [8] | 27.104 |
| `1 2 - - - - 7 8 9` | 18 | 9 | [9] | 27.104 | [9] | 27.104 |
| `1 2 - - - - 7 8 9` | 18 | 10 | [1, 9] | 27.104 | [1, 9] | 27.104 |
| `1 2 - - - - 7 8 9` | 18 | 11 | [2, 9] | 27.104 | [2, 9] | 27.104 |
| `1 2 - - - - 7 8 9` | 18 | 12 | [1, 2, 9] | 27.104 | [1, 2, 9] | 27.104 |
| `1 2 - 4 5 - 7 8 -` | 18 | 2 | [2] | 35.748 | [2] | 35.704 |
| `1 2 - 4 5 - 7 8 -` | 18 | 3 | [1, 2] | 35.748 | [1, 2] | 35.704 |
| `1 2 - 4 5 - 7 8 -` | 18 | 4 | [4] | 35.748 | [4] | 35.704 |
| `1 2 - 4 5 - 7 8 -` | 18 | 5 | [5] | 35.748 | [5] | 35.704 |
| `1 2 - 4 5 - 7 8 -` | 18 | 6 | [1, 5] | 35.748 | [1, 5] | 35.704 |
| `1 2 - 4 5 - 7 8 -` | 18 | 7 | [7] | 35.748 | [7] | 35.704 |
| `1 2 - 4 5 - 7 8 -` | 18 | 8 | [8] | 35.748 | [8] | 35.704 |
| `1 2 - 4 5 - 7 8 -` | 18 | 9 | [1, 8] | 35.748 | [1, 8] | 35.704 |
| `1 2 - 4 5 - 7 8 -` | 18 | 10 | [2, 8] | 35.748 | [2, 8] | 35.704 |
| `1 2 - 4 5 - 7 8 -` | 18 | 11 | [4, 7] | 35.748 | [4, 7] | 35.704 |
| `1 2 - 4 5 - 7 8 -` | 18 | 12 | [5, 7] | 35.748 | [4, 8] | 35.704 |
| `1 2 - 4 5 6 - - 9` | 18 | 2 | [2] | 34.288 | [2] | 34.120 |
| `1 2 - 4 5 6 - - 9` | 18 | 3 | [1, 2] | 34.288 | [1, 2] | 34.120 |
| `1 2 - 4 5 6 - - 9` | 18 | 4 | [4] | 34.288 | [4] | 34.120 |
| `1 2 - 4 5 6 - - 9` | 18 | 5 | [5] | 34.288 | [5] | 34.120 |
| `1 2 - 4 5 6 - - 9` | 18 | 6 | [6] | 34.288 | [6] | 34.120 |
| `1 2 - 4 5 6 - - 9` | 18 | 7 | [2, 5] | 34.288 | [1, 6] | 34.120 |
| `1 2 - 4 5 6 - - 9` | 18 | 8 | [2, 6] | 34.288 | [2, 6] | 34.120 |
| `1 2 - 4 5 6 - - 9` | 18 | 9 | [9] | 34.288 | [9] | 34.120 |
| `1 2 - 4 5 6 - - 9` | 18 | 10 | [1, 9] | 34.288 | [1, 9] | 34.120 |
| `1 2 - 4 5 6 - - 9` | 18 | 11 | [2, 9] | 34.288 | [2, 9] | 34.120 |
| `1 2 - 4 5 6 - - 9` | 18 | 12 | [1, 2, 9] | 34.288 | [1, 2, 9] | 34.120 |
| `1 2 3 - - 6 7 8 -` | 18 | 2 | [2] | 35.176 | [2] | 35.169 |
| `1 2 3 - - 6 7 8 -` | 18 | 3 | [3] | 35.176 | [3] | 35.169 |
| `1 2 3 - - 6 7 8 -` | 18 | 4 | [1, 3] | 35.176 | [1, 3] | 35.169 |
| `1 2 3 - - 6 7 8 -` | 18 | 5 | [2, 3] | 35.176 | [2, 3] | 35.169 |
| `1 2 3 - - 6 7 8 -` | 18 | 6 | [6] | 35.176 | [6] | 35.169 |
| `1 2 3 - - 6 7 8 -` | 18 | 7 | [7] | 35.176 | [7] | 35.169 |
| `1 2 3 - - 6 7 8 -` | 18 | 8 | [8] | 35.176 | [8] | 35.169 |
| `1 2 3 - - 6 7 8 -` | 18 | 9 | [1, 8] | 35.176 | [1, 8] | 35.169 |
| `1 2 3 - - 6 7 8 -` | 18 | 10 | [2, 8] | 35.176 | [2, 8] | 35.169 |
| `1 2 3 - - 6 7 8 -` | 18 | 11 | [3, 8] | 35.176 | [3, 8] | 35.169 |
| `1 2 3 - - 6 7 8 -` | 18 | 12 | [1, 3, 8] | 35.176 | [1, 3, 8] | 35.169 |
| `1 2 3 - 5 - 7 - 9` | 18 | 2 | [2] | 34.571 | [2] | 34.465 |
| `1 2 3 - 5 - 7 - 9` | 18 | 3 | [3] | 34.571 | [3] | 34.465 |
| `1 2 3 - 5 - 7 - 9` | 18 | 4 | [1, 3] | 34.571 | [1, 3] | 34.465 |
| `1 2 3 - 5 - 7 - 9` | 18 | 5 | [5] | 34.571 | [5] | 34.465 |
| `1 2 3 - 5 - 7 - 9` | 18 | 6 | [1, 5] | 34.571 | [1, 5] | 34.465 |
| `1 2 3 - 5 - 7 - 9` | 18 | 7 | [7] | 34.571 | [7] | 34.465 |
| `1 2 3 - 5 - 7 - 9` | 18 | 8 | [1, 7] | 34.571 | [1, 7] | 34.465 |
| `1 2 3 - 5 - 7 - 9` | 18 | 9 | [9] | 34.571 | [9] | 34.465 |
| `1 2 3 - 5 - 7 - 9` | 18 | 10 | [3, 7] | 34.571 | [1, 9] | 34.465 |
| `1 2 3 - 5 - 7 - 9` | 18 | 11 | [2, 9] | 34.571 | [2, 9] | 34.465 |
| `1 2 3 - 5 - 7 - 9` | 18 | 12 | [3, 9] | 34.571 | [3, 9] | 34.465 |
| `1 2 3 4 - - - 8 9` | 18 | 2 | [2] | 33.497 | [2] | 33.477 |
| `1 2 3 4 - - - 8 9` | 18 | 3 | [3] | 33.497 | [3] | 33.477 |
| `1 2 3 4 - - - 8 9` | 18 | 4 | [4] | 33.497 | [4] | 33.477 |
| `1 2 3 4 - - - 8 9` | 18 | 5 | [1, 4] | 33.497 | [1, 4] | 33.477 |
| `1 2 3 4 - - - 8 9` | 18 | 6 | [2, 4] | 33.497 | [2, 4] | 33.477 |
| `1 2 3 4 - - - 8 9` | 18 | 7 | [3, 4] | 33.497 | [3, 4] | 33.477 |
| `1 2 3 4 - - - 8 9` | 18 | 8 | [8] | 33.497 | [8] | 33.477 |
| `1 2 3 4 - - - 8 9` | 18 | 9 | [9] | 33.497 | [9] | 33.477 |
| `1 2 3 4 - - - 8 9` | 18 | 10 | [1, 9] | 33.497 | [1, 9] | 33.477 |
| `1 2 3 4 - - - 8 9` | 18 | 11 | [3, 8] | 33.497 | [2, 9] | 33.477 |
| `1 2 3 4 - - - 8 9` | 18 | 12 | [3, 9] | 33.497 | [3, 9] | 33.477 |
| `- - - - 5 6 7 8 -` | 19 | 5 | [5] | 26.144 | [5] | 26.144 |
| `- - - - 5 6 7 8 -` | 19 | 6 | [6] | 26.144 | [6] | 26.144 |
| `- - - - 5 6 7 8 -` | 19 | 7 | [7] | 26.144 | [7] | 26.144 |
| `- - - - 5 6 7 8 -` | 19 | 8 | [8] | 26.144 | [8] | 26.144 |
| `- - - - 5 6 7 8 -` | 19 | 11 | [5, 6] | 26.144 | [5, 6] | 26.144 |
| `- - - - 5 6 7 8 -` | 19 | 12 | [5, 7] | 26.144 | [5, 7] | 26.144 |
| `- - - 4 - 6 7 - 9` | 19 | 4 | [4] | 26.110 | [4] | 26.110 |
| `- - - 4 - 6 7 - 9` | 19 | 6 | [6] | 26.110 | [6] | 26.110 |
| `- - - 4 - 6 7 - 9` | 19 | 7 | [7] | 26.110 | [7] | 26.110 |
| `- - - 4 - 6 7 - 9` | 19 | 9 | [9] | 26.110 | [9] | 26.110 |
| `- - - 4 - 6 7 - 9` | 19 | 10 | [4, 6] | 26.110 | [4, 6] | 26.110 |
| `- - - 4 - 6 7 - 9` | 19 | 11 | [4, 7] | 26.110 | [4, 7] | 26.110 |
| `- - - 4 5 - - 8 9` | 19 | 4 | [4] | 23.835 | [4] | 23.835 |
| `- - - 4 5 - - 8 9` | 19 | 5 | [5] | 23.835 | [5] | 23.835 |
| `- - - 4 5 - - 8 9` | 19 | 8 | [8] | 23.835 | [8] | 23.835 |
| `- - - 4 5 - - 8 9` | 19 | 9 | [9] | 23.835 | [9] | 23.835 |
| `- - - 4 5 - - 8 9` | 19 | 12 | [4, 8] | 23.835 | [4, 8] | 23.835 |
| `- - 3 - - 6 - 8 9` | 19 | 3 | [3] | 24.896 | [3] | 24.896 |
| `- - 3 - - 6 - 8 9` | 19 | 6 | [6] | 24.896 | [6] | 24.896 |
| `- - 3 - - 6 - 8 9` | 19 | 8 | [8] | 24.896 | [8] | 24.896 |
| `- - 3 - - 6 - 8 9` | 19 | 9 | [9] | 24.896 | [9] | 24.896 |
| `- - 3 - - 6 - 8 9` | 19 | 11 | [3, 8] | 24.896 | [3, 8] | 24.896 |
| `- - 3 - - 6 - 8 9` | 19 | 12 | [3, 9] | 24.896 | [3, 9] | 24.896 |
| `- - 3 4 5 6 - 8 -` | 19 | 3 | [3] | 32.285 | [3] | 32.233 |
| `- - 3 4 5 6 - 8 -` | 19 | 4 | [4] | 32.285 | [4] | 32.233 |
| `- - 3 4 5 6 - 8 -` | 19 | 5 | [5] | 32.285 | [5] | 32.233 |
| `- - 3 4 5 6 - 8 -` | 19 | 6 | [6] | 32.285 | [6] | 32.233 |
| `- - 3 4 5 6 - 8 -` | 19 | 7 | [3, 4] | 32.285 | [3, 4] | 32.233 |
| `- - 3 4 5 6 - 8 -` | 19 | 8 | [8] | 32.285 | [8] | 32.233 |
| `- - 3 4 5 6 - 8 -` | 19 | 9 | [4, 5] | 32.285 | [3, 6] | 32.233 |
| `- - 3 4 5 6 - 8 -` | 19 | 10 | [4, 6] | 32.285 | [4, 6] | 32.233 |
| `- - 3 4 5 6 - 8 -` | 19 | 11 | [3, 8] | 32.285 | [3, 8] | 32.233 |
| `- - 3 4 5 6 - 8 -` | 19 | 12 | [4, 8] | 32.285 | [4, 8] | 32.233 |
| `- 2 - - - - 7 8 9` | 19 | 2 | [2] | 26.017 | [2] | 26.017 |
| `- 2 - - - - 7 8 9` | 19 | 7 | [7] | 26.017 | [7] | 26.017 |
| `- 2 - - - - 7 8 9` | 19 | 8 | [8] | 26.017 | [8] | 26.017 |
| `- 2 - - - - 7 8 9` | 19 | 9 | [9] | 26.017 | [9] | 26.017 |
| `- 2 - - - - 7 8 9` | 19 | 10 | [2, 8] | 26.017 | [2, 8] | 26.017 |
| `- 2 - - - - 7 8 9` | 19 | 11 | [2, 9] | 26.017 | [2, 9] | 26.017 |
| `- 2 - 4 5 - 7 8 -` | 19 | 2 | [2] | 32.217 | [2] | 32.093 |
| `- 2 - 4 5 - 7 8 -` | 19 | 4 | [4] | 32.217 | [4] | 32.093 |
| `- 2 - 4 5 - 7 8 -` | 19 | 5 | [5] | 32.217 | [5] | 32.093 |
| `- 2 - 4 5 - 7 8 -` | 19 | 6 | [2, 4] | 32.217 | [2, 4] | 32.093 |
| `- 2 - 4 5 - 7 8 -` | 19 | 7 | [7] | 32.217 | [7] | 32.093 |
| `- 2 - 4 5 - 7 8 -` | 19 | 8 | [8] | 32.217 | [8] | 32.093 |
| `- 2 - 4 5 - 7 8 -` | 19 | 9 | [4, 5] | 32.217 | [2, 7] | 32.093 |
| `- 2 - 4 5 - 7 8 -` | 19 | 10 | [2, 8] | 32.217 | [2, 8] | 32.093 |
| `- 2 - 4 5 - 7 8 -` | 19 | 11 | [4, 7] | 32.217 | [4, 7] | 32.093 |
| `- 2 - 4 5 - 7 8 -` | 19 | 12 | [5, 7] | 32.217 | [4, 8] | 32.093 |
| `- 2 - 4 5 6 - - 9` | 19 | 2 | [2] | 31.000 | [2] | 31.000 |
| `- 2 - 4 5 6 - - 9` | 19 | 4 | [4] | 31.000 | [4] | 31.000 |
| `- 2 - 4 5 6 - - 9` | 19 | 5 | [5] | 31.000 | [5] | 31.000 |
| `- 2 - 4 5 6 - - 9` | 19 | 6 | [6] | 31.000 | [6] | 31.000 |
| `- 2 - 4 5 6 - - 9` | 19 | 7 | [2, 5] | 31.000 | [2, 5] | 31.000 |
| `- 2 - 4 5 6 - - 9` | 19 | 8 | [2, 6] | 31.000 | [2, 6] | 31.000 |
| `- 2 - 4 5 6 - - 9` | 19 | 9 | [9] | 31.000 | [9] | 31.000 |
| `- 2 - 4 5 6 - - 9` | 19 | 10 | [4, 6] | 31.000 | [4, 6] | 31.000 |
| `- 2 - 4 5 6 - - 9` | 19 | 11 | [2, 9] | 31.000 | [2, 9] | 31.000 |
| `- 2 - 4 5 6 - - 9` | 19 | 12 | [2, 4, 6] | 31.000 | [2, 4, 6] | 31.000 |
| `- 2 3 - - 6 7 8 -` | 19 | 2 | [2] | 31.854 | [2] | 31.792 |
| `- 2 3 - - 6 7 8 -` | 19 | 3 | [3] | 31.854 | [3] | 31.792 |
| `- 2 3 - - 6 7 8 -` | 19 | 5 | [2, 3] | 31.854 | [2, 3] | 31.792 |
| `- 2 3 - - 6 7 8 -` | 19 | 6 | [6] | 31.854 | [6] | 31.792 |
| `- 2 3 - - 6 7 8 -` | 19 | 7 | [7] | 31.854 | [7] | 31.792 |
| `- 2 3 - - 6 7 8 -` | 19 | 8 | [8] | 31.854 | [8] | 31.792 |
| `- 2 3 - - 6 7 8 -` | 19 | 9 | [3, 6] | 31.854 | [2, 7] | 31.792 |
| `- 2 3 - - 6 7 8 -` | 19 | 10 | [2, 8] | 31.854 | [2, 8] | 31.792 |
| `- 2 3 - - 6 7 8 -` | 19 | 11 | [3, 8] | 31.854 | [3, 8] | 31.792 |
| `- 2 3 - - 6 7 8 -` | 19 | 12 | [2, 3, 7] | 31.854 | [2, 3, 7] | 31.792 |
| `- 2 3 - 5 - 7 - 9` | 19 | 2 | [2] | 29.412 | [2] | 29.412 |
| `- 2 3 - 5 - 7 - 9` | 19 | 3 | [3] | 29.412 | [3] | 29.412 |
| `- 2 3 - 5 - 7 - 9` | 19 | 5 | [5] | 29.412 | [5] | 29.412 |
| `- 2 3 - 5 - 7 - 9` | 19 | 7 | [7] | 29.412 | [7] | 29.412 |
| `- 2 3 - 5 - 7 - 9` | 19 | 8 | [3, 5] | 29.412 | [3, 5] | 29.412 |
| `- 2 3 - 5 - 7 - 9` | 19 | 9 | [9] | 29.412 | [9] | 29.412 |
| `- 2 3 - 5 - 7 - 9` | 19 | 10 | [3, 7] | 29.412 | [3, 7] | 29.412 |
| `- 2 3 - 5 - 7 - 9` | 19 | 11 | [2, 9] | 29.412 | [2, 9] | 29.412 |
| `- 2 3 - 5 - 7 - 9` | 19 | 12 | [3, 9] | 29.412 | [3, 9] | 29.412 |
| `- 2 3 4 - - - 8 9` | 19 | 2 | [2] | 31.411 | [2] | 31.411 |
| `- 2 3 4 - - - 8 9` | 19 | 3 | [3] | 31.411 | [3] | 31.411 |
| `- 2 3 4 - - - 8 9` | 19 | 4 | [4] | 31.411 | [4] | 31.411 |
| `- 2 3 4 - - - 8 9` | 19 | 5 | [2, 3] | 31.411 | [2, 3] | 31.411 |
| `- 2 3 4 - - - 8 9` | 19 | 6 | [2, 4] | 31.411 | [2, 4] | 31.411 |
| `- 2 3 4 - - - 8 9` | 19 | 7 | [3, 4] | 31.411 | [3, 4] | 31.411 |
| `- 2 3 4 - - - 8 9` | 19 | 8 | [8] | 31.411 | [8] | 31.411 |
| `- 2 3 4 - - - 8 9` | 19 | 9 | [9] | 31.411 | [9] | 31.411 |
| `- 2 3 4 - - - 8 9` | 19 | 10 | [2, 8] | 31.411 | [2, 8] | 31.411 |
| `- 2 3 4 - - - 8 9` | 19 | 11 | [2, 9] | 31.411 | [2, 9] | 31.411 |
| `- 2 3 4 - - - 8 9` | 19 | 12 | [3, 9] | 31.411 | [3, 9] | 31.411 |
| `1 - - 4 - 6 7 8 -` | 19 | 4 | [4] | 31.801 | [4] | 31.801 |
| `1 - - 4 - 6 7 8 -` | 19 | 5 | [1, 4] | 31.801 | [1, 4] | 31.801 |
| `1 - - 4 - 6 7 8 -` | 19 | 6 | [6] | 31.801 | [6] | 31.801 |
| `1 - - 4 - 6 7 8 -` | 19 | 7 | [7] | 31.801 | [7] | 31.801 |
| `1 - - 4 - 6 7 8 -` | 19 | 8 | [8] | 31.801 | [8] | 31.801 |
| `1 - - 4 - 6 7 8 -` | 19 | 9 | [1, 8] | 31.801 | [1, 8] | 31.801 |
| `1 - - 4 - 6 7 8 -` | 19 | 10 | [4, 6] | 31.801 | [4, 6] | 31.801 |
| `1 - - 4 - 6 7 8 -` | 19 | 11 | [4, 7] | 31.801 | [4, 7] | 31.801 |
| `1 - - 4 - 6 7 8 -` | 19 | 12 | [4, 8] | 31.801 | [4, 8] | 31.801 |
| `1 - - 4 5 - 7 - 9` | 19 | 4 | [4] | 30.837 | [4] | 30.837 |
| `1 - - 4 5 - 7 - 9` | 19 | 5 | [5] | 30.837 | [5] | 30.837 |
| `1 - - 4 5 - 7 - 9` | 19 | 6 | [1, 5] | 30.837 | [1, 5] | 30.837 |
| `1 - - 4 5 - 7 - 9` | 19 | 7 | [7] | 30.837 | [7] | 30.837 |
| `1 - - 4 5 - 7 - 9` | 19 | 8 | [1, 7] | 30.837 | [1, 7] | 30.837 |
| `1 - - 4 5 - 7 - 9` | 19 | 9 | [9] | 30.837 | [9] | 30.837 |
| `1 - - 4 5 - 7 - 9` | 19 | 10 | [1, 9] | 30.837 | [1, 9] | 30.837 |
| `1 - - 4 5 - 7 - 9` | 19 | 11 | [4, 7] | 30.837 | [4, 7] | 30.837 |
| `1 - - 4 5 - 7 - 9` | 19 | 12 | [5, 7] | 30.837 | [5, 7] | 30.837 |
| `1 - 3 - - 6 7 - 9` | 19 | 3 | [3] | 30.256 | [3] | 30.256 |
| `1 - 3 - - 6 7 - 9` | 19 | 4 | [1, 3] | 30.256 | [1, 3] | 30.256 |
| `1 - 3 - - 6 7 - 9` | 19 | 6 | [6] | 30.256 | [6] | 30.256 |
| `1 - 3 - - 6 7 - 9` | 19 | 7 | [7] | 30.256 | [7] | 30.256 |
| `1 - 3 - - 6 7 - 9` | 19 | 8 | [1, 7] | 30.256 | [1, 7] | 30.256 |
| `1 - 3 - - 6 7 - 9` | 19 | 9 | [9] | 30.256 | [9] | 30.256 |
| `1 - 3 - - 6 7 - 9` | 19 | 10 | [1, 9] | 30.256 | [1, 9] | 30.256 |
| `1 - 3 - - 6 7 - 9` | 19 | 11 | [1, 3, 7] | 30.256 | [1, 3, 7] | 30.256 |
| `1 - 3 - - 6 7 - 9` | 19 | 12 | [3, 9] | 30.256 | [3, 9] | 30.256 |
| `1 - 3 - 5 - - 8 9` | 19 | 3 | [3] | 28.920 | [3] | 28.920 |
| `1 - 3 - 5 - - 8 9` | 19 | 4 | [1, 3] | 28.920 | [1, 3] | 28.920 |
| `1 - 3 - 5 - - 8 9` | 19 | 5 | [5] | 28.920 | [5] | 28.920 |
| `1 - 3 - 5 - - 8 9` | 19 | 6 | [1, 5] | 28.920 | [1, 5] | 28.920 |
| `1 - 3 - 5 - - 8 9` | 19 | 8 | [8] | 28.920 | [8] | 28.920 |
| `1 - 3 - 5 - - 8 9` | 19 | 9 | [9] | 28.920 | [9] | 28.920 |
| `1 - 3 - 5 - - 8 9` | 19 | 10 | [1, 9] | 28.920 | [1, 9] | 28.920 |
| `1 - 3 - 5 - - 8 9` | 19 | 11 | [3, 8] | 28.920 | [3, 8] | 28.920 |
| `1 - 3 - 5 - - 8 9` | 19 | 12 | [3, 9] | 28.920 | [3, 9] | 28.920 |
| `1 - 3 4 5 6 7 - -` | 19 | 3 | [3] | 37.126 | [3] | 36.957 |
| `1 - 3 4 5 6 7 - -` | 19 | 4 | [4] | 37.126 | [4] | 36.957 |
| `1 - 3 4 5 6 7 - -` | 19 | 5 | [5] | 37.126 | [5] | 36.957 |
| `1 - 3 4 5 6 7 - -` | 19 | 6 | [6] | 37.126 | [6] | 36.957 |
| `1 - 3 4 5 6 7 - -` | 19 | 7 | [7] | 37.126 | [7] | 36.957 |
| `1 - 3 4 5 6 7 - -` | 19 | 8 | [1, 7] | 37.126 | [1, 7] | 36.957 |
| `1 - 3 4 5 6 7 - -` | 19 | 9 | [3, 6] | 37.126 | [3, 6] | 36.957 |
| `1 - 3 4 5 6 7 - -` | 19 | 10 | [4, 6] | 37.126 | [3, 7] | 36.957 |
| `1 - 3 4 5 6 7 - -` | 19 | 11 | [4, 7] | 37.126 | [4, 7] | 36.957 |
| `1 - 3 4 5 6 7 - -` | 19 | 12 | [5, 7] | 37.126 | [5, 7] | 36.957 |
| `1 2 - - - 6 - 8 9` | 19 | 2 | [2] | 29.737 | [2] | 29.652 |
| `1 2 - - - 6 - 8 9` | 19 | 3 | [1, 2] | 29.737 | [1, 2] | 29.652 |
| `1 2 - - - 6 - 8 9` | 19 | 6 | [6] | 29.737 | [6] | 29.652 |
| `1 2 - - - 6 - 8 9` | 19 | 7 | [1, 6] | 29.737 | [1, 6] | 29.652 |
| `1 2 - - - 6 - 8 9` | 19 | 8 | [8] | 29.737 | [8] | 29.652 |
| `1 2 - - - 6 - 8 9` | 19 | 9 | [9] | 29.737 | [9] | 29.652 |
| `1 2 - - - 6 - 8 9` | 19 | 10 | [2, 8] | 29.737 | [1, 9] | 29.652 |
| `1 2 - - - 6 - 8 9` | 19 | 11 | [2, 9] | 29.737 | [2, 9] | 29.652 |
| `1 2 - - - 6 - 8 9` | 19 | 12 | [1, 2, 9] | 29.737 | [1, 2, 9] | 29.652 |
| `1 2 - 4 5 6 - 8 -` | 19 | 2 | [2] | 36.835 | [2] | 36.761 |
| `1 2 - 4 5 6 - 8 -` | 19 | 3 | [1, 2] | 36.835 | [1, 2] | 36.761 |
| `1 2 - 4 5 6 - 8 -` | 19 | 4 | [4] | 36.835 | [4] | 36.761 |
| `1 2 - 4 5 6 - 8 -` | 19 | 5 | [5] | 36.835 | [5] | 36.761 |
| `1 2 - 4 5 6 - 8 -` | 19 | 6 | [6] | 36.835 | [6] | 36.761 |
| `1 2 - 4 5 6 - 8 -` | 19 | 7 | [1, 6] | 36.835 | [1, 6] | 36.761 |
| `1 2 - 4 5 6 - 8 -` | 19 | 8 | [8] | 36.835 | [8] | 36.761 |
| `1 2 - 4 5 6 - 8 -` | 19 | 9 | [1, 8] | 36.835 | [1, 8] | 36.761 |
| `1 2 - 4 5 6 - 8 -` | 19 | 10 | [4, 6] | 36.835 | [2, 8] | 36.761 |
| `1 2 - 4 5 6 - 8 -` | 19 | 11 | [5, 6] | 36.835 | [5, 6] | 36.761 |
| `1 2 - 4 5 6 - 8 -` | 19 | 12 | [4, 8] | 36.835 | [4, 8] | 36.761 |
| `1 2 3 - 5 - 7 8 -` | 19 | 2 | [2] | 36.241 | [2] | 36.227 |
| `1 2 3 - 5 - 7 8 -` | 19 | 3 | [3] | 36.241 | [3] | 36.227 |
| `1 2 3 - 5 - 7 8 -` | 19 | 4 | [1, 3] | 36.241 | [1, 3] | 36.227 |
| `1 2 3 - 5 - 7 8 -` | 19 | 5 | [5] | 36.241 | [5] | 36.227 |
| `1 2 3 - 5 - 7 8 -` | 19 | 6 | [1, 5] | 36.241 | [1, 5] | 36.227 |
| `1 2 3 - 5 - 7 8 -` | 19 | 7 | [7] | 36.241 | [7] | 36.227 |
| `1 2 3 - 5 - 7 8 -` | 19 | 8 | [8] | 36.241 | [8] | 36.227 |
| `1 2 3 - 5 - 7 8 -` | 19 | 9 | [1, 8] | 36.241 | [1, 8] | 36.227 |
| `1 2 3 - 5 - 7 8 -` | 19 | 10 | [2, 8] | 36.241 | [2, 8] | 36.227 |
| `1 2 3 - 5 - 7 8 -` | 19 | 11 | [3, 8] | 36.241 | [3, 8] | 36.227 |
| `1 2 3 - 5 - 7 8 -` | 19 | 12 | [5, 7] | 36.241 | [5, 7] | 36.227 |
| `1 2 3 - 5 6 - - 9` | 19 | 2 | [2] | 35.815 | [2] | 35.633 |
| `1 2 3 - 5 6 - - 9` | 19 | 3 | [3] | 35.815 | [3] | 35.633 |
| `1 2 3 - 5 6 - - 9` | 19 | 4 | [1, 3] | 35.815 | [1, 3] | 35.633 |
| `1 2 3 - 5 6 - - 9` | 19 | 5 | [5] | 35.815 | [5] | 35.633 |
| `1 2 3 - 5 6 - - 9` | 19 | 6 | [6] | 35.815 | [6] | 35.633 |
| `1 2 3 - 5 6 - - 9` | 19 | 7 | [1, 6] | 35.815 | [1, 6] | 35.633 |
| `1 2 3 - 5 6 - - 9` | 19 | 8 | [3, 5] | 35.815 | [2, 6] | 35.633 |
| `1 2 3 - 5 6 - - 9` | 19 | 9 | [9] | 35.815 | [9] | 35.633 |
| `1 2 3 - 5 6 - - 9` | 19 | 10 | [1, 9] | 35.815 | [1, 9] | 35.633 |
| `1 2 3 - 5 6 - - 9` | 19 | 11 | [2, 9] | 35.815 | [2, 9] | 35.633 |
| `1 2 3 - 5 6 - - 9` | 19 | 12 | [3, 9] | 35.815 | [3, 9] | 35.633 |
| `1 2 3 4 - - 7 - 9` | 19 | 2 | [2] | 35.391 | [2] | 35.220 |
| `1 2 3 4 - - 7 - 9` | 19 | 3 | [3] | 35.391 | [3] | 35.220 |
| `1 2 3 4 - - 7 - 9` | 19 | 4 | [4] | 35.391 | [4] | 35.220 |
| `1 2 3 4 - - 7 - 9` | 19 | 5 | [2, 3] | 35.391 | [1, 4] | 35.220 |
| `1 2 3 4 - - 7 - 9` | 19 | 6 | [2, 4] | 35.391 | [2, 4] | 35.220 |
| `1 2 3 4 - - 7 - 9` | 19 | 7 | [7] | 35.391 | [7] | 35.220 |
| `1 2 3 4 - - 7 - 9` | 19 | 8 | [1, 7] | 35.391 | [1, 7] | 35.220 |
| `1 2 3 4 - - 7 - 9` | 19 | 9 | [9] | 35.391 | [9] | 35.220 |
| `1 2 3 4 - - 7 - 9` | 19 | 10 | [1, 9] | 35.391 | [1, 9] | 35.220 |
| `1 2 3 4 - - 7 - 9` | 19 | 11 | [2, 9] | 35.391 | [2, 9] | 35.220 |
| `1 2 3 4 - - 7 - 9` | 19 | 12 | [3, 9] | 35.391 | [3, 9] | 35.220 |
| `- - - 4 - 6 7 8 -` | 20 | 4 | [4] | 27.982 | [4] | 27.982 |
| `- - - 4 - 6 7 8 -` | 20 | 6 | [6] | 27.982 | [6] | 27.982 |
| `- - - 4 - 6 7 8 -` | 20 | 7 | [7] | 27.982 | [7] | 27.982 |
| `- - - 4 - 6 7 8 -` | 20 | 8 | [8] | 27.982 | [8] | 27.982 |
| `- - - 4 - 6 7 8 -` | 20 | 10 | [4, 6] | 27.982 | [4, 6] | 27.982 |
| `- - - 4 - 6 7 8 -` | 20 | 11 | [4, 7] | 27.982 | [4, 7] | 27.982 |
| `- - - 4 - 6 7 8 -` | 20 | 12 | [4, 8] | 27.982 | [4, 8] | 27.982 |
| `- - - 4 5 - 7 - 9` | 20 | 4 | [4] | 25.842 | [4] | 25.842 |
| `- - - 4 5 - 7 - 9` | 20 | 5 | [5] | 25.842 | [5] | 25.842 |
| `- - - 4 5 - 7 - 9` | 20 | 7 | [7] | 25.842 | [7] | 25.842 |
| `- - - 4 5 - 7 - 9` | 20 | 9 | [9] | 25.842 | [9] | 25.842 |
| `- - - 4 5 - 7 - 9` | 20 | 11 | [4, 7] | 25.842 | [4, 7] | 25.842 |
| `- - - 4 5 - 7 - 9` | 20 | 12 | [5, 7] | 25.842 | [5, 7] | 25.842 |
| `- - 3 - - 6 7 - 9` | 20 | 3 | [3] | 26.421 | [3] | 26.421 |
| `- - 3 - - 6 7 - 9` | 20 | 6 | [6] | 26.421 | [6] | 26.421 |
| `- - 3 - - 6 7 - 9` | 20 | 7 | [7] | 26.421 | [7] | 26.421 |
| `- - 3 - - 6 7 - 9` | 20 | 9 | [9] | 26.421 | [9] | 26.421 |
| `- - 3 - - 6 7 - 9` | 20 | 10 | [3, 7] | 26.421 | [3, 7] | 26.421 |
| `- - 3 - - 6 7 - 9` | 20 | 12 | [3, 9] | 26.421 | [3, 9] | 26.421 |
| `- - 3 - 5 - - 8 9` | 20 | 3 | [3] | 25.412 | [3] | 25.412 |
| `- - 3 - 5 - - 8 9` | 20 | 5 | [5] | 25.412 | [5] | 25.412 |
| `- - 3 - 5 - - 8 9` | 20 | 8 | [8] | 25.412 | [8] | 25.412 |
| `- - 3 - 5 - - 8 9` | 20 | 9 | [9] | 25.412 | [9] | 25.412 |
| `- - 3 - 5 - - 8 9` | 20 | 11 | [3, 8] | 25.412 | [3, 8] | 25.412 |
| `- - 3 - 5 - - 8 9` | 20 | 12 | [3, 9] | 25.412 | [3, 9] | 25.412 |
| `- - 3 4 5 6 7 - -` | 20 | 3 | [3] | 34.087 | [3] | 34.073 |
| `- - 3 4 5 6 7 - -` | 20 | 4 | [4] | 34.087 | [4] | 34.073 |
| `- - 3 4 5 6 7 - -` | 20 | 5 | [5] | 34.087 | [5] | 34.073 |
| `- - 3 4 5 6 7 - -` | 20 | 6 | [6] | 34.087 | [6] | 34.073 |
| `- - 3 4 5 6 7 - -` | 20 | 7 | [7] | 34.087 | [7] | 34.073 |
| `- - 3 4 5 6 7 - -` | 20 | 8 | [3, 5] | 34.087 | [3, 5] | 34.073 |
| `- - 3 4 5 6 7 - -` | 20 | 9 | [4, 5] | 34.087 | [3, 6] | 34.073 |
| `- - 3 4 5 6 7 - -` | 20 | 10 | [4, 6] | 34.087 | [3, 7] | 34.073 |
| `- - 3 4 5 6 7 - -` | 20 | 11 | [4, 7] | 34.087 | [4, 7] | 34.073 |
| `- - 3 4 5 6 7 - -` | 20 | 12 | [5, 7] | 34.087 | [5, 7] | 34.073 |
| `- 2 - - - 6 - 8 9` | 20 | 2 | [2] | 26.432 | [2] | 26.432 |
| `- 2 - - - 6 - 8 9` | 20 | 6 | [6] | 26.432 | [6] | 26.432 |
| `- 2 - - - 6 - 8 9` | 20 | 8 | [8] | 26.432 | [8] | 26.432 |
| `- 2 - - - 6 - 8 9` | 20 | 9 | [9] | 26.432 | [9] | 26.432 |
| `- 2 - - - 6 - 8 9` | 20 | 10 | [2, 8] | 26.432 | [2, 8] | 26.432 |
| `- 2 - - - 6 - 8 9` | 20 | 11 | [2, 9] | 26.432 | [2, 9] | 26.432 |
| `- 2 - 4 5 6 - 8 -` | 20 | 2 | [2] | 32.845 | [2] | 32.845 |
| `- 2 - 4 5 6 - 8 -` | 20 | 4 | [4] | 32.845 | [4] | 32.845 |
| `- 2 - 4 5 6 - 8 -` | 20 | 5 | [5] | 32.845 | [5] | 32.845 |
| `- 2 - 4 5 6 - 8 -` | 20 | 6 | [6] | 32.845 | [6] | 32.845 |
| `- 2 - 4 5 6 - 8 -` | 20 | 7 | [2, 5] | 32.845 | [2, 5] | 32.845 |
| `- 2 - 4 5 6 - 8 -` | 20 | 8 | [8] | 32.845 | [8] | 32.845 |
| `- 2 - 4 5 6 - 8 -` | 20 | 9 | [4, 5] | 32.845 | [4, 5] | 32.845 |
| `- 2 - 4 5 6 - 8 -` | 20 | 10 | [2, 8] | 32.845 | [2, 8] | 32.845 |
| `- 2 - 4 5 6 - 8 -` | 20 | 11 | [5, 6] | 32.845 | [5, 6] | 32.845 |
| `- 2 - 4 5 6 - 8 -` | 20 | 12 | [4, 8] | 32.845 | [4, 8] | 32.845 |
| `- 2 3 - 5 - 7 8 -` | 20 | 2 | [2] | 30.559 | [2] | 30.559 |
| `- 2 3 - 5 - 7 8 -` | 20 | 3 | [3] | 30.559 | [3] | 30.559 |
| `- 2 3 - 5 - 7 8 -` | 20 | 5 | [5] | 30.559 | [5] | 30.559 |
| `- 2 3 - 5 - 7 8 -` | 20 | 7 | [7] | 30.559 | [7] | 30.559 |
| `- 2 3 - 5 - 7 8 -` | 20 | 8 | [8] | 30.559 | [8] | 30.559 |
| `- 2 3 - 5 - 7 8 -` | 20 | 9 | [2, 7] | 30.559 | [2, 7] | 30.559 |
| `- 2 3 - 5 - 7 8 -` | 20 | 10 | [2, 8] | 30.559 | [2, 8] | 30.559 |
| `- 2 3 - 5 - 7 8 -` | 20 | 11 | [3, 8] | 30.559 | [3, 8] | 30.559 |
| `- 2 3 - 5 - 7 8 -` | 20 | 12 | [5, 7] | 30.559 | [5, 7] | 30.559 |
| `- 2 3 - 5 6 - - 9` | 20 | 2 | [2] | 31.740 | [2] | 31.667 |
| `- 2 3 - 5 6 - - 9` | 20 | 3 | [3] | 31.740 | [3] | 31.667 |
| `- 2 3 - 5 6 - - 9` | 20 | 5 | [5] | 31.740 | [5] | 31.667 |
| `- 2 3 - 5 6 - - 9` | 20 | 6 | [6] | 31.740 | [6] | 31.667 |
| `- 2 3 - 5 6 - - 9` | 20 | 7 | [2, 5] | 31.740 | [2, 5] | 31.667 |
| `- 2 3 - 5 6 - - 9` | 20 | 8 | [3, 5] | 31.740 | [2, 6] | 31.667 |
| `- 2 3 - 5 6 - - 9` | 20 | 9 | [9] | 31.740 | [9] | 31.667 |
| `- 2 3 - 5 6 - - 9` | 20 | 10 | [2, 3, 5] | 31.740 | [2, 3, 5] | 31.667 |
| `- 2 3 - 5 6 - - 9` | 20 | 11 | [2, 9] | 31.740 | [2, 9] | 31.667 |
| `- 2 3 - 5 6 - - 9` | 20 | 12 | [3, 9] | 31.740 | [3, 9] | 31.667 |
| `- 2 3 4 - - 7 - 9` | 20 | 2 | [2] | 30.648 | [2] | 30.648 |
| `- 2 3 4 - - 7 - 9` | 20 | 3 | [3] | 30.648 | [3] | 30.648 |
| `- 2 3 4 - - 7 - 9` | 20 | 4 | [4] | 30.648 | [4] | 30.648 |
| `- 2 3 4 - - 7 - 9` | 20 | 5 | [2, 3] | 30.648 | [2, 3] | 30.648 |
| `- 2 3 4 - - 7 - 9` | 20 | 6 | [2, 4] | 30.648 | [2, 4] | 30.648 |
| `- 2 3 4 - - 7 - 9` | 20 | 7 | [7] | 30.648 | [7] | 30.648 |
| `- 2 3 4 - - 7 - 9` | 20 | 9 | [9] | 30.648 | [9] | 30.648 |
| `- 2 3 4 - - 7 - 9` | 20 | 10 | [3, 7] | 30.648 | [3, 7] | 30.648 |
| `- 2 3 4 - - 7 - 9` | 20 | 11 | [2, 9] | 30.648 | [2, 9] | 30.648 |
| `- 2 3 4 - - 7 - 9` | 20 | 12 | [3, 9] | 30.648 | [3, 9] | 30.648 |
| `1 - - - - - 7 8 9` | 20 | 7 | [7] | 26.052 | [7] | 26.052 |
| `1 - - - - - 7 8 9` | 20 | 8 | [8] | 26.052 | [8] | 26.052 |
| `1 - - - - - 7 8 9` | 20 | 9 | [9] | 26.052 | [9] | 26.052 |
| `1 - - - - - 7 8 9` | 20 | 10 | [1, 9] | 26.052 | [1, 9] | 26.052 |
| `1 - - 4 5 - 7 8 -` | 20 | 4 | [4] | 32.438 | [4] | 32.435 |
| `1 - - 4 5 - 7 8 -` | 20 | 5 | [5] | 32.438 | [5] | 32.435 |
| `1 - - 4 5 - 7 8 -` | 20 | 6 | [1, 5] | 32.438 | [1, 5] | 32.435 |
| `1 - - 4 5 - 7 8 -` | 20 | 7 | [7] | 32.438 | [7] | 32.435 |
| `1 - - 4 5 - 7 8 -` | 20 | 8 | [8] | 32.438 | [8] | 32.435 |
| `1 - - 4 5 - 7 8 -` | 20 | 9 | [1, 8] | 32.438 | [1, 8] | 32.435 |
| `1 - - 4 5 - 7 8 -` | 20 | 10 | [1, 4, 5] | 32.438 | [1, 4, 5] | 32.435 |
| `1 - - 4 5 - 7 8 -` | 20 | 11 | [4, 7] | 32.438 | [4, 7] | 32.435 |
| `1 - - 4 5 - 7 8 -` | 20 | 12 | [4, 8] | 32.438 | [4, 8] | 32.435 |
| `1 - - 4 5 6 - - 9` | 20 | 4 | [4] | 29.697 | [4] | 29.697 |
| `1 - - 4 5 6 - - 9` | 20 | 5 | [5] | 29.697 | [5] | 29.697 |
| `1 - - 4 5 6 - - 9` | 20 | 6 | [6] | 29.697 | [6] | 29.697 |
| `1 - - 4 5 6 - - 9` | 20 | 7 | [1, 6] | 29.697 | [1, 6] | 29.697 |
| `1 - - 4 5 6 - - 9` | 20 | 9 | [9] | 29.697 | [9] | 29.697 |
| `1 - - 4 5 6 - - 9` | 20 | 10 | [1, 9] | 29.697 | [1, 9] | 29.697 |
| `1 - - 4 5 6 - - 9` | 20 | 11 | [5, 6] | 29.697 | [5, 6] | 29.697 |
| `1 - - 4 5 6 - - 9` | 20 | 12 | [1, 5, 6] | 29.697 | [1, 5, 6] | 29.697 |
| `1 - 3 - - 6 7 8 -` | 20 | 3 | [3] | 32.090 | [3] | 32.089 |
| `1 - 3 - - 6 7 8 -` | 20 | 4 | [1, 3] | 32.090 | [1, 3] | 32.089 |
| `1 - 3 - - 6 7 8 -` | 20 | 6 | [6] | 32.090 | [6] | 32.089 |
| `1 - 3 - - 6 7 8 -` | 20 | 7 | [7] | 32.090 | [7] | 32.089 |
| `1 - 3 - - 6 7 8 -` | 20 | 8 | [8] | 32.090 | [8] | 32.089 |
| `1 - 3 - - 6 7 8 -` | 20 | 9 | [1, 8] | 32.090 | [1, 8] | 32.089 |
| `1 - 3 - - 6 7 8 -` | 20 | 10 | [3, 7] | 32.090 | [3, 7] | 32.089 |
| `1 - 3 - - 6 7 8 -` | 20 | 11 | [3, 8] | 32.090 | [3, 8] | 32.089 |
| `1 - 3 - - 6 7 8 -` | 20 | 12 | [1, 3, 8] | 32.090 | [1, 3, 8] | 32.089 |
| `1 - 3 - 5 - 7 - 9` | 20 | 3 | [3] | 32.718 | [3] | 32.558 |
| `1 - 3 - 5 - 7 - 9` | 20 | 4 | [1, 3] | 32.718 | [1, 3] | 32.558 |
| `1 - 3 - 5 - 7 - 9` | 20 | 5 | [5] | 32.718 | [5] | 32.558 |
| `1 - 3 - 5 - 7 - 9` | 20 | 6 | [1, 5] | 32.718 | [1, 5] | 32.558 |
| `1 - 3 - 5 - 7 - 9` | 20 | 7 | [7] | 32.718 | [7] | 32.558 |
| `1 - 3 - 5 - 7 - 9` | 20 | 8 | [3, 5] | 32.718 | [1, 7] | 32.558 |
| `1 - 3 - 5 - 7 - 9` | 20 | 9 | [9] | 32.718 | [9] | 32.558 |
| `1 - 3 - 5 - 7 - 9` | 20 | 10 | [1, 9] | 32.718 | [1, 9] | 32.558 |
| `1 - 3 - 5 - 7 - 9` | 20 | 11 | [1, 3, 7] | 32.718 | [1, 3, 7] | 32.558 |
| `1 - 3 - 5 - 7 - 9` | 20 | 12 | [3, 9] | 32.718 | [3, 9] | 32.558 |
| `1 - 3 4 - - - 8 9` | 20 | 3 | [3] | 30.369 | [3] | 30.369 |
| `1 - 3 4 - - - 8 9` | 20 | 4 | [4] | 30.369 | [4] | 30.369 |
| `1 - 3 4 - - - 8 9` | 20 | 5 | [1, 4] | 30.369 | [1, 4] | 30.369 |
| `1 - 3 4 - - - 8 9` | 20 | 7 | [3, 4] | 30.369 | [3, 4] | 30.369 |
| `1 - 3 4 - - - 8 9` | 20 | 8 | [8] | 30.369 | [8] | 30.369 |
| `1 - 3 4 - - - 8 9` | 20 | 9 | [9] | 30.369 | [9] | 30.369 |
| `1 - 3 4 - - - 8 9` | 20 | 10 | [1, 9] | 30.369 | [1, 9] | 30.369 |
| `1 - 3 4 - - - 8 9` | 20 | 11 | [3, 8] | 30.369 | [3, 8] | 30.369 |
| `1 - 3 4 - - - 8 9` | 20 | 12 | [3, 9] | 30.369 | [3, 9] | 30.369 |
| `1 2 - - - 6 7 - 9` | 20 | 2 | [2] | 31.090 | [2] | 31.004 |
| `1 2 - - - 6 7 - 9` | 20 | 3 | [1, 2] | 31.090 | [1, 2] | 31.004 |
| `1 2 - - - 6 7 - 9` | 20 | 6 | [6] | 31.090 | [6] | 31.004 |
| `1 2 - - - 6 7 - 9` | 20 | 7 | [7] | 31.090 | [7] | 31.004 |
| `1 2 - - - 6 7 - 9` | 20 | 8 | [2, 6] | 31.090 | [1, 7] | 31.004 |
| `1 2 - - - 6 7 - 9` | 20 | 9 | [9] | 31.090 | [9] | 31.004 |
| `1 2 - - - 6 7 - 9` | 20 | 10 | [1, 9] | 31.090 | [1, 9] | 31.004 |
| `1 2 - - - 6 7 - 9` | 20 | 11 | [2, 9] | 31.090 | [2, 9] | 31.004 |
| `1 2 - - - 6 7 - 9` | 20 | 12 | [1, 2, 9] | 31.090 | [1, 2, 9] | 31.004 |
| `1 2 - - 5 - - 8 9` | 20 | 2 | [2] | 31.619 | [2] | 31.619 |
| `1 2 - - 5 - - 8 9` | 20 | 3 | [1, 2] | 31.619 | [1, 2] | 31.619 |
| `1 2 - - 5 - - 8 9` | 20 | 5 | [5] | 31.619 | [5] | 31.619 |
| `1 2 - - 5 - - 8 9` | 20 | 6 | [1, 5] | 31.619 | [1, 5] | 31.619 |
| `1 2 - - 5 - - 8 9` | 20 | 7 | [2, 5] | 31.619 | [2, 5] | 31.619 |
| `1 2 - - 5 - - 8 9` | 20 | 8 | [8] | 31.619 | [8] | 31.619 |
| `1 2 - - 5 - - 8 9` | 20 | 9 | [9] | 31.619 | [9] | 31.619 |
| `1 2 - - 5 - - 8 9` | 20 | 10 | [1, 9] | 31.619 | [1, 9] | 31.619 |
| `1 2 - - 5 - - 8 9` | 20 | 11 | [2, 9] | 31.619 | [2, 9] | 31.619 |
| `1 2 - - 5 - - 8 9` | 20 | 12 | [1, 2, 9] | 31.619 | [1, 2, 9] | 31.619 |
| `1 2 - 4 5 6 7 - -` | 20 | 2 | [2] | 37.863 | [2] | 37.757 |
| `1 2 - 4 5 6 7 - -` | 20 | 3 | [1, 2] | 37.863 | [1, 2] | 37.757 |
| `1 2 - 4 5 6 7 - -` | 20 | 4 | [4] | 37.863 | [4] | 37.757 |
| `1 2 - 4 5 6 7 - -` | 20 | 5 | [5] | 37.863 | [5] | 37.757 |
| `1 2 - 4 5 6 7 - -` | 20 | 6 | [6] | 37.863 | [6] | 37.757 |
| `1 2 - 4 5 6 7 - -` | 20 | 7 | [7] | 37.863 | [7] | 37.757 |
| `1 2 - 4 5 6 7 - -` | 20 | 8 | [1, 7] | 37.863 | [1, 7] | 37.757 |
| `1 2 - 4 5 6 7 - -` | 20 | 9 | [2, 7] | 37.863 | [2, 7] | 37.757 |
| `1 2 - 4 5 6 7 - -` | 20 | 10 | [4, 6] | 37.863 | [4, 6] | 37.757 |
| `1 2 - 4 5 6 7 - -` | 20 | 11 | [5, 6] | 37.863 | [4, 7] | 37.757 |
| `1 2 - 4 5 6 7 - -` | 20 | 12 | [5, 7] | 37.863 | [5, 7] | 37.757 |
| `1 2 3 - 5 6 - 8 -` | 20 | 2 | [2] | 37.228 | [2] | 36.801 |
| `1 2 3 - 5 6 - 8 -` | 20 | 3 | [3] | 37.228 | [3] | 36.801 |
| `1 2 3 - 5 6 - 8 -` | 20 | 4 | [1, 3] | 37.228 | [1, 3] | 36.801 |
| `1 2 3 - 5 6 - 8 -` | 20 | 5 | [5] | 37.228 | [5] | 36.801 |
| `1 2 3 - 5 6 - 8 -` | 20 | 6 | [6] | 37.228 | [6] | 36.801 |
| `1 2 3 - 5 6 - 8 -` | 20 | 7 | [2, 5] | 37.228 | [1, 6] | 36.801 |
| `1 2 3 - 5 6 - 8 -` | 20 | 8 | [8] | 37.228 | [8] | 36.801 |
| `1 2 3 - 5 6 - 8 -` | 20 | 9 | [1, 8] | 37.228 | [1, 8] | 36.801 |
| `1 2 3 - 5 6 - 8 -` | 20 | 10 | [2, 8] | 37.228 | [2, 8] | 36.801 |
| `1 2 3 - 5 6 - 8 -` | 20 | 11 | [3, 8] | 37.228 | [3, 8] | 36.801 |
| `1 2 3 - 5 6 - 8 -` | 20 | 12 | [1, 3, 8] | 37.228 | [1, 3, 8] | 36.801 |
| `1 2 3 4 - - 7 8 -` | 20 | 2 | [2] | 37.140 | [2] | 36.971 |
| `1 2 3 4 - - 7 8 -` | 20 | 3 | [3] | 37.140 | [3] | 36.971 |
| `1 2 3 4 - - 7 8 -` | 20 | 4 | [4] | 37.140 | [4] | 36.971 |
| `1 2 3 4 - - 7 8 -` | 20 | 5 | [1, 4] | 37.140 | [1, 4] | 36.971 |
| `1 2 3 4 - - 7 8 -` | 20 | 6 | [2, 4] | 37.140 | [2, 4] | 36.971 |
| `1 2 3 4 - - 7 8 -` | 20 | 7 | [7] | 37.140 | [7] | 36.971 |
| `1 2 3 4 - - 7 8 -` | 20 | 8 | [8] | 37.140 | [8] | 36.971 |
| `1 2 3 4 - - 7 8 -` | 20 | 9 | [1, 8] | 37.140 | [1, 8] | 36.971 |
| `1 2 3 4 - - 7 8 -` | 20 | 10 | [3, 7] | 37.140 | [2, 8] | 36.971 |
| `1 2 3 4 - - 7 8 -` | 20 | 11 | [3, 8] | 37.140 | [3, 8] | 36.971 |
| `1 2 3 4 - - 7 8 -` | 20 | 12 | [4, 8] | 37.140 | [4, 8] | 36.971 |
| `1 2 3 4 - 6 - - 9` | 20 | 2 | [2] | 36.481 | [2] | 36.169 |
| `1 2 3 4 - 6 - - 9` | 20 | 3 | [3] | 36.481 | [3] | 36.169 |
| `1 2 3 4 - 6 - - 9` | 20 | 4 | [4] | 36.481 | [4] | 36.169 |
| `1 2 3 4 - 6 - - 9` | 20 | 5 | [2, 3] | 36.481 | [1, 4] | 36.169 |
| `1 2 3 4 - 6 - - 9` | 20 | 6 | [6] | 36.481 | [6] | 36.169 |
| `1 2 3 4 - 6 - - 9` | 20 | 7 | [3, 4] | 36.481 | [1, 6] | 36.169 |
| `1 2 3 4 - 6 - - 9` | 20 | 8 | [2, 6] | 36.481 | [2, 6] | 36.169 |
| `1 2 3 4 - 6 - - 9` | 20 | 9 | [9] | 36.481 | [9] | 36.169 |
| `1 2 3 4 - 6 - - 9` | 20 | 10 | [1, 9] | 36.481 | [1, 9] | 36.169 |
| `1 2 3 4 - 6 - - 9` | 20 | 11 | [2, 9] | 36.481 | [2, 9] | 36.169 |
| `1 2 3 4 - 6 - - 9` | 20 | 12 | [3, 9] | 36.481 | [3, 9] | 36.169 |
| `- - - - - - 7 8 9` | 21 | 7 | [7] | 25.307 | [7] | 25.307 |
| `- - - - - - 7 8 9` | 21 | 8 | [8] | 25.307 | [8] | 25.307 |
| `- - - - - - 7 8 9` | 21 | 9 | [9] | 25.307 | [9] | 25.307 |
| `- - - 4 5 - 7 8 -` | 21 | 4 | [4] | 28.785 | [4] | 28.785 |
| `- - - 4 5 - 7 8 -` | 21 | 5 | [5] | 28.785 | [5] | 28.785 |
| `- - - 4 5 - 7 8 -` | 21 | 7 | [7] | 28.785 | [7] | 28.785 |
| `- - - 4 5 - 7 8 -` | 21 | 8 | [8] | 28.785 | [8] | 28.785 |
| `- - - 4 5 - 7 8 -` | 21 | 9 | [4, 5] | 28.785 | [4, 5] | 28.785 |
| `- - - 4 5 - 7 8 -` | 21 | 11 | [4, 7] | 28.785 | [4, 7] | 28.785 |
| `- - - 4 5 - 7 8 -` | 21 | 12 | [4, 8] | 28.785 | [4, 8] | 28.785 |
| `- - - 4 5 6 - - 9` | 21 | 4 | [4] | 27.008 | [4] | 27.008 |
| `- - - 4 5 6 - - 9` | 21 | 5 | [5] | 27.008 | [5] | 27.008 |
| `- - - 4 5 6 - - 9` | 21 | 6 | [6] | 27.008 | [6] | 27.008 |
| `- - - 4 5 6 - - 9` | 21 | 9 | [9] | 27.008 | [9] | 27.008 |
| `- - - 4 5 6 - - 9` | 21 | 10 | [4, 6] | 27.008 | [4, 6] | 27.008 |
| `- - - 4 5 6 - - 9` | 21 | 11 | [5, 6] | 27.008 | [5, 6] | 27.008 |
| `- - 3 - - 6 7 8 -` | 21 | 3 | [3] | 29.684 | [3] | 29.684 |
| `- - 3 - - 6 7 8 -` | 21 | 6 | [6] | 29.684 | [6] | 29.684 |
| `- - 3 - - 6 7 8 -` | 21 | 7 | [7] | 29.684 | [7] | 29.684 |
| `- - 3 - - 6 7 8 -` | 21 | 8 | [8] | 29.684 | [8] | 29.684 |
| `- - 3 - - 6 7 8 -` | 21 | 9 | [3, 6] | 29.684 | [3, 6] | 29.684 |
| `- - 3 - - 6 7 8 -` | 21 | 10 | [3, 7] | 29.684 | [3, 7] | 29.684 |
| `- - 3 - - 6 7 8 -` | 21 | 11 | [3, 8] | 29.684 | [3, 8] | 29.684 |
| `- - 3 - 5 - 7 - 9` | 21 | 3 | [3] | 28.604 | [3] | 28.604 |
| `- - 3 - 5 - 7 - 9` | 21 | 5 | [5] | 28.604 | [5] | 28.604 |
| `- - 3 - 5 - 7 - 9` | 21 | 7 | [7] | 28.604 | [7] | 28.604 |
| `- - 3 - 5 - 7 - 9` | 21 | 8 | [3, 5] | 28.604 | [3, 5] | 28.604 |
| `- - 3 - 5 - 7 - 9` | 21 | 9 | [9] | 28.604 | [9] | 28.604 |
| `- - 3 - 5 - 7 - 9` | 21 | 10 | [3, 7] | 28.604 | [3, 7] | 28.604 |
| `- - 3 - 5 - 7 - 9` | 21 | 12 | [3, 9] | 28.604 | [3, 9] | 28.604 |
| `- - 3 4 - - - 8 9` | 21 | 3 | [3] | 27.755 | [3] | 27.755 |
| `- - 3 4 - - - 8 9` | 21 | 4 | [4] | 27.755 | [4] | 27.755 |
| `- - 3 4 - - - 8 9` | 21 | 7 | [3, 4] | 27.755 | [3, 4] | 27.755 |
| `- - 3 4 - - - 8 9` | 21 | 8 | [8] | 27.755 | [8] | 27.755 |
| `- - 3 4 - - - 8 9` | 21 | 9 | [9] | 27.755 | [9] | 27.755 |
| `- - 3 4 - - - 8 9` | 21 | 11 | [3, 8] | 27.755 | [3, 8] | 27.755 |
| `- - 3 4 - - - 8 9` | 21 | 12 | [3, 9] | 27.755 | [3, 9] | 27.755 |
| `- 2 - - - 6 7 - 9` | 21 | 2 | [2] | 28.134 | [2] | 28.134 |
| `- 2 - - - 6 7 - 9` | 21 | 6 | [6] | 28.134 | [6] | 28.134 |
| `- 2 - - - 6 7 - 9` | 21 | 7 | [7] | 28.134 | [7] | 28.134 |
| `- 2 - - - 6 7 - 9` | 21 | 8 | [2, 6] | 28.134 | [2, 6] | 28.134 |
| `- 2 - - - 6 7 - 9` | 21 | 9 | [9] | 28.134 | [9] | 28.134 |
| `- 2 - - - 6 7 - 9` | 21 | 11 | [2, 9] | 28.134 | [2, 9] | 28.134 |
| `- 2 - - 5 - - 8 9` | 21 | 2 | [2] | 28.603 | [2] | 28.603 |
| `- 2 - - 5 - - 8 9` | 21 | 5 | [5] | 28.603 | [5] | 28.603 |
| `- 2 - - 5 - - 8 9` | 21 | 7 | [2, 5] | 28.603 | [2, 5] | 28.603 |
| `- 2 - - 5 - - 8 9` | 21 | 8 | [8] | 28.603 | [8] | 28.603 |
| `- 2 - - 5 - - 8 9` | 21 | 9 | [9] | 28.603 | [9] | 28.603 |
| `- 2 - - 5 - - 8 9` | 21 | 10 | [2, 8] | 28.603 | [2, 8] | 28.603 |
| `- 2 - - 5 - - 8 9` | 21 | 11 | [2, 9] | 28.603 | [2, 9] | 28.603 |
| `- 2 - 4 5 6 7 - -` | 21 | 2 | [2] | 34.382 | [2] | 34.363 |
| `- 2 - 4 5 6 7 - -` | 21 | 4 | [4] | 34.382 | [4] | 34.363 |
| `- 2 - 4 5 6 7 - -` | 21 | 5 | [5] | 34.382 | [5] | 34.363 |
| `- 2 - 4 5 6 7 - -` | 21 | 6 | [6] | 34.382 | [6] | 34.363 |
| `- 2 - 4 5 6 7 - -` | 21 | 7 | [7] | 34.382 | [7] | 34.363 |
| `- 2 - 4 5 6 7 - -` | 21 | 8 | [2, 6] | 34.382 | [2, 6] | 34.363 |
| `- 2 - 4 5 6 7 - -` | 21 | 9 | [4, 5] | 34.382 | [2, 7] | 34.363 |
| `- 2 - 4 5 6 7 - -` | 21 | 10 | [4, 6] | 34.382 | [4, 6] | 34.363 |
| `- 2 - 4 5 6 7 - -` | 21 | 11 | [4, 7] | 34.382 | [4, 7] | 34.363 |
| `- 2 - 4 5 6 7 - -` | 21 | 12 | [5, 7] | 34.382 | [5, 7] | 34.363 |
| `- 2 3 - 5 6 - 8 -` | 21 | 2 | [2] | 32.844 | [2] | 32.841 |
| `- 2 3 - 5 6 - 8 -` | 21 | 3 | [3] | 32.844 | [3] | 32.841 |
| `- 2 3 - 5 6 - 8 -` | 21 | 5 | [5] | 32.844 | [5] | 32.841 |
| `- 2 3 - 5 6 - 8 -` | 21 | 6 | [6] | 32.844 | [6] | 32.841 |
| `- 2 3 - 5 6 - 8 -` | 21 | 7 | [2, 5] | 32.844 | [2, 5] | 32.841 |
| `- 2 3 - 5 6 - 8 -` | 21 | 8 | [8] | 32.844 | [8] | 32.841 |
| `- 2 3 - 5 6 - 8 -` | 21 | 9 | [3, 6] | 32.844 | [3, 6] | 32.841 |
| `- 2 3 - 5 6 - 8 -` | 21 | 10 | [2, 8] | 32.844 | [2, 8] | 32.841 |
| `- 2 3 - 5 6 - 8 -` | 21 | 11 | [3, 8] | 32.844 | [3, 8] | 32.841 |
| `- 2 3 4 - - 7 8 -` | 21 | 2 | [2] | 34.507 | [2] | 34.479 |
| `- 2 3 4 - - 7 8 -` | 21 | 3 | [3] | 34.507 | [3] | 34.479 |
| `- 2 3 4 - - 7 8 -` | 21 | 4 | [4] | 34.507 | [4] | 34.479 |
| `- 2 3 4 - - 7 8 -` | 21 | 5 | [2, 3] | 34.507 | [2, 3] | 34.479 |
| `- 2 3 4 - - 7 8 -` | 21 | 6 | [2, 4] | 34.507 | [2, 4] | 34.479 |
| `- 2 3 4 - - 7 8 -` | 21 | 7 | [7] | 34.507 | [7] | 34.479 |
| `- 2 3 4 - - 7 8 -` | 21 | 8 | [8] | 34.507 | [8] | 34.479 |
| `- 2 3 4 - - 7 8 -` | 21 | 9 | [2, 7] | 34.507 | [2, 7] | 34.479 |
| `- 2 3 4 - - 7 8 -` | 21 | 10 | [3, 7] | 34.507 | [2, 8] | 34.479 |
| `- 2 3 4 - - 7 8 -` | 21 | 11 | [3, 8] | 34.507 | [3, 8] | 34.479 |
| `- 2 3 4 - - 7 8 -` | 21 | 12 | [4, 8] | 34.507 | [4, 8] | 34.479 |
| `- 2 3 4 - 6 - - 9` | 21 | 2 | [2] | 33.390 | [2] | 33.390 |
| `- 2 3 4 - 6 - - 9` | 21 | 3 | [3] | 33.390 | [3] | 33.390 |
| `- 2 3 4 - 6 - - 9` | 21 | 4 | [4] | 33.390 | [4] | 33.390 |
| `- 2 3 4 - 6 - - 9` | 21 | 5 | [2, 3] | 33.390 | [2, 3] | 33.390 |
| `- 2 3 4 - 6 - - 9` | 21 | 6 | [6] | 33.390 | [6] | 33.390 |
| `- 2 3 4 - 6 - - 9` | 21 | 7 | [3, 4] | 33.390 | [3, 4] | 33.390 |
| `- 2 3 4 - 6 - - 9` | 21 | 8 | [2, 6] | 33.390 | [2, 6] | 33.390 |
| `- 2 3 4 - 6 - - 9` | 21 | 9 | [9] | 33.390 | [9] | 33.390 |
| `- 2 3 4 - 6 - - 9` | 21 | 10 | [4, 6] | 33.390 | [4, 6] | 33.390 |
| `- 2 3 4 - 6 - - 9` | 21 | 11 | [2, 9] | 33.390 | [2, 9] | 33.390 |
| `- 2 3 4 - 6 - - 9` | 21 | 12 | [3, 9] | 33.390 | [3, 9] | 33.390 |
| `1 - - - - 6 - 8 9` | 21 | 6 | [6] | 28.197 | [6] | 28.197 |
| `1 - - - - 6 - 8 9` | 21 | 7 | [1, 6] | 28.197 | [1, 6] | 28.197 |
| `1 - - - - 6 - 8 9` | 21 | 8 | [8] | 28.197 | [8] | 28.197 |
| `1 - - - - 6 - 8 9` | 21 | 9 | [9] | 28.197 | [9] | 28.197 |
| `1 - - - - 6 - 8 9` | 21 | 10 | [1, 9] | 28.197 | [1, 9] | 28.197 |
| `1 - - 4 5 6 - 8 -` | 21 | 4 | [4] | 33.451 | [4] | 33.449 |
| `1 - - 4 5 6 - 8 -` | 21 | 5 | [5] | 33.451 | [5] | 33.449 |
| `1 - - 4 5 6 - 8 -` | 21 | 6 | [6] | 33.451 | [6] | 33.449 |
| `1 - - 4 5 6 - 8 -` | 21 | 7 | [1, 6] | 33.451 | [1, 6] | 33.449 |
| `1 - - 4 5 6 - 8 -` | 21 | 8 | [8] | 33.451 | [8] | 33.449 |
| `1 - - 4 5 6 - 8 -` | 21 | 9 | [1, 8] | 33.451 | [1, 8] | 33.449 |
| `1 - - 4 5 6 - 8 -` | 21 | 10 | [4, 6] | 33.451 | [4, 6] | 33.449 |
| `1 - - 4 5 6 - 8 -` | 21 | 11 | [5, 6] | 33.451 | [5, 6] | 33.449 |
| `1 - - 4 5 6 - 8 -` | 21 | 12 | [4, 8] | 33.451 | [4, 8] | 33.449 |
| `1 - 3 - 5 - 7 8 -` | 21 | 3 | [3] | 34.002 | [3] | 33.994 |
| `1 - 3 - 5 - 7 8 -` | 21 | 4 | [1, 3] | 34.002 | [1, 3] | 33.994 |
| `1 - 3 - 5 - 7 8 -` | 21 | 5 | [5] | 34.002 | [5] | 33.994 |
| `1 - 3 - 5 - 7 8 -` | 21 | 6 | [1, 5] | 34.002 | [1, 5] | 33.994 |
| `1 - 3 - 5 - 7 8 -` | 21 | 7 | [7] | 34.002 | [7] | 33.994 |
| `1 - 3 - 5 - 7 8 -` | 21 | 8 | [8] | 34.002 | [8] | 33.994 |
| `1 - 3 - 5 - 7 8 -` | 21 | 9 | [1, 8] | 34.002 | [1, 8] | 33.994 |
| `1 - 3 - 5 - 7 8 -` | 21 | 10 | [3, 7] | 34.002 | [3, 7] | 33.994 |
| `1 - 3 - 5 - 7 8 -` | 21 | 11 | [3, 8] | 34.002 | [3, 8] | 33.994 |
| `1 - 3 - 5 - 7 8 -` | 21 | 12 | [5, 7] | 34.002 | [5, 7] | 33.994 |
| `1 - 3 - 5 6 - - 9` | 21 | 3 | [3] | 33.353 | [3] | 33.353 |
| `1 - 3 - 5 6 - - 9` | 21 | 4 | [1, 3] | 33.353 | [1, 3] | 33.353 |
| `1 - 3 - 5 6 - - 9` | 21 | 5 | [5] | 33.353 | [5] | 33.353 |
| `1 - 3 - 5 6 - - 9` | 21 | 6 | [6] | 33.353 | [6] | 33.353 |
| `1 - 3 - 5 6 - - 9` | 21 | 7 | [1, 6] | 33.353 | [1, 6] | 33.353 |
| `1 - 3 - 5 6 - - 9` | 21 | 8 | [3, 5] | 33.353 | [3, 5] | 33.353 |
| `1 - 3 - 5 6 - - 9` | 21 | 9 | [9] | 33.353 | [9] | 33.353 |
| `1 - 3 - 5 6 - - 9` | 21 | 10 | [1, 9] | 33.353 | [1, 9] | 33.353 |
| `1 - 3 - 5 6 - - 9` | 21 | 11 | [5, 6] | 33.353 | [5, 6] | 33.353 |
| `1 - 3 - 5 6 - - 9` | 21 | 12 | [3, 9] | 33.353 | [3, 9] | 33.353 |
| `1 - 3 4 - - 7 - 9` | 21 | 3 | [3] | 31.718 | [3] | 31.718 |
| `1 - 3 4 - - 7 - 9` | 21 | 4 | [4] | 31.718 | [4] | 31.718 |
| `1 - 3 4 - - 7 - 9` | 21 | 5 | [1, 4] | 31.718 | [1, 4] | 31.718 |
| `1 - 3 4 - - 7 - 9` | 21 | 7 | [7] | 31.718 | [7] | 31.718 |
| `1 - 3 4 - - 7 - 9` | 21 | 8 | [1, 7] | 31.718 | [1, 7] | 31.718 |
| `1 - 3 4 - - 7 - 9` | 21 | 9 | [9] | 31.718 | [9] | 31.718 |
| `1 - 3 4 - - 7 - 9` | 21 | 10 | [1, 9] | 31.718 | [1, 9] | 31.718 |
| `1 - 3 4 - - 7 - 9` | 21 | 11 | [4, 7] | 31.718 | [4, 7] | 31.718 |
| `1 - 3 4 - - 7 - 9` | 21 | 12 | [3, 9] | 31.718 | [3, 9] | 31.718 |
| `1 2 - - - 6 7 8 -` | 21 | 2 | [2] | 31.533 | [2] | 31.528 |
| `1 2 - - - 6 7 8 -` | 21 | 3 | [1, 2] | 31.533 | [1, 2] | 31.528 |
| `1 2 - - - 6 7 8 -` | 21 | 6 | [6] | 31.533 | [6] | 31.528 |
| `1 2 - - - 6 7 8 -` | 21 | 7 | [7] | 31.533 | [7] | 31.528 |
| `1 2 - - - 6 7 8 -` | 21 | 8 | [8] | 31.533 | [8] | 31.528 |
| `1 2 - - - 6 7 8 -` | 21 | 9 | [1, 8] | 31.533 | [1, 8] | 31.528 |
| `1 2 - - - 6 7 8 -` | 21 | 10 | [2, 8] | 31.533 | [2, 8] | 31.528 |
| `1 2 - - - 6 7 8 -` | 21 | 11 | [1, 2, 8] | 31.533 | [1, 2, 8] | 31.528 |
| `1 2 - - 5 - 7 - 9` | 21 | 2 | [2] | 32.968 | [2] | 32.968 |
| `1 2 - - 5 - 7 - 9` | 21 | 3 | [1, 2] | 32.968 | [1, 2] | 32.968 |
| `1 2 - - 5 - 7 - 9` | 21 | 5 | [5] | 32.968 | [5] | 32.968 |
| `1 2 - - 5 - 7 - 9` | 21 | 6 | [1, 5] | 32.968 | [1, 5] | 32.968 |
| `1 2 - - 5 - 7 - 9` | 21 | 7 | [7] | 32.968 | [7] | 32.968 |
| `1 2 - - 5 - 7 - 9` | 21 | 8 | [1, 7] | 32.968 | [1, 7] | 32.968 |
| `1 2 - - 5 - 7 - 9` | 21 | 9 | [9] | 32.968 | [9] | 32.968 |
| `1 2 - - 5 - 7 - 9` | 21 | 10 | [1, 9] | 32.968 | [1, 9] | 32.968 |
| `1 2 - - 5 - 7 - 9` | 21 | 11 | [2, 9] | 32.968 | [2, 9] | 32.968 |
| `1 2 - - 5 - 7 - 9` | 21 | 12 | [5, 7] | 32.968 | [5, 7] | 32.968 |
| `1 2 - 4 - - - 8 9` | 21 | 2 | [2] | 32.836 | [2] | 32.836 |
| `1 2 - 4 - - - 8 9` | 21 | 3 | [1, 2] | 32.836 | [1, 2] | 32.836 |
| `1 2 - 4 - - - 8 9` | 21 | 4 | [4] | 32.836 | [4] | 32.836 |
| `1 2 - 4 - - - 8 9` | 21 | 5 | [1, 4] | 32.836 | [1, 4] | 32.836 |
| `1 2 - 4 - - - 8 9` | 21 | 6 | [2, 4] | 32.836 | [2, 4] | 32.836 |
| `1 2 - 4 - - - 8 9` | 21 | 7 | [1, 2, 4] | 32.836 | [1, 2, 4] | 32.836 |
| `1 2 - 4 - - - 8 9` | 21 | 8 | [8] | 32.836 | [8] | 32.836 |
| `1 2 - 4 - - - 8 9` | 21 | 9 | [9] | 32.836 | [9] | 32.836 |
| `1 2 - 4 - - - 8 9` | 21 | 10 | [1, 9] | 32.836 | [1, 9] | 32.836 |
| `1 2 - 4 - - - 8 9` | 21 | 11 | [2, 9] | 32.836 | [2, 9] | 32.836 |
| `1 2 - 4 - - - 8 9` | 21 | 12 | [4, 8] | 32.836 | [4, 8] | 32.836 |
| `1 2 3 - 5 6 7 - -` | 21 | 2 | [2] | 38.638 | [2] | 38.501 |
| `1 2 3 - 5 6 7 - -` | 21 | 3 | [3] | 38.638 | [3] | 38.501 |
| `1 2 3 - 5 6 7 - -` | 21 | 4 | [1, 3] | 38.638 | [1, 3] | 38.501 |
| `1 2 3 - 5 6 7 - -` | 21 | 5 | [5] | 38.638 | [5] | 38.501 |
| `1 2 3 - 5 6 7 - -` | 21 | 6 | [6] | 38.638 | [6] | 38.501 |
| `1 2 3 - 5 6 7 - -` | 21 | 7 | [7] | 38.638 | [7] | 38.501 |
| `1 2 3 - 5 6 7 - -` | 21 | 8 | [2, 6] | 38.638 | [1, 7] | 38.501 |
| `1 2 3 - 5 6 7 - -` | 21 | 9 | [2, 7] | 38.638 | [2, 7] | 38.501 |
| `1 2 3 - 5 6 7 - -` | 21 | 10 | [3, 7] | 38.638 | [3, 7] | 38.501 |
| `1 2 3 - 5 6 7 - -` | 21 | 11 | [5, 6] | 38.638 | [5, 6] | 38.501 |
| `1 2 3 - 5 6 7 - -` | 21 | 12 | [5, 7] | 38.638 | [5, 7] | 38.501 |
| `1 2 3 4 - 6 - 8 -` | 21 | 2 | [2] | 38.476 | [2] | 38.236 |
| `1 2 3 4 - 6 - 8 -` | 21 | 3 | [3] | 38.476 | [3] | 38.236 |
| `1 2 3 4 - 6 - 8 -` | 21 | 4 | [4] | 38.476 | [4] | 38.236 |
| `1 2 3 4 - 6 - 8 -` | 21 | 5 | [2, 3] | 38.476 | [1, 4] | 38.236 |
| `1 2 3 4 - 6 - 8 -` | 21 | 6 | [6] | 38.476 | [6] | 38.236 |
| `1 2 3 4 - 6 - 8 -` | 21 | 7 | [1, 6] | 38.476 | [1, 6] | 38.236 |
| `1 2 3 4 - 6 - 8 -` | 21 | 8 | [8] | 38.476 | [8] | 38.236 |
| `1 2 3 4 - 6 - 8 -` | 21 | 9 | [1, 8] | 38.476 | [1, 8] | 38.236 |
| `1 2 3 4 - 6 - 8 -` | 21 | 10 | [2, 8] | 38.476 | [2, 8] | 38.236 |
| `1 2 3 4 - 6 - 8 -` | 21 | 11 | [3, 8] | 38.476 | [3, 8] | 38.236 |
| `1 2 3 4 - 6 - 8 -` | 21 | 12 | [4, 8] | 38.476 | [4, 8] | 38.236 |
| `1 2 3 4 5 - - - 9` | 21 | 2 | [2] | 37.234 | [2] | 36.952 |
| `1 2 3 4 5 - - - 9` | 21 | 3 | [3] | 37.234 | [3] | 36.952 |
| `1 2 3 4 5 - - - 9` | 21 | 4 | [4] | 37.234 | [4] | 36.952 |
| `1 2 3 4 5 - - - 9` | 21 | 5 | [5] | 37.234 | [5] | 36.952 |
| `1 2 3 4 5 - - - 9` | 21 | 6 | [2, 4] | 37.234 | [1, 5] | 36.952 |
| `1 2 3 4 5 - - - 9` | 21 | 7 | [3, 4] | 37.234 | [2, 5] | 36.952 |
| `1 2 3 4 5 - - - 9` | 21 | 8 | [3, 5] | 37.234 | [3, 5] | 36.952 |
| `1 2 3 4 5 - - - 9` | 21 | 9 | [9] | 37.234 | [9] | 36.952 |
| `1 2 3 4 5 - - - 9` | 21 | 10 | [1, 9] | 37.234 | [1, 9] | 36.952 |
| `1 2 3 4 5 - - - 9` | 21 | 11 | [2, 9] | 37.234 | [2, 9] | 36.952 |
| `1 2 3 4 5 - - - 9` | 21 | 12 | [3, 9] | 37.234 | [3, 9] | 36.952 |
| `- - - - - 6 - 8 9` | 22 | 6 | [6] | 25.807 | [6] | 25.807 |
| `- - - - - 6 - 8 9` | 22 | 8 | [8] | 25.807 | [8] | 25.807 |
| `- - - - - 6 - 8 9` | 22 | 9 | [9] | 25.807 | [9] | 25.807 |
| `- - - 4 5 6 - 8 -` | 22 | 4 | [4] | 30.257 | [4] | 30.257 |
| `- - - 4 5 6 - 8 -` | 22 | 5 | [5] | 30.257 | [5] | 30.257 |
| `- - - 4 5 6 - 8 -` | 22 | 6 | [6] | 30.257 | [6] | 30.257 |
| `- - - 4 5 6 - 8 -` | 22 | 8 | [8] | 30.257 | [8] | 30.257 |
| `- - - 4 5 6 - 8 -` | 22 | 9 | [4, 5] | 30.257 | [4, 5] | 30.257 |
| `- - - 4 5 6 - 8 -` | 22 | 10 | [4, 6] | 30.257 | [4, 6] | 30.257 |
| `- - - 4 5 6 - 8 -` | 22 | 11 | [5, 6] | 30.257 | [5, 6] | 30.257 |
| `- - - 4 5 6 - 8 -` | 22 | 12 | [4, 8] | 30.257 | [4, 8] | 30.257 |
| `- - 3 - 5 - 7 8 -` | 22 | 3 | [3] | 29.095 | [3] | 29.095 |
| `- - 3 - 5 - 7 8 -` | 22 | 5 | [5] | 29.095 | [5] | 29.095 |
| `- - 3 - 5 - 7 8 -` | 22 | 7 | [7] | 29.095 | [7] | 29.095 |
| `- - 3 - 5 - 7 8 -` | 22 | 8 | [8] | 29.095 | [8] | 29.095 |
| `- - 3 - 5 - 7 8 -` | 22 | 10 | [3, 7] | 29.095 | [3, 7] | 29.095 |
| `- - 3 - 5 - 7 8 -` | 22 | 11 | [3, 8] | 29.095 | [3, 8] | 29.095 |
| `- - 3 - 5 - 7 8 -` | 22 | 12 | [5, 7] | 29.095 | [5, 7] | 29.095 |
| `- - 3 - 5 6 - - 9` | 22 | 3 | [3] | 28.671 | [3] | 28.671 |
| `- - 3 - 5 6 - - 9` | 22 | 5 | [5] | 28.671 | [5] | 28.671 |
| `- - 3 - 5 6 - - 9` | 22 | 6 | [6] | 28.671 | [6] | 28.671 |
| `- - 3 - 5 6 - - 9` | 22 | 8 | [3, 5] | 28.671 | [3, 5] | 28.671 |
| `- - 3 - 5 6 - - 9` | 22 | 9 | [9] | 28.671 | [9] | 28.671 |
| `- - 3 - 5 6 - - 9` | 22 | 11 | [5, 6] | 28.671 | [5, 6] | 28.671 |
| `- - 3 - 5 6 - - 9` | 22 | 12 | [3, 9] | 28.671 | [3, 9] | 28.671 |
| `- - 3 4 - - 7 - 9` | 22 | 3 | [3] | 28.298 | [3] | 28.298 |
| `- - 3 4 - - 7 - 9` | 22 | 4 | [4] | 28.298 | [4] | 28.298 |
| `- - 3 4 - - 7 - 9` | 22 | 7 | [7] | 28.298 | [7] | 28.298 |
| `- - 3 4 - - 7 - 9` | 22 | 9 | [9] | 28.298 | [9] | 28.298 |
| `- - 3 4 - - 7 - 9` | 22 | 10 | [3, 7] | 28.298 | [3, 7] | 28.298 |
| `- - 3 4 - - 7 - 9` | 22 | 11 | [4, 7] | 28.298 | [4, 7] | 28.298 |
| `- - 3 4 - - 7 - 9` | 22 | 12 | [3, 9] | 28.298 | [3, 9] | 28.298 |
| `- 2 - - - 6 7 8 -` | 22 | 2 | [2] | 29.566 | [2] | 29.566 |
| `- 2 - - - 6 7 8 -` | 22 | 6 | [6] | 29.566 | [6] | 29.566 |
| `- 2 - - - 6 7 8 -` | 22 | 7 | [7] | 29.566 | [7] | 29.566 |
| `- 2 - - - 6 7 8 -` | 22 | 8 | [8] | 29.566 | [8] | 29.566 |
| `- 2 - - - 6 7 8 -` | 22 | 9 | [2, 7] | 29.566 | [2, 7] | 29.566 |
| `- 2 - - - 6 7 8 -` | 22 | 10 | [2, 8] | 29.566 | [2, 8] | 29.566 |
| `- 2 - - 5 - 7 - 9` | 22 | 2 | [2] | 27.485 | [2] | 27.485 |
| `- 2 - - 5 - 7 - 9` | 22 | 5 | [5] | 27.485 | [5] | 27.485 |
| `- 2 - - 5 - 7 - 9` | 22 | 7 | [7] | 27.485 | [7] | 27.485 |
| `- 2 - - 5 - 7 - 9` | 22 | 9 | [9] | 27.485 | [9] | 27.485 |
| `- 2 - - 5 - 7 - 9` | 22 | 11 | [2, 9] | 27.485 | [2, 9] | 27.485 |
| `- 2 - - 5 - 7 - 9` | 22 | 12 | [5, 7] | 27.485 | [5, 7] | 27.485 |
| `- 2 - 4 - - - 8 9` | 22 | 2 | [2] | 29.081 | [2] | 29.081 |
| `- 2 - 4 - - - 8 9` | 22 | 4 | [4] | 29.081 | [4] | 29.081 |
| `- 2 - 4 - - - 8 9` | 22 | 6 | [2, 4] | 29.081 | [2, 4] | 29.081 |
| `- 2 - 4 - - - 8 9` | 22 | 8 | [8] | 29.081 | [8] | 29.081 |
| `- 2 - 4 - - - 8 9` | 22 | 9 | [9] | 29.081 | [9] | 29.081 |
| `- 2 - 4 - - - 8 9` | 22 | 10 | [2, 8] | 29.081 | [2, 8] | 29.081 |
| `- 2 - 4 - - - 8 9` | 22 | 11 | [2, 9] | 29.081 | [2, 9] | 29.081 |
| `- 2 - 4 - - - 8 9` | 22 | 12 | [4, 8] | 29.081 | [4, 8] | 29.081 |
| `- 2 3 - 5 6 7 - -` | 22 | 2 | [2] | 35.071 | [2] | 35.056 |
| `- 2 3 - 5 6 7 - -` | 22 | 3 | [3] | 35.071 | [3] | 35.056 |
| `- 2 3 - 5 6 7 - -` | 22 | 5 | [5] | 35.071 | [5] | 35.056 |
| `- 2 3 - 5 6 7 - -` | 22 | 6 | [6] | 35.071 | [6] | 35.056 |
| `- 2 3 - 5 6 7 - -` | 22 | 7 | [7] | 35.071 | [7] | 35.056 |
| `- 2 3 - 5 6 7 - -` | 22 | 8 | [3, 5] | 35.071 | [2, 6] | 35.056 |
| `- 2 3 - 5 6 7 - -` | 22 | 9 | [2, 7] | 35.071 | [2, 7] | 35.056 |
| `- 2 3 - 5 6 7 - -` | 22 | 10 | [3, 7] | 35.071 | [3, 7] | 35.056 |
| `- 2 3 - 5 6 7 - -` | 22 | 11 | [5, 6] | 35.071 | [5, 6] | 35.056 |
| `- 2 3 - 5 6 7 - -` | 22 | 12 | [5, 7] | 35.071 | [5, 7] | 35.056 |
| `- 2 3 4 - 6 - 8 -` | 22 | 2 | [2] | 35.077 | [2] | 35.077 |
| `- 2 3 4 - 6 - 8 -` | 22 | 3 | [3] | 35.077 | [3] | 35.077 |
| `- 2 3 4 - 6 - 8 -` | 22 | 4 | [4] | 35.077 | [4] | 35.077 |
| `- 2 3 4 - 6 - 8 -` | 22 | 5 | [2, 3] | 35.077 | [2, 3] | 35.077 |
| `- 2 3 4 - 6 - 8 -` | 22 | 6 | [6] | 35.077 | [6] | 35.077 |
| `- 2 3 4 - 6 - 8 -` | 22 | 7 | [3, 4] | 35.077 | [3, 4] | 35.077 |
| `- 2 3 4 - 6 - 8 -` | 22 | 8 | [8] | 35.077 | [8] | 35.077 |
| `- 2 3 4 - 6 - 8 -` | 22 | 9 | [3, 6] | 35.077 | [3, 6] | 35.077 |
| `- 2 3 4 - 6 - 8 -` | 22 | 10 | [2, 8] | 35.077 | [2, 8] | 35.077 |
| `- 2 3 4 - 6 - 8 -` | 22 | 11 | [3, 8] | 35.077 | [3, 8] | 35.077 |
| `- 2 3 4 - 6 - 8 -` | 22 | 12 | [4, 8] | 35.077 | [4, 8] | 35.077 |
| `- 2 3 4 5 - - - 9` | 22 | 2 | [2] | 34.175 | [2] | 34.099 |
| `- 2 3 4 5 - - - 9` | 22 | 3 | [3] | 34.175 | [3] | 34.099 |
| `- 2 3 4 5 - - - 9` | 22 | 4 | [4] | 34.175 | [4] | 34.099 |
| `- 2 3 4 5 - - - 9` | 22 | 5 | [5] | 34.175 | [5] | 34.099 |
| `- 2 3 4 5 - - - 9` | 22 | 6 | [2, 4] | 34.175 | [2, 4] | 34.099 |
| `- 2 3 4 5 - - - 9` | 22 | 7 | [3, 4] | 34.175 | [2, 5] | 34.099 |
| `- 2 3 4 5 - - - 9` | 22 | 8 | [3, 5] | 34.175 | [3, 5] | 34.099 |
| `- 2 3 4 5 - - - 9` | 22 | 9 | [9] | 34.175 | [9] | 34.099 |
| `- 2 3 4 5 - - - 9` | 22 | 10 | [2, 3, 5] | 34.175 | [2, 3, 5] | 34.099 |
| `- 2 3 4 5 - - - 9` | 22 | 11 | [2, 9] | 34.175 | [2, 9] | 34.099 |
| `- 2 3 4 5 - - - 9` | 22 | 12 | [3, 9] | 34.175 | [3, 9] | 34.099 |
| `1 - - - - 6 7 - 9` | 22 | 6 | [6] | 29.293 | [6] | 29.293 |
| `1 - - - - 6 7 - 9` | 22 | 7 | [7] | 29.293 | [7] | 29.293 |
| `1 - - - - 6 7 - 9` | 22 | 8 | [1, 7] | 29.293 | [1, 7] | 29.293 |
| `1 - - - - 6 7 - 9` | 22 | 9 | [9] | 29.293 | [9] | 29.293 |
| `1 - - - - 6 7 - 9` | 22 | 10 | [1, 9] | 29.293 | [1, 9] | 29.293 |
| `1 - - - 5 - - 8 9` | 22 | 5 | [5] | 28.194 | [5] | 28.194 |
| `1 - - - 5 - - 8 9` | 22 | 6 | [1, 5] | 28.194 | [1, 5] | 28.194 |
| `1 - - - 5 - - 8 9` | 22 | 8 | [8] | 28.194 | [8] | 28.194 |
| `1 - - - 5 - - 8 9` | 22 | 9 | [9] | 28.194 | [9] | 28.194 |
| `1 - - - 5 - - 8 9` | 22 | 10 | [1, 9] | 28.194 | [1, 9] | 28.194 |
| `1 - - 4 5 6 7 - -` | 22 | 4 | [4] | 34.791 | [4] | 34.757 |
| `1 - - 4 5 6 7 - -` | 22 | 5 | [5] | 34.791 | [5] | 34.757 |
| `1 - - 4 5 6 7 - -` | 22 | 6 | [6] | 34.791 | [6] | 34.757 |
| `1 - - 4 5 6 7 - -` | 22 | 7 | [7] | 34.791 | [7] | 34.757 |
| `1 - - 4 5 6 7 - -` | 22 | 8 | [1, 7] | 34.791 | [1, 7] | 34.757 |
| `1 - - 4 5 6 7 - -` | 22 | 9 | [4, 5] | 34.791 | [4, 5] | 34.757 |
| `1 - - 4 5 6 7 - -` | 22 | 10 | [4, 6] | 34.791 | [4, 6] | 34.757 |
| `1 - - 4 5 6 7 - -` | 22 | 11 | [5, 6] | 34.791 | [4, 7] | 34.757 |
| `1 - - 4 5 6 7 - -` | 22 | 12 | [5, 7] | 34.791 | [5, 7] | 34.757 |
| `1 - 3 - 5 6 - 8 -` | 22 | 3 | [3] | 34.460 | [3] | 34.460 |
| `1 - 3 - 5 6 - 8 -` | 22 | 4 | [1, 3] | 34.460 | [1, 3] | 34.460 |
| `1 - 3 - 5 6 - 8 -` | 22 | 5 | [5] | 34.460 | [5] | 34.460 |
| `1 - 3 - 5 6 - 8 -` | 22 | 6 | [6] | 34.460 | [6] | 34.460 |
| `1 - 3 - 5 6 - 8 -` | 22 | 7 | [1, 6] | 34.460 | [1, 6] | 34.460 |
| `1 - 3 - 5 6 - 8 -` | 22 | 8 | [8] | 34.460 | [8] | 34.460 |
| `1 - 3 - 5 6 - 8 -` | 22 | 9 | [1, 8] | 34.460 | [1, 8] | 34.460 |
| `1 - 3 - 5 6 - 8 -` | 22 | 10 | [1, 3, 6] | 34.460 | [1, 3, 6] | 34.460 |
| `1 - 3 - 5 6 - 8 -` | 22 | 11 | [3, 8] | 34.460 | [3, 8] | 34.460 |
| `1 - 3 - 5 6 - 8 -` | 22 | 12 | [1, 3, 8] | 34.460 | [1, 3, 8] | 34.460 |
| `1 - 3 4 - - 7 8 -` | 22 | 3 | [3] | 32.978 | [3] | 32.978 |
| `1 - 3 4 - - 7 8 -` | 22 | 4 | [4] | 32.978 | [4] | 32.978 |
| `1 - 3 4 - - 7 8 -` | 22 | 5 | [1, 4] | 32.978 | [1, 4] | 32.978 |
| `1 - 3 4 - - 7 8 -` | 22 | 7 | [7] | 32.978 | [7] | 32.978 |
| `1 - 3 4 - - 7 8 -` | 22 | 8 | [8] | 32.978 | [8] | 32.978 |
| `1 - 3 4 - - 7 8 -` | 22 | 9 | [1, 8] | 32.978 | [1, 8] | 32.978 |
| `1 - 3 4 - - 7 8 -` | 22 | 10 | [3, 7] | 32.978 | [3, 7] | 32.978 |
| `1 - 3 4 - - 7 8 -` | 22 | 11 | [3, 8] | 32.978 | [3, 8] | 32.978 |
| `1 - 3 4 - - 7 8 -` | 22 | 12 | [4, 8] | 32.978 | [4, 8] | 32.978 |
| `1 - 3 4 - 6 - - 9` | 22 | 3 | [3] | 33.842 | [3] | 33.674 |
| `1 - 3 4 - 6 - - 9` | 22 | 4 | [4] | 33.842 | [4] | 33.674 |
| `1 - 3 4 - 6 - - 9` | 22 | 5 | [1, 4] | 33.842 | [1, 4] | 33.674 |
| `1 - 3 4 - 6 - - 9` | 22 | 6 | [6] | 33.842 | [6] | 33.674 |
| `1 - 3 4 - 6 - - 9` | 22 | 7 | [3, 4] | 33.842 | [1, 6] | 33.674 |
| `1 - 3 4 - 6 - - 9` | 22 | 8 | [1, 3, 4] | 33.842 | [1, 3, 4] | 33.674 |
| `1 - 3 4 - 6 - - 9` | 22 | 9 | [9] | 33.842 | [9] | 33.674 |
| `1 - 3 4 - 6 - - 9` | 22 | 10 | [1, 9] | 33.842 | [1, 9] | 33.674 |
| `1 - 3 4 - 6 - - 9` | 22 | 11 | [1, 4, 6] | 33.842 | [1, 4, 6] | 33.674 |
| `1 - 3 4 - 6 - - 9` | 22 | 12 | [3, 9] | 33.842 | [3, 9] | 33.674 |
| `1 2 - - 5 - 7 8 -` | 22 | 2 | [2] | 34.247 | [2] | 34.207 |
| `1 2 - - 5 - 7 8 -` | 22 | 3 | [1, 2] | 34.247 | [1, 2] | 34.207 |
| `1 2 - - 5 - 7 8 -` | 22 | 5 | [5] | 34.247 | [5] | 34.207 |
| `1 2 - - 5 - 7 8 -` | 22 | 6 | [1, 5] | 34.247 | [1, 5] | 34.207 |
| `1 2 - - 5 - 7 8 -` | 22 | 7 | [7] | 34.247 | [7] | 34.207 |
| `1 2 - - 5 - 7 8 -` | 22 | 8 | [8] | 34.247 | [8] | 34.207 |
| `1 2 - - 5 - 7 8 -` | 22 | 9 | [2, 7] | 34.247 | [1, 8] | 34.207 |
| `1 2 - - 5 - 7 8 -` | 22 | 10 | [2, 8] | 34.247 | [2, 8] | 34.207 |
| `1 2 - - 5 - 7 8 -` | 22 | 11 | [1, 2, 8] | 34.247 | [1, 2, 8] | 34.207 |
| `1 2 - - 5 - 7 8 -` | 22 | 12 | [5, 7] | 34.247 | [5, 7] | 34.207 |
| `1 2 - - 5 6 - - 9` | 22 | 2 | [2] | 34.022 | [2] | 33.931 |
| `1 2 - - 5 6 - - 9` | 22 | 3 | [1, 2] | 34.022 | [1, 2] | 33.931 |
| `1 2 - - 5 6 - - 9` | 22 | 5 | [5] | 34.022 | [5] | 33.931 |
| `1 2 - - 5 6 - - 9` | 22 | 6 | [6] | 34.022 | [6] | 33.931 |
| `1 2 - - 5 6 - - 9` | 22 | 7 | [2, 5] | 34.022 | [1, 6] | 33.931 |
| `1 2 - - 5 6 - - 9` | 22 | 8 | [2, 6] | 34.022 | [2, 6] | 33.931 |
| `1 2 - - 5 6 - - 9` | 22 | 9 | [9] | 34.022 | [9] | 33.931 |
| `1 2 - - 5 6 - - 9` | 22 | 10 | [1, 9] | 34.022 | [1, 9] | 33.931 |
| `1 2 - - 5 6 - - 9` | 22 | 11 | [2, 9] | 34.022 | [2, 9] | 33.931 |
| `1 2 - - 5 6 - - 9` | 22 | 12 | [1, 2, 9] | 34.022 | [1, 2, 9] | 33.931 |
| `1 2 - 4 - - 7 - 9` | 22 | 2 | [2] | 34.522 | [2] | 34.522 |
| `1 2 - 4 - - 7 - 9` | 22 | 3 | [1, 2] | 34.522 | [1, 2] | 34.522 |
| `1 2 - 4 - - 7 - 9` | 22 | 4 | [4] | 34.522 | [4] | 34.522 |
| `1 2 - 4 - - 7 - 9` | 22 | 5 | [1, 4] | 34.522 | [1, 4] | 34.522 |
| `1 2 - 4 - - 7 - 9` | 22 | 6 | [2, 4] | 34.522 | [2, 4] | 34.522 |
| `1 2 - 4 - - 7 - 9` | 22 | 7 | [7] | 34.522 | [7] | 34.522 |
| `1 2 - 4 - - 7 - 9` | 22 | 8 | [1, 7] | 34.522 | [1, 7] | 34.522 |
| `1 2 - 4 - - 7 - 9` | 22 | 9 | [9] | 34.522 | [9] | 34.522 |
| `1 2 - 4 - - 7 - 9` | 22 | 10 | [1, 9] | 34.522 | [1, 9] | 34.522 |
| `1 2 - 4 - - 7 - 9` | 22 | 11 | [2, 9] | 34.522 | [2, 9] | 34.522 |
| `1 2 - 4 - - 7 - 9` | 22 | 12 | [1, 2, 9] | 34.522 | [1, 2, 9] | 34.522 |
| `1 2 3 - - - - 8 9` | 22 | 2 | [2] | 31.616 | [2] | 31.616 |
| `1 2 3 - - - - 8 9` | 22 | 3 | [3] | 31.616 | [3] | 31.616 |
| `1 2 3 - - - - 8 9` | 22 | 4 | [1, 3] | 31.616 | [1, 3] | 31.616 |
| `1 2 3 - - - - 8 9` | 22 | 5 | [2, 3] | 31.616 | [2, 3] | 31.616 |
| `1 2 3 - - - - 8 9` | 22 | 6 | [1, 2, 3] | 31.616 | [1, 2, 3] | 31.616 |
| `1 2 3 - - - - 8 9` | 22 | 8 | [8] | 31.616 | [8] | 31.616 |
| `1 2 3 - - - - 8 9` | 22 | 9 | [9] | 31.616 | [9] | 31.616 |
| `1 2 3 - - - - 8 9` | 22 | 10 | [1, 9] | 31.616 | [1, 9] | 31.616 |
| `1 2 3 - - - - 8 9` | 22 | 11 | [2, 9] | 31.616 | [2, 9] | 31.616 |
| `1 2 3 - - - - 8 9` | 22 | 12 | [3, 9] | 31.616 | [3, 9] | 31.616 |
| `1 2 3 4 - 6 7 - -` | 22 | 2 | [2] | 39.474 | [2] | 39.399 |
| `1 2 3 4 - 6 7 - -` | 22 | 3 | [3] | 39.474 | [3] | 39.399 |
| `1 2 3 4 - 6 7 - -` | 22 | 4 | [4] | 39.474 | [4] | 39.399 |
| `1 2 3 4 - 6 7 - -` | 22 | 5 | [1, 4] | 39.474 | [1, 4] | 39.399 |
| `1 2 3 4 - 6 7 - -` | 22 | 6 | [6] | 39.474 | [6] | 39.399 |
| `1 2 3 4 - 6 7 - -` | 22 | 7 | [7] | 39.474 | [7] | 39.399 |
| `1 2 3 4 - 6 7 - -` | 22 | 8 | [1, 7] | 39.474 | [1, 7] | 39.399 |
| `1 2 3 4 - 6 7 - -` | 22 | 9 | [3, 6] | 39.474 | [2, 7] | 39.399 |
| `1 2 3 4 - 6 7 - -` | 22 | 10 | [3, 7] | 39.474 | [3, 7] | 39.399 |
| `1 2 3 4 - 6 7 - -` | 22 | 11 | [4, 7] | 39.474 | [4, 7] | 39.399 |
| `1 2 3 4 - 6 7 - -` | 22 | 12 | [2, 3, 7] | 39.474 | [1, 4, 7] | 39.399 |
| `1 2 3 4 5 - - 8 -` | 22 | 2 | [2] | 38.987 | [2] | 38.733 |
| `1 2 3 4 5 - - 8 -` | 22 | 3 | [3] | 38.987 | [3] | 38.733 |
| `1 2 3 4 5 - - 8 -` | 22 | 4 | [4] | 38.987 | [4] | 38.733 |
| `1 2 3 4 5 - - 8 -` | 22 | 5 | [5] | 38.987 | [5] | 38.733 |
| `1 2 3 4 5 - - 8 -` | 22 | 6 | [1, 5] | 38.987 | [1, 5] | 38.733 |
| `1 2 3 4 5 - - 8 -` | 22 | 7 | [3, 4] | 38.987 | [2, 5] | 38.733 |
| `1 2 3 4 5 - - 8 -` | 22 | 8 | [8] | 38.987 | [8] | 38.733 |
| `1 2 3 4 5 - - 8 -` | 22 | 9 | [1, 8] | 38.987 | [1, 8] | 38.733 |
| `1 2 3 4 5 - - 8 -` | 22 | 10 | [2, 8] | 38.987 | [2, 8] | 38.733 |
| `1 2 3 4 5 - - 8 -` | 22 | 11 | [3, 8] | 38.987 | [3, 8] | 38.733 |
| `1 2 3 4 5 - - 8 -` | 22 | 12 | [4, 8] | 38.987 | [4, 8] | 38.733 |
| `- - - - - 6 7 - 9` | 23 | 6 | [6] | 26.942 | [6] | 26.942 |
| `- - - - - 6 7 - 9` | 23 | 7 | [7] | 26.942 | [7] | 26.942 |
| `- - - - - 6 7 - 9` | 23 | 9 | [9] | 26.942 | [9] | 26.942 |
| `- - - - 5 - - 8 9` | 23 | 5 | [5] | 26.378 | [5] | 26.378 |
| `- - - - 5 - - 8 9` | 23 | 8 | [8] | 26.378 | [8] | 26.378 |
| `- - - - 5 - - 8 9` | 23 | 9 | [9] | 26.378 | [9] | 26.378 |
| `- - - 4 5 6 7 - -` | 23 | 4 | [4] | 31.726 | [4] | 31.721 |
| `- - - 4 5 6 7 - -` | 23 | 5 | [5] | 31.726 | [5] | 31.721 |
| `- - - 4 5 6 7 - -` | 23 | 6 | [6] | 31.726 | [6] | 31.721 |
| `- - - 4 5 6 7 - -` | 23 | 7 | [7] | 31.726 | [7] | 31.721 |
| `- - - 4 5 6 7 - -` | 23 | 9 | [4, 5] | 31.726 | [4, 5] | 31.721 |
| `- - - 4 5 6 7 - -` | 23 | 10 | [4, 6] | 31.726 | [4, 6] | 31.721 |
| `- - - 4 5 6 7 - -` | 23 | 11 | [5, 6] | 31.726 | [4, 7] | 31.721 |
| `- - - 4 5 6 7 - -` | 23 | 12 | [5, 7] | 31.726 | [5, 7] | 31.721 |
| `- - 3 - 5 6 - 8 -` | 23 | 3 | [3] | 29.465 | [3] | 29.465 |
| `- - 3 - 5 6 - 8 -` | 23 | 5 | [5] | 29.465 | [5] | 29.465 |
| `- - 3 - 5 6 - 8 -` | 23 | 6 | [6] | 29.465 | [6] | 29.465 |
| `- - 3 - 5 6 - 8 -` | 23 | 8 | [8] | 29.465 | [8] | 29.465 |
| `- - 3 - 5 6 - 8 -` | 23 | 9 | [3, 6] | 29.465 | [3, 6] | 29.465 |
| `- - 3 - 5 6 - 8 -` | 23 | 11 | [3, 8] | 29.465 | [3, 8] | 29.465 |
| `- - 3 4 - - 7 8 -` | 23 | 3 | [3] | 29.817 | [3] | 29.817 |
| `- - 3 4 - - 7 8 -` | 23 | 4 | [4] | 29.817 | [4] | 29.817 |
| `- - 3 4 - - 7 8 -` | 23 | 7 | [7] | 29.817 | [7] | 29.817 |
| `- - 3 4 - - 7 8 -` | 23 | 8 | [8] | 29.817 | [8] | 29.817 |
| `- - 3 4 - - 7 8 -` | 23 | 10 | [3, 7] | 29.817 | [3, 7] | 29.817 |
| `- - 3 4 - - 7 8 -` | 23 | 11 | [3, 8] | 29.817 | [3, 8] | 29.817 |
| `- - 3 4 - - 7 8 -` | 23 | 12 | [4, 8] | 29.817 | [4, 8] | 29.817 |
| `- - 3 4 - 6 - - 9` | 23 | 3 | [3] | 29.707 | [3] | 29.707 |
| `- - 3 4 - 6 - - 9` | 23 | 4 | [4] | 29.707 | [4] | 29.707 |
| `- - 3 4 - 6 - - 9` | 23 | 6 | [6] | 29.707 | [6] | 29.707 |
| `- - 3 4 - 6 - - 9` | 23 | 7 | [3, 4] | 29.707 | [3, 4] | 29.707 |
| `- - 3 4 - 6 - - 9` | 23 | 9 | [9] | 29.707 | [9] | 29.707 |
| `- - 3 4 - 6 - - 9` | 23 | 10 | [4, 6] | 29.707 | [4, 6] | 29.707 |
| `- - 3 4 - 6 - - 9` | 23 | 12 | [3, 9] | 29.707 | [3, 9] | 29.707 |
| `- 2 - - 5 - 7 8 -` | 23 | 2 | [2] | 30.465 | [2] | 30.465 |
| `- 2 - - 5 - 7 8 -` | 23 | 5 | [5] | 30.465 | [5] | 30.465 |
| `- 2 - - 5 - 7 8 -` | 23 | 7 | [7] | 30.465 | [7] | 30.465 |
| `- 2 - - 5 - 7 8 -` | 23 | 8 | [8] | 30.465 | [8] | 30.465 |
| `- 2 - - 5 - 7 8 -` | 23 | 9 | [2, 7] | 30.465 | [2, 7] | 30.465 |
| `- 2 - - 5 - 7 8 -` | 23 | 10 | [2, 8] | 30.465 | [2, 8] | 30.465 |
| `- 2 - - 5 - 7 8 -` | 23 | 12 | [5, 7] | 30.465 | [5, 7] | 30.465 |
| `- 2 - - 5 6 - - 9` | 23 | 2 | [2] | 30.746 | [2] | 30.746 |
| `- 2 - - 5 6 - - 9` | 23 | 5 | [5] | 30.746 | [5] | 30.746 |
| `- 2 - - 5 6 - - 9` | 23 | 6 | [6] | 30.746 | [6] | 30.746 |
| `- 2 - - 5 6 - - 9` | 23 | 7 | [2, 5] | 30.746 | [2, 5] | 30.746 |
| `- 2 - - 5 6 - - 9` | 23 | 8 | [2, 6] | 30.746 | [2, 6] | 30.746 |
| `- 2 - - 5 6 - - 9` | 23 | 9 | [9] | 30.746 | [9] | 30.746 |
| `- 2 - - 5 6 - - 9` | 23 | 11 | [2, 9] | 30.746 | [2, 9] | 30.746 |
| `- 2 - 4 - - 7 - 9` | 23 | 2 | [2] | 28.936 | [2] | 28.936 |
| `- 2 - 4 - - 7 - 9` | 23 | 4 | [4] | 28.936 | [4] | 28.936 |
| `- 2 - 4 - - 7 - 9` | 23 | 6 | [2, 4] | 28.936 | [2, 4] | 28.936 |
| `- 2 - 4 - - 7 - 9` | 23 | 7 | [7] | 28.936 | [7] | 28.936 |
| `- 2 - 4 - - 7 - 9` | 23 | 9 | [9] | 28.936 | [9] | 28.936 |
| `- 2 - 4 - - 7 - 9` | 23 | 11 | [2, 9] | 28.936 | [2, 9] | 28.936 |
| `- 2 3 - - - - 8 9` | 23 | 2 | [2] | 29.417 | [2] | 29.417 |
| `- 2 3 - - - - 8 9` | 23 | 3 | [3] | 29.417 | [3] | 29.417 |
| `- 2 3 - - - - 8 9` | 23 | 5 | [2, 3] | 29.417 | [2, 3] | 29.417 |
| `- 2 3 - - - - 8 9` | 23 | 8 | [8] | 29.417 | [8] | 29.417 |
| `- 2 3 - - - - 8 9` | 23 | 9 | [9] | 29.417 | [9] | 29.417 |
| `- 2 3 - - - - 8 9` | 23 | 10 | [2, 8] | 29.417 | [2, 8] | 29.417 |
| `- 2 3 - - - - 8 9` | 23 | 11 | [2, 9] | 29.417 | [2, 9] | 29.417 |
| `- 2 3 - - - - 8 9` | 23 | 12 | [3, 9] | 29.417 | [3, 9] | 29.417 |
| `- 2 3 4 - 6 7 - -` | 23 | 2 | [2] | 36.545 | [2] | 36.486 |
| `- 2 3 4 - 6 7 - -` | 23 | 3 | [3] | 36.545 | [3] | 36.486 |
| `- 2 3 4 - 6 7 - -` | 23 | 4 | [4] | 36.545 | [4] | 36.486 |
| `- 2 3 4 - 6 7 - -` | 23 | 5 | [2, 3] | 36.545 | [2, 3] | 36.486 |
| `- 2 3 4 - 6 7 - -` | 23 | 6 | [6] | 36.545 | [6] | 36.486 |
| `- 2 3 4 - 6 7 - -` | 23 | 7 | [7] | 36.545 | [7] | 36.486 |
| `- 2 3 4 - 6 7 - -` | 23 | 8 | [2, 6] | 36.545 | [2, 6] | 36.486 |
| `- 2 3 4 - 6 7 - -` | 23 | 9 | [2, 7] | 36.545 | [2, 7] | 36.486 |
| `- 2 3 4 - 6 7 - -` | 23 | 10 | [4, 6] | 36.545 | [3, 7] | 36.486 |
| `- 2 3 4 - 6 7 - -` | 23 | 11 | [4, 7] | 36.545 | [4, 7] | 36.486 |
| `- 2 3 4 - 6 7 - -` | 23 | 12 | [2, 4, 6] | 36.545 | [2, 3, 7] | 36.486 |
| `- 2 3 4 5 - - 8 -` | 23 | 2 | [2] | 36.019 | [2] | 36.010 |
| `- 2 3 4 5 - - 8 -` | 23 | 3 | [3] | 36.019 | [3] | 36.010 |
| `- 2 3 4 5 - - 8 -` | 23 | 4 | [4] | 36.019 | [4] | 36.010 |
| `- 2 3 4 5 - - 8 -` | 23 | 5 | [5] | 36.019 | [5] | 36.010 |
| `- 2 3 4 5 - - 8 -` | 23 | 6 | [2, 4] | 36.019 | [2, 4] | 36.010 |
| `- 2 3 4 5 - - 8 -` | 23 | 7 | [3, 4] | 36.019 | [2, 5] | 36.010 |
| `- 2 3 4 5 - - 8 -` | 23 | 8 | [8] | 36.019 | [8] | 36.010 |
| `- 2 3 4 5 - - 8 -` | 23 | 9 | [4, 5] | 36.019 | [4, 5] | 36.010 |
| `- 2 3 4 5 - - 8 -` | 23 | 10 | [2, 8] | 36.019 | [2, 8] | 36.010 |
| `- 2 3 4 5 - - 8 -` | 23 | 11 | [3, 8] | 36.019 | [3, 8] | 36.010 |
| `- 2 3 4 5 - - 8 -` | 23 | 12 | [4, 8] | 36.019 | [4, 8] | 36.010 |
| `1 - - - - 6 7 8 -` | 23 | 6 | [6] | 29.298 | [6] | 29.298 |
| `1 - - - - 6 7 8 -` | 23 | 7 | [7] | 29.298 | [7] | 29.298 |
| `1 - - - - 6 7 8 -` | 23 | 8 | [8] | 29.298 | [8] | 29.298 |
| `1 - - - - 6 7 8 -` | 23 | 9 | [1, 8] | 29.298 | [1, 8] | 29.298 |
| `1 - - - 5 - 7 - 9` | 23 | 5 | [5] | 31.354 | [5] | 31.354 |
| `1 - - - 5 - 7 - 9` | 23 | 6 | [1, 5] | 31.354 | [1, 5] | 31.354 |
| `1 - - - 5 - 7 - 9` | 23 | 7 | [7] | 31.354 | [7] | 31.354 |
| `1 - - - 5 - 7 - 9` | 23 | 8 | [1, 7] | 31.354 | [1, 7] | 31.354 |
| `1 - - - 5 - 7 - 9` | 23 | 9 | [9] | 31.354 | [9] | 31.354 |
| `1 - - - 5 - 7 - 9` | 23 | 10 | [1, 9] | 31.354 | [1, 9] | 31.354 |
| `1 - - - 5 - 7 - 9` | 23 | 12 | [5, 7] | 31.354 | [5, 7] | 31.354 |
| `1 - - 4 - - - 8 9` | 23 | 4 | [4] | 28.789 | [4] | 28.789 |
| `1 - - 4 - - - 8 9` | 23 | 5 | [1, 4] | 28.789 | [1, 4] | 28.789 |
| `1 - - 4 - - - 8 9` | 23 | 8 | [8] | 28.789 | [8] | 28.789 |
| `1 - - 4 - - - 8 9` | 23 | 9 | [9] | 28.789 | [9] | 28.789 |
| `1 - - 4 - - - 8 9` | 23 | 10 | [1, 9] | 28.789 | [1, 9] | 28.789 |
| `1 - - 4 - - - 8 9` | 23 | 12 | [4, 8] | 28.789 | [4, 8] | 28.789 |
| `1 - 3 - 5 6 7 - -` | 23 | 3 | [3] | 36.472 | [3] | 36.464 |
| `1 - 3 - 5 6 7 - -` | 23 | 4 | [1, 3] | 36.472 | [1, 3] | 36.464 |
| `1 - 3 - 5 6 7 - -` | 23 | 5 | [5] | 36.472 | [5] | 36.464 |
| `1 - 3 - 5 6 7 - -` | 23 | 6 | [6] | 36.472 | [6] | 36.464 |
| `1 - 3 - 5 6 7 - -` | 23 | 7 | [7] | 36.472 | [7] | 36.464 |
| `1 - 3 - 5 6 7 - -` | 23 | 8 | [1, 7] | 36.472 | [1, 7] | 36.464 |
| `1 - 3 - 5 6 7 - -` | 23 | 9 | [3, 6] | 36.472 | [3, 6] | 36.464 |
| `1 - 3 - 5 6 7 - -` | 23 | 10 | [3, 7] | 36.472 | [3, 7] | 36.464 |
| `1 - 3 - 5 6 7 - -` | 23 | 11 | [5, 6] | 36.472 | [5, 6] | 36.464 |
| `1 - 3 - 5 6 7 - -` | 23 | 12 | [5, 7] | 36.472 | [5, 7] | 36.464 |
| `1 - 3 4 - 6 - 8 -` | 23 | 3 | [3] | 35.993 | [3] | 35.899 |
| `1 - 3 4 - 6 - 8 -` | 23 | 4 | [4] | 35.993 | [4] | 35.899 |
| `1 - 3 4 - 6 - 8 -` | 23 | 5 | [1, 4] | 35.993 | [1, 4] | 35.899 |
| `1 - 3 4 - 6 - 8 -` | 23 | 6 | [6] | 35.993 | [6] | 35.899 |
| `1 - 3 4 - 6 - 8 -` | 23 | 7 | [3, 4] | 35.993 | [1, 6] | 35.899 |
| `1 - 3 4 - 6 - 8 -` | 23 | 8 | [8] | 35.993 | [8] | 35.899 |
| `1 - 3 4 - 6 - 8 -` | 23 | 9 | [1, 8] | 35.993 | [1, 8] | 35.899 |
| `1 - 3 4 - 6 - 8 -` | 23 | 10 | [4, 6] | 35.993 | [4, 6] | 35.899 |
| `1 - 3 4 - 6 - 8 -` | 23 | 11 | [3, 8] | 35.993 | [3, 8] | 35.899 |
| `1 - 3 4 - 6 - 8 -` | 23 | 12 | [4, 8] | 35.993 | [4, 8] | 35.899 |
| `1 - 3 4 5 - - - 9` | 23 | 3 | [3] | 33.965 | [3] | 33.965 |
| `1 - 3 4 5 - - - 9` | 23 | 4 | [4] | 33.965 | [4] | 33.965 |
| `1 - 3 4 5 - - - 9` | 23 | 5 | [5] | 33.965 | [5] | 33.965 |
| `1 - 3 4 5 - - - 9` | 23 | 6 | [1, 5] | 33.965 | [1, 5] | 33.965 |
| `1 - 3 4 5 - - - 9` | 23 | 7 | [3, 4] | 33.965 | [3, 4] | 33.965 |
| `1 - 3 4 5 - - - 9` | 23 | 8 | [3, 5] | 33.965 | [3, 5] | 33.965 |
| `1 - 3 4 5 - - - 9` | 23 | 9 | [9] | 33.965 | [9] | 33.965 |
| `1 - 3 4 5 - - - 9` | 23 | 10 | [1, 9] | 33.965 | [1, 9] | 33.965 |
| `1 - 3 4 5 - - - 9` | 23 | 12 | [3, 9] | 33.965 | [3, 9] | 33.965 |
| `1 2 - - 5 6 - 8 -` | 23 | 2 | [2] | 35.254 | [2] | 35.170 |
| `1 2 - - 5 6 - 8 -` | 23 | 3 | [1, 2] | 35.254 | [1, 2] | 35.170 |
| `1 2 - - 5 6 - 8 -` | 23 | 5 | [5] | 35.254 | [5] | 35.170 |
| `1 2 - - 5 6 - 8 -` | 23 | 6 | [6] | 35.254 | [6] | 35.170 |
| `1 2 - - 5 6 - 8 -` | 23 | 7 | [2, 5] | 35.254 | [1, 6] | 35.170 |
| `1 2 - - 5 6 - 8 -` | 23 | 8 | [8] | 35.254 | [8] | 35.170 |
| `1 2 - - 5 6 - 8 -` | 23 | 9 | [1, 8] | 35.254 | [1, 8] | 35.170 |
| `1 2 - - 5 6 - 8 -` | 23 | 10 | [2, 8] | 35.254 | [2, 8] | 35.170 |
| `1 2 - - 5 6 - 8 -` | 23 | 11 | [5, 6] | 35.254 | [5, 6] | 35.170 |
| `1 2 - - 5 6 - 8 -` | 23 | 12 | [1, 5, 6] | 35.254 | [1, 5, 6] | 35.170 |
| `1 2 - 4 - - 7 8 -` | 23 | 2 | [2] | 36.164 | [2] | 36.164 |
| `1 2 - 4 - - 7 8 -` | 23 | 3 | [1, 2] | 36.164 | [1, 2] | 36.164 |
| `1 2 - 4 - - 7 8 -` | 23 | 4 | [4] | 36.164 | [4] | 36.164 |
| `1 2 - 4 - - 7 8 -` | 23 | 5 | [1, 4] | 36.164 | [1, 4] | 36.164 |
| `1 2 - 4 - - 7 8 -` | 23 | 6 | [2, 4] | 36.164 | [2, 4] | 36.164 |
| `1 2 - 4 - - 7 8 -` | 23 | 7 | [7] | 36.164 | [7] | 36.164 |
| `1 2 - 4 - - 7 8 -` | 23 | 8 | [8] | 36.164 | [8] | 36.164 |
| `1 2 - 4 - - 7 8 -` | 23 | 9 | [1, 8] | 36.164 | [1, 8] | 36.164 |
| `1 2 - 4 - - 7 8 -` | 23 | 10 | [2, 8] | 36.164 | [2, 8] | 36.164 |
| `1 2 - 4 - - 7 8 -` | 23 | 11 | [4, 7] | 36.164 | [4, 7] | 36.164 |
| `1 2 - 4 - - 7 8 -` | 23 | 12 | [4, 8] | 36.164 | [4, 8] | 36.164 |
| `1 2 - 4 - 6 - - 9` | 23 | 2 | [2] | 35.202 | [2] | 35.202 |
| `1 2 - 4 - 6 - - 9` | 23 | 3 | [1, 2] | 35.202 | [1, 2] | 35.202 |
| `1 2 - 4 - 6 - - 9` | 23 | 4 | [4] | 35.202 | [4] | 35.202 |
| `1 2 - 4 - 6 - - 9` | 23 | 5 | [1, 4] | 35.202 | [1, 4] | 35.202 |
| `1 2 - 4 - 6 - - 9` | 23 | 6 | [6] | 35.202 | [6] | 35.202 |
| `1 2 - 4 - 6 - - 9` | 23 | 7 | [1, 6] | 35.202 | [1, 6] | 35.202 |
| `1 2 - 4 - 6 - - 9` | 23 | 8 | [2, 6] | 35.202 | [2, 6] | 35.202 |
| `1 2 - 4 - 6 - - 9` | 23 | 9 | [9] | 35.202 | [9] | 35.202 |
| `1 2 - 4 - 6 - - 9` | 23 | 10 | [1, 9] | 35.202 | [1, 9] | 35.202 |
| `1 2 - 4 - 6 - - 9` | 23 | 11 | [2, 9] | 35.202 | [2, 9] | 35.202 |
| `1 2 - 4 - 6 - - 9` | 23 | 12 | [1, 2, 9] | 35.202 | [1, 2, 9] | 35.202 |
| `1 2 3 - - - 7 - 9` | 23 | 2 | [2] | 34.880 | [2] | 34.880 |
| `1 2 3 - - - 7 - 9` | 23 | 3 | [3] | 34.880 | [3] | 34.880 |
| `1 2 3 - - - 7 - 9` | 23 | 4 | [1, 3] | 34.880 | [1, 3] | 34.880 |
| `1 2 3 - - - 7 - 9` | 23 | 5 | [2, 3] | 34.880 | [2, 3] | 34.880 |
| `1 2 3 - - - 7 - 9` | 23 | 6 | [1, 2, 3] | 34.880 | [1, 2, 3] | 34.880 |
| `1 2 3 - - - 7 - 9` | 23 | 7 | [7] | 34.880 | [7] | 34.880 |
| `1 2 3 - - - 7 - 9` | 23 | 8 | [1, 7] | 34.880 | [1, 7] | 34.880 |
| `1 2 3 - - - 7 - 9` | 23 | 9 | [9] | 34.880 | [9] | 34.880 |
| `1 2 3 - - - 7 - 9` | 23 | 10 | [1, 9] | 34.880 | [1, 9] | 34.880 |
| `1 2 3 - - - 7 - 9` | 23 | 11 | [2, 9] | 34.880 | [2, 9] | 34.880 |
| `1 2 3 - - - 7 - 9` | 23 | 12 | [3, 9] | 34.880 | [3, 9] | 34.880 |
| `1 2 3 4 5 - 7 - -` | 23 | 2 | [2] | 40.138 | [2] | 39.869 |
| `1 2 3 4 5 - 7 - -` | 23 | 3 | [3] | 40.138 | [3] | 39.869 |
| `1 2 3 4 5 - 7 - -` | 23 | 4 | [4] | 40.138 | [4] | 39.869 |
| `1 2 3 4 5 - 7 - -` | 23 | 5 | [5] | 40.138 | [5] | 39.869 |
| `1 2 3 4 5 - 7 - -` | 23 | 6 | [2, 4] | 40.138 | [1, 5] | 39.869 |
| `1 2 3 4 5 - 7 - -` | 23 | 7 | [7] | 40.138 | [7] | 39.869 |
| `1 2 3 4 5 - 7 - -` | 23 | 8 | [1, 7] | 40.138 | [1, 7] | 39.869 |
| `1 2 3 4 5 - 7 - -` | 23 | 9 | [4, 5] | 40.138 | [2, 7] | 39.869 |
| `1 2 3 4 5 - 7 - -` | 23 | 10 | [3, 7] | 40.138 | [3, 7] | 39.869 |
| `1 2 3 4 5 - 7 - -` | 23 | 11 | [4, 7] | 40.138 | [4, 7] | 39.869 |
| `1 2 3 4 5 - 7 - -` | 23 | 12 | [5, 7] | 40.138 | [5, 7] | 39.869 |
| `- - - - - 6 7 8 -` | 24 | 6 | [6] | 28.164 | [6] | 28.164 |
| `- - - - - 6 7 8 -` | 24 | 7 | [7] | 28.164 | [7] | 28.164 |
| `- - - - - 6 7 8 -` | 24 | 8 | [8] | 28.164 | [8] | 28.164 |
| `- - - - 5 - 7 - 9` | 24 | 5 | [5] | 27.898 | [5] | 27.898 |
| `- - - - 5 - 7 - 9` | 24 | 7 | [7] | 27.898 | [7] | 27.898 |
| `- - - - 5 - 7 - 9` | 24 | 9 | [9] | 27.898 | [9] | 27.898 |
| `- - - - 5 - 7 - 9` | 24 | 12 | [5, 7] | 27.898 | [5, 7] | 27.898 |
| `- - - 4 - - - 8 9` | 24 | 4 | [4] | 27.418 | [4] | 27.418 |
| `- - - 4 - - - 8 9` | 24 | 8 | [8] | 27.418 | [8] | 27.418 |
| `- - - 4 - - - 8 9` | 24 | 9 | [9] | 27.418 | [9] | 27.418 |
| `- - - 4 - - - 8 9` | 24 | 12 | [4, 8] | 27.418 | [4, 8] | 27.418 |
| `- - 3 - 5 6 7 - -` | 24 | 3 | [3] | 33.903 | [3] | 33.903 |
| `- - 3 - 5 6 7 - -` | 24 | 5 | [5] | 33.903 | [5] | 33.903 |
| `- - 3 - 5 6 7 - -` | 24 | 6 | [6] | 33.903 | [6] | 33.903 |
| `- - 3 - 5 6 7 - -` | 24 | 7 | [7] | 33.903 | [7] | 33.903 |
| `- - 3 - 5 6 7 - -` | 24 | 8 | [3, 5] | 33.903 | [3, 5] | 33.903 |
| `- - 3 - 5 6 7 - -` | 24 | 9 | [3, 6] | 33.903 | [3, 6] | 33.903 |
| `- - 3 - 5 6 7 - -` | 24 | 10 | [3, 7] | 33.903 | [3, 7] | 33.903 |
| `- - 3 - 5 6 7 - -` | 24 | 11 | [5, 6] | 33.903 | [5, 6] | 33.903 |
| `- - 3 - 5 6 7 - -` | 24 | 12 | [5, 7] | 33.903 | [5, 7] | 33.903 |
| `- - 3 4 - 6 - 8 -` | 24 | 3 | [3] | 33.249 | [3] | 33.249 |
| `- - 3 4 - 6 - 8 -` | 24 | 4 | [4] | 33.249 | [4] | 33.249 |
| `- - 3 4 - 6 - 8 -` | 24 | 6 | [6] | 33.249 | [6] | 33.249 |
| `- - 3 4 - 6 - 8 -` | 24 | 7 | [3, 4] | 33.249 | [3, 4] | 33.249 |
| `- - 3 4 - 6 - 8 -` | 24 | 8 | [8] | 33.249 | [8] | 33.249 |
| `- - 3 4 - 6 - 8 -` | 24 | 9 | [3, 6] | 33.249 | [3, 6] | 33.249 |
| `- - 3 4 - 6 - 8 -` | 24 | 10 | [4, 6] | 33.249 | [4, 6] | 33.249 |
| `- - 3 4 - 6 - 8 -` | 24 | 11 | [3, 8] | 33.249 | [3, 8] | 33.249 |
| `- - 3 4 - 6 - 8 -` | 24 | 12 | [4, 8] | 33.249 | [4, 8] | 33.249 |
| `- - 3 4 5 - - - 9` | 24 | 3 | [3] | 30.646 | [3] | 30.646 |
| `- - 3 4 5 - - - 9` | 24 | 4 | [4] | 30.646 | [4] | 30.646 |
| `- - 3 4 5 - - - 9` | 24 | 5 | [5] | 30.646 | [5] | 30.646 |
| `- - 3 4 5 - - - 9` | 24 | 7 | [3, 4] | 30.646 | [3, 4] | 30.646 |
| `- - 3 4 5 - - - 9` | 24 | 8 | [3, 5] | 30.646 | [3, 5] | 30.646 |
| `- - 3 4 5 - - - 9` | 24 | 9 | [9] | 30.646 | [9] | 30.646 |
| `- - 3 4 5 - - - 9` | 24 | 12 | [3, 9] | 30.646 | [3, 9] | 30.646 |
| `- 2 - - 5 6 - 8 -` | 24 | 2 | [2] | 31.662 | [2] | 31.662 |
| `- 2 - - 5 6 - 8 -` | 24 | 5 | [5] | 31.662 | [5] | 31.662 |
| `- 2 - - 5 6 - 8 -` | 24 | 6 | [6] | 31.662 | [6] | 31.662 |
| `- 2 - - 5 6 - 8 -` | 24 | 7 | [2, 5] | 31.662 | [2, 5] | 31.662 |
| `- 2 - - 5 6 - 8 -` | 24 | 8 | [8] | 31.662 | [8] | 31.662 |
| `- 2 - - 5 6 - 8 -` | 24 | 10 | [2, 8] | 31.662 | [2, 8] | 31.662 |
| `- 2 - - 5 6 - 8 -` | 24 | 11 | [5, 6] | 31.662 | [5, 6] | 31.662 |
| `- 2 - 4 - - 7 8 -` | 24 | 2 | [2] | 33.098 | [2] | 33.098 |
| `- 2 - 4 - - 7 8 -` | 24 | 4 | [4] | 33.098 | [4] | 33.098 |
| `- 2 - 4 - - 7 8 -` | 24 | 6 | [2, 4] | 33.098 | [2, 4] | 33.098 |
| `- 2 - 4 - - 7 8 -` | 24 | 7 | [7] | 33.098 | [7] | 33.098 |
| `- 2 - 4 - - 7 8 -` | 24 | 8 | [8] | 33.098 | [8] | 33.098 |
| `- 2 - 4 - - 7 8 -` | 24 | 9 | [2, 7] | 33.098 | [2, 7] | 33.098 |
| `- 2 - 4 - - 7 8 -` | 24 | 10 | [2, 8] | 33.098 | [2, 8] | 33.098 |
| `- 2 - 4 - - 7 8 -` | 24 | 11 | [4, 7] | 33.098 | [4, 7] | 33.098 |
| `- 2 - 4 - - 7 8 -` | 24 | 12 | [4, 8] | 33.098 | [4, 8] | 33.098 |
| `- 2 - 4 - 6 - - 9` | 24 | 2 | [2] | 30.961 | [2] | 30.961 |
| `- 2 - 4 - 6 - - 9` | 24 | 4 | [4] | 30.961 | [4] | 30.961 |
| `- 2 - 4 - 6 - - 9` | 24 | 6 | [6] | 30.961 | [6] | 30.961 |
| `- 2 - 4 - 6 - - 9` | 24 | 8 | [2, 6] | 30.961 | [2, 6] | 30.961 |
| `- 2 - 4 - 6 - - 9` | 24 | 9 | [9] | 30.961 | [9] | 30.961 |
| `- 2 - 4 - 6 - - 9` | 24 | 10 | [4, 6] | 30.961 | [4, 6] | 30.961 |
| `- 2 - 4 - 6 - - 9` | 24 | 11 | [2, 9] | 30.961 | [2, 9] | 30.961 |
| `- 2 - 4 - 6 - - 9` | 24 | 12 | [2, 4, 6] | 30.961 | [2, 4, 6] | 30.961 |
| `- 2 3 - - - 7 - 9` | 24 | 2 | [2] | 30.660 | [2] | 30.660 |
| `- 2 3 - - - 7 - 9` | 24 | 3 | [3] | 30.660 | [3] | 30.660 |
| `- 2 3 - - - 7 - 9` | 24 | 5 | [2, 3] | 30.660 | [2, 3] | 30.660 |
| `- 2 3 - - - 7 - 9` | 24 | 7 | [7] | 30.660 | [7] | 30.660 |
| `- 2 3 - - - 7 - 9` | 24 | 9 | [9] | 30.660 | [9] | 30.660 |
| `- 2 3 - - - 7 - 9` | 24 | 10 | [3, 7] | 30.660 | [3, 7] | 30.660 |
| `- 2 3 - - - 7 - 9` | 24 | 11 | [2, 9] | 30.660 | [2, 9] | 30.660 |
| `- 2 3 - - - 7 - 9` | 24 | 12 | [3, 9] | 30.660 | [3, 9] | 30.660 |
| `- 2 3 4 5 - 7 - -` | 24 | 2 | [2] | 37.432 | [2] | 37.428 |
| `- 2 3 4 5 - 7 - -` | 24 | 3 | [3] | 37.432 | [3] | 37.428 |
| `- 2 3 4 5 - 7 - -` | 24 | 4 | [4] | 37.432 | [4] | 37.428 |
| `- 2 3 4 5 - 7 - -` | 24 | 5 | [5] | 37.432 | [5] | 37.428 |
| `- 2 3 4 5 - 7 - -` | 24 | 6 | [2, 4] | 37.432 | [2, 4] | 37.428 |
| `- 2 3 4 5 - 7 - -` | 24 | 7 | [7] | 37.432 | [7] | 37.428 |
| `- 2 3 4 5 - 7 - -` | 24 | 8 | [3, 5] | 37.432 | [3, 5] | 37.428 |
| `- 2 3 4 5 - 7 - -` | 24 | 9 | [2, 7] | 37.432 | [2, 7] | 37.428 |
| `- 2 3 4 5 - 7 - -` | 24 | 10 | [3, 7] | 37.432 | [3, 7] | 37.428 |
| `- 2 3 4 5 - 7 - -` | 24 | 11 | [4, 7] | 37.432 | [4, 7] | 37.428 |
| `- 2 3 4 5 - 7 - -` | 24 | 12 | [5, 7] | 37.432 | [5, 7] | 37.428 |
| `1 - - - 5 - 7 8 -` | 24 | 5 | [5] | 31.476 | [5] | 31.476 |
| `1 - - - 5 - 7 8 -` | 24 | 6 | [1, 5] | 31.476 | [1, 5] | 31.476 |
| `1 - - - 5 - 7 8 -` | 24 | 7 | [7] | 31.476 | [7] | 31.476 |
| `1 - - - 5 - 7 8 -` | 24 | 8 | [8] | 31.476 | [8] | 31.476 |
| `1 - - - 5 - 7 8 -` | 24 | 9 | [1, 8] | 31.476 | [1, 8] | 31.476 |
| `1 - - - 5 - 7 8 -` | 24 | 12 | [5, 7] | 31.476 | [5, 7] | 31.476 |
| `1 - - - 5 6 - - 9` | 24 | 5 | [5] | 31.405 | [5] | 31.405 |
| `1 - - - 5 6 - - 9` | 24 | 6 | [6] | 31.405 | [6] | 31.405 |
| `1 - - - 5 6 - - 9` | 24 | 7 | [1, 6] | 31.405 | [1, 6] | 31.405 |
| `1 - - - 5 6 - - 9` | 24 | 9 | [9] | 31.405 | [9] | 31.405 |
| `1 - - - 5 6 - - 9` | 24 | 10 | [1, 9] | 31.405 | [1, 9] | 31.405 |
| `1 - - - 5 6 - - 9` | 24 | 11 | [5, 6] | 31.405 | [5, 6] | 31.405 |
| `1 - - - 5 6 - - 9` | 24 | 12 | [1, 5, 6] | 31.405 | [1, 5, 6] | 31.405 |
| `1 - - 4 - - 7 - 9` | 24 | 4 | [4] | 32.180 | [4] | 32.180 |
| `1 - - 4 - - 7 - 9` | 24 | 5 | [1, 4] | 32.180 | [1, 4] | 32.180 |
| `1 - - 4 - - 7 - 9` | 24 | 7 | [7] | 32.180 | [7] | 32.180 |
| `1 - - 4 - - 7 - 9` | 24 | 8 | [1, 7] | 32.180 | [1, 7] | 32.180 |
| `1 - - 4 - - 7 - 9` | 24 | 9 | [9] | 32.180 | [9] | 32.180 |
| `1 - - 4 - - 7 - 9` | 24 | 10 | [1, 9] | 32.180 | [1, 9] | 32.180 |
| `1 - - 4 - - 7 - 9` | 24 | 11 | [4, 7] | 32.180 | [4, 7] | 32.180 |
| `1 - - 4 - - 7 - 9` | 24 | 12 | [1, 4, 7] | 32.180 | [1, 4, 7] | 32.180 |
| `1 - 3 - - - - 8 9` | 24 | 3 | [3] | 29.941 | [3] | 29.941 |
| `1 - 3 - - - - 8 9` | 24 | 4 | [1, 3] | 29.941 | [1, 3] | 29.941 |
| `1 - 3 - - - - 8 9` | 24 | 8 | [8] | 29.941 | [8] | 29.941 |
| `1 - 3 - - - - 8 9` | 24 | 9 | [9] | 29.941 | [9] | 29.941 |
| `1 - 3 - - - - 8 9` | 24 | 10 | [1, 9] | 29.941 | [1, 9] | 29.941 |
| `1 - 3 - - - - 8 9` | 24 | 11 | [3, 8] | 29.941 | [3, 8] | 29.941 |
| `1 - 3 - - - - 8 9` | 24 | 12 | [3, 9] | 29.941 | [3, 9] | 29.941 |
| `1 - 3 4 - 6 7 - -` | 24 | 3 | [3] | 36.999 | [3] | 36.990 |
| `1 - 3 4 - 6 7 - -` | 24 | 4 | [4] | 36.999 | [4] | 36.990 |
| `1 - 3 4 - 6 7 - -` | 24 | 5 | [1, 4] | 36.999 | [1, 4] | 36.990 |
| `1 - 3 4 - 6 7 - -` | 24 | 6 | [6] | 36.999 | [6] | 36.990 |
| `1 - 3 4 - 6 7 - -` | 24 | 7 | [7] | 36.999 | [7] | 36.990 |
| `1 - 3 4 - 6 7 - -` | 24 | 8 | [1, 7] | 36.999 | [1, 7] | 36.990 |
| `1 - 3 4 - 6 7 - -` | 24 | 9 | [3, 6] | 36.999 | [3, 6] | 36.990 |
| `1 - 3 4 - 6 7 - -` | 24 | 10 | [3, 7] | 36.999 | [3, 7] | 36.990 |
| `1 - 3 4 - 6 7 - -` | 24 | 11 | [4, 7] | 36.999 | [4, 7] | 36.990 |
| `1 - 3 4 - 6 7 - -` | 24 | 12 | [1, 4, 7] | 36.999 | [1, 4, 7] | 36.990 |
| `1 - 3 4 5 - - 8 -` | 24 | 3 | [3] | 36.185 | [3] | 36.184 |
| `1 - 3 4 5 - - 8 -` | 24 | 4 | [4] | 36.185 | [4] | 36.184 |
| `1 - 3 4 5 - - 8 -` | 24 | 5 | [5] | 36.185 | [5] | 36.184 |
| `1 - 3 4 5 - - 8 -` | 24 | 6 | [1, 5] | 36.185 | [1, 5] | 36.184 |
| `1 - 3 4 5 - - 8 -` | 24 | 7 | [3, 4] | 36.185 | [3, 4] | 36.184 |
| `1 - 3 4 5 - - 8 -` | 24 | 8 | [8] | 36.185 | [8] | 36.184 |
| `1 - 3 4 5 - - 8 -` | 24 | 9 | [1, 8] | 36.185 | [1, 8] | 36.184 |
| `1 - 3 4 5 - - 8 -` | 24 | 10 | [1, 4, 5] | 36.185 | [1, 4, 5] | 36.184 |
| `1 - 3 4 5 - - 8 -` | 24 | 11 | [3, 8] | 36.185 | [3, 8] | 36.184 |
| `1 - 3 4 5 - - 8 -` | 24 | 12 | [4, 8] | 36.185 | [4, 8] | 36.184 |
| `1 2 - - 5 6 7 - -` | 24 | 2 | [2] | 36.068 | [2] | 36.059 |
| `1 2 - - 5 6 7 - -` | 24 | 3 | [1, 2] | 36.068 | [1, 2] | 36.059 |
| `1 2 - - 5 6 7 - -` | 24 | 5 | [5] | 36.068 | [5] | 36.059 |
| `1 2 - - 5 6 7 - -` | 24 | 6 | [6] | 36.068 | [6] | 36.059 |
| `1 2 - - 5 6 7 - -` | 24 | 7 | [7] | 36.068 | [7] | 36.059 |
| `1 2 - - 5 6 7 - -` | 24 | 8 | [1, 7] | 36.068 | [1, 7] | 36.059 |
| `1 2 - - 5 6 7 - -` | 24 | 9 | [2, 7] | 36.068 | [2, 7] | 36.059 |
| `1 2 - - 5 6 7 - -` | 24 | 10 | [1, 2, 7] | 36.068 | [1, 2, 7] | 36.059 |
| `1 2 - - 5 6 7 - -` | 24 | 11 | [5, 6] | 36.068 | [5, 6] | 36.059 |
| `1 2 - - 5 6 7 - -` | 24 | 12 | [5, 7] | 36.068 | [5, 7] | 36.059 |
| `1 2 - 4 - 6 - 8 -` | 24 | 2 | [2] | 36.763 | [2] | 36.763 |
| `1 2 - 4 - 6 - 8 -` | 24 | 3 | [1, 2] | 36.763 | [1, 2] | 36.763 |
| `1 2 - 4 - 6 - 8 -` | 24 | 4 | [4] | 36.763 | [4] | 36.763 |
| `1 2 - 4 - 6 - 8 -` | 24 | 5 | [1, 4] | 36.763 | [1, 4] | 36.763 |
| `1 2 - 4 - 6 - 8 -` | 24 | 6 | [6] | 36.763 | [6] | 36.763 |
| `1 2 - 4 - 6 - 8 -` | 24 | 7 | [1, 6] | 36.763 | [1, 6] | 36.763 |
| `1 2 - 4 - 6 - 8 -` | 24 | 8 | [8] | 36.763 | [8] | 36.763 |
| `1 2 - 4 - 6 - 8 -` | 24 | 9 | [1, 8] | 36.763 | [1, 8] | 36.763 |
| `1 2 - 4 - 6 - 8 -` | 24 | 10 | [2, 8] | 36.763 | [2, 8] | 36.763 |
| `1 2 - 4 - 6 - 8 -` | 24 | 11 | [1, 2, 8] | 36.763 | [1, 2, 8] | 36.763 |
| `1 2 - 4 - 6 - 8 -` | 24 | 12 | [4, 8] | 36.763 | [4, 8] | 36.763 |
| `1 2 - 4 5 - - - 9` | 24 | 2 | [2] | 35.670 | [2] | 35.604 |
| `1 2 - 4 5 - - - 9` | 24 | 3 | [1, 2] | 35.670 | [1, 2] | 35.604 |
| `1 2 - 4 5 - - - 9` | 24 | 4 | [4] | 35.670 | [4] | 35.604 |
| `1 2 - 4 5 - - - 9` | 24 | 5 | [5] | 35.670 | [5] | 35.604 |
| `1 2 - 4 5 - - - 9` | 24 | 6 | [2, 4] | 35.670 | [1, 5] | 35.604 |
| `1 2 - 4 5 - - - 9` | 24 | 7 | [2, 5] | 35.670 | [2, 5] | 35.604 |
| `1 2 - 4 5 - - - 9` | 24 | 8 | [1, 2, 5] | 35.670 | [1, 2, 5] | 35.604 |
| `1 2 - 4 5 - - - 9` | 24 | 9 | [9] | 35.670 | [9] | 35.604 |
| `1 2 - 4 5 - - - 9` | 24 | 10 | [1, 9] | 35.670 | [1, 9] | 35.604 |
| `1 2 - 4 5 - - - 9` | 24 | 11 | [2, 9] | 35.670 | [2, 9] | 35.604 |
| `1 2 - 4 5 - - - 9` | 24 | 12 | [1, 2, 9] | 35.670 | [1, 2, 9] | 35.604 |
| `1 2 3 - - - 7 8 -` | 24 | 2 | [2] | 36.421 | [2] | 36.421 |
| `1 2 3 - - - 7 8 -` | 24 | 3 | [3] | 36.421 | [3] | 36.421 |
| `1 2 3 - - - 7 8 -` | 24 | 4 | [1, 3] | 36.421 | [1, 3] | 36.421 |
| `1 2 3 - - - 7 8 -` | 24 | 5 | [2, 3] | 36.421 | [2, 3] | 36.421 |
| `1 2 3 - - - 7 8 -` | 24 | 6 | [1, 2, 3] | 36.421 | [1, 2, 3] | 36.421 |
| `1 2 3 - - - 7 8 -` | 24 | 7 | [7] | 36.421 | [7] | 36.421 |
| `1 2 3 - - - 7 8 -` | 24 | 8 | [8] | 36.421 | [8] | 36.421 |
| `1 2 3 - - - 7 8 -` | 24 | 9 | [1, 8] | 36.421 | [1, 8] | 36.421 |
| `1 2 3 - - - 7 8 -` | 24 | 10 | [2, 8] | 36.421 | [2, 8] | 36.421 |
| `1 2 3 - - - 7 8 -` | 24 | 11 | [3, 8] | 36.421 | [3, 8] | 36.421 |
| `1 2 3 - - - 7 8 -` | 24 | 12 | [1, 3, 8] | 36.421 | [1, 3, 8] | 36.421 |
| `1 2 3 - - 6 - - 9` | 24 | 2 | [2] | 35.911 | [2] | 35.911 |
| `1 2 3 - - 6 - - 9` | 24 | 3 | [3] | 35.911 | [3] | 35.911 |
| `1 2 3 - - 6 - - 9` | 24 | 4 | [1, 3] | 35.911 | [1, 3] | 35.911 |
| `1 2 3 - - 6 - - 9` | 24 | 5 | [2, 3] | 35.911 | [2, 3] | 35.911 |
| `1 2 3 - - 6 - - 9` | 24 | 6 | [6] | 35.911 | [6] | 35.911 |
| `1 2 3 - - 6 - - 9` | 24 | 7 | [1, 6] | 35.911 | [1, 6] | 35.911 |
| `1 2 3 - - 6 - - 9` | 24 | 8 | [2, 6] | 35.911 | [2, 6] | 35.911 |
| `1 2 3 - - 6 - - 9` | 24 | 9 | [9] | 35.911 | [9] | 35.911 |
| `1 2 3 - - 6 - - 9` | 24 | 10 | [1, 9] | 35.911 | [1, 9] | 35.911 |
| `1 2 3 - - 6 - - 9` | 24 | 11 | [2, 9] | 35.911 | [2, 9] | 35.911 |
| `1 2 3 - - 6 - - 9` | 24 | 12 | [3, 9] | 35.911 | [3, 9] | 35.911 |
| `1 2 3 4 5 6 - - -` | 24 | 2 | [2] | 40.698 | [2] | 40.554 |
| `1 2 3 4 5 6 - - -` | 24 | 3 | [3] | 40.698 | [3] | 40.554 |
| `1 2 3 4 5 6 - - -` | 24 | 4 | [4] | 40.698 | [4] | 40.554 |
| `1 2 3 4 5 6 - - -` | 24 | 5 | [5] | 40.698 | [5] | 40.554 |
| `1 2 3 4 5 6 - - -` | 24 | 6 | [6] | 40.698 | [6] | 40.554 |
| `1 2 3 4 5 6 - - -` | 24 | 7 | [1, 6] | 40.698 | [1, 6] | 40.554 |
| `1 2 3 4 5 6 - - -` | 24 | 8 | [3, 5] | 40.698 | [2, 6] | 40.554 |
| `1 2 3 4 5 6 - - -` | 24 | 9 | [3, 6] | 40.698 | [3, 6] | 40.554 |
| `1 2 3 4 5 6 - - -` | 24 | 10 | [4, 6] | 40.698 | [4, 6] | 40.554 |
| `1 2 3 4 5 6 - - -` | 24 | 11 | [5, 6] | 40.698 | [5, 6] | 40.554 |
| `1 2 3 4 5 6 - - -` | 24 | 12 | [3, 4, 5] | 40.698 | [1, 5, 6] | 40.554 |
| `- - - - 5 - 7 8 -` | 25 | 5 | [5] | 29.117 | [5] | 29.117 |
| `- - - - 5 - 7 8 -` | 25 | 7 | [7] | 29.117 | [7] | 29.117 |
| `- - - - 5 - 7 8 -` | 25 | 8 | [8] | 29.117 | [8] | 29.117 |
| `- - - - 5 - 7 8 -` | 25 | 12 | [5, 7] | 29.117 | [5, 7] | 29.117 |
| `- - - - 5 6 - - 9` | 25 | 5 | [5] | 28.766 | [5] | 28.766 |
| `- - - - 5 6 - - 9` | 25 | 6 | [6] | 28.766 | [6] | 28.766 |
| `- - - - 5 6 - - 9` | 25 | 9 | [9] | 28.766 | [9] | 28.766 |
| `- - - - 5 6 - - 9` | 25 | 11 | [5, 6] | 28.766 | [5, 6] | 28.766 |
| `- - - 4 - - 7 - 9` | 25 | 4 | [4] | 28.866 | [4] | 28.866 |
| `- - - 4 - - 7 - 9` | 25 | 7 | [7] | 28.866 | [7] | 28.866 |
| `- - - 4 - - 7 - 9` | 25 | 9 | [9] | 28.866 | [9] | 28.866 |
| `- - - 4 - - 7 - 9` | 25 | 11 | [4, 7] | 28.866 | [4, 7] | 28.866 |
| `- - 3 - - - - 8 9` | 25 | 3 | [3] | 28.878 | [3] | 28.878 |
| `- - 3 - - - - 8 9` | 25 | 8 | [8] | 28.878 | [8] | 28.878 |
| `- - 3 - - - - 8 9` | 25 | 9 | [9] | 28.878 | [9] | 28.878 |
| `- - 3 - - - - 8 9` | 25 | 11 | [3, 8] | 28.878 | [3, 8] | 28.878 |
| `- - 3 - - - - 8 9` | 25 | 12 | [3, 9] | 28.878 | [3, 9] | 28.878 |
| `- - 3 4 - 6 7 - -` | 25 | 3 | [3] | 32.542 | [3] | 32.530 |
| `- - 3 4 - 6 7 - -` | 25 | 4 | [4] | 32.542 | [4] | 32.530 |
| `- - 3 4 - 6 7 - -` | 25 | 6 | [6] | 32.542 | [6] | 32.530 |
| `- - 3 4 - 6 7 - -` | 25 | 7 | [7] | 32.542 | [7] | 32.530 |
| `- - 3 4 - 6 7 - -` | 25 | 9 | [3, 6] | 32.542 | [3, 6] | 32.530 |
| `- - 3 4 - 6 7 - -` | 25 | 10 | [4, 6] | 32.542 | [3, 7] | 32.530 |
| `- - 3 4 - 6 7 - -` | 25 | 11 | [4, 7] | 32.542 | [4, 7] | 32.530 |
| `- - 3 4 5 - - 8 -` | 25 | 3 | [3] | 32.680 | [3] | 32.680 |
| `- - 3 4 5 - - 8 -` | 25 | 4 | [4] | 32.680 | [4] | 32.680 |
| `- - 3 4 5 - - 8 -` | 25 | 5 | [5] | 32.680 | [5] | 32.680 |
| `- - 3 4 5 - - 8 -` | 25 | 7 | [3, 4] | 32.680 | [3, 4] | 32.680 |
| `- - 3 4 5 - - 8 -` | 25 | 8 | [8] | 32.680 | [8] | 32.680 |
| `- - 3 4 5 - - 8 -` | 25 | 9 | [4, 5] | 32.680 | [4, 5] | 32.680 |
| `- - 3 4 5 - - 8 -` | 25 | 11 | [3, 8] | 32.680 | [3, 8] | 32.680 |
| `- - 3 4 5 - - 8 -` | 25 | 12 | [4, 8] | 32.680 | [4, 8] | 32.680 |
| `- 2 - - 5 6 7 - -` | 25 | 2 | [2] | 33.477 | [2] | 33.477 |
| `- 2 - - 5 6 7 - -` | 25 | 5 | [5] | 33.477 | [5] | 33.477 |
| `- 2 - - 5 6 7 - -` | 25 | 6 | [6] | 33.477 | [6] | 33.477 |
| `- 2 - - 5 6 7 - -` | 25 | 7 | [7] | 33.477 | [7] | 33.477 |
| `- 2 - - 5 6 7 - -` | 25 | 8 | [2, 6] | 33.477 | [2, 6] | 33.477 |
| `- 2 - - 5 6 7 - -` | 25 | 9 | [2, 7] | 33.477 | [2, 7] | 33.477 |
| `- 2 - - 5 6 7 - -` | 25 | 11 | [5, 6] | 33.477 | [5, 6] | 33.477 |
| `- 2 - - 5 6 7 - -` | 25 | 12 | [5, 7] | 33.477 | [5, 7] | 33.477 |
| `- 2 - 4 - 6 - 8 -` | 25 | 2 | [2] | 30.262 | [2] | 30.262 |
| `- 2 - 4 - 6 - 8 -` | 25 | 4 | [4] | 30.262 | [4] | 30.262 |
| `- 2 - 4 - 6 - 8 -` | 25 | 6 | [6] | 30.262 | [6] | 30.262 |
| `- 2 - 4 - 6 - 8 -` | 25 | 8 | [8] | 30.262 | [8] | 30.262 |
| `- 2 - 4 - 6 - 8 -` | 25 | 10 | [2, 8] | 30.262 | [2, 8] | 30.262 |
| `- 2 - 4 - 6 - 8 -` | 25 | 12 | [4, 8] | 30.262 | [4, 8] | 30.262 |
| `- 2 - 4 5 - - - 9` | 25 | 2 | [2] | 31.477 | [2] | 31.477 |
| `- 2 - 4 5 - - - 9` | 25 | 4 | [4] | 31.477 | [4] | 31.477 |
| `- 2 - 4 5 - - - 9` | 25 | 5 | [5] | 31.477 | [5] | 31.477 |
| `- 2 - 4 5 - - - 9` | 25 | 6 | [2, 4] | 31.477 | [2, 4] | 31.477 |
| `- 2 - 4 5 - - - 9` | 25 | 7 | [2, 5] | 31.477 | [2, 5] | 31.477 |
| `- 2 - 4 5 - - - 9` | 25 | 9 | [9] | 31.477 | [9] | 31.477 |
| `- 2 - 4 5 - - - 9` | 25 | 11 | [2, 9] | 31.477 | [2, 9] | 31.477 |
| `- 2 3 - - - 7 8 -` | 25 | 2 | [2] | 33.369 | [2] | 33.369 |
| `- 2 3 - - - 7 8 -` | 25 | 3 | [3] | 33.369 | [3] | 33.369 |
| `- 2 3 - - - 7 8 -` | 25 | 5 | [2, 3] | 33.369 | [2, 3] | 33.369 |
| `- 2 3 - - - 7 8 -` | 25 | 7 | [7] | 33.369 | [7] | 33.369 |
| `- 2 3 - - - 7 8 -` | 25 | 8 | [8] | 33.369 | [8] | 33.369 |
| `- 2 3 - - - 7 8 -` | 25 | 9 | [2, 7] | 33.369 | [2, 7] | 33.369 |
| `- 2 3 - - - 7 8 -` | 25 | 10 | [2, 8] | 33.369 | [2, 8] | 33.369 |
| `- 2 3 - - - 7 8 -` | 25 | 11 | [3, 8] | 33.369 | [3, 8] | 33.369 |
| `- 2 3 - - - 7 8 -` | 25 | 12 | [2, 3, 7] | 33.369 | [2, 3, 7] | 33.369 |
| `- 2 3 - - 6 - - 9` | 25 | 2 | [2] | 31.559 | [2] | 31.559 |
| `- 2 3 - - 6 - - 9` | 25 | 3 | [3] | 31.559 | [3] | 31.559 |
| `- 2 3 - - 6 - - 9` | 25 | 5 | [2, 3] | 31.559 | [2, 3] | 31.559 |
| `- 2 3 - - 6 - - 9` | 25 | 6 | [6] | 31.559 | [6] | 31.559 |
| `- 2 3 - - 6 - - 9` | 25 | 8 | [2, 6] | 31.559 | [2, 6] | 31.559 |
| `- 2 3 - - 6 - - 9` | 25 | 9 | [9] | 31.559 | [9] | 31.559 |
| `- 2 3 - - 6 - - 9` | 25 | 11 | [2, 9] | 31.559 | [2, 9] | 31.559 |
| `- 2 3 - - 6 - - 9` | 25 | 12 | [3, 9] | 31.559 | [3, 9] | 31.559 |
| `- 2 3 4 5 6 - - -` | 25 | 2 | [2] | 38.645 | [2] | 38.630 |
| `- 2 3 4 5 6 - - -` | 25 | 3 | [3] | 38.645 | [3] | 38.630 |
| `- 2 3 4 5 6 - - -` | 25 | 4 | [4] | 38.645 | [4] | 38.630 |
| `- 2 3 4 5 6 - - -` | 25 | 5 | [5] | 38.645 | [5] | 38.630 |
| `- 2 3 4 5 6 - - -` | 25 | 6 | [6] | 38.645 | [6] | 38.630 |
| `- 2 3 4 5 6 - - -` | 25 | 7 | [3, 4] | 38.645 | [2, 5] | 38.630 |
| `- 2 3 4 5 6 - - -` | 25 | 8 | [2, 6] | 38.645 | [2, 6] | 38.630 |
| `- 2 3 4 5 6 - - -` | 25 | 9 | [3, 6] | 38.645 | [3, 6] | 38.630 |
| `- 2 3 4 5 6 - - -` | 25 | 10 | [4, 6] | 38.645 | [4, 6] | 38.630 |
| `- 2 3 4 5 6 - - -` | 25 | 11 | [5, 6] | 38.645 | [5, 6] | 38.630 |
| `- 2 3 4 5 6 - - -` | 25 | 12 | [3, 4, 5] | 38.645 | [2, 4, 6] | 38.630 |
| `1 - - - 5 6 - 8 -` | 25 | 5 | [5] | 32.995 | [5] | 32.995 |
| `1 - - - 5 6 - 8 -` | 25 | 6 | [6] | 32.995 | [6] | 32.995 |
| `1 - - - 5 6 - 8 -` | 25 | 7 | [1, 6] | 32.995 | [1, 6] | 32.995 |
| `1 - - - 5 6 - 8 -` | 25 | 8 | [8] | 32.995 | [8] | 32.995 |
| `1 - - - 5 6 - 8 -` | 25 | 9 | [1, 8] | 32.995 | [1, 8] | 32.995 |
| `1 - - - 5 6 - 8 -` | 25 | 11 | [5, 6] | 32.995 | [5, 6] | 32.995 |
| `1 - - - 5 6 - 8 -` | 25 | 12 | [1, 5, 6] | 32.995 | [1, 5, 6] | 32.995 |
| `1 - - 4 - - 7 8 -` | 25 | 4 | [4] | 32.449 | [4] | 32.449 |
| `1 - - 4 - - 7 8 -` | 25 | 5 | [1, 4] | 32.449 | [1, 4] | 32.449 |
| `1 - - 4 - - 7 8 -` | 25 | 7 | [7] | 32.449 | [7] | 32.449 |
| `1 - - 4 - - 7 8 -` | 25 | 8 | [8] | 32.449 | [8] | 32.449 |
| `1 - - 4 - - 7 8 -` | 25 | 9 | [1, 8] | 32.449 | [1, 8] | 32.449 |
| `1 - - 4 - - 7 8 -` | 25 | 11 | [4, 7] | 32.449 | [4, 7] | 32.449 |
| `1 - - 4 - - 7 8 -` | 25 | 12 | [4, 8] | 32.449 | [4, 8] | 32.449 |
| `1 - - 4 - 6 - - 9` | 25 | 4 | [4] | 32.378 | [4] | 32.378 |
| `1 - - 4 - 6 - - 9` | 25 | 5 | [1, 4] | 32.378 | [1, 4] | 32.378 |
| `1 - - 4 - 6 - - 9` | 25 | 6 | [6] | 32.378 | [6] | 32.378 |
| `1 - - 4 - 6 - - 9` | 25 | 7 | [1, 6] | 32.378 | [1, 6] | 32.378 |
| `1 - - 4 - 6 - - 9` | 25 | 9 | [9] | 32.378 | [9] | 32.378 |
| `1 - - 4 - 6 - - 9` | 25 | 10 | [1, 9] | 32.378 | [1, 9] | 32.378 |
| `1 - - 4 - 6 - - 9` | 25 | 11 | [1, 4, 6] | 32.378 | [1, 4, 6] | 32.378 |
| `1 - 3 - - - 7 - 9` | 25 | 3 | [3] | 32.552 | [3] | 32.552 |
| `1 - 3 - - - 7 - 9` | 25 | 4 | [1, 3] | 32.552 | [1, 3] | 32.552 |
| `1 - 3 - - - 7 - 9` | 25 | 7 | [7] | 32.552 | [7] | 32.552 |
| `1 - 3 - - - 7 - 9` | 25 | 8 | [1, 7] | 32.552 | [1, 7] | 32.552 |
| `1 - 3 - - - 7 - 9` | 25 | 9 | [9] | 32.552 | [9] | 32.552 |
| `1 - 3 - - - 7 - 9` | 25 | 10 | [1, 9] | 32.552 | [1, 9] | 32.552 |
| `1 - 3 - - - 7 - 9` | 25 | 11 | [1, 3, 7] | 32.552 | [1, 3, 7] | 32.552 |
| `1 - 3 - - - 7 - 9` | 25 | 12 | [3, 9] | 32.552 | [3, 9] | 32.552 |
| `1 - 3 4 5 - 7 - -` | 25 | 3 | [3] | 37.756 | [3] | 37.751 |
| `1 - 3 4 5 - 7 - -` | 25 | 4 | [4] | 37.756 | [4] | 37.751 |
| `1 - 3 4 5 - 7 - -` | 25 | 5 | [5] | 37.756 | [5] | 37.751 |
| `1 - 3 4 5 - 7 - -` | 25 | 6 | [1, 5] | 37.756 | [1, 5] | 37.751 |
| `1 - 3 4 5 - 7 - -` | 25 | 7 | [7] | 37.756 | [7] | 37.751 |
| `1 - 3 4 5 - 7 - -` | 25 | 8 | [1, 7] | 37.756 | [1, 7] | 37.751 |
| `1 - 3 4 5 - 7 - -` | 25 | 9 | [4, 5] | 37.756 | [4, 5] | 37.751 |
| `1 - 3 4 5 - 7 - -` | 25 | 10 | [3, 7] | 37.756 | [3, 7] | 37.751 |
| `1 - 3 4 5 - 7 - -` | 25 | 11 | [4, 7] | 37.756 | [4, 7] | 37.751 |
| `1 - 3 4 5 - 7 - -` | 25 | 12 | [5, 7] | 37.756 | [5, 7] | 37.751 |
| `1 2 - - - - - 8 9` | 25 | 2 | [2] | 30.532 | [2] | 30.532 |
| `1 2 - - - - - 8 9` | 25 | 3 | [1, 2] | 30.532 | [1, 2] | 30.532 |
| `1 2 - - - - - 8 9` | 25 | 8 | [8] | 30.532 | [8] | 30.532 |
| `1 2 - - - - - 8 9` | 25 | 9 | [9] | 30.532 | [9] | 30.532 |
| `1 2 - - - - - 8 9` | 25 | 10 | [1, 9] | 30.532 | [1, 9] | 30.532 |
| `1 2 - - - - - 8 9` | 25 | 11 | [2, 9] | 30.532 | [2, 9] | 30.532 |
| `1 2 - - - - - 8 9` | 25 | 12 | [1, 2, 9] | 30.532 | [1, 2, 9] | 30.532 |
| `1 2 - 4 - 6 7 - -` | 25 | 2 | [2] | 38.247 | [2] | 38.144 |
| `1 2 - 4 - 6 7 - -` | 25 | 3 | [1, 2] | 38.247 | [1, 2] | 38.144 |
| `1 2 - 4 - 6 7 - -` | 25 | 4 | [4] | 38.247 | [4] | 38.144 |
| `1 2 - 4 - 6 7 - -` | 25 | 5 | [1, 4] | 38.247 | [1, 4] | 38.144 |
| `1 2 - 4 - 6 7 - -` | 25 | 6 | [6] | 38.247 | [6] | 38.144 |
| `1 2 - 4 - 6 7 - -` | 25 | 7 | [7] | 38.247 | [7] | 38.144 |
| `1 2 - 4 - 6 7 - -` | 25 | 8 | [2, 6] | 38.247 | [1, 7] | 38.144 |
| `1 2 - 4 - 6 7 - -` | 25 | 9 | [2, 7] | 38.247 | [2, 7] | 38.144 |
| `1 2 - 4 - 6 7 - -` | 25 | 10 | [4, 6] | 38.247 | [4, 6] | 38.144 |
| `1 2 - 4 - 6 7 - -` | 25 | 11 | [4, 7] | 38.247 | [4, 7] | 38.144 |
| `1 2 - 4 - 6 7 - -` | 25 | 12 | [2, 4, 6] | 38.247 | [1, 4, 7] | 38.144 |
| `1 2 - 4 5 - - 8 -` | 25 | 2 | [2] | 37.592 | [2] | 37.571 |
| `1 2 - 4 5 - - 8 -` | 25 | 3 | [1, 2] | 37.592 | [1, 2] | 37.571 |
| `1 2 - 4 5 - - 8 -` | 25 | 4 | [4] | 37.592 | [4] | 37.571 |
| `1 2 - 4 5 - - 8 -` | 25 | 5 | [5] | 37.592 | [5] | 37.571 |
| `1 2 - 4 5 - - 8 -` | 25 | 6 | [2, 4] | 37.592 | [1, 5] | 37.571 |
| `1 2 - 4 5 - - 8 -` | 25 | 7 | [2, 5] | 37.592 | [2, 5] | 37.571 |
| `1 2 - 4 5 - - 8 -` | 25 | 8 | [8] | 37.592 | [8] | 37.571 |
| `1 2 - 4 5 - - 8 -` | 25 | 9 | [1, 8] | 37.592 | [1, 8] | 37.571 |
| `1 2 - 4 5 - - 8 -` | 25 | 10 | [2, 8] | 37.592 | [2, 8] | 37.571 |
| `1 2 - 4 5 - - 8 -` | 25 | 11 | [2, 4, 5] | 37.592 | [1, 2, 8] | 37.571 |
| `1 2 - 4 5 - - 8 -` | 25 | 12 | [4, 8] | 37.592 | [4, 8] | 37.571 |
| `1 2 3 - - 6 - 8 -` | 25 | 2 | [2] | 37.514 | [2] | 37.514 |
| `1 2 3 - - 6 - 8 -` | 25 | 3 | [3] | 37.514 | [3] | 37.514 |
| `1 2 3 - - 6 - 8 -` | 25 | 4 | [1, 3] | 37.514 | [1, 3] | 37.514 |
| `1 2 3 - - 6 - 8 -` | 25 | 5 | [2, 3] | 37.514 | [2, 3] | 37.514 |
| `1 2 3 - - 6 - 8 -` | 25 | 6 | [6] | 37.514 | [6] | 37.514 |
| `1 2 3 - - 6 - 8 -` | 25 | 7 | [1, 6] | 37.514 | [1, 6] | 37.514 |
| `1 2 3 - - 6 - 8 -` | 25 | 8 | [8] | 37.514 | [8] | 37.514 |
| `1 2 3 - - 6 - 8 -` | 25 | 9 | [1, 8] | 37.514 | [1, 8] | 37.514 |
| `1 2 3 - - 6 - 8 -` | 25 | 10 | [2, 8] | 37.514 | [2, 8] | 37.514 |
| `1 2 3 - - 6 - 8 -` | 25 | 11 | [3, 8] | 37.514 | [3, 8] | 37.514 |
| `1 2 3 - - 6 - 8 -` | 25 | 12 | [1, 3, 8] | 37.514 | [1, 3, 8] | 37.514 |
| `1 2 3 - 5 - - - 9` | 25 | 2 | [2] | 36.533 | [2] | 36.533 |
| `1 2 3 - 5 - - - 9` | 25 | 3 | [3] | 36.533 | [3] | 36.533 |
| `1 2 3 - 5 - - - 9` | 25 | 4 | [1, 3] | 36.533 | [1, 3] | 36.533 |
| `1 2 3 - 5 - - - 9` | 25 | 5 | [5] | 36.533 | [5] | 36.533 |
| `1 2 3 - 5 - - - 9` | 25 | 6 | [1, 5] | 36.533 | [1, 5] | 36.533 |
| `1 2 3 - 5 - - - 9` | 25 | 7 | [2, 5] | 36.533 | [2, 5] | 36.533 |
| `1 2 3 - 5 - - - 9` | 25 | 8 | [3, 5] | 36.533 | [3, 5] | 36.533 |
| `1 2 3 - 5 - - - 9` | 25 | 9 | [9] | 36.533 | [9] | 36.533 |
| `1 2 3 - 5 - - - 9` | 25 | 10 | [1, 9] | 36.533 | [1, 9] | 36.533 |
| `1 2 3 - 5 - - - 9` | 25 | 11 | [2, 9] | 36.533 | [2, 9] | 36.533 |
| `1 2 3 - 5 - - - 9` | 25 | 12 | [3, 9] | 36.533 | [3, 9] | 36.533 |
| `- - - - 5 6 - 8 -` | 26 | 5 | [5] | 29.980 | [5] | 29.980 |
| `- - - - 5 6 - 8 -` | 26 | 6 | [6] | 29.980 | [6] | 29.980 |
| `- - - - 5 6 - 8 -` | 26 | 8 | [8] | 29.980 | [8] | 29.980 |
| `- - - - 5 6 - 8 -` | 26 | 11 | [5, 6] | 29.980 | [5, 6] | 29.980 |
| `- - - 4 - - 7 8 -` | 26 | 4 | [4] | 30.502 | [4] | 30.502 |
| `- - - 4 - - 7 8 -` | 26 | 7 | [7] | 30.502 | [7] | 30.502 |
| `- - - 4 - - 7 8 -` | 26 | 8 | [8] | 30.502 | [8] | 30.502 |
| `- - - 4 - - 7 8 -` | 26 | 11 | [4, 7] | 30.502 | [4, 7] | 30.502 |
| `- - - 4 - - 7 8 -` | 26 | 12 | [4, 8] | 30.502 | [4, 8] | 30.502 |
| `- - - 4 - 6 - - 9` | 26 | 4 | [4] | 29.692 | [4] | 29.692 |
| `- - - 4 - 6 - - 9` | 26 | 6 | [6] | 29.692 | [6] | 29.692 |
| `- - - 4 - 6 - - 9` | 26 | 9 | [9] | 29.692 | [9] | 29.692 |
| `- - - 4 - 6 - - 9` | 26 | 10 | [4, 6] | 29.692 | [4, 6] | 29.692 |
| `- - 3 - - - 7 - 9` | 26 | 3 | [3] | 30.266 | [3] | 30.266 |
| `- - 3 - - - 7 - 9` | 26 | 7 | [7] | 30.266 | [7] | 30.266 |
| `- - 3 - - - 7 - 9` | 26 | 9 | [9] | 30.266 | [9] | 30.266 |
| `- - 3 - - - 7 - 9` | 26 | 10 | [3, 7] | 30.266 | [3, 7] | 30.266 |
| `- - 3 - - - 7 - 9` | 26 | 12 | [3, 9] | 30.266 | [3, 9] | 30.266 |
| `- - 3 4 5 - 7 - -` | 26 | 3 | [3] | 34.986 | [3] | 34.986 |
| `- - 3 4 5 - 7 - -` | 26 | 4 | [4] | 34.986 | [4] | 34.986 |
| `- - 3 4 5 - 7 - -` | 26 | 5 | [5] | 34.986 | [5] | 34.986 |
| `- - 3 4 5 - 7 - -` | 26 | 7 | [7] | 34.986 | [7] | 34.986 |
| `- - 3 4 5 - 7 - -` | 26 | 8 | [3, 5] | 34.986 | [3, 5] | 34.986 |
| `- - 3 4 5 - 7 - -` | 26 | 9 | [4, 5] | 34.986 | [4, 5] | 34.986 |
| `- - 3 4 5 - 7 - -` | 26 | 10 | [3, 7] | 34.986 | [3, 7] | 34.986 |
| `- - 3 4 5 - 7 - -` | 26 | 11 | [4, 7] | 34.986 | [4, 7] | 34.986 |
| `- - 3 4 5 - 7 - -` | 26 | 12 | [5, 7] | 34.986 | [5, 7] | 34.986 |
| `- 2 - - - - - 8 9` | 26 | 2 | [2] | 30.285 | [2] | 30.285 |
| `- 2 - - - - - 8 9` | 26 | 8 | [8] | 30.285 | [8] | 30.285 |
| `- 2 - - - - - 8 9` | 26 | 9 | [9] | 30.285 | [9] | 30.285 |
| `- 2 - - - - - 8 9` | 26 | 10 | [2, 8] | 30.285 | [2, 8] | 30.285 |
| `- 2 - - - - - 8 9` | 26 | 11 | [2, 9] | 30.285 | [2, 9] | 30.285 |
| `- 2 - 4 - 6 7 - -` | 26 | 2 | [2] | 35.028 | [2] | 35.028 |
| `- 2 - 4 - 6 7 - -` | 26 | 4 | [4] | 35.028 | [4] | 35.028 |
| `- 2 - 4 - 6 7 - -` | 26 | 6 | [6] | 35.028 | [6] | 35.028 |
| `- 2 - 4 - 6 7 - -` | 26 | 7 | [7] | 35.028 | [7] | 35.028 |
| `- 2 - 4 - 6 7 - -` | 26 | 8 | [2, 6] | 35.028 | [2, 6] | 35.028 |
| `- 2 - 4 - 6 7 - -` | 26 | 9 | [2, 7] | 35.028 | [2, 7] | 35.028 |
| `- 2 - 4 - 6 7 - -` | 26 | 10 | [4, 6] | 35.028 | [4, 6] | 35.028 |
| `- 2 - 4 - 6 7 - -` | 26 | 11 | [4, 7] | 35.028 | [4, 7] | 35.028 |
| `- 2 - 4 - 6 7 - -` | 26 | 12 | [2, 4, 6] | 35.028 | [2, 4, 6] | 35.028 |
| `- 2 - 4 5 - - 8 -` | 26 | 2 | [2] | 35.555 | [2] | 35.555 |
| `- 2 - 4 5 - - 8 -` | 26 | 4 | [4] | 35.555 | [4] | 35.555 |
| `- 2 - 4 5 - - 8 -` | 26 | 5 | [5] | 35.555 | [5] | 35.555 |
| `- 2 - 4 5 - - 8 -` | 26 | 6 | [2, 4] | 35.555 | [2, 4] | 35.555 |
| `- 2 - 4 5 - - 8 -` | 26 | 7 | [2, 5] | 35.555 | [2, 5] | 35.555 |
| `- 2 - 4 5 - - 8 -` | 26 | 8 | [8] | 35.555 | [8] | 35.555 |
| `- 2 - 4 5 - - 8 -` | 26 | 9 | [4, 5] | 35.555 | [4, 5] | 35.555 |
| `- 2 - 4 5 - - 8 -` | 26 | 10 | [2, 8] | 35.555 | [2, 8] | 35.555 |
| `- 2 - 4 5 - - 8 -` | 26 | 11 | [2, 4, 5] | 35.555 | [2, 4, 5] | 35.555 |
| `- 2 - 4 5 - - 8 -` | 26 | 12 | [4, 8] | 35.555 | [4, 8] | 35.555 |
| `- 2 3 - - 6 - 8 -` | 26 | 2 | [2] | 33.426 | [2] | 33.426 |
| `- 2 3 - - 6 - 8 -` | 26 | 3 | [3] | 33.426 | [3] | 33.426 |
| `- 2 3 - - 6 - 8 -` | 26 | 5 | [2, 3] | 33.426 | [2, 3] | 33.426 |
| `- 2 3 - - 6 - 8 -` | 26 | 6 | [6] | 33.426 | [6] | 33.426 |
| `- 2 3 - - 6 - 8 -` | 26 | 8 | [8] | 33.426 | [8] | 33.426 |
| `- 2 3 - - 6 - 8 -` | 26 | 9 | [3, 6] | 33.426 | [3, 6] | 33.426 |
| `- 2 3 - - 6 - 8 -` | 26 | 10 | [2, 8] | 33.426 | [2, 8] | 33.426 |
| `- 2 3 - - 6 - 8 -` | 26 | 11 | [3, 8] | 33.426 | [3, 8] | 33.426 |
| `- 2 3 - 5 - - - 9` | 26 | 2 | [2] | 33.703 | [2] | 33.703 |
| `- 2 3 - 5 - - - 9` | 26 | 3 | [3] | 33.703 | [3] | 33.703 |
| `- 2 3 - 5 - - - 9` | 26 | 5 | [5] | 33.703 | [5] | 33.703 |
| `- 2 3 - 5 - - - 9` | 26 | 7 | [2, 5] | 33.703 | [2, 5] | 33.703 |
| `- 2 3 - 5 - - - 9` | 26 | 8 | [3, 5] | 33.703 | [3, 5] | 33.703 |
| `- 2 3 - 5 - - - 9` | 26 | 9 | [9] | 33.703 | [9] | 33.703 |
| `- 2 3 - 5 - - - 9` | 26 | 10 | [2, 3, 5] | 33.703 | [2, 3, 5] | 33.703 |
| `- 2 3 - 5 - - - 9` | 26 | 11 | [2, 9] | 33.703 | [2, 9] | 33.703 |
| `- 2 3 - 5 - - - 9` | 26 | 12 | [3, 9] | 33.703 | [3, 9] | 33.703 |
| `1 - - - 5 6 7 - -` | 26 | 5 | [5] | 32.849 | [5] | 32.849 |
| `1 - - - 5 6 7 - -` | 26 | 6 | [6] | 32.849 | [6] | 32.849 |
| `1 - - - 5 6 7 - -` | 26 | 7 | [7] | 32.849 | [7] | 32.849 |
| `1 - - - 5 6 7 - -` | 26 | 8 | [1, 7] | 32.849 | [1, 7] | 32.849 |
| `1 - - - 5 6 7 - -` | 26 | 11 | [5, 6] | 32.849 | [5, 6] | 32.849 |
| `1 - - - 5 6 7 - -` | 26 | 12 | [5, 7] | 32.849 | [5, 7] | 32.849 |
| `1 - - 4 - 6 - 8 -` | 26 | 4 | [4] | 35.474 | [4] | 35.474 |
| `1 - - 4 - 6 - 8 -` | 26 | 5 | [1, 4] | 35.474 | [1, 4] | 35.474 |
| `1 - - 4 - 6 - 8 -` | 26 | 6 | [6] | 35.474 | [6] | 35.474 |
| `1 - - 4 - 6 - 8 -` | 26 | 7 | [1, 6] | 35.474 | [1, 6] | 35.474 |
| `1 - - 4 - 6 - 8 -` | 26 | 8 | [8] | 35.474 | [8] | 35.474 |
| `1 - - 4 - 6 - 8 -` | 26 | 9 | [1, 8] | 35.474 | [1, 8] | 35.474 |
| `1 - - 4 - 6 - 8 -` | 26 | 10 | [4, 6] | 35.474 | [4, 6] | 35.474 |
| `1 - - 4 - 6 - 8 -` | 26 | 11 | [1, 4, 6] | 35.474 | [1, 4, 6] | 35.474 |
| `1 - - 4 - 6 - 8 -` | 26 | 12 | [4, 8] | 35.474 | [4, 8] | 35.474 |
| `1 - - 4 5 - - - 9` | 26 | 4 | [4] | 31.005 | [4] | 31.005 |
| `1 - - 4 5 - - - 9` | 26 | 5 | [5] | 31.005 | [5] | 31.005 |
| `1 - - 4 5 - - - 9` | 26 | 6 | [1, 5] | 31.005 | [1, 5] | 31.005 |
| `1 - - 4 5 - - - 9` | 26 | 9 | [9] | 31.005 | [9] | 31.005 |
| `1 - - 4 5 - - - 9` | 26 | 10 | [1, 9] | 31.005 | [1, 9] | 31.005 |
| `1 - 3 - - - 7 8 -` | 26 | 3 | [3] | 33.861 | [3] | 33.861 |
| `1 - 3 - - - 7 8 -` | 26 | 4 | [1, 3] | 33.861 | [1, 3] | 33.861 |
| `1 - 3 - - - 7 8 -` | 26 | 7 | [7] | 33.861 | [7] | 33.861 |
| `1 - 3 - - - 7 8 -` | 26 | 8 | [8] | 33.861 | [8] | 33.861 |
| `1 - 3 - - - 7 8 -` | 26 | 9 | [1, 8] | 33.861 | [1, 8] | 33.861 |
| `1 - 3 - - - 7 8 -` | 26 | 10 | [3, 7] | 33.861 | [3, 7] | 33.861 |
| `1 - 3 - - - 7 8 -` | 26 | 11 | [3, 8] | 33.861 | [3, 8] | 33.861 |
| `1 - 3 - - - 7 8 -` | 26 | 12 | [1, 3, 8] | 33.861 | [1, 3, 8] | 33.861 |
| `1 - 3 - - 6 - - 9` | 26 | 3 | [3] | 32.500 | [3] | 32.500 |
| `1 - 3 - - 6 - - 9` | 26 | 4 | [1, 3] | 32.500 | [1, 3] | 32.500 |
| `1 - 3 - - 6 - - 9` | 26 | 6 | [6] | 32.500 | [6] | 32.500 |
| `1 - 3 - - 6 - - 9` | 26 | 7 | [1, 6] | 32.500 | [1, 6] | 32.500 |
| `1 - 3 - - 6 - - 9` | 26 | 9 | [9] | 32.500 | [9] | 32.500 |
| `1 - 3 - - 6 - - 9` | 26 | 10 | [1, 9] | 32.500 | [1, 9] | 32.500 |
| `1 - 3 - - 6 - - 9` | 26 | 12 | [3, 9] | 32.500 | [3, 9] | 32.500 |
| `1 - 3 4 5 6 - - -` | 26 | 3 | [3] | 38.856 | [3] | 38.754 |
| `1 - 3 4 5 6 - - -` | 26 | 4 | [4] | 38.856 | [4] | 38.754 |
| `1 - 3 4 5 6 - - -` | 26 | 5 | [5] | 38.856 | [5] | 38.754 |
| `1 - 3 4 5 6 - - -` | 26 | 6 | [6] | 38.856 | [6] | 38.754 |
| `1 - 3 4 5 6 - - -` | 26 | 7 | [1, 6] | 38.856 | [1, 6] | 38.754 |
| `1 - 3 4 5 6 - - -` | 26 | 8 | [3, 5] | 38.856 | [3, 5] | 38.754 |
| `1 - 3 4 5 6 - - -` | 26 | 9 | [4, 5] | 38.856 | [3, 6] | 38.754 |
| `1 - 3 4 5 6 - - -` | 26 | 10 | [4, 6] | 38.856 | [4, 6] | 38.754 |
| `1 - 3 4 5 6 - - -` | 26 | 11 | [5, 6] | 38.856 | [5, 6] | 38.754 |
| `1 - 3 4 5 6 - - -` | 26 | 12 | [3, 4, 5] | 38.856 | [1, 5, 6] | 38.754 |
| `1 2 - - - - 7 - 9` | 26 | 2 | [2] | 33.158 | [2] | 33.158 |
| `1 2 - - - - 7 - 9` | 26 | 3 | [1, 2] | 33.158 | [1, 2] | 33.158 |
| `1 2 - - - - 7 - 9` | 26 | 7 | [7] | 33.158 | [7] | 33.158 |
| `1 2 - - - - 7 - 9` | 26 | 8 | [1, 7] | 33.158 | [1, 7] | 33.158 |
| `1 2 - - - - 7 - 9` | 26 | 9 | [9] | 33.158 | [9] | 33.158 |
| `1 2 - - - - 7 - 9` | 26 | 10 | [1, 9] | 33.158 | [1, 9] | 33.158 |
| `1 2 - - - - 7 - 9` | 26 | 11 | [2, 9] | 33.158 | [2, 9] | 33.158 |
| `1 2 - - - - 7 - 9` | 26 | 12 | [1, 2, 9] | 33.158 | [1, 2, 9] | 33.158 |
| `1 2 - 4 5 - 7 - -` | 26 | 2 | [2] | 38.968 | [2] | 38.854 |
| `1 2 - 4 5 - 7 - -` | 26 | 3 | [1, 2] | 38.968 | [1, 2] | 38.854 |
| `1 2 - 4 5 - 7 - -` | 26 | 4 | [4] | 38.968 | [4] | 38.854 |
| `1 2 - 4 5 - 7 - -` | 26 | 5 | [5] | 38.968 | [5] | 38.854 |
| `1 2 - 4 5 - 7 - -` | 26 | 6 | [2, 4] | 38.968 | [1, 5] | 38.854 |
| `1 2 - 4 5 - 7 - -` | 26 | 7 | [7] | 38.968 | [7] | 38.854 |
| `1 2 - 4 5 - 7 - -` | 26 | 8 | [1, 7] | 38.968 | [1, 7] | 38.854 |
| `1 2 - 4 5 - 7 - -` | 26 | 9 | [4, 5] | 38.968 | [2, 7] | 38.854 |
| `1 2 - 4 5 - 7 - -` | 26 | 10 | [1, 4, 5] | 38.968 | [1, 2, 7] | 38.854 |
| `1 2 - 4 5 - 7 - -` | 26 | 11 | [4, 7] | 38.968 | [4, 7] | 38.854 |
| `1 2 - 4 5 - 7 - -` | 26 | 12 | [5, 7] | 38.968 | [5, 7] | 38.854 |
| `1 2 3 - - 6 7 - -` | 26 | 2 | [2] | 38.792 | [2] | 38.776 |
| `1 2 3 - - 6 7 - -` | 26 | 3 | [3] | 38.792 | [3] | 38.776 |
| `1 2 3 - - 6 7 - -` | 26 | 4 | [1, 3] | 38.792 | [1, 3] | 38.776 |
| `1 2 3 - - 6 7 - -` | 26 | 5 | [2, 3] | 38.792 | [2, 3] | 38.776 |
| `1 2 3 - - 6 7 - -` | 26 | 6 | [6] | 38.792 | [6] | 38.776 |
| `1 2 3 - - 6 7 - -` | 26 | 7 | [7] | 38.792 | [7] | 38.776 |
| `1 2 3 - - 6 7 - -` | 26 | 8 | [1, 7] | 38.792 | [1, 7] | 38.776 |
| `1 2 3 - - 6 7 - -` | 26 | 9 | [2, 7] | 38.792 | [2, 7] | 38.776 |
| `1 2 3 - - 6 7 - -` | 26 | 10 | [3, 7] | 38.792 | [3, 7] | 38.776 |
| `1 2 3 - - 6 7 - -` | 26 | 11 | [2, 3, 6] | 38.792 | [1, 3, 7] | 38.776 |
| `1 2 3 - - 6 7 - -` | 26 | 12 | [2, 3, 7] | 38.792 | [2, 3, 7] | 38.776 |
| `1 2 3 - 5 - - 8 -` | 26 | 2 | [2] | 38.136 | [2] | 38.136 |
| `1 2 3 - 5 - - 8 -` | 26 | 3 | [3] | 38.136 | [3] | 38.136 |
| `1 2 3 - 5 - - 8 -` | 26 | 4 | [1, 3] | 38.136 | [1, 3] | 38.136 |
| `1 2 3 - 5 - - 8 -` | 26 | 5 | [5] | 38.136 | [5] | 38.136 |
| `1 2 3 - 5 - - 8 -` | 26 | 6 | [1, 5] | 38.136 | [1, 5] | 38.136 |
| `1 2 3 - 5 - - 8 -` | 26 | 7 | [2, 5] | 38.136 | [2, 5] | 38.136 |
| `1 2 3 - 5 - - 8 -` | 26 | 8 | [8] | 38.136 | [8] | 38.136 |
| `1 2 3 - 5 - - 8 -` | 26 | 9 | [1, 8] | 38.136 | [1, 8] | 38.136 |
| `1 2 3 - 5 - - 8 -` | 26 | 10 | [2, 8] | 38.136 | [2, 8] | 38.136 |
| `1 2 3 - 5 - - 8 -` | 26 | 11 | [3, 8] | 38.136 | [3, 8] | 38.136 |
| `1 2 3 - 5 - - 8 -` | 26 | 12 | [1, 3, 8] | 38.136 | [1, 3, 8] | 38.136 |
| `1 2 3 4 - - - - 9` | 26 | 2 | [2] | 36.991 | [2] | 36.984 |
| `1 2 3 4 - - - - 9` | 26 | 3 | [3] | 36.991 | [3] | 36.984 |
| `1 2 3 4 - - - - 9` | 26 | 4 | [4] | 36.991 | [4] | 36.984 |
| `1 2 3 4 - - - - 9` | 26 | 5 | [2, 3] | 36.991 | [1, 4] | 36.984 |
| `1 2 3 4 - - - - 9` | 26 | 6 | [2, 4] | 36.991 | [2, 4] | 36.984 |
| `1 2 3 4 - - - - 9` | 26 | 7 | [3, 4] | 36.991 | [3, 4] | 36.984 |
| `1 2 3 4 - - - - 9` | 26 | 8 | [1, 3, 4] | 36.991 | [1, 3, 4] | 36.984 |
| `1 2 3 4 - - - - 9` | 26 | 9 | [9] | 36.991 | [9] | 36.984 |
| `1 2 3 4 - - - - 9` | 26 | 10 | [1, 9] | 36.991 | [1, 9] | 36.984 |
| `1 2 3 4 - - - - 9` | 26 | 11 | [2, 9] | 36.991 | [2, 9] | 36.984 |
| `1 2 3 4 - - - - 9` | 26 | 12 | [3, 9] | 36.991 | [3, 9] | 36.984 |
| `- - - - 5 6 7 - -` | 27 | 5 | [5] | 31.522 | [5] | 31.522 |
| `- - - - 5 6 7 - -` | 27 | 6 | [6] | 31.522 | [6] | 31.522 |
| `- - - - 5 6 7 - -` | 27 | 7 | [7] | 31.522 | [7] | 31.522 |
| `- - - - 5 6 7 - -` | 27 | 11 | [5, 6] | 31.522 | [5, 6] | 31.522 |
| `- - - - 5 6 7 - -` | 27 | 12 | [5, 7] | 31.522 | [5, 7] | 31.522 |
| `- - - 4 - 6 - 8 -` | 27 | 4 | [4] | 31.305 | [4] | 31.305 |
| `- - - 4 - 6 - 8 -` | 27 | 6 | [6] | 31.305 | [6] | 31.305 |
| `- - - 4 - 6 - 8 -` | 27 | 8 | [8] | 31.305 | [8] | 31.305 |
| `- - - 4 - 6 - 8 -` | 27 | 10 | [4, 6] | 31.305 | [4, 6] | 31.305 |
| `- - - 4 - 6 - 8 -` | 27 | 12 | [4, 8] | 31.305 | [4, 8] | 31.305 |
| `- - - 4 5 - - - 9` | 27 | 4 | [4] | 29.414 | [4] | 29.414 |
| `- - - 4 5 - - - 9` | 27 | 5 | [5] | 29.414 | [5] | 29.414 |
| `- - - 4 5 - - - 9` | 27 | 9 | [9] | 29.414 | [9] | 29.414 |
| `- - 3 - - - 7 8 -` | 27 | 3 | [3] | 31.835 | [3] | 31.835 |
| `- - 3 - - - 7 8 -` | 27 | 7 | [7] | 31.835 | [7] | 31.835 |
| `- - 3 - - - 7 8 -` | 27 | 8 | [8] | 31.835 | [8] | 31.835 |
| `- - 3 - - - 7 8 -` | 27 | 10 | [3, 7] | 31.835 | [3, 7] | 31.835 |
| `- - 3 - - - 7 8 -` | 27 | 11 | [3, 8] | 31.835 | [3, 8] | 31.835 |
| `- - 3 - - 6 - - 9` | 27 | 3 | [3] | 29.920 | [3] | 29.920 |
| `- - 3 - - 6 - - 9` | 27 | 6 | [6] | 29.920 | [6] | 29.920 |
| `- - 3 - - 6 - - 9` | 27 | 9 | [9] | 29.920 | [9] | 29.920 |
| `- - 3 - - 6 - - 9` | 27 | 12 | [3, 9] | 29.920 | [3, 9] | 29.920 |
| `- - 3 4 5 6 - - -` | 27 | 3 | [3] | 37.100 | [3] | 37.089 |
| `- - 3 4 5 6 - - -` | 27 | 4 | [4] | 37.100 | [4] | 37.089 |
| `- - 3 4 5 6 - - -` | 27 | 5 | [5] | 37.100 | [5] | 37.089 |
| `- - 3 4 5 6 - - -` | 27 | 6 | [6] | 37.100 | [6] | 37.089 |
| `- - 3 4 5 6 - - -` | 27 | 7 | [3, 4] | 37.100 | [3, 4] | 37.089 |
| `- - 3 4 5 6 - - -` | 27 | 8 | [3, 5] | 37.100 | [3, 5] | 37.089 |
| `- - 3 4 5 6 - - -` | 27 | 9 | [4, 5] | 37.100 | [3, 6] | 37.089 |
| `- - 3 4 5 6 - - -` | 27 | 10 | [4, 6] | 37.100 | [4, 6] | 37.089 |
| `- - 3 4 5 6 - - -` | 27 | 11 | [5, 6] | 37.100 | [5, 6] | 37.089 |
| `- - 3 4 5 6 - - -` | 27 | 12 | [3, 4, 5] | 37.100 | [3, 4, 5] | 37.089 |
| `- 2 - - - - 7 - 9` | 27 | 2 | [2] | 30.502 | [2] | 30.502 |
| `- 2 - - - - 7 - 9` | 27 | 7 | [7] | 30.502 | [7] | 30.502 |
| `- 2 - - - - 7 - 9` | 27 | 9 | [9] | 30.502 | [9] | 30.502 |
| `- 2 - - - - 7 - 9` | 27 | 11 | [2, 9] | 30.502 | [2, 9] | 30.502 |
| `- 2 - 4 5 - 7 - -` | 27 | 2 | [2] | 34.401 | [2] | 34.369 |
| `- 2 - 4 5 - 7 - -` | 27 | 4 | [4] | 34.401 | [4] | 34.369 |
| `- 2 - 4 5 - 7 - -` | 27 | 5 | [5] | 34.401 | [5] | 34.369 |
| `- 2 - 4 5 - 7 - -` | 27 | 6 | [2, 4] | 34.401 | [2, 4] | 34.369 |
| `- 2 - 4 5 - 7 - -` | 27 | 7 | [7] | 34.401 | [7] | 34.369 |
| `- 2 - 4 5 - 7 - -` | 27 | 9 | [4, 5] | 34.401 | [2, 7] | 34.369 |
| `- 2 - 4 5 - 7 - -` | 27 | 11 | [4, 7] | 34.401 | [4, 7] | 34.369 |
| `- 2 - 4 5 - 7 - -` | 27 | 12 | [5, 7] | 34.401 | [5, 7] | 34.369 |
| `- 2 3 - - 6 7 - -` | 27 | 2 | [2] | 36.594 | [2] | 36.573 |
| `- 2 3 - - 6 7 - -` | 27 | 3 | [3] | 36.594 | [3] | 36.573 |
| `- 2 3 - - 6 7 - -` | 27 | 5 | [2, 3] | 36.594 | [2, 3] | 36.573 |
| `- 2 3 - - 6 7 - -` | 27 | 6 | [6] | 36.594 | [6] | 36.573 |
| `- 2 3 - - 6 7 - -` | 27 | 7 | [7] | 36.594 | [7] | 36.573 |
| `- 2 3 - - 6 7 - -` | 27 | 8 | [2, 6] | 36.594 | [2, 6] | 36.573 |
| `- 2 3 - - 6 7 - -` | 27 | 9 | [3, 6] | 36.594 | [2, 7] | 36.573 |
| `- 2 3 - - 6 7 - -` | 27 | 10 | [3, 7] | 36.594 | [3, 7] | 36.573 |
| `- 2 3 - - 6 7 - -` | 27 | 11 | [2, 3, 6] | 36.594 | [2, 3, 6] | 36.573 |
| `- 2 3 - - 6 7 - -` | 27 | 12 | [2, 3, 7] | 36.594 | [2, 3, 7] | 36.573 |
| `- 2 3 - 5 - - 8 -` | 27 | 2 | [2] | 33.415 | [2] | 33.415 |
| `- 2 3 - 5 - - 8 -` | 27 | 3 | [3] | 33.415 | [3] | 33.415 |
| `- 2 3 - 5 - - 8 -` | 27 | 5 | [5] | 33.415 | [5] | 33.415 |
| `- 2 3 - 5 - - 8 -` | 27 | 7 | [2, 5] | 33.415 | [2, 5] | 33.415 |
| `- 2 3 - 5 - - 8 -` | 27 | 8 | [8] | 33.415 | [8] | 33.415 |
| `- 2 3 - 5 - - 8 -` | 27 | 10 | [2, 8] | 33.415 | [2, 8] | 33.415 |
| `- 2 3 - 5 - - 8 -` | 27 | 11 | [3, 8] | 33.415 | [3, 8] | 33.415 |
| `- 2 3 4 - - - - 9` | 27 | 2 | [2] | 33.872 | [2] | 33.872 |
| `- 2 3 4 - - - - 9` | 27 | 3 | [3] | 33.872 | [3] | 33.872 |
| `- 2 3 4 - - - - 9` | 27 | 4 | [4] | 33.872 | [4] | 33.872 |
| `- 2 3 4 - - - - 9` | 27 | 5 | [2, 3] | 33.872 | [2, 3] | 33.872 |
| `- 2 3 4 - - - - 9` | 27 | 6 | [2, 4] | 33.872 | [2, 4] | 33.872 |
| `- 2 3 4 - - - - 9` | 27 | 7 | [3, 4] | 33.872 | [3, 4] | 33.872 |
| `- 2 3 4 - - - - 9` | 27 | 9 | [9] | 33.872 | [9] | 33.872 |
| `- 2 3 4 - - - - 9` | 27 | 11 | [2, 9] | 33.872 | [2, 9] | 33.872 |
| `- 2 3 4 - - - - 9` | 27 | 12 | [3, 9] | 33.872 | [3, 9] | 33.872 |
| `1 - - - - - - 8 9` | 27 | 8 | [8] | 30.526 | [8] | 30.526 |
| `1 - - - - - - 8 9` | 27 | 9 | [9] | 30.526 | [9] | 30.526 |
| `1 - - - - - - 8 9` | 27 | 10 | [1, 9] | 30.526 | [1, 9] | 30.526 |
| `1 - - 4 - 6 7 - -` | 27 | 4 | [4] | 35.412 | [4] | 35.412 |
| `1 - - 4 - 6 7 - -` | 27 | 5 | [1, 4] | 35.412 | [1, 4] | 35.412 |
| `1 - - 4 - 6 7 - -` | 27 | 6 | [6] | 35.412 | [6] | 35.412 |
| `1 - - 4 - 6 7 - -` | 27 | 7 | [7] | 35.412 | [7] | 35.412 |
| `1 - - 4 - 6 7 - -` | 27 | 8 | [1, 7] | 35.412 | [1, 7] | 35.412 |
| `1 - - 4 - 6 7 - -` | 27 | 10 | [4, 6] | 35.412 | [4, 6] | 35.412 |
| `1 - - 4 - 6 7 - -` | 27 | 11 | [4, 7] | 35.412 | [4, 7] | 35.412 |
| `1 - - 4 - 6 7 - -` | 27 | 12 | [1, 4, 7] | 35.412 | [1, 4, 7] | 35.412 |
| `1 - - 4 5 - - 8 -` | 27 | 4 | [4] | 33.929 | [4] | 33.913 |
| `1 - - 4 5 - - 8 -` | 27 | 5 | [5] | 33.929 | [5] | 33.913 |
| `1 - - 4 5 - - 8 -` | 27 | 6 | [1, 5] | 33.929 | [1, 5] | 33.913 |
| `1 - - 4 5 - - 8 -` | 27 | 8 | [8] | 33.929 | [8] | 33.913 |
| `1 - - 4 5 - - 8 -` | 27 | 9 | [4, 5] | 33.929 | [1, 8] | 33.913 |
| `1 - - 4 5 - - 8 -` | 27 | 10 | [1, 4, 5] | 33.929 | [1, 4, 5] | 33.913 |
| `1 - - 4 5 - - 8 -` | 27 | 12 | [4, 8] | 33.929 | [4, 8] | 33.913 |
| `1 - 3 - - 6 - 8 -` | 27 | 3 | [3] | 35.815 | [3] | 35.810 |
| `1 - 3 - - 6 - 8 -` | 27 | 4 | [1, 3] | 35.815 | [1, 3] | 35.810 |
| `1 - 3 - - 6 - 8 -` | 27 | 6 | [6] | 35.815 | [6] | 35.810 |
| `1 - 3 - - 6 - 8 -` | 27 | 7 | [1, 6] | 35.815 | [1, 6] | 35.810 |
| `1 - 3 - - 6 - 8 -` | 27 | 8 | [8] | 35.815 | [8] | 35.810 |
| `1 - 3 - - 6 - 8 -` | 27 | 9 | [3, 6] | 35.815 | [1, 8] | 35.810 |
| `1 - 3 - - 6 - 8 -` | 27 | 10 | [1, 3, 6] | 35.815 | [1, 3, 6] | 35.810 |
| `1 - 3 - - 6 - 8 -` | 27 | 11 | [3, 8] | 35.815 | [3, 8] | 35.810 |
| `1 - 3 - - 6 - 8 -` | 27 | 12 | [1, 3, 8] | 35.815 | [1, 3, 8] | 35.810 |
| `1 - 3 - 5 - - - 9` | 27 | 3 | [3] | 33.980 | [3] | 33.980 |
| `1 - 3 - 5 - - - 9` | 27 | 4 | [1, 3] | 33.980 | [1, 3] | 33.980 |
| `1 - 3 - 5 - - - 9` | 27 | 5 | [5] | 33.980 | [5] | 33.980 |
| `1 - 3 - 5 - - - 9` | 27 | 6 | [1, 5] | 33.980 | [1, 5] | 33.980 |
| `1 - 3 - 5 - - - 9` | 27 | 8 | [3, 5] | 33.980 | [3, 5] | 33.980 |
| `1 - 3 - 5 - - - 9` | 27 | 9 | [9] | 33.980 | [9] | 33.980 |
| `1 - 3 - 5 - - - 9` | 27 | 10 | [1, 9] | 33.980 | [1, 9] | 33.980 |
| `1 - 3 - 5 - - - 9` | 27 | 12 | [3, 9] | 33.980 | [3, 9] | 33.980 |
| `1 2 - - - - 7 8 -` | 27 | 2 | [2] | 33.991 | [2] | 33.991 |
| `1 2 - - - - 7 8 -` | 27 | 3 | [1, 2] | 33.991 | [1, 2] | 33.991 |
| `1 2 - - - - 7 8 -` | 27 | 7 | [7] | 33.991 | [7] | 33.991 |
| `1 2 - - - - 7 8 -` | 27 | 8 | [8] | 33.991 | [8] | 33.991 |
| `1 2 - - - - 7 8 -` | 27 | 9 | [1, 8] | 33.991 | [1, 8] | 33.991 |
| `1 2 - - - - 7 8 -` | 27 | 10 | [2, 8] | 33.991 | [2, 8] | 33.991 |
| `1 2 - - - - 7 8 -` | 27 | 11 | [1, 2, 8] | 33.991 | [1, 2, 8] | 33.991 |
| `1 2 - - - 6 - - 9` | 27 | 2 | [2] | 35.126 | [2] | 35.126 |
| `1 2 - - - 6 - - 9` | 27 | 3 | [1, 2] | 35.126 | [1, 2] | 35.126 |
| `1 2 - - - 6 - - 9` | 27 | 6 | [6] | 35.126 | [6] | 35.126 |
| `1 2 - - - 6 - - 9` | 27 | 7 | [1, 6] | 35.126 | [1, 6] | 35.126 |
| `1 2 - - - 6 - - 9` | 27 | 8 | [2, 6] | 35.126 | [2, 6] | 35.126 |
| `1 2 - - - 6 - - 9` | 27 | 9 | [9] | 35.126 | [9] | 35.126 |
| `1 2 - - - 6 - - 9` | 27 | 10 | [1, 9] | 35.126 | [1, 9] | 35.126 |
| `1 2 - - - 6 - - 9` | 27 | 11 | [2, 9] | 35.126 | [2, 9] | 35.126 |
| `1 2 - - - 6 - - 9` | 27 | 12 | [1, 2, 9] | 35.126 | [1, 2, 9] | 35.126 |
| `1 2 - 4 5 6 - - -` | 27 | 2 | [2] | 39.672 | [2] | 39.662 |
| `1 2 - 4 5 6 - - -` | 27 | 3 | [1, 2] | 39.672 | [1, 2] | 39.662 |
| `1 2 - 4 5 6 - - -` | 27 | 4 | [4] | 39.672 | [4] | 39.662 |
| `1 2 - 4 5 6 - - -` | 27 | 5 | [5] | 39.672 | [5] | 39.662 |
| `1 2 - 4 5 6 - - -` | 27 | 6 | [6] | 39.672 | [6] | 39.662 |
| `1 2 - 4 5 6 - - -` | 27 | 7 | [1, 6] | 39.672 | [1, 6] | 39.662 |
| `1 2 - 4 5 6 - - -` | 27 | 8 | [2, 6] | 39.672 | [2, 6] | 39.662 |
| `1 2 - 4 5 6 - - -` | 27 | 9 | [4, 5] | 39.672 | [4, 5] | 39.662 |
| `1 2 - 4 5 6 - - -` | 27 | 10 | [4, 6] | 39.672 | [4, 6] | 39.662 |
| `1 2 - 4 5 6 - - -` | 27 | 11 | [5, 6] | 39.672 | [5, 6] | 39.662 |
| `1 2 - 4 5 6 - - -` | 27 | 12 | [2, 4, 6] | 39.672 | [1, 5, 6] | 39.662 |
| `1 2 3 - 5 - 7 - -` | 27 | 2 | [2] | 39.467 | [2] | 39.418 |
| `1 2 3 - 5 - 7 - -` | 27 | 3 | [3] | 39.467 | [3] | 39.418 |
| `1 2 3 - 5 - 7 - -` | 27 | 4 | [1, 3] | 39.467 | [1, 3] | 39.418 |
| `1 2 3 - 5 - 7 - -` | 27 | 5 | [5] | 39.467 | [5] | 39.418 |
| `1 2 3 - 5 - 7 - -` | 27 | 6 | [1, 5] | 39.467 | [1, 5] | 39.418 |
| `1 2 3 - 5 - 7 - -` | 27 | 7 | [7] | 39.467 | [7] | 39.418 |
| `1 2 3 - 5 - 7 - -` | 27 | 8 | [3, 5] | 39.467 | [1, 7] | 39.418 |
| `1 2 3 - 5 - 7 - -` | 27 | 9 | [2, 7] | 39.467 | [2, 7] | 39.418 |
| `1 2 3 - 5 - 7 - -` | 27 | 10 | [3, 7] | 39.467 | [3, 7] | 39.418 |
| `1 2 3 - 5 - 7 - -` | 27 | 11 | [1, 3, 7] | 39.467 | [1, 3, 7] | 39.418 |
| `1 2 3 - 5 - 7 - -` | 27 | 12 | [5, 7] | 39.467 | [5, 7] | 39.418 |
| `1 2 3 4 - - - 8 -` | 27 | 2 | [2] | 38.835 | [2] | 38.828 |
| `1 2 3 4 - - - 8 -` | 27 | 3 | [3] | 38.835 | [3] | 38.828 |
| `1 2 3 4 - - - 8 -` | 27 | 4 | [4] | 38.835 | [4] | 38.828 |
| `1 2 3 4 - - - 8 -` | 27 | 5 | [2, 3] | 38.835 | [1, 4] | 38.828 |
| `1 2 3 4 - - - 8 -` | 27 | 6 | [2, 4] | 38.835 | [2, 4] | 38.828 |
| `1 2 3 4 - - - 8 -` | 27 | 7 | [3, 4] | 38.835 | [3, 4] | 38.828 |
| `1 2 3 4 - - - 8 -` | 27 | 8 | [8] | 38.835 | [8] | 38.828 |
| `1 2 3 4 - - - 8 -` | 27 | 9 | [1, 8] | 38.835 | [1, 8] | 38.828 |
| `1 2 3 4 - - - 8 -` | 27 | 10 | [2, 8] | 38.835 | [2, 8] | 38.828 |
| `1 2 3 4 - - - 8 -` | 27 | 11 | [3, 8] | 38.835 | [3, 8] | 38.828 |
| `1 2 3 4 - - - 8 -` | 27 | 12 | [4, 8] | 38.835 | [4, 8] | 38.828 |
| `- - - - - - - 8 9` | 28 | 8 | [8] | 30.373 | [8] | 30.373 |
| `- - - - - - - 8 9` | 28 | 9 | [9] | 30.373 | [9] | 30.373 |
| `- - - 4 - 6 7 - -` | 28 | 4 | [4] | 32.780 | [4] | 32.780 |
| `- - - 4 - 6 7 - -` | 28 | 6 | [6] | 32.780 | [6] | 32.780 |
| `- - - 4 - 6 7 - -` | 28 | 7 | [7] | 32.780 | [7] | 32.780 |
| `- - - 4 - 6 7 - -` | 28 | 10 | [4, 6] | 32.780 | [4, 6] | 32.780 |
| `- - - 4 - 6 7 - -` | 28 | 11 | [4, 7] | 32.780 | [4, 7] | 32.780 |
| `- - - 4 5 - - 8 -` | 28 | 4 | [4] | 32.115 | [4] | 32.115 |
| `- - - 4 5 - - 8 -` | 28 | 5 | [5] | 32.115 | [5] | 32.115 |
| `- - - 4 5 - - 8 -` | 28 | 8 | [8] | 32.115 | [8] | 32.115 |
| `- - - 4 5 - - 8 -` | 28 | 9 | [4, 5] | 32.115 | [4, 5] | 32.115 |
| `- - - 4 5 - - 8 -` | 28 | 12 | [4, 8] | 32.115 | [4, 8] | 32.115 |
| `- - 3 - - 6 - 8 -` | 28 | 3 | [3] | 32.577 | [3] | 32.577 |
| `- - 3 - - 6 - 8 -` | 28 | 6 | [6] | 32.577 | [6] | 32.577 |
| `- - 3 - - 6 - 8 -` | 28 | 8 | [8] | 32.577 | [8] | 32.577 |
| `- - 3 - - 6 - 8 -` | 28 | 9 | [3, 6] | 32.577 | [3, 6] | 32.577 |
| `- - 3 - - 6 - 8 -` | 28 | 11 | [3, 8] | 32.577 | [3, 8] | 32.577 |
| `- - 3 - 5 - - - 9` | 28 | 3 | [3] | 31.801 | [3] | 31.801 |
| `- - 3 - 5 - - - 9` | 28 | 5 | [5] | 31.801 | [5] | 31.801 |
| `- - 3 - 5 - - - 9` | 28 | 8 | [3, 5] | 31.801 | [3, 5] | 31.801 |
| `- - 3 - 5 - - - 9` | 28 | 9 | [9] | 31.801 | [9] | 31.801 |
| `- - 3 - 5 - - - 9` | 28 | 12 | [3, 9] | 31.801 | [3, 9] | 31.801 |
| `- 2 - - - - 7 8 -` | 28 | 2 | [2] | 33.114 | [2] | 33.114 |
| `- 2 - - - - 7 8 -` | 28 | 7 | [7] | 33.114 | [7] | 33.114 |
| `- 2 - - - - 7 8 -` | 28 | 8 | [8] | 33.114 | [8] | 33.114 |
| `- 2 - - - - 7 8 -` | 28 | 9 | [2, 7] | 33.114 | [2, 7] | 33.114 |
| `- 2 - - - - 7 8 -` | 28 | 10 | [2, 8] | 33.114 | [2, 8] | 33.114 |
| `- 2 - - - 6 - - 9` | 28 | 2 | [2] | 32.316 | [2] | 32.316 |
| `- 2 - - - 6 - - 9` | 28 | 6 | [6] | 32.316 | [6] | 32.316 |
| `- 2 - - - 6 - - 9` | 28 | 8 | [2, 6] | 32.316 | [2, 6] | 32.316 |
| `- 2 - - - 6 - - 9` | 28 | 9 | [9] | 32.316 | [9] | 32.316 |
| `- 2 - - - 6 - - 9` | 28 | 11 | [2, 9] | 32.316 | [2, 9] | 32.316 |
| `- 2 - 4 5 6 - - -` | 28 | 2 | [2] | 37.621 | [2] | 37.621 |
| `- 2 - 4 5 6 - - -` | 28 | 4 | [4] | 37.621 | [4] | 37.621 |
| `- 2 - 4 5 6 - - -` | 28 | 5 | [5] | 37.621 | [5] | 37.621 |
| `- 2 - 4 5 6 - - -` | 28 | 6 | [6] | 37.621 | [6] | 37.621 |
| `- 2 - 4 5 6 - - -` | 28 | 7 | [2, 5] | 37.621 | [2, 5] | 37.621 |
| `- 2 - 4 5 6 - - -` | 28 | 8 | [2, 6] | 37.621 | [2, 6] | 37.621 |
| `- 2 - 4 5 6 - - -` | 28 | 9 | [4, 5] | 37.621 | [4, 5] | 37.621 |
| `- 2 - 4 5 6 - - -` | 28 | 10 | [4, 6] | 37.621 | [4, 6] | 37.621 |
| `- 2 - 4 5 6 - - -` | 28 | 11 | [5, 6] | 37.621 | [5, 6] | 37.621 |
| `- 2 - 4 5 6 - - -` | 28 | 12 | [2, 4, 6] | 37.621 | [2, 4, 6] | 37.621 |
| `- 2 3 - 5 - 7 - -` | 28 | 2 | [2] | 35.472 | [2] | 35.472 |
| `- 2 3 - 5 - 7 - -` | 28 | 3 | [3] | 35.472 | [3] | 35.472 |
| `- 2 3 - 5 - 7 - -` | 28 | 5 | [5] | 35.472 | [5] | 35.472 |
| `- 2 3 - 5 - 7 - -` | 28 | 7 | [7] | 35.472 | [7] | 35.472 |
| `- 2 3 - 5 - 7 - -` | 28 | 8 | [3, 5] | 35.472 | [3, 5] | 35.472 |
| `- 2 3 - 5 - 7 - -` | 28 | 9 | [2, 7] | 35.472 | [2, 7] | 35.472 |
| `- 2 3 - 5 - 7 - -` | 28 | 10 | [3, 7] | 35.472 | [3, 7] | 35.472 |
| `- 2 3 - 5 - 7 - -` | 28 | 12 | [5, 7] | 35.472 | [5, 7] | 35.472 |
| `- 2 3 4 - - - 8 -` | 28 | 2 | [2] | 37.451 | [2] | 37.451 |
| `- 2 3 4 - - - 8 -` | 28 | 3 | [3] | 37.451 | [3] | 37.451 |
| `- 2 3 4 - - - 8 -` | 28 | 4 | [4] | 37.451 | [4] | 37.451 |
| `- 2 3 4 - - - 8 -` | 28 | 5 | [2, 3] | 37.451 | [2, 3] | 37.451 |
| `- 2 3 4 - - - 8 -` | 28 | 6 | [2, 4] | 37.451 | [2, 4] | 37.451 |
| `- 2 3 4 - - - 8 -` | 28 | 7 | [3, 4] | 37.451 | [3, 4] | 37.451 |
| `- 2 3 4 - - - 8 -` | 28 | 8 | [8] | 37.451 | [8] | 37.451 |
| `- 2 3 4 - - - 8 -` | 28 | 9 | [2, 3, 4] | 37.451 | [2, 3, 4] | 37.451 |
| `- 2 3 4 - - - 8 -` | 28 | 10 | [2, 8] | 37.451 | [2, 8] | 37.451 |
| `- 2 3 4 - - - 8 -` | 28 | 11 | [3, 8] | 37.451 | [3, 8] | 37.451 |
| `- 2 3 4 - - - 8 -` | 28 | 12 | [4, 8] | 37.451 | [4, 8] | 37.451 |
| `1 - - - - - 7 - 9` | 28 | 7 | [7] | 32.906 | [7] | 32.906 |
| `1 - - - - - 7 - 9` | 28 | 8 | [1, 7] | 32.906 | [1, 7] | 32.906 |
| `1 - - - - - 7 - 9` | 28 | 9 | [9] | 32.906 | [9] | 32.906 |
| `1 - - - - - 7 - 9` | 28 | 10 | [1, 9] | 32.906 | [1, 9] | 32.906 |
| `1 - - 4 5 - 7 - -` | 28 | 4 | [4] | 37.362 | [4] | 37.362 |
| `1 - - 4 5 - 7 - -` | 28 | 5 | [5] | 37.362 | [5] | 37.362 |
| `1 - - 4 5 - 7 - -` | 28 | 6 | [1, 5] | 37.362 | [1, 5] | 37.362 |
| `1 - - 4 5 - 7 - -` | 28 | 7 | [7] | 37.362 | [7] | 37.362 |
| `1 - - 4 5 - 7 - -` | 28 | 8 | [1, 7] | 37.362 | [1, 7] | 37.362 |
| `1 - - 4 5 - 7 - -` | 28 | 9 | [4, 5] | 37.362 | [4, 5] | 37.362 |
| `1 - - 4 5 - 7 - -` | 28 | 10 | [1, 4, 5] | 37.362 | [1, 4, 5] | 37.362 |
| `1 - - 4 5 - 7 - -` | 28 | 11 | [4, 7] | 37.362 | [4, 7] | 37.362 |
| `1 - - 4 5 - 7 - -` | 28 | 12 | [5, 7] | 37.362 | [5, 7] | 37.362 |
| `1 - 3 - - 6 7 - -` | 28 | 3 | [3] | 36.617 | [3] | 36.617 |
| `1 - 3 - - 6 7 - -` | 28 | 4 | [1, 3] | 36.617 | [1, 3] | 36.617 |
| `1 - 3 - - 6 7 - -` | 28 | 6 | [6] | 36.617 | [6] | 36.617 |
| `1 - 3 - - 6 7 - -` | 28 | 7 | [7] | 36.617 | [7] | 36.617 |
| `1 - 3 - - 6 7 - -` | 28 | 8 | [1, 7] | 36.617 | [1, 7] | 36.617 |
| `1 - 3 - - 6 7 - -` | 28 | 9 | [3, 6] | 36.617 | [3, 6] | 36.617 |
| `1 - 3 - - 6 7 - -` | 28 | 10 | [3, 7] | 36.617 | [3, 7] | 36.617 |
| `1 - 3 - - 6 7 - -` | 28 | 11 | [1, 3, 7] | 36.617 | [1, 3, 7] | 36.617 |
| `1 - 3 - 5 - - 8 -` | 28 | 3 | [3] | 34.918 | [3] | 34.918 |
| `1 - 3 - 5 - - 8 -` | 28 | 4 | [1, 3] | 34.918 | [1, 3] | 34.918 |
| `1 - 3 - 5 - - 8 -` | 28 | 5 | [5] | 34.918 | [5] | 34.918 |
| `1 - 3 - 5 - - 8 -` | 28 | 6 | [1, 5] | 34.918 | [1, 5] | 34.918 |
| `1 - 3 - 5 - - 8 -` | 28 | 8 | [8] | 34.918 | [8] | 34.918 |
| `1 - 3 - 5 - - 8 -` | 28 | 9 | [1, 8] | 34.918 | [1, 8] | 34.918 |
| `1 - 3 - 5 - - 8 -` | 28 | 11 | [3, 8] | 34.918 | [3, 8] | 34.918 |
| `1 - 3 - 5 - - 8 -` | 28 | 12 | [1, 3, 8] | 34.918 | [1, 3, 8] | 34.918 |
| `1 - 3 4 - - - - 9` | 28 | 3 | [3] | 35.085 | [3] | 35.085 |
| `1 - 3 4 - - - - 9` | 28 | 4 | [4] | 35.085 | [4] | 35.085 |
| `1 - 3 4 - - - - 9` | 28 | 5 | [1, 4] | 35.085 | [1, 4] | 35.085 |
| `1 - 3 4 - - - - 9` | 28 | 7 | [3, 4] | 35.085 | [3, 4] | 35.085 |
| `1 - 3 4 - - - - 9` | 28 | 8 | [1, 3, 4] | 35.085 | [1, 3, 4] | 35.085 |
| `1 - 3 4 - - - - 9` | 28 | 9 | [9] | 35.085 | [9] | 35.085 |
| `1 - 3 4 - - - - 9` | 28 | 10 | [1, 9] | 35.085 | [1, 9] | 35.085 |
| `1 - 3 4 - - - - 9` | 28 | 12 | [3, 9] | 35.085 | [3, 9] | 35.085 |
| `1 2 - - - 6 - 8 -` | 28 | 2 | [2] | 35.985 | [2] | 35.985 |
| `1 2 - - - 6 - 8 -` | 28 | 3 | [1, 2] | 35.985 | [1, 2] | 35.985 |
| `1 2 - - - 6 - 8 -` | 28 | 6 | [6] | 35.985 | [6] | 35.985 |
| `1 2 - - - 6 - 8 -` | 28 | 7 | [1, 6] | 35.985 | [1, 6] | 35.985 |
| `1 2 - - - 6 - 8 -` | 28 | 8 | [8] | 35.985 | [8] | 35.985 |
| `1 2 - - - 6 - 8 -` | 28 | 9 | [1, 8] | 35.985 | [1, 8] | 35.985 |
| `1 2 - - - 6 - 8 -` | 28 | 10 | [2, 8] | 35.985 | [2, 8] | 35.985 |
| `1 2 - - - 6 - 8 -` | 28 | 11 | [1, 2, 8] | 35.985 | [1, 2, 8] | 35.985 |
| `1 2 - - 5 - - - 9` | 28 | 2 | [2] | 36.582 | [2] | 36.582 |
| `1 2 - - 5 - - - 9` | 28 | 3 | [1, 2] | 36.582 | [1, 2] | 36.582 |
| `1 2 - - 5 - - - 9` | 28 | 5 | [5] | 36.582 | [5] | 36.582 |
| `1 2 - - 5 - - - 9` | 28 | 6 | [1, 5] | 36.582 | [1, 5] | 36.582 |
| `1 2 - - 5 - - - 9` | 28 | 7 | [2, 5] | 36.582 | [2, 5] | 36.582 |
| `1 2 - - 5 - - - 9` | 28 | 8 | [1, 2, 5] | 36.582 | [1, 2, 5] | 36.582 |
| `1 2 - - 5 - - - 9` | 28 | 9 | [9] | 36.582 | [9] | 36.582 |
| `1 2 - - 5 - - - 9` | 28 | 10 | [1, 9] | 36.582 | [1, 9] | 36.582 |
| `1 2 - - 5 - - - 9` | 28 | 11 | [2, 9] | 36.582 | [2, 9] | 36.582 |
| `1 2 - - 5 - - - 9` | 28 | 12 | [1, 2, 9] | 36.582 | [1, 2, 9] | 36.582 |
| `1 2 3 - 5 6 - - -` | 28 | 2 | [2] | 40.217 | [2] | 40.086 |
| `1 2 3 - 5 6 - - -` | 28 | 3 | [3] | 40.217 | [3] | 40.086 |
| `1 2 3 - 5 6 - - -` | 28 | 4 | [1, 3] | 40.217 | [1, 3] | 40.086 |
| `1 2 3 - 5 6 - - -` | 28 | 5 | [5] | 40.217 | [5] | 40.086 |
| `1 2 3 - 5 6 - - -` | 28 | 6 | [6] | 40.217 | [6] | 40.086 |
| `1 2 3 - 5 6 - - -` | 28 | 7 | [2, 5] | 40.217 | [1, 6] | 40.086 |
| `1 2 3 - 5 6 - - -` | 28 | 8 | [3, 5] | 40.217 | [2, 6] | 40.086 |
| `1 2 3 - 5 6 - - -` | 28 | 9 | [3, 6] | 40.217 | [3, 6] | 40.086 |
| `1 2 3 - 5 6 - - -` | 28 | 10 | [2, 3, 5] | 40.217 | [1, 3, 6] | 40.086 |
| `1 2 3 - 5 6 - - -` | 28 | 11 | [5, 6] | 40.217 | [5, 6] | 40.086 |
| `1 2 3 - 5 6 - - -` | 28 | 12 | [1, 5, 6] | 40.217 | [1, 5, 6] | 40.086 |
| `1 2 3 4 - - 7 - -` | 28 | 2 | [2] | 40.054 | [2] | 40.044 |
| `1 2 3 4 - - 7 - -` | 28 | 3 | [3] | 40.054 | [3] | 40.044 |
| `1 2 3 4 - - 7 - -` | 28 | 4 | [4] | 40.054 | [4] | 40.044 |
| `1 2 3 4 - - 7 - -` | 28 | 5 | [2, 3] | 40.054 | [1, 4] | 40.044 |
| `1 2 3 4 - - 7 - -` | 28 | 6 | [2, 4] | 40.054 | [2, 4] | 40.044 |
| `1 2 3 4 - - 7 - -` | 28 | 7 | [7] | 40.054 | [7] | 40.044 |
| `1 2 3 4 - - 7 - -` | 28 | 8 | [1, 7] | 40.054 | [1, 7] | 40.044 |
| `1 2 3 4 - - 7 - -` | 28 | 9 | [2, 7] | 40.054 | [2, 7] | 40.044 |
| `1 2 3 4 - - 7 - -` | 28 | 10 | [3, 7] | 40.054 | [3, 7] | 40.044 |
| `1 2 3 4 - - 7 - -` | 28 | 11 | [4, 7] | 40.054 | [4, 7] | 40.044 |
| `1 2 3 4 - - 7 - -` | 28 | 12 | [2, 3, 7] | 40.054 | [1, 4, 7] | 40.044 |
| `- - - - - - 7 - 9` | 29 | 7 | [7] | 31.463 | [7] | 31.463 |
| `- - - - - - 7 - 9` | 29 | 9 | [9] | 31.463 | [9] | 31.463 |
| `- - - 4 5 - 7 - -` | 29 | 4 | [4] | 33.940 | [4] | 33.940 |
| `- - - 4 5 - 7 - -` | 29 | 5 | [5] | 33.940 | [5] | 33.940 |
| `- - - 4 5 - 7 - -` | 29 | 7 | [7] | 33.940 | [7] | 33.940 |
| `- - - 4 5 - 7 - -` | 29 | 9 | [4, 5] | 33.940 | [4, 5] | 33.940 |
| `- - - 4 5 - 7 - -` | 29 | 11 | [4, 7] | 33.940 | [4, 7] | 33.940 |
| `- - - 4 5 - 7 - -` | 29 | 12 | [5, 7] | 33.940 | [5, 7] | 33.940 |
| `- - 3 - - 6 7 - -` | 29 | 3 | [3] | 33.986 | [3] | 33.986 |
| `- - 3 - - 6 7 - -` | 29 | 6 | [6] | 33.986 | [6] | 33.986 |
| `- - 3 - - 6 7 - -` | 29 | 7 | [7] | 33.986 | [7] | 33.986 |
| `- - 3 - - 6 7 - -` | 29 | 9 | [3, 6] | 33.986 | [3, 6] | 33.986 |
| `- - 3 - - 6 7 - -` | 29 | 10 | [3, 7] | 33.986 | [3, 7] | 33.986 |
| `- - 3 - 5 - - 8 -` | 29 | 3 | [3] | 32.060 | [3] | 32.060 |
| `- - 3 - 5 - - 8 -` | 29 | 5 | [5] | 32.060 | [5] | 32.060 |
| `- - 3 - 5 - - 8 -` | 29 | 8 | [8] | 32.060 | [8] | 32.060 |
| `- - 3 - 5 - - 8 -` | 29 | 11 | [3, 8] | 32.060 | [3, 8] | 32.060 |
| `- - 3 4 - - - - 9` | 29 | 3 | [3] | 32.577 | [3] | 32.577 |
| `- - 3 4 - - - - 9` | 29 | 4 | [4] | 32.577 | [4] | 32.577 |
| `- - 3 4 - - - - 9` | 29 | 7 | [3, 4] | 32.577 | [3, 4] | 32.577 |
| `- - 3 4 - - - - 9` | 29 | 9 | [9] | 32.577 | [9] | 32.577 |
| `- - 3 4 - - - - 9` | 29 | 12 | [3, 9] | 32.577 | [3, 9] | 32.577 |
| `- 2 - - - 6 - 8 -` | 29 | 2 | [2] | 32.529 | [2] | 32.529 |
| `- 2 - - - 6 - 8 -` | 29 | 6 | [6] | 32.529 | [6] | 32.529 |
| `- 2 - - - 6 - 8 -` | 29 | 8 | [8] | 32.529 | [8] | 32.529 |
| `- 2 - - - 6 - 8 -` | 29 | 10 | [2, 8] | 32.529 | [2, 8] | 32.529 |
| `- 2 - - 5 - - - 9` | 29 | 2 | [2] | 33.023 | [2] | 33.023 |
| `- 2 - - 5 - - - 9` | 29 | 5 | [5] | 33.023 | [5] | 33.023 |
| `- 2 - - 5 - - - 9` | 29 | 7 | [2, 5] | 33.023 | [2, 5] | 33.023 |
| `- 2 - - 5 - - - 9` | 29 | 9 | [9] | 33.023 | [9] | 33.023 |
| `- 2 - - 5 - - - 9` | 29 | 11 | [2, 9] | 33.023 | [2, 9] | 33.023 |
| `- 2 3 - 5 6 - - -` | 29 | 2 | [2] | 37.801 | [2] | 37.781 |
| `- 2 3 - 5 6 - - -` | 29 | 3 | [3] | 37.801 | [3] | 37.781 |
| `- 2 3 - 5 6 - - -` | 29 | 5 | [5] | 37.801 | [5] | 37.781 |
| `- 2 3 - 5 6 - - -` | 29 | 6 | [6] | 37.801 | [6] | 37.781 |
| `- 2 3 - 5 6 - - -` | 29 | 7 | [2, 5] | 37.801 | [2, 5] | 37.781 |
| `- 2 3 - 5 6 - - -` | 29 | 8 | [3, 5] | 37.801 | [2, 6] | 37.781 |
| `- 2 3 - 5 6 - - -` | 29 | 9 | [3, 6] | 37.801 | [3, 6] | 37.781 |
| `- 2 3 - 5 6 - - -` | 29 | 10 | [2, 3, 5] | 37.801 | [2, 3, 5] | 37.781 |
| `- 2 3 - 5 6 - - -` | 29 | 11 | [5, 6] | 37.801 | [5, 6] | 37.781 |
| `- 2 3 4 - - 7 - -` | 29 | 2 | [2] | 37.271 | [2] | 37.271 |
| `- 2 3 4 - - 7 - -` | 29 | 3 | [3] | 37.271 | [3] | 37.271 |
| `- 2 3 4 - - 7 - -` | 29 | 4 | [4] | 37.271 | [4] | 37.271 |
| `- 2 3 4 - - 7 - -` | 29 | 5 | [2, 3] | 37.271 | [2, 3] | 37.271 |
| `- 2 3 4 - - 7 - -` | 29 | 6 | [2, 4] | 37.271 | [2, 4] | 37.271 |
| `- 2 3 4 - - 7 - -` | 29 | 7 | [7] | 37.271 | [7] | 37.271 |
| `- 2 3 4 - - 7 - -` | 29 | 9 | [2, 7] | 37.271 | [2, 7] | 37.271 |
| `- 2 3 4 - - 7 - -` | 29 | 10 | [3, 7] | 37.271 | [3, 7] | 37.271 |
| `- 2 3 4 - - 7 - -` | 29 | 11 | [4, 7] | 37.271 | [4, 7] | 37.271 |
| `- 2 3 4 - - 7 - -` | 29 | 12 | [2, 3, 7] | 37.271 | [2, 3, 7] | 37.271 |
| `1 - - - - - 7 8 -` | 29 | 7 | [7] | 33.076 | [7] | 33.076 |
| `1 - - - - - 7 8 -` | 29 | 8 | [8] | 33.076 | [8] | 33.076 |
| `1 - - - - - 7 8 -` | 29 | 9 | [1, 8] | 33.076 | [1, 8] | 33.076 |
| `1 - - - - 6 - - 9` | 29 | 6 | [6] | 33.546 | [6] | 33.546 |
| `1 - - - - 6 - - 9` | 29 | 7 | [1, 6] | 33.546 | [1, 6] | 33.546 |
| `1 - - - - 6 - - 9` | 29 | 9 | [9] | 33.546 | [9] | 33.546 |
| `1 - - - - 6 - - 9` | 29 | 10 | [1, 9] | 33.546 | [1, 9] | 33.546 |
| `1 - - 4 5 6 - - -` | 29 | 4 | [4] | 36.786 | [4] | 36.786 |
| `1 - - 4 5 6 - - -` | 29 | 5 | [5] | 36.786 | [5] | 36.786 |
| `1 - - 4 5 6 - - -` | 29 | 6 | [6] | 36.786 | [6] | 36.786 |
| `1 - - 4 5 6 - - -` | 29 | 7 | [1, 6] | 36.786 | [1, 6] | 36.786 |
| `1 - - 4 5 6 - - -` | 29 | 9 | [4, 5] | 36.786 | [4, 5] | 36.786 |
| `1 - - 4 5 6 - - -` | 29 | 10 | [4, 6] | 36.786 | [4, 6] | 36.786 |
| `1 - - 4 5 6 - - -` | 29 | 11 | [5, 6] | 36.786 | [5, 6] | 36.786 |
| `1 - - 4 5 6 - - -` | 29 | 12 | [1, 5, 6] | 36.786 | [1, 5, 6] | 36.786 |
| `1 - 3 - 5 - 7 - -` | 29 | 3 | [3] | 38.523 | [3] | 38.468 |
| `1 - 3 - 5 - 7 - -` | 29 | 4 | [1, 3] | 38.523 | [1, 3] | 38.468 |
| `1 - 3 - 5 - 7 - -` | 29 | 5 | [5] | 38.523 | [5] | 38.468 |
| `1 - 3 - 5 - 7 - -` | 29 | 6 | [1, 5] | 38.523 | [1, 5] | 38.468 |
| `1 - 3 - 5 - 7 - -` | 29 | 7 | [7] | 38.523 | [7] | 38.468 |
| `1 - 3 - 5 - 7 - -` | 29 | 8 | [3, 5] | 38.523 | [1, 7] | 38.468 |
| `1 - 3 - 5 - 7 - -` | 29 | 9 | [1, 3, 5] | 38.523 | [1, 3, 5] | 38.468 |
| `1 - 3 - 5 - 7 - -` | 29 | 10 | [3, 7] | 38.523 | [3, 7] | 38.468 |
| `1 - 3 - 5 - 7 - -` | 29 | 11 | [1, 3, 7] | 38.523 | [1, 3, 7] | 38.468 |
| `1 - 3 - 5 - 7 - -` | 29 | 12 | [5, 7] | 38.523 | [5, 7] | 38.468 |
| `1 - 3 4 - - - 8 -` | 29 | 3 | [3] | 36.111 | [3] | 36.111 |
| `1 - 3 4 - - - 8 -` | 29 | 4 | [4] | 36.111 | [4] | 36.111 |
| `1 - 3 4 - - - 8 -` | 29 | 5 | [1, 4] | 36.111 | [1, 4] | 36.111 |
| `1 - 3 4 - - - 8 -` | 29 | 7 | [3, 4] | 36.111 | [3, 4] | 36.111 |
| `1 - 3 4 - - - 8 -` | 29 | 8 | [8] | 36.111 | [8] | 36.111 |
| `1 - 3 4 - - - 8 -` | 29 | 9 | [1, 8] | 36.111 | [1, 8] | 36.111 |
| `1 - 3 4 - - - 8 -` | 29 | 11 | [3, 8] | 36.111 | [3, 8] | 36.111 |
| `1 - 3 4 - - - 8 -` | 29 | 12 | [4, 8] | 36.111 | [4, 8] | 36.111 |
| `1 2 - - - 6 7 - -` | 29 | 2 | [2] | 36.382 | [2] | 36.348 |
| `1 2 - - - 6 7 - -` | 29 | 3 | [1, 2] | 36.382 | [1, 2] | 36.348 |
| `1 2 - - - 6 7 - -` | 29 | 6 | [6] | 36.382 | [6] | 36.348 |
| `1 2 - - - 6 7 - -` | 29 | 7 | [7] | 36.382 | [7] | 36.348 |
| `1 2 - - - 6 7 - -` | 29 | 8 | [2, 6] | 36.382 | [1, 7] | 36.348 |
| `1 2 - - - 6 7 - -` | 29 | 9 | [2, 7] | 36.382 | [2, 7] | 36.348 |
| `1 2 - - - 6 7 - -` | 29 | 10 | [1, 2, 7] | 36.382 | [1, 2, 7] | 36.348 |
| `1 2 - - 5 - - 8 -` | 29 | 2 | [2] | 37.525 | [2] | 37.525 |
| `1 2 - - 5 - - 8 -` | 29 | 3 | [1, 2] | 37.525 | [1, 2] | 37.525 |
| `1 2 - - 5 - - 8 -` | 29 | 5 | [5] | 37.525 | [5] | 37.525 |
| `1 2 - - 5 - - 8 -` | 29 | 6 | [1, 5] | 37.525 | [1, 5] | 37.525 |
| `1 2 - - 5 - - 8 -` | 29 | 7 | [2, 5] | 37.525 | [2, 5] | 37.525 |
| `1 2 - - 5 - - 8 -` | 29 | 8 | [8] | 37.525 | [8] | 37.525 |
| `1 2 - - 5 - - 8 -` | 29 | 9 | [1, 8] | 37.525 | [1, 8] | 37.525 |
| `1 2 - - 5 - - 8 -` | 29 | 10 | [2, 8] | 37.525 | [2, 8] | 37.525 |
| `1 2 - - 5 - - 8 -` | 29 | 11 | [1, 2, 8] | 37.525 | [1, 2, 8] | 37.525 |
| `1 2 - 4 - - - - 9` | 29 | 2 | [2] | 36.443 | [2] | 36.443 |
| `1 2 - 4 - - - - 9` | 29 | 3 | [1, 2] | 36.443 | [1, 2] | 36.443 |
| `1 2 - 4 - - - - 9` | 29 | 4 | [4] | 36.443 | [4] | 36.443 |
| `1 2 - 4 - - - - 9` | 29 | 5 | [1, 4] | 36.443 | [1, 4] | 36.443 |
| `1 2 - 4 - - - - 9` | 29 | 6 | [2, 4] | 36.443 | [2, 4] | 36.443 |
| `1 2 - 4 - - - - 9` | 29 | 7 | [1, 2, 4] | 36.443 | [1, 2, 4] | 36.443 |
| `1 2 - 4 - - - - 9` | 29 | 9 | [9] | 36.443 | [9] | 36.443 |
| `1 2 - 4 - - - - 9` | 29 | 10 | [1, 9] | 36.443 | [1, 9] | 36.443 |
| `1 2 - 4 - - - - 9` | 29 | 11 | [2, 9] | 36.443 | [2, 9] | 36.443 |
| `1 2 - 4 - - - - 9` | 29 | 12 | [1, 2, 9] | 36.443 | [1, 2, 9] | 36.443 |
| `1 2 3 4 - 6 - - -` | 29 | 2 | [2] | 40.676 | [2] | 40.644 |
| `1 2 3 4 - 6 - - -` | 29 | 3 | [3] | 40.676 | [3] | 40.644 |
| `1 2 3 4 - 6 - - -` | 29 | 4 | [4] | 40.676 | [4] | 40.644 |
| `1 2 3 4 - 6 - - -` | 29 | 5 | [2, 3] | 40.676 | [1, 4] | 40.644 |
| `1 2 3 4 - 6 - - -` | 29 | 6 | [6] | 40.676 | [6] | 40.644 |
| `1 2 3 4 - 6 - - -` | 29 | 7 | [3, 4] | 40.676 | [1, 6] | 40.644 |
| `1 2 3 4 - 6 - - -` | 29 | 8 | [2, 6] | 40.676 | [2, 6] | 40.644 |
| `1 2 3 4 - 6 - - -` | 29 | 9 | [3, 6] | 40.676 | [3, 6] | 40.644 |
| `1 2 3 4 - 6 - - -` | 29 | 10 | [4, 6] | 40.676 | [4, 6] | 40.644 |
| `1 2 3 4 - 6 - - -` | 29 | 11 | [2, 3, 6] | 40.676 | [1, 4, 6] | 40.644 |
| `1 2 3 4 - 6 - - -` | 29 | 12 | [2, 4, 6] | 40.676 | [2, 4, 6] | 40.644 |
| `- - - - - - 7 8 -` | 30 | 7 | [7] | 32.625 | [7] | 32.625 |
| `- - - - - - 7 8 -` | 30 | 8 | [8] | 32.625 | [8] | 32.625 |
| `- - - - - 6 - - 9` | 30 | 6 | [6] | 32.065 | [6] | 32.065 |
| `- - - - - 6 - - 9` | 30 | 9 | [9] | 32.065 | [9] | 32.065 |
| `- - - 4 5 6 - - -` | 30 | 4 | [4] | 35.014 | [4] | 35.014 |
| `- - - 4 5 6 - - -` | 30 | 5 | [5] | 35.014 | [5] | 35.014 |
| `- - - 4 5 6 - - -` | 30 | 6 | [6] | 35.014 | [6] | 35.014 |
| `- - - 4 5 6 - - -` | 30 | 9 | [4, 5] | 35.014 | [4, 5] | 35.014 |
| `- - - 4 5 6 - - -` | 30 | 10 | [4, 6] | 35.014 | [4, 6] | 35.014 |
| `- - - 4 5 6 - - -` | 30 | 11 | [5, 6] | 35.014 | [5, 6] | 35.014 |
| `- - 3 - 5 - 7 - -` | 30 | 3 | [3] | 35.071 | [3] | 35.071 |
| `- - 3 - 5 - 7 - -` | 30 | 5 | [5] | 35.071 | [5] | 35.071 |
| `- - 3 - 5 - 7 - -` | 30 | 7 | [7] | 35.071 | [7] | 35.071 |
| `- - 3 - 5 - 7 - -` | 30 | 8 | [3, 5] | 35.071 | [3, 5] | 35.071 |
| `- - 3 - 5 - 7 - -` | 30 | 10 | [3, 7] | 35.071 | [3, 7] | 35.071 |
| `- - 3 - 5 - 7 - -` | 30 | 12 | [5, 7] | 35.071 | [5, 7] | 35.071 |
| `- - 3 4 - - - 8 -` | 30 | 3 | [3] | 34.438 | [3] | 34.438 |
| `- - 3 4 - - - 8 -` | 30 | 4 | [4] | 34.438 | [4] | 34.438 |
| `- - 3 4 - - - 8 -` | 30 | 7 | [3, 4] | 34.438 | [3, 4] | 34.438 |
| `- - 3 4 - - - 8 -` | 30 | 8 | [8] | 34.438 | [8] | 34.438 |
| `- - 3 4 - - - 8 -` | 30 | 11 | [3, 8] | 34.438 | [3, 8] | 34.438 |
| `- - 3 4 - - - 8 -` | 30 | 12 | [4, 8] | 34.438 | [4, 8] | 34.438 |
| `- 2 - - - 6 7 - -` | 30 | 2 | [2] | 35.138 | [2] | 35.138 |
| `- 2 - - - 6 7 - -` | 30 | 6 | [6] | 35.138 | [6] | 35.138 |
| `- 2 - - - 6 7 - -` | 30 | 7 | [7] | 35.138 | [7] | 35.138 |
| `- 2 - - - 6 7 - -` | 30 | 8 | [2, 6] | 35.138 | [2, 6] | 35.138 |
| `- 2 - - - 6 7 - -` | 30 | 9 | [2, 7] | 35.138 | [2, 7] | 35.138 |
| `- 2 - - 5 - - 8 -` | 30 | 2 | [2] | 34.482 | [2] | 34.482 |
| `- 2 - - 5 - - 8 -` | 30 | 5 | [5] | 34.482 | [5] | 34.482 |
| `- 2 - - 5 - - 8 -` | 30 | 7 | [2, 5] | 34.482 | [2, 5] | 34.482 |
| `- 2 - - 5 - - 8 -` | 30 | 8 | [8] | 34.482 | [8] | 34.482 |
| `- 2 - - 5 - - 8 -` | 30 | 10 | [2, 8] | 34.482 | [2, 8] | 34.482 |
| `- 2 - 4 - - - - 9` | 30 | 2 | [2] | 33.310 | [2] | 33.310 |
| `- 2 - 4 - - - - 9` | 30 | 4 | [4] | 33.310 | [4] | 33.310 |
| `- 2 - 4 - - - - 9` | 30 | 6 | [2, 4] | 33.310 | [2, 4] | 33.310 |
| `- 2 - 4 - - - - 9` | 30 | 9 | [9] | 33.310 | [9] | 33.310 |
| `- 2 - 4 - - - - 9` | 30 | 11 | [2, 9] | 33.310 | [2, 9] | 33.310 |
| `- 2 3 4 - 6 - - -` | 30 | 2 | [2] | 39.396 | [2] | 39.396 |
| `- 2 3 4 - 6 - - -` | 30 | 3 | [3] | 39.396 | [3] | 39.396 |
| `- 2 3 4 - 6 - - -` | 30 | 4 | [4] | 39.396 | [4] | 39.396 |
| `- 2 3 4 - 6 - - -` | 30 | 5 | [2, 3] | 39.396 | [2, 3] | 39.396 |
| `- 2 3 4 - 6 - - -` | 30 | 6 | [6] | 39.396 | [6] | 39.396 |
| `- 2 3 4 - 6 - - -` | 30 | 7 | [3, 4] | 39.396 | [3, 4] | 39.396 |
| `- 2 3 4 - 6 - - -` | 30 | 8 | [2, 6] | 39.396 | [2, 6] | 39.396 |
| `- 2 3 4 - 6 - - -` | 30 | 9 | [3, 6] | 39.396 | [3, 6] | 39.396 |
| `- 2 3 4 - 6 - - -` | 30 | 10 | [4, 6] | 39.396 | [4, 6] | 39.396 |
| `- 2 3 4 - 6 - - -` | 30 | 11 | [2, 3, 6] | 39.396 | [2, 3, 6] | 39.396 |
| `- 2 3 4 - 6 - - -` | 30 | 12 | [2, 4, 6] | 39.396 | [2, 4, 6] | 39.396 |
| `1 - - - - 6 - 8 -` | 30 | 6 | [6] | 34.960 | [6] | 34.960 |
| `1 - - - - 6 - 8 -` | 30 | 7 | [1, 6] | 34.960 | [1, 6] | 34.960 |
| `1 - - - - 6 - 8 -` | 30 | 8 | [8] | 34.960 | [8] | 34.960 |
| `1 - - - - 6 - 8 -` | 30 | 9 | [1, 8] | 34.960 | [1, 8] | 34.960 |
| `1 - - - 5 - - - 9` | 30 | 5 | [5] | 33.765 | [5] | 33.765 |
| `1 - - - 5 - - - 9` | 30 | 6 | [1, 5] | 33.765 | [1, 5] | 33.765 |
| `1 - - - 5 - - - 9` | 30 | 9 | [9] | 33.765 | [9] | 33.765 |
| `1 - - - 5 - - - 9` | 30 | 10 | [1, 9] | 33.765 | [1, 9] | 33.765 |
| `1 - 3 - 5 6 - - -` | 30 | 3 | [3] | 39.268 | [3] | 39.268 |
| `1 - 3 - 5 6 - - -` | 30 | 4 | [1, 3] | 39.268 | [1, 3] | 39.268 |
| `1 - 3 - 5 6 - - -` | 30 | 5 | [5] | 39.268 | [5] | 39.268 |
| `1 - 3 - 5 6 - - -` | 30 | 6 | [6] | 39.268 | [6] | 39.268 |
| `1 - 3 - 5 6 - - -` | 30 | 7 | [1, 6] | 39.268 | [1, 6] | 39.268 |
| `1 - 3 - 5 6 - - -` | 30 | 8 | [3, 5] | 39.268 | [3, 5] | 39.268 |
| `1 - 3 - 5 6 - - -` | 30 | 9 | [3, 6] | 39.268 | [3, 6] | 39.268 |
| `1 - 3 - 5 6 - - -` | 30 | 10 | [1, 3, 6] | 39.268 | [1, 3, 6] | 39.268 |
| `1 - 3 - 5 6 - - -` | 30 | 11 | [5, 6] | 39.268 | [5, 6] | 39.268 |
| `1 - 3 - 5 6 - - -` | 30 | 12 | [1, 5, 6] | 39.268 | [1, 5, 6] | 39.268 |
| `1 - 3 4 - - 7 - -` | 30 | 3 | [3] | 36.915 | [3] | 36.915 |
| `1 - 3 4 - - 7 - -` | 30 | 4 | [4] | 36.915 | [4] | 36.915 |
| `1 - 3 4 - - 7 - -` | 30 | 5 | [1, 4] | 36.915 | [1, 4] | 36.915 |
| `1 - 3 4 - - 7 - -` | 30 | 7 | [7] | 36.915 | [7] | 36.915 |
| `1 - 3 4 - - 7 - -` | 30 | 8 | [1, 7] | 36.915 | [1, 7] | 36.915 |
| `1 - 3 4 - - 7 - -` | 30 | 10 | [3, 7] | 36.915 | [3, 7] | 36.915 |
| `1 - 3 4 - - 7 - -` | 30 | 11 | [4, 7] | 36.915 | [4, 7] | 36.915 |
| `1 - 3 4 - - 7 - -` | 30 | 12 | [1, 4, 7] | 36.915 | [1, 4, 7] | 36.915 |
| `1 2 - - 5 - 7 - -` | 30 | 2 | [2] | 38.250 | [2] | 38.250 |
| `1 2 - - 5 - 7 - -` | 30 | 3 | [1, 2] | 38.250 | [1, 2] | 38.250 |
| `1 2 - - 5 - 7 - -` | 30 | 5 | [5] | 38.250 | [5] | 38.250 |
| `1 2 - - 5 - 7 - -` | 30 | 6 | [1, 5] | 38.250 | [1, 5] | 38.250 |
| `1 2 - - 5 - 7 - -` | 30 | 7 | [7] | 38.250 | [7] | 38.250 |
| `1 2 - - 5 - 7 - -` | 30 | 8 | [1, 7] | 38.250 | [1, 7] | 38.250 |
| `1 2 - - 5 - 7 - -` | 30 | 9 | [2, 7] | 38.250 | [2, 7] | 38.250 |
| `1 2 - - 5 - 7 - -` | 30 | 10 | [1, 2, 7] | 38.250 | [1, 2, 7] | 38.250 |
| `1 2 - - 5 - 7 - -` | 30 | 12 | [5, 7] | 38.250 | [5, 7] | 38.250 |
| `1 2 - 4 - - - 8 -` | 30 | 2 | [2] | 38.922 | [2] | 38.922 |
| `1 2 - 4 - - - 8 -` | 30 | 3 | [1, 2] | 38.922 | [1, 2] | 38.922 |
| `1 2 - 4 - - - 8 -` | 30 | 4 | [4] | 38.922 | [4] | 38.922 |
| `1 2 - 4 - - - 8 -` | 30 | 5 | [1, 4] | 38.922 | [1, 4] | 38.922 |
| `1 2 - 4 - - - 8 -` | 30 | 6 | [2, 4] | 38.922 | [2, 4] | 38.922 |
| `1 2 - 4 - - - 8 -` | 30 | 7 | [1, 2, 4] | 38.922 | [1, 2, 4] | 38.922 |
| `1 2 - 4 - - - 8 -` | 30 | 8 | [8] | 38.922 | [8] | 38.922 |
| `1 2 - 4 - - - 8 -` | 30 | 9 | [1, 8] | 38.922 | [1, 8] | 38.922 |
| `1 2 - 4 - - - 8 -` | 30 | 10 | [2, 8] | 38.922 | [2, 8] | 38.922 |
| `1 2 - 4 - - - 8 -` | 30 | 11 | [1, 2, 8] | 38.922 | [1, 2, 8] | 38.922 |
| `1 2 - 4 - - - 8 -` | 30 | 12 | [4, 8] | 38.922 | [4, 8] | 38.922 |
| `1 2 3 - - - - - 9` | 30 | 2 | [2] | 35.781 | [2] | 35.781 |
| `1 2 3 - - - - - 9` | 30 | 3 | [3] | 35.781 | [3] | 35.781 |
| `1 2 3 - - - - - 9` | 30 | 4 | [1, 3] | 35.781 | [1, 3] | 35.781 |
| `1 2 3 - - - - - 9` | 30 | 5 | [2, 3] | 35.781 | [2, 3] | 35.781 |
| `1 2 3 - - - - - 9` | 30 | 6 | [1, 2, 3] | 35.781 | [1, 2, 3] | 35.781 |
| `1 2 3 - - - - - 9` | 30 | 9 | [9] | 35.781 | [9] | 35.781 |
| `1 2 3 - - - - - 9` | 30 | 10 | [1, 9] | 35.781 | [1, 9] | 35.781 |
| `1 2 3 - - - - - 9` | 30 | 11 | [2, 9] | 35.781 | [2, 9] | 35.781 |
| `1 2 3 - - - - - 9` | 30 | 12 | [3, 9] | 35.781 | [3, 9] | 35.781 |
| `1 2 3 4 5 - - - -` | 30 | 2 | [2] | 41.101 | [2] | 40.999 |
| `1 2 3 4 5 - - - -` | 30 | 3 | [3] | 41.101 | [3] | 40.999 |
| `1 2 3 4 5 - - - -` | 30 | 4 | [4] | 41.101 | [4] | 40.999 |
| `1 2 3 4 5 - - - -` | 30 | 5 | [5] | 41.101 | [5] | 40.999 |
| `1 2 3 4 5 - - - -` | 30 | 6 | [1, 5] | 41.101 | [1, 5] | 40.999 |
| `1 2 3 4 5 - - - -` | 30 | 7 | [3, 4] | 41.101 | [2, 5] | 40.999 |
| `1 2 3 4 5 - - - -` | 30 | 8 | [3, 5] | 41.101 | [3, 5] | 40.999 |
| `1 2 3 4 5 - - - -` | 30 | 9 | [4, 5] | 41.101 | [4, 5] | 40.999 |
| `1 2 3 4 5 - - - -` | 30 | 10 | [2, 3, 5] | 41.101 | [1, 4, 5] | 40.999 |
| `1 2 3 4 5 - - - -` | 30 | 11 | [2, 4, 5] | 41.101 | [2, 4, 5] | 40.999 |
| `1 2 3 4 5 - - - -` | 30 | 12 | [3, 4, 5] | 41.101 | [3, 4, 5] | 40.999 |
| `- - - - - 6 - 8 -` | 31 | 6 | [6] | 33.215 | [6] | 33.215 |
| `- - - - - 6 - 8 -` | 31 | 8 | [8] | 33.215 | [8] | 33.215 |
| `- - - - 5 - - - 9` | 31 | 5 | [5] | 32.728 | [5] | 32.728 |
| `- - - - 5 - - - 9` | 31 | 9 | [9] | 32.728 | [9] | 32.728 |
| `- - 3 - 5 6 - - -` | 31 | 3 | [3] | 36.076 | [3] | 36.076 |
| `- - 3 - 5 6 - - -` | 31 | 5 | [5] | 36.076 | [5] | 36.076 |
| `- - 3 - 5 6 - - -` | 31 | 6 | [6] | 36.076 | [6] | 36.076 |
| `- - 3 - 5 6 - - -` | 31 | 8 | [3, 5] | 36.076 | [3, 5] | 36.076 |
| `- - 3 - 5 6 - - -` | 31 | 9 | [3, 6] | 36.076 | [3, 6] | 36.076 |
| `- - 3 - 5 6 - - -` | 31 | 11 | [5, 6] | 36.076 | [5, 6] | 36.076 |
| `- - 3 4 - - 7 - -` | 31 | 3 | [3] | 34.745 | [3] | 34.745 |
| `- - 3 4 - - 7 - -` | 31 | 4 | [4] | 34.745 | [4] | 34.745 |
| `- - 3 4 - - 7 - -` | 31 | 7 | [7] | 34.745 | [7] | 34.745 |
| `- - 3 4 - - 7 - -` | 31 | 10 | [3, 7] | 34.745 | [3, 7] | 34.745 |
| `- - 3 4 - - 7 - -` | 31 | 11 | [4, 7] | 34.745 | [4, 7] | 34.745 |
| `- 2 - - 5 - 7 - -` | 31 | 2 | [2] | 34.789 | [2] | 34.789 |
| `- 2 - - 5 - 7 - -` | 31 | 5 | [5] | 34.789 | [5] | 34.789 |
| `- 2 - - 5 - 7 - -` | 31 | 7 | [7] | 34.789 | [7] | 34.789 |
| `- 2 - - 5 - 7 - -` | 31 | 9 | [2, 7] | 34.789 | [2, 7] | 34.789 |
| `- 2 - - 5 - 7 - -` | 31 | 12 | [5, 7] | 34.789 | [5, 7] | 34.789 |
| `- 2 - 4 - - - 8 -` | 31 | 2 | [2] | 35.078 | [2] | 35.078 |
| `- 2 - 4 - - - 8 -` | 31 | 4 | [4] | 35.078 | [4] | 35.078 |
| `- 2 - 4 - - - 8 -` | 31 | 6 | [2, 4] | 35.078 | [2, 4] | 35.078 |
| `- 2 - 4 - - - 8 -` | 31 | 8 | [8] | 35.078 | [8] | 35.078 |
| `- 2 - 4 - - - 8 -` | 31 | 10 | [2, 8] | 35.078 | [2, 8] | 35.078 |
| `- 2 - 4 - - - 8 -` | 31 | 12 | [4, 8] | 35.078 | [4, 8] | 35.078 |
| `- 2 3 - - - - - 9` | 31 | 2 | [2] | 34.070 | [2] | 34.070 |
| `- 2 3 - - - - - 9` | 31 | 3 | [3] | 34.070 | [3] | 34.070 |
| `- 2 3 - - - - - 9` | 31 | 5 | [2, 3] | 34.070 | [2, 3] | 34.070 |
| `- 2 3 - - - - - 9` | 31 | 9 | [9] | 34.070 | [9] | 34.070 |
| `- 2 3 - - - - - 9` | 31 | 11 | [2, 9] | 34.070 | [2, 9] | 34.070 |
| `- 2 3 - - - - - 9` | 31 | 12 | [3, 9] | 34.070 | [3, 9] | 34.070 |
| `- 2 3 4 5 - - - -` | 31 | 2 | [2] | 40.117 | [2] | 40.100 |
| `- 2 3 4 5 - - - -` | 31 | 3 | [3] | 40.117 | [3] | 40.100 |
| `- 2 3 4 5 - - - -` | 31 | 4 | [4] | 40.117 | [4] | 40.100 |
| `- 2 3 4 5 - - - -` | 31 | 5 | [5] | 40.117 | [5] | 40.100 |
| `- 2 3 4 5 - - - -` | 31 | 6 | [2, 4] | 40.117 | [2, 4] | 40.100 |
| `- 2 3 4 5 - - - -` | 31 | 7 | [3, 4] | 40.117 | [2, 5] | 40.100 |
| `- 2 3 4 5 - - - -` | 31 | 8 | [3, 5] | 40.117 | [3, 5] | 40.100 |
| `- 2 3 4 5 - - - -` | 31 | 9 | [4, 5] | 40.117 | [4, 5] | 40.100 |
| `- 2 3 4 5 - - - -` | 31 | 10 | [2, 3, 5] | 40.117 | [2, 3, 5] | 40.100 |
| `- 2 3 4 5 - - - -` | 31 | 11 | [2, 4, 5] | 40.117 | [2, 4, 5] | 40.100 |
| `- 2 3 4 5 - - - -` | 31 | 12 | [3, 4, 5] | 40.117 | [3, 4, 5] | 40.100 |
| `1 - - - - 6 7 - -` | 31 | 6 | [6] | 34.877 | [6] | 34.877 |
| `1 - - - - 6 7 - -` | 31 | 7 | [7] | 34.877 | [7] | 34.877 |
| `1 - - - - 6 7 - -` | 31 | 8 | [1, 7] | 34.877 | [1, 7] | 34.877 |
| `1 - - - 5 - - 8 -` | 31 | 5 | [5] | 35.144 | [5] | 35.144 |
| `1 - - - 5 - - 8 -` | 31 | 6 | [1, 5] | 35.144 | [1, 5] | 35.144 |
| `1 - - - 5 - - 8 -` | 31 | 8 | [8] | 35.144 | [8] | 35.144 |
| `1 - - - 5 - - 8 -` | 31 | 9 | [1, 8] | 35.144 | [1, 8] | 35.144 |
| `1 - - 4 - - - - 9` | 31 | 4 | [4] | 34.113 | [4] | 34.113 |
| `1 - - 4 - - - - 9` | 31 | 5 | [1, 4] | 34.113 | [1, 4] | 34.113 |
| `1 - - 4 - - - - 9` | 31 | 9 | [9] | 34.113 | [9] | 34.113 |
| `1 - - 4 - - - - 9` | 31 | 10 | [1, 9] | 34.113 | [1, 9] | 34.113 |
| `1 - 3 4 - 6 - - -` | 31 | 3 | [3] | 39.567 | [3] | 39.517 |
| `1 - 3 4 - 6 - - -` | 31 | 4 | [4] | 39.567 | [4] | 39.517 |
| `1 - 3 4 - 6 - - -` | 31 | 5 | [1, 4] | 39.567 | [1, 4] | 39.517 |
| `1 - 3 4 - 6 - - -` | 31 | 6 | [6] | 39.567 | [6] | 39.517 |
| `1 - 3 4 - 6 - - -` | 31 | 7 | [3, 4] | 39.567 | [1, 6] | 39.517 |
| `1 - 3 4 - 6 - - -` | 31 | 8 | [1, 3, 4] | 39.567 | [1, 3, 4] | 39.517 |
| `1 - 3 4 - 6 - - -` | 31 | 9 | [3, 6] | 39.567 | [3, 6] | 39.517 |
| `1 - 3 4 - 6 - - -` | 31 | 10 | [4, 6] | 39.567 | [4, 6] | 39.517 |
| `1 - 3 4 - 6 - - -` | 31 | 11 | [1, 4, 6] | 39.567 | [1, 4, 6] | 39.517 |
| `1 2 - - 5 6 - - -` | 31 | 2 | [2] | 38.719 | [2] | 38.685 |
| `1 2 - - 5 6 - - -` | 31 | 3 | [1, 2] | 38.719 | [1, 2] | 38.685 |
| `1 2 - - 5 6 - - -` | 31 | 5 | [5] | 38.719 | [5] | 38.685 |
| `1 2 - - 5 6 - - -` | 31 | 6 | [6] | 38.719 | [6] | 38.685 |
| `1 2 - - 5 6 - - -` | 31 | 7 | [2, 5] | 38.719 | [1, 6] | 38.685 |
| `1 2 - - 5 6 - - -` | 31 | 8 | [2, 6] | 38.719 | [2, 6] | 38.685 |
| `1 2 - - 5 6 - - -` | 31 | 9 | [1, 2, 6] | 38.719 | [1, 2, 6] | 38.685 |
| `1 2 - - 5 6 - - -` | 31 | 11 | [5, 6] | 38.719 | [5, 6] | 38.685 |
| `1 2 - - 5 6 - - -` | 31 | 12 | [1, 5, 6] | 38.719 | [1, 5, 6] | 38.685 |
| `1 2 - 4 - - 7 - -` | 31 | 2 | [2] | 40.040 | [2] | 40.040 |
| `1 2 - 4 - - 7 - -` | 31 | 3 | [1, 2] | 40.040 | [1, 2] | 40.040 |
| `1 2 - 4 - - 7 - -` | 31 | 4 | [4] | 40.040 | [4] | 40.040 |
| `1 2 - 4 - - 7 - -` | 31 | 5 | [1, 4] | 40.040 | [1, 4] | 40.040 |
| `1 2 - 4 - - 7 - -` | 31 | 6 | [2, 4] | 40.040 | [2, 4] | 40.040 |
| `1 2 - 4 - - 7 - -` | 31 | 7 | [7] | 40.040 | [7] | 40.040 |
| `1 2 - 4 - - 7 - -` | 31 | 8 | [1, 7] | 40.040 | [1, 7] | 40.040 |
| `1 2 - 4 - - 7 - -` | 31 | 9 | [2, 7] | 40.040 | [2, 7] | 40.040 |
| `1 2 - 4 - - 7 - -` | 31 | 10 | [1, 2, 7] | 40.040 | [1, 2, 7] | 40.040 |
| `1 2 - 4 - - 7 - -` | 31 | 11 | [4, 7] | 40.040 | [4, 7] | 40.040 |
| `1 2 - 4 - - 7 - -` | 31 | 12 | [1, 4, 7] | 40.040 | [1, 4, 7] | 40.040 |
| `1 2 3 - - - - 8 -` | 31 | 2 | [2] | 38.135 | [2] | 38.135 |
| `1 2 3 - - - - 8 -` | 31 | 3 | [3] | 38.135 | [3] | 38.135 |
| `1 2 3 - - - - 8 -` | 31 | 4 | [1, 3] | 38.135 | [1, 3] | 38.135 |
| `1 2 3 - - - - 8 -` | 31 | 5 | [2, 3] | 38.135 | [2, 3] | 38.135 |
| `1 2 3 - - - - 8 -` | 31 | 6 | [1, 2, 3] | 38.135 | [1, 2, 3] | 38.135 |
| `1 2 3 - - - - 8 -` | 31 | 8 | [8] | 38.135 | [8] | 38.135 |
| `1 2 3 - - - - 8 -` | 31 | 9 | [1, 8] | 38.135 | [1, 8] | 38.135 |
| `1 2 3 - - - - 8 -` | 31 | 10 | [2, 8] | 38.135 | [2, 8] | 38.135 |
| `1 2 3 - - - - 8 -` | 31 | 11 | [3, 8] | 38.135 | [3, 8] | 38.135 |
| `1 2 3 - - - - 8 -` | 31 | 12 | [1, 3, 8] | 38.135 | [1, 3, 8] | 38.135 |
| `- - - - - 6 7 - -` | 32 | 6 | [6] | 34.301 | [6] | 34.301 |
| `- - - - - 6 7 - -` | 32 | 7 | [7] | 34.301 | [7] | 34.301 |
| `- - - - 5 - - 8 -` | 32 | 5 | [5] | 33.867 | [5] | 33.867 |
| `- - - - 5 - - 8 -` | 32 | 8 | [8] | 33.867 | [8] | 33.867 |
| `- - - 4 - - - - 9` | 32 | 4 | [4] | 33.454 | [4] | 33.454 |
| `- - - 4 - - - - 9` | 32 | 9 | [9] | 33.454 | [9] | 33.454 |
| `- - 3 4 - 6 - - -` | 32 | 3 | [3] | 37.049 | [3] | 37.049 |
| `- - 3 4 - 6 - - -` | 32 | 4 | [4] | 37.049 | [4] | 37.049 |
| `- - 3 4 - 6 - - -` | 32 | 6 | [6] | 37.049 | [6] | 37.049 |
| `- - 3 4 - 6 - - -` | 32 | 7 | [3, 4] | 37.049 | [3, 4] | 37.049 |
| `- - 3 4 - 6 - - -` | 32 | 9 | [3, 6] | 37.049 | [3, 6] | 37.049 |
| `- - 3 4 - 6 - - -` | 32 | 10 | [4, 6] | 37.049 | [4, 6] | 37.049 |
| `- 2 - - 5 6 - - -` | 32 | 2 | [2] | 37.088 | [2] | 37.088 |
| `- 2 - - 5 6 - - -` | 32 | 5 | [5] | 37.088 | [5] | 37.088 |
| `- 2 - - 5 6 - - -` | 32 | 6 | [6] | 37.088 | [6] | 37.088 |
| `- 2 - - 5 6 - - -` | 32 | 7 | [2, 5] | 37.088 | [2, 5] | 37.088 |
| `- 2 - - 5 6 - - -` | 32 | 8 | [2, 6] | 37.088 | [2, 6] | 37.088 |
| `- 2 - - 5 6 - - -` | 32 | 11 | [5, 6] | 37.088 | [5, 6] | 37.088 |
| `- 2 - 4 - - 7 - -` | 32 | 2 | [2] | 36.660 | [2] | 36.660 |
| `- 2 - 4 - - 7 - -` | 32 | 4 | [4] | 36.660 | [4] | 36.660 |
| `- 2 - 4 - - 7 - -` | 32 | 6 | [2, 4] | 36.660 | [2, 4] | 36.660 |
| `- 2 - 4 - - 7 - -` | 32 | 7 | [7] | 36.660 | [7] | 36.660 |
| `- 2 - 4 - - 7 - -` | 32 | 9 | [2, 7] | 36.660 | [2, 7] | 36.660 |
| `- 2 - 4 - - 7 - -` | 32 | 11 | [4, 7] | 36.660 | [4, 7] | 36.660 |
| `- 2 3 - - - - 8 -` | 32 | 2 | [2] | 35.751 | [2] | 35.751 |
| `- 2 3 - - - - 8 -` | 32 | 3 | [3] | 35.751 | [3] | 35.751 |
| `- 2 3 - - - - 8 -` | 32 | 5 | [2, 3] | 35.751 | [2, 3] | 35.751 |
| `- 2 3 - - - - 8 -` | 32 | 8 | [8] | 35.751 | [8] | 35.751 |
| `- 2 3 - - - - 8 -` | 32 | 10 | [2, 8] | 35.751 | [2, 8] | 35.751 |
| `- 2 3 - - - - 8 -` | 32 | 11 | [3, 8] | 35.751 | [3, 8] | 35.751 |
| `1 - - - 5 - 7 - -` | 32 | 5 | [5] | 36.724 | [5] | 36.724 |
| `1 - - - 5 - 7 - -` | 32 | 6 | [1, 5] | 36.724 | [1, 5] | 36.724 |
| `1 - - - 5 - 7 - -` | 32 | 7 | [7] | 36.724 | [7] | 36.724 |
| `1 - - - 5 - 7 - -` | 32 | 8 | [1, 7] | 36.724 | [1, 7] | 36.724 |
| `1 - - - 5 - 7 - -` | 32 | 12 | [5, 7] | 36.724 | [5, 7] | 36.724 |
| `1 - - 4 - - - 8 -` | 32 | 4 | [4] | 35.793 | [4] | 35.793 |
| `1 - - 4 - - - 8 -` | 32 | 5 | [1, 4] | 35.793 | [1, 4] | 35.793 |
| `1 - - 4 - - - 8 -` | 32 | 8 | [8] | 35.793 | [8] | 35.793 |
| `1 - - 4 - - - 8 -` | 32 | 9 | [1, 8] | 35.793 | [1, 8] | 35.793 |
| `1 - - 4 - - - 8 -` | 32 | 12 | [4, 8] | 35.793 | [4, 8] | 35.793 |
| `1 - 3 - - - - - 9` | 32 | 3 | [3] | 34.921 | [3] | 34.921 |
| `1 - 3 - - - - - 9` | 32 | 4 | [1, 3] | 34.921 | [1, 3] | 34.921 |
| `1 - 3 - - - - - 9` | 32 | 9 | [9] | 34.921 | [9] | 34.921 |
| `1 - 3 - - - - - 9` | 32 | 10 | [1, 9] | 34.921 | [1, 9] | 34.921 |
| `1 - 3 - - - - - 9` | 32 | 12 | [3, 9] | 34.921 | [3, 9] | 34.921 |
| `1 - 3 4 5 - - - -` | 32 | 3 | [3] | 39.938 | [3] | 39.938 |
| `1 - 3 4 5 - - - -` | 32 | 4 | [4] | 39.938 | [4] | 39.938 |
| `1 - 3 4 5 - - - -` | 32 | 5 | [5] | 39.938 | [5] | 39.938 |
| `1 - 3 4 5 - - - -` | 32 | 6 | [1, 5] | 39.938 | [1, 5] | 39.938 |
| `1 - 3 4 5 - - - -` | 32 | 7 | [3, 4] | 39.938 | [3, 4] | 39.938 |
| `1 - 3 4 5 - - - -` | 32 | 8 | [3, 5] | 39.938 | [3, 5] | 39.938 |
| `1 - 3 4 5 - - - -` | 32 | 9 | [4, 5] | 39.938 | [4, 5] | 39.938 |
| `1 - 3 4 5 - - - -` | 32 | 10 | [1, 4, 5] | 39.938 | [1, 4, 5] | 39.938 |
| `1 - 3 4 5 - - - -` | 32 | 12 | [3, 4, 5] | 39.938 | [3, 4, 5] | 39.938 |
| `1 2 - 4 - 6 - - -` | 32 | 2 | [2] | 40.678 | [2] | 40.678 |
| `1 2 - 4 - 6 - - -` | 32 | 3 | [1, 2] | 40.678 | [1, 2] | 40.678 |
| `1 2 - 4 - 6 - - -` | 32 | 4 | [4] | 40.678 | [4] | 40.678 |
| `1 2 - 4 - 6 - - -` | 32 | 5 | [1, 4] | 40.678 | [1, 4] | 40.678 |
| `1 2 - 4 - 6 - - -` | 32 | 6 | [6] | 40.678 | [6] | 40.678 |
| `1 2 - 4 - 6 - - -` | 32 | 7 | [1, 6] | 40.678 | [1, 6] | 40.678 |
| `1 2 - 4 - 6 - - -` | 32 | 8 | [2, 6] | 40.678 | [2, 6] | 40.678 |
| `1 2 - 4 - 6 - - -` | 32 | 9 | [1, 2, 6] | 40.678 | [1, 2, 6] | 40.678 |
| `1 2 - 4 - 6 - - -` | 32 | 10 | [4, 6] | 40.678 | [4, 6] | 40.678 |
| `1 2 - 4 - 6 - - -` | 32 | 11 | [1, 4, 6] | 40.678 | [1, 4, 6] | 40.678 |
| `1 2 - 4 - 6 - - -` | 32 | 12 | [2, 4, 6] | 40.678 | [2, 4, 6] | 40.678 |
| `1 2 3 - - - 7 - -` | 32 | 2 | [2] | 40.496 | [2] | 40.496 |
| `1 2 3 - - - 7 - -` | 32 | 3 | [3] | 40.496 | [3] | 40.496 |
| `1 2 3 - - - 7 - -` | 32 | 4 | [1, 3] | 40.496 | [1, 3] | 40.496 |
| `1 2 3 - - - 7 - -` | 32 | 5 | [2, 3] | 40.496 | [2, 3] | 40.496 |
| `1 2 3 - - - 7 - -` | 32 | 6 | [1, 2, 3] | 40.496 | [1, 2, 3] | 40.496 |
| `1 2 3 - - - 7 - -` | 32 | 7 | [7] | 40.496 | [7] | 40.496 |
| `1 2 3 - - - 7 - -` | 32 | 8 | [1, 7] | 40.496 | [1, 7] | 40.496 |
| `1 2 3 - - - 7 - -` | 32 | 9 | [2, 7] | 40.496 | [2, 7] | 40.496 |
| `1 2 3 - - - 7 - -` | 32 | 10 | [3, 7] | 40.496 | [3, 7] | 40.496 |
| `1 2 3 - - - 7 - -` | 32 | 11 | [1, 3, 7] | 40.496 | [1, 3, 7] | 40.496 |
| `1 2 3 - - - 7 - -` | 32 | 12 | [2, 3, 7] | 40.496 | [2, 3, 7] | 40.496 |
| `- - - - 5 - 7 - -` | 33 | 5 | [5] | 35.278 | [5] | 35.278 |
| `- - - - 5 - 7 - -` | 33 | 7 | [7] | 35.278 | [7] | 35.278 |
| `- - - - 5 - 7 - -` | 33 | 12 | [5, 7] | 35.278 | [5, 7] | 35.278 |
| `- - - 4 - - - 8 -` | 33 | 4 | [4] | 34.917 | [4] | 34.917 |
| `- - - 4 - - - 8 -` | 33 | 8 | [8] | 34.917 | [8] | 34.917 |
| `- - - 4 - - - 8 -` | 33 | 12 | [4, 8] | 34.917 | [4, 8] | 34.917 |
| `- - 3 - - - - - 9` | 33 | 3 | [3] | 34.574 | [3] | 34.574 |
| `- - 3 - - - - - 9` | 33 | 9 | [9] | 34.574 | [9] | 34.574 |
| `- - 3 - - - - - 9` | 33 | 12 | [3, 9] | 34.574 | [3, 9] | 34.574 |
| `- - 3 4 5 - - - -` | 33 | 3 | [3] | 38.279 | [3] | 38.279 |
| `- - 3 4 5 - - - -` | 33 | 4 | [4] | 38.279 | [4] | 38.279 |
| `- - 3 4 5 - - - -` | 33 | 5 | [5] | 38.279 | [5] | 38.279 |
| `- - 3 4 5 - - - -` | 33 | 7 | [3, 4] | 38.279 | [3, 4] | 38.279 |
| `- - 3 4 5 - - - -` | 33 | 8 | [3, 5] | 38.279 | [3, 5] | 38.279 |
| `- - 3 4 5 - - - -` | 33 | 9 | [4, 5] | 38.279 | [4, 5] | 38.279 |
| `- - 3 4 5 - - - -` | 33 | 12 | [3, 4, 5] | 38.279 | [3, 4, 5] | 38.279 |
| `- 2 - 4 - 6 - - -` | 33 | 2 | [2] | 36.951 | [2] | 36.951 |
| `- 2 - 4 - 6 - - -` | 33 | 4 | [4] | 36.951 | [4] | 36.951 |
| `- 2 - 4 - 6 - - -` | 33 | 6 | [6] | 36.951 | [6] | 36.951 |
| `- 2 - 4 - 6 - - -` | 33 | 8 | [2, 6] | 36.951 | [2, 6] | 36.951 |
| `- 2 - 4 - 6 - - -` | 33 | 10 | [4, 6] | 36.951 | [4, 6] | 36.951 |
| `- 2 - 4 - 6 - - -` | 33 | 12 | [2, 4, 6] | 36.951 | [2, 4, 6] | 36.951 |
| `- 2 3 - - - 7 - -` | 33 | 2 | [2] | 37.583 | [2] | 37.583 |
| `- 2 3 - - - 7 - -` | 33 | 3 | [3] | 37.583 | [3] | 37.583 |
| `- 2 3 - - - 7 - -` | 33 | 5 | [2, 3] | 37.583 | [2, 3] | 37.583 |
| `- 2 3 - - - 7 - -` | 33 | 7 | [7] | 37.583 | [7] | 37.583 |
| `- 2 3 - - - 7 - -` | 33 | 9 | [2, 7] | 37.583 | [2, 7] | 37.583 |
| `- 2 3 - - - 7 - -` | 33 | 10 | [3, 7] | 37.583 | [3, 7] | 37.583 |
| `- 2 3 - - - 7 - -` | 33 | 12 | [2, 3, 7] | 37.583 | [2, 3, 7] | 37.583 |
| `1 - - - 5 6 - - -` | 33 | 5 | [5] | 37.008 | [5] | 37.008 |
| `1 - - - 5 6 - - -` | 33 | 6 | [6] | 37.008 | [6] | 37.008 |
| `1 - - - 5 6 - - -` | 33 | 7 | [1, 6] | 37.008 | [1, 6] | 37.008 |
| `1 - - - 5 6 - - -` | 33 | 11 | [5, 6] | 37.008 | [5, 6] | 37.008 |
| `1 - - - 5 6 - - -` | 33 | 12 | [1, 5, 6] | 37.008 | [1, 5, 6] | 37.008 |
| `1 - - 4 - - 7 - -` | 33 | 4 | [4] | 37.625 | [4] | 37.625 |
| `1 - - 4 - - 7 - -` | 33 | 5 | [1, 4] | 37.625 | [1, 4] | 37.625 |
| `1 - - 4 - - 7 - -` | 33 | 7 | [7] | 37.625 | [7] | 37.625 |
| `1 - - 4 - - 7 - -` | 33 | 8 | [1, 7] | 37.625 | [1, 7] | 37.625 |
| `1 - - 4 - - 7 - -` | 33 | 11 | [4, 7] | 37.625 | [4, 7] | 37.625 |
| `1 - - 4 - - 7 - -` | 33 | 12 | [1, 4, 7] | 37.625 | [1, 4, 7] | 37.625 |
| `1 - 3 - - - - 8 -` | 33 | 3 | [3] | 36.853 | [3] | 36.853 |
| `1 - 3 - - - - 8 -` | 33 | 4 | [1, 3] | 36.853 | [1, 3] | 36.853 |
| `1 - 3 - - - - 8 -` | 33 | 8 | [8] | 36.853 | [8] | 36.853 |
| `1 - 3 - - - - 8 -` | 33 | 9 | [1, 8] | 36.853 | [1, 8] | 36.853 |
| `1 - 3 - - - - 8 -` | 33 | 11 | [3, 8] | 36.853 | [3, 8] | 36.853 |
| `1 - 3 - - - - 8 -` | 33 | 12 | [1, 3, 8] | 36.853 | [1, 3, 8] | 36.853 |
| `1 2 - - - - - - 9` | 33 | 2 | [2] | 36.136 | [2] | 36.136 |
| `1 2 - - - - - - 9` | 33 | 3 | [1, 2] | 36.136 | [1, 2] | 36.136 |
| `1 2 - - - - - - 9` | 33 | 9 | [9] | 36.136 | [9] | 36.136 |
| `1 2 - - - - - - 9` | 33 | 10 | [1, 9] | 36.136 | [1, 9] | 36.136 |
| `1 2 - - - - - - 9` | 33 | 11 | [2, 9] | 36.136 | [2, 9] | 36.136 |
| `1 2 - - - - - - 9` | 33 | 12 | [1, 2, 9] | 36.136 | [1, 2, 9] | 36.136 |
| `1 2 - 4 5 - - - -` | 33 | 2 | [2] | 41.318 | [2] | 41.297 |
| `1 2 - 4 5 - - - -` | 33 | 3 | [1, 2] | 41.318 | [1, 2] | 41.297 |
| `1 2 - 4 5 - - - -` | 33 | 4 | [4] | 41.318 | [4] | 41.297 |
| `1 2 - 4 5 - - - -` | 33 | 5 | [5] | 41.318 | [5] | 41.297 |
| `1 2 - 4 5 - - - -` | 33 | 6 | [2, 4] | 41.318 | [1, 5] | 41.297 |
| `1 2 - 4 5 - - - -` | 33 | 7 | [2, 5] | 41.318 | [2, 5] | 41.297 |
| `1 2 - 4 5 - - - -` | 33 | 8 | [1, 2, 5] | 41.318 | [1, 2, 5] | 41.297 |
| `1 2 - 4 5 - - - -` | 33 | 9 | [4, 5] | 41.318 | [4, 5] | 41.297 |
| `1 2 - 4 5 - - - -` | 33 | 10 | [1, 4, 5] | 41.318 | [1, 4, 5] | 41.297 |
| `1 2 - 4 5 - - - -` | 33 | 11 | [2, 4, 5] | 41.318 | [2, 4, 5] | 41.297 |
| `1 2 - 4 5 - - - -` | 33 | 12 | [1, 2, 4, 5] | 41.318 | [1, 2, 4, 5] | 41.297 |
| `1 2 3 - - 6 - - -` | 33 | 2 | [2] | 41.272 | [2] | 41.272 |
| `1 2 3 - - 6 - - -` | 33 | 3 | [3] | 41.272 | [3] | 41.272 |
| `1 2 3 - - 6 - - -` | 33 | 4 | [1, 3] | 41.272 | [1, 3] | 41.272 |
| `1 2 3 - - 6 - - -` | 33 | 5 | [2, 3] | 41.272 | [2, 3] | 41.272 |
| `1 2 3 - - 6 - - -` | 33 | 6 | [6] | 41.272 | [6] | 41.272 |
| `1 2 3 - - 6 - - -` | 33 | 7 | [1, 6] | 41.272 | [1, 6] | 41.272 |
| `1 2 3 - - 6 - - -` | 33 | 8 | [2, 6] | 41.272 | [2, 6] | 41.272 |
| `1 2 3 - - 6 - - -` | 33 | 9 | [3, 6] | 41.272 | [3, 6] | 41.272 |
| `1 2 3 - - 6 - - -` | 33 | 10 | [1, 3, 6] | 41.272 | [1, 3, 6] | 41.272 |
| `1 2 3 - - 6 - - -` | 33 | 11 | [2, 3, 6] | 41.272 | [2, 3, 6] | 41.272 |
| `1 2 3 - - 6 - - -` | 33 | 12 | [1, 2, 3, 6] | 41.272 | [1, 2, 3, 6] | 41.272 |
| `- - - - 5 6 - - -` | 34 | 5 | [5] | 36.170 | [5] | 36.170 |
| `- - - - 5 6 - - -` | 34 | 6 | [6] | 36.170 | [6] | 36.170 |
| `- - - - 5 6 - - -` | 34 | 11 | [5, 6] | 36.170 | [5, 6] | 36.170 |
| `- - - 4 - - 7 - -` | 34 | 4 | [4] | 36.264 | [4] | 36.264 |
| `- - - 4 - - 7 - -` | 34 | 7 | [7] | 36.264 | [7] | 36.264 |
| `- - - 4 - - 7 - -` | 34 | 11 | [4, 7] | 36.264 | [4, 7] | 36.264 |
| `- - 3 - - - - 8 -` | 34 | 3 | [3] | 35.974 | [3] | 35.974 |
| `- - 3 - - - - 8 -` | 34 | 8 | [8] | 35.974 | [8] | 35.974 |
| `- - 3 - - - - 8 -` | 34 | 11 | [3, 8] | 35.974 | [3, 8] | 35.974 |
| `- 2 - - - - - - 9` | 34 | 2 | [2] | 35.701 | [2] | 35.701 |
| `- 2 - - - - - - 9` | 34 | 9 | [9] | 35.701 | [9] | 35.701 |
| `- 2 - - - - - - 9` | 34 | 11 | [2, 9] | 35.701 | [2, 9] | 35.701 |
| `- 2 - 4 5 - - - -` | 34 | 2 | [2] | 39.037 | [2] | 39.037 |
| `- 2 - 4 5 - - - -` | 34 | 4 | [4] | 39.037 | [4] | 39.037 |
| `- 2 - 4 5 - - - -` | 34 | 5 | [5] | 39.037 | [5] | 39.037 |
| `- 2 - 4 5 - - - -` | 34 | 6 | [2, 4] | 39.037 | [2, 4] | 39.037 |
| `- 2 - 4 5 - - - -` | 34 | 7 | [2, 5] | 39.037 | [2, 5] | 39.037 |
| `- 2 - 4 5 - - - -` | 34 | 9 | [4, 5] | 39.037 | [4, 5] | 39.037 |
| `- 2 - 4 5 - - - -` | 34 | 11 | [2, 4, 5] | 39.037 | [2, 4, 5] | 39.037 |
| `- 2 3 - - 6 - - -` | 34 | 2 | [2] | 38.735 | [2] | 38.735 |
| `- 2 3 - - 6 - - -` | 34 | 3 | [3] | 38.735 | [3] | 38.735 |
| `- 2 3 - - 6 - - -` | 34 | 5 | [2, 3] | 38.735 | [2, 3] | 38.735 |
| `- 2 3 - - 6 - - -` | 34 | 6 | [6] | 38.735 | [6] | 38.735 |
| `- 2 3 - - 6 - - -` | 34 | 8 | [2, 6] | 38.735 | [2, 6] | 38.735 |
| `- 2 3 - - 6 - - -` | 34 | 9 | [3, 6] | 38.735 | [3, 6] | 38.735 |
| `- 2 3 - - 6 - - -` | 34 | 11 | [2, 3, 6] | 38.735 | [2, 3, 6] | 38.735 |
| `1 - - 4 - 6 - - -` | 34 | 4 | [4] | 38.772 | [4] | 38.772 |
| `1 - - 4 - 6 - - -` | 34 | 5 | [1, 4] | 38.772 | [1, 4] | 38.772 |
| `1 - - 4 - 6 - - -` | 34 | 6 | [6] | 38.772 | [6] | 38.772 |
| `1 - - 4 - 6 - - -` | 34 | 7 | [1, 6] | 38.772 | [1, 6] | 38.772 |
| `1 - - 4 - 6 - - -` | 34 | 10 | [4, 6] | 38.772 | [4, 6] | 38.772 |
| `1 - - 4 - 6 - - -` | 34 | 11 | [1, 4, 6] | 38.772 | [1, 4, 6] | 38.772 |
| `1 - 3 - - - 7 - -` | 34 | 3 | [3] | 38.552 | [3] | 38.552 |
| `1 - 3 - - - 7 - -` | 34 | 4 | [1, 3] | 38.552 | [1, 3] | 38.552 |
| `1 - 3 - - - 7 - -` | 34 | 7 | [7] | 38.552 | [7] | 38.552 |
| `1 - 3 - - - 7 - -` | 34 | 8 | [1, 7] | 38.552 | [1, 7] | 38.552 |
| `1 - 3 - - - 7 - -` | 34 | 10 | [3, 7] | 38.552 | [3, 7] | 38.552 |
| `1 - 3 - - - 7 - -` | 34 | 11 | [1, 3, 7] | 38.552 | [1, 3, 7] | 38.552 |
| `1 2 - - - - - 8 -` | 34 | 2 | [2] | 37.935 | [2] | 37.935 |
| `1 2 - - - - - 8 -` | 34 | 3 | [1, 2] | 37.935 | [1, 2] | 37.935 |
| `1 2 - - - - - 8 -` | 34 | 8 | [8] | 37.935 | [8] | 37.935 |
| `1 2 - - - - - 8 -` | 34 | 9 | [1, 8] | 37.935 | [1, 8] | 37.935 |
| `1 2 - - - - - 8 -` | 34 | 10 | [2, 8] | 37.935 | [2, 8] | 37.935 |
| `1 2 - - - - - 8 -` | 34 | 11 | [1, 2, 8] | 37.935 | [1, 2, 8] | 37.935 |
| `1 2 3 - 5 - - - -` | 34 | 2 | [2] | 41.604 | [2] | 41.604 |
| `1 2 3 - 5 - - - -` | 34 | 3 | [3] | 41.604 | [3] | 41.604 |
| `1 2 3 - 5 - - - -` | 34 | 4 | [1, 3] | 41.604 | [1, 3] | 41.604 |
| `1 2 3 - 5 - - - -` | 34 | 5 | [5] | 41.604 | [5] | 41.604 |
| `1 2 3 - 5 - - - -` | 34 | 6 | [1, 5] | 41.604 | [1, 5] | 41.604 |
| `1 2 3 - 5 - - - -` | 34 | 7 | [2, 5] | 41.604 | [2, 5] | 41.604 |
| `1 2 3 - 5 - - - -` | 34 | 8 | [3, 5] | 41.604 | [3, 5] | 41.604 |
| `1 2 3 - 5 - - - -` | 34 | 9 | [1, 3, 5] | 41.604 | [1, 3, 5] | 41.604 |
| `1 2 3 - 5 - - - -` | 34 | 10 | [2, 3, 5] | 41.604 | [2, 3, 5] | 41.604 |
| `1 2 3 - 5 - - - -` | 34 | 11 | [1, 2, 3, 5] | 41.604 | [1, 2, 3, 5] | 41.604 |
| `- - - 4 - 6 - - -` | 35 | 4 | [4] | 37.116 | [4] | 37.116 |
| `- - - 4 - 6 - - -` | 35 | 6 | [6] | 37.116 | [6] | 37.116 |
| `- - - 4 - 6 - - -` | 35 | 10 | [4, 6] | 37.116 | [4, 6] | 37.116 |
| `- - 3 - - - 7 - -` | 35 | 3 | [3] | 37.259 | [3] | 37.259 |
| `- - 3 - - - 7 - -` | 35 | 7 | [7] | 37.259 | [7] | 37.259 |
| `- - 3 - - - 7 - -` | 35 | 10 | [3, 7] | 37.259 | [3, 7] | 37.259 |
| `- 2 - - - - - 8 -` | 35 | 2 | [2] | 37.039 | [2] | 37.039 |
| `- 2 - - - - - 8 -` | 35 | 8 | [8] | 37.039 | [8] | 37.039 |
| `- 2 - - - - - 8 -` | 35 | 10 | [2, 8] | 37.039 | [2, 8] | 37.039 |
| `- 2 3 - 5 - - - -` | 35 | 2 | [2] | 39.164 | [2] | 39.164 |
| `- 2 3 - 5 - - - -` | 35 | 3 | [3] | 39.164 | [3] | 39.164 |
| `- 2 3 - 5 - - - -` | 35 | 5 | [5] | 39.164 | [5] | 39.164 |
| `- 2 3 - 5 - - - -` | 35 | 7 | [2, 5] | 39.164 | [2, 5] | 39.164 |
| `- 2 3 - 5 - - - -` | 35 | 8 | [3, 5] | 39.164 | [3, 5] | 39.164 |
| `- 2 3 - 5 - - - -` | 35 | 10 | [2, 3, 5] | 39.164 | [2, 3, 5] | 39.164 |
| `1 - - - - - - - 9` | 35 | 9 | [9] | 36.833 | [9] | 36.833 |
| `1 - - - - - - - 9` | 35 | 10 | [1, 9] | 36.833 | [1, 9] | 36.833 |
| `1 - - 4 5 - - - -` | 35 | 4 | [4] | 38.816 | [4] | 38.816 |
| `1 - - 4 5 - - - -` | 35 | 5 | [5] | 38.816 | [5] | 38.816 |
| `1 - - 4 5 - - - -` | 35 | 6 | [1, 5] | 38.816 | [1, 5] | 38.816 |
| `1 - - 4 5 - - - -` | 35 | 9 | [4, 5] | 38.816 | [4, 5] | 38.816 |
| `1 - - 4 5 - - - -` | 35 | 10 | [1, 4, 5] | 38.816 | [1, 4, 5] | 38.816 |
| `1 - 3 - - 6 - - -` | 35 | 3 | [3] | 39.611 | [3] | 39.611 |
| `1 - 3 - - 6 - - -` | 35 | 4 | [1, 3] | 39.611 | [1, 3] | 39.611 |
| `1 - 3 - - 6 - - -` | 35 | 6 | [6] | 39.611 | [6] | 39.611 |
| `1 - 3 - - 6 - - -` | 35 | 7 | [1, 6] | 39.611 | [1, 6] | 39.611 |
| `1 - 3 - - 6 - - -` | 35 | 9 | [3, 6] | 39.611 | [3, 6] | 39.611 |
| `1 - 3 - - 6 - - -` | 35 | 10 | [1, 3, 6] | 39.611 | [1, 3, 6] | 39.611 |
| `1 2 - - - - 7 - -` | 35 | 2 | [2] | 39.506 | [2] | 39.506 |
| `1 2 - - - - 7 - -` | 35 | 3 | [1, 2] | 39.506 | [1, 2] | 39.506 |
| `1 2 - - - - 7 - -` | 35 | 7 | [7] | 39.506 | [7] | 39.506 |
| `1 2 - - - - 7 - -` | 35 | 8 | [1, 7] | 39.506 | [1, 7] | 39.506 |
| `1 2 - - - - 7 - -` | 35 | 9 | [2, 7] | 39.506 | [2, 7] | 39.506 |
| `1 2 - - - - 7 - -` | 35 | 10 | [1, 2, 7] | 39.506 | [1, 2, 7] | 39.506 |
| `1 2 3 4 - - - - -` | 35 | 2 | [2] | 41.705 | [2] | 41.694 |
| `1 2 3 4 - - - - -` | 35 | 3 | [3] | 41.705 | [3] | 41.694 |
| `1 2 3 4 - - - - -` | 35 | 4 | [4] | 41.705 | [4] | 41.694 |
| `1 2 3 4 - - - - -` | 35 | 5 | [2, 3] | 41.705 | [1, 4] | 41.694 |
| `1 2 3 4 - - - - -` | 35 | 6 | [2, 4] | 41.705 | [2, 4] | 41.694 |
| `1 2 3 4 - - - - -` | 35 | 7 | [3, 4] | 41.705 | [3, 4] | 41.694 |
| `1 2 3 4 - - - - -` | 35 | 8 | [1, 3, 4] | 41.705 | [1, 3, 4] | 41.694 |
| `1 2 3 4 - - - - -` | 35 | 9 | [2, 3, 4] | 41.705 | [2, 3, 4] | 41.694 |
| `1 2 3 4 - - - - -` | 35 | 10 | [1, 2, 3, 4] | 41.705 | [1, 2, 3, 4] | 41.694 |
| `- - - - - - - - 9` | 36 | 9 | [9] | 37.000 | [9] | 37.000 |
| `- - - 4 5 - - - -` | 36 | 4 | [4] | 37.972 | [4] | 37.972 |
| `- - - 4 5 - - - -` | 36 | 5 | [5] | 37.972 | [5] | 37.972 |
| `- - - 4 5 - - - -` | 36 | 9 | [4, 5] | 37.972 | [4, 5] | 37.972 |
| `- - 3 - - 6 - - -` | 36 | 3 | [3] | 38.069 | [3] | 38.069 |
| `- - 3 - - 6 - - -` | 36 | 6 | [6] | 38.069 | [6] | 38.069 |
| `- - 3 - - 6 - - -` | 36 | 9 | [3, 6] | 38.069 | [3, 6] | 38.069 |
| `- 2 - - - - 7 - -` | 36 | 2 | [2] | 38.264 | [2] | 38.264 |
| `- 2 - - - - 7 - -` | 36 | 7 | [7] | 38.264 | [7] | 38.264 |
| `- 2 - - - - 7 - -` | 36 | 9 | [2, 7] | 38.264 | [2, 7] | 38.264 |
| `- 2 3 4 - - - - -` | 36 | 2 | [2] | 40.362 | [2] | 40.362 |
| `- 2 3 4 - - - - -` | 36 | 3 | [3] | 40.362 | [3] | 40.362 |
| `- 2 3 4 - - - - -` | 36 | 4 | [4] | 40.362 | [4] | 40.362 |
| `- 2 3 4 - - - - -` | 36 | 5 | [2, 3] | 40.362 | [2, 3] | 40.362 |
| `- 2 3 4 - - - - -` | 36 | 6 | [2, 4] | 40.362 | [2, 4] | 40.362 |
| `- 2 3 4 - - - - -` | 36 | 7 | [3, 4] | 40.362 | [3, 4] | 40.362 |
| `- 2 3 4 - - - - -` | 36 | 9 | [2, 3, 4] | 40.362 | [2, 3, 4] | 40.362 |
| `1 - - - - - - 8 -` | 36 | 8 | [8] | 38.111 | [8] | 38.111 |
| `1 - - - - - - 8 -` | 36 | 9 | [1, 8] | 38.111 | [1, 8] | 38.111 |
| `1 - 3 - 5 - - - -` | 36 | 3 | [3] | 40.202 | [3] | 40.202 |
| `1 - 3 - 5 - - - -` | 36 | 4 | [1, 3] | 40.202 | [1, 3] | 40.202 |
| `1 - 3 - 5 - - - -` | 36 | 5 | [5] | 40.202 | [5] | 40.202 |
| `1 - 3 - 5 - - - -` | 36 | 6 | [1, 5] | 40.202 | [1, 5] | 40.202 |
| `1 - 3 - 5 - - - -` | 36 | 8 | [3, 5] | 40.202 | [3, 5] | 40.202 |
| `1 - 3 - 5 - - - -` | 36 | 9 | [1, 3, 5] | 40.202 | [1, 3, 5] | 40.202 |
| `1 2 - - - 6 - - -` | 36 | 2 | [2] | 40.475 | [2] | 40.475 |
| `1 2 - - - 6 - - -` | 36 | 3 | [1, 2] | 40.475 | [1, 2] | 40.475 |
| `1 2 - - - 6 - - -` | 36 | 6 | [6] | 40.475 | [6] | 40.475 |
| `1 2 - - - 6 - - -` | 36 | 7 | [1, 6] | 40.475 | [1, 6] | 40.475 |
| `1 2 - - - 6 - - -` | 36 | 8 | [2, 6] | 40.475 | [2, 6] | 40.475 |
| `1 2 - - - 6 - - -` | 36 | 9 | [1, 2, 6] | 40.475 | [1, 2, 6] | 40.475 |
| `- - - - - - - 8 -` | 37 | 8 | [8] | 38.111 | [8] | 38.111 |
| `- - 3 - 5 - - - -` | 37 | 3 | [3] | 38.883 | [3] | 38.883 |
| `- - 3 - 5 - - - -` | 37 | 5 | [5] | 38.883 | [5] | 38.883 |
| `- - 3 - 5 - - - -` | 37 | 8 | [3, 5] | 38.883 | [3, 5] | 38.883 |
| `- 2 - - - 6 - - -` | 37 | 2 | [2] | 39.031 | [2] | 39.031 |
| `- 2 - - - 6 - - -` | 37 | 6 | [6] | 39.031 | [6] | 39.031 |
| `- 2 - - - 6 - - -` | 37 | 8 | [2, 6] | 39.031 | [2, 6] | 39.031 |
| `1 - - - - - 7 - -` | 37 | 7 | [7] | 39.278 | [7] | 39.278 |
| `1 - - - - - 7 - -` | 37 | 8 | [1, 7] | 39.278 | [1, 7] | 39.278 |
| `1 - 3 4 - - - - -` | 37 | 3 | [3] | 40.443 | [3] | 40.443 |
| `1 - 3 4 - - - - -` | 37 | 4 | [4] | 40.443 | [4] | 40.443 |
| `1 - 3 4 - - - - -` | 37 | 5 | [1, 4] | 40.443 | [1, 4] | 40.443 |
| `1 - 3 4 - - - - -` | 37 | 7 | [3, 4] | 40.443 | [3, 4] | 40.443 |
| `1 - 3 4 - - - - -` | 37 | 8 | [1, 3, 4] | 40.443 | [1, 3, 4] | 40.443 |
| `1 2 - - 5 - - - -` | 37 | 2 | [2] | 40.991 | [2] | 40.991 |
| `1 2 - - 5 - - - -` | 37 | 3 | [1, 2] | 40.991 | [1, 2] | 40.991 |
| `1 2 - - 5 - - - -` | 37 | 5 | [5] | 40.991 | [5] | 40.991 |
| `1 2 - - 5 - - - -` | 37 | 6 | [1, 5] | 40.991 | [1, 5] | 40.991 |
| `1 2 - - 5 - - - -` | 37 | 7 | [2, 5] | 40.991 | [2, 5] | 40.991 |
| `1 2 - - 5 - - - -` | 37 | 8 | [1, 2, 5] | 40.991 | [1, 2, 5] | 40.991 |
| `- - - - - - 7 - -` | 38 | 7 | [7] | 39.167 | [7] | 39.167 |
| `- - 3 4 - - - - -` | 38 | 3 | [3] | 39.699 | [3] | 39.699 |
| `- - 3 4 - - - - -` | 38 | 4 | [4] | 39.699 | [4] | 39.699 |
| `- - 3 4 - - - - -` | 38 | 7 | [3, 4] | 39.699 | [3, 4] | 39.699 |
| `- 2 - - 5 - - - -` | 38 | 2 | [2] | 39.799 | [2] | 39.799 |
| `- 2 - - 5 - - - -` | 38 | 5 | [5] | 39.799 | [5] | 39.799 |
| `- 2 - - 5 - - - -` | 38 | 7 | [2, 5] | 39.799 | [2, 5] | 39.799 |
| `1 - - - - 6 - - -` | 38 | 6 | [6] | 40.000 | [6] | 40.000 |
| `1 - - - - 6 - - -` | 38 | 7 | [1, 6] | 40.000 | [1, 6] | 40.000 |
| `1 2 - 4 - - - - -` | 38 | 2 | [2] | 41.179 | [2] | 41.179 |
| `1 2 - 4 - - - - -` | 38 | 3 | [1, 2] | 41.179 | [1, 2] | 41.179 |
| `1 2 - 4 - - - - -` | 38 | 4 | [4] | 41.179 | [4] | 41.179 |
| `1 2 - 4 - - - - -` | 38 | 5 | [1, 4] | 41.179 | [1, 4] | 41.179 |
| `1 2 - 4 - - - - -` | 38 | 6 | [2, 4] | 41.179 | [2, 4] | 41.179 |
| `1 2 - 4 - - - - -` | 38 | 7 | [1, 2, 4] | 41.179 | [1, 2, 4] | 41.179 |
| `- - - - - 6 - - -` | 39 | 6 | [6] | 39.833 | [6] | 39.833 |
| `- 2 - 4 - - - - -` | 39 | 2 | [2] | 40.236 | [2] | 40.236 |
| `- 2 - 4 - - - - -` | 39 | 4 | [4] | 40.236 | [4] | 40.236 |
| `- 2 - 4 - - - - -` | 39 | 6 | [2, 4] | 40.236 | [2, 4] | 40.236 |
| `1 - - - 5 - - - -` | 39 | 5 | [5] | 40.389 | [5] | 40.389 |
| `1 - - - 5 - - - -` | 39 | 6 | [1, 5] | 40.389 | [1, 5] | 40.389 |
| `1 2 3 - - - - - -` | 39 | 2 | [2] | 40.975 | [2] | 40.975 |
| `1 2 3 - - - - - -` | 39 | 3 | [3] | 40.975 | [3] | 40.975 |
| `1 2 3 - - - - - -` | 39 | 4 | [1, 3] | 40.975 | [1, 3] | 40.975 |
| `1 2 3 - - - - - -` | 39 | 5 | [2, 3] | 40.975 | [2, 3] | 40.975 |
| `1 2 3 - - - - - -` | 39 | 6 | [1, 2, 3] | 40.975 | [1, 2, 3] | 40.975 |
| `- - - - 5 - - - -` | 40 | 5 | [5] | 40.556 | [5] | 40.556 |
| `- 2 3 - - - - - -` | 40 | 2 | [2] | 40.785 | [2] | 40.785 |
| `- 2 3 - - - - - -` | 40 | 3 | [3] | 40.785 | [3] | 40.785 |
| `- 2 3 - - - - - -` | 40 | 5 | [2, 3] | 40.785 | [2, 3] | 40.785 |
| `1 - - 4 - - - - -` | 40 | 4 | [4] | 40.889 | [4] | 40.889 |
| `1 - - 4 - - - - -` | 40 | 5 | [1, 4] | 40.889 | [1, 4] | 40.889 |
| `- - - 4 - - - - -` | 41 | 4 | [4] | 41.333 | [4] | 41.333 |
| `1 - 3 - - - - - -` | 41 | 3 | [3] | 41.500 | [3] | 41.500 |
| `1 - 3 - - - - - -` | 41 | 4 | [1, 3] | 41.500 | [1, 3] | 41.500 |
| `- - 3 - - - - - -` | 42 | 3 | [3] | 42.167 | [3] | 42.167 |
| `1 2 - - - - - - -` | 42 | 2 | [2] | 42.222 | [2] | 42.222 |
| `1 2 - - - - - - -` | 42 | 3 | [1, 2] | 42.222 | [1, 2] | 42.222 |
| `- 2 - - - - - - -` | 43 | 2 | [2] | 43.056 | [2] | 43.056 |
